In [83]:
from fastlane_bot import Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.helpers import TxHelper
from fastlane_bot.helpers import TxHelpers
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonBot))
from fastlane_bot.testing import *
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("2.0", __VERSION__)


"""
To run this spreadsheet you need the following:

Install dependencies:
    pip install gspread
    pip install oauth2client


Add Carbon API key to .env file:
    x-carbon-auth-key=    

    
To create your own spreadsheet, see the Google Sheet API setup instructions: https://medium.com/daily-python/python-script-to-edit-google-sheets-daily-python-7-aadce27846c0

Add the path to the JSON keystore file that enables access to the Google Sheet in my_file_path below


"""

my_file_path = 'C:\\Users\\Kveen\\Documents\\Bancor_Carbon\\arbon-carbon-report-557e80a643e4.json'


ConstantProductCurve v2.10.1 (07/May/2023)
CarbonBot v3-b2.1 (03/May/2023)


C:\Users\Kveen\AppData\Local\Temp\ipykernel_33912\1611382630.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


Version = 3-b2.1 [requirements >= 2.0 is met]


# Testing the TRIANGLE functions on MAINNET [NBTest024]

## MAINNET Configuration

### Set up the bot and curves

In [84]:
C = Config.new(config=Config.CONFIG_MAINNET)
assert C.DATABASE == C.DATABASE_POSTGRES
assert C.POSTGRES_DB == "defaultdb"
assert C.NETWORK == C.NETWORK_MAINNET
assert C.PROVIDER == C.PROVIDER_ALCHEMY
bot = CarbonBot(ConfigObj=C)
assert str(type(bot.db)) == "<class 'fastlane_bot.db.manager.DatabaseManager'>"

In [85]:
controller = C.CARBON_CONTROLLER_CONTRACT

In [86]:
all_pairs = controller.pairs()


In [87]:
from dataclasses import dataclass
@dataclass
class ERC20Token():
    """Represents an ERC20 token on the Ethereum network. Also used for native ETH."""

    symbol: str = None
    """Symbol such as ETH, DAI, etc."""

    amt: int or Decimal = 0
    """Amount of token."""

    decimals: int = None
    """Decimals used to denominate the token."""

    address: str = None
    """Address of the token contract."""


In [88]:
def _quantize(amount: Decimal, decimals: int) -> Decimal:


    if "." not in str(amount):
        return Decimal(str(amount))
    amount_num = str(amount).split(".")[0]
    amount_dec = str(amount).split(".")[1]
    amount_dec = str(amount_dec)[:decimals]
    try:
        return Decimal(f"{str(amount_num)}.{amount_dec}")
    except Exception as e:
        print("Error quantizing amount: ", f"{str(amount_num)}.{amount_dec}")

In [89]:
@dataclass
class EncodedOrder:
    """
    a single curve as encoded by the SDK

    :token:      token address
    :y:          number of token wei to sell on the curve
    :z:          curve capacity in number of token wei
    :A:          curve parameter A, multiplied by 2**48, encoded
    :B:          curve parameter B, multiplied by 2**48, encoded
    ----
    :A_:         curve parameter A in proper units
    :B_:         curve parameter B in proper units
    :p_start:    start token wei price of the order (in dy/dx)
    :p_end:      end token wei price of the order (in dy/dx)
    """

    token_in: ERC20Token
    token_out: ERC20Token
    y: int
    z: int
    A: int
    B: int
    ONE = 2**48

    @classmethod
    def decodeFloat(cls, value):
        """undoes the mantisse/exponent encoding in A,B"""
        return (value % cls.ONE) << (value // cls.ONE)

    @classmethod
    def decode(cls, value):
        """decodes A,B to float"""
        return cls.decodeFloat(int(value)) / cls.ONE

    @dataclass
    class DecodedOrder:
        """
        a single curve with the values of A,B decoded and as floats
        """

        y: Decimal
        z: Decimal
        A: Decimal
        B: Decimal
        token_in: ERC20Token
        token_out: ERC20Token

    @property
    def decoded(self):
        """
        returns a the order with A, B decoded as floats
        """
        return self.DecodedOrder(
            y=self.y_,
            z=self.z_,
            A=self.A_,
            B=self.B_,
            token_in=self.token_in,
            token_out=self.token_out,
        )

    @property
    def A_(self):
        return Decimal(str(self.decode(self.A))) * Decimal("10") ** (
            (self.token_in.decimals - self.token_out.decimals) / Decimal("2")
        )

    @property
    def B_(self):
        return Decimal(str(self.decode(self.B))) * Decimal("10") ** (
            (self.token_in.decimals - self.token_out.decimals) / Decimal("2")
        )

    @property
    def z_(self):
        return Decimal(str(self.z / 10**self.token_out.decimals))

    @property
    def y_(self):
        return Decimal(str(self.y / 10**self.token_out.decimals))

@dataclass
class CarbonV1Order:
    """
    The fee is a global variable on Carbon. At the smart-contract level, this is represented as a percentage in PPM (parts-per-million).
    Here, for simplicity, we represent it as a value we can multiply by. For example a 1% fee would be: 0.99
    """

    order_id: str = None
    exchange: str = "carbon_v1"
    pair_name: str = None
    exchange_id: int = 6

    def __init__(
        self,
        tkn_out: ERC20Token,
        tkn_in: ERC20Token,
        order_id: str,
        block_updated: int,
        y: Decimal = None,
        z: Decimal = None,
        A: Decimal = None,
        B: Decimal = None,
        fee: Decimal = Decimal("0.002"),
        encoded_order: EncodedOrder = None,
    ):
        """
        param: y: y in the contract
        param: z: this is the label given to the y intercept in the contract.
        param: A: position variable
        param: B: position variable
        param: logger: the logger to log for loggers!
        """
        self.tkn0 = tkn_in
        self.tkn1 = tkn_out
        self.order_id = order_id
        self.id = order_id
        self.pair_name = tkn_out.symbol + "_" + tkn_in.symbol
        self.fee = fee

        if encoded_order is not None:
            decoded = encoded_order.decoded
            self.y, self.z, self.A, self.B = decoded.y, decoded.z, decoded.A, decoded.B
        else:
            self.y = y
            self.B = B
            self.A = A
            self.z = z
            if not self.valid_position:
                raise Exception(
                    f"Invalid Carbon position, y is greater than z or less than 0: y={self.y}, z={self.z}"
                )
        # print(f'Carbon order: y={self.y}, z={self.z}, A={self.A}, B={self.B}')
        # if self.y > 0:
        #     print(f"Carbon order: {self.y} of token: {self.tkn1}")
        
        self.c = self.get_c
        self.d = self.get_d
        self.max_in = self.get_max_in
        self.marg_price = self.get_marginal_price
        self.marg_price_inverted = self.get_marginal_price_inverted

    @property
    def get_c(self):
        """
        Returns the value of C for an order. C represents the following order parameters:
        C = (B * y_int + A * y)^2
        """
        return Decimal((self.B * self.z + self.A * self.y) ** 2)

    @property
    def get_d(self) -> Decimal:
        """
        Returns the value of D for an order. D represents the following order parameters:
        D = B * A * y_int + A^2 * y
        """
        return Decimal(self.B * self.A * self.z + self.A**2 * self.y)

    @property
    def is_active(self) -> bool:
        """
        Returns true if the order currently has liquidity that can be traded
        """
        return self.y > 0

    def update_y(self, y_delta):
        """
        Updates the y (remaining liquidity) in the order.
        When selling tokens, the y_delta should be negative, and when updating the linked order, it should be positive.
        """
        if self.y + y_delta < 0 or self.y + y_delta > self.z:
            raise Exception("Invalid update! y cannot go below 0 or above y_int")

        self.y = self.y + y_delta

    def update_z(self, new_y_int):
        """
        Updates z, the y intercept for instances in which an order size is expanded due to trading.
        """
        self.z = new_y_int

    @property
    def get_marginal_price(self) -> Decimal:
        """
        Calculates the current marginal price (Dy/Dx) of the order.
        """

        return Decimal((self.B * self.z + self.A * self.y) ** 2 / (self.z**2))

    @property
    def get_marginal_price_inverted(self) -> Decimal:
        """
        Calculates the current marginal price inverted - 1 / (Dy/Dx) of the order. This is used to compare marginal prices of orders trading the opposite tokens, IE TKN0 -> TKN1 and TKN1 -> TKN0.
        """

        return Decimal("1") / self.get_marginal_price

    @property
    def get_max_in(self) -> Decimal:
        """
        This returns the maximum amount that can be traded into the order.
        """
        y, z, A, B = self.y, self.z, self.A, self.B
        tkns_in = ((y * z**2) / ((A * y + B * z) * (A * y + B * z - A * y))) * (
            1 - self.fee
        )

        tkns_in = _quantize(tkns_in, self.tkn1.decimals)
        assert type(tkns_in) == Decimal

        return tkns_in

    @property
    def valid_position(self) -> bool:
        """
        Validates the position by checking if the value y is below 0 or above z.
        """
        if self.y < 0 or self.y > self.z or self.B == 0:
            return False
        return True

    def to_pandas(
        self, idx: int = 0, amt_in: Decimal = None, amt_out: Decimal = None
    ) -> pd.DataFrame:
        """
        Exports values for inspection...
        """

        dic = {
            f"{idx}_id": self.order_id,
            f"{idx}_exchange": self.exchange,
            f"{idx}_pair_name": self.pair_name,
            f"{idx}_tkn0_amt": str(0),
            f"{idx}_tkn1_amt": str(0),
            f"{idx}_tkn0_symbol": self.tkn0.symbol,
            f"{idx}_tkn1_symbol": self.tkn1.symbol,
            f"{idx}_fee": self.fee,
            f"{idx}_liquidity": str(0),
            f"{idx}_amt_in": str(amt_in),
            f"{idx}_amt_out": str(amt_out),
            f"{idx}_sqrt_price_q96": str(0) if idx == 1 else None,
            f"{idx}y": self.y,
            f"{idx}z": self.z,
            f"{idx}A": self.A,
            f"{idx}B": self.B,
            f"{idx}c": self.c,
            f"{idx}d": self.d,
            f"{idx}fee": self.fee,
            f"{idx}marg_price": self.marg_price,
            f"{idx}marg_price_inverted": self.marg_price_inverted,
            f"{idx}max_in": self.max_in,
        }

        return pd.DataFrame(dic, index=[0])

In [90]:
token_address_lookup = {
    '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE': {'symbol': 'ETH', 'decimals': 18},
    '0xC98835e792553e505AE46e73a6fD27a23985aCcA': {'symbol': 'LBR', 'decimals': 18},
    '0x7e7E112A68d8D2E221E11047a72fFC1065c38e1a': {'symbol': 'bDIGG', 'decimals': 18},
    '0x444d6088B0F625f8C20192623B3C43001135E0fa': {'symbol': 'CROWN', 'decimals': 18},
    '0x6c862f803fF42A97D4A483AB761256ad8c90f4F8': {'symbol': 'XLS', 'decimals': 18},
    '0x20910e5b5f087f6439DFcB0ddA4e27d1014Ac2b8': {'symbol': 'BNA', 'decimals': 18},
    '0x7884F51dC1410387371ce61747CB6264E1dAeE0B': {'symbol': 'BDOT', 'decimals': 10},
    '0x0FCbC31c503b4A9eD90E87f8Ff46C318a4a14260': {'symbol': 'QTF', 'decimals': 8},
    '0xB6eDA82597808c96969C21a88bf6c749B441c44A': {'symbol': 'SMC', 'decimals': 18},
    '0x1E5193ccC53f25638Aa22a940af899B692e10B09': {'symbol': 'ALLUO', 'decimals': 18},
    '0x9Ce115f0341ae5daBC8B477b74E83db2018A6f42': {'symbol': 'HAIR', 'decimals': 18},
    '0xd24DFf6117936B6ff97108CF26c1dD8865743d87': {'symbol': 'MDM', 'decimals': 18},
    '0xC6065B9fc8171Ad3D29bad510709249681758972': {'symbol': 'WFAIR', 'decimals': 18},
    '0x946551DD05C5Abd7CC808927480225ce36D8c475': {'symbol': 'ONE', 'decimals': 18},
    '0x9cEB84f92A0561fa3Cc4132aB9c0b76A59787544': {'symbol': 'DOKI', 'decimals': 18},
    '0x44F5909E97E1CBf5FBbDF0FC92FD83cde5d5c58A': {'symbol': 'ACRIA', 'decimals': 18},
    '0x7e794eD35788b698AE60cefC98eE48015C4876dA': {'symbol': 'SHINTAMA', 'decimals': 9},
    '0x4da08a1Bff50BE96bdeD5C7019227164b49C2bFc': {'symbol': 'MONONOKE-INU', 'decimals': 9},
    '0x92d529163c5e880b9De86f01De0cB8924D790357': {'symbol': 'EYE', 'decimals': 18},
    '0x996229D0c6a485c7F4B52E092EAa907cB2def5C6': {'symbol': 'BHIG', 'decimals': 18},
    '0x5DD0D493EA59d512EfC13D5C1F9d325775192877': {'symbol': 'PUSUKE', 'decimals': 18},
    '0xa258C4606Ca8206D8aA700cE2143D7db854D168c': {'symbol': 'YVWETH', 'decimals': 18},
    '0xd33D0EB1c0c0295c3A2409a9101dd511823Bf217': {'symbol': 'FANS', 'decimals': 18},
    '0x2d64750eC85f6eC442E3093D74C7b1b0a133BE6a': {'symbol': 'POINT', 'decimals': 18},
    '0x94e496474F1725f1c1824cB5BDb92d7691A4F03a': {'symbol': 'BANANA', 'decimals': 18},
    '0x8a2eAd49D2123eDEE893181FB69527A022235C36': {'symbol': 'MAXR', 'decimals': 9},
    '0xb06B8186cc008A79FD6722B1eefad07c14E97dA0': {'symbol': 'SIGN', 'decimals': 18},
    '0xdd974D5C2e2928deA5F71b9825b8b646686BD200': {'symbol': 'KNCL', 'decimals': 18},
    '0xbb1EE07d6c7BAeB702949904080eb61f5D5e7732': {'symbol': 'DINU', 'decimals': 18},
    '0x5aaEFe84E0fB3DD1f0fCfF6fA7468124986B91bd': {'symbol': 'EVED', 'decimals': 18},
    '0x8E30ea2329D95802Fd804f4291220b0e2F579812': {'symbol': 'DVP', 'decimals': 18},
    '0x6839aC5Ea8082688B706C773D5d2aFd7d503FDD4': {'symbol': 'NXL', 'decimals': 0},
    '0x355a824bEa1adc22733978A3748271E1bbB34130': {'symbol': 'NEPT', 'decimals': 18},
    '0x7f3141c4D6b047fb930991b450f1eD996a51CB26': {'symbol': 'X', 'decimals': 18},
    '0x9BE89D2a4cd102D8Fecc6BF9dA793be995C22541': {'symbol': 'BBTC', 'decimals': 8},
    '0x6aEB95F06CDA84cA345c2dE0F3B7f96923a44f4c': {'symbol': 'BERRY', 'decimals': 14},
    '0xCC1F757E3fa67B70E761C71F4b75B1E9F72263AF': {'symbol': 'GET', 'decimals': 18},
    '0x0B4C2708F052dca413600e237675e4d6778A9375': {'symbol': 'CLM', 'decimals': 16},
    '0xCB56b52316041A62B6b5D0583DcE4A8AE7a3C629': {'symbol': 'CIG', 'decimals': 18},
    '0x485d17A6f1B8780392d53D64751824253011A260': {'symbol': 'TIME', 'decimals': 8},
    '0x580c8520dEDA0a441522AEAe0f9F7A5f29629aFa': {'symbol': 'DAWN', 'decimals': 18},
    '0x407a3E019c655B779ccD098Ff50377E4C5F1C334': {'symbol': 'OTHR', 'decimals': 9},
    '0xb8db81B84d30E2387de0FF330420A4AAA6688134': {'symbol': 'AAMMUNILINKWETH', 'decimals': 18},
    '0xfe98C2B85cAaFE841ca7Fa8b3B912a178A3f68eD': {'symbol': 'RR', 'decimals': 18},
    '0xa6a840E50bCaa50dA017b91A0D86B8b2d41156EE': {'symbol': 'EKO', 'decimals': 18},
    '0x3391bc034f2935eF0E1e41619445F998b2680D35': {'symbol': 'IDLEUSDCSAFE', 'decimals': 18},
    '0xEB50455805ebF8396d9177BBC4A371A376D00ECC': {'symbol': 'BSI', 'decimals': 18},
    '0xaa99199d1e9644b588796F3215089878440D58e0': {'symbol': 'ALPHR', 'decimals': 18},
    '0xdE4CE5447ce0c67920a1371605A39187Cb6847c8': {'symbol': 'LOVE', 'decimals': 18},
    '0x3392D8A60B77F8d3eAa4FB58F09d835bD31ADD29': {'symbol': 'INDI', 'decimals': 18},
    '0xc37E8a31BA2d110c12f09f0239954A68b00bC599': {'symbol': 'EUBC', 'decimals': 8},
    '0xB9e7F8568e08d5659f5D29C4997173d84CdF2607': {'symbol': 'SWT', 'decimals': 18},
    '0x657B83A0336561C8f64389a6f5aDE675C04b0C3b': {'symbol': 'PCNT', 'decimals': 18},
    '0xba386A4Ca26B85FD057ab1Ef86e3DC7BdeB5ce70': {'symbol': 'JESUS', 'decimals': 18},
    '0x82fdedfB7635441aA5A92791D001fA7388da8025': {'symbol': 'DTX', 'decimals': 18},
    '0xc7fF1e126cC81e816915FF48c940ED9d4E6d05d6': {'symbol': 'IJC', 'decimals': 18},
    '0x51A673e060E1ea091B23B53d192058fFE0F2D6b6': {'symbol': 'ZZC', 'decimals': 18},
    '0x4824A7b64E3966B0133f4f4FFB1b9D6bEb75FFF7': {'symbol': 'TCT', 'decimals': 18},
    '0xFD020998A1bB316dfE7B136Fe59Ae4B365D79978': {'symbol': 'GDG', 'decimals': 18},
    '0x97A3BD8a445cC187c6A751F392e15C3B2134D695': {'symbol': 'BXR', 'decimals': 18},
    '0x9c197c4b58527fAAAb67CB35E3145166B23D242e': {'symbol': 'HNB', 'decimals': 18},
    '0x4Dd672e77c795844fe3A464eF8eF0FAAe617C8fB': {'symbol': 'CON', 'decimals': 18},
    '0x99fE3B1391503A1bC1788051347A1324bff41452': {'symbol': 'SX', 'decimals': 18},
    '0xe3A46b2Bc1d83C731D58cAb765D3b45bCe789095': {'symbol': 'DST', 'decimals': 18},
    '0xDD16eC0F66E54d453e6756713E533355989040E4': {'symbol': 'TEN', 'decimals': 18},
    '0x5b71BEE9D961b1B848f8485EEC8d8787f80217F5': {'symbol': 'BF', 'decimals': 18},
    '0x70e36f6BF80a52b3B46b3aF8e106CC0ed743E8e4': {'symbol': 'CCOMP', 'decimals': 8},
    '0x6D52DfeFb16BB9Cdc78bfCA09061e44574886626': {'symbol': 'CPU', 'decimals': 18},
    '0x76BC677d444f1E9d57dAf5187ee2b7dC852745aE': {'symbol': 'XFT', 'decimals': 18},
    '0x7db5af2B9624e1b3B4Bb69D6DeBd9aD1016A58Ac': {'symbol': 'VOLT', 'decimals': 9},
    '0x5F0bc16D50F72d10b719dBF6845DE2E599eb5624': {'symbol': 'VENT', 'decimals': 18},
    '0xaf9f549774ecEDbD0966C52f250aCc548D3F36E5': {'symbol': 'RFUEL', 'decimals': 18},
    '0x48AF7b1c9dac8871C064f62FcEC0d9d6F7c269f5': {'symbol': 'ALPHA', 'decimals': 18},
    '0xE29F1241E6015A4949cb7E3f07234BA794006181': {'symbol': 'BIGCAP', 'decimals': 18},
    '0xFf743A38efCF6f46fC0768B46a5Bdc264F7e92B3': {'symbol': 'AMDG', 'decimals': 18},
    '0x8cA9a0fbd8DB501F013F2e9e33a1B9dC129A48E0': {'symbol': 'DDDD', 'decimals': 18},
    '0xF56b164efd3CFc02BA739b719B6526A6FA1cA32a': {'symbol': 'CGT', 'decimals': 18},
    '0x65Ef703f5594D2573eb71Aaf55BC0CB548492df4': {'symbol': 'MULTI', 'decimals': 18},
    '0x9F5F3CFD7a32700C93F971637407ff17b91c7342': {'symbol': 'DDD', 'decimals': 18},
    '0x8CC0F052fff7eaD7f2EdCCcaC895502E884a8a71': {'symbol': 'ARTH', 'decimals': 18},
    '0x644192291cc835A93d6330b24EA5f5FEdD0eEF9e': {'symbol': 'NXRA', 'decimals': 18},
    '0x786e995A975C2ab7C1dd252A9019fB0c91c9Ca83': {'symbol': 'LWD', 'decimals': 18},
    '0x4e352cF164E64ADCBad318C3a1e222E9EBa4Ce42': {'symbol': 'MCB', 'decimals': 18},
    '0xaaAEBE6Fe48E54f431b0C390CfaF0b017d09D42d': {'symbol': 'CEL', 'decimals': 4},
    '0xe2bA8693cE7474900A045757fe0efCa900F6530b': {'symbol': 'FDAI', 'decimals': 8},
    '0x15Dd37a83564559927bBDcDbfcF85F60A2C6a419': {'symbol': 'AEGIS', 'decimals': 18},
    '0x6E765D26388A17A6e86c49A8E41DF3F58aBcd337': {'symbol': 'KANGAL', 'decimals': 18},
    '0x7169bA45cF03E0377B464462AACbccdf1f721125': {'symbol': '1PECO', 'decimals': 18},
    '0xadd5E881984783dD432F80381Fb52F45B53f3e70': {'symbol': 'VITE', 'decimals': 18},
    '0x72B886d09C117654aB7dA13A14d603001dE0B777': {'symbol': 'XDEFI', 'decimals': 18},
    '0x3C6A7aB47B5F058Be0e7C7fE1A4b7925B8aCA40e': {'symbol': 'CAJ', 'decimals': 18},
    '0xA9d2927d3a04309E008B6af6E2e282AE2952e7fD': {'symbol': 'ZIP', 'decimals': 18},
    '0x9DFAD1b7102D46b1b197b90095B5c4E9f5845BBA': {'symbol': 'BOTTO', 'decimals': 18},
    '0x7D8146cf21e8D7cbe46054e01588207b51198729': {'symbol': 'BOB', 'decimals': 18},
    '0x7e80e4d7d5725499791CF8b17A4586F1f0672A0C': {'symbol': 'ZOGI', 'decimals': 18},
    '0x16587cF43F044aBa0165ffA00AcF412631194e4b': {'symbol': 'SRC', 'decimals': 18},
    '0x73374Ea518De7adDD4c2B624C0e8B113955ee041': {'symbol': 'JGN', 'decimals': 18},
    '0xbd15Ad921e1B480209AF549874a2fCb80DC312Bf': {'symbol': 'HRP', 'decimals': 18},
    '0x09ce2B746C32528B7d864A1e3979Bd97d2f095AB': {'symbol': 'DFL', 'decimals': 18},
    '0x3E5D9D8a63CC8a88748f229999CF59487e90721e': {'symbol': 'XMT', 'decimals': 18},
    '0xd1b624f07a4D9B3e3746E33cb58f42dF079b5444': {'symbol': 'NKCLC', 'decimals': 18},
    '0x19ebAA7F212b09de2aee2a32D40338553C70e2e3': {'symbol': 'ARTM', 'decimals': 18},
    '0xd87D72248093597df8D56d2a53C1ab7c1A0Cc8DA': {'symbol': 'HAHA', 'decimals': 18},
    '0x15F20f9DFDf96CCf6AC96653b7C0aBfe4A9C9f0f': {'symbol': 'WSB', 'decimals': 18},
    '0x275f5Ad03be0Fa221B4C6649B8AeE09a42D9412A': {'symbol': 'MONA', 'decimals': 18},
    '0xbF776e4FCa664D791C4Ee3A71e2722990E003283': {'symbol': 'SMTY', 'decimals': 18},
    '0x5978708d6ccE1CC9640Eed47422D64c91BbD5171': {'symbol': 'LOL', 'decimals': 18},
    '0x19E2a43Fbbc643C3B2D9667D858D49cAd17bc2b5': {'symbol': 'BNS', 'decimals': 8},
    '0xeA7aA1eDd21735A5ab05EE3E90869016191e274E': {'symbol': 'JCC', 'decimals': 18},
    '0x557B933a7C2c45672B610F8954A3deB39a51A8Ca': {'symbol': 'REVV', 'decimals': 18},
    '0x31Fdd1c6607F47C14a2821f599211c67AC20FA96': {'symbol': 'BUY', 'decimals': 18},
    '0x2a03A891ADD2dc6d0F7b94419086630ba5cB65b6': {'symbol': 'DV', 'decimals': 18},
    '0xBaD96Ea5D43156a99A94Cd1Fd36a330aA7E2273e': {'symbol': 'JDB', 'decimals': 18},
    '0x41dBECc1cdC5517C6f76f6a6E836aDbEe2754DE3': {'symbol': 'MTN', 'decimals': 18},
    '0x1712Aad2C773ee04bdC9114b32163c058321CD85': {'symbol': 'LIMIT', 'decimals': 18},
    '0x1337DEF16F9B486fAEd0293eb623Dc8395dFE46a': {'symbol': 'ARMOR', 'decimals': 18},
    '0x310C8F00b9dE3c31Ab95ea68feb6C877538f7947': {'symbol': 'UNDEAD', 'decimals': 18},
    '0x7AC168c81F4F3820Fa3F22603ce5864D6aB3C547': {'symbol': 'STACME', 'decimals': 8},
    '0x4F08705FB8F33AffC231ed66e626B40E84A71870': {'symbol': 'FLUT', 'decimals': 11},
    '0x40FD72257597aA14C7231A7B1aaa29Fce868F677': {'symbol': 'XOR', 'decimals': 18},
    '0x001A8Ffcb0f03e99141652eBCdecDb0384E3bd6c': {'symbol': 'PKR', 'decimals': 18},
    '0x4B48068864F77261838C7849A12853FB94c77a91': {'symbol': 'LITX', 'decimals': 18},
    '0x4295c8556AFEE00264C0789dde2ddd2dba71AcFe': {'symbol': 'BISC', 'decimals': 18},
    '0x5891599664Ed15C6e88041B4F5bc08594f026f0e': {'symbol': 'JPGC', 'decimals': 18},
    '0x20a68F9e34076b2dc15ce726d7eEbB83b694702d': {'symbol': 'ISLA', 'decimals': 18},
    '0xB1191F691A355b43542Bea9B8847bc73e7Abb137': {'symbol': 'KIRO', 'decimals': 18},
    '0x0A7b89E66a1DC16633abdfd132bAe05827d3BFa5': {'symbol': 'HIMOONBIRDS', 'decimals': 18},
    '0x72F020f8f3E8fd9382705723Cd26380f8D0c66Bb': {'symbol': 'PLOT', 'decimals': 18},
    '0x72953a5C32413614d24C29c84a66AE4B59581Bbf': {'symbol': 'CLEV', 'decimals': 18},
    '0x909E34d3f6124C324ac83DccA84b74398a6fa173': {'symbol': 'ZKP', 'decimals': 18},
    '0xD85AD783cc94bd04196a13DC042A3054a9B52210': {'symbol': 'HAKA', 'decimals': 18},
    '0x64D91f12Ece7362F91A6f8E7940Cd55F05060b92': {'symbol': 'ASH', 'decimals': 18},
    '0x21381e026Ad6d8266244f2A583b35F9E4413FA2a': {'symbol': 'FORM', 'decimals': 18},
    '0x8727c112C712c4a03371AC87a74dD6aB104Af768': {'symbol': 'JET', 'decimals': 18},
    '0xd0929d411954c47438dc1d871dd6081F5C5e149c': {'symbol': 'RFR', 'decimals': 4},
    '0x2ED2Cc2c858A8a8219fD2F2d9E170285dBd02756': {'symbol': 'SBET', 'decimals': 18},
    '0x400b1d8a7dd8C471026B2c8CBe1062b27d120538': {'symbol': 'LIMEX', 'decimals': 8},
    '0xBc5991cCd8cAcEba01edC44C2BB9832712c29cAB': {'symbol': 'UUSDC', 'decimals': 6},
    '0x94501b6A153C8973fd1f321fCC8188d40dC5d72d': {'symbol': 'DDRT', 'decimals': 8},
    '0xB63B606Ac810a52cCa15e44bB630fd42D8d1d83d': {'symbol': 'MCO', 'decimals': 8},
    '0x67B66C99D3Eb37Fa76Aa3Ed1ff33E8e39F0b9c7A': {'symbol': 'IBETH', 'decimals': 18},
    '0xf9FBE825BFB2bF3E387af0Dc18caC8d87F29DEa8': {'symbol': 'RADAR', 'decimals': 18},
    '0x461B71cff4d4334BbA09489acE4b5Dc1A1813445': {'symbol': 'HRD', 'decimals': 9},
    '0x4fABf135bCF8111671870d4399af739683198f96': {'symbol': 'XVC', 'decimals': 18},
    '0xC538143202f3b11382D8606aae90a96b042a19DB': {'symbol': 'CNB', 'decimals': 18},
    '0x60F5672A271C7E39E787427A18353ba59A4A3578': {'symbol': 'PIKA', 'decimals': 18},
    '0xbD9908b0Cdd50386F92efCC8e1d71766C2782Df0': {'symbol': 'RICE', 'decimals': 18},
    '0x66A64a8A9cdc54EF97e45a051bA1c8f3D4b41A1D': {'symbol': 'BABYSHIBA', 'decimals': 18},
    '0xa4cb0dCe4849BdcAd2d553E9e68644Cf40E26ccE': {'symbol': 'BAKED', 'decimals': 18},
    '0xDe7D85157d9714EADf595045CC12Ca4A5f3E2aDb': {'symbol': 'STPT', 'decimals': 18},
    '0x13B2f6928D7204328b0E8E4BCd0379aA06EA21FA': {'symbol': 'AAMMWBTC', 'decimals': 8},
    '0x7b123f53421b1bF8533339BFBdc7C98aA94163db': {'symbol': 'BUIDL', 'decimals': 18},
    '0x083D41d6DD21EE938f0c055CA4fb12268DF0EfaC': {'symbol': 'GOL', 'decimals': 4},
    '0x72377f31e30a405282b522d588AEbbea202b4f23': {'symbol': 'VRN', 'decimals': 18},
    '0xd7d706ED3598a354A4aDFce5D8C5383dF99a4461': {'symbol': 'DAIN', 'decimals': 18},
    '0xdc524e3c6910257744C1F93Cf15E9F472b5bD236': {'symbol': 'WITCH', 'decimals': 18},
    '0xEb7355C2f217b3485a591332Fe13C8c5A76A581D': {'symbol': 'JT', 'decimals': 18},
    '0x4F9254C83EB525f9FCf346490bbb3ed28a81C667': {'symbol': 'CELR', 'decimals': 18},
    '0x1856BC58f5511269aFE2CA1e89Bf08CBDCD01100': {'symbol': 'BDS', 'decimals': 18},
    '0xe7eF051C6EA1026A70967E8F04da143C67Fa4E1f': {'symbol': 'VETME', 'decimals': 9},
    '0xbD434a09191D401da3283a5545bB3515d033B8c4': {'symbol': 'GIX', 'decimals': 18},
    '0x4B0181102A0112A2ef11AbEE5563bb4a3176c9d7': {'symbol': 'CSUSHI', 'decimals': 8},
    '0xba358B6f5b4c0215650444B8C30D870B55050D2D': {'symbol': 'HUB', 'decimals': 18},
    '0x518b63Da813D46556FEa041A88b52e3CAa8C16a8': {'symbol': 'ATF', 'decimals': 18},
    '0x763Fa6806e1acf68130D2D0f0df754C93cC546B2': {'symbol': 'LIT', 'decimals': 18},
    '0x00C2999c8B2AdF4ABC835cc63209533973718eB1': {'symbol': 'STATE', 'decimals': 18},
    '0x6aB4A7d75B0A42B6Bc83E852daB9E121F9C610Aa': {'symbol': 'EUM', 'decimals': 18},
    '0x07baC35846e5eD502aA91AdF6A9e7aA210F2DcbE': {'symbol': 'EROWAN', 'decimals': 18},
    '0xA4Bdb11dc0a2bEC88d24A3aa1E6Bb17201112eBe': {'symbol': 'USDS', 'decimals': 6},
    '0x2A54bA2964C8Cd459Dc568853F79813a60761B58': {'symbol': 'USDI', 'decimals': 18},
    '0xBE56ab825fD35678A32dc35bc4EB17e238e1404F': {'symbol': 'DIGITS', 'decimals': 18},
    '0xd6bD97a26232bA02172Ff86b055d5D7bE789335B': {'symbol': 'OMC', 'decimals': 18},
    '0x1977bE49C33dFacF6590C16ca9a9CfA0463f663c': {'symbol': 'DRE', 'decimals': 18},
    '0x36E43065e977bC72CB86Dbd8405fae7057CDC7fD': {'symbol': 'ARCHA', 'decimals': 9},
    '0xf1f955016EcbCd7321c7266BccFB96c68ea5E49b': {'symbol': 'RLY', 'decimals': 18},
    '0x77132Bb08B02a7a87732ec289EFaB45edF49884F': {'symbol': 'RELIC', 'decimals': 18},
    '0x5dc60C4D5e75D22588FA17fFEB90A63E535efCE0': {'symbol': 'DKA', 'decimals': 18},
    '0x29C56E7CB9C840d2b2371b17e28BaB44AD3c3ead': {'symbol': 'ESPRO', 'decimals': 18},
    '0x7714f320Adca62B149df2579361AfEC729c5FE6A': {'symbol': 'TUP', 'decimals': 18},
    '0x4CF488387F035FF08c371515562CBa712f9015d4': {'symbol': 'WPR', 'decimals': 18},
    '0xa16a609fF4e1A15b6cCb469E7a5dD14E89305283': {'symbol': 'SPUME', 'decimals': 18},
    '0xCda2f16C6Aa895D533506B426AFF827b709c87F5': {'symbol': 'FAI', 'decimals': 18},
    '0x028171bCA77440897B824Ca71D1c56caC55b68A3': {'symbol': 'ADAI', 'decimals': 18},
    '0xe481f2311C774564D517d015e678c2736A25Ddd3': {'symbol': 'DEFO', 'decimals': 18},
    '0x70EDF1c215D0ce69E7F16FD4E6276ba0d99d4de7': {'symbol': 'CHEQ', 'decimals': 9},
    '0xF29AE508698bDeF169B89834F76704C3B205aedf': {'symbol': 'YVSNX', 'decimals': 18},
    '0x61fd1c62551850D0c04C76FcE614cBCeD0094498': {'symbol': 'IDK', 'decimals': 8},
    '0xD1b5651E55D4CeeD36251c61c50C889B36F6abB5': {'symbol': 'SDCRV', 'decimals': 18},
    '0x3Ed3B47Dd13EC9a98b44e6204A523E766B225811': {'symbol': 'AUSDT', 'decimals': 6},
    '0x358bD0d980E031E23ebA9AA793926857703783BD': {'symbol': 'AAMMBPTWBTCWETH', 'decimals': 18},
    '0x38d9eb07A7b8Df7D86F440A4A5c4a4c1a27E1a08': {'symbol': 'BLXM', 'decimals': 18},
    '0x26fb86579e371c7AEdc461b2DdEF0A8628c93d3B': {'symbol': 'BORA', 'decimals': 18},
    '0xEaDC218Ac4cB7895a5A56e6484646b48f841C45a': {'symbol': 'MFI', 'decimals': 18},
    '0x9ff58f4fFB29fA2266Ab25e75e2A8b3503311656': {'symbol': 'AWBTC', 'decimals': 8},
    '0xBe00734799A67a62Af2819825580318aC1B1E4Ec': {'symbol': 'ORD', 'decimals': 18},
    '0x41A08648C3766F9F9d85598fF102a08f4ef84F84': {'symbol': 'ABPT', 'decimals': 18},
    '0x000000007a58f5f58E697e51Ab0357BC9e260A04': {'symbol': 'CNV', 'decimals': 18},
    '0x86911B82BfA596e9377836838AC1f0d9AD4Ecc19': {'symbol': 'LCMS', 'decimals': 18},
    '0xD2e5decc08A80be6538F89f9AB8ff296e2f724Df': {'symbol': 'STIMA', 'decimals': 6},
    '0x9b6dB7597a74602a5A806E33408e7E2DAFa58193': {'symbol': 'SPICE', 'decimals': 18},
    '0xdbDD6F355A37b94e6C7D32fef548e98A280B8Df5': {'symbol': 'UWL', 'decimals': 18},
    '0x73484A262730d1d422610729e828346F9B2ff480': {'symbol': 'BRCP', 'decimals': 18},
    '0x67954768E721FAD0f0f21E33e874497C73ED6a82': {'symbol': 'KEK', 'decimals': 18},
    '0x3f5294DF68F871241c4B18fcF78ebD8Ac18aB654': {'symbol': 'STZ', 'decimals': 18},
    '0x41C21693e60FC1a5dBb7c50e54E7A6016aA44C99': {'symbol': 'SIMP', 'decimals': 18},
    '0xD16fd95D949F996e3808EeEa0e3881C59E76EF1E': {'symbol': 'PARA', 'decimals': 18},
    '0x59c6900949aD1835f07a04321f4d9934a054E114': {'symbol': 'TROVE', 'decimals': 18},
    '0x0414D8C87b271266a5864329fb4932bBE19c0c49': {'symbol': 'WSB', 'decimals': 18},
    '0xbcBf09b4b83EB3893D58F0665d21460C04e39D8E': {'symbol': 'SLOCK', 'decimals': 18},
    '0xAeC65404DdC3af3C897AD89571d5772C1A695F22': {'symbol': 'PHX', 'decimals': 18},
    '0xcAfE001067cDEF266AfB7Eb5A286dCFD277f3dE5': {'symbol': 'PSP', 'decimals': 18},
    '0x8954D907520532c1f0d89d42569232Fd0f995Fdf': {'symbol': 'TX', 'decimals': 8},
    '0xB04DFdb8271ed2d5e13858562C44A77D3CEb9e57': {'symbol': 'BUP', 'decimals': 18},
    '0x95652e39E6FB55d41f6A975079035829293e0E16': {'symbol': 'XRT', 'decimals': 8},
    '0x26a604DFFE3ddaB3BEE816097F81d3C4a2A4CF97': {'symbol': 'CORX', 'decimals': 8},
    '0x9e04F519b094F5F8210441e285f603f4d2b50084': {'symbol': '1EARTH', 'decimals': 18},
    '0x76960Dccd5a1fe799F7c29bE9F19ceB4627aEb2f': {'symbol': 'RED', 'decimals': 18},
    '0x48C276e8d03813224bb1e55F953adB6d02FD3E02': {'symbol': 'KUMA', 'decimals': 18},
    '0x47e67BA66b0699500f18A53F94E2b9dB3D47437e': {'symbol': 'PXG', 'decimals': 18},
    '0x226f7b842E0F0120b7E194D05432b3fd14773a9D': {'symbol': 'UNN', 'decimals': 18},
    '0x41B25Ff6431074959532dB7435DAdacA65a21D1C': {'symbol': 'CLAW', 'decimals': 18},
    '0x1A403E1c96792dFedb8232cF56400Eb72Ab95Acb': {'symbol': 'SML', 'decimals': 18},
    '0xa2881F7F441267042f9778fFA0d4F834693426be': {'symbol': 'HUSL', 'decimals': 18},
    '0xc58F53A8adff2fB4eb16ED56635772075E2EE123': {'symbol': 'AAMMUNIWBTCWETH', 'decimals': 18},
    '0x3A810ff7211b40c4fA76205a14efe161615d0385': {'symbol': 'AIN', 'decimals': 18},
    '0xc0AE17Eb994fa828540FFa53776B3830233A1B02': {'symbol': 'ELT', 'decimals': 18},
    '0xBBeca80A4c99c3b1BD3C10E64595d0Fc7dC26eE0': {'symbol': 'HIMEEBITS', 'decimals': 18},
    '0xaF4DcE16Da2877f8c9e00544c93B62Ac40631F16': {'symbol': 'MTH', 'decimals': 5},
    '0x419c4dB4B9e25d6Db2AD9691ccb832C8D9fDA05E': {'symbol': 'DRGN', 'decimals': 18},
    '0xcccF837f40D334F8602f031e64B52AD4CD2b6601': {'symbol': 'UFEWO', 'decimals': 18},
    '0xb622400807765e73107B7196F444866D7EdF6f62': {'symbol': 'HEROS', 'decimals': 9},
    '0x8f3470A7388c05eE4e7AF3d01D8C722b0FF52374': {'symbol': 'VERI', 'decimals': 18},
    '0xC1322D8aE3B0e2E437e0AE36388D0CFD2C02f1c9': {'symbol': 'PMT', 'decimals': 4},
    '0xD39a2CeCBA2657e125Ba6a5c98ad2F6b6D7E83FD': {'symbol': 'LUXO', 'decimals': 18},
    '0x57c411e9a358e2d2d0a6b058cEDb709175e8fd16': {'symbol': 'KODACHI', 'decimals': 18},
    '0xA0008F510fE9eE696E7E320C9e5cbf61E27791Ee': {'symbol': 'GMB', 'decimals': 18},
    '0xa1a36D3537bBe375Cc9694795F663DDC8D516Db9': {'symbol': 'POLI', 'decimals': 18},
    '0xCe391315b414D4c7555956120461D21808A69F3A': {'symbol': 'BAO', 'decimals': 18},
    '0x3175Df0976dFA876431C2E9eE6Bc45b65d3473CC': {'symbol': 'CRVFRAX', 'decimals': 18},
    '0xB13d747d783BF1A9F1D65Df74C080C890045f17e': {'symbol': 'FND', 'decimals': 7},
    '0xdfe691F37b6264a90Ff507EB359C45d55037951C': {'symbol': 'KARMA', 'decimals': 4},
    '0x37E83a94c6B1Bdb816B59aC71dd02CF154d8111F': {'symbol': 'PHCR', 'decimals': 18},
    '0xE692c8D72bd4aC7764090d54842a305546dd1dE5': {'symbol': 'ABLOCK', 'decimals': 8},
    '0x68749665FF8D2d112Fa859AA293F07A622782F38': {'symbol': 'XAUT', 'decimals': 6},
    '0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0': {'symbol': 'MATIC', 'decimals': 18},
    '0x77271813bd9167e75B5DF9c230cf58d64f0A58Fd': {'symbol': 'BKR', 'decimals': 18},
    '0xdc8aF07A7861bedD104B8093Ae3e9376fc8596D2': {'symbol': 'RVF', 'decimals': 18},
    '0x22B48E1f20043D1Db5F2a11CbF1d520A4F20b198': {'symbol': 'XOT', 'decimals': 18},
    '0x94804dc4948184fFd7355f62Ccbb221c9765886F': {'symbol': 'RAGE', 'decimals': 18},
    '0x963035453633f6f7433032c958F82Eb3043D8cD2': {'symbol': 'USDF', 'decimals': 18},
    '0x15874d65e649880c2614e7a480cb7c9A55787FF6': {'symbol': 'EMAX', 'decimals': 18},
    '0x6B3595068778DD592e39A122f4f5a5cF09C90fE2': {'symbol': 'SUSHI', 'decimals': 18},
    '0x4123a133ae3c521FD134D7b13A2dEC35b56c2463': {'symbol': 'QRDO', 'decimals': 8},
    '0x5165d24277cD063F5ac44Efd447B27025e888f37': {'symbol': 'AYFI', 'decimals': 18},
    '0xc9BC48c72154ef3e5425641a3c747242112a46AF': {'symbol': 'ARAI', 'decimals': 18},
    '0xD34c9281585866A9fb2af0931adb1fA35ed44039': {'symbol': 'BGC', 'decimals': 8},
    '0x8379f52d09b9998eCbA731288EE4fE532fD91C0B': {'symbol': 'BAKT', 'decimals': 18},
    '0xAF75d880b3128981D1fEd3292Fc02E3FB37Acd53': {'symbol': 'TRUTH', 'decimals': 9},
    '0xA86a0Da9D05d0771955DF05B44Ca120661aF16DE': {'symbol': 'OTB', 'decimals': 18},
    '0x1c48f86ae57291F7686349F12601910BD8D470bb': {'symbol': 'USDK', 'decimals': 18},
    '0x5150956E082C748Ca837a5dFa0a7C10CA4697f9c': {'symbol': 'ZDEX', 'decimals': 18},
    '0x6361F338Ab8DEf2AF3f2A1Be7BD8A7dB3156F7E7': {'symbol': 'PRBLY', 'decimals': 18},
    '0xee772CEc929D8430b4Fa7a01cD7FbD159a68Aa83': {'symbol': 'SHANG', 'decimals': 18},
    '0x2370f9d504c7a6E775bf6E14B3F12846b594cD53': {'symbol': 'JPYC', 'decimals': 18},
    '0xeAd482da0793B00bbAe0E34C8cfaE6DAf29a44b2': {'symbol': 'VRH', 'decimals': 18},
    '0x8254C1C134436F74047F79eaAeA97E3324eF78B5': {'symbol': 'ICHIGO', 'decimals': 9},
    '0x84679bc467DC6c2c40ab04538813AfF3796351f1': {'symbol': 'CHONK', 'decimals': 18},
    '0xF94b5C5651c888d928439aB6514B93944eEE6F48': {'symbol': 'YLD', 'decimals': 18},
    '0xa1d6Df714F91DeBF4e0802A542E13067f31b8262': {'symbol': 'RFOX', 'decimals': 18},
    '0x465e07d6028830124BE2E4aA551fBe12805dB0f5': {'symbol': 'WXMR', 'decimals': 18},
    '0x58b6A8A3302369DAEc383334672404Ee733aB239': {'symbol': 'LPT', 'decimals': 18},
    '0xB8C3B7A2A618C552C23B1E4701109a9E756Bab67': {'symbol': 'YV1INCH', 'decimals': 18},
    '0x3D9Ac8e7a9C9bE11DFac1677dDa901E28d44527f': {'symbol': 'PAA', 'decimals': 8},
    '0x820802Fa8a99901F52e39acD21177b0BE6EE2974': {'symbol': 'EUROE', 'decimals': 6},
    '0x916c5DE09cF63f6602d1e1793FB41F6437814A62': {'symbol': 'JACY', 'decimals': 9},
    '0xa20f77B7ad5A88badC48800C56507B7274c06Fdc': {'symbol': 'CHER', 'decimals': 18},
    '0x5Cb888182fBfFdb62C08fb4B5a343914F00FdfeE': {'symbol': 'BIPS', 'decimals': 18},
    '0xBa21Ef4c9f433Ede00badEFcC2754B8E74bd538A': {'symbol': 'SWFL', 'decimals': 18},
    '0xB4a677B0E363c3815d46326954a4E4d2B1ACe357': {'symbol': 'THE', 'decimals': 18},
    '0x6C5024Cd4F8A59110119C56f8933403A539555EB': {'symbol': 'ASUSD', 'decimals': 18},
    '0xB9D7CB55f463405CDfBe4E90a6D2Df01C2B92BF1': {'symbol': 'AUNI', 'decimals': 18},
    '0x8290333ceF9e6D528dD5618Fb97a76f268f3EDD4': {'symbol': 'ANKR', 'decimals': 18},
    '0x2863916C6ebDBBf0c6f02F87b7eB478509299868': {'symbol': 'SST', 'decimals': 18},
    '0x653430560bE843C4a3D143d0110e896c2Ab8ac0D': {'symbol': 'MOF', 'decimals': 16},
    '0xF406F7A9046793267bc276908778B29563323996': {'symbol': 'VISION', 'decimals': 18},
    '0xeE6b9CF11D968E0bAc7BFfF547577B8AE35B8065': {'symbol': 'MKONG', 'decimals': 9},
    '0xdb25cA703181E7484a155DD612b06f57E12Be5F0': {'symbol': 'YVYFI', 'decimals': 18},
    '0x5C1d9aA868a30795F92fAe903eDc9eFF269044bf': {'symbol': 'CNG', 'decimals': 18},
    '0xf8C3527CC04340b208C854E985240c02F7B7793f': {'symbol': 'FRONT', 'decimals': 18},
    '0xDadb4aE5B5D3099Dd1f586f990B845F2404A1c4c': {'symbol': '', 'decimals': 18},
    '0x0BF43350076F95e0d16120b4D6bdfA1C9D50BDBD': {'symbol': 'AGT', 'decimals': 18},
    '0x0f2D719407FdBeFF09D87557AbB7232601FD9F29': {'symbol': 'SYN', 'decimals': 18},
    '0xD37EE7e4f452C6638c96536e68090De8cBcdb583': {'symbol': 'AGUSD', 'decimals': 2},
    '0x444444444444C1a66F394025Ac839A535246FCc8': {'symbol': 'GENI', 'decimals': 9},
    '0x171b1DAefac13A0A3524Fcb6BEDdC7B31E58e079': {'symbol': 'FCC', 'decimals': 18},
    '0x570936840fA942C96ca4C57A170dDa494DD9B9C6': {'symbol': 'ART', 'decimals': 18},
    '0xD43Be54C1aedf7Ee4099104f2DaE4eA88B18A249': {'symbol': 'TRAXX', 'decimals': 18},
    '0x72E5390EDb7727E3d4e3436451DADafF675dBCC0': {'symbol': 'HANU', 'decimals': 12},
    '0xC949fC82A15964FB5B97e5cf8f9ffed139086821': {'symbol': 'PGPAY', 'decimals': 18},
    '0x882e5b370D595E50c24b2a0e7a94e87Cc32ADdA1': {'symbol': 'GAME', 'decimals': 18},
    '0x177BA0cac51bFC7eA24BAd39d81dcEFd59d74fAa': {'symbol': 'KIF', 'decimals': 18},
    '0xB685145D7F127b9093d7f9278BaE902EF59Ff486': {'symbol': 'FREQAI', 'decimals': 18},
    '0x5F0E628B693018f639D10e4A4F59BD4d8B2B6B44': {'symbol': 'WHITE', 'decimals': 18},
    '0x836A808d4828586A69364065A1e064609F5078c7': {'symbol': 'PETH', 'decimals': 18},
    '0x9625cE7753ace1fa1865A47aAe2c5C2Ce4418569': {'symbol': 'KAP', 'decimals': 18},
    '0x0fF6ffcFDa92c53F615a4A75D982f399C989366b': {'symbol': 'LAYER', 'decimals': 18},
    '0x06450dEe7FD2Fb8E39061434BAbCFC05599a6Fb8': {'symbol': 'XEN', 'decimals': 18},
    '0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B': {'symbol': 'CVX', 'decimals': 18},
    '0xA823E6722006afe99E91c30FF5295052fe6b8E32': {'symbol': 'NEU', 'decimals': 18},
    '0xe8Ff5C9c75dEb346acAc493C463C8950Be03Dfba': {'symbol': 'VIBE', 'decimals': 18},
    '0x227c7DF69D3ed1ae7574A1a7685fDEd90292EB48': {'symbol': 'MILADY', 'decimals': 18},
    '0x0F00f1696218EaeFa2D2330Df3D6D1f94813b38f': {'symbol': 'SEDO', 'decimals': 8},
    '0x524Ebc93BeEf838F70b4Ae54b675d3e971D5884E': {'symbol': 'HERO', 'decimals': 9},
    '0x40d1F63B5D2048e67E9bEdB1B4c2F1A9fb4b6817': {'symbol': 'GOLD', 'decimals': 18},
    '0xAbeA7663c472648d674bd3403D94C858dFeEF728': {'symbol': 'PUDGY', 'decimals': 18},
    '0x93ad9b819C88D98B4c9641470A96E24769Ae7922': {'symbol': 'KRX', 'decimals': 9},
    '0x7A58c0Be72BE218B41C608b7Fe7C5bB630736C71': {'symbol': 'PEOPLE', 'decimals': 18},
    '0x501e8726d06Cdef66f3e0cb67f54924CcA1cC894': {'symbol': 'BRMV', 'decimals': 18},
    '0xb78B3320493a4EFaa1028130C5Ba26f0B6085Ef8': {'symbol': 'DRC', 'decimals': 18},
    '0x970cf867Ca0530a989f222bE01FdD67C1ab5b2bF': {'symbol': 'BASAN', 'decimals': 18},
    '0x5274891bEC421B39D23760c04A6755eCB444797C': {'symbol': 'IDLEUSDCYIELD', 'decimals': 18},
    '0xf911a7ec46a2c6fa49193212fe4a2a9B95851c27': {'symbol': 'XAMP', 'decimals': 9},
    '0x9D65fF81a3c488d585bBfb0Bfe3c7707c7917f54': {'symbol': 'SSV', 'decimals': 18},
    '0x3136eF851592aCf49CA4C825131E364170FA32b3': {'symbol': 'COFI', 'decimals': 18},
    '0x6f259637dcD74C767781E37Bc6133cd6A68aa161': {'symbol': 'HT', 'decimals': 18},
    '0x9B83f827928aBdf18cF1F7e67053572b9bceff3a': {'symbol': 'ARTEM', 'decimals': 18},
    '0x351cAa9045D65107b9d311D922D15887cfd634E4': {'symbol': 'ARW', 'decimals': 18},
    '0x4f3AfEC4E5a3F2A6a1A411DEF7D7dFe50eE057bF': {'symbol': 'DGX', 'decimals': 9},
    '0xBBa39Fd2935d5769116ce38d46a71bde9cf03099': {'symbol': 'CHO', 'decimals': 18},
    '0x8fc9b6354E839AB1c8B31F4afa53607092B8C2e5': {'symbol': 'ECU', 'decimals': 18},
    '0x3af33bEF05C2dCb3C7288b77fe1C8d2AeBA4d789': {'symbol': 'KROM', 'decimals': 18},
    '0x88ACDd2a6425c3FaAE4Bc9650Fd7E27e0Bebb7aB': {'symbol': 'MIST', 'decimals': 18},
    '0x9565c2036963697786705120Fc59310F747bCfD0': {'symbol': 'PP', 'decimals': 18},
    '0x1ce270557C1f68Cfb577b856766310Bf8B47FD9C': {'symbol': 'MONG', 'decimals': 18},
    '0x993864E43Caa7F7F12953AD6fEb1d1Ca635B875F': {'symbol': 'SDAO', 'decimals': 18},
    '0x03042ae6fcFd53E3A0BAa1FaB5Ce70E0cb74E6fb': {'symbol': 'TBC', 'decimals': 18},
    '0x1614F18Fc94f47967A3Fbe5FfcD46d4e7Da3D787': {'symbol': 'PAID', 'decimals': 18},
    '0x8b79656FC38a04044E495e22fAD747126ca305C4': {'symbol': 'AGVC', 'decimals': 18},
    '0x44f262622248027f8E2a8Fb1090c4Cf85072392C': {'symbol': 'XIV', 'decimals': 18},
    '0x998FFE1E43fAcffb941dc337dD0468d52bA5b48A': {'symbol': 'IDRT', 'decimals': 2},
    '0x4Fb721eF3Bf99e0f2c193847afA296b9257d3C30': {'symbol': 'TOK', 'decimals': 8},
    '0xAD038Eb671c44b853887A7E32528FaB35dC5D710': {'symbol': 'DBR', 'decimals': 18},
    '0x22987407FD1fC5A971e3FDA3B3e74C88666cDa91': {'symbol': 'SRT', 'decimals': 18},
    '0xCF8829AE9384540c886A151fac3a865794cB9a01': {'symbol': 'SDG', 'decimals': 18},
    '0x944eeE930933BE5E23b690c8589021Ec8619a301': {'symbol': 'MUNCH', 'decimals': 9},
    '0x8790f2fc7Ca2e7Db841307FB3f4e72a03Baf7b47': {'symbol': 'SPILLWAYS', 'decimals': 18},
    '0xCcbA0b2bc4BAbe4cbFb6bD2f1Edc2A9e86b7845f': {'symbol': 'WINTER', 'decimals': 18},
    '0x37A15C92e67686aA268df03d4C881A76340907E8': {'symbol': 'PIXIU', 'decimals': 0},
    '0x374CB8C27130E2c9E04F44303f3c8351B9De61C1': {'symbol': 'BAO', 'decimals': 18},
    '0x08F7be99ED83369541501d60f4e66F8e34c3F736': {'symbol': 'CKU', 'decimals': 18},
    '0x767FE9EDC9E0dF98E07454847909b5E959D7ca0E': {'symbol': 'ILV', 'decimals': 18},
    '0x010D14d36C3eA6570D240ae3ac9d660398f7C48e': {'symbol': 'XCF', 'decimals': 18},
    '0x4db2c02831c9ac305FF9311Eb661f80f1dF61e07': {'symbol': 'ONEICHI', 'decimals': 18},
    '0xA67E9F021B9d208F7e3365B2A155E3C55B27de71': {'symbol': 'KLEE', 'decimals': 9},
    '0x940BdCb99A0Ee5Fb008A606778AE87Ed9789F257': {'symbol': 'JFIN', 'decimals': 18},
    '0x8442E0e292186854BB6875b2A0fc1308b9Ded793': {'symbol': 'PP', 'decimals': 9},
    '0x8C81121B15197fA0eEaEE1DC75533419DcfD3151': {'symbol': 'IDLEWBTCYIELD', 'decimals': 18},
    '0x3625b55f653F8C7c884aC668696881df43631D44': {'symbol': 'ARTI', 'decimals': 8},
    '0x69e37422cB87d963367F73A119C8ce9a4D529b72': {'symbol': 'ADVT', 'decimals': 9},
    '0xc278041fDD8249FE4c1Aad1193876857EEa3D68c': {'symbol': 'IDLETUSDYIELD', 'decimals': 18},
    '0x350D3f0f41b5B21F0e252fe2645AE9D55562150a': {'symbol': 'BCT', 'decimals': 9},
    '0x0000000005c6B7C1fd10915a05f034F90d524D6E': {'symbol': 'TRYC', 'decimals': 6},
    '0x34bdf48A8F753de4822a6CFB1FEE275F9b4D662e': {'symbol': 'BKC', 'decimals': 18},
    '0x3Be65bd36C1aD28D33Acc1C9dD8338a2706cA000': {'symbol': 'KAIROS', 'decimals': 9},
    '0x6fc73113fC1aFaB4C28d3DD4c537A1DA6045d47D': {'symbol': 'TRENDAI', 'decimals': 18},
    '0x9ba4CBcBb920F67f54A8F8f0001D721E0B8944b9': {'symbol': 'REDPANDA', 'decimals': 9},
    '0x2ef52Ed7De8c5ce03a4eF0efbe9B7450F2D7Edc9': {'symbol': 'REV', 'decimals': 6},
    '0x0AF55d5fF28A3269d69B98680Fd034f115dd53Ac': {'symbol': 'BSL', 'decimals': 8},
    '0x024B6e7DC26F4d5579bDD936F8d7BC31f2339999': {'symbol': 'MIS', 'decimals': 18},
    '0x92e52a1A235d9A103D970901066CE910AAceFD37': {'symbol': 'UCASH', 'decimals': 8},
    '0x1da4858ad385cc377165A298CC2CE3fce0C5fD31': {'symbol': 'CCS', 'decimals': 0},
    '0xf92364c2369a2633FfcD7dB1B18D1Fafff6bCbaB': {'symbol': 'BABIL', 'decimals': 8},
    '0xf5555732B3925356964695578feFcFfCD31bCbb8': {'symbol': 'PMD', 'decimals': 9},
    '0x9a0587EaE7eF64b2B38A10442a44CfA43EDd7D2A': {'symbol': 'WTL', 'decimals': 18},
    '0x0D8775F648430679A709E98d2b0Cb6250d2887EF': {'symbol': 'BAT', 'decimals': 18},
    '0x213C53C96A01a89E6Dcc5683cF16473203E17513': {'symbol': 'DSS', 'decimals': 18},
    '0x86d1d12523B65203851c571FcC029bF90903fB6d': {'symbol': 'BNTX', 'decimals': 18},
    '0x4aa41bC1649C9C3177eD16CaaA11482295fC7441': {'symbol': 'XFIT', 'decimals': 18},
    '0x602f65BB8B8098Ad804E99DB6760Fd36208cD967': {'symbol': 'MOPS', 'decimals': 18},
    '0xc4c7Ea4FAB34BD9fb9a5e1B1a98Df76E26E6407c': {'symbol': 'COCOS', 'decimals': 18},
    '0x19D97D8fA813EE2f51aD4B4e04EA08bAf4DFfC28': {'symbol': 'BBADGER', 'decimals': 18},
    '0xa2847348b58CEd0cA58d23c7e9106A49f1427Df6': {'symbol': 'CVXFPIS', 'decimals': 18},
    '0x305DE070488C8469dFAC957226c9c900c4BFbA22': {'symbol': 'MORE', 'decimals': 8},
    '0x8dBF9A4c99580fC7Fd4024ee08f3994420035727': {'symbol': 'ECO', 'decimals': 18},
    '0x957891C11616D3E0B0A76a76fb42724C382E0eF3': {'symbol': 'COLL', 'decimals': 18},
    '0x86F0cdB38a853420be4dbF6a3a9a3fe0756bc701': {'symbol': 'LEVX', 'decimals': 9},
    '0xC014186Cf1Ba36032aaEc7f96088f09eb3934347': {'symbol': 'WCX', 'decimals': 18},
    '0x80D55c03180349Fff4a229102F62328220A96444': {'symbol': 'OPUL', 'decimals': 18},
    '0xE41d2489571d322189246DaFA5ebDe1F4699F498': {'symbol': 'ZRX', 'decimals': 18},
    '0x4e4a47cAC6A28A62dcC20990ed2cdA9BC659469F': {'symbol': 'SHIT', 'decimals': 18},
    '0xDEf1CA1fb7FBcDC777520aa7f396b4E015F497aB': {'symbol': 'COW', 'decimals': 18},
    '0xfF20817765cB7f73d4bde2e66e067E58D11095C2': {'symbol': 'AMP', 'decimals': 18},
    '0x114f1388fAB456c4bA31B1850b244Eedcd024136': {'symbol': 'COOL', 'decimals': 18},
    '0xc56c2b7e71B54d38Aab6d52E94a04Cbfa8F604fA': {'symbol': 'ZUSD', 'decimals': 6},
    '0x55C08ca52497e2f1534B59E2917BF524D4765257': {'symbol': 'UWU', 'decimals': 18},
    '0x08F40811C7d6C013744166f3d4CB1A9A92D3D54E': {'symbol': 'NVG', 'decimals': 18},
    '0xE910c2a090516Fb7a7Be07f96a464785f2D5Dc18': {'symbol': 'CMT', 'decimals': 18},
    '0xF88Baf18FAB7e330fa0C4F83949E23F52FECECce': {'symbol': 'GRAIN', 'decimals': 18},
    '0x49D72e3973900A195A155a46441F0C08179FdB64': {'symbol': 'CRETH2', 'decimals': 18},
    '0xD0660cD418a64a1d44E9214ad8e459324D8157f1': {'symbol': 'WOOFY', 'decimals': 12},
    '0x75387e1287Dd85482aB66102DA9f6577E027f609': {'symbol': 'MAI', 'decimals': 18},
    '0x06f3CDabae564B0546529b4DD8FeF1bcD4235753': {'symbol': 'TLW', 'decimals': 8},
    '0x29CeDDCF0Da3c1D8068a7DFbD0FB06c2E438FF70': {'symbol': 'FREL', 'decimals': 18},
    '0xCFEAead4947f0705A14ec42aC3D44129E1Ef3eD5': {'symbol': 'NOTE', 'decimals': 8},
    '0x910Dfc18D6EA3D6a7124A6F8B5458F281060fa4c': {'symbol': 'X8X', 'decimals': 18},
    '0x6570fFe19dA7e2b425329B157d9109b87f18304b': {'symbol': 'UNM', 'decimals': 18},
    '0x456D8f0D25A4e787eE60c401F8B963a465148f70': {'symbol': 'CAVA', 'decimals': 9},
    '0x47b9F01B16E9C9cb99191DCA68c9cc5bF6403957': {'symbol': 'ONSTON', 'decimals': 18},
    '0xa41161AF8D4Ee421ba5fED4328F2B12034796A21': {'symbol': 'AFY', 'decimals': 18},
    '0x2c644C3bBeA053ed95a6bC04A94C9CE928Ff9881': {'symbol': 'LCE', 'decimals': 8},
    '0x76306F029f8F99EFFE509534037Ba7030999E3CF': {'symbol': 'ACR', 'decimals': 18},
    '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9': {'symbol': 'AAVE', 'decimals': 18},
    '0x0107C4aAa31940eF88760aCb1F32424CA8d8bD80': {'symbol': 'AUTH', 'decimals': 8},
    '0xcB8fb2438A805664cD8c3e640b85AC473DA5BE87': {'symbol': 'CTI', 'decimals': 18},
    '0xfFbF315f70E458e49229654DeA4cE192d26f9b25': {'symbol': 'VOLT', 'decimals': 18},
    '0x9ab165D795019b6d8B3e971DdA91071421305e5a': {'symbol': 'AOA', 'decimals': 18},
    '0xC0e6737A29DE7a00e2f6011924eB257106CB082f': {'symbol': 'GLO', 'decimals': 18},
    '0xd5A98E77d1fEB091344096301Ea336a5C07a6A41': {'symbol': 'ACAP', 'decimals': 18},
    '0x4b9278b94a1112cAD404048903b8d343a810B07e': {'symbol': 'HIFI', 'decimals': 18},
    '0x3b544e6fcf6C8dCE9D8B45A4FdF21C9B02f9fDa9': {'symbol': 'GHD', 'decimals': 18},
    '0xD13432fdD2ae4eeD84C3A0233D6c5265087C96c3': {'symbol': 'STSR', 'decimals': 18},
    '0x15Eabb7500E44B7Fdb6e4051cA8DecA430cF9FB8': {'symbol': 'DXF', 'decimals': 18},
    '0x8810C63470d38639954c6B41AaC545848C46484a': {'symbol': 'ADI', 'decimals': 18},
    '0x2781246fe707bB15CeE3e5ea354e2154a2877B16': {'symbol': 'EL', 'decimals': 18},
    '0x79C9e0410b6615e7Ba9DD69614B0519325a2B047': {'symbol': 'HIFLUF', 'decimals': 18},
    '0x419E35E3515c2fDB652C898bF7A0B21FB20497dC': {'symbol': 'OFI', 'decimals': 9},
    '0xC6d1f1D5a46de07E73091f1c8793293B203f01a1': {'symbol': 'HELIOS', 'decimals': 18},
    '0x5aF28EB9deC75e86CDFBD2e41a474b54B211C1C2': {'symbol': 'TNGL', 'decimals': 9},
    '0xE9966C1184f8552Fcb16F65addba9Dd08fe8f4EA': {'symbol': 'MYCE', 'decimals': 18},
    '0x90E2Fa98DfC518317600Dd3DC571DE8D071a7238': {'symbol': 'RHLLOR', 'decimals': 9},
    '0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE': {'symbol': 'SHIB', 'decimals': 18},
    '0xfAE4Ee59CDd86e3Be9e8b90b53AA866327D7c090': {'symbol': 'CPC', 'decimals': 18},
    '0xf134519cBE2042B06Ee7cE20Df51d09b55559896': {'symbol': 'MOCHI', 'decimals': 18},
    '0xd9Fcd98c322942075A5C3860693e9f4f03AAE07b': {'symbol': 'EUL', 'decimals': 18},
    '0x9e5BD9D9fAd182ff0A93bA8085b664bcab00fA68': {'symbol': 'DINGER', 'decimals': 9},
    '0x107c4504cd79C5d2696Ea0030a8dD4e92601B82e': {'symbol': 'BLT', 'decimals': 18},
    '0xcDcA1B81DbB543BAA92c97b701396Cd3bA315E5D': {'symbol': 'SHUEY', 'decimals': 18},
    '0xF83301c5Cd1CCBB86f466A6B3c53316ED2f8465a': {'symbol': 'CMS', 'decimals': 6},
    '0x793786e2dd4Cc492ed366a94B88a3Ff9ba5E7546': {'symbol': 'AXIAV3', 'decimals': 18},
    '0x4aD0b81f92B16624BBcF46FC0030cFBBf8d02376': {'symbol': 'UDAI', 'decimals': 18},
    '0xac3E018457B222d93114458476f3E3416Abbe38F': {'symbol': 'SFRXETH', 'decimals': 18},
    '0xeBF4CA5319F406602EEFf68da16261f1216011B5': {'symbol': 'YO', 'decimals': 18},
    '0x667102BD3413bFEaa3Dffb48fa8288819E480a88': {'symbol': 'TKX', 'decimals': 8},
    '0x9bA00D6856a4eDF4665BcA2C2309936572473B7E': {'symbol': 'AUSDC', 'decimals': 6},
    '0xF1e345Ea7c33fd6c05F5512a780Eb5839EE31674': {'symbol': 'TELE', 'decimals': 18},
    '0x3Ecab35B64345bfC472477A653e4A3abE70532D9': {'symbol': 'ENTC', 'decimals': 18},
    '0xf009f5531dE69067435e32c4b9D36077F4C4A673': {'symbol': 'UNV', 'decimals': 18},
    '0xBf9e72eEb5adB8B558334c8672950B7a379D4266': {'symbol': 'CUBT', 'decimals': 18},
    '0x2fe39f22EAC6d3c1C86DD9D143640EbB94609FCE': {'symbol': 'JDC', 'decimals': 18},
    '0xeEEE2a622330E6d2036691e983DEe87330588603': {'symbol': 'ASKO', 'decimals': 18},
    '0x193408cA0576B73156Ed42A2EA7D6fD3f6507162': {'symbol': 'INFS', 'decimals': 1},
    '0x3446Dd70B2D52A6Bf4a5a192D9b0A161295aB7F9': {'symbol': 'SUDO', 'decimals': 18},
    '0x40986a85B4cFCDb054A6CBFB1210194fee51af88': {'symbol': 'UFARM', 'decimals': 18},
    '0xE6f143a0e0A8f24F6294CE3432eA10FaD0206920': {'symbol': 'PKEX', 'decimals': 18},
    '0x29A5c1Db7321C5c9EaE57F9366eE8eef00cA11fb': {'symbol': 'SHKOOBY', 'decimals': 18},
    '0x685Fb6960Ea7F7b24F02e17c8d893B8c33239d48': {'symbol': 'SAIKO', 'decimals': 9},
    '0x6b785a0322126826d8226d77e173d75DAfb84d11': {'symbol': 'VLT', 'decimals': 18},
    '0xE51b8Ab09008285a0380Dd2680cD9Dd5e13924D3': {'symbol': 'BSP', 'decimals': 18},
    '0x035bfe6057E15Ea692c0DfdcaB3BB41a64Dd2aD4': {'symbol': 'ULU', 'decimals': 18},
    '0x67B6D479c7bB412C54e03dCA8E1Bc6740ce6b99C': {'symbol': 'KYL', 'decimals': 18},
    '0x6De037ef9aD2725EB40118Bb1702EBb27e4Aeb24': {'symbol': 'RNDR', 'decimals': 18},
    '0xd55236D48606c295adEbF129dAD04Fc74BFaA708': {'symbol': 'CW', 'decimals': 18},
    '0x668f7dfb8c8D716839fa5dBEa317D8723Ebe6110': {'symbol': 'SMSCT', 'decimals': 0},
    '0xD1E06952708771f71E6dd18F06EE418F6e8FC564': {'symbol': 'GAZE', 'decimals': 18},
    '0xF0ACF8949e705E0ebb6CB42c2164B0B986454223': {'symbol': 'BRTR', 'decimals': 8},
    '0x009668A9691e456972c8ec4Cc84e99486308b84D': {'symbol': 'TERA', 'decimals': 18},
    '0x40284109c3309A7C3439111bFD93BF5E0fBB706c': {'symbol': 'MOV', 'decimals': 18},
    '0xE5B826Ca2Ca02F09c1725e9bd98d9a8874C30532': {'symbol': 'ZEON', 'decimals': 18},
    '0x58fcAa970339a9b1f8c0a5b4F3Fcd7af2bA3075E': {'symbol': 'POLAR', 'decimals': 18},
    '0xc690F7C7FcfFA6a82b79faB7508c466FEfdfc8c5': {'symbol': 'LYM', 'decimals': 18},
    '0xbEEf3bB9dA340EbdF0f5bae2E85368140d7D85D0': {'symbol': 'MORE', 'decimals': 18},
    '0x9F52c8ecbEe10e00D9faaAc5Ee9Ba0fF6550F511': {'symbol': 'SIPHER', 'decimals': 18},
    '0xD212046F89680aC9F106B9c63f314cc9808e18d5': {'symbol': 'SUKI', 'decimals': 18},
    '0xb3e6Ee8d2c586FA03Ab70Aef96B8aE6D12D64eC7': {'symbol': 'HIFIDENZA', 'decimals': 18},
    '0xa88842Ae47dbe87116cF7001DDdD1b246fcd8262': {'symbol': 'HIENS3', 'decimals': 18},
    '0x0d86EB9f43C57f6FF3BC9E23D8F9d82503f0e84b': {'symbol': 'MAXI', 'decimals': 8},
    '0x47110d43175f7f2C2425E7d15792acC5817EB44f': {'symbol': 'GMI', 'decimals': 18},
    '0x3aF5Ba94C29a8407785f5f6d90eF5d69a8EB2436': {'symbol': 'UWBTC', 'decimals': 8},
    '0xd47bDF574B4F76210ed503e0EFe81B58Aa061F3d': {'symbol': 'TRVL', 'decimals': 18},
    '0xFbe878CED08132bd8396988671b450793C44bC12': {'symbol': 'FOXT', 'decimals': 18},
    '0xc91B523A59ACC63A64f61fc7bBfB4bfc82DD25f2': {'symbol': 'AI', 'decimals': 18},
    '0x0263994554Ec08cc60473dbf8ceE60f9EedbF093': {'symbol': 'POWER', 'decimals': 18},
    '0xD533a949740bb3306d119CC777fa900bA034cd52': {'symbol': 'CRV', 'decimals': 18},
    '0x38E491A71291CD43E8DE63b7253E482622184894': {'symbol': 'AAMMUNISNXWETH', 'decimals': 18},
    '0x7aF89c8A06719271A96e62E290Ea9Ed192E73FC1': {'symbol': 'GMM', 'decimals': 18},
    '0xB4Bd4628e6EFb0Cb521D9ec35050C75840320374': {'symbol': 'FRETH', 'decimals': 18},
    '0xd075e95423C5c4BA1E122CaE0f4CdFA19b82881b': {'symbol': 'WPE', 'decimals': 18},
    '0x913c9509b94D32Ee38B0d98431C1FF109d24ff16': {'symbol': 'AIPRO', 'decimals': 18},
    '0x14cC8DfaF2258E1B8B2869300dBa1B734dc0fE43': {'symbol': 'KTT', 'decimals': 18},
    '0xe2f2a5C287993345a840Db3B0845fbC70f5935a5': {'symbol': 'MUSD', 'decimals': 18},
    '0x38d1B0D157529Bd5D936719A8a5F8379aFB24fAA': {'symbol': 'DKYC', 'decimals': 18},
    '0xf51ccB15d3F2D304Ae6C409303e8b3a3e397A80b': {'symbol': 'OPP', 'decimals': 18},
    '0xe5867608b51A2c9C78B9587355cC093140A49B0A': {'symbol': 'SMS', 'decimals': 3},
    '0x12fD19DAC0Fab61bEd5e0F09091B470C452D4d61': {'symbol': 'EC', 'decimals': 18},
    '0x0c93B616933b0CD03B201b29Cd8a22681DD9E0d9': {'symbol': 'HGOLD', 'decimals': 8},
    '0x82698aeCc9E28e9Bb27608Bd52cF57f704BD1B83': {'symbol': 'BB-A-USDC', 'decimals': 18},
    '0x66186008C1050627F979d464eABb258860563dbE': {'symbol': 'MDS', 'decimals': 18},
    '0x8765b1A0eb57ca49bE7EACD35b24A574D0203656': {'symbol': 'MGH', 'decimals': 18},
    '0xAa4e3edb11AFa93c41db59842b29de64b72E355B': {'symbol': 'MFI', 'decimals': 18},
    '0x8610632B514761597d91ECF20F485cd8163a3033': {'symbol': 'BET', 'decimals': 18},
    '0xaD5Fe5B0B8eC8fF4565204990E4405B2Da117d8e': {'symbol': 'TRXC', 'decimals': 0},
    '0xCee1d3c3A02267e37E6B373060F79d5d7b9e1669': {'symbol': 'YFFI', 'decimals': 18},
    '0x1a23a6BfBAdB59fa563008c0fB7cf96dfCF34Ea1': {'symbol': 'COFI', 'decimals': 18},
    '0x7a2Bc711E19ba6aff6cE8246C546E8c4B4944DFD': {'symbol': 'WAXE', 'decimals': 8},
    '0x677ddbd918637E5F2c79e164D402454dE7dA8619': {'symbol': 'VUSD', 'decimals': 18},
    '0x14449dE7937fE1C1207006E92f89dEe17BbE418A': {'symbol': 'MLC', 'decimals': 18},
    '0xa291B47EFE802de81Ac425c67D2B8d2925FA31A4': {'symbol': 'MVRC', 'decimals': 18},
    '0xDB5C3C46E28B53a39C255AA39A411dD64e5fed9c': {'symbol': 'NCR', 'decimals': 18},
    '0x2602278EE1882889B946eb11DC0E810075650983': {'symbol': 'VADER', 'decimals': 18},
    '0x27778E14Ce36d3B85e1efFeB43816a17bBB7088A': {'symbol': 'LGOLD', 'decimals': 18},
    '0xd109b2A304587569c84308c55465cd9fF0317bFB': {'symbol': 'AAMMBPTBALWETH', 'decimals': 18},
    '0x337AF08bb6980Ecb68389C5ed8876D08643aBF8a': {'symbol': 'VACHI', 'decimals': 18},
    '0xB755D5bC7de83232b9df1886bD5CdB38895933B0': {'symbol': 'HIMFERS', 'decimals': 18},
    '0xC28E27870558cF22ADD83540d2126da2e4b464c2': {'symbol': 'SASHIMI', 'decimals': 18},
    '0x6286A9e6f7e745A6D884561D88F94542d6715698': {'symbol': 'TECH', 'decimals': 18},
    '0x35bD01FC9d6D5D81CA9E055Db88Dc49aa2c699A8': {'symbol': 'FWB', 'decimals': 18},
    '0x727f064A78DC734D33eEc18d5370aef32Ffd46e4': {'symbol': 'ORION', 'decimals': 18},
    '0x8686525d6627A25C68De82c228448f43c97999F2': {'symbol': 'LY', 'decimals': 9},
    '0xFC4B8ED459e00e5400be803A9BB3954234FD50e3': {'symbol': 'AWBTC', 'decimals': 8},
    '0xC0F9bD5Fa5698B6505F643900FFA515Ea5dF54A9': {'symbol': 'DONUT', 'decimals': 18},
    '0x187Eff9690E1F1A61d578C7c492296eaAB82701a': {'symbol': 'MOAR', 'decimals': 18},
    '0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72': {'symbol': 'ENS', 'decimals': 18},
    '0xA361718326c15715591c299427c62086F69923D9': {'symbol': 'ABUSD', 'decimals': 18},
    '0x3FAb0bBAa03BCEAF7C49E2b12877dB0142BE65FC': {'symbol': 'CAST', 'decimals': 8},
    '0x054D64b73d3D8A21Af3D764eFd76bCaA774f3Bb2': {'symbol': 'PPAY', 'decimals': 18},
    '0x329cf160F30D21006bCD24b67EAde561E54CDE4c': {'symbol': 'CARE', 'decimals': 18},
    '0x3d6F0DEa3AC3C607B3998e6Ce14b6350721752d9': {'symbol': 'CARDS', 'decimals': 18},
    '0xe5eF42D0E5E4Aa6B36c613d00Db8dad303d505F3': {'symbol': 'MSOT', 'decimals': 18},
    '0xE1c7E30C42C24582888C758984f6e382096786bd': {'symbol': 'XCUR', 'decimals': 8},
    '0xba58444C8050Ed9385b7417533A73644036d21Eb': {'symbol': 'LOGT', 'decimals': 18},
    '0x3231Cb76718CDeF2155FC47b5286d82e6eDA273f': {'symbol': 'EURE', 'decimals': 18},
    '0x090FC4D5436D98E13473c4d6109De6A18BCfB0d4': {'symbol': 'SKY', 'decimals': 18},
    '0x4AF5ff1A60a6eF6C7c8f9C4E304cD9051fCa3Ec0': {'symbol': 'RGP', 'decimals': 18},
    '0xdab396cCF3d84Cf2D07C4454e10C8A6F5b008D2b': {'symbol': 'GFI', 'decimals': 18},
    '0xB56A1f3310578f23120182Fb2e58c087EFE6e147': {'symbol': 'ACYC', 'decimals': 18},
    '0x41D5D79431A913C4aE7d69a668ecdfE5fF9DFB68': {'symbol': 'INV', 'decimals': 18},
    '0xe0bCEEF36F3a6eFDd5EEBFACD591423f8549B9D5': {'symbol': 'FACTR', 'decimals': 18},
    '0xB620Be8a1949AA9532e6a3510132864EF9Bc3F82': {'symbol': 'LFT', 'decimals': 18},
    '0x5026F006B85729a8b14553FAE6af249aD16c9aaB': {'symbol': 'WOJAK', 'decimals': 18},
    '0x80CE3027a70e0A928d9268994e9B85d03Bd4CDcf': {'symbol': 'LKR', 'decimals': 18},
    '0xBA11D00c5f74255f56a5E366F4F77f5A186d7f55': {'symbol': 'BAND', 'decimals': 18},
    '0xA9639160481b625ba43677bE753E0A70bf58c647': {'symbol': 'RBXS', 'decimals': 18},
    '0x823556202e86763853b40e9cDE725f412e294689': {'symbol': 'ASTO', 'decimals': 18},
    '0xEA38eAa3C86c8F9B751533Ba2E562deb9acDED40': {'symbol': 'FUEL', 'decimals': 18},
    '0x16594930D16f3970e1a4317c6016555cb2e7b7Fc': {'symbol': 'TKB', 'decimals': 18},
    '0x07F9702ce093db82dfdC92c2C6E578d6EA8D5E22': {'symbol': 'OBT', 'decimals': 18},
    '0xA5E412ba6FcA1e07b15dEFcaA4236Ff7B5A7f086': {'symbol': 'CBANK', 'decimals': 18},
    '0xf552b656022c218C26dAd43ad88881Fc04116F76': {'symbol': 'MORK', 'decimals': 4},
    '0xCDb9D30a3BA48CdfCB0EcbE19317E6cf783672f1': {'symbol': 'MNDCC', 'decimals': 18},
    '0x53011E93F21EC7A74cdfBB7E6548F1ABCE306833': {'symbol': 'LEXE', 'decimals': 18},
    '0xb6c4267C4877BB0D6b1685Cfd85b0FBe82F105ec': {'symbol': 'REL', 'decimals': 18},
    '0x8A2279d4A90B6fe1C4B30fa660cC9f926797bAA2': {'symbol': 'CHR', 'decimals': 6},
    '0x73A9fb46e228628f8f9BB9004eCa4f4F529D3998': {'symbol': 'WLEO', 'decimals': 3},
    '0x719e7f0daDFdeA25B78595da944F44d15d7E6795': {'symbol': 'MUSK', 'decimals': 18},
    '0xCA0e7269600d353F70b14Ad118A49575455C0f2f': {'symbol': 'AMLT', 'decimals': 18},
    '0xB396C717105f2F9Ba81007c7FB774fb06d0fb937': {'symbol': 'MMI', 'decimals': 18},
    '0x5D843Fa9495d23dE997C394296ac7B4D721E841c': {'symbol': 'RELAY', 'decimals': 18},
    '0x77FbA179C79De5B7653F68b5039Af940AdA60ce0': {'symbol': 'FORTH', 'decimals': 18},
    '0x1Af2eAEAf2B1d9dDA800861268e6bbB3995a6C3b': {'symbol': '1ECO', 'decimals': 18},
    '0x92868A5255C628dA08F550a858A802f5351C5223': {'symbol': 'BRIDGE', 'decimals': 18},
    '0x0ea20e7fFB006d4Cfe84df2F72d8c7bD89247DB0': {'symbol': 'AAMMUNICRVWETH', 'decimals': 18},
    '0xaeE433ADeBe0FBB88dAa47eF0C1A513CaA52EF02': {'symbol': 'TOON', 'decimals': 18},
    '0x634239cfA331Df0291653139d1a6083B9cf705e3': {'symbol': 'DES', 'decimals': 18},
    '0xC5253C32BB72921b77ae18CF329dAd5e6C3de5A4': {'symbol': 'NOBI', 'decimals': 18},
    '0xF063fE1aB7a291c5d06a86e14730b00BF24cB589': {'symbol': 'SALE', 'decimals': 18},
    '0xf4F618Eff5eF36Cde2FCa4FBD86554c62Fb1382B': {'symbol': 'AGV', 'decimals': 18},
    '0xa3BeD4E1c75D00fa6f4E5E6922DB7261B5E9AcD2': {'symbol': 'MTA', 'decimals': 18},
    '0x7A647637E75d34A7798Ba06BD1aDF5c084bb5dd5': {'symbol': 'BSY', 'decimals': 18},
    '0x8C6bf16C273636523C29Db7DB04396143770F6A0': {'symbol': 'AAA', 'decimals': 18},
    '0x967B0c95295ead8fAef70d26A7846AeCD349AAFf': {'symbol': 'HACHI', 'decimals': 18},
    '0xeD35af169aF46a02eE13b9d79Eb57d6D68C1749e': {'symbol': 'OMI', 'decimals': 18},
    '0x9dAeF41E08Bc88183572b03796C96a7ace1Ec9d4': {'symbol': 'KOYO', 'decimals': 18},
    '0x850aAB69f0e0171A9a49dB8BE3E71351c8247Df4': {'symbol': 'KONO', 'decimals': 18},
    '0x9416bA76e88D873050A06e5956A3EBF10386b863': {'symbol': 'VIRTUE', 'decimals': 18},
    '0x2C10c0dE3362FF21F8ED6bC7F4AC5e391153fD2c': {'symbol': 'VIZ', 'decimals': 18},
    '0xE3a64A3c4216B83255b53Ec7eA078B13f21a7daD': {'symbol': 'DFGL', 'decimals': 18},
    '0x8167D3B1024cB51A2DD1B4d889ddf7023420796a': {'symbol': 'NOKU', 'decimals': 18},
    '0xc324a2f6b05880503444451B8b27e6f9e63287Cb': {'symbol': 'XUC', 'decimals': 18},
    '0x6B4c7A5e3f0B99FCD83e9c089BDDD6c7FCe5c611': {'symbol': 'MM', 'decimals': 18},
    '0x39142c18B6db2a8A41B7018f49E1478837560caD': {'symbol': 'STATS', 'decimals': 9},
    '0x36919a60a2B67B6d2329863093D180d23d5A0308': {'symbol': 'KUSUNOKI', 'decimals': 18},
    '0xe3818504c1B32bF1557b16C238B2E01Fd3149C17': {'symbol': 'PLR', 'decimals': 18},
    '0x69d47846bb7B884062FaBc08A4A7dEADCd98c45B': {'symbol': 'RUSHAI', 'decimals': 18},
    '0xb29663Aa4E2e81e425294193616c1B102B70a158': {'symbol': 'LDN', 'decimals': 18},
    '0x44709a920fCcF795fbC57BAA433cc3dd53C44DbE': {'symbol': 'RADAR', 'decimals': 18},
    '0x6982508145454Ce325dDbE47a25d4ec3d2311933': {'symbol': 'PEPE', 'decimals': 18},
    '0x24E89bDf2f65326b94E36978A7EDeAc63623DAFA': {'symbol': 'TKING', 'decimals': 18},
    '0xbB97e381F1d1e94ffa2A5844F6875e6146981009': {'symbol': 'WBX', 'decimals': 18},
    '0x6a8fee0E33Cb65a7E8D21BadCA62E87639ef74B3': {'symbol': 'PDX', 'decimals': 18},
    '0x1FCdcE58959f536621d76f5b7FfB955baa5A672F': {'symbol': 'FOR', 'decimals': 18},
    '0x1cF4592ebfFd730c7dc92c1bdFFDfc3B9EfCf29a': {'symbol': 'WAVES', 'decimals': 18},
    '0x6069c9223e8a5DA1ec49ac5525d4BB757Af72Cd8': {'symbol': 'MUSK', 'decimals': 18},
    '0x46be62D1Fc217063c1bF1f35D788690ceC7a60cA': {'symbol': 'SUKO', 'decimals': 18},
    '0xF5717f5DF41eA67Ef67DFD3c1d02F9940bcF5d08': {'symbol': 'SNN', 'decimals': 3},
    '0x120a3879da835A5aF037bB2d1456beBd6B54d4bA': {'symbol': 'RVST', 'decimals': 18},
    '0xB64fde8f199F073F41c132B9eC7aD5b61De0B1B7': {'symbol': 'PRV', 'decimals': 9},
    '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984': {'symbol': 'UNI', 'decimals': 18},
    '0x0cf75471B0FbF76A315A75eBDa6600ab92Fdc924': {'symbol': 'DTH', 'decimals': 18},
    '0xbe7A4f88c6B15a722aa960B487BAdF9880184Ec5': {'symbol': 'SPLC', 'decimals': 18},
    '0xC775C0C30840Cb9F51e21061B054ebf1A00aCC29': {'symbol': 'PSL', 'decimals': 5},
    '0x997507cc49FBf0CD6ce5e1EE543218556fAFdEBc': {'symbol': 'BT', 'decimals': 18},
    '0xED0e2041BFb5a426e5ED426A73765624E08BbB75': {'symbol': 'FX1', 'decimals': 18},
    '0xF4134146AF2d511Dd5EA8cDB1C4AC88C57D60404': {'symbol': 'SNC', 'decimals': 18},
    '0xbC396689893D065F41bc2C6EcbeE5e0085233447': {'symbol': 'PERP', 'decimals': 18},
    '0x95A1796437BAd6502d1c1cce165Cd76E522409a9': {'symbol': 'OCCT', 'decimals': 18},
    '0xB8fA12f8409DA31A4fc43D15c4c78C33d8213B9B': {'symbol': 'CALI', 'decimals': 18},
    '0x2AF5D2aD76741191D15Dfe7bF6aC92d4Bd912Ca3': {'symbol': 'LEO', 'decimals': 18},
    '0x5f4361971667C8198706dEc8C59cF7607e405C6e': {'symbol': 'FSC', 'decimals': 8},
    '0x0eb3032bcAc2BE1fa95E296442F225edb80fc3CD': {'symbol': 'ATC', 'decimals': 18},
    '0x418D75f65a02b3D53B2418FB8E1fe493759c7605': {'symbol': 'BNB', 'decimals': 18},
    '0xEd279fDD11cA84bEef15AF5D39BB4d4bEE23F0cA': {'symbol': 'LUSD3CRV', 'decimals': 18},
    '0xA9e201A4e269d6cd5E9F0FcbcB78520cf815878B': {'symbol': 'AAMMUNIRENWETH', 'decimals': 18},
    '0x9b8e9d523D1D6bC8EB209301c82C7D64D10b219E': {'symbol': 'SPROUT', 'decimals': 18},
    '0xB2279B6769CFBa691416F00609b16244c0cF4b20': {'symbol': 'WAIF', 'decimals': 18},
    '0xd8446236FA95b9b5f9fd0f8E7Df1a944823c683d': {'symbol': 'NEEO', 'decimals': 18},
    '0x0AfFa06e7Fbe5bC9a764C979aA66E8256A631f02': {'symbol': 'PLBT', 'decimals': 6},
    '0xe4D75e9b493458D032A5C3CC1eE9B0712C1EcE06': {'symbol': 'BITCCA', 'decimals': 2},
    '0x11afE7Fa792589dd1236257f99ba09f510460Ad9': {'symbol': 'LNKO', 'decimals': 8},
    '0xE63684BcF2987892CEfB4caA79BD21b34e98A291': {'symbol': 'KFC', 'decimals': 18},
    '0x4D953cf077c0C95Ba090226E59A18FcF97db44EC': {'symbol': 'MINI', 'decimals': 18},
    '0x0AeE8703D34DD9aE107386d3eFF22AE75Dd616D1': {'symbol': 'SLICE', 'decimals': 18},
    '0xDF4Ef6EE483953fE3B84ABd08C6A060445c01170': {'symbol': 'WACME', 'decimals': 8},
    '0x64CdF819d3E75Ac8eC217B3496d7cE167Be42e80': {'symbol': 'IPL', 'decimals': 18},
    '0xcbE771323587EA16dACB6016e269D7F08A7ACC4E': {'symbol': 'SPO', 'decimals': 18},
    '0xf7B098298f7C69Fc14610bf71d5e02c60792894C': {'symbol': 'GUP', 'decimals': 3},
    '0xF4d861575ecC9493420A3f5a14F85B13f0b50EB3': {'symbol': 'FCL', 'decimals': 18},
    '0x52928C95C4C7e934E0EfcfAB08853A0E4558861d': {'symbol': 'HART', 'decimals': 18},
    '0x1559FA1b8F28238FD5D76D9f434ad86FD20D1559': {'symbol': 'EDEN', 'decimals': 18},
    '0xFA3118B34522580c35Ae27F6cf52da1dBb756288': {'symbol': 'LET', 'decimals': 6},
    '0xA2cd3D43c775978A96BdBf12d733D5A1ED94fb18': {'symbol': 'XCN', 'decimals': 18},
    '0x3B5e2DF103a037509dcC8430E5462B9CD8F75f49': {'symbol': 'GEMX', 'decimals': 9},
    '0xEA01906843Ea8D910658a2c485ffCe7C104AB2b6': {'symbol': 'QTO', 'decimals': 18},
    '0x467719aD09025FcC6cF6F8311755809d45a5E5f3': {'symbol': 'AXL', 'decimals': 6},
    '0x030bA81f1c18d280636F32af80b9AAd02Cf0854e': {'symbol': 'AWETH', 'decimals': 18},
    '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F': {'symbol': 'SNX', 'decimals': 18},
    '0x84342e932797FC62814189f01F0Fb05F52519708': {'symbol': 'NHT', 'decimals': 18},
    '0x52662717e448be36Cb54588499D5A8328BD95292': {'symbol': 'TENSHI', 'decimals': 18},
    '0x2bBA3CF6DE6058cc1B4457Ce00deb359E2703d7F': {'symbol': 'HSC', 'decimals': 18},
    '0x0B4680e848f842b86E4Df38D9FA0d9f9419F37EC': {'symbol': 'BST', 'decimals': 18},
    '0x58f9102bF53Cf186682Bd9a281d3Cd3c616eEc41': {'symbol': 'TRL', 'decimals': 18},
    '0xa41F142b6eb2b164f8164CAE0716892Ce02f311f': {'symbol': 'AVG', 'decimals': 18},
    '0xDA9FDAb21bC4A5811134A6E0Ba6CA06624e67c07': {'symbol': 'QUIDD', 'decimals': 18},
    '0x6a6c2adA3Ce053561C2FbC3eE211F23d9b8C520a': {'symbol': 'TON', 'decimals': 18},
    '0x77D0cb0ab54F9E74b9405a5B3f60DA06A78F1AAd': {'symbol': 'WMLX', 'decimals': 0},
    '0xD417144312DbF50465b1C641d016962017Ef6240': {'symbol': 'CQT', 'decimals': 18},
    '0xBEaB712832112bd7664226db7CD025B153D3af55': {'symbol': 'BRIGHT', 'decimals': 18},
    '0xD49ff13661451313cA1553fd6954BD1d9b6E02b9': {'symbol': 'ELEC', 'decimals': 18},
    '0x1dD80016e3d4ae146Ee2EBB484e8edD92dacC4ce': {'symbol': 'LEAD', 'decimals': 18},
    '0x16f812Be7FfF02cAF662B85d5d58a5da6572D4Df': {'symbol': 'UTT', 'decimals': 8},
    '0x3B604747ad1720C01ded0455728b62c0d2F100F0': {'symbol': 'WAGMIGAMES', 'decimals': 18},
    '0xc5019E129b75D380d3d837B8e609dEc6c8f5d044': {'symbol': 'GN', 'decimals': 9},
    '0x7b3D36Eb606f873A75A6aB68f8c999848B04F935': {'symbol': 'LOOT', 'decimals': 18},
    '0xF1d1A5306daaE314AF6c5d027a492B313e07e1A0': {'symbol': 'ENV', 'decimals': 18},
    '0x16F78145AD0B9Af58747e9A97EBd99175378bd3D': {'symbol': 'GDO', 'decimals': 18},
    '0xf058501585023d040eA9493134ED72C083553eED': {'symbol': 'DMX', 'decimals': 18},
    '0x228ba514309FFDF03A81a205a6D040E429d6E80C': {'symbol': 'GSC', 'decimals': 18},
    '0xCBe7142F5c16755D8683BA329EFA1ABF7b54482d': {'symbol': 'MVEDA', 'decimals': 8},
    '0xf6c0aA7eBFE9992200C67E5388E4F42da49E1783': {'symbol': 'USD1', 'decimals': 2},
    '0x628aB8b061FEA2AF1239B68eFA5e46135D186666': {'symbol': 'SILV', 'decimals': 8},
    '0xF7e40AF830a2EcbFe72C5210C1Ca272e3Aa7cA1B': {'symbol': 'CTPL', 'decimals': 18},
    '0x4c3bAe16c79c30eEB1004Fb03C878d89695e3a99': {'symbol': 'AUTUMN', 'decimals': 18},
    '0xe9B076B476D8865cDF79D1Cf7DF420EE397a7f75': {'symbol': 'FUND', 'decimals': 9},
    '0x525794473F7ab5715C81d06d10f52d11cC052804': {'symbol': 'TSHP', 'decimals': 18},
    '0xfFffFffF2ba8F66D4e51811C5190992176930278': {'symbol': 'COMBO', 'decimals': 18},
    '0x29FA1fee0F4F0Ab0e36eF7Ab8d7a35439eC6be75': {'symbol': 'DVT', 'decimals': 18},
    '0xDa007777D86AC6d989cC9f79A73261b3fC5e0DA0': {'symbol': 'NODE', 'decimals': 18},
    '0xE340B25fE32B1011616bb8EC495A4d503e322177': {'symbol': 'AAMMUNIDAIUSDC', 'decimals': 18},
    '0x809826cceAb68c387726af962713b64Cb5Cb3CCA': {'symbol': 'NCASH', 'decimals': 18},
    '0x308564DC5217c39386F5eaE96545159e1D396661': {'symbol': 'HLP', 'decimals': 18},
    '0xD56daC73A4d6766464b38ec6D91eB45Ce7457c44': {'symbol': 'PAN', 'decimals': 18},
    '0x6E0615a03eD9527a6013FcD5B556E36EF4DaB1FF': {'symbol': 'HNB', 'decimals': 18},
    '0x1C9A2d6b33B4826757273D47ebEe0e2DddcD978B': {'symbol': 'FFRAX', 'decimals': 8},
    '0x4Fabb145d64652a948d72533023f6E7A623C7C53': {'symbol': 'BUSD', 'decimals': 18},
    '0x2F0F622a93c85C43Fd47f4Ab119360f5846026C4': {'symbol': 'MATSUSWAP', 'decimals': 9},
    '0x5d60d8d7eF6d37E16EBABc324de3bE57f135e0BC': {'symbol': 'MYB', 'decimals': 18},
    '0xE477292f1B3268687A29376116B0ED27A9c76170': {'symbol': 'PLAY', 'decimals': 18},
    '0x2b67D1a87A8D8B280A23e97bC55095215eE0ec53': {'symbol': 'CPI', 'decimals': 18},
    '0x0B342C51d1592C41068d5D4b4DA4A68C0a04d5A4': {'symbol': 'ONES', 'decimals': 18},
    '0xF527d24391C767B86b8e91385e1cE9C54D230A2B': {'symbol': 'TZKI', 'decimals': 9},
    '0x83249c6794BCa5a77Eb8c0AF9F1A86E055459cea': {'symbol': 'GIGA', 'decimals': 9},
    '0x8052327F1BAF94A9DC8B26b9100f211eE3774f54': {'symbol': 'ATD', 'decimals': 18},
    '0x32E6C34Cd57087aBBD59B5A4AECC4cB495924356': {'symbol': 'BTBS', 'decimals': 18},
    '0x12A7530D6f9e1a9B0351D78aB711f7C2c033873A': {'symbol': 'HOOT', 'decimals': 18},
    '0x1E9D0bB190Ac34492aa11B80D28c1C86487a341F': {'symbol': 'NEKO', 'decimals': 18},
    '0x5c147e74D63B1D31AA3Fd78Eb229B65161983B2b': {'symbol': 'WFLOW', 'decimals': 18},
    '0x425c5b7B55F9C981C71935418ED044E79D8080E2': {'symbol': 'MADHAT', 'decimals': 9},
    '0xD101dCC414F310268c37eEb4cD376CcFA507F571': {'symbol': 'RSC', 'decimals': 18},
    '0xE5CAeF4Af8780E59Df925470b050Fb23C43CA68C': {'symbol': 'FRM', 'decimals': 6},
    '0x467Bccd9d29f223BcE8043b84E8C8B282827790F': {'symbol': 'TEL', 'decimals': 2},
    '0x3c72fca8523686fd9e5740b0826FA4bB376E0241': {'symbol': 'FIT', 'decimals': 18},
    '0x3fE7940616e5Bc47b0775a0dccf6237893353bB4': {'symbol': 'IDLEDAIYIELD', 'decimals': 18},
    '0x25B6325f5BB1c1E03cfbC3e53F470E1F1ca022E3': {'symbol': 'LML', 'decimals': 18},
    '0x1f961BCEAEF8eDF6fb2797C0293FfBDe3E994614': {'symbol': 'DINGO', 'decimals': 9},
    '0x0d438F3b5175Bebc262bF23753C1E53d03432bDE': {'symbol': 'WNXM', 'decimals': 18},
    '0x6251E725CD45Fb1AF99354035a414A2C0890B929': {'symbol': 'MXT', 'decimals': 18},
    '0x24B47299E756aF0571F512232A3629E0DaBb52Ed': {'symbol': 'CVT', 'decimals': 18},
    '0xf278c1CA969095ffddDED020290cf8B5C424AcE2': {'symbol': 'RUFF', 'decimals': 18},
    '0xdD0020B1D5Ba47A54E2EB16800D73Beb6546f91A': {'symbol': 'AXPR', 'decimals': 18},
    '0x14da230d6726C50F759Bc1838717F8CE6373509C': {'symbol': 'KAT', 'decimals': 18},
    '0xb8919522331C59f5C16bDfAA6A121a6E03A91F62': {'symbol': 'HOME', 'decimals': 6},
    '0x26524e35E44e265eB181C7Ba6eC084983181aaE3': {'symbol': 'DOTR', 'decimals': 9},
    '0xFD6C31bb6F05Fc8dB64F4b740Ab758605c271FD8': {'symbol': 'CTCN', 'decimals': 18},
    '0x173E552Bf97BBD50b455514ac52991Ef639ba703': {'symbol': 'SHIDO', 'decimals': 9},
    '0x3540DFCaD7CF102a2E44AA0e2132FAb1c89d7eAe': {'symbol': 'SWAI', 'decimals': 18},
    '0xc5102fE9359FD9a28f877a67E36B0F050d81a3CC': {'symbol': 'HOP', 'decimals': 18},
    '0x4C19596f5aAfF459fA38B0f7eD92F11AE6543784': {'symbol': 'TRU', 'decimals': 8},
    '0xc1AD0aA69454603a5dEe55Cf9BD9341e01328544': {'symbol': 'HIGAZERS', 'decimals': 18},
    '0x824E35f7A75324f99300aFAC75ECF7354E17Ea26': {'symbol': 'TIA', 'decimals': 9},
    '0x8EEdEFe828A0f16C8fc80e46a87Bc0f1De2d960c': {'symbol': 'DGMV', 'decimals': 18},
    '0xd32641191578Ea9b208125dDD4EC5E7B84FcaB4C': {'symbol': 'TMED', 'decimals': 18},
    '0xb23d80f5FefcDDaa212212F028021B41DEd428CF': {'symbol': 'PRIME', 'decimals': 18},
    '0xf6Fd82dEdBBe0ffadb5e1ecc2a283AB52B9ED2B0': {'symbol': 'HEAL', 'decimals': 18},
    '0x0e0989b1f9B8A38983c2BA8053269Ca62Ec9B195': {'symbol': 'POE', 'decimals': 8},
    '0xF1290473E210b2108A85237fbCd7b6eb42Cc654F': {'symbol': 'HEDG', 'decimals': 18},
    '0x00B7db6B4431e345eee5cc23D21E8dbC1d5cADA3': {'symbol': 'CTC', 'decimals': 18},
    '0xf7e945FcE8F19302AaCc7E1418b0A0bdef89327B': {'symbol': 'IZE', 'decimals': 8},
    '0x5224F552f110eC78E6E0468138950AE5F3040942': {'symbol': 'ANOM', 'decimals': 18},
    '0xeaf61FC150CD5c3BeA75744e830D916E60EA5A9F': {'symbol': 'TYPE', 'decimals': 4},
    '0x5AC83BfbFcebb3397A40fD259dBE7a4bE04237d3': {'symbol': 'LBT', 'decimals': 18},
    '0xd7C9F0e536dC865Ae858b0C0453Fe76D13c3bEAc': {'symbol': 'XAI', 'decimals': 18},
    '0x05030203674173Fa6dF6F9f7E34D6E70E9a761D7': {'symbol': 'MU', 'decimals': 18},
    '0xC62DEF1701309bb76e6B39B6ab8B5fAC910A3C87': {'symbol': 'MTRA', 'decimals': 18},
    '0x9040e237C3bF18347bb00957Dc22167D0f2b999d': {'symbol': 'STND', 'decimals': 18},
    '0x21cf09BC065082478Dcc9ccB5fd215A978Dc8d86': {'symbol': 'JEM', 'decimals': 18},
    '0xFe4BEb9217cdDf2422d4bd65449b76d807b30fe1': {'symbol': 'WHITE', 'decimals': 9},
    '0x7ca62545a380e7d71F8F5CFa14B9211002075930': {'symbol': 'SVS', 'decimals': 18},
    '0xf3AE5d769e153Ef72b4e3591aC004E89F48107a1': {'symbol': 'DPR', 'decimals': 18},
    '0x5F18ea482ad5cc6BC65803817C99f477043DcE85': {'symbol': 'AGI', 'decimals': 18},
    '0xa774FFB4AF6B0A91331C084E1aebAE6Ad535e6F3': {'symbol': 'FLEXUSD', 'decimals': 18},
    '0xB62D46A21fef0fADb5b92fC5eF7077E71e65631c': {'symbol': 'ACOIN', 'decimals': 18},
    '0x5d43b66da68706D39f6C97F7f1415615672b446b': {'symbol': 'ROG', 'decimals': 18},
    '0x2aF1dF3AB0ab157e1E2Ad8F88A7D04fbea0c7dc6': {'symbol': 'BED', 'decimals': 18},
    '0x196c81385Bc536467433014042788Eb707703934': {'symbol': 'CTASK', 'decimals': 18},
    '0xE66b3AA360bB78468c00Bebe163630269DB3324F': {'symbol': 'MTO', 'decimals': 18},
    '0xd1ba9BAC957322D6e8c07a160a3A8dA11A0d2867': {'symbol': 'HMT', 'decimals': 18},
    '0x7D1356BfcAA08AF4Ab0D643AF1B6b4cd75Ccc2d0': {'symbol': 'OKINU', 'decimals': 18},
    '0x2A69655C22eDa32ff48d315bB26ED45f150700b4': {'symbol': 'WXTZ', 'decimals': 6},
    '0xF3b3Cad094B89392fcE5faFD40bC03b80F2Bc624': {'symbol': 'PAT', 'decimals': 18},
    '0xeCe8531550Fc599ccB2c7734494FBc358d0a9483': {'symbol': 'PP', 'decimals': 18},
    '0x72955eCFf76E48F2C8AbCCe11d54e5734D6f3657': {'symbol': 'TRV', 'decimals': 18},
    '0xDa70FBaC9e1C9Ac3475c24A3A847f85f1388C429': {'symbol': 'YOURWALLET', 'decimals': 18},
    '0x00a8b738E453fFd858a7edf03bcCfe20412f0Eb0': {'symbol': 'ALBT', 'decimals': 18},
    '0x1D96Fd43EE07Aa79f8Fd003cbdF404fb5ce41AD2': {'symbol': 'QWLA', 'decimals': 18},
    '0x269616D549D7e8Eaa82DFb17028d0B212D11232A': {'symbol': 'PUNK', 'decimals': 18},
    '0xfB559CE67Ff522ec0b9Ba7f5dC9dc7EF6c139803': {'symbol': 'PROB', 'decimals': 18},
    '0x4D67EDef87a5fF910954899f4e5a0AaF107afd42': {'symbol': 'BLUESPARROW', 'decimals': 9},
    '0x130914E1B240a7F4c5D460B7d3a2Fd3846b576fa': {'symbol': 'ANG', 'decimals': 18},
    '0x8cb1d155a5a1d5d667611b7710920fD9D1CD727F': {'symbol': 'AIRX', 'decimals': 8},
    '0xDDB3422497E61e13543BeA06989C0789117555c5': {'symbol': 'COTI', 'decimals': 18},
    '0x15F0EEDF9Ce24fc4b6826E590A8292CE5524a1DA': {'symbol': 'DENA', 'decimals': 18},
    '0xcFCFfE432A48dB53F59c301422d2EdD77B2A88d7': {'symbol': 'TEXAN', 'decimals': 18},
    '0x4c601dc69AfFb0D4Fc8dE1Ac303705e432A4A27E': {'symbol': 'KCT', 'decimals': 18},
    '0xf974b5f9Ac9c6632FeE8b76C61B0242ce69C839D': {'symbol': 'ZYX', 'decimals': 18},
    '0x9E78b8274e1D6a76a0dBbf90418894DF27cBCEb5': {'symbol': 'UNIFI', 'decimals': 18},
    '0x60C24407d01782C2175D32fe7C8921ed732371D1': {'symbol': 'LEMO', 'decimals': 18},
    '0xbD0a4bf098261673d5E6e600Fd87ddcd756e6764': {'symbol': 'HINA', 'decimals': 9},
    '0xE9F721E7419423f11863e83DbD710b5D6127b5b0': {'symbol': 'EPHIAT', 'decimals': 18},
    '0x7409856CAE628f5d578B285B45669b36E7005283': {'symbol': 'JGBP', 'decimals': 18},
    '0x3D3af44cf092a49280e316f09c8f20ecf97BC933': {'symbol': 'UCX', 'decimals': 18},
    '0x4DA9b813057D04BAef4e5800E36083717b4a0341': {'symbol': 'ATUSD', 'decimals': 18},
    '0xa7DE087329BFcda5639247F96140f9DAbe3DeED1': {'symbol': 'STA', 'decimals': 18},
    '0x95b3497bBcCcc46a8F45F5Cf54b0878b39f8D96C': {'symbol': 'UNIDX', 'decimals': 18},
    '0xF0c5831EC3Da15f3696B4DAd8B21c7Ce2f007f28': {'symbol': 'AXIS', 'decimals': 8},
    '0x549020a9Cb845220D66d3E9c6D9F9eF61C981102': {'symbol': 'SIDUS', 'decimals': 18},
    '0x3A82D3111aB5faF39d847D46023d9090261A658F': {'symbol': 'TYC', 'decimals': 18},
    '0xBa3335588D9403515223F109EdC4eB7269a9Ab5D': {'symbol': 'GEAR', 'decimals': 18},
    '0x9d5963ba32e877871dfF3e2E697283dc64066271': {'symbol': 'EDC', 'decimals': 18},
    '0x900dB999074d9277c5DA2A43F252D74366230DA0': {'symbol': 'GIV', 'decimals': 18},
    '0x60Eb57d085C59932d5fAa6c6026268A4386927d0': {'symbol': 'LOCG', 'decimals': 18},
    '0xee573a945B01B788B9287CE062A0CFC15bE9fd86': {'symbol': 'XED', 'decimals': 18},
    '0x0000000000095413afC295d19EDeb1Ad7B71c952': {'symbol': 'LON', 'decimals': 18},
    '0x97872EAfd79940C7b24f7BCc1EADb1457347ADc9': {'symbol': 'STRP', 'decimals': 18},
    '0x777E2ae845272a2F540ebf6a3D03734A5a8f618e': {'symbol': 'RYOSHI', 'decimals': 18},
    '0x72e203a17adD19A3099137c9d7015fD3e2b7DBa9': {'symbol': 'BCP', 'decimals': 18},
    '0x64B5a241b829Bca14Eb42d00097Ba8fD8135841E': {'symbol': 'MEF', 'decimals': 18},
    '0x1C95b093d6C236d3EF7c796fE33f9CC6b8606714': {'symbol': 'BOMB', 'decimals': 0},
    '0x9303EabC860a743aABcc3A1629014CaBcc3F8D36': {'symbol': 'AAMMUNIDAIWETH', 'decimals': 18},
    '0x927159670C50042109d7C0f4aEd0Cee89452433E': {'symbol': 'DGP', 'decimals': 18},
    '0x9B9647431632AF44be02ddd22477Ed94d14AacAa': {'symbol': 'KOK', 'decimals': 18},
    '0xAA2d8c9a8bd0F7945143bfD509bE3ff23dd78918': {'symbol': 'ATNT', 'decimals': 3},
    '0xB870679a7Fa65b924026F496De7f27c1DD0e5C5F': {'symbol': 'PET', 'decimals': 18},
    '0xB9d99C33eA2d86EC5eC6b8A4dD816EBBA64404AF': {'symbol': 'K21', 'decimals': 18},
    '0x4674a4F24C5f63D53F22490Fb3A08eAAAD739ff8': {'symbol': 'BRKL', 'decimals': 18},
    '0xBEDE1F59Fa4412556fEF69f1B9969F2003E3f8c1': {'symbol': 'MVRS', 'decimals': 18},
    '0x33909C9CE97Ce509daB3A038B3eC7ac3d1Be3231': {'symbol': 'TERA2', 'decimals': 4},
    '0x049715C70fDbdD2be4814f76a53Dc3D6F4367756': {'symbol': 'NEZUKO', 'decimals': 18},
    '0xACf3D402e5E2C3eDD5b8129e966017D293F12a4C': {'symbol': 'BAXS', 'decimals': 18},
    '0xf98AB0874b13a7Fdc39D7295DEdd49850A5D426B': {'symbol': 'KIRA', 'decimals': 8},
    '0x0FD3822072ad001aac1c90A09d9506f097F24458': {'symbol': 'NOA', 'decimals': 9},
    '0x6Dca182ac5E3f99985bC4Ee0f726d6472aB1ec55': {'symbol': 'USHI', 'decimals': 18},
    '0x0B319dB00d07C8fAdfaAEf13C910141a5dA0aa8F': {'symbol': 'FTUSD', 'decimals': 18},
    '0x3Da932456D082CBa208FEB0B096d49b202Bf89c8': {'symbol': 'DEGO', 'decimals': 18},
    '0xEBe807bFDf8b2092E36e964C75463A8Aaa642139': {'symbol': 'NPAS', 'decimals': 18},
    '0xbbBBBBB5AA847A2003fbC6b5C16DF0Bd1E725f61': {'symbol': 'BPRO', 'decimals': 18},
    '0xb41380174d0B06181513A5677b60200b93b5Efb4': {'symbol': 'CHS', 'decimals': 18},
    '0xb6EE9668771a79be7967ee29a63D4184F8097143': {'symbol': 'CXO', 'decimals': 18},
    '0x405CE8B2eAeeA7D4Ba5Fc160848cB2A6569e03f0': {'symbol': 'METR', 'decimals': 18},
    '0xEC213F83defB583af3A000B1c0ada660b1902A0F': {'symbol': 'PRE', 'decimals': 18},
    '0xb4FBed161bEbcb37afB1Cb4a6F7cA18b977cCB25': {'symbol': 'DOGES', 'decimals': 18},
    '0x987441856BA4f463544FC68ccBF6a80F434a7956': {'symbol': 'NOWAI', 'decimals': 18},
    '0x667088b212ce3d06a1b553a7221E1fD19000d9aF': {'symbol': 'WINGS', 'decimals': 18},
    '0x6d1DC3928604b00180Bb570BdAe94b9698d33b79': {'symbol': 'UCT', 'decimals': 18},
    '0x58002A6B6E659A16dE9F02F529B10536E307b0d9': {'symbol': 'CHFT', 'decimals': 18},
    '0x1982b2F5814301d4e9a8b0201555376e62F82428': {'symbol': 'ASTETH', 'decimals': 18},
    '0x4E4bFfAA8df6F0dC3e5600bBACf7DA55f37134FC': {'symbol': 'BST', 'decimals': 18},
    '0x765f0C16D1Ddc279295c1a7C24B0883F62d33F75': {'symbol': 'DTX', 'decimals': 18},
    '0x9eEC1A4814323a7396c938bc86aeC46b97F1BD82': {'symbol': 'TOKU', 'decimals': 18},
    '0x15D4c048F83bd7e37d49eA4C83a07267Ec4203dA': {'symbol': 'GALA', 'decimals': 8},
    '0x03042482d64577A7bdb282260e2eA4c8a89C064B': {'symbol': 'CNTR', 'decimals': 18},
    '0x5bBe36152d3CD3eB7183A82470b39b29EedF068B': {'symbol': 'HETH', 'decimals': 18},
    '0x108a850856Db3f85d0269a2693D896B394C80325': {'symbol': 'TGT', 'decimals': 18},
    '0xAF691508BA57d416f895e32a1616dA1024e882D2': {'symbol': 'PNODE', 'decimals': 18},
    '0xc13F4F0F865bAc08F62654B57E38669EbC4747a3': {'symbol': 'CREDS', 'decimals': 18},
    '0x4F604735c1cF31399C6E711D5962b2B3E0225AD3': {'symbol': 'USDGLO', 'decimals': 18},
    '0xd7f5CABdF696D7d1bf384D7688926A4bdB092c67': {'symbol': 'DRC', 'decimals': 18},
    '0xAFe7131a57E44f832cb2dE78ade38CaD644aaC2f': {'symbol': 'MAUSDT', 'decimals': 18},
    '0x04Fa0d235C4abf4BcF4787aF4CF447DE572eF828': {'symbol': 'UMA', 'decimals': 18},
    '0x3C2A309d9005433c1BC2C92EF1bE06489e5bf258': {'symbol': 'WPCI', 'decimals': 8},
    '0x16ECCfDbb4eE1A85A33f3A9B21175Cd7Ae753dB4': {'symbol': 'ROUTE', 'decimals': 18},
    '0x389999216860AB8E0175387A0c90E5c52522C945': {'symbol': 'FEG', 'decimals': 9},
    '0x24E3794605C84E580EEA4972738D633E8a7127c8': {'symbol': 'KTLYO', 'decimals': 18},
    '0x49642110B712C1FD7261Bc074105E9E44676c68F': {'symbol': 'DINO', 'decimals': 18},
    '0xa456b515303B2Ce344E9d2601f91270f8c2Fea5E': {'symbol': 'CORN', 'decimals': 18},
    '0x8811e4Dd5EC5Eb8764b97CC814B1339089717adA': {'symbol': 'BITH', 'decimals': 8},
    '0x50bC2Ecc0bfDf5666640048038C1ABA7B7525683': {'symbol': 'CV', 'decimals': 18},
    '0x8e1b448EC7aDFc7Fa35FC2e885678bD323176E34': {'symbol': 'EGT', 'decimals': 18},
    '0xB2dbF14D0b47ED3Ba02bDb7C954e05A72deB7544': {'symbol': 'MOFI', 'decimals': 18},
    '0x382A1667C9062F0621362F49076Ef6e4fE4C9eC7': {'symbol': 'RUSH', 'decimals': 18},
    '0x0100546F2cD4C9D97f798fFC9755E47865FF7Ee6': {'symbol': 'ALETH', 'decimals': 18},
    '0x6C4C193bff0a117F0C2B516802abbA961A1Eeb12': {'symbol': 'PAPA', 'decimals': 18},
    '0x272F97b7a56a387aE942350bBC7Df5700f8a4576': {'symbol': 'ABAL', 'decimals': 18},
    '0xb2617246d0c6c0087f18703d576831899ca94f01': {'symbol': 'ZIG', 'decimals': 18},
    '0x0d8ca4b20b115D4DA5c13DC45Dd582A5de3e78BF': {'symbol': 'GAI', 'decimals': 18},
    '0xB444Cb2bcB19180602328fA774738cB8DbD6677B': {'symbol': 'MG', 'decimals': 9},
    '0x7458fd786B2fe8CD801C0381f88b61C5071A006F': {'symbol': 'LOA', 'decimals': 18},
    '0x408e41876cCCDC0F92210600ef50372656052a38': {'symbol': 'REN', 'decimals': 18},
    '0xb9EF770B6A5e12E45983C5D80545258aA38F3B78': {'symbol': 'ZCN', 'decimals': 10},
    '0xc8D3DCb63C38607Cb0c9d3F55E8eccE628A01C36': {'symbol': 'MATRIX', 'decimals': 18},
    '0x7420B4b9a0110cdC71fB720908340C03F9Bc03EC': {'symbol': 'JASMY', 'decimals': 18},
    '0xfFE510a92434a0Df346C5E72a3494b043Cf249eB': {'symbol': 'LBXC', 'decimals': 18},
    '0xa117000000f279D81A1D3cc75430fAA017FA5A2e': {'symbol': 'ANT', 'decimals': 18},
    '0x875773784Af8135eA0ef43b5a374AaD105c5D39e': {'symbol': 'IDLE', 'decimals': 18},
    '0x57dB3FfCa78dBbE0eFa0EC745D55f62aa0Cbd345': {'symbol': 'SYMM', 'decimals': 18},
    '0x508626d9a29D13EbA26F843A2BD7bF7b00a45be5': {'symbol': 'KALE', 'decimals': 18},
    '0x8CB924583681cbFE487A62140a994A49F833c244': {'symbol': 'SWAPP', 'decimals': 18},
    '0xdF35988d795d90711e785B488Bb2127692e6f956': {'symbol': 'BABYFLOKI', 'decimals': 9},
    '0x943ED852DadB5C3938ECdC6883718df8142DE4C8': {'symbol': 'FTI', 'decimals': 18},
    '0xc96DF921009B790dfFcA412375251ed1A2b75c60': {'symbol': 'ORME', 'decimals': 8},
    '0xf6af18F94d55B8888B557cb3da0Ec623BA245F80': {'symbol': 'SBLX', 'decimals': 18},
    '0x297E4e5e59Ad72B1B0A2fd446929e76117be0E0a': {'symbol': 'VALOR', 'decimals': 18},
    '0x32DebF31D5ED90ca2e287D306fc23FaFAD9361c6': {'symbol': 'GTR', 'decimals': 18},
    '0x3C03b4EC9477809072FF9CC9292C9B25d4A8e6c6': {'symbol': 'CVR', 'decimals': 18},
    '0x6bd361e10c1afed0d95259e7C0115f3A60e4ea99': {'symbol': 'BOLLY', 'decimals': 18},
    '0x2B89bF8ba858cd2FCee1faDa378D5cd6936968Be': {'symbol': 'WSCRT', 'decimals': 6},
    '0xc36b4311B21Fc0c2eAd46f1eA6ce97c9C4D98D3d': {'symbol': 'CRE8', 'decimals': 18},
    '0xe7aB45162f5979F09B0bDa1CC7DfC97C270EA3d5': {'symbol': 'DOBE', 'decimals': 18},
    '0x8e9A29e7Ed21DB7c5B2E1cd75e676dA0236dfB45': {'symbol': 'HUB', 'decimals': 18},
    '0x14d4c7A788908fbbBD3c1a4Bac4AFf86fE1573EB': {'symbol': 'TAIRYO', 'decimals': 9},
    '0xC626e0619aC79AFEa9281c8eB9b1a9f9D3Fab532': {'symbol': 'FR', 'decimals': 18},
    '0xceEB07Dd26b36287B6d109f0b06d7e8202Ce8c1D': {'symbol': 'GOTG', 'decimals': 18},
    '0x0E8d6b471e332F140e7d9dbB99E5E3822F728DA6': {'symbol': 'ABYSS', 'decimals': 18},
    '0x0FE20E0Fa9C78278702B05c333Cc000034bb69E2': {'symbol': 'ETHMAXY', 'decimals': 18},
    '0x4290563C2D7c255B5EEC87f2D3bD10389f991d68': {'symbol': 'UIP', 'decimals': 18},
    '0x2b1D36f5B61AdDAf7DA7ebbd11B35FD8cfb0DE31': {'symbol': 'ITP', 'decimals': 18},
    '0x6Aa40D02115090D40Dc33C7C5F3Cf05112fA4f83': {'symbol': 'PWRC', 'decimals': 18},
    '0x2559813BBb508C4C79e9CCCE4703Bcb1f149EDd7': {'symbol': 'WAIT', 'decimals': 9},
    '0xc0F1728d9513EFC316D0E93A0758c992f88b0809': {'symbol': 'SWAT', 'decimals': 8},
    '0x02814F435dD04e254Be7ae69F61FCa19881a780D': {'symbol': 'PINA', 'decimals': 18},
    '0xe857734840dc188B4283d5aF14aB8685467aB87D': {'symbol': 'ESGC', 'decimals': 18},
    '0x244517Dc59943E8CdFbD424Bdb3262c5f04a1387': {'symbol': 'FDAI', 'decimals': 18},
    '0x8c1BEd5b9a0928467c9B1341Da1D7BD5e10b6549': {'symbol': 'LSETH', 'decimals': 18},
    '0x048Aa6E69D5117BD665aF4128383a6982B8367b3': {'symbol': 'VXON', 'decimals': 4},
    '0x8947da500Eb47F82df21143D0C01A29862a8C3c5': {'symbol': 'THALES', 'decimals': 18},
    '0xA7dE94227Eb6828497f4343B9923959E41Ae9CD1': {'symbol': 'MUNI', 'decimals': 18},
    '0x1CC30e2EAc975416060Ec6FE682041408420d414': {'symbol': 'KOL', 'decimals': 18},
    '0x9aeB50f542050172359A0e1a25a9933Bc8c01259': {'symbol': 'OIN', 'decimals': 8},
    '0x072987D5B36aD8d45552aEd98879a7101cCdd749': {'symbol': 'BNV', 'decimals': 18},
    '0xA8b12Cc90AbF65191532a12bb5394A714A46d358': {'symbol': 'PBTC35A', 'decimals': 18},
    '0x56b4F8C39E07D4d5D91692acf9d0F6d4d3493763': {'symbol': 'TRISM', 'decimals': 18},
    '0x049399a6B048D52971F7D122aE21A1532722285F': {'symbol': 'FLOT', 'decimals': 18},
    '0xc2BdE1A2fA26890c8E6AcB10C91CC6D9c11F4a73': {'symbol': 'MASK20', 'decimals': 18},
    '0x0adA190c81b814548ddC2F6AdC4a689ce7C1FE73': {'symbol': 'YAXIS', 'decimals': 18},
    '0xC237868a9c5729bdF3173dDDacaa336a0a5BB6e0': {'symbol': 'WGR', 'decimals': 8},
    '0x967da4048cD07aB37855c090aAF366e4ce1b9F48': {'symbol': 'OCEAN', 'decimals': 18},
    '0xFc1C93a2507975E98b9d0E9260Ded61A00152BF1': {'symbol': 'NAVI', 'decimals': 18},
    '0x40370aEd88933021e20CF7C4D67E00417CdA2202': {'symbol': 'XAV', 'decimals': 18},
    '0x656C00e1BcD96f256F224AD9112FF426Ef053733': {'symbol': 'EFI', 'decimals': 18},
    '0xf6c2A37AcC8Ba45874808837486110dC0Afc63C8': {'symbol': 'X1', 'decimals': 2},
    '0x9C4A4204B79dd291D6b6571C5BE8BbcD0622F050': {'symbol': 'TCR', 'decimals': 18},
    '0xD7394087E1DBBE477FE4F1CF373B9Ac9459565fF': {'symbol': 'RET', 'decimals': 8},
    '0xa4Cb3ef5f41a4D89D6FCed22ea8a1C57957629Aa': {'symbol': 'MR', 'decimals': 18},
    '0x4fEe21439F2b95b72da2F9f901b3956f27fE91D5': {'symbol': 'FROG', 'decimals': 18},
    '0xF1376bceF0f78459C0Ed0ba5ddce976F1ddF51F4': {'symbol': 'UNIETH', 'decimals': 18},
    '0x00A7EC2F2b451cb0233E8aDbf4C9a951027C2B02': {'symbol': 'HIENS4', 'decimals': 18},
    '0xdCB5645eDa1ed34C5641d81b927D33EBaE9cF2A4': {'symbol': 'PAYB', 'decimals': 18},
    '0x8254e26e453EB5aBd29B3c37AC9E8Da32E5d3299': {'symbol': 'RBX', 'decimals': 18},
    '0xA891CF72AEDa692570928eFe1A832342e9783CDC': {'symbol': 'IFEX', 'decimals': 18},
    '0x4A8f5F96D5436e43112c2fBC6a9f70DA9e4E16D4': {'symbol': 'INXT', 'decimals': 8},
    '0x189fC141854BFc6146777406d53fbC89F4e70407': {'symbol': 'CEJI', 'decimals': 18},
    '0xfAc0403a24229d7e2Edd994D50F5940624CBeac2': {'symbol': 'THEO', 'decimals': 9},
    '0x36c86a1c23f4D17f3B3F28B7266cD7DB1D74A0B4': {'symbol': 'N', 'decimals': 9},
    '0x7101a9392EAc53B01e7c07ca3baCa945A56EE105': {'symbol': 'X7101', 'decimals': 18},
    '0x3383c5a8969Dc413bfdDc9656Eb80A1408E4bA20': {'symbol': 'WANATHA', 'decimals': 18},
    '0x7220e92D418E2EB59D0C25d195FA004bfD3aFC42': {'symbol': 'ADF', 'decimals': 18},
    '0x3684b581dB1F94b721EE0022624329FEb16Ab653': {'symbol': 'GUNTHY', 'decimals': 18},
    '0x433FCe7dfbEc729A79999EAf056Cb073B2153ebA': {'symbol': 'CNW', 'decimals': 6},
    '0xA885B6d3C87535de688194Ec3645199ee76D6772': {'symbol': 'TOKR', 'decimals': 9},
    '0x5B7533812759B45C2B44C19e320ba2cD2681b542': {'symbol': 'AGIX', 'decimals': 8},
    '0xFD0877d9095789cAF24c98F7CCe092fa8E120775': {'symbol': 'YVTUSD', 'decimals': 18},
    '0x923b83c26B3809d960fF80332Ed00aA46D7Ed375': {'symbol': 'CTR', 'decimals': 18},
    '0xC3DCA8F61B275d1e88f3ea31B3e311c49f56b24B': {'symbol': 'NPICK', 'decimals': 18},
    '0x705EE96c1c160842C92c1aeCfCFfccc9C412e3D9': {'symbol': 'POLL', 'decimals': 18},
    '0x2e9d63788249371f1DFC918a52f8d799F4a38C94': {'symbol': 'TOKE', 'decimals': 18},
    '0x60c7AEa107EA3cDAb21455e187cFB7e54E09b760': {'symbol': 'MEME', 'decimals': 18},
    '0xa5cA62D95D24A4a350983D5B8ac4EB8638887396': {'symbol': 'YVSUSD', 'decimals': 18},
    '0x95b4eF2869eBD94BEb4eEE400a99824BF5DC325b': {'symbol': 'CMKR', 'decimals': 8},
    '0x9aF15D7B8776fa296019979E70a5BE53c714A7ec': {'symbol': 'EVN', 'decimals': 18},
    '0xaeC2E87E0A235266D9C5ADc9DEb4b2E29b54D009': {'symbol': 'SNGLS', 'decimals': 0},
    '0x48783486ddD7fa85ECa6B0C4AE8920Bc25DfbcD7': {'symbol': 'GOM2', 'decimals': 0},
    '0xF32aa187d5Bc16A2C02A6aFb7Df1459d0D107574': {'symbol': 'INU', 'decimals': 18},
    '0xB8A768Cd332e4461FCcFbdb1d0292cEFFbB2Bb8A': {'symbol': 'COLLIE', 'decimals': 18},
    '0x33d20575f20C6A1881F8ab08E69F6fbAFfAEedF2': {'symbol': 'WIZ', 'decimals': 12},
    '0xd7DCd9B99787C619b4D57979521258D1A7267ad7': {'symbol': 'EVRY', 'decimals': 18},
    '0xeDEec5691f23E4914cF0183A4196bBEb30d027a0': {'symbol': 'WSTA', 'decimals': 18},
    '0x43D4A3cd90ddD2F8f4f693170C9c8098163502ad': {'symbol': 'D2D', 'decimals': 18},
    '0x65032604Dab202aFf9ADf89300CdB4bD0d059F55': {'symbol': 'SOBA', 'decimals': 18},
    '0x2Ec3275f7aCe4044e499823F511cd58250be8E3d': {'symbol': 'PRVG', 'decimals': 8},
    '0xFEead860FFa43e5660Ca62ea5953a74F695C1D5B': {'symbol': 'MDCY', 'decimals': 18},
    '0x1045F5cCb01DAEA4f8eaB055F5FcBB7C0E7c89F0': {'symbol': 'DFIAT', 'decimals': 18},
    '0x9280abF1AF455d6228def9c93b5E6D85F3F1ecc8': {'symbol': 'FLIKS', 'decimals': 18},
    '0x8A9C67fee641579dEbA04928c4BC45F66e26343A': {'symbol': 'JRT', 'decimals': 18},
    '0xA95C5ebB86E0dE73B4fB8c47A45B792CFeA28C23': {'symbol': 'SDL', 'decimals': 18},
    '0xcC6F15Be8573cB8243C42d300565566D328213Dd': {'symbol': 'OWN', 'decimals': 18},
    '0x0888Fd2960Ddf57Cfb561B59d73Dd2b9524F6A26': {'symbol': 'MIKAWA', 'decimals': 9},
    '0x97D4F49eEb0E2c96d5EbAA71AB8418e563ecd9fd': {'symbol': 'LSD', 'decimals': 9},
    '0xA6D5C720a9af5A405dFB6b9F44Fc44FaB5D4A58D': {'symbol': 'EZY', 'decimals': 8},
    '0xdA816459F1AB5631232FE5e97a05BBBb94970c95': {'symbol': 'YVDAI', 'decimals': 18},
    '0xB4272071eCAdd69d933AdcD19cA99fe80664fc08': {'symbol': 'XCHF', 'decimals': 18},
    '0xc1f976B91217E240885536aF8b63bc8b5269a9BE': {'symbol': 'PIN', 'decimals': 18},
    '0xc2A81eb482cB4677136D8812cC6Db6E0cB580883': {'symbol': 'TFS', 'decimals': 18},
    '0x578Dc16c47327429fF1594fA3eB77A96F38438E0': {'symbol': 'PZ', 'decimals': 9},
    '0x8c15Ef5b4B21951d50E53E4fbdA8298FFAD25057': {'symbol': 'FX', 'decimals': 18},
    '0x17837004eA685690b32dbEaD02a274Ec4333A26a': {'symbol': 'BEAR', 'decimals': 18},
    '0xb53ecF1345caBeE6eA1a65100Ebb153cEbcac40f': {'symbol': 'O', 'decimals': 18},
    '0x0316EB71485b0Ab14103307bf65a021042c6d380': {'symbol': 'HBTC', 'decimals': 18},
    '0xd2adC1C84443Ad06f0017aDCa346Bd9b6fc52CaB': {'symbol': 'DFND', 'decimals': 18},
    '0xf34B1Db61ACa1a371fE97BAd2606c9f534fb9D7D': {'symbol': 'RBIS', 'decimals': 18},
    '0x7C5A0CE9267ED19B22F8cae653F198e3E8daf098': {'symbol': 'SAN', 'decimals': 18},
    '0x3b9e094D56103611f0ACEfDAb43182347BA60dF4': {'symbol': 'XPN', 'decimals': 18},
    '0xE6b7743e2B9Aa2d0A9B163C4e69186ABb57817D9': {'symbol': 'KUKY', 'decimals': 18},
    '0xfDBc1aDc26F0F8f8606a5d63b7D3a3CD21c22B23': {'symbol': '1WO', 'decimals': 8},
    '0x9AB7bb7FdC60f4357ECFef43986818A2A3569c62': {'symbol': 'GOG', 'decimals': 18},
    '0xAd7Ca17e23f13982796D27d1E6406366Def6eE5f': {'symbol': 'RHEGIC2', 'decimals': 18},
    '0x92CfbEC26C206C90aeE3b7C66A9AE673754FaB7e': {'symbol': 'OLE', 'decimals': 18},
    '0xbDbf245c690d54b67C6e610A28486A2C6dE08bE6': {'symbol': 'SUNDER', 'decimals': 18},
    '0x050D94685c6B0477E1Fc555888AF6e2bB8dFBda5': {'symbol': 'INU', 'decimals': 18},
    '0xCdF7028ceAB81fA0C6971208e83fa7872994beE5': {'symbol': 'T', 'decimals': 18},
    '0xa06bC25B5805d5F8d82847D191Cb4Af5A3e873E0': {'symbol': 'ALINK', 'decimals': 18},
    '0xa499648fD0e80FD911972BbEb069e4c20e68bF22': {'symbol': 'UJENNY', 'decimals': 18},
    '0x0018d5E01e53878F90FEAB02F1b2019a21adf8b1': {'symbol': 'SHADOWCATS', 'decimals': 18},
    '0x79d4f0232A66c4c91b89c76362016A1707CFBF4f': {'symbol': 'VCHF', 'decimals': 18},
    '0xc6e145421FD494B26dCF2BFeB1b02b7c5721978f': {'symbol': 'CPRX', 'decimals': 18},
    '0x0E5C8C387C5EBa2eCbc137aD012aeD5Fe729e251': {'symbol': 'RPG', 'decimals': 18},
    '0x6f3277ad0782a7DA3eb676b85a8346A100BF9C1c': {'symbol': 'DOGPAD', 'decimals': 18},
    '0x64A77277e37D44957FE5815d6FF442ab8b16cC29': {'symbol': 'DAWGS', 'decimals': 9},
    '0x5CAc718A3AE330d361e39244BF9e67AB17514CE8': {'symbol': 'COT', 'decimals': 18},
    '0x52A047Ee205701895Ee06A375492490EC9C597cE': {'symbol': 'PULSE', 'decimals': 18},
    '0x3D0293f06daF4311B482564330D57C8Db6C10893': {'symbol': 'YCO', 'decimals': 8},
    '0xdDF7Fd345D54ff4B40079579d4C4670415DbfD0A': {'symbol': 'SG', 'decimals': 18},
    '0x5EeAA2DCb23056F4E8654a349E57eBE5e76b5e6e': {'symbol': 'VPP', 'decimals': 18},
    '0xfb5453340C03db5aDe474b27E68B6a9c6b2823Eb': {'symbol': 'ROBOT', 'decimals': 18},
    '0x9B39A0B97319a9bd5fed217c1dB7b030453bac91': {'symbol': 'TCH', 'decimals': 18},
    '0x71dc40668682a124231301414167E4CF7f55383C': {'symbol': 'MIMIR', 'decimals': 18},
    '0x0b38210ea11411557c13457D4dA7dC6ea731B88a': {'symbol': 'API3', 'decimals': 18},
    '0x00d8318E44780EdEEFcF3020A5448F636788883C': {'symbol': 'DAPPX', 'decimals': 18},
    '0x081F67aFA0cCF8c7B17540767BBe95DF2bA8D97F': {'symbol': 'CET', 'decimals': 18},
    '0xcbCC0F036ED4788F63FC0fEE32873d6A7487b908': {'symbol': 'HMQ', 'decimals': 8},
    '0x0168703872fa06741Ecaa9Dff7803168e83f7aE0': {'symbol': 'MIRO', 'decimals': 8},
    '0x701C244b988a513c945973dEFA05de933b23Fe1D': {'symbol': 'OAX', 'decimals': 18},
    '0x7797C85b46F548eAcC07C229f6Cd207d6370442f': {'symbol': 'TAG', 'decimals': 18},
    '0x3540abE4f288B280A0740ad5121aEc337c404D15': {'symbol': 'TPRO', 'decimals': 18},
    '0x01e0E2e61f554eCAaeC0cC933E739Ad90f24a86d': {'symbol': 'GTON', 'decimals': 18},
    '0x9A2Af0AbB12bee5369B180976Be01E8c80D0e7B6': {'symbol': 'EMPIRE', 'decimals': 9},
    '0x668C50B1c7f46EFFBE3f242687071d7908AAB00A': {'symbol': 'COSHI', 'decimals': 9},
    '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0': {'symbol': 'WSTETH', 'decimals': 18},
    '0xFF603F43946A3A28DF5E6A73172555D8C8b02386': {'symbol': 'RNT', 'decimals': 18},
    '0xeb2f5A4e1441dF330670Dc7B0cf4eaC0f7ab5fa5': {'symbol': 'HIFRIENDS', 'decimals': 18},
    '0x4FbB350052Bca5417566f188eB2EBCE5b19BC964': {'symbol': 'GRG', 'decimals': 18},
    '0x96ef7f9cF1B6eCC66E482A6598fc9F009E9277DA': {'symbol': 'POMI', 'decimals': 8},
    '0x39795344CBCc76cC3Fb94B9D1b15C23c2070C66D': {'symbol': 'SHARE', 'decimals': 9},
    '0x16484d73Ac08d2355F466d448D2b79D2039F6EBB': {'symbol': 'FKX', 'decimals': 18},
    '0xb683D83a532e2Cb7DFa5275eED3698436371cc9f': {'symbol': 'BTU', 'decimals': 18},
    '0xb1f136a74e18E4E2921feBbf25820d1Bb65B5647': {'symbol': 'HIPUNKS', 'decimals': 18},
    '0xf40eDF22e4aB146a32eCD8d659Cedc7A2a55Cd43': {'symbol': 'ONEPIECE', 'decimals': 9},
    '0xd578779dbC9252218E12d18D628e3cb27E4a56f2': {'symbol': 'BRT', 'decimals': 18},
    '0xad32A8e6220741182940c5aBF610bDE99E737b2D': {'symbol': 'DOUGH', 'decimals': 18},
    '0x24cCeDEBF841544C9e6a62Af4E8c2fA6e5a46FdE': {'symbol': 'BLUESPARROW', 'decimals': 9},
    '0x77607588222e01bf892a29Abab45796A2047fc7b': {'symbol': 'UETH', 'decimals': 18},
    '0x09617F6fD6cF8A71278ec86e23bBab29C04353a7': {'symbol': 'ULT', 'decimals': 18},
    '0x1122B6a0E00DCe0563082b6e2953f3A943855c1F': {'symbol': 'CENNZ', 'decimals': 18},
    '0x53340a1EF3a0dDeBA1D94BBd1E2fF55936F0ea60': {'symbol': 'BARK', 'decimals': 18},
    '0xe9f84d418B008888A992Ff8c6D22389C2C3504e0': {'symbol': 'BASE', 'decimals': 8},
    '0x6B0956258fF7bd7645aa35369B55B61b8e6d6140': {'symbol': 'LUCKY', 'decimals': 8},
    '0x785c34312dfA6B74F6f1829f79ADe39042222168': {'symbol': 'BUMP', 'decimals': 18},
    '0xB5c578947de0fd71303F71F2C3d41767438bD0de': {'symbol': 'DEVT', 'decimals': 18},
    '0xfd0205066521550D7d7AB19DA8F72bb004b4C341': {'symbol': 'LIT', 'decimals': 18},
    '0x74bE64B45D394FA57816c1950E94dBB8d7A7B306': {'symbol': 'GINU', 'decimals': 9},
    '0xB7Df0f42FAe30acf30C9A5BA147D6B792b5eB9d9': {'symbol': 'AISC', 'decimals': 18},
    '0xc383a3833A87009fD9597F8184979AF5eDFad019': {'symbol': 'IETH', 'decimals': 18},
    '0xdfdDF7a69716124Bc346bA556d4b9f9E74C4A8Bc': {'symbol': 'SCCN', 'decimals': 18},
    '0x1a57367C6194199e5D9aEa1cE027431682dfB411': {'symbol': 'MDF', 'decimals': 18},
    '0x01FB70eA8AcFDd5435c8e8bd1531d96D57cEd0ec': {'symbol': 'SELECT', 'decimals': 18},
    '0x6E98E5401ADCB0D76F4DEBfc3d794B3031F48790': {'symbol': 'AUR', 'decimals': 18},
    '0xeDF6568618A00C6F0908Bf7758A16F76B6E04aF9': {'symbol': 'ARIA20', 'decimals': 18},
    '0x2be5e8c109e2197D077D13A82dAead6a9b3433C5': {'symbol': 'TON', 'decimals': 18},
    '0x0A913beaD80F321E7Ac35285Ee10d9d922659cB7': {'symbol': 'DOS', 'decimals': 18},
    '0x8C661806f716652B637728355cC4e2620D428F99': {'symbol': 'DSP', 'decimals': 18},
    '0xBa1ED22C69ad00739EE2b4aBD70e270BE9e87eE2': {'symbol': 'WFX', 'decimals': 18},
    '0x62406995CaFd18f57e7375e8E0060725Acebce58': {'symbol': 'FIWT', 'decimals': 18},
    '0x340D2bdE5Eb28c1eed91B2f790723E3B160613B7': {'symbol': 'VEE', 'decimals': 18},
    '0x0fE0Ed7F146Cb12e4B9759afF4FA8d34571802ca': {'symbol': 'PARTY', 'decimals': 18},
    '0xfCC63e6459936216b7b887F6dA5fCD90c0C62b7c': {'symbol': 'KEES', 'decimals': 18},
    '0xb0280743b44bF7db4B6bE482b2Ba7b75E5dA096C': {'symbol': 'TNS', 'decimals': 18},
    '0x9Ac5C63dDcb93612E316ab31dFc8192bC8961988': {'symbol': 'ARA', 'decimals': 18},
    '0xf45f6c8bB3d77eA762175b8f7ca4D251941649fA': {'symbol': 'LEMD', 'decimals': 18},
    '0x6006FC2a849fEdABa8330ce36F5133DE01F96189': {'symbol': 'SHAKE', 'decimals': 18},
    '0xC82E3dB60A52CF7529253b4eC688f631aad9e7c2': {'symbol': 'ARC', 'decimals': 18},
    '0xC8EF1460277EA47d179dEc66D1c5f8b7f7aE5a28': {'symbol': 'RIFI', 'decimals': 18},
    '0xF17A3fE536F8F7847F1385ec1bC967b2Ca9caE8D': {'symbol': 'AMKT', 'decimals': 18},
    '0x4D8046B2f7986FEB5a4b0781EbF5Fa2D7a808617': {'symbol': 'DOGEZ', 'decimals': 18},
    '0x4161725D019690a3E0de50f6bE67b07a86A9fAe1': {'symbol': 'TPT', 'decimals': 4},
    '0xa44E5137293E855B1b7bC7E2C6f8cD796fFCB037': {'symbol': 'DVPN', 'decimals': 8},
    '0x32C4ADB9cF57f972bc375129de91C897b4F364F1': {'symbol': 'FLC', 'decimals': 18},
    '0x6226e00bCAc68b0Fe55583B90A1d727C14fAB77f': {'symbol': 'MTV', 'decimals': 18},
    '0xb20043F149817bff5322F1b928e89aBFC65A9925': {'symbol': 'EXRT', 'decimals': 8},
    '0xa143ac515dCA260A46C742C7251eF3B268639593': {'symbol': 'BULK', 'decimals': 18},
    '0xBD04ccc050058a6A422851fA6c0F92BB65EB06ca': {'symbol': 'PRTG', 'decimals': 18},
    '0xf203Ca1769ca8e9e8FE1DA9D147DB68B6c919817': {'symbol': 'WNCG', 'decimals': 18},
    '0xF52CDcD458bf455aeD77751743180eC4A595Fd3F': {'symbol': 'IDLESUSDYIELD', 'decimals': 18},
    '0xbb70AdbE39408cB1E5258702ea8ADa7c81165b73': {'symbol': 'ANTE', 'decimals': 18},
    '0xcC779bbBD17b2E68ECdCCaeA3d8C791B1573c66b': {'symbol': 'FETCH', 'decimals': 18},
    '0x6769D86f9C430f5AC6d9c861a0173613F1C5544C': {'symbol': 'KOACOMBAT', 'decimals': 9},
    '0x0AD896863ce4Cd84f10a9D30D4f509cefFd53c84': {'symbol': 'CLB', 'decimals': 18},
    '0x3802c218221390025BCEABBaD5D8C59F40eB74B8': {'symbol': 'GETH', 'decimals': 18},
    '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704': {'symbol': 'CBETH', 'decimals': 18},
    '0x6FCE4A401B6B80ACe52baAefE4421Bd188e76F6f': {'symbol': 'AMANA', 'decimals': 18},
    '0x72430A612Adc007c50e3b6946dBb1Bb0fd3101D1': {'symbol': 'TIC', 'decimals': 8},
    '0x296233E84c1D7bfF11121BF6D60f0FFA39C3f0cf': {'symbol': 'NOONE', 'decimals': 9},
    '0x75858677e27C930FB622759FeafFeE2b754Af07F': {'symbol': 'SOUL', 'decimals': 8},
    '0x3212b29E33587A00FB1C83346f5dBFA69A458923': {'symbol': 'IMBTC', 'decimals': 8},
    '0xB5FE099475d3030DDe498c3BB6F3854F762A48Ad': {'symbol': 'FNK', 'decimals': 18},
    '0xa6422E3E219ee6d4C1B18895275FE43556fd50eD': {'symbol': 'STBU', 'decimals': 18},
    '0x9e10f61749c4952C320412A6B26901605Ff6Da1d': {'symbol': 'THEOS', 'decimals': 18},
    '0xc1ABb8c93be6811aFfC70675b0432926c4BFBb5D': {'symbol': 'UERII', 'decimals': 18},
    '0x16CC8367055aE7e9157DBcB9d86Fd6CE82522b31': {'symbol': 'VXL', 'decimals': 18},
    '0x0bb217E40F8a5Cb79Adf04E1aAb60E5abd0dfC1e': {'symbol': 'SWFTC', 'decimals': 8},
    '0x965d79F1A1016B574a62986e13Ca8Ab04DfdD15C': {'symbol': 'M2', 'decimals': 18},
    '0x6B175474E89094C44Da98b954EedeAC495271d0F': {'symbol': 'DAI', 'decimals': 18},
    '0xe9F1d62c671EFe99896492766c0B416bD3FB9e52': {'symbol': 'XOY', 'decimals': 8},
    '0x71de2285ec83BB6517D2B3eC8DEb2A6c44D8C5da': {'symbol': 'UMOON', 'decimals': 18},
    '0x6BFf2fE249601ed0Db3a87424a2E923118BB0312': {'symbol': 'FYZ', 'decimals': 18},
    '0x8971f9fd7196e5cEE2C1032B50F656855af7Dd26': {'symbol': 'LAMB', 'decimals': 18},
    '0x39Fa206c1648944f92E8F7B626e1CBdf78d7E9dB': {'symbol': 'DXY', 'decimals': 18},
    '0x5DE8ab7E27f6E7A1fFf3E5B337584Aa43961BEeF': {'symbol': 'SDEX', 'decimals': 18},
    '0x6d6554939D646f274d0FC3cEcB7dab5d76bc908F': {'symbol': 'MS', 'decimals': 18},
    '0x7B7983967409Fce461EA8bbDF9ED37631b1d59C9': {'symbol': 'KPOP', 'decimals': 18},
    '0x3c79EB9d7a76F4EB7B2a5286aAcFbbD4FeAbe79A': {'symbol': 'DOGTIC', 'decimals': 18},
    '0xb8e103b60A33597136EA9511F46b6dBeB643a3a5': {'symbol': 'SBTC', 'decimals': 18},
    '0x4bceA5E4d0F6eD53cf45e7a28FebB2d3621D7438': {'symbol': 'MODEX', 'decimals': 18},
    '0x24EC2Ca132abf8F6f8a6E24A1B97943e31f256a7': {'symbol': 'MOOV', 'decimals': 18},
    '0x635d081fD8F6670135D8a3640E2cF78220787d56': {'symbol': 'ADD', 'decimals': 18},
    '0x760Ea31711aC14cb9Ba81aC25453C6f378fA5794': {'symbol': 'BOWL', 'decimals': 9},
    '0x8888801aF4d980682e47f1A9036e589479e835C5': {'symbol': 'MPH', 'decimals': 18},
    '0x865ec58b06bF6305B886793AA20A2da31D034E68': {'symbol': 'MOC', 'decimals': 18},
    '0xEd91879919B71bB6905f23af0A68d231EcF87b14': {'symbol': 'DMG', 'decimals': 18},
    '0xA64BD6C70Cb9051F6A9ba1F163Fdc07E0DfB5F84': {'symbol': 'ALINK', 'decimals': 18},
    '0x82A77710495A35549D2aDD797412b4a4497D33eF': {'symbol': 'DOGZ', 'decimals': 18},
    '0x3700Adfd26d5bC062cB8b8a77e68fbd43F58eCAB': {'symbol': 'HIDOODLES', 'decimals': 18},
    '0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78': {'symbol': 'IETHV2', 'decimals': 18},
    '0xa8B0F154A688c22142E361707df64277e0A0bE66': {'symbol': 'RAK', 'decimals': 18},
    '0x3DBB00c9bE5A327E25CAf4f650844C5DbA81E34B': {'symbol': 'RMATIC', 'decimals': 18},
    '0x3810A4Ddf41E586Fa0dbA1463A7951B748cEcFca': {'symbol': 'MPAY', 'decimals': 18},
    '0x86ed939B500E121C0C5f493F399084Db596dAd20': {'symbol': 'SPC', 'decimals': 18},
    '0xf79F9020560963422eCC9C0c04D3a21190BBf045': {'symbol': 'BABYSAITAMA', 'decimals': 9},
    '0xf8aD7dFe656188A23e89da09506Adf7ad9290D5d': {'symbol': 'BLY', 'decimals': 18},
    '0x6295Ab2BE04A617747481B292c390BfcA592Cf28': {'symbol': 'TDS', 'decimals': 18},
    '0xa4bBE66f151B22B167127c770016b15fF97Dd35C': {'symbol': 'UMBR', 'decimals': 18},
    '0x8b0FdE007458Ee153bd0F66CD448Af5FB3D99b43': {'symbol': 'MASTERMIND', 'decimals': 18},
    '0x4b1D0b9F081468D780Ca1d5d79132b64301085d1': {'symbol': 'LMR', 'decimals': 8},
    '0xd346E8ADA104093ADcf5f4186087E1AD309BB3B3': {'symbol': 'SHIBN', 'decimals': 18},
    '0xDecade1c6Bf2cD9fb89aFad73e4a519C867adcF5': {'symbol': 'WIS', 'decimals': 18},
    '0x91a5de30e57831529a3c1aF636A78a7E4E83f3aa': {'symbol': 'HULK', 'decimals': 18},
    '0x42726d074BBa68Ccc15200442B72Afa2D495A783': {'symbol': 'ISIKC', 'decimals': 4},
    '0xDb05EA0877A2622883941b939f0bb11d1ac7c400': {'symbol': 'OPCT', 'decimals': 18},
    '0x3A0b022f32b3191D44E5847da12dc0B63fb07C91': {'symbol': 'SPELLFIRE', 'decimals': 18},
    '0xB16d3Ed603D62b125c6bd45519EDa40829549489': {'symbol': 'ISR', 'decimals': 18},
    '0xE83D5FB2c60B3A2597452e248CF7b2f52a7E731e': {'symbol': 'ARTIC', 'decimals': 18},
    '0xdC4c9bFBB37C7cDeeBc53C0BA5C7474FEA5E106D': {'symbol': 'FRANK', 'decimals': 9},
    '0x2942E3B38E33123965bfbc21E802bE943a76bbC6': {'symbol': 'EHASH', 'decimals': 18},
    '0x035dF12E0F3ac6671126525f1015E47D79dFEDDF': {'symbol': '0XMR', 'decimals': 18},
    '0xbededDf2eF49E87037c4fb2cA34d1FF3D3992A11': {'symbol': 'FEG', 'decimals': 18},
    '0xF433089366899D83a9f26A773D59ec7eCF30355e': {'symbol': 'MTL', 'decimals': 8},
    '0xDc0327D50E6C73db2F8117760592C8BBf1CDCF38': {'symbol': 'STRNGR', 'decimals': 18},
    '0x3330BFb7332cA23cd071631837dC289B09C33333': {'symbol': 'RBC', 'decimals': 18},
    '0x56694577564FdD577a0ABB20FE95C1E2756C2a11': {'symbol': 'ASW', 'decimals': 18},
    '0x9A0aBA393aac4dFbFf4333B06c407458002C6183': {'symbol': 'AC', 'decimals': 18},
    '0x9f549ebFD4974cD4eD4A1550D40394B44A7382AA': {'symbol': 'LKN', 'decimals': 18},
    '0x7FBEC0bb6A7152e77C30D005B5D49cbC08A602C3': {'symbol': 'DDOS', 'decimals': 18},
    '0xcFEB09C3c5F0f78aD72166D55f9e6E9A60e96eEC': {'symbol': 'VEMP', 'decimals': 18},
    '0xBDC5bAC39Dbe132B1E030e898aE3830017D7d969': {'symbol': 'SNOV', 'decimals': 18},
    '0xa2E3356610840701BDf5611a53974510Ae27E2e1': {'symbol': 'WBETH', 'decimals': 18},
    '0x7E9c15C43f0D6C4a12E6bdfF7c7D55D0f80e3E23': {'symbol': 'ASTRA', 'decimals': 18},
    '0xDfDd3459D4F87234751696840092ee20C970Fb07': {'symbol': 'MYOBU', 'decimals': 9},
    '0x8EF47555856f6Ce2E0cd7C36AeF4FAb317d2e2E2': {'symbol': 'PAYT', 'decimals': 18},
    '0xfC98e825A2264D890F9a1e68ed50E1526abCcacD': {'symbol': 'MCO2', 'decimals': 18},
    '0x38b7Bf4eeCF3EB530b1529c9401FC37d2a71a912': {'symbol': 'CSMATIC', 'decimals': 18},
    '0x04A6b6DE116Fb8bF57e5eE8b05e0293EA3639fE8': {'symbol': 'ETH20', 'decimals': 18},
    '0xE67F943af5Eb6051ef56f05979cc30B732717fa6': {'symbol': 'WATT', 'decimals': 4},
    '0x138C2F1123cF3f82E4596d097c118eAc6684940B': {'symbol': 'ALPHA', 'decimals': 18},
    '0x83E9F223e1edb3486F876EE888d76bFba26c475A': {'symbol': 'GUILD', 'decimals': 18},
    '0x7F3EDcdD180Dbe4819Bd98FeE8929b5cEdB3AdEB': {'symbol': 'XTK', 'decimals': 18},
    '0xe9F84dE264E91529aF07Fa2C746e934397810334': {'symbol': 'SAK3', 'decimals': 18},
    '0xf1b8762a7fa8C244e36F7234EDF40cFaE24394e3': {'symbol': 'GOLD', 'decimals': 18},
    '0x62A116B9fFe7338bEe13c87E69552De9cC67627D': {'symbol': 'FUC', 'decimals': 18},
    '0x4a220E6096B25EADb88358cb44068A3248254675': {'symbol': 'QNT', 'decimals': 18},
    '0x822f1A51E534723ec1305E6EB5C96AEcfeb4c629': {'symbol': 'MTU', 'decimals': 18},
    '0xa350DA05405cc204e551C4eeD19C3039646528d5': {'symbol': 'BSPT', 'decimals': 18},
    '0xc7BbA5b765581eFb2Cdd2679DB5Bea9eE79b201f': {'symbol': 'GEM', 'decimals': 18},
    '0x8806926Ab68EB5a7b909DcAf6FdBe5d93271D6e2': {'symbol': 'UQC', 'decimals': 18},
    '0x7Cca2e1c9B0519f52029467914A15E782BF66971': {'symbol': 'CPTINU', 'decimals': 18},
    '0xbe30F684d62C9F7883a75A29c162c332c0d98f23': {'symbol': 'GHT', 'decimals': 18},
    '0x9AAb071B4129B083B01cB5A0Cb513Ce7ecA26fa5': {'symbol': 'HUNT', 'decimals': 18},
    '0x7f0c8B125040f707441cad9e5eD8a8408673b455': {'symbol': 'NEBO', 'decimals': 18},
    '0x7105E64bF67ECA3Ae9b123F0e5Ca2b83b2eF2dA0': {'symbol': 'X7DAO', 'decimals': 18},
    '0x73968b9a57c6E53d41345FD57a6E6ae27d6CDB2F': {'symbol': 'SDT', 'decimals': 18},
    '0xDF2C7238198Ad8B389666574f2d8bc411A4b7428': {'symbol': 'MFT', 'decimals': 18},
    '0x9501BFc48897DCEEadf73113EF635d2fF7ee4B97': {'symbol': 'EMT', 'decimals': 18},
    '0xCAEaf8381D4B20b43AFA42061D6f80319A8881F6': {'symbol': 'R34P', 'decimals': 8},
    '0x7A73839bd0e5cdED853cB01aa9773F8989381065': {'symbol': 'UBXN', 'decimals': 18},
    '0x6ef460EB3563CfCc73F8147B0A77DAFFeE71F867': {'symbol': 'ZEUS', 'decimals': 18},
    '0xe59A1fe295f940E5226DcF6411Ef2Bdf3089C254': {'symbol': 'YMNT', 'decimals': 9},
    '0x8A9c4dfe8b9D8962B31e4e16F8321C44d48e246E': {'symbol': 'NCT', 'decimals': 18},
    '0x78223d31298107f3E310B09797b07967832046A6': {'symbol': 'RSFT', 'decimals': 18},
    '0xfF770E4c68e35DB85c6e0E89a43750eC02bDB2AC': {'symbol': 'OBTC', 'decimals': 18},
    '0x556d4F40982CB95E0714989E0c229C42be8b1499': {'symbol': 'GLTM', 'decimals': 18},
    '0x4Cf89ca06ad997bC732Dc876ed2A7F26a9E7f361': {'symbol': 'MYST', 'decimals': 18},
    '0x2001f2A0Cf801EcFda622f6C28fb6E10d803D969': {'symbol': 'CLT', 'decimals': 8},
    '0x083DFb3566bde80380446718eEDa55B800464636': {'symbol': 'ASX', 'decimals': 18},
    '0x24ffe459F51ea20C5d8ad49843529fc33654e7E4': {'symbol': 'POM', 'decimals': 9},
    '0xC77b230F31b517F1ef362e59c173C2BE6540B5E8': {'symbol': 'VIDY', 'decimals': 18},
    '0x6393E822874728f8Afa7e1C9944E417D37CA5878': {'symbol': 'CIX100', 'decimals': 18},
    '0x5d259E3FbcAdef1abfeA5582e456Bc7F0aebC2a1': {'symbol': 'AI', 'decimals': 18},
    '0x7d8d7c26179b7A6aebbF66A91C38ED92d5B4996b': {'symbol': 'FUND', 'decimals': 18},
    '0x4b0f027d0b694Aae2761ED2d426295d4f949F5d0': {'symbol': 'POLL', 'decimals': 18},
    '0x1F17D72cBe65Df609315dF5c4f5F729eFbd00Ade': {'symbol': 'GYOSHI', 'decimals': 18},
    '0x805C2077f3ab224D889f9c3992B41B2F4722c787': {'symbol': 'ARTEQ', 'decimals': 0},
    '0x66fD97a78d8854fEc445cd1C80a07896B0b4851f': {'symbol': 'LMY', 'decimals': 18},
    '0xB64ef51C888972c908CFacf59B47C1AfBC0Ab8aC': {'symbol': 'STORJ', 'decimals': 8},
    '0x618E75Ac90b12c6049Ba3b27f5d5F8651b0037F6': {'symbol': 'QASH', 'decimals': 6},
    '0x111111111117dC0aa78b770fA6A738034120C302': {'symbol': '1INCH', 'decimals': 18},
    '0x97b65710D03E12775189F0D113202cc1443b0aa2': {'symbol': 'ELONONE', 'decimals': 9},
    '0x5ba19d656B65f1684cFea4Af428C23B9f3628F97': {'symbol': 'AAG', 'decimals': 18},
    '0x8d96B4ab6C741a4C8679AE323a100d74f085BA8F': {'symbol': 'BZR', 'decimals': 18},
    '0x998b3B82bC9dBA173990Be7afb772788B5aCB8Bd': {'symbol': 'BANCA', 'decimals': 18},
    '0x88303FeD02b31dB9C7a9eafB711dA9eF4A03e5D3': {'symbol': 'ZIK', 'decimals': 18},
    '0x3B7f247f21BF3A07088C2D3423F64233d4B069F7': {'symbol': 'DYNMT', 'decimals': 2},
    '0x6E605c269E0C92e70BEeB85486f1fC550f9380BD': {'symbol': 'CATT', 'decimals': 18},
    '0x56bd0C900acAF04125Ee26f546d6214634fD970F': {'symbol': 'HUB', 'decimals': 18},
    '0x9aE380F0272E2162340a5bB646c354271c0F5cFC': {'symbol': 'CNC', 'decimals': 18},
    '0xb269Ab3B20E09e6fd17C3ED5B4CECF3dEA1a5141': {'symbol': 'SAK', 'decimals': 18},
    '0xc6Cc3d07C705E39D11c7f60d8836C7C78D4aC5f1': {'symbol': 'OLEA', 'decimals': 18},
    '0x954b890704693af242613edEf1B603825afcD708': {'symbol': 'CARD', 'decimals': 18},
    '0x5D21eF5f25a985380B65c8e943A0082fEDa0Db84': {'symbol': 'ECASH', 'decimals': 18},
    '0x6FC2f1044A3b9bB3e43A43EC8F840843Ed753061': {'symbol': 'ROBO', 'decimals': 18},
    '0xf7790914Dc335B20Aa19D7c9C9171e14e278A134': {'symbol': 'EUR-C', 'decimals': 18},
    '0x33D6064F0DFb62462A74049f30909DDD4f683Ba2': {'symbol': 'SEREN', 'decimals': 9},
    '0x96Eb50804d0eF2790f2E1A33670FEff6040CF89D': {'symbol': 'SX', 'decimals': 18},
    '0x97E19E2a5458294842036404E6A05571D8Bc2fA3': {'symbol': 'G', 'decimals': 18},
    '0xdD3A88EF8535Ff7b19178B735C52eA277eF25623': {'symbol': 'DFX', 'decimals': 18},
    '0x51cB253744189f11241becb29BeDd3F1b5384fdB': {'symbol': 'DMTR', 'decimals': 18},
    '0x75231F58b43240C9718Dd58B4967c5114342a86c': {'symbol': 'OKB', 'decimals': 18},
    '0xfa5B75a9e13Df9775cf5b996A049D9cc07c15731': {'symbol': 'VCK', 'decimals': 18},
    '0x251457b7c5d85251Ca1aB384361c821330bE2520': {'symbol': 'HATI', 'decimals': 18},
    '0x7a83FF237e7870d8d6C3485920EBe654D2841315': {'symbol': 'HIOD', 'decimals': 18},
    '0x4d224452801ACEd8B2F0aebE155379bb5D594381': {'symbol': 'APE', 'decimals': 18},
    '0xaea46A60368A7bD060eec7DF8CBa43b7EF41Ad85': {'symbol': 'FET', 'decimals': 18},
    '0xf8497f1dab075A3674363B5609Ba679F7dc9C885': {'symbol': 'SODATSU', 'decimals': 18},
    '0x75Ecb52e403C617679FBd3e77A50f9d10A842387': {'symbol': 'CSR', 'decimals': 18},
    '0x996Dc5dfc819408Dd98Cd92c9a76f64b0738Dc3D': {'symbol': 'SKI', 'decimals': 18},
    '0x0DB8D8b76BC361bAcbB72E2C491E06085A97Ab31': {'symbol': 'IQN', 'decimals': 18},
    '0x57C75ECCc8557136D32619a191fBCDc88560d711': {'symbol': 'VDG', 'decimals': 0},
    '0x814e0908b12A99FeCf5BC101bB5d0b8B5cDf7d26': {'symbol': 'MDT', 'decimals': 18},
    '0xebF2096E01455108bAdCbAF86cE30b6e5A72aa52': {'symbol': 'XIDR', 'decimals': 6},
    '0x64C1C9a38038AE73C3699B8dc347fEFDa2221E93': {'symbol': 'SWISSNFTFUND', 'decimals': 9},
    '0xFcF8eda095e37A41e002E266DaAD7efC1579bc0A': {'symbol': 'FLEX', 'decimals': 18},
    '0x7aE1D57b58fA6411F32948314BadD83583eE0e8C': {'symbol': 'PAPER', 'decimals': 18},
    '0x7662c015D845EF487fDe32cb884653bE9c9E0110': {'symbol': 'BOXA', 'decimals': 9},
    '0x259AB9b9EAB62b0fD98729B97BE121073D5B3479': {'symbol': 'EST', 'decimals': 18},
    '0x018fb5Af9d015Af25592a014C4266a84143De7a0': {'symbol': 'MP3', 'decimals': 18},
    '0x42a501903AFaa1086B5975773375c80E363f4063': {'symbol': 'CTK', 'decimals': 8},
    '0x2341Dd0A96a0Dab62Aa1efB93D59FF7F3bDB8932': {'symbol': 'PROT', 'decimals': 18},
    '0xbEA269038Eb75BdAB47a9C04D0F5c572d94b93D5': {'symbol': 'WFIO', 'decimals': 9},
    '0xB39185e33E8c28e0BB3DbBCe24DA5dEA6379Ae91': {'symbol': 'PHUNK', 'decimals': 18},
    '0xEb6026D3BEAA308D5822C44cDd2Ca8c7714237EC': {'symbol': 'SOC', 'decimals': 18},
    '0x607F4C5BB672230e8672085532f7e901544a7375': {'symbol': 'RLC', 'decimals': 9},
    '0x25f8087EAD173b73D6e8B84329989A8eEA16CF73': {'symbol': 'YGG', 'decimals': 18},
    '0x9669890e48f330ACD88b78D63E1A6b3482652CD9': {'symbol': 'BCNT', 'decimals': 18},
    '0x3ADfc4999F77D04c8341BAC5F3A76f58DfF5B37A': {'symbol': 'PRIX', 'decimals': 8},
    '0xeA068Fba19CE95f12d252aD8Cb2939225C4Ea02D': {'symbol': 'FIEF', 'decimals': 18},
    '0xC25a3A3b969415c80451098fa907EC722572917F': {'symbol': 'SCURVE', 'decimals': 18},
    '0xD38e031f4529a07996aaB977d2B79f0e00656C56': {'symbol': 'WTBT', 'decimals': 18},
    '0x888888435FDe8e7d4c54cAb67f206e4199454c60': {'symbol': 'DFX', 'decimals': 18},
    '0x3A707d56D538e85B783E8CE12B346e7fB6511F90': {'symbol': 'IETHV', 'decimals': 18},
    '0xecC0F1F860a82aB3b442382D93853C02d6384389': {'symbol': 'AXIS', 'decimals': 18},
    '0x752Efadc0a7E05ad1BCCcDA22c141D01a75EF1e4': {'symbol': 'COMFI', 'decimals': 18},
    '0xE0B7927c4aF23765Cb51314A0E0521A9645F0E2A': {'symbol': 'DGD', 'decimals': 9},
    '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48': {'symbol': 'USDC', 'decimals': 6},
    '0x79B87d52a03ED26A19C1914be4CE37F812e2056c': {'symbol': 'USPC', 'decimals': 18},
    '0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56': {'symbol': 'B-80BAL-20WETH', 'decimals': 18},
    '0x149caC67F1cd5d80651E7c9Bb359ec285D821a05': {'symbol': 'MNT', 'decimals': 18},
    '0x49Bf0220C9Ce17E52dCcA3d217231746D676085B': {'symbol': 'VIX', 'decimals': 18},
    '0x6876EbA317272FE221C67405C5e8EB3B24535547': {'symbol': 'MCT', 'decimals': 18},
    '0x4b13006980aCB09645131b91D259eaA111eaF5Ba': {'symbol': 'MYC', 'decimals': 18},
    '0x5cAf454Ba92e6F2c929DF14667Ee360eD9fD5b26': {'symbol': 'DEV', 'decimals': 18},
    '0x2b100Be232F90Dd33d3B6e4b9ad70D1E8AC0E208': {'symbol': 'MARX', 'decimals': 18},
    '0xbc4171f45EF0EF66E76F979dF021a34B46DCc81d': {'symbol': 'DORA', 'decimals': 18},
    '0x2610F0bFC21EF389fe4D03CFB7De9ac1E6C99D6E': {'symbol': 'SKYRIM', 'decimals': 18},
    '0x348B7f3106B5Da47405332534d06069fF9CE4d1B': {'symbol': 'ELONGD', 'decimals': 9},
    '0xd3C89cAC4A4283EDbA6927e2910fD1eBC14fe006': {'symbol': 'MOOI', 'decimals': 18},
    '0x13C99770694f07279607A6274F28a28c33086424': {'symbol': 'MTRL', 'decimals': 18},
    '0xe77f82f288A6a7EB4Ac295a86FccB1F6D240499B': {'symbol': 'PLANB', 'decimals': 18},
    '0x6f40d4A6237C257fff2dB00FA0510DeEECd303eb': {'symbol': 'INST', 'decimals': 18},
    '0x02AD335DD3ca11c18cEBBbB583b9613b6289d75F': {'symbol': 'BPEG', 'decimals': 18},
    '0xF519381791C03DD7666C142D4E49Fd94d3536011': {'symbol': 'ASIA', 'decimals': 18},
    '0xFbEEa1C75E4c4465CB2FCCc9c6d6afe984558E20': {'symbol': 'DDIM', 'decimals': 18},
    '0x08711D3B02C8758F2FB3ab4e80228418a7F8e39c': {'symbol': 'EDG', 'decimals': 0},
    '0x3aFfCCa64c2A6f4e3B6Bd9c64CD2C969EFd1ECBe': {'symbol': 'DSLA', 'decimals': 18},
    '0x9343e24716659A3551eB10Aff9472A2dcAD5Db2d': {'symbol': 'STFX', 'decimals': 18},
    '0x89bD2E7e388fAB44AE88BEf4e1AD12b4F1E0911c': {'symbol': 'NUX', 'decimals': 18},
    '0xC477D038d5420C6A9e0b031712f61c5120090de9': {'symbol': 'BOSON', 'decimals': 18},
    '0x675BBC7514013E2073DB7a919F6e4cbeF576de37': {'symbol': 'CLS', 'decimals': 18},
    '0xB57420FaD6731B004309D5a0ec7C6C906Adb8df7': {'symbol': 'DTO', 'decimals': 18},
    '0xB8366948B4A3F07BcBf14EB1739daA42A26b07c4': {'symbol': 'VBIT', 'decimals': 18},
    '0x21eAD867C8c5181854f6f8Ce71f75b173d2Bc16A': {'symbol': 'ETHX', 'decimals': 18},
    '0xEBd9D99A3982d547C5Bb4DB7E3b1F9F14b67Eb83': {'symbol': 'ID', 'decimals': 18},
    '0xF2f9A7e93f845b3ce154EfbeB64fB9346FCCE509': {'symbol': 'POWER', 'decimals': 18},
    '0x66bcD0C22Bb15cD1bA7199C7a383DC89528f4427': {'symbol': 'BUH', 'decimals': 18},
    '0xaE0585a259A3BCAB258D6EE02FB583f7B33C2a12': {'symbol': 'TEM', 'decimals': 18},
    '0xfb130d93E49DcA13264344966A611dc79a456Bc5': {'symbol': 'DOGEGF', 'decimals': 18},
    '0x012E0e6342308b247F36Ee500ECb14dc77a7a8C1': {'symbol': 'SKT', 'decimals': 8},
    '0x80a2AE356fc9ef4305676f7a3E2Ed04e12C33946': {'symbol': 'CYFI', 'decimals': 8},
    '0x3d9a4D8ab4F5bd1D5D08Ae3A95e8ED8BB4d7e3B9': {'symbol': 'STONKS', 'decimals': 18},
    '0x188E817B02e635D482AE4D81e25DdA98A97C4a42': {'symbol': 'LITH', 'decimals': 18},
    '0x77252494C25444F8598A0c74Ffc90ADc535291a9': {'symbol': 'NMS', 'decimals': 18},
    '0x431D5dfF03120AFA4bDf332c61A6e1766eF37BDB': {'symbol': 'JPYC', 'decimals': 18},
    '0x397b102dECcACE4AA8E5bA63EeDb8e65ad83E20c': {'symbol': '0XENCRYPT', 'decimals': 9},
    '0x3d38F87BeF0123f5764635C7E11048027E0721a8': {'symbol': 'EWW', 'decimals': 9},
    '0xB8E2e2101eD11e9138803cd3e06e16dd19910647': {'symbol': 'ARDX', 'decimals': 2},
    '0x56d811088235F11C8920698a204A5010a788f4b3': {'symbol': 'BZRX', 'decimals': 18},
    '0x514910771AF9Ca656af840dff83E8264EcF986CA': {'symbol': 'LINK', 'decimals': 18},
    '0xDd2e93924BDD4E20C3CF4A8736E5955224FA450E': {'symbol': 'FOHO', 'decimals': 8},
    '0x29127fE04ffa4c32AcAC0fFe17280ABD74eAC313': {'symbol': 'SIFU', 'decimals': 18},
    '0x7707aAdA3Ce7722Ac63B91727DAF1999849F6835': {'symbol': 'BNK', 'decimals': 8},
    '0x6B9f031D718dDed0d681c20cB754F97b3BB81b78': {'symbol': 'GEEQ', 'decimals': 18},
    '0x582d872A1B094FC48F5DE31D3B73F2D9bE47def1': {'symbol': 'TON', 'decimals': 9},
    '0xd9788f3931Ede4D5018184E198699dC6d66C1915': {'symbol': 'YVAAVE', 'decimals': 18},
    '0x30cF203b48edaA42c3B4918E955fED26Cd012A3F': {'symbol': 'SEED', 'decimals': 18},
    '0x0B44547be0A0Df5dCd5327de8EA73680517c5a54': {'symbol': 'DC', 'decimals': 18},
    '0xFDb15e5E6799Be72798b1ccFAEcbF186Bf73A0c4': {'symbol': 'NTX', 'decimals': 8},
    '0x5f944B0c4315Cb7c3a846b025AB4045da44abf6c': {'symbol': 'GCAKE', 'decimals': 18},
    '0xdc77aCC82ccE1Cc095CbA197474Cc06824adE6F7': {'symbol': 'OVL', 'decimals': 18},
    '0x410e731c2970Dce3AdD351064AcF5cE9E33FDBf0': {'symbol': 'ONIT', 'decimals': 18},
    '0xC3FFbE26E9446aC52008FfC8C1191d60A12bdC48': {'symbol': 'FAKT', 'decimals': 18},
    '0x76dbCba41e11784BFA95cE30e529fA533C3fA4aD': {'symbol': 'SMT', 'decimals': 18},
    '0x5f4c148D17Effd165C2e2d46b46d2BD6e3eBDC3e': {'symbol': 'RUN', 'decimals': 18},
    '0x1e4746dC744503b53b4A082cB3607B169a289090': {'symbol': 'IPOR', 'decimals': 18},
    '0xd7c302fc3ac829C7E896a32c4Bd126f3e8Bd0a1f': {'symbol': 'B2M', 'decimals': 18},
    '0x2F4eb100552ef93840d5aDC30560E5513DFfFACb': {'symbol': 'BB-A-USDT', 'decimals': 18},
    '0xbe1dBe6741fB988fb571AB1E28CfFB36e3C62629': {'symbol': 'MAV', 'decimals': 18},
    '0x0ab87046fBb341D058F17CBC4c1133F25a20a52f': {'symbol': 'GOHM', 'decimals': 18},
    '0x5dbcF33D8c2E976c6b560249878e6F1491Bca25c': {'symbol': 'YVAULT-LP-YCURVE', 'decimals': 18},
    '0x8503a7b00B4b52692cC6c14e5b96F142E30547b7': {'symbol': 'MEED', 'decimals': 18},
    '0x6A68DE599E8E0b1856E322CE5Bd11c5C3C79712B': {'symbol': 'IBY', 'decimals': 18},
    '0x4e432A62733a7ee38Ad2e16b3cc0731457eA5B55': {'symbol': 'QDT', 'decimals': 4},
    '0x207e14389183A94343942de7aFbC607F57460618': {'symbol': 'THOL', 'decimals': 18},
    '0xab306326bC72c2335Bd08F42cbec383691eF8446': {'symbol': 'PPIZZA', 'decimals': 18},
    '0x968Cbe62c830A0Ccf4381614662398505657A2A9': {'symbol': 'TPY', 'decimals': 8},
    '0xc133529e57681b2999708F9458Be5634e293995E': {'symbol': 'NAAL', 'decimals': 18},
    '0x337DD23d05c27bff099d3604938BFc23A9B25820': {'symbol': 'MFC', 'decimals': 18},
    '0xfC1Cb4920dC1110fD61AfaB75Cf085C1f871b8C6': {'symbol': 'ELT', 'decimals': 18},
    '0x14Da7b27b2E0FedEfe0a664118b0c9bc68e2E9AF': {'symbol': 'BCUG', 'decimals': 18},
    '0x7cB683151A83c2B10A30cBB003CDA9996228a2bA': {'symbol': 'IYKYK', 'decimals': 18},
    '0x2bE1e42BF263AaB47D27Ba92E72c14823e101D7C': {'symbol': 'FFRAX', 'decimals': 18},
    '0x5dD57Da40e6866C9FcC34F4b6DDC89F1BA740DfE': {'symbol': 'BRIGHT', 'decimals': 18},
    '0x5Ca135cB8527d76e932f34B5145575F9d8cbE08E': {'symbol': 'FPI', 'decimals': 18},
    '0x2C0da41C89AdB5a1d4430E5761b9B400911426B0': {'symbol': 'DAMA', 'decimals': 18},
    '0xa49811140E1d6f653dEc28037Be0924C811C4538': {'symbol': 'PRUF', 'decimals': 18},
    '0x63b4f3e3fa4e438698CE330e365E831F7cCD1eF4': {'symbol': 'CFI', 'decimals': 18},
    '0x12E951934246186F50146235d541D3bD1D463E4d': {'symbol': 'CRI', 'decimals': 18},
    '0x9Cb2f26A23b8d89973F08c957C4d7cdf75CD341c': {'symbol': 'DZAR', 'decimals': 6},
    '0xcbd55D4fFc43467142761A764763652b48b969ff': {'symbol': 'ASTRO', 'decimals': 18},
    '0xB6c3DC857845a713d3531cea5ac546F6767992F4': {'symbol': 'ADCO', 'decimals': 6},
    '0x455F7Ef6D8BCfc35f9337e85aEe1B0600a59FabE': {'symbol': 'ALOHA', 'decimals': 18},
    '0xf3dcbc6D72a4E1892f7917b7C43b74131Df8480e': {'symbol': 'BDP', 'decimals': 18},
    '0x190fB342aa6a15eB82903323ae78066fF8616746': {'symbol': 'UMC', 'decimals': 6},
    '0x0D15009896EfE9972F8E086bdD3BcBa5c1f74bF3': {'symbol': 'SONO', 'decimals': 8},
    '0xfA93660C3f6a848556Bb8E265f994160A1F2B289': {'symbol': 'CBT', 'decimals': 18},
    '0x651e44D0df89055c00513d0efb9E8d1A0C843378': {'symbol': 'HAPPY', 'decimals': 18},
    '0x09970aec766b6f3223aCA9111555E99DC50Ff13a': {'symbol': 'LEVL', 'decimals': 18},
    '0x87a92428bBc876d463C21C8e51b903f127d9A9F4': {'symbol': 'AUC', 'decimals': 18},
    '0x7105FAA4a26eD1c67B8B2b41BEc98F06Ee21D105': {'symbol': 'X7105', 'decimals': 18},
    '0x4bD70556ae3F8a6eC6C4080A0C327B24325438f3': {'symbol': 'HXRO', 'decimals': 18},
    '0xe87e15B9c7d989474Cb6d8c56b3DB4eFAD5b21E8': {'symbol': 'HOKK', 'decimals': 18},
    '0x27Ad8e47c48063b209fa5CA14c8c46eCe49B82d2': {'symbol': 'AWRT', 'decimals': 9},
    '0xDF347911910b6c9A4286bA8E2EE5ea4a39eB2134': {'symbol': 'BOB', 'decimals': 18},
    '0x2fd61567c29E7ADB4Ca17e60E1f4a3Fcfe68aCb8': {'symbol': 'SYM', 'decimals': 18},
    '0x6c5bA91642F10282b576d91922Ae6448C9d52f4E': {'symbol': 'PHA', 'decimals': 18},
    '0x34103e1190b824a44C6a638438d425cbA21143BA': {'symbol': 'BITE', 'decimals': 18},
    '0x0F5D2fB29fb7d3CFeE444a200298f468908cC942': {'symbol': 'MANA', 'decimals': 18},
    '0x4527a3B4A8A150403090a99b87efFC96F2195047': {'symbol': 'P2PS', 'decimals': 8},
    '0xaA7a9CA87d3694B5755f213B5D04094b8d0F0A6F': {'symbol': 'TRAC', 'decimals': 18},
    '0x476c5E26a75bd202a9683ffD34359C0CC15be0fF': {'symbol': 'SRM', 'decimals': 6},
    '0x84777600c51996B2ce92a514FF2583B7d7C2239b': {'symbol': 'RVUSD', 'decimals': 18},
    '0x47481c1b44F2A1c0135c45AA402CE4F4dDE4D30e': {'symbol': 'MPT', 'decimals': 18},
    '0xf0a163a26601D9dc8Aef26b388Eadb7B1c620d24': {'symbol': 'TICKR', 'decimals': 9},
    '0xBEE709aa42375acE642338F7f657ff859a19c9bC': {'symbol': 'MCT', 'decimals': 18},
    '0x34612903Db071e888a4dADcaA416d3EE263a87b9': {'symbol': 'ARTE', 'decimals': 18},
    '0xd945d2031b4C63C0E363304FB771F709b502DC0a': {'symbol': 'BMC', 'decimals': 18},
    '0x85Ffb35957203dfD12061eAeCD708dB623Bd567C': {'symbol': 'LTY', 'decimals': 18},
    '0x10B506Ae42b5466d548EdC4C02a4CE9828386C75': {'symbol': 'EHIVE', 'decimals': 18},
    '0x8F0921f30555624143d427b340b1156914882C10': {'symbol': 'FYP', 'decimals': 18},
    '0xe11ba472F74869176652C35D30dB89854b5ae84D': {'symbol': 'YVHEGIC', 'decimals': 18},
    '0x74faaB6986560fD1140508e4266D8a7b87274Ffd': {'symbol': 'HDAO', 'decimals': 18},
    '0x36b00c4c6CE3653a091c7940fC98C3acb0043871': {'symbol': 'SHIBAMON', 'decimals': 9},
    '0x07327a00ba28D413f745C931bbe6bE053B0AD2a6': {'symbol': 'HUM', 'decimals': 18},
    '0xa1faa113cbE53436Df28FF0aEe54275c13B40975': {'symbol': 'ALPHA', 'decimals': 18},
    '0xe9fa21E671BcfB04E6868784b89C19d5aa2424Ea': {'symbol': 'ECTE', 'decimals': 18},
    '0x5283D291DBCF85356A21bA090E6db59121208b44': {'symbol': 'BLUR', 'decimals': 18},
    '0xC3c221fE28c33814c28c822b631fd76047eF1a63': {'symbol': 'MM', 'decimals': 18},
    '0xcc7ab8d78dBA187dC95bF3bB86e65E0C26d0041f': {'symbol': 'SPACE', 'decimals': 18},
    '0x604026696fDB3C6720AE3049C46d59AC604dEA0A': {'symbol': 'XJP', 'decimals': 18},
    '0x328C4c80BC7aCa0834Db37e6600A6c49E12Da4DE': {'symbol': 'ASNX', 'decimals': 18},
    '0x0AB39AC604f992aAeC3C36dE337c3CD3917a7d26': {'symbol': 'KVERSE', 'decimals': 18},
    '0x6E109E9dD7Fa1a58BC3eff667e8e41fC3cc07AEF': {'symbol': 'CNHT', 'decimals': 6},
    '0x79126d32a86e6663F3aAac4527732d0701c1AE6c': {'symbol': 'DMT', 'decimals': 18},
    '0x37d299eF5548BF19503Cb72C085799e331F9097d': {'symbol': 'GRNC', 'decimals': 18},
    '0xF61BF4d1A948487d61b8fa63808AAC06bda55F98': {'symbol': 'TR3', 'decimals': 18},
    '0xA5f1DbB0e55bc31f32c6d032BeE330288490e722': {'symbol': 'DBD', 'decimals': 18},
    '0x6a969D379700B2E5EA4E684d273d63C1c050Ba49': {'symbol': 'PAF', 'decimals': 18},
    '0x856c4Efb76C1D1AE02e20CEB03A2A6a08b0b8dC3': {'symbol': 'OETH', 'decimals': 18},
    '0x9a96E767bFCcE8E80370BE00821ED5BA283D4A17': {'symbol': 'GOGO', 'decimals': 18},
    '0xbE393AA534F82c0ffAC31BF06A23e283AcB3352B': {'symbol': 'NTB', 'decimals': 18},
    '0xa8262Eb913FccEa4C3f77fc95b8b4043B384cFbB': {'symbol': 'KGC', 'decimals': 18},
    '0x60EF10EDfF6D600cD91caeCA04caED2a2e605Fe5': {'symbol': 'MOCHI', 'decimals': 18},
    '0xF5238462E7235c7B62811567E63Dd17d12C2EAA0': {'symbol': 'CGT', 'decimals': 8},
    '0x8640353CDc9778DEaB0dF45D12FB3013dEAC079C': {'symbol': 'LIFE', 'decimals': 18},
    '0x0eC78ED49C2D27b315D462d43B5BAB94d2C79bf8': {'symbol': 'ZERO', 'decimals': 18},
    '0x51Fa2efd62ee56a493f24AE963eAce7D0051929E': {'symbol': 'PSDNOCEAN', 'decimals': 18},
    '0xCFe4EB08e33272d98cb31e37A7BE78d5C1b740c1': {'symbol': 'VB', 'decimals': 18},
    '0x4eE438be38F8682ABB089F2BFeA48851C5E71EAF': {'symbol': 'YAE', 'decimals': 18},
    '0xFf19138b039D938db46bDDA0067DC4BA132ec71C': {'symbol': 'SNET', 'decimals': 8},
    '0x492798Fb464e77Cb3cDA62b9a2C3c65162db198e': {'symbol': 'AGG', 'decimals': 18},
    '0x391E86e2C002C70dEe155eAceB88F7A3c38f5976': {'symbol': 'AAMMUNIUSDCWETH', 'decimals': 18},
    '0xDD782657af86492771BdB369CaEB8c246bE362f5': {'symbol': 'RUG', 'decimals': 12},
    '0xE6D2c3cB986db66818c14C7032DB05D1d2A6ee74': {'symbol': 'FNB', 'decimals': 8},
    '0xB26631c6dda06aD89B93C71400D25692de89c068': {'symbol': 'MINDS', 'decimals': 18},
    '0xF74941046389c78F12fE44784C0EC9ca7CEb7dc2': {'symbol': 'DMR', 'decimals': 18},
    '0x70c5f366dB60A2a0C59C4C24754803Ee47Ed7284': {'symbol': 'GLO', 'decimals': 18},
    '0xd031edafac6a6ae5425e77F936022E506444C242': {'symbol': 'TSANGNYON', 'decimals': 18},
    '0x095cF7f3e82a1dcAdBF0FbC59023F419883eA296': {'symbol': 'POC', 'decimals': 18},
    '0xf12EC0D3Dab64DdEfBdC96474bDe25af3FE1B327': {'symbol': 'STACY', 'decimals': 18},
    '0xa92E7c82B11d10716aB534051B271D2f6aEf7Df5': {'symbol': 'ARA', 'decimals': 18},
    '0x68d57c9a1C35f63E2c83eE8e49A64e9d70528D25': {'symbol': 'SRN', 'decimals': 18},
    '0x8916187112fd6e3624a80C42FA354Cd9756E695f': {'symbol': 'SHIRD', 'decimals': 9},
    '0xf4d2888d29D722226FafA5d9B24F9164c092421E': {'symbol': 'LOOKS', 'decimals': 18},
    '0xfC1E690f61EFd961294b3e1Ce3313fBD8aa4f85d': {'symbol': 'ADAI', 'decimals': 18},
    '0x89303500a7Abfb178B274FD89F2469C264951e1f': {'symbol': 'REF', 'decimals': 8},
    '0x678e840C640F619E17848045D23072844224dD37': {'symbol': 'CRTS', 'decimals': 18},
    '0x99B600D0a4abdbc4a6796225a160bCf3D5Ce2a89': {'symbol': 'SRM', 'decimals': 18},
    '0x0642026E7f0B6cCaC5925b4E7Fa61384250e1701': {'symbol': 'H2O', 'decimals': 18},
    '0x3E828ac5C480069D4765654Fb4b8733b910b13b2': {'symbol': 'CLNY', 'decimals': 18},
    '0x307d45Afbb7E84F82ef3D251A6bb0F00Edf632E4': {'symbol': 'PLA', 'decimals': 18},
    '0x69BEaB403438253f13b6e92Db91F7FB849258263': {'symbol': 'NTK', 'decimals': 18},
    '0x1E8E29CA51363D923725aB9DaC73Bd7e9C440f71': {'symbol': 'MTAO', 'decimals': 9},
    '0x8D26ccA0D58913932157dD4b294d4C95066172dF': {'symbol': 'MSTO', 'decimals': 18},
    '0x246908BfF0b1ba6ECaDCF57fb94F6AE2FcD43a77': {'symbol': 'DIVI', 'decimals': 8},
    '0x66a0f676479Cee1d7373f3DC2e2952778BfF5bd6': {'symbol': 'WISE', 'decimals': 18},
    '0x142a774E8b52550E88E196CedD7A5835acB646D0': {'symbol': 'SRLTY', 'decimals': 9},
    '0xdb69c3f8977f78952303987E87153c1988CF65a9': {'symbol': 'PUGL', 'decimals': 18},
    '0xf680429328caaaCabee69b7A9FdB21a71419c063': {'symbol': 'BFLY', 'decimals': 18},
    '0xeCcf15A4B5976a1365BAeD5297058B4cA42777C0': {'symbol': 'NTRS', 'decimals': 18},
    '0xb208b1a5664574F72d22E9e9981837310cACFeD6': {'symbol': 'SAKURA', 'decimals': 9},
    '0x4FD51Cb87ffEFDF1711112b5Bd8aB682E54988eA': {'symbol': 'WPT', 'decimals': 18},
    '0xe64b47931f28f89Cc7A0C6965Ecf89EaDB4975f5': {'symbol': 'LUD', 'decimals': 18},
    '0x26CBC7008cD879f4B63B69a915378f2D9b17bBF0': {'symbol': 'NFUP', 'decimals': 18},
    '0xdc9Ac3C20D1ed0B540dF9b1feDC10039Df13F99c': {'symbol': 'UTK', 'decimals': 18},
    '0x6988A804C74Fd04F37DA1EA4781CEa68c9C00F86': {'symbol': 'TRIBL', 'decimals': 18},
    '0xA7fBd9254f10F8e20a31a593C9E8BC0D041e15F6': {'symbol': 'ORBN', 'decimals': 9},
    '0x9E32b13ce7f2E80A01932B42553652E053D6ed8e': {'symbol': 'METIS', 'decimals': 18},
    '0x3c48Ca59bf2699E51d4974d4B6D284AE52076e5e': {'symbol': 'CDS', 'decimals': 18},
    '0x0C37Bcf456bC661C14D596683325623076D7e283': {'symbol': 'ARNX', 'decimals': 18},
    '0x5F474906637bdCDA05f29C74653F6962bb0f8eDa': {'symbol': 'DEFX', 'decimals': 18},
    '0x1f22AD99C334e6689bAe751CC225bCC38fe54F82': {'symbol': 'CMS', 'decimals': 18},
    '0xADE00C28244d5CE17D72E40330B1c318cD12B7c3': {'symbol': 'ADX', 'decimals': 18},
    '0x9c6666d5ff4B53b5eB3bd866664c15d0BFcECaa7': {'symbol': 'ZENI', 'decimals': 18},
    '0x03806Ce5ef69Bd9780EDFb04c29da1F23Db96294': {'symbol': 'TSL', 'decimals': 18},
    '0x5e74C9036fb86BD7eCdcb084a0673EFc32eA31cb': {'symbol': 'SETH', 'decimals': 18},
    '0x9d38F670D15C14716BE1F109a4F453E966A2B6D4': {'symbol': 'QUID', 'decimals': 9},
    '0x88d50B466BE55222019D71F9E8fAe17f5f45FCA1': {'symbol': 'CPT', 'decimals': 8},
    '0x4Efe8665e564bF454cCF5C90Ee16817F7485d5Cf': {'symbol': 'BDT', 'decimals': 18},
    '0x7aE0d42f23C33338dE15bFa89C7405c068d9dC0a': {'symbol': 'VERSE', 'decimals': 18},
    '0x17Ce5A8187c817D659B21083455455A092a173BB': {'symbol': 'WBI', 'decimals': 18},
    '0x1F31DD60b84ea2a273cfC56876437E069aE80afd': {'symbol': 'SHINJI', 'decimals': 9},
    '0xDcEe70654261AF21C44c093C300eD3Bb97b78192': {'symbol': 'WOETH', 'decimals': 18},
    '0xbE601dD49da9EE1d2F64D422c4AECf8EB83c119f': {'symbol': 'JCG', 'decimals': 18},
    '0x2f4eb47A1b1F4488C71fc10e39a4aa56AF33Dd49': {'symbol': 'UNCL', 'decimals': 18},
    '0xa203eB78Fee91c8459C6d4eF3a899d8724Ee5B35': {'symbol': 'MERC', 'decimals': 18},
    '0xa3BA969DD831D4eFf2bAF930e7F66D2eef5D929c': {'symbol': 'SHIBGUN', 'decimals': 18},
    '0x20cD2E7Ec8F5d8b337fe46a4F565Ccef1561b9a9': {'symbol': 'ESG', 'decimals': 18},
    '0xdfbc9050F5B01DF53512DCC39B4f2B2BBaCD517A': {'symbol': 'JOB', 'decimals': 8},
    '0xb31eF9e52d94D4120eb44Fe1ddfDe5B4654A6515': {'symbol': 'DOSE', 'decimals': 18},
    '0x40d2025Ed2E89632D3A41d8541DF9Ed2AC0e2b1c': {'symbol': 'CON', 'decimals': 18},
    '0x4fD5b9B5dcc9a5D5931d007ba4aE573e760d9B64': {'symbol': 'MO', 'decimals': 18},
    '0x1426cC6D52D1B14e2B3b1Cb04d57ea42B39c4c7c': {'symbol': 'JSHIBA', 'decimals': 9},
    '0x869DBe51Dc214fcb663604b0F7B548592F8C71f5': {'symbol': 'DOKI', 'decimals': 9},
    '0x469084939d1c20Fae3C73704FE963941C51bE863': {'symbol': 'VIS', 'decimals': 18},
    '0xcf9c692F7e62Af3c571D4173fd4ABf9A3E5330D0': {'symbol': 'ONIGIRI', 'decimals': 18},
    '0x3C4008eca800Ec1283E4cf500E68D06BFaBc00a8': {'symbol': 'HAO', 'decimals': 18},
    '0x7865af71cf0b288b4E7F654f4F7851EB46a2B7F8': {'symbol': 'SNTVT', 'decimals': 18},
    '0xa4eF4b0B23C1fc81d3f9ecF93510e64f58A4A016': {'symbol': '1MIL', 'decimals': 18},
    '0x6fC13EACE26590B80cCCAB1ba5d51890577D83B2': {'symbol': 'UMB', 'decimals': 18},
    '0xC18c07a18198A6340cf4d94855fE5eb6DD33b46E': {'symbol': 'QLINDO', 'decimals': 0},
    '0x106538CC16F938776c7c180186975BCA23875287': {'symbol': 'BAS', 'decimals': 18},
    '0xe75ad3aAB14E4B0dF8c5da4286608DaBb21Bd864': {'symbol': 'AAC', 'decimals': 5},
    '0x297D33e17e61C2Ddd812389C2105193f8348188a': {'symbol': 'TRDL', 'decimals': 18},
    '0xC89d9aa9D9E54bb196319c6195AEA1038d2bc936': {'symbol': 'TRENDX', 'decimals': 18},
    '0x5E031a5bC1c6b6E86A49e0B0F2e757800F1D0FFf': {'symbol': 'DAF', 'decimals': 18},
    '0x06874F973Dc3c96dc22A10eF0D0609F877f335EA': {'symbol': 'STA', 'decimals': 18},
    '0xa10740ff9FF6852eac84cdcfF9184e1D6d27C057': {'symbol': 'WG0', 'decimals': 18},
    '0x26A9c0c6F28ae2e69270BC39F13dB87A27Db4CE5': {'symbol': 'DBOMB', 'decimals': 18},
    '0x33840024177A7DacA3468912363BeD8b425015c5': {'symbol': 'EBOX', 'decimals': 18},
    '0xc3B5284B2c0cfa1871a6ac63B6d6ee43c08BDC79': {'symbol': 'BGAN', 'decimals': 18},
    '0xcf0C122c6b73ff809C693DB761e7BaeBe62b6a2E': {'symbol': 'FLOKI', 'decimals': 9},
    '0xc8d07671aFbA9492Da95819De4AD10859e00aB7F': {'symbol': 'CCP', 'decimals': 18},
    '0xc39a048353842CAf1B94ce5967A04149Bb435b98': {'symbol': 'NEWS', 'decimals': 9},
    '0x159A1dFAe19057de57dFfFcbB3DA1aE784678965': {'symbol': 'RFX', 'decimals': 8},
    '0x9b4e2B4B13d125238Aa0480dD42B4f6fC71b37CC': {'symbol': 'MT', 'decimals': 18},
    '0xC382e04099A435439725BB40647e2B32dC136806': {'symbol': 'COGE', 'decimals': 18},
    '0x3155BA85D5F96b2d030a4966AF206230e46849cb': {'symbol': 'RUNE', 'decimals': 18},
    '0x490BBbc2485e99989Ba39b34802faFa58e26ABa4': {'symbol': 'MAWETH', 'decimals': 18},
    '0x77C6E4a580c0dCE4E5c7a17d0bc077188a83A059': {'symbol': 'SWUSD', 'decimals': 18},
    '0x2Fc246aA66F0da5bB1368F688548ecBBE9bdee5d': {'symbol': 'TEMCO', 'decimals': 18},
    '0xaAAf91D9b90dF800Df4F55c205fd6989c977E73a': {'symbol': 'TKN', 'decimals': 8},
    '0x84cA8bc7997272c7CfB4D0Cd3D55cd942B3c9419': {'symbol': 'DIA', 'decimals': 18},
    '0xf293d23BF2CDc05411Ca0edDD588eb1977e8dcd4': {'symbol': 'SYLO', 'decimals': 18},
    '0xc3CaB0d0918c39E29088c495a077EcD022758798': {'symbol': 'KMASK', 'decimals': 18},
    '0xcB8d1260F9c92A3A545d409466280fFdD7AF7042': {'symbol': 'NFT', 'decimals': 18},
    '0xfAC77A24E52B463bA9857d6b758ba41aE20e31FF': {'symbol': 'LSD', 'decimals': 18},
    '0x7E9D8f07A64e363e97A648904a89fb4cd5fB94CD': {'symbol': 'FF', 'decimals': 18},
    '0x664C6E221c77313307467B121528ad563107bD01': {'symbol': 'TAXHAVENINU', 'decimals': 9},
    '0xF0dC76C22139ab22618dDFb498BE1283254612B1': {'symbol': 'WSTR', 'decimals': 18},
    '0xd61143652aF94f570C7D9429356662dD859cA6EC': {'symbol': 'FINT', 'decimals': 18},
    '0xac3211a5025414Af2866FF09c23FC18bc97e79b1': {'symbol': 'DOV', 'decimals': 18},
    '0xdAC17F958D2ee523a2206206994597C13D831ec7': {'symbol': 'USDT', 'decimals': 6},
    '0x37E808F084101F75783612407e7C3f5F92d8ee3F': {'symbol': 'RI', 'decimals': 18},
    '0x1f19f83fC9a25F3C861260143E36c17706257986': {'symbol': 'VEST', 'decimals': 18},
    '0x1c4481750daa5Ff521A2a7490d9981eD46465Dbd': {'symbol': 'BCPT', 'decimals': 18},
    '0xA5269A8e31B93Ff27B887B56720A25F844db0529': {'symbol': 'MAUSDC', 'decimals': 18},
    '0xE9287543684f440b2f29983A3472987bF7b0ed12': {'symbol': 'SHINO', 'decimals': 18},
    '0x68037790A0229e9Ce6EaA8A99ea92964106C4703': {'symbol': 'PAR', 'decimals': 18},
    '0xA1B0edF4460CC4d8bFAA18Ed871bFF15E5b57Eb4': {'symbol': 'AAMMUNIBATWETH', 'decimals': 18},
    '0x6D0F5149c502faf215C89ab306ec3E50b15e2892': {'symbol': 'PRT', 'decimals': 18},
    '0x62A8C2818BD7034DC24CD22368C3E53E8eB47c18': {'symbol': 'INNIT', 'decimals': 18},
    '0x045109cF1Be9eDEC048AA0B3D7a323154a1aEA65': {'symbol': 'ELEV', 'decimals': 18},
    '0xD31a59c85aE9D8edEFeC411D448f90841571b89c': {'symbol': 'SOL', 'decimals': 9},
    '0x37E8789bB9996CaC9156cD5F5Fd32599E6b91289': {'symbol': 'AID', 'decimals': 18},
    '0xA15C7Ebe1f07CaF6bFF097D8a589fb8AC49Ae5B3': {'symbol': 'NPXS', 'decimals': 18},
    '0xe65cdB6479BaC1e22340E4E755fAE7E509EcD06c': {'symbol': 'CAAVE', 'decimals': 8},
    '0x6be61833FC4381990e82D7D4a9F4c9B3F67eA941': {'symbol': 'HTB', 'decimals': 18},
    '0xc2a2E9B87b37938D934Fd51CfEC0C5Fd5cB14c31': {'symbol': 'FLUFFY', 'decimals': 18},
    '0x187D1018E8ef879BE4194d6eD7590987463eAD85': {'symbol': 'FUZE', 'decimals': 18},
    '0xE7f58A92476056627f9FdB92286778aBd83b285F': {'symbol': 'DWEB', 'decimals': 18},
    '0x7a5ce6abD131EA6B148a022CB76fc180ae3315A6': {'symbol': 'BALPHA', 'decimals': 18},
    '0x8B1F49491477e0fB46a29fef53F1EA320D13c349': {'symbol': 'AMM', 'decimals': 6},
    '0x790bFaCaE71576107C068f494c8A6302aea640cb': {'symbol': 'CBC', 'decimals': 18},
    '0x721A1B990699eE9D90b6327FaaD0A3E840aE8335': {'symbol': 'LOOT', 'decimals': 18},
    '0x42d6622deCe394b54999Fbd73D108123806f6a18': {'symbol': 'SPANK', 'decimals': 18},
    '0x9196E18Bc349B1F64Bc08784eaE259525329a1ad': {'symbol': 'PUSSY', 'decimals': 18},
    '0x515d7E9D75E2b76DB60F8a051Cd890eBa23286Bc': {'symbol': 'GDAO', 'decimals': 18},
    '0xB6eD7644C69416d67B522e20bC294A9a9B405B31': {'symbol': '0XBTC', 'decimals': 8},
    '0xaDB2437e6F65682B85F814fBc12FeC0508A7B1D0': {'symbol': 'UNCX', 'decimals': 18},
    '0x04E0Af0af1b7f0023c6B12af5a94Df59B0e8cF59': {'symbol': 'SETS', 'decimals': 18},
    '0x25d4e7fe624FcC6E7F0b52D142F40c9ACEcF039a': {'symbol': 'KENNEL', 'decimals': 18},
    '0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32': {'symbol': 'LDO', 'decimals': 18},
    '0x892A6f9dF0147e5f079b0993F486F9acA3c87881': {'symbol': 'XFUND', 'decimals': 9},
    '0x7Ddc52c4De30e94Be3A6A0A2b259b2850f421989': {'symbol': 'GMT', 'decimals': 18},
    '0x01b23286FF60a543ec29366aE8D6B6274cA20541': {'symbol': 'BMP', 'decimals': 18},
    '0x21018CBC9ad730542130bE180b577b74DB2a9397': {'symbol': 'PAPER', 'decimals': 18},
    '0x198d7387Fa97A73F05b8578CdEFf8F2A1f34Cd1F': {'symbol': 'WJAURA', 'decimals': 18},
    '0xa59e341e8047498700eD244814b01b34547fb21B': {'symbol': 'MOH', 'decimals': 18},
    '0x4521C9aD6A3D4230803aB752Ed238BE11F8B342F': {'symbol': 'SANI', 'decimals': 18},
    '0xf42965f82f9e3171d1205C5e3058Caf324a09432': {'symbol': 'ALIEN', 'decimals': 9},
    '0xb3Cc3d7E656893F22d2372b0aE57106F6b155Cbe': {'symbol': 'DOGGER', 'decimals': 18},
    '0x40897C872214303b6F479a37E549eE1516B264A2': {'symbol': 'FLETA', 'decimals': 18},
    '0xa92c49C403386111C1629AEe00936eED2A9E74a6': {'symbol': 'KLTR', 'decimals': 18},
    '0xD3c625F54dec647DB8780dBBe0E880eF21BA4329': {'symbol': 'XHT', 'decimals': 18},
    '0x612E1726435fE38dD49A0B35b4065B56f49c8F11': {'symbol': 'CCV2', 'decimals': 18},
    '0xf2ddae89449b7D26309a5D54614B1FC99C608AF5': {'symbol': 'ASTA', 'decimals': 18},
    '0xAec7d1069e3a914a3EB50f0BFB1796751f2ce48a': {'symbol': 'S4F', 'decimals': 18},
    '0x0fbe9Cc858D7AD6e246Fe9D01Aed22abD2A66f0B': {'symbol': 'DNC', 'decimals': 18},
    '0x07Ac55797D4F43f57cA92a49E65ca582cC287c27': {'symbol': 'TBT', 'decimals': 18},
    '0xd559f20296FF4895da39b5bd9ADd54b442596a61': {'symbol': 'FTX', 'decimals': 18},
    '0x18084fbA666a33d37592fA2633fD49a74DD93a88': {'symbol': 'TBTC', 'decimals': 18},
    '0x56de8BC61346321D4F2211e3aC3c0A7F00dB9b76': {'symbol': 'RENA', 'decimals': 18},
    '0xcE593a29905951E8Fc579bC092ecA72577dA575c': {'symbol': 'GR', 'decimals': 6},
    '0x7A41e0517a5ecA4FdbC7FbebA4D4c47B9fF6DC63': {'symbol': 'ZSC', 'decimals': 18},
    '0x9008064E6cf73e27a3Aba4B10E69f855A4F8efcC': {'symbol': 'GEM', 'decimals': 18},
    '0x6e5970DBd6fc7eb1f29C6D2eDF2bC4c36124C0C1': {'symbol': 'TRADE', 'decimals': 18},
    '0x5CB3ce6D081fB00d5f6677d196f2d70010EA3f4a': {'symbol': 'BUSY', 'decimals': 18},
    '0x8F12Dfc7981DE79A8A34070a732471f2D335EecE': {'symbol': 'CE', 'decimals': 18},
    '0x5218E472cFCFE0b64A064F055B43b4cdC9EfD3A6': {'symbol': 'ERSDL', 'decimals': 18},
    '0xd52AAE39a2b5CC7812F7B9450EBB61DfEf702B15': {'symbol': 'MAGE', 'decimals': 18},
    '0x4946Fcea7C692606e8908002e55A582af44AC121': {'symbol': 'FOAM', 'decimals': 18},
    '0x77777777772cf0455fB38eE0e75f38034dFa50DE': {'symbol': 'XY', 'decimals': 18},
    '0x238A1FcC2F65ed50c1e1797cF5857CBA3E82a55B': {'symbol': 'MOETA', 'decimals': 9},
    '0x1c7E83f8C581a967940DBfa7984744646AE46b29': {'symbol': 'RND', 'decimals': 18},
    '0x70Bef3bB2f001dA2fDDb207dAe696cD9FAFf3f5d': {'symbol': 'NST', 'decimals': 18},
    '0x1a87077C4F834884691B8ba4fc808D2eC93A9F30': {'symbol': 'WINRY', 'decimals': 9},
    '0x888888888889C00c67689029D7856AAC1065eC11': {'symbol': 'OPIUM', 'decimals': 18},
    '0x8013266cb5c9dd48bE3Ad7D1CE832874d64B3Ce1': {'symbol': 'BOOP', 'decimals': 18},
    '0xb24cd494faE4C180A89975F1328Eab2a7D5d8f11': {'symbol': 'CODE', 'decimals': 18},
    '0x64aa3364F17a4D01c6f1751Fd97C2BD3D7e7f1D5': {'symbol': 'OHM', 'decimals': 9},
    '0x23352036E911A22Cfc692B5E2E196692658ADED9': {'symbol': 'FDZ', 'decimals': 18},
    '0x6fe56C0bcdD471359019FcBC48863d6c3e9d4F41': {'symbol': 'PROPS', 'decimals': 18},
    '0x0acC0FEE1D86D2cD5AF372615bf59b298D50cd69': {'symbol': 'ILSI', 'decimals': 18},
    '0xBfB2b6870501a6Ff17121D676A0A45a38c9eeD1e': {'symbol': 'TOAD', 'decimals': 9},
    '0x553539d40AE81FD0d9C4b991B0b77bE6f6Bc030e': {'symbol': 'SMOL', 'decimals': 18},
    '0xB705268213D593B8FD88d3FDEFF93AFF5CbDcfAE': {'symbol': 'IDEX', 'decimals': 18},
    '0xd4Df22556e07148e591B4c7b4f555a17188CF5cF': {'symbol': 'TWT', 'decimals': 9},
    '0xd714d91A169127e11D8FAb3665d72E8b7ef9Dbe2': {'symbol': 'BLACK', 'decimals': 18},
    '0x0563DCe613D559a47877fFD1593549fb9d3510D6': {'symbol': 'SUPERBID', 'decimals': 18},
    '0x530824DA86689C9C17CdC2871Ff29B058345b44a': {'symbol': 'STBT', 'decimals': 18},
    '0xcbC1Aa6961B28cCcb11E1cdDd84Ad8B42Ec920b1': {'symbol': 'ZEUS10000', 'decimals': 4},
    '0xD33526068D116cE69F19A9ee46F0bd304F21A51f': {'symbol': 'RPL', 'decimals': 18},
    '0x8c088775e4139af116Ac1FA6f281Bbf71E8c1c73': {'symbol': 'PUMLX', 'decimals': 18},
    '0x045414e728067Ab3da4bCeafC0D992D59183463A': {'symbol': 'BOX', 'decimals': 18},
    '0xbA3E5F8b4200a5eb856FF2C3E001aB29444491AA': {'symbol': 'MBASE', 'decimals': 18},
    '0xB9D7DdDca9a4AC480991865EfEf82E01273F79C3': {'symbol': 'BLUSD', 'decimals': 18},
    '0x39Ea10E507720783C27eDd5F96Bf2D6e199579B8': {'symbol': 'HONR', 'decimals': 18},
    '0xB90cb79B72EB10c39CbDF86e50B1C89F6a235f2e': {'symbol': 'AUDT', 'decimals': 18},
    '0x383B0b2ce2e1757b5e1D087D8f36013eA595541a': {'symbol': 'HEAL', 'decimals': 9},
    '0x2F6081E3552b1c86cE4479B80062A1ddA8EF23E3': {'symbol': 'USDX', 'decimals': 9},
    '0x2e1E15C44Ffe4Df6a0cb7371CD00d5028e571d14': {'symbol': 'MTLX', 'decimals': 18},
    '0xEe9E5eFF401ee921b138490d00CA8D1F13f67A72': {'symbol': 'AFIN', 'decimals': 8},
    '0x406B9DCA8b52f08385014eC1eD1cf6A0D5c01289': {'symbol': 'MEISHU', 'decimals': 9},
    '0xB69d946D8cdC5238B0658188e1baBB41b8652A29': {'symbol': 'BXB', 'decimals': 18},
    '0x903bEF1736CDdf2A537176cf3C64579C3867A881': {'symbol': 'ICHI', 'decimals': 9},
    '0x5d285F735998F36631F678FF41fb56A10A4d0429': {'symbol': 'MIX', 'decimals': 18},
    '0x0Cf0Ee63788A0849fE5297F3407f701E122cC023': {'symbol': 'XDATA', 'decimals': 18},
    '0x33d63Ba1E57E54779F7dDAeaA7109349344cf5F1': {'symbol': 'DATA', 'decimals': 18},
    '0x3BE7bF1A5F23BD8336787D0289B70602f1940875': {'symbol': 'VIDT', 'decimals': 18},
    '0xc12d099be31567add4e4e4d0D45691C3F58f5663': {'symbol': 'AUC', 'decimals': 18},
    '0x39207D2E2fEEF178FBdA8083914554C59D9f8c00': {'symbol': 'INUS', 'decimals': 18},
    '0x4dAeb4a06F70f4b1A5C329115731fE4b89C0B227': {'symbol': 'QUA', 'decimals': 18},
    '0x1F4cb968b76931C494Ff92ED80CCB169ad641cb1': {'symbol': 'STF', 'decimals': 18},
    '0xb19Dd661F076998e3B0456935092a233e12C2280': {'symbol': 'UM', 'decimals': 18},
    '0x243cACb4D5fF6814AD668C3e225246efA886AD5a': {'symbol': 'SHI', 'decimals': 18},
    '0xCAE7F3aa6D456463bEf0E5B508542b69D96f2904': {'symbol': 'INVECTAI', 'decimals': 8},
    '0xf1Dc500FdE233A4055e25e5BbF516372BC4F6871': {'symbol': 'SDL', 'decimals': 18},
    '0x00059AE69c1622A7542EdC15E8d17b060fE307b6': {'symbol': 'AMON', 'decimals': 18},
    '0x6810e776880C02933D47DB1b9fc05908e5386b96': {'symbol': 'GNO', 'decimals': 18},
    '0x7f280daC515121DcdA3EaC69eB4C13a52392CACE': {'symbol': 'FNC', 'decimals': 18},
    '0x7E77dCb127F99ECe88230a64Db8d595F31F1b068': {'symbol': 'SILV2', 'decimals': 18},
    '0x3aadc3BCe49724ce299fD9F3850211211c399C8c': {'symbol': 'CSX', 'decimals': 18},
    '0x69570f3E84f51Ea70b7B68055c8d667e77735a25': {'symbol': 'BSGG', 'decimals': 18},
    '0x178Bf8fD04b47D2De3eF3f6b3D112106375ad584': {'symbol': 'UUSDT', 'decimals': 6},
    '0x00281dFCe4cfD72c0b6FDA2AAaF077258743f9E8': {'symbol': 'NRFB', 'decimals': 0},
    '0xC5FDf3569af74f3B3e97E46A187a626352D2d508': {'symbol': 'ZINU', 'decimals': 9},
    '0x4a527d8fc13C5203AB24BA0944F4Cb14658D1Db6': {'symbol': 'MITX', 'decimals': 18},
    '0x3a1311B8C404629E38f61D566cefEFed083B9670': {'symbol': 'PINU', 'decimals': 9},
    '0x370adc71f67f581158Dc56f539dF5F399128Ddf9': {'symbol': 'AAMMUNIMKRWETH', 'decimals': 18},
    '0xcAa9Ed6D7502595B555113D4517668aE24038C8a': {'symbol': 'MARVIN', 'decimals': 18},
    '0x0198f46f520F33cd4329bd4bE380a25a90536CD5': {'symbol': 'PLA', 'decimals': 18},
    '0x4d4d883f920f7c0C36a1bE71a02Aa0cde2AA22d1': {'symbol': 'OPCH', 'decimals': 18},
    '0xa1d65E8fB6e87b60FECCBc582F7f97804B725521': {'symbol': 'DXD', 'decimals': 18},
    '0x430EF9263E76DAE63c84292C3409D61c598E9682': {'symbol': 'PYR', 'decimals': 18},
    '0x56322c14cb5F53531c231b8F9Ab178bf5d893A5e': {'symbol': 'AZA', 'decimals': 18},
    '0xb7C2fcD6d7922eddd2A7A9B0524074A60D5b472C': {'symbol': 'VST', 'decimals': 18},
    '0xC3f65b1c45B0D42a472CD0Fbd37dD6678AEf667a': {'symbol': 'CIMO', 'decimals': 9},
    '0x3A880652F47bFaa771908C07Dd8673A787dAEd3A': {'symbol': 'DDX', 'decimals': 18},
    '0xef6344de1fcfC5F48c30234C16c1389e8CdC572C': {'symbol': 'DNA', 'decimals': 18},
    '0x5B322514FF727253292637D9054301600c2C81e8': {'symbol': 'DAD', 'decimals': 9},
    '0x6BeA7CFEF803D1e3d5f7C0103f7ded065644e197': {'symbol': 'GAMMA', 'decimals': 18},
    '0xdF574c24545E5FfEcb9a659c229253D4111d87e1': {'symbol': 'HUSD', 'decimals': 8},
    '0x77D9046ee15FAACeB89439FFdDE4be071C2F07bD': {'symbol': 'JIYUU', 'decimals': 18},
    '0xE3feDAeCD47aa8EAb6b23227b0eE56F092C967a9': {'symbol': 'PST', 'decimals': 18},
    '0x23ddBD36547d43627afa9B42d4e9FB0515F48B09': {'symbol': 'HIBEANZ', 'decimals': 18},
    '0xa0b93B9e90aB887E53F9FB8728c009746e989B53': {'symbol': 'TST', 'decimals': 18},
    '0xD5e7D22362BCC9881D06512d3189eAe79DD98d70': {'symbol': 'MIYAZAKI', 'decimals': 9},
    '0x558EC3152e2eb2174905cd19AeA4e34A23DE9aD6': {'symbol': 'BRD', 'decimals': 18},
    '0x299698B4B44BD6d023981A7317798dEe12860834': {'symbol': 'NFP', 'decimals': 9},
    '0x82089a9c7c4a07352F7433fBce1D4Ee9a722fF29': {'symbol': 'PROXY', 'decimals': 9},
    '0x3449FC1Cd036255BA1EB19d65fF4BA2b8903A69a': {'symbol': 'BAC', 'decimals': 18},
    '0x9D91BE44C06d373a8a226E1f3b146956083803eB': {'symbol': 'AKNC', 'decimals': 18},
    '0x65cCD72c0813CE6f2703593B633202a0F3Ca6a0c': {'symbol': 'EGG', 'decimals': 18},
    '0x40955D77F87123b71b145098358A60573ac7BE96': {'symbol': 'DAISY', 'decimals': 18},
    '0x6944d3e38973C4831dA24E954fbD790c7E688bDd': {'symbol': 'IZE', 'decimals': 18},
    '0xe8272210954eA85DE6D2Ae739806Ab593B5d9c51': {'symbol': 'A5T', 'decimals': 18},
    '0x21BfBDa47A0B4B5b1248c767Ee49F7caA9B23697': {'symbol': 'OVR', 'decimals': 18},
    '0xd2be3722B17b616c51ed9B8944A227D1ce579C24': {'symbol': 'DTUBE', 'decimals': 2},
    '0xA1E770be76BDe604F8Ebb66f640250a787B9422B': {'symbol': 'DEB', 'decimals': 18},
    '0xC2456D2118299a2efDFE6522Ff79aA48fC5d2b00': {'symbol': 'MARU', 'decimals': 9},
    '0x95E40E065AFB3059dcabe4aaf404c1F92756603a': {'symbol': 'KDAG', 'decimals': 18},
    '0x5408245a4D7c685F59cE6D3b8B35916dd6c11A99': {'symbol': 'AZK', 'decimals': 18},
    '0x0CDF9acd87E940837ff21BB40c9fd55F68bba059': {'symbol': 'MINT', 'decimals': 18},
    '0xf920e4F3FBEF5B3aD0A25017514B769bDc4Ac135': {'symbol': 'BAX', 'decimals': 18},
    '0x491604c0FDF08347Dd1fa4Ee062a822A5DD06B5D': {'symbol': 'CTSI', 'decimals': 18},
    '0x450E7F6e3a2f247A51b98C39297a9a5BfbDb3170': {'symbol': 'EGT', 'decimals': 9},
    '0x39fBBABf11738317a448031930706cd3e612e1B9': {'symbol': 'WXRP', 'decimals': 18},
    '0x37F74e99794853777a10ea1dc08a64C86958F06a': {'symbol': 'DILI', 'decimals': 18},
    '0x4cfF49d0A19ed6Ff845a9122fa912AbcFB1F68A6': {'symbol': 'WTK', 'decimals': 18},
    '0xCAabCaA4ca42e1d86dE1a201c818639def0ba7A7': {'symbol': 'TALK', 'decimals': 18},
    '0xea3983Fc6D0fbbC41fb6F6091f68F3e08894dC06': {'symbol': 'UDO', 'decimals': 18},
    '0x4DB26817c88A57D2ef84cFd90168f535Ee83bDA9': {'symbol': 'HIAZUKI', 'decimals': 18},
    '0xEFd113CeA2f656Fe7899A25a06243A6E40406E08': {'symbol': 'THE9', 'decimals': 18},
    '0xA31B1767e09f842ECFd4bc471Fe44F830E3891AA': {'symbol': 'ROOBEE', 'decimals': 18},
    '0x9b00e6E8D787b13756eb919786c9745054DB64f9': {'symbol': 'WSIENNA', 'decimals': 18},
    '0x1FbD3dF007eB8A7477A1Eab2c63483dCc24EfFD6': {'symbol': 'SCA', 'decimals': 18},
    '0xB009BFaAF85e53F55d8657781Eb69feAaed83672': {'symbol': 'EGS', 'decimals': 18},
    '0x7bE00ed6796B21656732E8f739Fc1b8F1C53DA0D': {'symbol': 'ACXT', 'decimals': 18},
    '0x7C5d5100B339Fe7D995a893AF6CB496B9474373c': {'symbol': 'LOON', 'decimals': 18},
    '0xd6327ce1fb9D6020E8C2c0E124A1eC23DCAb7536': {'symbol': 'CUMINU', 'decimals': 18},
    '0x440238CC07186aDEA6653a2E8cb9a24737615609': {'symbol': 'SHIBMERICAN', 'decimals': 9},
    '0x02dE007D412266a2e0Fa9287C103474170F06560': {'symbol': 'EXD', 'decimals': 18},
    '0x32353A6C91143bfd6C7d363B546e62a9A2489A20': {'symbol': 'AGLD', 'decimals': 18},
    '0x27702a26126e0B3702af63Ee09aC4d1A084EF628': {'symbol': 'ALEPH', 'decimals': 18},
    '0xdeFA4e8a7bcBA345F687a2f1456F5Edd9CE97202': {'symbol': 'KNC', 'decimals': 18},
    '0x9A6aeA7C2b0D563EaB701E3255110d30F08B5a64': {'symbol': 'MAPS', 'decimals': 8},
    '0xa3EE21C306A700E682AbCdfe9BaA6A08F3820419': {'symbol': 'CTC', 'decimals': 18},
    '0x3893b9422Cd5D70a81eDeFfe3d5A1c6A978310BB': {'symbol': 'MITH', 'decimals': 18},
    '0x50D1c9771902476076eCFc8B2A83Ad6b9355a4c9': {'symbol': 'FTT', 'decimals': 18},
    '0x4fE83213D56308330EC302a8BD641f1d0113A4Cc': {'symbol': 'NU', 'decimals': 18},
    '0x5C559F3ee9A81Da83e069c0093471cB05D84052A': {'symbol': 'BABYPEPE', 'decimals': 18},
    '0x5c8190b76E90B4DD0702740cF6Eb0F7eE01Ab5E9': {'symbol': 'ARCAI', 'decimals': 9},
    '0xe973E453977195422B48e1852A207b7eE9C913C7': {'symbol': 'AD', 'decimals': 18},
    '0x28fAc5334C9f7262b3A3Fe707e250E01053e07b5': {'symbol': 'IDLEUSDTSAFE', 'decimals': 18},
    '0xcccD1Ba9f7acD6117834E0D28F25645dECb1736a': {'symbol': 'ECOX', 'decimals': 18},
    '0xbD668EC47e10D3bf23b1d68139E545162a0dfCFA': {'symbol': 'GRANX', 'decimals': 18},
    '0x2bDC0D42996017fCe214b21607a515DA41A9E0C5': {'symbol': 'SKIN', 'decimals': 6},
    '0x1FEE5588cb1De19c70B6aD5399152D8C643FAe7b': {'symbol': 'PHTK', 'decimals': 18},
    '0xBf6Ff49FfD3d104302Ef0AB0F10f5a84324c091c': {'symbol': 'NFTFY', 'decimals': 18},
    '0xaE377ed00DF9b9663B253bE93035EF1CfECBc1fB': {'symbol': 'ALERT', 'decimals': 9},
    '0x26DB5439F651CAF491A87d48799dA81F191bDB6b': {'symbol': 'CBC', 'decimals': 8},
    '0x1A4b46696b2bB4794Eb3D4c26f1c55F9170fa4C5': {'symbol': 'BIT', 'decimals': 18},
    '0x1e797Ce986C3CFF4472F7D38d5C4aba55DfEFE40': {'symbol': 'BCDN', 'decimals': 15},
    '0x4086E77C5E993FDB90a406285d00111a974F877a': {'symbol': 'BRWL', 'decimals': 4},
    '0xba5BDe662c17e2aDFF1075610382B9B691296350': {'symbol': 'RARE', 'decimals': 18},
    '0x102203517ce35AC5Cab9a2cda80DF03f26c7419b': {'symbol': 'SMTX', 'decimals': 18},
    '0x1d2d542e6D9d85A712deB4D1a7D96a16CE00B8cE': {'symbol': 'POA', 'decimals': 18},
    '0xd87de4cCef2c2FE651bc4d130cb1A365248f21fA': {'symbol': 'LYFE', 'decimals': 18},
    '0x9B99CcA871Be05119B2012fd4474731dd653FEBe': {'symbol': 'MATTER', 'decimals': 18},
    '0xADc234a4e90E2045f353F5d4fCdE66144d23b458': {'symbol': 'FUSDT', 'decimals': 6},
    '0xdffA3a7f5B40789C7A437DbE7B31b47F9B08FE75': {'symbol': 'HOODIE', 'decimals': 18},
    '0xB3319f5D18Bc0D84dD1b4825Dcde5d5f7266d407': {'symbol': 'CZRX', 'decimals': 8},
    '0x5046E860ff274fb8c66106B0Ffb8155849fB0787': {'symbol': 'JS', 'decimals': 8},
    '0xAd0891Abb1687Fb994a2C1c9699520422573996F': {'symbol': 'ETHER', 'decimals': 18},
    '0x5136C98A80811C3f46bDda8B5c4555CFd9f812F0': {'symbol': 'IDH', 'decimals': 6},
    '0x9469D013805bFfB7D3DEBe5E7839237e535ec483': {'symbol': 'RING', 'decimals': 18},
    '0x6FF1BFa14A57594a5874B37ff6AC5efbD9F9599A': {'symbol': 'TOTM', 'decimals': 18},
    '0x7869c4A1a3f6F8684FBCC422a21aD7Abe3167834': {'symbol': 'PVT', 'decimals': 18},
    '0x888ceA2BBDD5D47a4032cf63668D7525C74af57A': {'symbol': 'POOF', 'decimals': 18},
    '0xFb83869f7EE52AEBadDEA254F647953AfB1F99F8': {'symbol': 'LEAN', 'decimals': 18},
    '0x05D3606d5c81EB9b7B18530995eC9B29da05FaBa': {'symbol': 'TOMOE', 'decimals': 18},
    '0xB087C2180e3134Db396977065817aed91FEa6EAD': {'symbol': 'HELLSING', 'decimals': 9},
    '0x84d821F7FbDD595c4C4A50842913e6b1E07d7a53': {'symbol': 'BNPL', 'decimals': 18},
    '0x044d078F1c86508e13328842Cc75AC021B272958': {'symbol': 'PPC', 'decimals': 6},
    '0x60bE1e1fE41c1370ADaF5d8e66f07Cf1C2Df2268': {'symbol': 'PERC', 'decimals': 18},
    '0xBbff34E47E559ef680067a6B1c980639EEb64D24': {'symbol': 'L2', 'decimals': 18},
    '0x33D0568941C0C64ff7e0FB4fbA0B11BD37deEd9f': {'symbol': 'RAMP', 'decimals': 18},
    '0xEA47B64e1BFCCb773A0420247C0aa0a3C1D2E5C5': {'symbol': 'BAYC', 'decimals': 18},
    '0xc0ea83113038987d974FE667831a36E442e661E7': {'symbol': 'LIBFX', 'decimals': 18},
    '0xa8c8CfB141A3bB59FEA1E2ea6B79b5ECBCD7b6ca': {'symbol': 'NOIA', 'decimals': 18},
    '0xD4f6f9Ae14399fD5Eb8DFc7725F0094a1A7F5d80': {'symbol': 'BST', 'decimals': 18},
    '0x5B685863494C33F344081F75e5430C260c224A32': {'symbol': 'CMCX', 'decimals': 18},
    '0x0258F474786DdFd37ABCE6df6BBb1Dd5dfC4434a': {'symbol': 'ORN', 'decimals': 8},
    '0x1936C91190E901B7dD55229A574AE22B58Ff498a': {'symbol': 'MEVFREE', 'decimals': 18},
    '0x47C1178F49140ECdBfbdF0aE2935cDB640D579F9': {'symbol': 'KCAL', 'decimals': 10},
    '0x618679dF9EfCd19694BB1daa8D00718Eacfa2883': {'symbol': 'XYZ', 'decimals': 18},
    '0xC32cC5b70BEe4bd54Aa62B9Aefb91346d18821C4': {'symbol': 'ITS', 'decimals': 18},
    '0x77E06c9eCCf2E797fd462A92B6D7642EF85b0A44': {'symbol': 'WTAO', 'decimals': 9},
    '0xa8EdA9D4Aee0eb882F8752C6bA7e16d9233C9Ad2': {'symbol': 'LVL', 'decimals': 8},
    '0x549E4D92285ff5A16c9484Ff79211E4358b1f202': {'symbol': 'TIC', 'decimals': 18},
    '0x9dc7094530cB1bcf5442c3b9389ee386738A190c': {'symbol': 'MACRV', 'decimals': 18},
    '0xD7d05bDa4bf5876bA1254b3Eaaf8b47D2F5676eb': {'symbol': 'STA', 'decimals': 18},
    '0x33e18a092a93ff21aD04746c7Da12e35D34DC7C4': {'symbol': 'PLAY', 'decimals': 18},
    '0x3A4CaB3DcfaB144FE7eB2B5a3e288Cc03dC07659': {'symbol': 'FTG', 'decimals': 18},
    '0x675ABaBD3A210566A5e213547523b740Be80041A': {'symbol': 'ENT', 'decimals': 18},
    '0x7d4b8Cce0591C9044a22ee543533b72E976E36C3': {'symbol': 'CAG', 'decimals': 18},
    '0xE61F6e39711cEc14f8D6c637c2f4568bAA9FF7Ee': {'symbol': 'HEY', 'decimals': 18},
    '0x9d409a0A012CFbA9B15F6D4B36Ac57A46966Ab9a': {'symbol': 'YVBOOST', 'decimals': 18},
    '0x2e516BA5Bf3b7eE47fb99B09eaDb60BDE80a82e0': {'symbol': 'EGGS', 'decimals': 18},
    '0x309627af60F0926daa6041B8279484312f2bf060': {'symbol': 'USDB', 'decimals': 18},
    '0x42476F744292107e34519F9c357927074Ea3F75D': {'symbol': 'LOOM', 'decimals': 18},
    '0x9AF4f26941677C706cfEcf6D3379FF01bB85D5Ab': {'symbol': 'DRT', 'decimals': 8},
    '0x7a3f7F6675514D4d611b442A4B76752f6AB77670': {'symbol': 'TORA', 'decimals': 18},
    '0xf16e81dce15B08F326220742020379B855B87DF9': {'symbol': 'ICE', 'decimals': 18},
    '0x77A9B858aA8417fd318ecEE01C1d6D9fEF31A71F': {'symbol': 'NET', 'decimals': 18},
    '0xE4CFE9eAa8Cdb0942A80B7bC68fD8Ab0F6D44903': {'symbol': 'XEND', 'decimals': 18},
    '0x255Aa6DF07540Cb5d3d297f0D0D4D84cb52bc8e6': {'symbol': 'RDN', 'decimals': 18},
    '0x36F8d0D0573ae92326827C4a82Fe4CE4C244cAb6': {'symbol': 'MADAI', 'decimals': 18},
    '0xc4C75F2A0cB1a9ACc33929512dc9733EA1Fd6fde': {'symbol': 'MSI', 'decimals': 18},
    '0x420412E765BFa6d85aaaC94b4f7b708C89be2e2B': {'symbol': 'BRZ', 'decimals': 4},
    '0xd73Cab6a16c67Cb79fc61F8c13DfEc4F32f9DCBF': {'symbol': 'INUSANITY', 'decimals': 9},
    '0x8D1ce361eb68e9E05573443C407D4A3Bed23B033': {'symbol': 'DEFI++', 'decimals': 18},
    '0x3D371413dd5489F3A04C07c0C2CE369c20986ceb': {'symbol': 'YOUC', 'decimals': 10},
    '0x443b29fc978058Abe3Fc2f4c3c6b76c57fdecC02': {'symbol': 'IDEAS', 'decimals': 18},
    '0xE1005BfBBC9A17d5d844C7a4371CBF6B2B387380': {'symbol': 'G', 'decimals': 8},
    '0xe304283C3e60Cefaf7eA514007Cf4E8fdC3d869d': {'symbol': 'GEC', 'decimals': 18},
    '0xf941D3AAbf2EE0F5589E68Ba6047b8329592B366': {'symbol': 'HEEL', 'decimals': 9},
    '0xE6F1966d04CfcB9cd1B1dC4E8256D8b501b11CbA': {'symbol': 'SAFEEARTH', 'decimals': 9},
    '0xAA19961b6B858D9F18a115f25aa1D98ABc1fdBA8': {'symbol': 'LCS', 'decimals': 18},
    '0x1a6921F7B92113c79306f1A9E148D83445ac42BC': {'symbol': 'ROCK', 'decimals': 9},
    '0xdB2F2bCCe3efa95EDA95a233aF45F3e0d4f00e2A': {'symbol': 'AGS', 'decimals': 8},
    '0x10f44a834097469AC340592d28c479c442E99bFe': {'symbol': 'ALCAZAR', 'decimals': 18},
    '0xAE36155a55F04a696b8362777620027882B31dB5': {'symbol': 'KISHIMOTO', 'decimals': 9},
    '0x00fC270C9cc13e878Ab5363D00354bebF6f05C15': {'symbol': 'VNXLU', 'decimals': 18},
    '0x9783B81438C24848f85848f8df31845097341771': {'symbol': 'COLLAR', 'decimals': 18},
    '0xA00425D3e2D3E9FF74F3e112B4D3A7978d7D88c2': {'symbol': 'GPT', 'decimals': 18},
    '0x6Fb0855c404E09c47C3fBCA25f08d4E41f9F062f': {'symbol': 'AZRX', 'decimals': 18},
    '0xf2CDF38e24738bA379ffa38D47bc88a941dF5627': {'symbol': 'ALY', 'decimals': 2},
    '0xcA3Ea3061d638E02113aA960340c98343b5aCd62': {'symbol': 'BWT', 'decimals': 18},
    '0xf8b358b3397a8ea5464f8cc753645d42e14b79EA': {'symbol': 'ABL', 'decimals': 18},
    '0x286BDA1413a2Df81731D4930ce2F862a35A609fE': {'symbol': 'WABI', 'decimals': 18},
    '0xd049206fB408a611E543791F2d8F102a8bC253dc': {'symbol': 'NAO', 'decimals': 18},
    '0xF9C12B27cE5058ab98ce11BD53900f84E18C0650': {'symbol': 'Y2B', 'decimals': 18},
    '0x115eC79F1de567eC68B7AE7eDA501b406626478e': {'symbol': 'CRE', 'decimals': 18},
    '0x0B498ff89709d3838a063f1dFA463091F9801c2b': {'symbol': 'BTC2X-FLI', 'decimals': 18},
    '0xf9Fb4AD91812b704Ba883B11d2B576E890a6730A': {'symbol': 'AAMMWETH', 'decimals': 18},
    '0xA92cDf011c895780733b2a6A6e15C4c1ed836148': {'symbol': 'DFCH', 'decimals': 18},
    '0xd433138d12beB9929FF6fd583DC83663eea6Aaa5': {'symbol': 'BTR', 'decimals': 18},
    '0xcd17fA52528f37FACB3028688E62ec82d9417581': {'symbol': 'MTRM', 'decimals': 0},
    '0x761D38e5ddf6ccf6Cf7c55759d5210750B5D60F3': {'symbol': 'ELON', 'decimals': 18},
    '0x7d8DafF6d70CEAd12c6f077048552Cf89130A2B1': {'symbol': 'ARX', 'decimals': 18},
    '0x0AF44e2784637218dD1D32A322D44e603A8f0c6A': {'symbol': 'MTX', 'decimals': 18},
    '0x93b1E78a3e652cd2e71C4a767595B77282344932': {'symbol': 'BITO', 'decimals': 18},
    '0xff8C479134A18918059493243943150776cF8CF2': {'symbol': 'RENQ', 'decimals': 18},
    '0xe1A4c5BBb704a92599FEdB191f451E0d3a1ed842': {'symbol': 'NAME', 'decimals': 18},
    '0x5b11aAcB6Bddb9ffab908FDCE739Bf4aed554327': {'symbol': 'TDP', 'decimals': 18},
    '0x5833dBB0749887174b254bA4a5df747FF523a905': {'symbol': 'XRUN', 'decimals': 18},
    '0x0763fdCCF1aE541A5961815C0872A8c5Bc6DE4d7': {'symbol': 'SUKU', 'decimals': 18},
    '0xf418588522d5dd018b425E472991E52EBBeEEEEE': {'symbol': 'PUSH', 'decimals': 18},
    '0x3c8D2FCE49906e11e71cB16Fa0fFeB2B16C29638': {'symbol': 'NFTL', 'decimals': 18},
    '0xdA30f261a962d5AAe94C9ecd170544600d193766': {'symbol': 'ORBR', 'decimals': 18},
    '0x47140a767A861F7a1f3B0Dd22A2F463421c28814': {'symbol': 'ME', 'decimals': 18},
    '0x0f8c45B896784A1E408526B9300519ef8660209c': {'symbol': 'XMX', 'decimals': 8},
    '0x2E95Cea14dd384429EB3c4331B776c4CFBB6FCD9': {'symbol': 'THN', 'decimals': 18},
    '0x77700005BEA4DE0A78b956517f099260C2CA9a26': {'symbol': 'ROOT', 'decimals': 18},
    '0x3a1Bda28AdB5B0a812a7CF10A1950c920F79BcD3': {'symbol': 'FLP', 'decimals': 18},
    '0xc4d5545392f5Fc57EBa3AF8981815669bb7E2A48': {'symbol': 'HDP', 'decimals': 4},
    '0x374fB05c96c36348b92E38fB088b26B8511e3b3D': {'symbol': 'CWD', 'decimals': 18},
    '0x320623b8E4fF03373931769A31Fc52A4E78B5d70': {'symbol': 'RSR', 'decimals': 18},
    '0x298d492e8c1d909D3F63Bc4A36C66c64ACB3d695': {'symbol': 'PBR', 'decimals': 18},
    '0x4236f8aaF2B1f3A28420Eb15B8e0ddF63201a95E': {'symbol': 'BMDA', 'decimals': 18},
    '0xd794DD1CAda4cf79C9EebaAb8327a1B0507ef7d4': {'symbol': 'HYVE', 'decimals': 18},
    '0x37E1160184F7dD29f00b78C050Bf13224780b0B0': {'symbol': 'YCC', 'decimals': 8},
    '0x147faF8De9d8D8DAAE129B187F0D02D819126750': {'symbol': 'GEO', 'decimals': 18},
    '0x9C2dc0c3CC2BADdE84B0025Cf4df1c5aF288D835': {'symbol': 'COR', 'decimals': 18},
    '0x6595b8fD9C920C81500dCa94e53Cdc712513Fb1f': {'symbol': 'OLY', 'decimals': 18},
    '0x26E43759551333e57F073bb0772F50329A957b30': {'symbol': 'DGVC', 'decimals': 18},
    '0x722f97A435278B7383a1e3c47F41773bebF3232C': {'symbol': 'UCM', 'decimals': 18},
    '0x5955f7d312c5d84B004d259D55072C4F0A478dBC': {'symbol': 'BALTO', 'decimals': 18},
    '0x5592C5Aa89492Ea918d55B804E177B5Ca7dCcd5a': {'symbol': 'WXBTC', 'decimals': 18},
    '0x96184d9C811Ea0624fC30C80233B1d749B9E485B': {'symbol': 'DAPPT', 'decimals': 18},
    '0xCeB53519a2B891d33b36E3826962D2a48f878e45': {'symbol': 'HRTS', 'decimals': 18},
    '0x1571eD0bed4D987fe2b498DdBaE7DFA19519F651': {'symbol': 'IFARM', 'decimals': 18},
    '0x286F851b049CcCE1419e09b6468dC3297F86A703': {'symbol': 'HISEALS', 'decimals': 18},
    '0x35073CA7bAC0c2eDf58fb34A2CBC39Ea2fbB7056': {'symbol': 'TOR', 'decimals': 18},
    '0x8eb5bD8c9Ab0F8ad28e94693F3c889F490bE2aB0': {'symbol': 'POS32', 'decimals': 18},
    '0x56c9D5F1E727de03643af220B5cE52DE23D4D973': {'symbol': 'DBOX', 'decimals': 18},
    '0xc944E90C64B2c07662A292be6244BDf05Cda44a7': {'symbol': 'GRT', 'decimals': 18},
    '0xD4c64f0Fe38ecdaF0Baefd7859E18185a73b9aa3': {'symbol': 'UZUMAKI', 'decimals': 9},
    '0xA888D9616C2222788fa19f05F77221A290eEf704': {'symbol': 'DARUMA', 'decimals': 9},
    '0x6d4CA1177087924edfE0908ef655169EA766FDc3': {'symbol': 'HEDGEHOG', 'decimals': 18},
    '0xba100000625a3754423978a60c9317c58a424e3D': {'symbol': 'BAL', 'decimals': 18},
    '0xB14eBF566511B9e6002bB286016AB2497B9b9c9D': {'symbol': 'HID', 'decimals': 18},
    '0x0c90C57aaf95A3A87eadda6ec3974c99D786511F': {'symbol': 'HAN', 'decimals': 18},
    '0xc6956b78e036b87Ba2aB9810BF081eb76EEDd17a': {'symbol': 'H', 'decimals': 18},
    '0x6692De64716a177c15360D8d010BC522bBc530a0': {'symbol': 'TNT', 'decimals': 18},
    '0x232FB065D9d24c34708eeDbF03724f2e95ABE768': {'symbol': 'SHEESHA', 'decimals': 18},
    '0x39AA39c021dfbaE8faC545936693aC917d5E7563': {'symbol': 'CUSDC', 'decimals': 8},
    '0xFa14Fa6958401314851A17d6C5360cA29f74B57B': {'symbol': 'SAITO', 'decimals': 18},
    '0xe59d2FF6995a926A574390824a657eEd36801E55': {'symbol': 'AAMMUNIAAVEWETH', 'decimals': 18},
    '0x345DADb10A200F10814Ad8523fCA0f2d958C3370': {'symbol': 'KORI', 'decimals': 9},
    '0x6c6EE5e31d828De241282B9606C8e98Ea48526E2': {'symbol': 'HOT', 'decimals': 18},
    '0xF9e5aF7B42D31D51677c75bbBD37c1986eC79AEE': {'symbol': 'QCX', 'decimals': 8},
    '0x8d137e3337eb1B58A222Fef2B2Cc7C423903d9cf': {'symbol': 'SQGL', 'decimals': 18},
    '0x22143217662334F073652Da5E2F1e0AE7349240c': {'symbol': 'CWT', 'decimals': 18},
    '0x670f9D9a26D3D42030794ff035d35a67AA092ead': {'symbol': 'GOLD', 'decimals': 8},
    '0x46d473a0B3eEEc9F55FADE641bC576d5bc0b2246': {'symbol': 'SURF', 'decimals': 8},
    '0x22B6C31c2bEB8f2d0d5373146Eed41Ab9eDe3caf': {'symbol': 'COC', 'decimals': 8},
    '0x63f88A2298a5c4AEE3c216Aa6D926B184a4b2437': {'symbol': 'GAME', 'decimals': 18},
    '0x34D6A0F5C2f5D0082141fE73d93B9dd00ca7CE11': {'symbol': 'GOLD', 'decimals': 18},
    '0x7CA5af5bA3472AF6049F63c1AbC324475D44EFC1': {'symbol': 'KNDX', 'decimals': 9},
    '0x5c59a5B139B0538CB106D775a022caD98Dd14b5a': {'symbol': 'ORT', 'decimals': 18},
    '0x3D382228C54736d831FAC2748F4734D9177c7332': {'symbol': 'ANV', 'decimals': 18},
    '0x1500205f50bf3FD976466d0662905c9ff254fc9c': {'symbol': 'BBT', 'decimals': 4},
    '0x7fC3eC3574d408F3b59CD88709baCb42575EBF2b': {'symbol': 'POP', 'decimals': 18},
    '0x1BE56412c9606E7285280F76A105EBa56996e491': {'symbol': 'Z', 'decimals': 9},
    '0xd6cAF5Bd23CF057f5FcCCE295Dcc50C01C198707': {'symbol': 'EVA', 'decimals': 18},
    '0xE8663A64A96169ff4d95b4299E7ae9a76b905B31': {'symbol': 'RATING', 'decimals': 8},
    '0x7495e5Cc8F27e0bd5bD4cb86d17F0D841cA58EE4': {'symbol': 'ARNC', 'decimals': 18},
    '0xc719d010B63E5bbF2C0551872CD5316ED26AcD83': {'symbol': 'DIP', 'decimals': 18},
    '0xe6fd75ff38Adca4B97FBCD938c86b98772431867': {'symbol': 'ELA', 'decimals': 18},
    '0x30f271C9E86D2B7d00a6376Cd96A1cFBD5F0b9b3': {'symbol': 'DEC', 'decimals': 18},
    '0x73EE6D7e6b203125aDd89320E9F343d65ec7c39a': {'symbol': 'AXI', 'decimals': 18},
    '0x7c8155909cd385F120A56eF90728dD50F9CcbE52': {'symbol': 'NII', 'decimals': 15},
    '0x2a9bDCFF37aB68B95A53435ADFd8892e86084F93': {'symbol': 'AQT', 'decimals': 18},
    '0xd137561C20BE8e870D62e2aE92269f0950702f77': {'symbol': 'EXP', 'decimals': 18},
    '0xe7aE6D0C56CACaf007b7e4d312f9af686a9E9a04': {'symbol': 'VAB', 'decimals': 18},
    '0x2c022e58C5E3Ee213F06f975fd8a0d3a6fE9Ca1C': {'symbol': 'FINU', 'decimals': 18},
    '0xEaDD9B69F96140283F9fF75DA5FD33bcF54E6296': {'symbol': 'CYCE', 'decimals': 6},
    '0x98585dFc8d9e7D48F0b1aE47ce33332CF4237D96': {'symbol': 'NEWO', 'decimals': 18},
    '0x848578e351D25B6Ec0d486E42677891521c3d743': {'symbol': 'MOSOLID', 'decimals': 18},
    '0x2730d6FdC86C95a74253BefFaA8306B40feDecbb': {'symbol': 'UNI', 'decimals': 8},
    '0xA13f0743951B4f6E3e3AA039f682E17279f52bc3': {'symbol': 'SENC', 'decimals': 18},
    '0x082646B22A3960DA69eF7A778c16dd6fb85Dd999': {'symbol': 'FKPEPE', 'decimals': 18},
    '0xA87Ed75C257f1ec38393bEA0A83d55Ac2279D79c': {'symbol': 'EARN', 'decimals': 9},
    '0xC1f33e0cf7e40a67375007104B929E49a581bafE': {'symbol': 'SPOT', 'decimals': 9},
    '0x96c645D3D3706f793Ef52C19bBACe441900eD47D': {'symbol': 'MPS', 'decimals': 0},
    '0x5C0590cDE44569bf39ef79e859B367e39cB000F1': {'symbol': 'HIVALHALLA', 'decimals': 18},
    '0x2c9acEb63181cd08a093d052ec041e191f229692': {'symbol': 'ANB', 'decimals': 18},
    '0x8E6cd950Ad6ba651F6DD608Dc70e5886B1AA6B24': {'symbol': 'STARL', 'decimals': 18},
    '0x295B42684F90c77DA7ea46336001010F2791Ec8c': {'symbol': 'XI', 'decimals': 18},
    '0x3ebb4A4e91Ad83BE51F8d596533818b246F4bEe1': {'symbol': 'SATA', 'decimals': 18},
    '0x63f584FA56E60e4D0fE8802b27C7e6E3b33E007f': {'symbol': 'BOX', 'decimals': 18},
    '0x9Ce84F6A69986a83d92C324df10bC8E64771030f': {'symbol': 'CHEX', 'decimals': 18},
    '0x8f693ca8D21b157107184d29D398A8D082b38b76': {'symbol': 'DATA', 'decimals': 18},
    '0xEc505C81D6a7567b5BDE804870b1038832fE6Da1': {'symbol': 'CND', 'decimals': 18},
    '0xc66cDac744916AFb6811C71c277d88dE90cE8D5b': {'symbol': 'MCD', 'decimals': 18},
    '0x4f640F2529ee0cF119A2881485845FA8e61A782A': {'symbol': 'ORE', 'decimals': 18},
    '0x5fCe9Fc9B5d62aF082A59D0823A062F7529eFA5a': {'symbol': 'SHAMAN', 'decimals': 18},
    '0x159751323A9E0415DD3d6D42a1212fe9F4a0848C': {'symbol': 'INFI', 'decimals': 18},
    '0x2C2f7e7C5604D162d75641256b80F1Bf6f4dC796': {'symbol': 'PRARE', 'decimals': 18},
    '0x06A01a4d579479Dd5D884EBf61A31727A3d8D442': {'symbol': 'SKEY', 'decimals': 8},
    '0x329c6E459FFa7475718838145e5e85802Db2a303': {'symbol': 'EMAID', 'decimals': 18},
    '0x1BBf25e71EC48B84d773809B4bA55B6F4bE946Fb': {'symbol': 'VOW', 'decimals': 18},
    '0x78132543D8E20D2417d8a07D9ae199D458A0D581': {'symbol': 'LINU', 'decimals': 18},
    '0xCd1fAFf6e578Fa5cAC469d2418C95671bA1a62Fe': {'symbol': 'XTM', 'decimals': 18},
    '0x9d561d63375672ABd02119b9Bc4FB90EB9E307Ca': {'symbol': 'DRCT', 'decimals': 18},
    '0x981dc247745800bD2cA28a4bF147f0385Eaa0bc0': {'symbol': 'NUTS', 'decimals': 18},
    '0xFca59Cd816aB1eaD66534D82bc21E7515cE441CF': {'symbol': 'RARI', 'decimals': 18},
    '0x9681ee0D91E737C3B60aCEbA7Fbdae61B5462f42': {'symbol': 'CYCE', 'decimals': 6},
    '0x6781a0F84c7E9e846DCb84A9a5bd49333067b104': {'symbol': 'ZAP', 'decimals': 18},
    '0xd7f0cC50AD69408aE58be033F4f85D2367C2e468': {'symbol': 'VERA', 'decimals': 18},
    '0x8530b66ca3DDf50E0447eae8aD7eA7d5e62762eD': {'symbol': 'METADOGE', 'decimals': 18},
    '0x5919DeA604631016c15c805e3D948A0384879892': {'symbol': 'PEEP', 'decimals': 9},
    '0x1b980e05943dE3dB3a459C72325338d327B6F5a9': {'symbol': 'GEAR', 'decimals': 18},
    '0x669c01CAF0eDcaD7c2b8Dc771474aD937A7CA4AF': {'symbol': 'WMINIMA', 'decimals': 18},
    '0x4332f8A38f14BD3D8D1553aF27D7c7Ac6C27278D': {'symbol': 'APEFI', 'decimals': 18},
    '0x5c872500c00565505F3624AB435c222E558E9ff8': {'symbol': 'COT', 'decimals': 18},
    '0x4561de8e0C2bbA725d38D266EF62426e62678d82': {'symbol': 'CONI', 'decimals': 18},
    '0x4Bc3263Eb5bb2Ef7Ad9aB6FB68be80E43b43801F': {'symbol': 'VETH', 'decimals': 18},
    '0xB83c27805aAcA5C7082eB45C868d955Cf04C337F': {'symbol': 'TIGER', 'decimals': 18},
    '0x2F9411088cEF82Fd9fB904Eb8092f28eB485C8F6': {'symbol': 'ATH', 'decimals': 18},
    '0x7102DC82EF61bfB0410B1b1bF8EA74575bf0A105': {'symbol': 'X7102', 'decimals': 18},
    '0x87869A9789291A6cEC99f3c3Ef2fF71fcEb12a8e': {'symbol': 'CMOS', 'decimals': 9},
    '0xa43a8FC632d809FB2c16f590c0ff6423e311c67D': {'symbol': 'MODS', 'decimals': 18},
    '0x1CC481cE2BD2EC7Bf67d1Be64d4878b16078F309': {'symbol': 'IBCHF', 'decimals': 18},
    '0x3D658390460295FB963f54dC0899cfb1c30776Df': {'symbol': 'COVAL', 'decimals': 8},
    '0x0f51bb10119727a7e5eA3538074fb341F56B09Ad': {'symbol': 'DAO', 'decimals': 18},
    '0xE5097D9baeAFB89f9bcB78C9290d545dB5f9e9CB': {'symbol': 'HBOT', 'decimals': 18},
    '0x471d113059324321749E097705197a2B44A070FC': {'symbol': 'KNG', 'decimals': 18},
    '0xF25c91C87e0B1fd9B4064Af0F427157AaB0193A7': {'symbol': 'BASIC', 'decimals': 18},
    '0xe28f0982eEeD9cCf816d8496fb742D438056d275': {'symbol': 'PKT', 'decimals': 18},
    '0xE23311294467654E0CaB14cD32A169A41be5ca8E': {'symbol': 'CRNO', 'decimals': 18},
    '0x8e57c27761EBBd381b0f9d09Bb92CeB51a358AbB': {'symbol': 'XDNA', 'decimals': 18},
    '0x4FaDC7A98f2Dc96510e42dD1A74141eEae0C1543': {'symbol': 'WAR', 'decimals': 12},
    '0xC0Eb85285d83217CD7c891702bcbC0FC401E2D9D': {'symbol': 'HVN', 'decimals': 8},
    '0x0C6Bf55F60143Fa1dA202AC8c9b8Ce484b5a5d90': {'symbol': 'ODA', 'decimals': 18},
    '0xA89ac6e529aCf391CfbBD377F3aC9D93eae9664e': {'symbol': 'KP4R', 'decimals': 18},
    '0x28dee01D53FED0Edf5f6E310BF8Ef9311513Ae40': {'symbol': 'XBP', 'decimals': 18},
    '0xf8e386EDa857484f5a12e4B5DAa9984E06E73705': {'symbol': 'IND', 'decimals': 18},
    '0x21d5678A62DFe63a47062469Ebb2fAc2817D8832': {'symbol': 'YLC', 'decimals': 8},
    '0x3D26dcd840fCC8e4B2193AcE8A092e4a65832F9f': {'symbol': 'AAMMUNIUNIWETH', 'decimals': 18},
    '0xD64809f5F7d772D9112A6BD379De00A77188199E': {'symbol': 'LSILVER', 'decimals': 18},
    '0xdB792B1D8869A7CFc34916d6c845Ff05A7C9b789': {'symbol': 'CCBCH', 'decimals': 8},
    '0x9b110Fda4E20DB18Ad7052f8468a455de7449eb6': {'symbol': 'RIA', 'decimals': 18},
    '0xa249DE6948022783765Fee4850d7b85E43118FCc': {'symbol': 'JAR', 'decimals': 18},
    '0xF0d33BeDa4d734C72684b5f9abBEbf715D0a7935': {'symbol': 'NTX', 'decimals': 6},
    '0x171d76d931529384639BC9aad5b77b77041ED604': {'symbol': 'MOTG', 'decimals': 18},
    '0x3C4B6E6e1eA3D4863700D7F76b36B7f3D3f13E3d': {'symbol': 'VGX', 'decimals': 8},
    '0xDe5eA375FFBDc8b25a80fe13D631E8Ba0AB4BB02': {'symbol': 'GERA', 'decimals': 18},
    '0xC146B7CdBaff065090077151d391f4c96Aa09e0C': {'symbol': 'MCC', 'decimals': 9},
    '0xbA6B0dbb2bA8dAA8F5D6817946393Aef8D3A4487': {'symbol': 'HSF', 'decimals': 18},
    '0x98E0438d3eE1404FEA48E38e92853BB08Cfa68bD': {'symbol': 'TVT', 'decimals': 8},
    '0xa9f9aCB92E4E2f16410511D56839A5Bd1d630a60': {'symbol': 'BLOCK-E', 'decimals': 9},
    '0x0e8D2EB7D6bDF28393c25a1966385Ad32Ff0259a': {'symbol': 'STREAMERINU', 'decimals': 18},
    '0x191557728e4d8CAa4Ac94f86af842148c0FA8F7E': {'symbol': 'ECO', 'decimals': 8},
    '0x30df7D7EE52c1b03cd009e656F00AB875AdCEeD2': {'symbol': 'RESET', 'decimals': 18},
    '0xa3C22370de5f9544f0c4De126b1e46cEadF0A51B': {'symbol': 'WSTRAX', 'decimals': 18},
    '0xEDc1004886d010751f74ec0Ad223819F9F3b1910': {'symbol': 'AGN', 'decimals': 18},
    '0x1bF7Fd22709733cCD7c45AB27Dd02C7EC8E50078': {'symbol': 'QTCON', 'decimals': 18},
    '0xc3Eb2622190c57429aac3901808994443b64B466': {'symbol': 'ORO', 'decimals': 18},
    '0x492baa7A6450712D4bbCCa01B87F029DEe3Ea3Ec': {'symbol': 'WAGIE', 'decimals': 9},
    '0x054f76beED60AB6dBEb23502178C52d6C5dEbE40': {'symbol': 'FIN', 'decimals': 18},
    '0x9b31bb425D8263fA1b8B9d090b83CF0C31665355': {'symbol': 'CPD', 'decimals': 18},
    '0x00166a5b93FD4f87ecA1c267d31D82dF3f133a7F': {'symbol': 'ICB', 'decimals': 18},
    '0x7deB5e830be29F91E298ba5FF1356BB7f8146998': {'symbol': 'AMKR', 'decimals': 18},
    '0xa80f2C8f61c56546001f5FC2eb8D6E4e72c45d4C': {'symbol': 'UNQT', 'decimals': 18},
    '0x8C543AED163909142695f2d2aCd0D55791a9Edb9': {'symbol': 'VLX', 'decimals': 18},
    '0x74B988156925937bD4E082f0eD7429Da8eAea8Db': {'symbol': 'MEME', 'decimals': 18},
    '0x294559FA758C88D639Fd085751e463FEe7806eab': {'symbol': 'METAL', 'decimals': 18},
    '0xB2a63A5Dd36C91eC2DA59B188ff047F66fac122a': {'symbol': 'FOLO', 'decimals': 18},
    '0x6c3BE406174349cfa4501654313d97e6a31072e1': {'symbol': 'CNNS', 'decimals': 18},
    '0x7eaF9C89037e4814DC0d9952Ac7F888C784548DB': {'symbol': 'ROYA', 'decimals': 18},
    '0x6afCFF9189e8ed3fCc1CFfa184FEB1276f6A82A5': {'symbol': 'PETS', 'decimals': 18},
    '0x15B543e986b8c34074DFc9901136d9355a537e7E': {'symbol': 'STC', 'decimals': 18},
    '0x9d7630aDF7ab0b0CB00Af747Db76864df0EC82E4': {'symbol': 'GATE', 'decimals': 18},
    '0x9F9913853f749b3fE6D6D4e16a1Cc3C1656B6D51': {'symbol': 'BITT', 'decimals': 18},
    '0x9F009D03E1b7F02065017C90e8e0D5Cb378eB015': {'symbol': 'FORTUNE', 'decimals': 9},
    '0x2620638EDA99F9e7E902Ea24a285456EE9438861': {'symbol': 'CSM', 'decimals': 18},
    '0xB893A8049f250b57eFA8C62D51527a22404D7c9A': {'symbol': 'USHIBA', 'decimals': 9},
    '0x513C3200F227ebB62e3B3d00B7a83779643a71CF': {'symbol': 'LIFT', 'decimals': 18},
    '0x1e4E46b7BF03ECE908c88FF7cC4975560010893A': {'symbol': 'IOEN', 'decimals': 18},
    '0x72e9D9038cE484EE986FEa183f8d8Df93f9aDA13': {'symbol': 'SMARTCREDIT', 'decimals': 18},
    '0xDf49C9f599A0A9049D97CFF34D0C30E468987389': {'symbol': 'SATT', 'decimals': 18},
    '0xCc4450EDEfbBEb450E2f205f4ECEfF7232D54e92': {'symbol': 'TGL', 'decimals': 18},
    '0xEF19F4E48830093Ce5bC8b3Ff7f903A0AE3E9Fa1': {'symbol': 'BOTX', 'decimals': 18},
    '0xA2085073878152aC3090eA13D1e41bD69e60Dc99': {'symbol': 'ELG', 'decimals': 18},
    '0x97AbEe33Cd075c58BFdd174e0885e08E8f03556F': {'symbol': 'SENT', 'decimals': 18},
    '0x1788430620960F9a70e3DC14202a3A35ddE1A316': {'symbol': 'OAP', 'decimals': 18},
    '0x547b2F82ceCfAb9C2B1D36fddA96eF9F58C63B8C': {'symbol': 'TXT', 'decimals': 18},
    '0x6B0b3a982b4634aC68dD83a4DBF02311cE324181': {'symbol': 'ALI', 'decimals': 18},
    '0xB197a6Fdd8AB825332eDEbA1AB5C4D1Bf97fb9A8': {'symbol': 'SHIBDAO', 'decimals': 18},
    '0x5d30aD9C6374Bf925D0A75454fa327AACf778492': {'symbol': 'PERI', 'decimals': 18},
    '0x28Cca76f6e8eC81e4550ecd761f899110b060E97': {'symbol': 'ARGO', 'decimals': 18},
    '0xAf91E8aFbe87642dC628786188a54B78580A4d76': {'symbol': 'FOY', 'decimals': 18},
    '0xf7413489c474ca4399eeE604716c72879Eea3615': {'symbol': 'APYS', 'decimals': 18},
    '0x817bbDbC3e8A1204f3691d14bB44992841e3dB35': {'symbol': 'CUDOS', 'decimals': 18},
    '0x2C9023bBc572ff8dc1228c7858A280046Ea8C9E5': {'symbol': 'VID', 'decimals': 18},
    '0x9D86b1B2554ec410ecCFfBf111A6994910111340': {'symbol': 'OPEN', 'decimals': 8},
    '0xa0ED3C520dC0632657AD2EaaF19E26C4fD431a84': {'symbol': 'HPO', 'decimals': 18},
    '0x6f8b23296394d20eC048FbDec8eBc0CA90f5c8f1': {'symbol': 'TUF', 'decimals': 18},
    '0xF5581dFeFD8Fb0e4aeC526bE659CFaB1f8c781dA': {'symbol': 'HOPR', 'decimals': 18},
    '0xD0D6D6C5Fe4a677D343cC433536BB717bAe167dD': {'symbol': 'ADT', 'decimals': 9},
    '0xD75f1f81B69bDd4Df8EfbB70e9C6F4609009D753': {'symbol': 'YASHA', 'decimals': 18},
    '0x674c964AC0E89d847d6B0aBd144b797bf78BA56b': {'symbol': 'GWD', 'decimals': 6},
    '0x4CD0c43B0D53bc318cc5342b77EB6f124E47f526': {'symbol': 'FREE', 'decimals': 18},
    '0x86B4dBE5D203e634a12364C0e428fa242A3FbA98': {'symbol': 'GBPT', 'decimals': 18},
    '0xEd40834A13129509A89be39a9bE9C0E96A0DDd71': {'symbol': 'WARP', 'decimals': 18},
    '0x12f649A9E821F90BB143089a6e56846945892ffB': {'symbol': 'UDOO', 'decimals': 18},
    '0x92830ef7c8d651Ed3A708053c602E807bAd7db22': {'symbol': 'BLUSGD', 'decimals': 18},
    '0x976bf616F61685e0E7E36419Aee3f7Cde24B0Dc9': {'symbol': 'SYRANT', 'decimals': 18},
    '0xE796d6ca1ceb1b022EcE5296226BF784110031Cd': {'symbol': 'BLES', 'decimals': 18},
    '0x2ccbFF3A042c68716Ed2a2Cb0c544A9f1d1935E1': {'symbol': 'DMT', 'decimals': 8},
    '0x9F195617fA8fbAD9540C5D113A99A0a0172aaEDC': {'symbol': 'NBC', 'decimals': 18},
    '0xd6929179D752d5D25c5efe2d9729eb77D7138A80': {'symbol': 'GOB', 'decimals': 18},
    '0xCfa0885131F602D11D4DA248D2c65A62063567A9': {'symbol': 'TORG', 'decimals': 18},
    '0x94E9EB8b5Ab9fd6B9ea3169D55FFAde62a01702e': {'symbol': 'BREED', 'decimals': 18},
    '0xf59257E961883636290411c11ec5Ae622d19455e': {'symbol': 'FLOOR', 'decimals': 9},
    '0x2D5C73f3597B07F23C2bB3F2422932E67eca4543': {'symbol': 'IMP', 'decimals': 18},
    '0x9Adc7710E9d1b29d8a78c04d52D32532297C2Ef3': {'symbol': 'QDT', 'decimals': 18},
    '0x584bC13c7D411c00c01A62e8019472dE68768430': {'symbol': 'HEGIC', 'decimals': 18},
    '0x6369c3DadfC00054A42BA8B2c09c48131dd4Aa38': {'symbol': 'MPH', 'decimals': 18},
    '0xe78BFF28D3c5195C752E734E95421bb4bd742f1d': {'symbol': 'NINEFI', 'decimals': 18},
    '0xe76C6c83af64e4C60245D8C7dE953DF673a7A33D': {'symbol': 'RAIL', 'decimals': 18},
    '0x03Be5C903c727Ee2C8C4e9bc0AcC860Cca4715e2': {'symbol': 'CAPS', 'decimals': 18},
    '0x6123B0049F904d730dB3C36a31167D9d4121fA6B': {'symbol': 'RBN', 'decimals': 18},
    '0x4B5f49487ea7B3609b1aD05459BE420548789f1f': {'symbol': 'LEVER', 'decimals': 18},
    '0xbc194e6f748a222754C3E8b9946922c09E7d4e91': {'symbol': 'LEV', 'decimals': 18},
    '0xF5B5eFc906513b4344EbAbCF47A04901f99F09f3': {'symbol': 'UBX', 'decimals': 0},
    '0x630d98424eFe0Ea27fB1b3Ab7741907DFFEaAd78': {'symbol': 'PEAK', 'decimals': 8},
    '0xdaC657ffD44a3B9d8aba8749830Bf14BEB66fF2D': {'symbol': 'HDAO', 'decimals': 18},
    '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C': {'symbol': 'BNT', 'decimals': 18},
    '0xF5cFBC74057C610c8EF151A439252680AC68c6DC': {'symbol': 'OCT', 'decimals': 18},
    '0xFBEB78a723b8087fD2ea7Ef1afEc93d35E8Bed42': {'symbol': 'YVUNI', 'decimals': 18},
    '0xDC98c5543F3004DEBfaad8966ec403093D0aa4A8': {'symbol': 'PEBBLE', 'decimals': 18},
    '0x7659CE147D0e714454073a5dd7003544234b6Aa0': {'symbol': 'XCAD', 'decimals': 18},
    '0x1d37986F252d0e349522EA6C3B98Cb935495E63E': {'symbol': 'CHART', 'decimals': 18},
    '0x1B8568FbB47708E9E9D31Ff303254f748805bF21': {'symbol': 'SCX', 'decimals': 18},
    '0x0913dDAE242839f8995c0375493f9a1A3Bddc977': {'symbol': 'MRI', 'decimals': 18},
    '0x00E679Ba63B509182c349F5614F0A07cDd0CE0c5': {'symbol': 'DAMEX', 'decimals': 18},
    '0xdBBE358Fead311b4089B8BF30F455c041E67f5f4': {'symbol': 'DED', 'decimals': 18},
    '0x54C890411e97755ca69321f28Bed31CAFBb3C5A0': {'symbol': 'STUB', 'decimals': 18},
    '0xA2A54f1Ec1f09316eF12c1770D32ed8F21B1Fb6A': {'symbol': 'DFT', 'decimals': 8},
    '0xaac41EC512808d64625576EDdd580e7Ea40ef8B2': {'symbol': 'GSWAP', 'decimals': 18},
    '0xE7AC8545E34771de3706598Abb3dB9a19af2A07F': {'symbol': 'MONKED', 'decimals': 8},
    '0x562E362876c8Aee4744FC2c6aaC8394C312d215d': {'symbol': 'OPTI', 'decimals': 9},
    '0x5A56Da75c50aA2733F5Fa9A2442AaEfcBc60B2e6': {'symbol': 'CXD', 'decimals': 18},
    '0x7C07F7aBe10CE8e33DC6C5aD68FE033085256A84': {'symbol': 'ICETH', 'decimals': 18},
    '0x746DdA2ea243400D5a63e0700F190aB79f06489e': {'symbol': 'BOA', 'decimals': 7},
    '0x00DD5Ad8F5d9c5118BcEeb526c55eA6b2C07e922': {'symbol': 'VFY', 'decimals': 18},
    '0x99ea4dB9EE77ACD40B119BD1dC4E33e1C070b80d': {'symbol': 'QSP', 'decimals': 18},
    '0x8a6D4C8735371EBAF8874fBd518b56Edd66024eB': {'symbol': 'BLOCKS', 'decimals': 18},
    '0x8A0cdfaB62eD35b836DC0633482798421C81b3Ec': {'symbol': 'SPHRI', 'decimals': 18},
    '0x5Cf04716BA20127F1E2297AdDCf4B5035000c9eb': {'symbol': 'NKN', 'decimals': 18},
    '0x07eF9E82721AC16809D24DAfBE1792Ce01654DB4': {'symbol': 'BNANA', 'decimals': 18},
    '0x97e6e31aFb2d93d437301e006D9DA714616766A5': {'symbol': 'MIDAS', 'decimals': 18},
    '0x5efcEA234f7547de4569aad1215FA5D2adACeD38': {'symbol': 'HONK', 'decimals': 18},
    '0x1De852CD18323Bc5eBDa842b8EEF46C6E551aa47': {'symbol': 'HIRENGA', 'decimals': 18},
    '0x146D8D942048ad517479C9bab1788712Af180Fde': {'symbol': 'MIB', 'decimals': 18},
    '0x3757232B55E60da4A8793183aC030CfCE4c3865d': {'symbol': 'YDR', 'decimals': 18},
    '0x25644eA5bE09d1266B5df3F96559Ecdc2D0ad69b': {'symbol': 'YSHORI', 'decimals': 18},
    '0x00AbA6fE5557De1a1d565658cBDdddf7C710a1eb': {'symbol': 'EZ', 'decimals': 18},
    '0x990E081A7B7d3Ccba26a2f49746A68CC4fF73280': {'symbol': 'KSC', 'decimals': 18},
    '0x89d3c0249307Ae570A316030764D12f53BB191FD': {'symbol': 'XGC', 'decimals': 14},
    '0x5BB29c33C4A3C29f56F8ACa40B4dB91d8a5fe2c5': {'symbol': 'ONS', 'decimals': 18},
    '0x68a9d92Fe19399FEEBEd6A9a0980a7ea7638074C': {'symbol': 'IQQ', 'decimals': 18},
    '0xDA7C0810cE6F8329786160bb3d1734cf6661CA6E': {'symbol': 'JAY', 'decimals': 18},
    '0x3Ba925fdeAe6B46d0BB4d424D829982Cb2F7309e': {'symbol': 'RBX', 'decimals': 18},
    '0xb0E99627bC29adEf1178f16117BF495351E81997': {'symbol': 'DXC', 'decimals': 18},
    '0xE35f19E4457A114A951781aaF421EC5266eF25Fe': {'symbol': 'VIDA', 'decimals': 18},
    '0x9b5161a41B58498Eb9c5FEBf89d60714089d2253': {'symbol': 'MF1', 'decimals': 18},
    '0xd084944d3c05CD115C09d072B9F44bA3E0E45921': {'symbol': 'FOLD', 'decimals': 18},
    '0xbC298DfAA2EDda095b924F1390cC38fB7c5f6250': {'symbol': 'NEZUKO', 'decimals': 9},
    '0x2F109021aFe75B949429fe30523Ee7C0D5B27207': {'symbol': 'OCC', 'decimals': 18},
    '0x9F284E1337A815fe77D2Ff4aE46544645B20c5ff': {'symbol': 'KTON', 'decimals': 18},
    '0xe84d9e32dC8cE819b8D6c83e50EDAfD46c6354dB': {'symbol': 'KOI', 'decimals': 18},
    '0x8F081Eb884Fd47b79536D28E2DD9d4886773F783': {'symbol': 'BECOIN', 'decimals': 6},
    '0x7533D63A2558965472398Ef473908e1320520AE2': {'symbol': 'INTX', 'decimals': 9},
    '0x8b937AF714aC7E2129bD33D93641F52b665Ca352': {'symbol': 'JIZZ', 'decimals': 18},
    '0x4dA34f8264CB33A5c9F17081B9EF5Ff6091116f4': {'symbol': 'ELFI', 'decimals': 18},
    '0xaC0968a3E2020aC8ca83e60CcF69081EBc6d3bC3': {'symbol': 'CIND', 'decimals': 18},
    '0xeF2e239b21A81719abAabC6239e2e90859B44D9B': {'symbol': 'TTX', 'decimals': 18},
    '0x9b68bFaE21DF5A510931A262CECf63f41338F264': {'symbol': 'DBET', 'decimals': 18},
    '0xAFFCDd96531bCd66faED95FC61e443D08F79eFEf': {'symbol': 'PMGT', 'decimals': 5},
    '0xFbbE9b1142C699512545f47937Ee6fae0e4B0aA9': {'symbol': 'EDDA', 'decimals': 18},
    '0xbfD815347d024F449886c171f78Fa5B8E6790811': {'symbol': 'AAPX', 'decimals': 18},
    '0x9cF77be84214beb066F26a4ea1c38ddcc2AFbcf7': {'symbol': 'MSHIBA', 'decimals': 18},
    '0x47DA5456bC2e1ce391b645Ce80F2E97192e4976a': {'symbol': 'PLUG', 'decimals': 18},
    '0xD9c2D319Cd7e6177336b0a9c93c21cb48d84Fb54': {'symbol': 'HAPI', 'decimals': 18},
    '0x1B19C19393e2d034D8Ff31ff34c81252FcBbee92': {'symbol': 'OUSG', 'decimals': 18},
    '0x5732046A883704404F284Ce41FfADd5b007FD668': {'symbol': 'BLZ', 'decimals': 18},
    '0x1a6658F40e51b372E593B7d2144c1402d5cf33E8': {'symbol': 'PUBLX', 'decimals': 18},
    '0xF725f73CAEe250AE384ec38bB2C77C38ef2CcCeA': {'symbol': 'AIR', 'decimals': 18},
    '0x1AD82FAc4dfE68e68f22Df9Cb892b018E4541DbA': {'symbol': 'MARBLEX7', 'decimals': 18},
    '0xAc0C8dA4A4748d8d821A0973d00b157aA78C473D': {'symbol': 'YFO', 'decimals': 18},
    '0x4092678e4E78230F46A1534C0fbc8fA39780892B': {'symbol': 'OCN', 'decimals': 18},
    '0x799ebfABE77a6E34311eeEe9825190B9ECe32824': {'symbol': 'BTRST', 'decimals': 18},
    '0xB04bf60E468743418E87291d7c9301a5299d984d': {'symbol': '4SHIBA', 'decimals': 18},
    '0x02e7F808990638E9e67E1f00313037EDe2362361': {'symbol': 'KIBSHI', 'decimals': 18},
    '0x5E8422345238F34275888049021821E8E08CAa1f': {'symbol': 'FRXETH', 'decimals': 18},
    '0x3252d4221f92b7Af3774Da8312F01600eC84B252': {'symbol': 'XOLO', 'decimals': 18},
    '0x2BA8349123de45E931a8C8264c332E6e9CF593F9': {'symbol': 'BCMC', 'decimals': 18},
    '0x56015BBE3C01fE05bc30A8a9a9Fd9A88917e7dB3': {'symbol': 'CAT', 'decimals': 18},
    '0x7968bc6a03017eA2de509AAA816F163Db0f35148': {'symbol': 'HGET', 'decimals': 6},
    '0xe74bE071f3b62f6A4aC23cA68E5E2A39797A3c30': {'symbol': 'RCG', 'decimals': 18},
    '0xe28b3B32B6c345A34Ff64674606124Dd5Aceca30': {'symbol': 'INJ', 'decimals': 18},
    '0x7c32DB0645A259FaE61353c1f891151A2e7f8c1e': {'symbol': 'PTM', 'decimals': 18},
    '0x04C17b9D3b29A78F7Bd062a57CF44FC633e71f85': {'symbol': 'IMPT', 'decimals': 18},
    '0xAA3573CC3761133375b0C504146B93F6827F945A': {'symbol': 'ATH', 'decimals': 18},
    '0x0488401c3F535193Fa8Df029d9fFe615A06E74E6': {'symbol': 'SRK', 'decimals': 18},
    '0x846C66cf71C43f80403B51fE3906B3599D63336f': {'symbol': 'PMA', 'decimals': 18},
    '0x428dca9537116148616a5A3E44035Af17238Fe9d': {'symbol': 'OXAI', 'decimals': 18},
    '0x16CDA4028e9E872a38AcB903176719299beAed87': {'symbol': 'MARS4', 'decimals': 18},
    '0x833850BE8858722Cfc5E5e75f2Fe6275E055d888': {'symbol': 'DBNB', 'decimals': 9},
    '0xc528c28FEC0A90C083328BC45f587eE215760A0F': {'symbol': 'EDR', 'decimals': 18},
    '0x74232704659ef37c08995e386A2E26cc27a8d7B1': {'symbol': 'STRK', 'decimals': 18},
    '0x434Cb4fC4b952872967914D430878Eee53Ebd502': {'symbol': 'APCG', 'decimals': 18},
    '0xda23d301761E4e2bF474951f978f6DFB6F3C9F14': {'symbol': 'TKINU', 'decimals': 9},
    '0x2008e3057BD734e10AD13c9EAe45Ff132aBc1722': {'symbol': 'ZCO', 'decimals': 8},
    '0x1341A2257fa7b770420Ef70616f888056f90926c': {'symbol': 'ZOOT', 'decimals': 9},
    '0x947AEb02304391f8fbE5B25D7D98D649b57b1788': {'symbol': 'MDX', 'decimals': 18},
    '0xaF1250fa68D7DECD34fD75dE8742Bc03B29BD58e': {'symbol': 'IHF', 'decimals': 18},
    '0xd83AE04c9eD29d6D3E6Bf720C71bc7BeB424393E': {'symbol': 'INSURE', 'decimals': 18},
    '0xa14eA0E11121e6E951E87c66AFe460A00BCD6A16': {'symbol': 'IDLEDAISAFE', 'decimals': 18},
    '0x34748FEDAd669C788aaBe4fb325A4587941F7B48': {'symbol': 'TOS', 'decimals': 18},
    '0x2A039B1D9bBDccBb91be28691b730ca893e5e743': {'symbol': 'RNB', 'decimals': 18},
    '0xDe30da39c46104798bB5aA3fe8B9e0e1F348163F': {'symbol': 'GTC', 'decimals': 18},
    '0x1ef6A7e2c966fb7C5403EFEFdE38338b1a95a084': {'symbol': 'SLD', 'decimals': 18},
    '0xdF5e0e81Dff6FAF3A7e52BA697820c5e32D806A8': {'symbol': 'YCURVE', 'decimals': 18},
    '0xbf52F2ab39e26E0951d2a02b49B7702aBe30406a': {'symbol': 'ODE', 'decimals': 18},
    '0x5AA7C403c7dE4B3bb0cc07079a03e389671a4771': {'symbol': 'IBZ', 'decimals': 18},
    '0xceb286C9604c542d3cc08b41AA6C9675B078A832': {'symbol': 'VTX', 'decimals': 18},
    '0xD5DF655087d99B7B720A5bc8711F296180A4F44b': {'symbol': 'OPTIG', 'decimals': 18},
    '0xD46bA6D942050d489DBd938a2C909A5d5039A161': {'symbol': 'AMPL', 'decimals': 9},
    '0xDE075D9ADbD0240b4462F124af926452Ad0BAC91': {'symbol': 'BBF', 'decimals': 18},
    '0xb4615AAD766f6De3c55330099E907fF7F13f1582': {'symbol': 'ONIGI', 'decimals': 9},
    '0x2791BfD60D232150Bff86b39B7146c0eaAA2BA81': {'symbol': 'BIFI', 'decimals': 18},
    '0xa58a4f5c4Bb043d2CC1E170613B74e767c94189B': {'symbol': 'UTU', 'decimals': 18},
    '0x2162f572B25f7358db9376AB58a947a4e45CeDE1': {'symbol': 'LBL', 'decimals': 18},
    '0x05079687D35b93538cbd59fe5596380cae9054A9': {'symbol': 'BTSG', 'decimals': 18},
    '0xB1e9157c2Fdcc5a856C8DA8b2d89b6C32b3c1229': {'symbol': 'ZEFU', 'decimals': 18},
    '0x9355372396e3F6daF13359B7b607a3374cc638e0': {'symbol': 'WHALE', 'decimals': 4},
    '0x6F87D756DAf0503d08Eb8993686c7Fc01Dc44fB1': {'symbol': 'TRADE', 'decimals': 18},
    '0x429881672B9AE42b8EbA0E26cD9C73711b891Ca5': {'symbol': 'PICKLE', 'decimals': 18},
    '0xE33AE4E795114279721047484E5ad5cc7DF24fcB': {'symbol': 'MCF', 'decimals': 10},
    '0x5AFFF9876C1F98b7d2b53bCB69EB57e92408319F': {'symbol': 'MESA', 'decimals': 18},
    '0x249cA82617eC3DfB2589c4c17ab7EC9765350a18': {'symbol': 'VERSE', 'decimals': 18},
    '0x77a90B04d64189d4D09508612C09219bC6816BdC': {'symbol': 'TOSHI', 'decimals': 9},
    '0x069f967be0CA21C7D793D8C343f71e597D9a49b3': {'symbol': 'HZM', 'decimals': 8},
    '0x8185Bc4757572Da2a610f887561c32298f1A5748': {'symbol': 'ALN', 'decimals': 18},
    '0x2A8e1E676Ec238d8A992307B495b45B3fEAa5e86': {'symbol': 'OUSD', 'decimals': 18},
    '0xb3ED706B564BBa9CAB64042f4e1b391bE7BEbCe5': {'symbol': 'AGF', 'decimals': 18},
    '0xe530441f4f73bDB6DC2fA5aF7c3fC5fD551Ec838': {'symbol': 'GSE', 'decimals': 4},
    '0x12300ED54F65648524c1bFAf86712927dA6cC6fC': {'symbol': 'MOVD', 'decimals': 18},
    '0x27C70Cd1946795B66be9d954418546998b546634': {'symbol': 'LEASH', 'decimals': 18},
    '0xEA097A2b1dB00627B2Fa17460Ad260c016016977': {'symbol': 'UFR', 'decimals': 18},
    '0x74b1Af114274335598da72f5C6ed7b954a016EeD': {'symbol': 'HIT', 'decimals': 18},
    '0x624d520BAB2E4aD83935Fa503fB130614374E850': {'symbol': 'SSP', 'decimals': 4},
    '0x30D20208d987713f46DFD34EF128Bb16C404D10f': {'symbol': 'SD', 'decimals': 18},
    '0xD34A2051aC17BDD44D76AA84EA7Fcf8419130E55': {'symbol': 'KABOSUINU', 'decimals': 18},
    '0x11EBe21e9d7BF541A18e1E3aC94939018Ce88F0b': {'symbol': 'PITCHFXS', 'decimals': 18},
    '0xA6586E19EF681b1AC0ED3D46413D199a555dBB95': {'symbol': 'LETSGO', 'decimals': 18},
    '0x30dcBa0405004cF124045793E1933C798Af9E66a': {'symbol': 'YDF', 'decimals': 18},
    '0xfe9A29aB92522D14Fc65880d817214261D8479AE': {'symbol': 'SNOW', 'decimals': 18},
    '0x3a4f40631a4f906c2BaD353Ed06De7A5D3fCb430': {'symbol': 'PLA', 'decimals': 18},
    '0xB17548c7B510427baAc4e267BEa62e800b247173': {'symbol': 'SMT', 'decimals': 18},
    '0xE5a3229CCb22b6484594973A03a3851dCd948756': {'symbol': 'RAE', 'decimals': 18},
    '0xB4b9DC1C77bdbb135eA907fd5a08094d98883A35': {'symbol': 'SWEAT', 'decimals': 18},
    '0x569424c5EE13884A193773fDC5d1c5f79C443a51': {'symbol': 'PINE', 'decimals': 18},
    '0xbA93EF534094F8b7001ECe2691168140965341ab': {'symbol': 'LOTT', 'decimals': 18},
    '0x7103eBdbF1f89be2d53EFF9B3CF996C9E775c105': {'symbol': 'X7103', 'decimals': 18},
    '0xB3e2Cb7CccfE139f8FF84013823Bf22dA6B6390A': {'symbol': 'ICNQ', 'decimals': 18},
    '0x1a7a8BD9106F2B8D977E08582DC7d24c723ab0DB': {'symbol': 'APPC', 'decimals': 18},
    '0x3CbF23c081fAA5419810ce0F6BC1ECb73006d848': {'symbol': 'DIGI', 'decimals': 18},
    '0x184b6f31aeAa6a19Eb207dFeA01F4289629e0f0f': {'symbol': 'KMT', 'decimals': 18},
    '0x7104D1f179Cc9cc7fb5c79Be6Da846E3FBC4C105': {'symbol': 'X7104', 'decimals': 18},
    '0xe03B2642A5111aD0EFc0cbCe766498c2dd562Ae9': {'symbol': 'BC', 'decimals': 9},
    '0xae37D54Ae477268B9997d4161B96b8200755935c': {'symbol': 'BB-A-DAI', 'decimals': 18},
    '0x442Bc47357919446eAbC18C7211E57a13d983469': {'symbol': 'CHAT', 'decimals': 18},
    '0x0cbA60Ca5eF4D42f92A5070A8fEDD13BE93E2861': {'symbol': 'THE', 'decimals': 18},
    '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84': {'symbol': 'STETH', 'decimals': 18},
    '0xE99De844EF3Ef72806cf006224EF3b813e82662f': {'symbol': 'UMINT', 'decimals': 18},
    '0x6C22910c6F75F828B305e57c6a54855D8adeAbf8': {'symbol': 'SATS', 'decimals': 9},
    '0x8b6CdA5CC518c904e8844f445E1A7C7d2DB0fF16': {'symbol': 'SFCP', 'decimals': 18},
    '0x3FD8f39A962eFDA04956981C31AB89FAB5FB8bC8': {'symbol': 'RTH', 'decimals': 18},
    '0xD90E69f67203EBE02c917B5128629E77B4cd92dc': {'symbol': 'ONC', 'decimals': 18},
    '0x9506d37f70eB4C3d79C398d326C871aBBf10521d': {'symbol': 'MLT', 'decimals': 18},
    '0xb12494C8824fc069757F47d177E666c571Cd49aE': {'symbol': 'IMPACTXP', 'decimals': 9},
    '0x16980b3B4a3f9D89E33311B5aa8f80303E5ca4F8': {'symbol': 'KEX', 'decimals': 6},
    '0x57Ab1ec28D129707052df4dF418D58a2D46d5f51': {'symbol': 'SUSD', 'decimals': 18},
    '0x0173661769325565d4f011b2e5Cda688689Cc87c': {'symbol': 'QLT', 'decimals': 9},
    '0x536381a8628dBcC8C70aC9A30A7258442eAb4c92': {'symbol': 'PAN', 'decimals': 8},
    '0xc0114F14638a333A4D5C3b04F09b96372348A842': {'symbol': 'KEI', 'decimals': 9},
    '0x51FE2E572e97BFEB1D719809d743Ec2675924EDc': {'symbol': 'VPAD', 'decimals': 18},
    '0xfc82bb4ba86045Af6F327323a46E80412b91b27d': {'symbol': 'PROM', 'decimals': 18},
    '0x9992eC3cF6A55b00978cdDF2b27BC6882d88D1eC': {'symbol': 'POLY', 'decimals': 18},
    '0xC19B6A4Ac7C7Cc24459F08984Bbd09664af17bD1': {'symbol': 'SENSO', 'decimals': 0},
    '0x2C5BC2Ba3614fD27FCc7022eA71d9172E2632c16': {'symbol': 'SOV', 'decimals': 18},
    '0x89509aA1D14a8E1E5364Ec4C3B041213bcDbe08D': {'symbol': 'ZURR', 'decimals': 18},
    '0x797de1Dc0b9fAf5E25c1f7Efe8dF9599138fA09d': {'symbol': 'GPOOL', 'decimals': 18},
    '0xD8E3FB3b08eBA982F2754988d70D57eDc0055ae6': {'symbol': 'ZORA', 'decimals': 9},
    '0x3F9BEC82c776c47405BcB38070d2395Fd18F89d3': {'symbol': 'PHM', 'decimals': 18},
    '0x044727e50ff30DB57fad06Ff4F5846eAb5eA52a2': {'symbol': 'KITTY', 'decimals': 9},
    '0x674C6Ad92Fd080e4004b2312b45f796a192D27a0': {'symbol': 'XTN', 'decimals': 18},
    '0x4dd28568D05f09b02220b09C2cb307bFd837cb95': {'symbol': 'PRINTS', 'decimals': 18},
    '0x7b39917f9562C8Bc83c7a6c2950FF571375D505D': {'symbol': 'LEAG', 'decimals': 18},
    '0x320d31183100280CcdF69366CD56180Ea442A3E8': {'symbol': 'LHC', 'decimals': 8},
    '0x23894DC9da6c94ECb439911cAF7d337746575A72': {'symbol': 'JAM', 'decimals': 18},
    '0xaC0104Cca91D167873B8601d2e71EB3D4D8c33e0': {'symbol': 'CWS', 'decimals': 18},
    '0xEfaB7248D36585e2340E5d25F8a8D243E6e3193F': {'symbol': 'DACXI', 'decimals': 18},
    '0xC28e931814725BbEB9e670676FaBBCb694Fe7DF2': {'symbol': 'EQUAD', 'decimals': 18},
    '0xF921ae2DAC5fa128DC0F6168Bf153ea0943d2D43': {'symbol': 'FIRE', 'decimals': 8},
    '0x0d88eD6E74bbFD96B831231638b66C05571e824F': {'symbol': 'AVT', 'decimals': 18},
    '0xa0bEd124a09ac2Bd941b10349d8d224fe3c955eb': {'symbol': 'DEPAY', 'decimals': 18},
    '0x7945b0A6674b175695e5d1D08aE1e6F13744Abb0': {'symbol': 'BAOUSD', 'decimals': 18},
    '0x99D8a9C45b2ecA8864373A26D1459e3Dff1e17F3': {'symbol': 'MIM', 'decimals': 18},
    '0x1C720999A4F8B2B0E9199039D353D2c9A77A73fC': {'symbol': 'TQUEEN', 'decimals': 18},
    '0x9cCf191169A613Eca433a249d3F40131178CAeBc': {'symbol': 'MCG', 'decimals': 18},
    '0x4a615bB7166210CCe20E6642a6f8Fb5d4D044496': {'symbol': 'NAOS', 'decimals': 18},
    '0x0F5dEF84CED3e9e295DAe28df96d0B846dE92C1A': {'symbol': 'SDG', 'decimals': 18},
    '0xb3999F658C0391d94A37f7FF328F3feC942BcADC': {'symbol': 'HFT', 'decimals': 18},
    '0x4EC1b60B96193a64AcAe44778e51f7BfF2007831': {'symbol': 'EDGE', 'decimals': 18},
    '0x0b63128C40737B13647552e0C926bCFEccC35f93': {'symbol': 'WLITI', 'decimals': 18},
    '0xEABB8996eA1662cAd2f7fB715127852cd3262Ae9': {'symbol': 'CNFI', 'decimals': 18},
    '0xcD74Cf66c43e45Abd5703f3642F73d0675D6AFF7': {'symbol': 'EWC', 'decimals': 18},
    '0x6431FA4B812a2DCC062A38CB55cc7D18135AdEAd': {'symbol': 'RANKER', 'decimals': 18},
    '0x0f17BC9a994b87b5225cFb6a2Cd4D667ADb4F20B': {'symbol': 'JEUR', 'decimals': 18},
    '0x51DB5Ad35C671a87207d88fC11d593AC0C8415bd': {'symbol': 'MDA', 'decimals': 18},
    '0x205eD31C867bF715E4182137Af95AFE9177Cd8E7': {'symbol': 'DEFY', 'decimals': 18},
    '0x4463e6A3dEd0dBE3F6e15bC8420dFc55e5FeA830': {'symbol': 'TXA', 'decimals': 18},
    '0x6bA75D640bEbfe5dA1197bb5A2aff3327789b5d3': {'symbol': 'VEUR', 'decimals': 18},
    '0xD567B5F02b9073aD3a982a099a23Bf019FF11d1c': {'symbol': 'GAME', 'decimals': 5},
    '0x69D9905B2e5f6F5433212b7F3c954433F23C1572': {'symbol': 'OOKS', 'decimals': 18},
    '0x1456688345527bE1f37E9e627DA0837D6f08C925': {'symbol': 'USDP', 'decimals': 18},
    '0xF7920B0768Ecb20A123fAc32311d07D193381d6f': {'symbol': 'TNB', 'decimals': 18},
    '0x02e7Ac540409D32C90BfB51114003a9E1fF0249c': {'symbol': 'JPG', 'decimals': 18},
    '0xbfC66D8cCE39E668fd5d3C10fd1B1eAbB82c27B7': {'symbol': 'OVO', 'decimals': 18},
    '0x71Ab77b7dbB4fa7e017BC15090b2163221420282': {'symbol': 'HIGH', 'decimals': 18},
    '0x1E18821E69B9FAA8e6e75DFFe54E7E25754beDa0': {'symbol': 'KIMCHI', 'decimals': 18},
    '0x5d5E244660cA05C42073c9a526616d99f2c99516': {'symbol': 'GTCOIN', 'decimals': 18},
    '0xCbfef8fdd706cde6F208460f2Bf39Aa9c785F05D': {'symbol': 'KINE', 'decimals': 18},
    '0xc73C167E7a4Ba109e4052f70D5466D0C312A344D': {'symbol': 'SANSHU', 'decimals': 9},
    '0xd084B83C305daFD76AE3E1b4E1F1fe2eCcCb3988': {'symbol': 'TVK', 'decimals': 18},
    '0x33Cf519030235F75A13F29aFcFF5D6ff4Fd10350': {'symbol': 'PEPEBET', 'decimals': 18},
    '0xf91AC30E9b517f6D57e99446ee44894e6C22C032': {'symbol': 'LOL', 'decimals': 9},
    '0x88A9A52F944315D5B4e917b9689e65445C401E83': {'symbol': 'FEAR', 'decimals': 18},
    '0xE1BDA0c3Bfa2bE7f740f0119B6a34F057BD58Eba': {'symbol': 'WNK', 'decimals': 18},
    '0x469eDA64aEd3A3Ad6f868c44564291aA415cB1d9': {'symbol': 'FLUX', 'decimals': 18},
    '0x65a8fbA02F641a13Bb7B01d5E1129b0521004f52': {'symbol': 'AMAS', 'decimals': 18},
    '0x72dD4b6bd852A3AA172Be4d6C5a6dbEc588cf131': {'symbol': 'NGC', 'decimals': 18},
    '0x8207c1FfC5B6804F6024322CcF34F29c3541Ae26': {'symbol': 'OGN', 'decimals': 18},
    '0x712DB54daA836B53Ef1EcBb9c6ba3b9Efb073F40': {'symbol': 'AENJ', 'decimals': 18},
    '0x0Ec72CD6690db40B16be166858299F19D4f8E5B0': {'symbol': 'GOLD1', 'decimals': 9},
    '0x65ad6A2288b2Dd23E466226397c8F5D1794e58fC': {'symbol': 'GFX', 'decimals': 18},
    '0xE22020F47B7378dfedcedd2C81d4137c22fE1152': {'symbol': 'MLP', 'decimals': 18},
    '0xed0889F7E1c7C7267407222Be277e1f1Ef4d4892': {'symbol': 'MEL', 'decimals': 18},
    '0x9c405acf8688AfB61B3197421cDeeC1A266c6839': {'symbol': 'DOGY', 'decimals': 18},
    '0x7BEC98609cB6378D6F995e8f8097Ee78376fbec9': {'symbol': 'LM', 'decimals': 18},
    '0xc6DdDB5bc6E61e0841C54f3e723Ae1f3A807260b': {'symbol': 'URUS', 'decimals': 18},
    '0x99c6e435eC259A7E8d65E1955C9423DB624bA54C': {'symbol': 'FMT', 'decimals': 18},
    '0x505B5eDa5E25a67E1c24A2BF1a527Ed9eb88Bf04': {'symbol': 'CWEB', 'decimals': 18},
    '0xd35Ce4FD815522Ff52243EB587020d557617f4dB': {'symbol': 'DFE', 'decimals': 18},
    '0xE95A203B1a91a908F9B9CE46459d101078c2c3cb': {'symbol': 'ANKRETH', 'decimals': 18},
    '0x940a2dB1B7008B6C776d4faaCa729d6d4A4AA551': {'symbol': 'DUSK', 'decimals': 18},
    '0x9FC8f0ca1668E87294941b7f627e9C15eA06B459': {'symbol': 'PNL', 'decimals': 18},
    '0xF0939011a9bb95c3B791f0cb546377Ed2693a574': {'symbol': 'ZERO', 'decimals': 18},
    '0xC631120155621Ee625835ec810B9885cDd764cd6': {'symbol': 'GLDX', 'decimals': 8},
    '0xAA2DEd323944b25C0B6f1F891Bc96F010b65622C': {'symbol': 'THERADIO', 'decimals': 18},
    '0xa3c56427683a19F7574b9fc219CFD27d5d6e87Fa': {'symbol': 'SAITOKI', 'decimals': 9},
    '0xA9B1Eb5908CfC3cdf91F9B8B3a74108598009096': {'symbol': 'AUCTION', 'decimals': 18},
    '0xC744df3419A8c9Bd4D6b9852A503eb1C5308A326': {'symbol': 'RED', 'decimals': 18},
    '0x697eF32B4a3F5a4C39dE1cB7563f24CA7BfC5947': {'symbol': 'ISLA', 'decimals': 18},
    '0xbA745513ACEbcBb977497C569D4F7d340f2A936B': {'symbol': 'MFTU', 'decimals': 18},
    '0x39C6b3e42d6A679d7D776778Fe880BC9487C2EDA': {'symbol': 'AKNC', 'decimals': 18},
    '0x69fa0feE221AD11012BAb0FdB45d444D3D2Ce71c': {'symbol': 'XRUNE', 'decimals': 18},
    '0x5BEfBB272290dD5b8521D4a938f6c4757742c430': {'symbol': 'XFI', 'decimals': 18},
    '0x8a40c222996f9F3431f63Bf80244C36822060f12': {'symbol': 'FXF', 'decimals': 18},
    '0x68FfB43026CEdC070bfC01B2203d90caB62CcF30': {'symbol': 'TUSHI', 'decimals': 18},
    '0xFe2e637202056d30016725477c5da089Ab0A043A': {'symbol': 'SETH2', 'decimals': 18},
    '0xE74dC43867E0cbEB208F1a012fc60DcBbF0E3044': {'symbol': 'CWAP', 'decimals': 18},
    '0xA0d69E286B938e21CBf7E51D71F6A4c8918f482F': {'symbol': 'EUSD', 'decimals': 18},
    '0xfd30C9BEA1A952FEeEd2eF2C6B2Ff8A8FC4aAD07': {'symbol': 'KALLY', 'decimals': 18},
    '0xd337382da15d12Bb6e56498e91DF64f86c8f1EA8': {'symbol': 'AEL', 'decimals': 18},
    '0xc502002aEB1b9309FcCb016adF50507987Fc6C2b': {'symbol': 'GNFT', 'decimals': 18},
    '0xa4E9584DAa093Cb1205E17bA737c3fd015748087': {'symbol': 'FOREX', 'decimals': 18},
    '0x970B9bB2C0444F5E81e9d0eFb84C8ccdcdcAf84d': {'symbol': 'FUSE', 'decimals': 18},
    '0xF75C7a59bCD9bd207C4Ab1BEB0b32EEd3B6392f3': {'symbol': 'KEI', 'decimals': 8},
    '0xd81b71cBb89B2800CDb000AA277Dc1491dc923C3': {'symbol': 'NMT', 'decimals': 18},
    '0x71a28feAEe902966DC8D355e7B8Aa427D421e7e0': {'symbol': 'LUNCH', 'decimals': 18},
    '0x6a445E9F40e0b97c92d0b8a3366cEF1d67F700BF': {'symbol': 'FIDU', 'decimals': 18},
    '0x37c4822DFcEa7211d1D9876dABd48c62D46dbAAb': {'symbol': 'MBET', 'decimals': 18},
    '0x44108f0223A3C3028F5Fe7AEC7f9bb2E66beF82F': {'symbol': 'ACX', 'decimals': 18},
    '0xb399511642FE1666c6a07f83483e6E4feAed9A00': {'symbol': 'SEURO', 'decimals': 18},
    '0xEEF9f339514298C6A857EfCfC1A762aF84438dEE': {'symbol': 'HEZ', 'decimals': 18},
    '0xe0a189C975e4928222978A74517442239a0b86ff': {'symbol': 'KEYS', 'decimals': 9},
    '0xA3ec7DF93090f4DE46FaEA09F73cc40ebe7dD714': {'symbol': 'WUKONG', 'decimals': 18},
    '0xfa05A73FfE78ef8f1a739473e462c54bae6567D9': {'symbol': 'LUN', 'decimals': 18},
    '0x7cA4408137eb639570F8E647d9bD7B7E8717514A': {'symbol': 'ALPA', 'decimals': 18},
    '0x179Cd91631d96e8faFEe6A744eAC6fFdbB923520': {'symbol': 'IXI', 'decimals': 8},
    '0x4EcE5C5CfB9B960A49aAe739E15cDB6cfDcc5782': {'symbol': 'DBUY', 'decimals': 9},
    '0x5022Cb6D39001CDD6F996e8a66500c86128f1cc4': {'symbol': 'SHIWA', 'decimals': 9},
    '0x628eBC64A38269E031AFBDd3C5BA857483B5d048': {'symbol': 'LSETH', 'decimals': 18},
    '0x6b1a8f210Ec6b7B6643cea3583Fb0c079f367898': {'symbol': 'BXX', 'decimals': 18},
    '0x0335A7610D817aeCA1bEBbEfbd392ecC2eD587B8': {'symbol': 'NITRO', 'decimals': 18},
    '0x2822f6D1B2f41F93f33d937bc7d84A8Dfa4f4C21': {'symbol': 'QQQ', 'decimals': 18},
    '0xb056c38f6b7Dc4064367403E26424CD2c60655e1': {'symbol': 'CEEK', 'decimals': 18},
    '0xC90906d45046059221e908de5Cc8fcfaCA859235': {'symbol': 'SHIBP', 'decimals': 18},
    '0xF17e65822b568B3903685a7c9F496CF7656Cc6C2': {'symbol': 'BICO', 'decimals': 18},
    '0x9cf98Eb8A8B28c83E8612046cf55701Ce3Eb0063': {'symbol': 'UGT', 'decimals': 18},
    '0x849C479D7A90Eb378dbD00E8f166371176244eb1': {'symbol': 'MUU', 'decimals': 9},
    '0x79bE75FFC64DD58e66787E4Eae470c8a1FD08ba4': {'symbol': 'AAMMDAI', 'decimals': 18},
    '0x6f80310CA7F2C654691D1383149Fa1A57d8AB1f8': {'symbol': 'SILO', 'decimals': 18},
    '0x3F68e7B44e9bCB486C2FeAdB7A2289D9cdFC9088': {'symbol': 'ICONS', 'decimals': 18},
    '0xff742d05420B6Aca4481F635aD8341F81A6300C2': {'symbol': 'ASD', 'decimals': 18},
    '0xeb986DA994E4a118d5956b02d8b7c3C7CE373674': {'symbol': 'GTH', 'decimals': 18},
    '0x0A3BB08b3a15A19b4De82F8AcFc862606FB69A2D': {'symbol': 'IUSD', 'decimals': 18},
    '0xe63d6B308BCe0F6193AeC6b7E6eBa005f41e36AB': {'symbol': 'STN', 'decimals': 18},
    '0xb46eda6219ba121cE9280388e7afb7dc84be3fF2': {'symbol': 'TYP', 'decimals': 0},
    '0xFEb6d5238Ed8F1d59DCaB2db381AA948e625966D': {'symbol': 'DGTV', 'decimals': 18},
    '0x949D48EcA67b17269629c7194F4b727d4Ef9E5d6': {'symbol': 'MC', 'decimals': 18},
    '0xB97048628DB6B661D4C2aA833e95Dbe1A905B280': {'symbol': 'PAY', 'decimals': 18},
    '0xf51EBf9a26DbC02B13F8B3a9110dac47a4d62D78': {'symbol': 'APIX', 'decimals': 18},
    '0x04b77A1be2981c1Ca353aaf251b0F11398413bFA': {'symbol': 'DOOMER', 'decimals': 18},
    '0x21413c119b0C11C5d96aE1bD328917bC5C8ED67E': {'symbol': 'GENE', 'decimals': 18},
    '0xFCc5c47bE19d06BF83eB04298b026F81069ff65b': {'symbol': 'YCRV', 'decimals': 18},
    '0x905E337c6c8645263D3521205Aa37bf4d034e745': {'symbol': 'MTC', 'decimals': 18},
    '0x4E08F03079c5CD3083eA331Ec61bCC87538B7665': {'symbol': 'DODI', 'decimals': 18},
    '0xAAA9214F675316182Eaa21C85f0Ca99160CC3AAA': {'symbol': 'QANX', 'decimals': 18},
    '0xB097Dd6a23175C10eAA45A1F410d7eB4cFDAFC8a': {'symbol': 'KAINET', 'decimals': 9},
    '0x0c7D5ae016f806603CB1782bEa29AC69471CAb9c': {'symbol': 'BFC', 'decimals': 18},
    '0x9d93692E826A4bd9e903e2A27D7FbD1e116efdad': {'symbol': 'POLY', 'decimals': 9},
    '0xfA5047c9c78B8877af97BDcb85Db743fD7313d4a': {'symbol': 'ROOK', 'decimals': 18},
    '0x368BF9F1A1CA767935E39F20439D9041707E2634': {'symbol': 'CFXT', 'decimals': 18},
    '0xA9598333B99d14D90Bc81CAd8Af82C4C70625e75': {'symbol': 'MTS', 'decimals': 18},
    '0x321C2fE4446C7c963dc41Dd58879AF648838f98D': {'symbol': 'CTX', 'decimals': 18},
    '0x4f5fa8f2d12e5eB780f6082Dd656C565C48E0f24': {'symbol': 'GUM', 'decimals': 18},
    '0x29d578CEc46B50Fa5C88a99C6A4B70184C062953': {'symbol': 'EVER', 'decimals': 9},
    '0x87931E7AD81914e7898d07c68F145fC0A553D8Fb': {'symbol': 'WIZARD', 'decimals': 18},
    '0x2Ebd53d035150f328bd754D6DC66B99B0eDB89aa': {'symbol': 'MET', 'decimals': 18},
    '0x4a1d542b52a95AD01Ddc70C2e7df0c7bBAAdc56f': {'symbol': 'NIFT', 'decimals': 18},
    '0x368C5290b13cAA10284Db58B4ad4F3E9eE8bf4c9': {'symbol': 'KKO', 'decimals': 18},
    '0x5fBc3cb8B428cC00A04808870295D39962CC7FeE': {'symbol': 'BTZ', 'decimals': 18},
    '0xa4Cf2aFD3B165975afFFBf7e487CDd40C894Ab6B': {'symbol': 'SHIBAKEN', 'decimals': 0},
    '0x35f6B052C598d933D69A4EEC4D04c73A191fE6c2': {'symbol': 'ASNX', 'decimals': 18},
    '0x805856f176625843d14097017618E224F4607D2D': {'symbol': 'BES', 'decimals': 18},
    '0xf028ADEe51533b1B47BEaa890fEb54a457f51E89': {'symbol': 'BMT', 'decimals': 18},
    '0xA696a63cc78DfFa1a63E9E50587C197387FF6C7E': {'symbol': 'YVWBTC', 'decimals': 8},
    '0xCc802c45B55581713cEcd1Eb17BE9Ab7fcCb0844': {'symbol': 'BHNY', 'decimals': 18},
    '0x8D75959f1E61EC2571aa72798237101F084DE63a': {'symbol': 'SUB', 'decimals': 18},
    '0x94D40B49F020BfeBbA1a80A0191EB3737B90E8d3': {'symbol': 'MNTE', 'decimals': 18},
    '0x33f391F4c4fE802b70B77AE37670037A92114A7c': {'symbol': 'BURP', 'decimals': 18},
    '0xfE18be6b3Bd88A2D2A7f928d00292E7a9963CfC6': {'symbol': 'SBTC', 'decimals': 18},
    '0x155040625D7ae3e9caDA9a73E3E44f76D3Ed1409': {'symbol': 'REVO', 'decimals': 18},
    '0x1b4dD5eA240732dDAc8d74FD1Cd9026Addc02e3c': {'symbol': 'DOGENS', 'decimals': 9},
    '0x4eED0fa8dE12D5a86517f214C2f11586Ba2ED88D': {'symbol': 'BITE', 'decimals': 18},
    '0x9ffc3bCDe7B68C46a6dC34f0718009925c1867cB': {'symbol': 'HDOT', 'decimals': 18},
    '0x038a68FF68c393373eC894015816e33Ad41BD564': {'symbol': 'GLCH', 'decimals': 18},
    '0xA130E3a33a4d84b04c3918c4E5762223Ae252F80': {'symbol': 'SWASH', 'decimals': 18},
    '0x96E61422b6A9bA0e068B6c5ADd4fFaBC6a4aae27': {'symbol': 'IBEUR', 'decimals': 18},
    '0xa11bD36801d8fa4448F0ac4ea7A62e3634cE8C7C': {'symbol': 'ABR', 'decimals': 18},
    '0x28b5E12CcE51f15594B0b91d5b5AdaA70F684a02': {'symbol': 'NPX', 'decimals': 2},
    '0xfB5c6815cA3AC72Ce9F5006869AE67f18bF77006': {'symbol': 'PSTAKE', 'decimals': 18},
    '0xAE1eaAE3F627AAca434127644371b67B18444051': {'symbol': 'YOP', 'decimals': 8},
    '0x626E8036dEB333b408Be468F951bdB42433cBF18': {'symbol': 'AIOZ', 'decimals': 18},
    '0x89d24A6b4CcB1B6fAA2625fE562bDD9a23260359': {'symbol': 'SAI', 'decimals': 18},
    '0x83031984B45553070a088273f341BfF7fB4f2F46': {'symbol': 'HIBAYC', 'decimals': 18},
    '0x6Aac8CB9861E42bf8259F5AbDC6aE3Ae89909E11': {'symbol': 'BTCRED', 'decimals': 8},
    '0xc07A150ECAdF2cc352f5586396e344A6b17625EB': {'symbol': 'BIOT', 'decimals': 9},
    '0xADf86E75d8f0F57e0288D0970E7407eaA49b3CAb': {'symbol': 'APOLLO', 'decimals': 9},
    '0x6BC08509B36A98E829dFfAD49Fde5e412645d0a3': {'symbol': 'WOOF', 'decimals': 18},
    '0xd01409314aCb3b245CEa9500eCE3F6Fd4d70ea30': {'symbol': 'LTO', 'decimals': 8},
    '0x2Ab6Bb8408ca3199B8Fa6C92d5b455F820Af03c4': {'symbol': 'TONE', 'decimals': 18},
    '0x6a4C76874e686A7d080D173987A35A9c48905583': {'symbol': 'LPNT', 'decimals': 18},
    '0xe0B9a2C3E9f40CF74B2C7F591B2b0CCa055c3112': {'symbol': 'GS', 'decimals': 18},
    '0x35A18000230DA775CAc24873d00Ff85BccdeD550': {'symbol': 'CUNI', 'decimals': 8},
    '0x64551fA4093c2D6c298cf1792740F884084a8F69': {'symbol': 'SPGBB', 'decimals': 18},
    '0x549044000a94870ab7c5017cd8fB0AEfa9239a13': {'symbol': 'UCOIL', 'decimals': 18},
    '0x887168120cb89Fb06F3E74Dc4AF20D67dF0977f6': {'symbol': 'SKRT', 'decimals': 18},
    '0xaeDf386B755465871fF874E3E37Af5976E247064': {'symbol': 'FTN', 'decimals': 18},
    '0xDb298285FE4C5410B05390cA80e8Fbe9DE1F259B': {'symbol': 'FOREX', 'decimals': 18},
    '0x425d105913a04f4F95311788195bA13c3E82c3B9': {'symbol': 'APOLLO', 'decimals': 9},
    '0xFF44b5719f0B77A9951636fc5e69d3a1fc9E7d73': {'symbol': '4ART', 'decimals': 18},
    '0xBd356a39BFf2cAda8E9248532DD879147221Cf76': {'symbol': 'WOM', 'decimals': 18},
    '0xA0CF46eb152656C7090e769916eb44a138aaa406': {'symbol': 'SPH', 'decimals': 18},
    '0x429acA1cCd47296483D1281c85B24e842de0b758': {'symbol': 'YEET', 'decimals': 18},
    '0x79A06aCb8bdd138BEEECcE0f1605971f3AC7c09B': {'symbol': 'LSVR', 'decimals': 18},
    '0xA5Ef74068d04ba0809B7379dD76Af5Ce34Ab7C57': {'symbol': 'LUCHOW', 'decimals': 18},
    '0x6e0daDE58D2d89eBBe7aFc384e3E4f15b70b14D8': {'symbol': 'QRX', 'decimals': 18},
    '0xA71DCaaf5ba390B1B2012323A8810fAc121F90dd': {'symbol': 'SFG', 'decimals': 18},
    '0xcb86c6A22CB56B6cf40CaFEDb06BA0DF188a416E': {'symbol': 'SURE', 'decimals': 18},
    '0x852e5427c86A3b46DD25e5FE027bb15f53c4BCb8': {'symbol': 'NIIFI', 'decimals': 15},
    '0xB1A30851E3f7d841b231B086479608e17198363A': {'symbol': 'HMR', 'decimals': 18},
    '0x5EE84583f67D5EcEa5420dBb42b462896E7f8D06': {'symbol': 'PLSB', 'decimals': 12},
    '0x3a8d5BC8A8948b68DfC0Ce9C14aC4150e083518c': {'symbol': 'PARA', 'decimals': 18},
    '0xC3E486F614e297d8E016aC2805e81707C627b2d5': {'symbol': 'M', 'decimals': 18},
    '0x8Ab7404063Ec4DBcfd4598215992DC3F8EC853d7': {'symbol': 'AKRO', 'decimals': 18},
    '0xeca82185adCE47f39c684352B0439f030f860318': {'symbol': 'PERL', 'decimals': 18},
    '0x08389495D7456E1951ddF7c3a1314A4bfb646d8B': {'symbol': 'CRPT', 'decimals': 18},
    '0xBc276f51184ec085cd1437601eCb4bBc3D981896': {'symbol': 'SHIBTAMA', 'decimals': 18},
    '0xDc63269eA166b70d4780b3A11F5C825C2b761B01': {'symbol': 'PAW', 'decimals': 18},
    '0x23B608675a2B2fB1890d3ABBd85c5775c51691d5': {'symbol': 'SOCKS', 'decimals': 18},
    '0xaB93dF617F51E1E415b5b4f8111f122d6b48e55C': {'symbol': 'DETO', 'decimals': 18},
    '0x9c5476Af06590A9277C7706fe70c0a42A480F8A0': {'symbol': 'CHOO', 'decimals': 18},
    '0x4a621d9f1b19296d1C0f87637b3A8D4978e9bf82': {'symbol': 'CYFM', 'decimals': 18},
    '0x43Ab765ee05075d78AD8aa79dcb1978CA3079258': {'symbol': 'POW', 'decimals': 18},
    '0x1e2D230C7A7F4C679Fb1378F1f51dEDeAe85Cd72': {'symbol': 'PYRO', 'decimals': 18},
    '0x3541A5C1b04AdABA0B83F161747815cd7B1516bC': {'symbol': 'KNIGHT', 'decimals': 18},
    '0xe1eC350ea16D1DdAFF57F31387B2d9708Eb7ce28': {'symbol': 'PC', 'decimals': 9},
    '0xC4C2614E694cF534D407Ee49F8E44D125E4681c4': {'symbol': 'CHAIN', 'decimals': 18},
    '0xF1cA9cb74685755965c7458528A36934Df52A3EF': {'symbol': 'AVINOC', 'decimals': 18},
    '0x80f0C1c49891dcFDD40b6e0F960F84E6042bcB6F': {'symbol': 'DXN', 'decimals': 18},
    '0x0f7F961648aE6Db43C75663aC7E5414Eb79b5704': {'symbol': 'XIO', 'decimals': 18},
    '0x84c722e6F1363E8D5C6dB3eA600bEF9a006Da824': {'symbol': 'MSB', 'decimals': 18},
    '0x6C8c6b02E7b2BE14d4fA6022Dfd6d75921D90E4E': {'symbol': 'CBAT', 'decimals': 8},
    '0x6fB1E018f107d3352506c23777e4cd62e063584a': {'symbol': 'IDTT', 'decimals': 18},
    '0x49E833337ECe7aFE375e44F4E3e8481029218E5c': {'symbol': 'VALUE', 'decimals': 18},
    '0xf5d126077096e5b01BC30FFa5d9324D7202d7Cb3': {'symbol': 'CHEW', 'decimals': 18},
    '0x131157c6760f78f7dDF877C0019Eba175BA4b6F6': {'symbol': 'BIGSB', 'decimals': 18},
    '0x8E964e35A76103Af4C7D7318e1B1a82c682ae296': {'symbol': 'FLZ', 'decimals': 18},
    '0x8A7aDc1B690E81c758F1BD0F72DFe27Ae6eC56A5': {'symbol': 'BLID', 'decimals': 18},
    '0x725C263e32c72dDC3A19bEa12C5a0479a81eE688': {'symbol': 'BMI', 'decimals': 18},
    '0x6EC8a24CaBdc339A06a172F8223ea557055aDAa5': {'symbol': 'GNX', 'decimals': 9},
    '0xB840d10D840eF47c233FEC1fd040F5B145a6DfA5': {'symbol': 'STREETH', 'decimals': 18},
    '0x000000000000d0151E748d25b766e77efe2A6c83': {'symbol': 'XDEX', 'decimals': 18},
    '0xBA50933C268F567BDC86E1aC131BE072C6B0b71a': {'symbol': 'ARPA', 'decimals': 18},
    '0xF34842d05A1c888Ca02769A633DF37177415C2f8': {'symbol': 'IDLEUSDTYIELD', 'decimals': 18},
    '0xb44377B74Ef1773639B663d0754cB8410A847d02': {'symbol': 'DREAM', 'decimals': 18},
    '0xb19189Fb36c816f3e0f16065057B07B790998fDC': {'symbol': 'SER', 'decimals': 18},
    '0x6Fbc20483b53CEa47839Bb8e171Abd6D67C3c696': {'symbol': 'EVOAI', 'decimals': 9},
    '0xC4cB5793BD58BaD06bF51FB37717b86B02CBe8A4': {'symbol': 'CREDIT', 'decimals': 18},
    '0x6d57B2E05F26C26b549231c866bdd39779e4a488': {'symbol': 'VNXAU', 'decimals': 18},
    '0xccC8cb5229B0ac8069C51fd58367Fd1e622aFD97': {'symbol': 'GODS', 'decimals': 18},
    '0xd780Ae2Bf04cD96E577D3D014762f831d97129d0': {'symbol': 'EVN', 'decimals': 18},
    '0xAcf8D5E515Ed005655DfeFA09C22673a37A7CDEE': {'symbol': 'FNF', 'decimals': 18},
    '0xd2877702675e6cEb975b4A1dFf9fb7BAF4C91ea9': {'symbol': 'LUNC', 'decimals': 18},
    '0x60F63B76E2Fc1649E57a3489162732A90ACf59FE': {'symbol': 'FLURRY', 'decimals': 18},
    '0x4c11249814f11b9346808179Cf06e71ac328c1b5': {'symbol': 'ORAI', 'decimals': 18},
    '0x2f7B618993cc3848d6C7ed9CdD5e835E4Fe22b98': {'symbol': 'NAMI', 'decimals': 18},
    '0x788B6D2B37Aa51D916F2837Ae25b05f0e61339d1': {'symbol': 'MVD', 'decimals': 9},
    '0x8793Fb615Eb92822F482f88B3137B00aad4C00D2': {'symbol': 'REVOAI', 'decimals': 9},
    '0xc3dD23A0a854b4f9aE80670f528094E9Eb607CCb': {'symbol': 'TRND', 'decimals': 18},
    '0xfc05987bd2be489ACCF0f509E44B0145d68240f7': {'symbol': 'ESS', 'decimals': 18},
    '0xe2Fc859C838F31C1450448f4FeD92e3284aE49E9': {'symbol': 'JIG', 'decimals': 18},
    '0x16756EC1DEb89A2106C35E0B586a799D0A61837D': {'symbol': 'CHEDDA', 'decimals': 18},
    '0x95a4492F028aa1fd432Ea71146b433E7B4446611': {'symbol': 'APY', 'decimals': 18},
    '0xceec4aE5CF5300aC9A49a816961Af9393b214674': {'symbol': 'INSAI', 'decimals': 6},
    '0x53fD2342B43eCD24AEf1535BC3797F509616Ce8c': {'symbol': 'ANARCHY', 'decimals': 9},
    '0xE66747a101bFF2dBA3697199DCcE5b743b454759': {'symbol': 'GT', 'decimals': 18},
    '0x5B09A0371C1DA44A8E24D36Bf5DEb1141a84d875': {'symbol': 'MAD', 'decimals': 18},
    '0xEd1480d12bE41d92F36f5f7bDd88212E381A3677': {'symbol': 'FDT', 'decimals': 18},
    '0x234D51eE02be808A0160b19b689660Fb7BFA871b': {'symbol': 'SCAN', 'decimals': 9},
    '0xbd1848e1491d4308Ad18287A745DD4DB2A4BD55B': {'symbol': 'MOMA', 'decimals': 18},
    '0xC52C326331E9Ce41F04484d3B5E5648158028804': {'symbol': 'ZCX', 'decimals': 18},
    '0xEB9A4B185816C354dB92DB09cC3B50bE60b901b6': {'symbol': 'ORS', 'decimals': 18},
    '0x5e9F35E8163c44cD7e606BdD716AbED32AD2F1C6': {'symbol': 'SAITANOBI', 'decimals': 9},
    '0xc0EC8CaEC55F37D47fBfA595727418868A21fd48': {'symbol': 'EGC', 'decimals': 8},
    '0x8Ae4BF2C33a8e667de34B54938B0ccD03Eb8CC06': {'symbol': 'PTOY', 'decimals': 8},
    '0x2C4e8f2D746113d0696cE89B35F0d8bF88E0AEcA': {'symbol': 'OST', 'decimals': 18},
    '0x8E0fE2947752BE0d5ACF73aaE77362Daf79cB379': {'symbol': 'NFTD', 'decimals': 18},
    '0x1E4EDE388cbc9F4b5c79681B7f94d36a11ABEBC9': {'symbol': 'X2Y2', 'decimals': 18},
    '0x0275E1001e293C46CFe158B3702AADe0B99f88a5': {'symbol': 'OIL', 'decimals': 18},
    '0xB131F337C45D386cEeC234e194b2663D5c3d9dCF': {'symbol': 'ICOM', 'decimals': 18},
    '0xd6A5aB46ead26f49b03bBB1F9EB1Ad5c1767974a': {'symbol': 'EMON', 'decimals': 18},
    '0xd3E133a0A14Bb8B595e5Fbf9851c7c783685BA69': {'symbol': 'LKT', 'decimals': 18},
    '0x5E6FFe7B174A50c81fF3f3c54c04fD3c11E20830': {'symbol': 'DVC', 'decimals': 18},
    '0xbEa98c05eEAe2f3bC8c3565Db7551Eb738c8CCAb': {'symbol': 'GYSR', 'decimals': 18},
    '0xef952363C1d990A2fa58F8b379A9fa33bad1dfd1': {'symbol': 'LYNK', 'decimals': 8},
    '0xBeD4AB0019ff361d83ddeB74883DAC8a70f5ea1e': {'symbol': 'MRCH', 'decimals': 18},
    '0xdD69DB25F6D620A7baD3023c5d32761D353D3De9': {'symbol': 'GETH', 'decimals': 18},
    '0xd1AFBCCC9A2c2187ea544363B986EA0AB6EF08B5': {'symbol': 'ETHY', 'decimals': 18},
    '0x7DD9c5Cba05E151C895FDe1CF355C9A1D5DA6429': {'symbol': 'GLM', 'decimals': 18},
    '0xD69F306549e9d96f183B1AecA30B8f4353c2ECC3': {'symbol': 'MCHC', 'decimals': 18},
    '0x5F64Ab1544D28732F0A24F4713c2C8ec0dA089f0': {'symbol': 'DEXTF', 'decimals': 18},
    '0x94c9cEb2F9741230FAD3a62781b27Cc79a9460d4': {'symbol': 'MAYC', 'decimals': 18},
    '0x93C9175E26F57d2888c7Df8B470C9eeA5C0b0A93': {'symbol': 'BCUBE', 'decimals': 18},
    '0x473037de59cf9484632f4A27B509CFE8d4a31404': {'symbol': 'GST-ETH', 'decimals': 8},
    '0x1F8A626883d7724DBd59eF51CBD4BF1Cf2016D13': {'symbol': 'STAK', 'decimals': 18},
    '0x2d787D4f5005Bd66aC910c2E821241625c406ed5': {'symbol': 'BERRY', 'decimals': 18},
    '0xAB2A7B5876D707e0126B3A75EF7781c77c8877EE': {'symbol': 'QUAD', 'decimals': 18},
    '0x4Ddc2D193948926D02f9B1fE9e1daa0718270ED5': {'symbol': 'CETH', 'decimals': 8},
    '0xA2b4C0Af19cC16a6CfAcCe81F192B024d625817D': {'symbol': 'KISHU', 'decimals': 9},
    '0xe616ADb3f729f6f3df19C876Eda66472a308D397': {'symbol': 'PPAI', 'decimals': 18},
    '0x00F29171D7bCDC464a0758cF3217fE83173772b9': {'symbol': 'INU', 'decimals': 9},
    '0x6C2adC2073994fb2CCC5032cC2906Fa221e9B391': {'symbol': 'DPY', 'decimals': 18},
    '0x01597E397605Bf280674Bf292623460b4204C375': {'symbol': 'BENT', 'decimals': 18},
    '0xd9b312D77Bc7BEd9B9CecB56636300bED4Fe5Ce9': {'symbol': 'GAINS', 'decimals': 18},
    '0xb8b295df2cd735b15BE5Eb419517Aa626fc43cD5': {'symbol': 'STLINK', 'decimals': 18},
    '0xec6d73557937974077911a0B6fDc436b0ff70296': {'symbol': 'SKRIMP', 'decimals': 18},
    '0xa66Daa57432024023DB65477BA87D4E7F5f95213': {'symbol': 'HPT', 'decimals': 18},
    '0xf23a2BF7619Ad93300499c3fC7d7145A06CcA562': {'symbol': 'NOISEGPT', 'decimals': 18},
    '0x53263d9EF74Db583b15fbC6D5D4e8B83833fa134': {'symbol': 'LEAP', 'decimals': 18},
    '0x34F0915a5f15a66Eba86F6a58bE1A471FB7836A7': {'symbol': 'PLSD', 'decimals': 12},
    '0x40803cEA2b2A32BdA1bE61d3604af6a814E70976': {'symbol': 'SPOOL', 'decimals': 18},
    '0x948c70Dc6169Bfb10028FdBE96cbC72E9562b2Ac': {'symbol': 'XP', 'decimals': 18},
    '0xf0f9D895aCa5c8678f706FB8216fa22957685A13': {'symbol': 'CULT', 'decimals': 18},
    '0xC6bDb96E29c38DC43f014Eed44dE4106A6A8eB5f': {'symbol': 'INUINU', 'decimals': 18},
    '0xEA1ea0972fa092dd463f2968F9bB51Cc4c981D71': {'symbol': 'MOD', 'decimals': 18},
    '0x741b0428Efdf4372A8DF6FB54B018dB5e5aB7710': {'symbol': 'ARTX', 'decimals': 18},
    '0x397Deb686C72384FAd502A81f4d7fDb89e1f1280': {'symbol': 'XELS', 'decimals': 8},
    '0xD71eCFF9342A5Ced620049e616c5035F1dB98620': {'symbol': 'SEUR', 'decimals': 18},
    '0xEf27252B567F6B3fe35b34A85bE322917abE524A': {'symbol': 'FLONA', 'decimals': 18},
    '0x71Fc1F555a39E0B698653AB0b475488EC3c34D57': {'symbol': 'RAIN', 'decimals': 18},
    '0x2eDf094dB69d6Dcd487f1B3dB9febE2eeC0dd4c5': {'symbol': 'ZEE', 'decimals': 18},
    '0x36905Fc93280f52362A1CBAB151F25DC46742Fb5': {'symbol': 'BTO', 'decimals': 18},
    '0x5b1D655C93185b06B00f7925791106132Cb3ad75': {'symbol': 'DMT', 'decimals': 18},
    '0x6399C842dD2bE3dE30BF99Bc7D1bBF6Fa3650E70': {'symbol': 'PREMIA', 'decimals': 18},
    '0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39': {'symbol': 'HEX', 'decimals': 8},
    '0x5a3e6A77ba2f983eC0d371ea3B475F8Bc0811AD5': {'symbol': '0X0', 'decimals': 9},
    '0x831091dA075665168E01898c6DAC004A867f1e1B': {'symbol': 'GFARM2', 'decimals': 18},
    '0xc713e5E149D5D0715DcD1c156a020976e7E56B88': {'symbol': 'AMKR', 'decimals': 18},
    '0xD04E772BC0d591fBD288f2E2a86aFA3D3CB647F8': {'symbol': 'GPT', 'decimals': 18},
    '0xC05d14442A510De4D3d71a3d316585aA0CE32b50': {'symbol': 'LINA', 'decimals': 18},
    '0xF45091f25d374BbE956c0bb64bB85e02D07Aa741': {'symbol': 'MNMC', 'decimals': 8},
    '0x58f7345b5295E43aA454911571f13be186655BE9': {'symbol': 'GRLC', 'decimals': 8},
    '0xC31CEBf8F9E825d1D1244D73d0a65e44bD5210DB': {'symbol': 'CRYN', 'decimals': 8},
    '0x1EA48B9965bb5086F3b468E50ED93888a661fc17': {'symbol': 'MON', 'decimals': 18},
    '0xEA26c4aC16D4a5A106820BC8AEE85fd0b7b2b664': {'symbol': 'QKC', 'decimals': 18},
    '0xBcc66ed2aB491e9aE7Bf8386541Fb17421Fa9d35': {'symbol': 'SKULL', 'decimals': 4},
    '0x4CC19356f2D37338b9802aa8E8fc58B0373296E7': {'symbol': 'KEY', 'decimals': 18},
    '0x08AD83D779BDf2BBE1ad9cc0f78aa0D24AB97802': {'symbol': 'RWS', 'decimals': 18},
    '0xa685a61171bb30d4072B338c80Cb7b2c865c873E': {'symbol': 'AMANA', 'decimals': 18},
    '0xdf1A2f85F3af80E85d14DdAAB2933C8caA09294a': {'symbol': 'DWAGON', 'decimals': 18},
    '0xB1f1F47061A7Be15C69f378CB3f69423bD58F2F8': {'symbol': 'FLASH', 'decimals': 18},
    '0x27B5739e22ad9033bcBf192059122d163b60349D': {'symbol': 'ST-YCRV', 'decimals': 18},
    '0x874D4C9B980f1a13dD44CBcDB912e24Ef0671eD0': {'symbol': 'GDR', 'decimals': 18},
    '0xC795fBa221f7920F1C6ac0f1598886742D8Ea661': {'symbol': 'SAITAX', 'decimals': 9},
    '0x7616113782AaDAB041d7B10d474F8A0c04EFf258': {'symbol': 'VEE', 'decimals': 18},
    '0xaC6Df26a590F08dcC95D5a4705ae8abbc88509Ef': {'symbol': 'AENJ', 'decimals': 18},
    '0xaCbd826394189Cf2623C6DF98a18b41fC8fFC16D': {'symbol': 'N1', 'decimals': 18},
    '0xEEd3aE7b0F8b5B9BB8C035A9941382B1822671CD': {'symbol': 'EVY', 'decimals': 12},
    '0x1Da87b114f35E1DC91F72bF57fc07A768Ad40Bb0': {'symbol': 'EQZ', 'decimals': 18},
    '0xAcfa209Fb73bF3Dd5bBfb1101B9Bc999C49062a5': {'symbol': 'BCDT', 'decimals': 18},
    '0x2F57430a6ceDA85a67121757785877b4a71b8E6D': {'symbol': 'DFP2', 'decimals': 18},
    '0xE4f726Adc8e89C6a6017F01eadA77865dB22dA14': {'symbol': 'BCP', 'decimals': 18},
    '0x3abF2A4f8452cCC2CF7b4C1e4663147600646f66': {'symbol': 'JBX', 'decimals': 18},
    '0x4104b135DBC9609Fc1A9490E61369036497660c8': {'symbol': 'APW', 'decimals': 18},
    '0xf9d4DAaE1300CfF251979722C4A3C45857973079': {'symbol': 'CASTLE', 'decimals': 18},
    '0xB69753c06BB5c366BE51E73bFc0cC2e3DC07E371': {'symbol': 'POOH', 'decimals': 18},
    '0x13A0599c493Cc502ED8C4Dd26F22F5cEDC248fC4': {'symbol': 'EMP', 'decimals': 18},
    '0x9F7fC686CfD64aA5Ae15b351d03071e91533094b': {'symbol': 'TRACE', 'decimals': 18},
    '0x8f828a0644f12Fa352888E645a90333d30f6FD7D': {'symbol': 'RINIA', 'decimals': 8},
    '0x419D0d8BdD9aF5e606Ae2232ed285Aff190E711b': {'symbol': 'FUN', 'decimals': 8},
    '0xf04aF3f4E4929F7CD25A751E6149A3318373d4FE': {'symbol': 'SPRING', 'decimals': 18},
    '0xC86D054809623432210c107af2e3F619DcFbf652': {'symbol': 'UPP', 'decimals': 18},
    '0x9994E35Db50125E0DF82e4c2dde62496CE330999': {'symbol': 'MORPHO', 'decimals': 18},
    '0xAB846Fb6C81370327e784Ae7CbB6d6a6af6Ff4BF': {'symbol': 'PAL', 'decimals': 18},
    '0x7d647b1A0dcD5525e9C6B3D14BE58f27674f8c95': {'symbol': 'BYTES', 'decimals': 18},
    '0x3A8cCCB969a61532d1E6005e2CE12C200caeCe87': {'symbol': 'TITAN', 'decimals': 18},
    '0xCA1308E38340C69E848061aA2C3880daeB958187': {'symbol': 'HLD', 'decimals': 9},
    '0xd79F43113B22D1eA9F29cfcC7BB287489F8EE5e0': {'symbol': 'ZRO', 'decimals': 18},
    '0x42b91f1d05afeA671a2dA3c780eDa2aBf0A2A366': {'symbol': 'MNB', 'decimals': 18},
    '0x08f5a9235B08173b7569F83645d2c7fB55e8cCD8': {'symbol': 'TNT', 'decimals': 8},
    '0x436dA116249044E8B4464F0Cf21Dd93311d88190': {'symbol': 'ZEUM', 'decimals': 18},
    '0x2FdA8c6783Aa36BeD645baD28a4cDC8769dCD252': {'symbol': 'D2O', 'decimals': 18},
    '0xF629cBd94d3791C9250152BD8dfBDF380E2a3B9c': {'symbol': 'ENJ', 'decimals': 18},
    '0xf1B99e3E573A1a9C5E6B2Ce818b617F0E664E86B': {'symbol': 'OSQTH', 'decimals': 18},
    '0x2565ae0385659badCada1031DB704442E1b69982': {'symbol': 'ASM', 'decimals': 18},
    '0xE9A95d175a5f4C9369f3b74222402eB1b837693b': {'symbol': 'NOW', 'decimals': 8},
    '0xAaEf88cEa01475125522e117BFe45cF32044E238': {'symbol': 'GF', 'decimals': 18},
    '0x009178997aFf09A67D4cACcFeB897Fb79d036214': {'symbol': '1SOL', 'decimals': 18},
    '0xe831F96A7a1DcE1aa2EB760b1e296c6A74CaA9d5': {'symbol': 'NEXM', 'decimals': 8},
    '0x73C69d24ad28e2d43D03CBf35F79fE26EBDE1011': {'symbol': 'ARCH', 'decimals': 18},
    '0x217ddEad61a42369A266F1Fb754EB5d3EBadc88a': {'symbol': 'DON', 'decimals': 18},
    '0x37FC4b48CE93469dbEA9918468993C735049642a': {'symbol': 'CBX', 'decimals': 18},
    '0xF9Ca9523E5b5A42C3018C62B084Db8543478C400': {'symbol': 'LAKE', 'decimals': 18},
    '0x38029C62DfA30D9FD3CaDf4C64e9b2ab21DbDa17': {'symbol': 'DUBBZ', 'decimals': 18},
    '0x2d94AA3e47d9D5024503Ca8491fcE9A2fB4DA198': {'symbol': 'BANK', 'decimals': 18},
    '0x0B452278223D3954F4AC050949D7998e373e7E43': {'symbol': 'SUZUME', 'decimals': 18},
    '0x8dB1D28Ee0d822367aF8d220C0dc7cB6fe9DC442': {'symbol': 'ETHPAD', 'decimals': 18},
    '0x865377367054516e17014CcdED1e7d814EDC9ce4': {'symbol': 'DOLA', 'decimals': 18},
    '0xBA41Ddf06B7fFD89D1267b5A93BFeF2424eb2003': {'symbol': 'MYTH', 'decimals': 18},
    '0x5D64D850c8368008aFB39224E92aD0DcEFf3CF38': {'symbol': 'MIN', 'decimals': 18},
    '0x2965395F71B7d97ede251E9B63e44dfA9647cC0A': {'symbol': 'SHINSHU', 'decimals': 18},
    '0xcf4C68Db4C2FA0bf58dF07B14f45cE7709a716AC': {'symbol': 'SHIELD', 'decimals': 18},
    '0xEC363faa5c4dd0e51f3D9B5d0101263760E7cdeB': {'symbol': 'IWBTC', 'decimals': 8},
    '0x873fB544277FD7b977B196a826459a69E27eA4ea': {'symbol': 'YVRAI', 'decimals': 18},
    '0x62d3c05b9C3d916FBC111819bbD3CEE52906C1Ae': {'symbol': 'EGAME', 'decimals': 18},
    '0xd4126f195a8de772EeFfa61a4aB6dd43462F4e39': {'symbol': 'HIKARI', 'decimals': 18},
    '0x953Cd009a490176FcEB3a26b9753e6F01645ff28': {'symbol': 'XDEUS', 'decimals': 18},
    '0xaA8330FB2B4D5D07ABFE7A72262752a8505C6B37': {'symbol': 'POLC', 'decimals': 18},
    '0x31ea0de8119307aA264Bb4b38727aAb4E36b074f': {'symbol': 'STORE', 'decimals': 18},
    '0x274E7Eb07B485CFdE53d02270555213447570aC6': {'symbol': 'GOV', 'decimals': 18},
    '0x220B71671b649c03714dA9c621285943f3cbcDC6': {'symbol': 'DIS', 'decimals': 18},
    '0xd98F75b1A3261dab9eEd4956c93F33749027a964': {'symbol': 'SHR', 'decimals': 2},
    '0xaBd4dc8fDe9848CBc4Ff2c0Ee81d4A49F4803Da4': {'symbol': 'SQUEEZE', 'decimals': 9},
    '0xF55a93b613D172b86c2Ba3981a849DaE2aeCDE2f': {'symbol': 'YFX', 'decimals': 18},
    '0x1DE5e000C41C8d35b9f1f4985C23988f05831057': {'symbol': 'BNF', 'decimals': 18},
    '0x426FC8BE95573230f6e6bc4af91873F0c67b21b4': {'symbol': 'BPLC', 'decimals': 18},
    '0x488E0369f9BC5C40C002eA7c1fe4fd01A198801c': {'symbol': 'GOF', 'decimals': 18},
    '0xC08512927D12348F6620a698105e1BAac6EcD911': {'symbol': 'GYEN', 'decimals': 6},
    '0x3AFA1902b1f8a802aBC18e5aD982D1bCd34AfE22': {'symbol': 'GST', 'decimals': 18},
    '0x0b5326Da634f9270FB84481DD6F94d3dC2cA7096': {'symbol': 'ETHO', 'decimals': 18},
    '0x40a9d39aa50871Df092538c5999b107f34409061': {'symbol': 'IDAI', 'decimals': 18},
    '0x53C8395465A84955c95159814461466053DedEDE': {'symbol': 'DG', 'decimals': 18},
    '0x8FfE40A3D0f80C0CE6b203D5cDC1A6a86d9AcaeA': {'symbol': 'IGG', 'decimals': 6},
    '0xFde00bD1Da57349b0DF85F134b18124266F3bC5b': {'symbol': 'DIA', 'decimals': 18},
    '0x318Ee488AF8881f9945B6d3D69e8B395Fb559bB1': {'symbol': 'MONTE', 'decimals': 18},
    '0x075b1bb99792c9E1041bA13afEf80C91a1e70fB3': {'symbol': 'CRVRENWSBTC', 'decimals': 18},
    '0x3a3A65aAb0dd2A17E3F1947bA16138cd37d08c04': {'symbol': 'AETH', 'decimals': 18},
    '0x45804880De22913dAFE09f4980848ECE6EcbAf78': {'symbol': 'PAXG', 'decimals': 18},
    '0x5380442d3C4EC4f5777f551f5EDD2FA0F691A27C': {'symbol': 'LOVE', 'decimals': 18},
    '0xa7A5c1058194Af8F00c187adB7FcC0c95f1C6c2d': {'symbol': 'SPIZ', 'decimals': 18},
    '0x1cEB5cB57C4D4E2b2433641b95Dd330A33185A44': {'symbol': 'KP3R', 'decimals': 18},
    '0xC5bcc8BA3F33Ab0d64f3473e861BDC0685b19Ef5': {'symbol': 'MECHA', 'decimals': 18},
    '0x53884b61963351C283118a8E1Fc05BA464a11959': {'symbol': 'MNS', 'decimals': 18},
    '0x95aA5d2DbD3c16ee3fdea82D5C6EC3E38CE3314f': {'symbol': 'PXP', 'decimals': 18},
    '0x591127253E40d4f63bF29CcF3D81FD062A149C8c': {'symbol': 'XMETA', 'decimals': 18},
    '0x308516e6dAaC88d719Af5a6D7eA2A82d8C8e7356': {'symbol': 'MSG', 'decimals': 18},
    '0x7866E48C74CbFB8183cd1a929cd9b95a7a5CB4F4': {'symbol': 'KIT', 'decimals': 18},
    '0x8EE325AE3E54e83956eF2d5952d3C8Bc1fa6ec27': {'symbol': 'TYRANT', 'decimals': 9},
    '0x9C5673002d854de18e9F6Feb7f9825F88c5bf1E4': {'symbol': 'TERRAFORM', 'decimals': 18},
    '0x9EA3b5b4EC044b70375236A281986106457b20EF': {'symbol': 'DELTA', 'decimals': 18},
    '0x33D203FA03bb30b133De0fE2d6533C268bA286B6': {'symbol': 'MANDOX', 'decimals': 9},
    '0xE38B72d6595FD3885d1D2F770aa23E94757F91a1': {'symbol': 'TCR', 'decimals': 8},
    '0x69bBC3F8787d573F1BBDd0a5f40C7bA0Aee9BCC9': {'symbol': 'YUP', 'decimals': 18},
    '0xD15A1A2A3211b58113e45809f05934252E34e2F8': {'symbol': 'WZM', 'decimals': 18},
    '0x519C1001D550C0a1DaE7d1fC220f7d14c2A521BB': {'symbol': 'PSWAP', 'decimals': 18},
    '0xF6650117017FFD48B725B4EC5A00B414097108A7': {'symbol': 'XIDO', 'decimals': 18},
    '0xe7F72Bc0252cA7B16dBb72eEee1AfcDb2429F2DD': {'symbol': 'NFTL', 'decimals': 18},
    '0xbcD4b7dE6fde81025f74426D43165a5b0D790Fdd': {'symbol': 'SPDR', 'decimals': 18},
    '0xc314b0E758D5FF74f63e307A86EbfE183C95767b': {'symbol': 'ADP', 'decimals': 18},
    '0xcbee6459728019CB1f2bB971dDe2eE3271BC7617': {'symbol': 'MRG', 'decimals': 18},
    '0x082716b6734b31791407d7Dd0e2a2c41260029b2': {'symbol': 'HICOOLCATS', 'decimals': 18},
    '0x2047ab3072B52561596Ce5E0131BDbB7C848538D': {'symbol': 'BORED', 'decimals': 9},
    '0x0E29e5AbbB5FD88e28b2d355774e73BD47dE3bcd': {'symbol': 'HAKKA', 'decimals': 18},
    '0x884DDBb5DC6c2cEf77d3E74c6CcCa315797d655B': {'symbol': 'CARR', 'decimals': 18},
    '0xEa5eDef1287AfDF9Eb8A46f9773AbFc10820c61c': {'symbol': 'EASE', 'decimals': 18},
    '0x2DCA19E944453e46d9130950Ca135461b3Bc0c30': {'symbol': 'EYES', 'decimals': 18},
    '0x01022591fCe0609B57670bebA60102dFd2b95D77': {'symbol': 'HITOP', 'decimals': 18},
    '0x25e1474170c4c0aA64fa98123bdc8dB49D7802fa': {'symbol': 'BID', 'decimals': 18},
    '0xE1BA0FB44CCb0D11b80F92f4f8Ed94CA3fF51D00': {'symbol': 'ABAT', 'decimals': 18},
    '0x9559Aaa82d9649C7A7b220E7c461d2E74c9a3593': {'symbol': 'RETH', 'decimals': 18},
    '0xFf44b937788215ecA197BAaf9AF69dbdC214aa04': {'symbol': 'ROCKI', 'decimals': 18},
    '0x15Ee120fD69BEc86C1d38502299af7366a41D1a6': {'symbol': 'BITANT', 'decimals': 18},
    '0x4612021C75809160Be60DB21fBc9D6adD0B32deF': {'symbol': 'BCP', 'decimals': 18},
    '0x4384b85FE228AE727B129230211194E4A50877c4': {'symbol': 'TAIL', 'decimals': 9},
    '0x20BC832ca081b91433ff6c17f85701B6e92486c5': {'symbol': 'RETH2', 'decimals': 18},
    '0x66C0DDEd8433c9EA86C8cf91237B14e10b4d70B7': {'symbol': 'MARS', 'decimals': 18},
    '0x42dbBd5ae373FEA2FC320F62d44C058522Bb3758': {'symbol': 'MEM', 'decimals': 18},
    '0xB113c6CF239F60D380359b762E95C13817275277': {'symbol': 'BMEX', 'decimals': 6},
    '0x93eEB426782BD88fCD4B48D7b0368CF061044928': {'symbol': 'TRG', 'decimals': 18},
    '0x85Eee30c52B0b379b046Fb0F85F4f3Dc3009aFEC': {'symbol': 'KEEP', 'decimals': 18},
    '0xcb4D749FD1FF081D6fEA3bd978de87603E098E63': {'symbol': 'JENNY', 'decimals': 18},
    '0x0f9b80fc3c8b9123D0aEf43Df58ebDBC034A8901': {'symbol': 'NFTE', 'decimals': 18},
    '0x4159862bcF6B4393A80550b1ed03DFfa6f90533C': {'symbol': 'OHMI', 'decimals': 18},
    '0x033e223870f766644f7f7a4B7dc2E91573707d06': {'symbol': 'ZIN', 'decimals': 18},
    '0xc4De189Abf94c57f396bD4c52ab13b954FebEfD8': {'symbol': 'B20', 'decimals': 18},
    '0x758B4684BE769E92eeFeA93f60DDA0181eA303Ec': {'symbol': 'PHONON', 'decimals': 18},
    '0x55b1e2D8b13E7acad03353FAD58fc3FA065C5822': {'symbol': 'FRZ', 'decimals': 18},
    '0x68909e586eeAC8F47315e84B4c9788DD54Ef65Bb': {'symbol': 'EVN', 'decimals': 18},
    '0x6243d8CEA23066d098a15582d81a598b4e8391F4': {'symbol': 'FLX', 'decimals': 18},
    '0x3f5dd1A1538a4F9f82E543098f01F22480B0A3a8': {'symbol': 'DKUMA', 'decimals': 18},
    '0xBBC7f7A6AADAc103769C66CBC69AB720f7F9Eae3': {'symbol': 'INX', 'decimals': 18},
    '0x4da0C48376C277cdBd7Fc6FdC6936DEE3e4AdF75': {'symbol': 'EPIK', 'decimals': 18},
    '0x2604FA406Be957E542BEb89E6754fCdE6815e83f': {'symbol': 'PKT', 'decimals': 18},
    '0x5A9780Bfe63f3ec57f01b087cD65BD656C9034A8': {'symbol': 'COM', 'decimals': 12},
    '0xC4EE0aA2d993ca7C9263eCFa26c6f7e13009d2b6': {'symbol': 'HOICHI', 'decimals': 18},
    '0xD487892BB4C57edBe7aB401d9fe801c8FE6473f5': {'symbol': 'HVE2', 'decimals': 18},
    '0x69681f8fde45345C3870BCD5eaf4A05a60E7D227': {'symbol': 'IBGBP', 'decimals': 18},
    '0x90B831fa3Bebf58E9744A14D638E25B4eE06f9Bc': {'symbol': 'MIMO', 'decimals': 18},
    '0x8798249c2E607446EfB7Ad49eC89dD1865Ff4272': {'symbol': 'XSUSHI', 'decimals': 18},
    '0xeB953eDA0DC65e3246f43DC8fa13f35623bDd5eD': {'symbol': 'RAINI', 'decimals': 18},
    '0x93ef1Ea305D11A9b2a3EbB9bB4FCc34695292E7d': {'symbol': 'QETH', 'decimals': 18},
    '0x70008F18Fc58928dcE982b0A69C2c21ff80Dca54': {'symbol': 'X7R', 'decimals': 18},
    '0x8fc6a71Be7f82373e045a97859287676a399934A': {'symbol': 'WIX', 'decimals': 18},
    '0xC4f6E93AEDdc11dc22268488465bAbcAF09399aC': {'symbol': 'HI', 'decimals': 18},
    '0x896e145568624a498c5a909187363AE947631503': {'symbol': 'WASABI', 'decimals': 18},
    '0xE94B97b6b43639E238c851A7e693F50033EfD75C': {'symbol': 'RNBW', 'decimals': 18},
    '0x4C25Bdf026Ea05F32713F00f73Ca55857Fbf6342': {'symbol': 'FONT', 'decimals': 18},
    '0xB1e93236ab6073fdAC58adA5564897177D4bcC43': {'symbol': 'SEELE', 'decimals': 18},
    '0xB98d4C97425d9908E66E53A6fDf673ACcA0BE986': {'symbol': 'ABT', 'decimals': 18},
    '0x8578530205CEcbe5DB83F7F29EcfEEC860C297C2': {'symbol': 'AOG', 'decimals': 18},
    '0x6C5Aa4244113E77414386630B7F963dbE1B4d73D': {'symbol': 'DNS', 'decimals': 18},
    '0x64b78325d7495D6d4be92f234fa3f3B8d8964B8b': {'symbol': 'SHOP', 'decimals': 18},
    '0xE89C20096b636fFec9fd26d1a623F42A33eaD309': {'symbol': 'OG', 'decimals': 18},
    '0x34950Ff2b487d9E5282c5aB342d08A2f712eb79F': {'symbol': 'WOZX', 'decimals': 18},
    '0x46ccA329970B33e1a007DD4ef0594A1cedb3E72a': {'symbol': 'YESP', 'decimals': 18},
    '0x6368e1E18c4C419DDFC608A0BEd1ccb87b9250fc': {'symbol': 'XTP', 'decimals': 18},
    '0x4297394c20800E8a38A619A243E9BbE7681Ff24E': {'symbol': 'HOTCROSS', 'decimals': 18},
    '0x235C8EE913d93c68D2902a8e0b5a643755705726': {'symbol': 'BAG', 'decimals': 18},
    '0xD478161C952357F05f0292B56012Cd8457F1cfbF': {'symbol': 'POLK', 'decimals': 18},
    '0x9eFdFC77017cB9b878300eCdcA21fAB0529a22A0': {'symbol': 'AKI', 'decimals': 9},
    '0x823E1B82cE1Dc147Bbdb25a203f046aFab1CE918': {'symbol': 'COIL', 'decimals': 18},
    '0x2653891204F463fb2a2F4f412564b19e955166aE': {'symbol': 'NGL', 'decimals': 18},
    '0x8a3d77e9d6968b780564936d15B09805827C21fa': {'symbol': 'UCO', 'decimals': 18},
    '0x4b520c812E8430659FC9f12f6d0c39026C83588D': {'symbol': 'DG', 'decimals': 18},
    '0xD5d86FC8d5C0Ea1aC1Ac5Dfab6E529c9967a45E9': {'symbol': 'WRLD', 'decimals': 18},
    '0x89Fb927240750c1B15d4743cd58440fc5f14A11C': {'symbol': 'ATT', 'decimals': 18},
    '0x45734927Fa2f616FbE19E65f42A0ef3d37d1c80A': {'symbol': 'ROGE', 'decimals': 9},
    '0x37C997B35C619C21323F3518B9357914E8B99525': {'symbol': 'PILOT', 'decimals': 18},
    '0x8Af78f0c818302164F73B2365fE152C2D1FE80e1': {'symbol': 'FNCT', 'decimals': 18},
    '0x4485561Db76614Ff727f8E0a3Ea95690b8b16022': {'symbol': 'INVOX', 'decimals': 18},
    '0x7Cc97Bf17C5AdaBe25F9D19D15A1eC8A1ad65f14': {'symbol': 'WOLVERINU', 'decimals': 18},
    '0x06e0feB0D74106c7adA8497754074D222Ec6BCDf': {'symbol': 'BTB', 'decimals': 18},
    '0xAD83b92cdD542Db07445597f7F06963B31Cf9444': {'symbol': 'BUMBLEC', 'decimals': 9},
    '0xB045f7f363fE4949954811b113bd56d208c67B23': {'symbol': 'SILK', 'decimals': 8},
    '0x11613b1f840bb5A40F8866d857e24DA126B79D73': {'symbol': 'CAPP', 'decimals': 2},
    '0x2596825a84888E8f24b747DF29e11b5dd03C81D7': {'symbol': 'FTRB', 'decimals': 18},
    '0xc691bc298a304d591Ad9b352c7A8D216dE9f2CED': {'symbol': 'POLA', 'decimals': 18},
    '0xA3665c69213BB9AffCf92AB90100fBe273e3aB74': {'symbol': 'DRNT', 'decimals': 18},
    '0xADe6FDAba1643E4D1eeF68Da7170F234470938c6': {'symbol': 'HARAMBE', 'decimals': 18},
    '0xEC681F28f4561c2a9534799AA38E0d36A83Cf478': {'symbol': 'YVS', 'decimals': 18},
    '0x9Ed8e7C9604790F7Ec589F99b94361d8AAB64E5E': {'symbol': 'UNISTAKE', 'decimals': 18},
    '0xEE06A81a695750E71a662B51066F2c74CF4478a0': {'symbol': 'DG', 'decimals': 18},
    '0x6c28AeF8977c9B773996d0e8376d2EE379446F2f': {'symbol': 'QUICK', 'decimals': 18},
    '0x5d65D971895Edc438f465c17DB6992698a52318D': {'symbol': 'NAS', 'decimals': 18},
    '0x28fDA76721a8077A5dE802Ab0212849B8c38429E': {'symbol': 'ARV', 'decimals': 18},
    '0xB8BAa0e4287890a5F79863aB62b7F175ceCbD433': {'symbol': 'SWRV', 'decimals': 18},
    '0x1fc5EF0337AEA85C5f9198853a6E3A579a7A6987': {'symbol': 'REAP', 'decimals': 18},
    '0x8716Fc5Da009D3A208f0178b637a50F4ef42400F': {'symbol': 'UGAS', 'decimals': 18},
    '0xDf801468a808a32656D2eD2D2d80B72A129739f4': {'symbol': 'CUBE', 'decimals': 8},
    '0xB668473944d2E25B6aF6D46917Eb0233DBaC53ae': {'symbol': 'NTO', 'decimals': 18},
    '0xF029Fd6b5ED35C825Dd02B7A02952C30002840E4': {'symbol': 'SIGIL', 'decimals': 18},
    '0x5Eaa69B29f99C84Fe5dE8200340b4e9b4Ab38EaC': {'symbol': 'RAZE', 'decimals': 18},
    '0xdacD69347dE42baBfAEcD09dC88958378780FB62': {'symbol': 'ATRI', 'decimals': 0},
    '0x4156D3342D5c385a87D264F90653733592000581': {'symbol': 'SALT', 'decimals': 8},
    '0x378cb52b00F9D0921cb46dFc099CFf73b42419dC': {'symbol': 'YVLUSD', 'decimals': 18},
    '0x34Be5b8C30eE4fDe069DC878989686aBE9884470': {'symbol': 'SENATE', 'decimals': 18},
    '0x09a3EcAFa817268f77BE1283176B946C4ff2E608': {'symbol': 'MIR', 'decimals': 18},
    '0x441761326490cACF7aF299725B6292597EE822c2': {'symbol': 'UNFI', 'decimals': 18},
    '0x5EA82C27efc7634f1C5Ad20a3561c453433a2f3a': {'symbol': 'BTR', 'decimals': 18},
    '0x116c4B65e14449947bC6fa1bbe844CB16A162d53': {'symbol': 'BMAX', 'decimals': 18},
    '0x358AA737e033F34df7c54306960a38d09AaBd523': {'symbol': 'ARES', 'decimals': 18},
    '0x45448E05020576929fcDEabC228E35b420098840': {'symbol': 'IDV', 'decimals': 18},
    '0x18a3563c21062897950BB09339C82b9686a35667': {'symbol': 'ENXS', 'decimals': 18},
    '0x8Eb94A06b4716093DBfE335cbdb098DEb2dcdE1b': {'symbol': 'SHIB05', 'decimals': 18},
    '0xEb57Bf569Ad976974C1F861a5923A59F40222451': {'symbol': 'LOOMI', 'decimals': 18},
    '0xe4815AE53B124e7263F08dcDBBB757d41Ed658c6': {'symbol': 'ZKS', 'decimals': 18},
    '0x757dA0E5c253082b0f2Bd5105119F71817Fe0911': {'symbol': 'VITO', 'decimals': 18},
    '0xc770EEfAd204B5180dF6a14Ee197D99d808ee52d': {'symbol': 'FOX', 'decimals': 18},
    '0x37fE0f067FA808fFBDd12891C0858532CFE7361d': {'symbol': 'CIV', 'decimals': 18},
    '0xDA0c94c73D127eE191955FB46bACd7FF999b2bcd': {'symbol': 'STANDARD', 'decimals': 18},
    '0x0fD67B4ceb9b607Ef206904eC73459c4880132c9': {'symbol': 'SHOE', 'decimals': 18},
    '0x84fE25f3921f3426395c883707950d0c00367576': {'symbol': 'INX', 'decimals': 18},
    '0x1892F6ff5FBE11c31158F8c6f6F6e33106c5B10E': {'symbol': 'MEGA', 'decimals': 18},
    '0x71E368Ed06814Bf35d4E663eFf946400a5BC8115': {'symbol': 'DGRN', 'decimals': 18},
    '0x4674672BcDdDA2ea5300F5207E1158185c944bc0': {'symbol': 'GXT', 'decimals': 18},
    '0x3656bD0f3f07623Bb7f429B390D208f894e44eCE': {'symbol': 'IFV', 'decimals': 18},
    '0xaD996A45fd2373ed0B10Efa4A8eCB9de445A4302': {'symbol': 'SHI', 'decimals': 18},
    '0x045da4bFe02B320f4403674B3b7d121737727A36': {'symbol': 'DCHF', 'decimals': 18},
    '0xaD22f63404f7305e4713CcBd4F296f34770513f4': {'symbol': 'AWC', 'decimals': 8},
    '0x2e85ae1C47602f7927bCabc2Ff99C40aA222aE15': {'symbol': 'KATA', 'decimals': 18},
    '0x252b9F56359901a0BDE52d0675b1f1130d86F471': {'symbol': 'PANDO', 'decimals': 18},
    '0x8c6Fa66c21aE3fC435790E451946a9Ea82E6e523': {'symbol': 'FABRIC', 'decimals': 18},
    '0xeEAA40B28A2d1b0B08f6f97bB1DD4B75316c6107': {'symbol': 'GOVI', 'decimals': 18},
    '0x19062190B1925b5b6689D7073fDfC8c2976EF8Cb': {'symbol': 'BZZ', 'decimals': 16},
    '0x07D9e49Ea402194bf48A8276dAfB16E4eD633317': {'symbol': 'DALC', 'decimals': 8},
    '0x910524678C0B1B23FFB9285a81f99C29C11CBaEd': {'symbol': 'AZUKI', 'decimals': 18},
    '0x6400B5522f8D448C0803e6245436DD1c81dF09ce': {'symbol': 'CVNT', 'decimals': 8},
    '0xFAce851a4921ce59e912d19329929CE6da6EB0c7': {'symbol': 'CLINK', 'decimals': 8},
    '0x2Dd1B9410c73E16b60240E529e38E30425F00d4E': {'symbol': 'PACHA', 'decimals': 10},
    '0xBC1A4555745e9C5334eBF9DcfEC0B51285aF8d17': {'symbol': 'CATAI', 'decimals': 18},
    '0x93dfaf57D986B9cA77Df9376c50878E013D9c7C8': {'symbol': 'RARE', 'decimals': 18},
    '0x525A8F6F3Ba4752868cde25164382BfbaE3990e1': {'symbol': 'NYM', 'decimals': 6},
    '0xaa0c5B3567fd1bac8a2a11EB16c3F81a49eea90F': {'symbol': 'MMAI', 'decimals': 7},
    '0x4448d67EA26a2Eaf286bc1045005f4CF15aaBa11': {'symbol': 'SJM', 'decimals': 9},
    '0xb3Cb8d5AEFF0F4d1f432f353309f47b885E404E3': {'symbol': 'MEV', 'decimals': 18},
    '0x336F646F87D9f6bC6Ed42Dd46E8b3fD9DbD15C22': {'symbol': 'CCT', 'decimals': 18},
    '0x383518188C0C6d7730D91b2c03a03C837814a899': {'symbol': 'OHM', 'decimals': 9},
    '0x61E90A50137E1F645c9eF4a0d3A4f01477738406': {'symbol': 'LOKA', 'decimals': 18},
    '0x635F15eb7Aa2e62d122f6B1f9f519fDCcf4aBdda': {'symbol': 'HIMAYC', 'decimals': 18},
    '0xC71D8BaAa436aa7E42DA1f40bEc48F11AB3c9E4A': {'symbol': 'IETH', 'decimals': 8},
    '0xc5bDdf9843308380375a611c18B50Fb9341f502A': {'symbol': 'YVE-CRVDAO', 'decimals': 18},
    '0x69fa8e7F6bf1ca1fB0de61e1366f7412b827CC51': {'symbol': 'NRCH', 'decimals': 9},
    '0xFD957F21bd95E723645C07C48a2d8ACB8Ffb3794': {'symbol': 'ETHM', 'decimals': 18},
    '0xA1AFFfE3F4D611d252010E3EAf6f4D77088b0cd7': {'symbol': 'RFI', 'decimals': 9},
    '0xf3Db5Fa2C66B7aF3Eb0C0b782510816cbe4813b8': {'symbol': 'EVX', 'decimals': 4},
    '0x8a7664E782860E856031D6c31EB3bde721Bc362b': {'symbol': 'REKT', 'decimals': 18},
    '0x3E9BC21C9b189C09dF3eF1B824798658d5011937': {'symbol': 'LINA', 'decimals': 18},
    '0x465a5a630482f3abD6d3b84B39B29b07214d19e5': {'symbol': 'FUSDC', 'decimals': 8},
    '0x0391D2021f89DC339F60Fff84546EA23E337750f': {'symbol': 'BOND', 'decimals': 18},
    '0x6307B25A665Efc992EC1C1bC403c38F3dDd7c661': {'symbol': 'GCR', 'decimals': 4},
    '0x9b62Ec1453cEa5Dde760AAf662048cA6eEB66E7f': {'symbol': 'ECELL', 'decimals': 2},
    '0xc22B30E4cce6b78aaaADae91E44E73593929a3e9': {'symbol': 'RAC', 'decimals': 18},
    '0xB30F5d11b94efBBfDeaa4de38eDFFCeEc0bE6513': {'symbol': 'PIF', 'decimals': 18},
    '0x579CEa1889991f68aCc35Ff5c3dd0621fF29b0C9': {'symbol': 'IQ', 'decimals': 18},
    '0xc55c2175E90A46602fD42e931f62B3Acc1A013Ca': {'symbol': 'STARS', 'decimals': 18},
    '0xe4dae00bc1c46ea2F44ae71B1BEb8b171C15d812': {'symbol': 'PRMX', 'decimals': 18},
    '0xC4c346eDc55504574cCEB00AA1091d22404A4bC3': {'symbol': 'MEZZ', 'decimals': 18},
    '0x83984d6142934bb535793A82ADB0a46EF0F66B6d': {'symbol': 'REM', 'decimals': 4},
    '0x3C6ff50c9Ec362efa359317009428d52115fe643': {'symbol': 'PERX', 'decimals': 18},
    '0x28cb7e841ee97947a86B06fA4090C8451f64c0be': {'symbol': 'YFL', 'decimals': 18},
    '0xaa602dE53347579f86b996D2Add74bb6F79462b2': {'symbol': 'ZMT', 'decimals': 18},
    '0x1F3f9D3068568F8040775be2e8C03C103C61f3aF': {'symbol': 'ARCH', 'decimals': 18},
    '0x17EF75AA22dD5f6C2763b8304Ab24f40eE54D48a': {'symbol': 'RVP', 'decimals': 18},
    '0x0778Cc2E8bBAd3d483E82371606D100cc8604522': {'symbol': 'LHCOIN', 'decimals': 8},
    '0x97de57eC338AB5d51557DA3434828C5DbFaDA371': {'symbol': 'EUSD', 'decimals': 18},
    '0x72108a8CC3254813C6BE2F1b77be53E185abFdD9': {'symbol': 'ES', 'decimals': 18},
    '0x24dA31e7BB182cb2cABfEF1d88db19C2AE1f5572': {'symbol': 'SHIK', 'decimals': 18},
    '0xFeeeef4D7b4Bf3cc8BD012D02D32Ba5fD3D51e31': {'symbol': 'TAIL', 'decimals': 18},
    '0xCae72A7A0Fd9046cf6b165CA54c9e3a3872109E0': {'symbol': 'ANRX', 'decimals': 18},
    '0x961C8c0B1aaD0c0b10a51FeF6a867E3091BCef17': {'symbol': 'DYP', 'decimals': 18},
    '0xc5fB36dd2fb59d3B98dEfF88425a3F425Ee469eD': {'symbol': 'TSUKA', 'decimals': 9},
    '0x6628606c321FaF52b7230A57b26c01B19aA68e82': {'symbol': 'BT', 'decimals': 18},
    '0xa5f2211B9b8170F694421f2046281775E8468044': {'symbol': 'THOR', 'decimals': 18},
    '0x543Ff227F64Aa17eA132Bf9886cAb5DB55DCAddf': {'symbol': 'GEN', 'decimals': 18},
    '0x73A83269b9bbAFC427E76Be0A2C1a1db2a26f4C2': {'symbol': '0NE', 'decimals': 18},
    '0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643': {'symbol': 'CDAI', 'decimals': 8},
    '0xd5281BB2d1eE94866B03A0fcCDd4e900c8Cb5091': {'symbol': 'HUSKY', 'decimals': 9},
    '0x068E3563b1c19590F822c0e13445c4FA1b9EEFa5': {'symbol': 'WUSD', 'decimals': 18},
    '0xE5A733681bbE6cd8c764Bb8078ef8E13a576Dd78': {'symbol': 'DPAY', 'decimals': 18},
    '0xEd3D4e446A96dC3b181B64B75c3c70dA41dC3CBe': {'symbol': 'VDR', 'decimals': 18},
    '0x06677Dc4fE12d3ba3C7CCfD0dF8Cd45e4D4095bF': {'symbol': 'WQT', 'decimals': 18},
    '0x8BCbef61ACd66537362f38167F11875134FfcD63': {'symbol': 'PEPEG', 'decimals': 18},
    '0x04abEdA201850aC0124161F037Efd70c74ddC74C': {'symbol': 'NEST', 'decimals': 18},
    '0x3472A5A71965499acd81997a54BBA8D852C6E53d': {'symbol': 'BADGER', 'decimals': 18},
    '0x06A00715E6F92210Af9d7680B584931FAF71A833': {'symbol': 'XNL', 'decimals': 18},
    '0x973e52691176d36453868D9d86572788d27041A9': {'symbol': 'DX', 'decimals': 18},
    '0x745407c86DF8DB893011912d3aB28e68B62E49B0': {'symbol': 'MAHA', 'decimals': 18},
    '0x67c597624B17b16fb77959217360B7cD18284253': {'symbol': 'MARK', 'decimals': 9},
    '0x7A8Ca2f815A260660158a38C34ca321A3605eCFE': {'symbol': 'BIZZ', 'decimals': 8},
    '0xc3D088842DcF02C13699F936BB83DFBBc6f721Ab': {'symbol': 'VETH', 'decimals': 18},
    '0xFd09911130e6930Bf87F2B0554c44F400bD80D3e': {'symbol': 'ETHIX', 'decimals': 18},
    '0xa23C1194d421F252b4e6D5edcc3205F7650a4eBE': {'symbol': 'LBP', 'decimals': 18},
    '0x5a7E6C8204A1359DB9AAcab7bA5Fc309B7981eFd': {'symbol': 'ANONUSD', 'decimals': 18},
    '0x0eDF9bc41Bbc1354c70e2107F80C42caE7FBBcA8': {'symbol': 'STRM', 'decimals': 18},
    '0xC57d533c50bC22247d49a368880fb49a1caA39F7': {'symbol': 'PTF', 'decimals': 18},
    '0x284b59cf2539544559C6EfA11E2795e06D535345': {'symbol': 'DAOVC', 'decimals': 18},
    '0xe516D78d784C77D479977BE58905B3f2b1111126': {'symbol': 'SPWN', 'decimals': 18},
    '0x00D1793D7C3aAE506257Ba985b34C76AaF642557': {'symbol': 'TACO', 'decimals': 18},
    '0xf7DD746A613Fb6362D44eCEDEB743f62ade6c3aa': {'symbol': 'FIFTY', 'decimals': 9},
    '0xdcD85914b8aE28c1E62f1C488E1D968D5aaFfE2b': {'symbol': 'TOP', 'decimals': 18},
    '0x671a912C10bba0CFA74Cfc2d6Fba9BA1ed9530B2': {'symbol': 'YVLINK', 'decimals': 18},
    '0x595832F8FC6BF59c85C527fEC3740A1b7a361269': {'symbol': 'POWR', 'decimals': 6},
    '0x91Af0fBB28ABA7E31403Cb457106Ce79397FD4E6': {'symbol': 'AERGO', 'decimals': 18},
    '0x0aA7eFE4945Db24d95cA6E117BBa65Ed326e291A': {'symbol': 'OJA', 'decimals': 18},
    '0xf2A22B900dde3ba18Ec2AeF67D4c8C1a0DAB6aAC': {'symbol': 'MONKEYS', 'decimals': 9},
    '0x7DAc25b1A665e1c70F25F1fC37d88C99274984ed': {'symbol': 'SHINJA', 'decimals': 9},
    '0x9D9e399e5385e2b9A58d4F775A1E16441b571afb': {'symbol': 'METANO', 'decimals': 18},
    '0xB37a769B37224449d92AAc57dE379E1267Cd3B00': {'symbol': 'COVA', 'decimals': 18},
    '0x6149C26Cd2f7b5CCdb32029aF817123F6E37Df5B': {'symbol': 'LPOOL', 'decimals': 18},
    '0x4BE10dA47A07716af28Ad199FbE020501BddD7aF': {'symbol': 'XT', 'decimals': 18},
    '0xc221b7E65FfC80DE234bbB6667aBDd46593D34F0': {'symbol': 'WCFG', 'decimals': 18},
    '0xC229c69eB3BB51828D0cAA3509A05a51083898dd': {'symbol': 'PTU', 'decimals': 18},
    '0x35e78b3982E87ecfD5b3f3265B601c046cDBe232': {'symbol': 'XAI', 'decimals': 18},
    '0x5CA9a71B1d01849C0a95490Cc00559717fCF0D1d': {'symbol': 'AE', 'decimals': 18},
    '0x841FB148863454A3b3570f515414759BE9091465': {'symbol': 'SHIH', 'decimals': 18},
    '0xa0F0546Eb5E3eE7e8cfC5DA12e5949F3AE622675': {'symbol': 'TOKO', 'decimals': 18},
    '0xc7283b66Eb1EB5FB86327f08e1B5816b0720212B': {'symbol': 'TRIBE', 'decimals': 18},
    '0x1a16B2B93B37A7EF5cB0A4F978CeD45FE14a5486': {'symbol': 'COLLAR', 'decimals': 18},
    '0x85f6eB2BD5a062f5F8560BE93FB7147e16c81472': {'symbol': 'FLY', 'decimals': 4},
    '0x8dfc8cc3201425669FaE803e1eB125cddd4189eC': {'symbol': 'OKAGE', 'decimals': 18},
    '0x01fF50f8b7f74E4f00580d9596cd3D0d6d6E326f': {'symbol': 'BFT', 'decimals': 18},
    '0x798D1bE841a82a273720CE31c822C61a67a601C3': {'symbol': 'DIGG', 'decimals': 9},
    '0x8e0E57DCb1ce8d9091dF38ec1BfC3b224529754A': {'symbol': 'CAH', 'decimals': 18},
    '0xa62cc35625B0C8dc1fAEA39d33625Bb4C15bD71C': {'symbol': 'STMX', 'decimals': 18},
    '0x95eFD1Fe6099F65a7ED524DEF487483221094947': {'symbol': 'CBM', 'decimals': 18},
    '0x4ABB9cC67BD3da9Eb966d1159A71a0e68BD15432': {'symbol': 'KEL', 'decimals': 18},
    '0xFA99A87b14B02e2240C79240C5a20F945ca5EF76': {'symbol': 'GGTK', 'decimals': 18},
    '0x5228a22e72ccC52d415EcFd199F99D0665E7733b': {'symbol': 'PBTC', 'decimals': 18},
    '0x10633216E7E8281e33c86F02Bf8e565a635D9770': {'symbol': 'DVI', 'decimals': 18},
    '0x67d85A291fcDC862A78812a3C26d55e28FFB2701': {'symbol': 'ASX', 'decimals': 18},
    '0xc50EF449171a51FbeAFd7c562b064B6471C36caA': {'symbol': 'ZINU', 'decimals': 9},
    '0x72De803b67B6AB05B61EFab2Efdcd414D16eBF6D': {'symbol': 'LED', 'decimals': 18},
    '0x7449c93ABe66457e83B3799BCf1A99e92D58A93B': {'symbol': 'CYMI', 'decimals': 18},
    '0x2731d151CBDf84A8A4C6d9D0BaE74012Db51E428': {'symbol': 'IFT', 'decimals': 18},
    '0xF5a56bD9Ee4a3E2D493ab8e072658794B38b6d19': {'symbol': 'SHFT', 'decimals': 18},
    '0x0202Be363B8a4820f3F4DE7FaF5224fF05943AB1': {'symbol': 'UFT', 'decimals': 18},
    '0x4740735AA98Dc8aa232BD049f8F0210458E7fCa3': {'symbol': 'RDT', 'decimals': 18},
    '0x221657776846890989a759BA2973e427DfF5C9bB': {'symbol': 'REP', 'decimals': 18},
    '0x62199B909FB8B8cf870f97BEf2cE6783493c4908': {'symbol': 'PBTC', 'decimals': 18},
    '0x6cD079F296c10EB84A64239b893D4EdfA2E1d779': {'symbol': 'PLSCX', 'decimals': 18},
    '0x72e364F2ABdC788b7E918bc238B21f109Cd634D7': {'symbol': 'MVI', 'decimals': 18},
    '0x93ED3FBe21207Ec2E8f2d3c3de6e058Cb73Bc04d': {'symbol': 'PNK', 'decimals': 18},
    '0xBDCFbf5C4D91Abc0bC9709C7286d00063c0e6F22': {'symbol': 'GUESS', 'decimals': 2},
    '0xf720E38F678B29B243F7D53B56Acbf5dE98F2385': {'symbol': 'UPR', 'decimals': 18},
    '0xf5b1Fd29d23e98Db2D9EBb8435E1082e3B38FB65': {'symbol': 'KISHIMOTO', 'decimals': 9},
    '0xc67B12049c2D0CF6e476BC64c7F82fc6C63cFFc5': {'symbol': 'GDT', 'decimals': 8},
    '0x26CB3641aaA43911f1D4cB2ce544eb652AAc7c47': {'symbol': 'CYL', 'decimals': 18},
    '0x38A2fDc11f526Ddd5a607C1F251C065f40fBF2f7': {'symbol': 'PHNX', 'decimals': 18},
    '0x8dAE6Cb04688C62d939ed9B68d32Bc62e49970b1': {'symbol': 'ACRV', 'decimals': 18},
    '0x7237C0B30B1355f1B76355582f182F6f04B08740': {'symbol': 'MGG', 'decimals': 18},
    '0xBEA0000029AD1c77D3d5D23Ba2D8893dB9d1Efab': {'symbol': 'BEAN', 'decimals': 6},
    '0xD0Cd466b34A24fcB2f87676278AF2005Ca8A78c4': {'symbol': 'POP', 'decimals': 18},
    '0xC74B43cC61b627667a608c3F650c2558F88028a1': {'symbol': 'ASTRAL', 'decimals': 18},
    '0x7ECbb21346C501Fd07eb165E406120fA32381C16': {'symbol': 'ECOREAL', 'decimals': 18},
    '0x7B32e70e8D73ac87c1B342e063528B2930b15ceB': {'symbol': 'RBIF', 'decimals': 9},
    '0xe8E06a5613dC86D459bC8Fb989e173bB8b256072': {'symbol': 'FEY', 'decimals': 18},
    '0xBcca60bB61934080951369a648Fb03DF4F96263C': {'symbol': 'AUSDC', 'decimals': 6},
    '0x9f4909Cc95FB870BF48C128C1Fdbb5F482797632': {'symbol': 'GZLR', 'decimals': 18},
    '0x20e7125677311Fca903A8897042b9983f22Ea295': {'symbol': 'FWT', 'decimals': 18},
    '0x5d929AA919E489505CcAAd8A199619c6DCA0c2de': {'symbol': 'BAAS', 'decimals': 18},
    '0xd7c49CEE7E9188cCa6AD8FF264C1DA2e69D4Cf3B': {'symbol': 'NXM', 'decimals': 18},
    '0x5f190F9082878cA141F858c1c90B4C59fe2782C5': {'symbol': 'KDOE', 'decimals': 18},
    '0x135783B60cf5d71DAFF7a377f9eb7dB8D2dEAb9e': {'symbol': 'CUT', 'decimals': 18},
    '0x46e98FFE40E408bA6412bEb670507e083C8B95ff': {'symbol': 'PRIMATE', 'decimals': 18},
    '0x006BeA43Baa3f7A6f765F14f10A1a1b08334EF45': {'symbol': 'STX', 'decimals': 18},
    '0xff56Cc6b1E6dEd347aA0B7676C85AB0B3D08B0FA': {'symbol': 'ORBS', 'decimals': 18},
    '0xec67005c4E498Ec7f55E092bd1d35cbC47C91892': {'symbol': 'MLN', 'decimals': 18},
    '0x0E5f00DA8AAef196a719d045DB89b5DA8F371b32': {'symbol': 'CNTM', 'decimals': 18},
    '0xE1bAD922F84b198A08292FB600319300ae32471b': {'symbol': 'FCT', 'decimals': 18},
    '0x1117aC6Ad6Cdf1A3BC543baD3B133724620522d5': {'symbol': 'MODA', 'decimals': 18},
    '0x48C3399719B582dD63eB5AADf12A40B4C3f52FA2': {'symbol': 'SWISE', 'decimals': 18},
    '0xB6Ca7399B4F9CA56FC27cBfF44F4d2e4Eef1fc81': {'symbol': 'MUSE', 'decimals': 18},
    '0x819c1A1568934Ee59d9F3C8B9640908556c44140': {'symbol': 'HOBBES', 'decimals': 18},
    '0x76c5449F4950f6338A393F53CdA8b53B0cd3Ca3a': {'symbol': 'BT', 'decimals': 18},
    '0x8AF5FedC0f263841C18F31D9DbCC97A47e1aB462': {'symbol': 'M87', 'decimals': 18},
    '0x80fB784B7eD66730e8b1DBd9820aFD29931aab03': {'symbol': 'LEND', 'decimals': 18},
    '0xb1f871Ae9462F1b2C6826E88A7827e76f86751d4': {'symbol': 'GNY', 'decimals': 18},
    '0x1dEa979ae76f26071870F824088dA78979eb91C8': {'symbol': 'SPD', 'decimals': 18},
    '0x3Ec8798B81485A254928B70CDA1cf0A2BB0B74D7': {'symbol': 'GRO', 'decimals': 18},
    '0xa150Db9b1Fa65b44799d4dD949D922c0a33Ee606': {'symbol': 'DRC', 'decimals': 0},
    '0x50DE6856358Cc35f3A9a57eAAA34BD4cB707d2cd': {'symbol': 'RAZOR', 'decimals': 18},
    '0x794Baab6b878467F93EF17e2f2851ce04E3E34C8': {'symbol': 'YIN', 'decimals': 18},
    '0xA8580F3363684d76055bdC6660CaeFe8709744e1': {'symbol': 'FOL', 'decimals': 18},
    '0x76fcA1adb104770B38581B64d55e67fA5A0f3966': {'symbol': 'ZKT', 'decimals': 9},
    '0x99295f1141d58A99e939F7bE6BBe734916a875B8': {'symbol': 'LPL', 'decimals': 18},
    '0x1A3cBdA3853494acAb67648EE59AfeB7ec3E9334': {'symbol': 'COLT', 'decimals': 18},
    '0xE22910D04bDc9d55570f3BC52E786b49DddF37b4': {'symbol': 'GATSBY', 'decimals': 18},
    '0x6bB61215298F296C55b19Ad842D3Df69021DA2ef': {'symbol': 'DOP', 'decimals': 18},
    '0x0352557B007A4Aae1511C114409b932F06F9E2f4': {'symbol': 'SRUNE', 'decimals': 18},
    '0x53dfEa0A8CC2A2A2e425E1C174Bc162999723ea0': {'symbol': 'JCHF', 'decimals': 18},
    '0x9AFb950948c2370975fb91a441F36FDC02737cD4': {'symbol': 'HFIL', 'decimals': 18},
    '0xD89310F4BaeDb33AfB36d7cc45BB8847f4463060': {'symbol': 'MEMEX', 'decimals': 18},
    '0x7e9e431a0B8c4D532C745B1043c7FA29a48D4fBa': {'symbol': 'EOSDAC', 'decimals': 18},
    '0xea7Cc765eBC94C4805e3BFf28D7E4aE48D06468A': {'symbol': 'PAD', 'decimals': 18},
    '0x3505F494c3f0fed0B594E01Fa41Dd3967645ca39': {'symbol': 'SWM', 'decimals': 18},
    '0x69948cC03f478B95283F7dbf1CE764d0fc7EC54C': {'symbol': 'AREN', 'decimals': 18},
    '0x33bD66c334274989b673a8e8C8d1A3f1B8De5889': {'symbol': 'HIODBS', 'decimals': 18},
    '0x9E976F211daea0D652912AB99b0Dc21a7fD728e4': {'symbol': 'MAP', 'decimals': 18},
    '0xD85a6Ae55a7f33B0ee113C234d2EE308EdeAF7fD': {'symbol': 'CBK', 'decimals': 18},
    '0x764104dc24DAdFf01150253a58C82337984B4319': {'symbol': 'HICLONEX', 'decimals': 18},
    '0x5d3a4F62124498092Ce665f865E0b38fF6F5FbEa': {'symbol': 'IDEA', 'decimals': 18},
    '0x41e5560054824eA6B0732E656E3Ad64E20e94E45': {'symbol': 'CVC', 'decimals': 8},
    '0x517ABf1fCDBD76BC75B532683ada9113E313A128': {'symbol': 'DOKE', 'decimals': 9},
    '0xF81421fc15300c5a8CCa9aFE12F5CBAd502Fa756': {'symbol': 'CRDC', 'decimals': 18},
    '0x86772b1409b61c639EaAc9Ba0AcfBb6E238e5F83': {'symbol': 'NDX', 'decimals': 18},
    '0x2223bF1D7c19EF7C06DAB88938EC7B85952cCd89': {'symbol': 'KXA', 'decimals': 18},
    '0x0e2ef8AeCB3c01Ad5D596f1B67134e178199984D': {'symbol': 'LAND', 'decimals': 18},
    '0xf6ec87DFE1Ed3a7256Cc0c38e3c8139103e9aF3b': {'symbol': 'GENE', 'decimals': 18},
    '0x3e7804c51a70Ba26E904c2e0ab440C5623a8A83F': {'symbol': 'GPX', 'decimals': 8},
    '0x669dB4c47f89f21554Ebd825A744888725FD9491': {'symbol': 'HIPENGUINS', 'decimals': 18},
    '0xd1cD47746B8e72359b28C1c84a4F6a19dc1A0ee5': {'symbol': 'SONIC', 'decimals': 18},
    '0xFd414e39155F91E94443a9Fe97E856569d0F5Eec': {'symbol': 'SERP', 'decimals': 9},
    '0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE': {'symbol': 'YVUSDC', 'decimals': 6},
    '0x04A020325024F130988782bd5276e53595e8d16E': {'symbol': 'HERB', 'decimals': 8},
    '0x661Ab0Ed68000491d98C796146bcF28c20d7c559': {'symbol': 'DOWS', 'decimals': 18},
    '0x7051faED0775f664a0286Af4F75ef5ed74e02754': {'symbol': 'CHANGE', 'decimals': 18},
    '0x603778e5e07e180E300bD10df7fA626313a206B2': {'symbol': 'WIZ', 'decimals': 4},
    '0x5D858bcd53E085920620549214a8b27CE2f04670': {'symbol': 'POP', 'decimals': 18},
    '0x16c52CeeCE2ed57dAd87319D91B5e3637d50aFa4': {'symbol': 'TCAP', 'decimals': 18},
    '0x111111517e4929D3dcbdfa7CCe55d30d4B6BC4d6': {'symbol': 'ICHI', 'decimals': 18},
    '0xAEcc217a749c2405b5ebC9857a16d58Bdc1c367F': {'symbol': 'PAWTH', 'decimals': 9},
    '0x71eebA415A523F5C952Cc2f06361D5443545Ad28': {'symbol': 'XDAO', 'decimals': 18},
    '0x4385328cc4D643Ca98DfEA734360C0F596C83449': {'symbol': 'TOMI', 'decimals': 18},
    '0x0000000DE40dfa9B17854cBC7869D80f9F98D823': {'symbol': 'DLTA', 'decimals': 18},
    '0x3F7Aff0EF20AA2E646290DfA4E67611B2220C597': {'symbol': 'VOLT', 'decimals': 9},
    '0x679BADc551626e01B23CeecEFBc9B877EA18fc46': {'symbol': 'CCO', 'decimals': 18},
    '0xF256CC7847E919FAc9B808cC216cAc87CCF2f47a': {'symbol': 'AXSUSHI', 'decimals': 18},
    '0x8c4E7f814d40f8929F9112C5D09016F923d34472': {'symbol': 'XLAB', 'decimals': 18},
    '0x7BEF710a5759d197EC0Bf621c3Df802C2D60D848': {'symbol': 'SHOPX', 'decimals': 18},
    '0xd54619E0b9899D74Cc9B981354Eb6b59732c43B1': {'symbol': 'GLR', 'decimals': 18},
    '0xC5b3D3231001a776123194Cf1290068e8b0C783b': {'symbol': 'LIT', 'decimals': 18},
    '0x33E07f5055173cF8FeBedE8B21B12D1e2b523205': {'symbol': 'ELAND', 'decimals': 18},
    '0x247Dc9CBBAADabce6e30E2A84eC6C53A419913Ad': {'symbol': 'EMS', 'decimals': 18},
    '0x048Fe49BE32adfC9ED68C37D32B5ec9Df17b3603': {'symbol': 'SKM', 'decimals': 18},
    '0x25C7b64A93Eb1261E130eC21a3e9918CaA38b611': {'symbol': 'WVG0', 'decimals': 18},
    '0x5F9123d661459AF6f398B6f1566f53222612541e': {'symbol': 'MARAN', 'decimals': 18},
    '0x166F1a7eCAe00bd43876A25B10a63C575e05c0e7': {'symbol': 'VERA', 'decimals': 18},
    '0x8dc89F4716E027394Bba225b82328C1ea2Ea58Bf': {'symbol': 'GVC', 'decimals': 18},
    '0xDE2F7766C8BF14CA67193128535E5c7454f8387C': {'symbol': 'META', 'decimals': 18},
    '0x734C90044a0bA31B3F2e640c10dC5d3540499Bfd': {'symbol': 'TSX', 'decimals': 18},
    '0x6911F552842236bd9E8ea8DDBB3fb414e2C5FA9d': {'symbol': 'SNP', 'decimals': 18},
    '0x80c8C3dCfB854f9542567c8Dac3f44D709eBc1de': {'symbol': 'MILK2', 'decimals': 18},
    '0x6e9730EcFfBed43fD876A264C982e254ef05a0DE': {'symbol': 'NORD', 'decimals': 18},
    '0x72630B1e3B42874bf335020Ba0249e3E9e47Bafc': {'symbol': 'EPAN', 'decimals': 18},
    '0xC53342fd7575f572b0fF4569e31941A5B821aC76': {'symbol': 'ETHV', 'decimals': 18},
    '0xE23FaEfcdAAe646527eb86b1C746A25Ae4a36f75': {'symbol': 'WGMI', 'decimals': 18},
    '0x6524B87960c2d573AE514fd4181777E7842435d4': {'symbol': 'BZN', 'decimals': 18},
    '0xcAe636167db2369bd746F2bcBa79a6e8b0d4E000': {'symbol': 'EAC', 'decimals': 18},
    '0xC88F47067dB2E25851317A2FDaE73a22c0777c37': {'symbol': 'ONEBTC', 'decimals': 9},
    '0xEa319e87Cf06203DAe107Dd8E5672175e3Ee976c': {'symbol': 'SURF', 'decimals': 18},
    '0x81f8f0bb1cB2A06649E51913A151F0E7Ef6FA321': {'symbol': 'VITA', 'decimals': 18},
    '0x27054b13b1B798B345b591a4d22e6562d47eA75a': {'symbol': 'AST', 'decimals': 4},
    '0xe469c4473af82217B30CF17b10BcDb6C8c796e75': {'symbol': 'EXRN', 'decimals': 0},
    '0x4A3FE75762017DB0eD73a71C9A06db7768DB5e66': {'symbol': 'YVCOMP', 'decimals': 18},
    '0x7815bDa662050D84718B988735218CFfd32f75ea': {'symbol': 'YEL', 'decimals': 18},
    '0x9C78EE466D6Cb57A4d01Fd887D2b5dFb2D46288f': {'symbol': 'MUST', 'decimals': 18},
    '0x0944D5848BD9F60A34Ba92aEa300d4286696Eb76': {'symbol': 'PLT', 'decimals': 18},
    '0x747f564d258612ec5c4E24742C5fd4110bCbe46B': {'symbol': 'NDS', 'decimals': 18},
    '0x01792e1548Dc317bDE6123fe92da1Fe6d7311C3c': {'symbol': 'SPIRAL', 'decimals': 9},
    '0x6100dd79fCAA88420750DceE3F735d168aBcB771': {'symbol': 'SOON', 'decimals': 18},
    '0x9bf02cF6B0435A0523E6f6e0D2f35A920144F5FA': {'symbol': 'D11', 'decimals': 18},
    '0xFb782396c9b20E564A64896181c7AC8d8979d5F4': {'symbol': 'DIVER', 'decimals': 18},
    '0xdFC3829b127761a3218bFceE7fc92e1232c9D116': {'symbol': 'PRCY', 'decimals': 8},
    '0xd24946147829DEaA935bE2aD85A3291dbf109c80': {'symbol': 'AAMMUSDC', 'decimals': 6},
    '0x52F4D5EE6c91e01Be67CA1F64b11eD0EE370817d': {'symbol': 'CIA', 'decimals': 9},
    '0xCC12AbE4ff81c9378D670De1b57F8e0Dd228D77a': {'symbol': 'AREN', 'decimals': 18},
    '0x7C84e62859D0715eb77d1b1C4154Ecd6aBB21BEC': {'symbol': 'SHPING', 'decimals': 18},
    '0x94d863173EE77439E4292284fF13fAD54b3BA182': {'symbol': 'ADEL', 'decimals': 18},
    '0x56A86d648c435DC707c8405B78e2Ae8eB4E60Ba4': {'symbol': 'STACK', 'decimals': 18},
    '0xC631bE100F6Cf9A7012C23De5a6ccb990EAFC133': {'symbol': 'BTFA', 'decimals': 9},
    '0xC581b735A1688071A1746c968e0798D642EDE491': {'symbol': 'EURT', 'decimals': 6},
    '0x08037036451C768465369431Da5C671ad9B37dBc': {'symbol': 'NFTS', 'decimals': 18},
    '0x3819f64f282bf135d62168C1e513280dAF905e06': {'symbol': 'HDRN', 'decimals': 9},
    '0x056354F3Ff20743aa4c0DA365603871c7000b081': {'symbol': 'TRIX', 'decimals': 18},
    '0x666d875C600AA06AC1cf15641361dEC3b00432Ef': {'symbol': 'BTSE', 'decimals': 8},
    '0x992D339532A9C42F1b0E59A57e95F38da38c66F6': {'symbol': 'SOUL', 'decimals': 18},
    '0xBb556b0eE2CBd89ed95DdEA881477723A3Aa8F8b': {'symbol': 'ALCA', 'decimals': 18},
    '0xfc4913214444aF5c715cc9F7b52655e788A569ed': {'symbol': 'ICSA', 'decimals': 9},
    '0xA5B947687163FE88C3e6af5b17Ae69896F4abccf': {'symbol': 'PSDN', 'decimals': 18},
    '0xf66Cd2f8755a21d3c8683a10269F795c0532Dd58': {'symbol': 'COREDAO', 'decimals': 18},
    '0xd26a9C3437f7D121098c8C05C7413F5Cc70BB070': {'symbol': 'AZUM', 'decimals': 18},
    '0x9b0B23b35AD8136E6181F22b346134cE5f426090': {'symbol': 'CINO', 'decimals': 18},
    '0xE61fDAF474Fac07063f2234Fb9e60C1163Cfa850': {'symbol': 'COIN', 'decimals': 18},
    '0xBBaB3bDb291b0F22BC9881895ff488A5Db67BeC8': {'symbol': 'ITAMCUBE', 'decimals': 18},
    '0x177d39AC676ED1C67A2b268AD7F1E58826E5B0af': {'symbol': 'CDT', 'decimals': 18},
    '0x97a9bac06f90940bCe9CAeC2b880ff17707519e4': {'symbol': 'MNTO', 'decimals': 18},
    '0x1df7AA5551E801e280007dc0Fc0454E2D06C1A89': {'symbol': 'BKK', 'decimals': 18},
    '0x6d765CbE5bC922694afE112C140b8878b9FB0390': {'symbol': 'YVSUSHI', 'decimals': 18},
    '0x0De05F6447ab4D22c8827449EE4bA2D5C288379B': {'symbol': 'OOKI', 'decimals': 18},
    '0x755be920943eA95e39eE2DC437b268917B580D6e': {'symbol': 'VVT', 'decimals': 18},
    '0xD7D8f3b8Bc8bC48d3AcC37879EABA7b85889FA52': {'symbol': 'CLH', 'decimals': 18},
    '0x1cCF27211e8bf052f6255329ed641B4E94E80603': {'symbol': 'BABY', 'decimals': 18},
    '0xd2d6158683aeE4Cc838067727209a0aAF4359de3': {'symbol': 'BNTY', 'decimals': 18},
    '0x939A7A577D93ad29B64C1595B1284ce660A479B9': {'symbol': 'JEJUDOGE', 'decimals': 18},
    '0x1A3496C18d558bd9C6C8f609E1B129f67AB08163': {'symbol': 'DEP', 'decimals': 18},
    '0x88dF592F8eb5D7Bd38bFeF7dEb0fBc02cf3778a0': {'symbol': 'TRB', 'decimals': 18},
    '0xe115401B985306837B9f1495c8e9d7f8f8328cc3': {'symbol': 'CLAW', 'decimals': 18},
    '0x16B0a1a87ae8aF5C792faBC429C4FE248834842B': {'symbol': 'ALG', 'decimals': 18},
    '0xdF96bDE075d59e9143b325C75aF38e208c986E6F': {'symbol': 'YETIC', 'decimals': 9},
    '0x9ee91F9f426fA633d227f7a9b000E28b9dfd8599': {'symbol': 'STMATIC', 'decimals': 18},
    '0x6f222E04F6c53Cc688FfB0Abe7206aAc66A8FF98': {'symbol': 'ROKO', 'decimals': 18},
    '0x2D80f5F5328FdcB6ECeb7Cacf5DD8AEDaEC94e20': {'symbol': 'AGA', 'decimals': 4},
    '0x09F098B155D561Fc9F7BcCc97038b7e3d20bAF74': {'symbol': 'ZOO', 'decimals': 18},
    '0x666CBfaA3baa2FAccFAC8854fea1E5Db140FB104': {'symbol': 'PLUMS', 'decimals': 18},
    '0x70D2b7C19352bB76e4409858FF5746e500f2B67c': {'symbol': 'UPI', 'decimals': 18},
    '0x5b985B4F827072FebE33091b42729522B557Bba1': {'symbol': 'T', 'decimals': 18},
    '0x8CE9137d39326AD0cD6491fb5CC0CbA0e089b6A9': {'symbol': 'SXP', 'decimals': 18},
    '0x53Bd789F2cDb846b227d8ffc7B46eD4263231FDf': {'symbol': 'SMBSWAP', 'decimals': 18},
    '0xa6a1Cc527d48585538b137E6Abc14B2a55489D11': {'symbol': 'LWC', 'decimals': 8},
    '0x5f98805A4E8be255a32880FDeC7F6728C6568bA0': {'symbol': 'LUSD', 'decimals': 18},
    '0x101cc05f4A51C0319f570d5E146a8C625198e636': {'symbol': 'ATUSD', 'decimals': 18},
    '0x2904b9b16652d7d0408EcCfA23A19D4A3358230f': {'symbol': 'PURE', 'decimals': 18},
    '0xA848a1D33D8ef1633397A6acf617620fab8E5dA8': {'symbol': 'MOUSEWORM', 'decimals': 6},
    '0x0000000000004946c0e9F43F4Dee607b0eF1fA1c': {'symbol': 'CHI', 'decimals': 0},
    '0x6Ee0f7BB50a54AB5253dA0667B0Dc2ee526C30a8': {'symbol': 'ABUSD', 'decimals': 18},
    '0x61bFC979EA8160Ede9b862798B7833a97baFa02a': {'symbol': 'REL', 'decimals': 18},
    '0x68E9C0d9aA450254aed2CD102503d4DFf6B3C37c': {'symbol': 'WCSOV', 'decimals': 18},
    '0xbE9Ab37a414c517b2be2bfA5945665bB07379054': {'symbol': 'TOMS', 'decimals': 18},
    '0x60BB16c4A931b1a0B8A7D945C651DD90f41D42Cf': {'symbol': 'FBX', 'decimals': 18},
    '0x0C9c7712C83B3C70e7c5E11100D33D9401BdF9dd': {'symbol': 'WOMBAT', 'decimals': 18},
    '0x1864cE27E9F7517047933CaAE530674e8C70b8A7': {'symbol': 'PIB', 'decimals': 18},
    '0x1735Db6AB5BAa19eA55d0AdcEeD7bcDc008B3136': {'symbol': 'URQA', 'decimals': 18},
    '0x32EA3Dc70E2962334864A9665254d2433E4ddbfD': {'symbol': 'SPN', 'decimals': 18},
    '0x910c4DA718CAF4EE38cE5c2490CDDAEca689204e': {'symbol': 'WACO', 'decimals': 18},
    '0x38c87AA89B2B8cD9B95b736e1Fa7b612EA972169': {'symbol': 'AMO', 'decimals': 18},
    '0x14a32f050FACF226Ec60882398A9BF36d91DBaC2': {'symbol': 'KYOKO', 'decimals': 18},
    '0x20d4DB1946859E2Adb0e5ACC2eac58047aD41395': {'symbol': 'MOONEY', 'decimals': 18},
    '0x51395AdE06Eae126f590e7B06Dc8F6Baf511f13F': {'symbol': 'HISQUIGGLE', 'decimals': 18},
    '0x3209d14Ff61766359E64aceFf91877cEC2AD968E': {'symbol': 'CUP', 'decimals': 18},
    '0x1460a58096d80a50a2F1f956DDA497611Fa4f165': {'symbol': 'CHX', 'decimals': 18},
    '0xBD2F0Cd039E0BFcf88901C98c0bFAc5ab27566e3': {'symbol': 'DSD', 'decimals': 18},
    '0x04969cD041C0cafB6AC462Bd65B536A5bDB3A670': {'symbol': 'WOMI', 'decimals': 18},
    '0x508df5aa4746bE37b5b6A69684DfD8BDC322219d': {'symbol': 'CRF', 'decimals': 18},
    '0xf1F508c7C9f0d1b15a76fbA564eEf2d956220cf7': {'symbol': 'PPDEX', 'decimals': 18},
    '0xddaC9C604BA6Bc4ACEc0FBB485B83f390ECF2f31': {'symbol': 'CYOP', 'decimals': 9},
    '0xB5F1457d6FBa1956Fb8d31B0B7CAcA14BDe0bE4b': {'symbol': 'STILT', 'decimals': 9},
    '0xb3207935FF56120f3499e8aD08461Dd403bF16b8': {'symbol': 'DAMM', 'decimals': 18},
    '0x066798d9ef0833ccc719076Dab77199eCbd178b0': {'symbol': 'SAKE', 'decimals': 18},
    '0x77777FeDdddFfC19Ff86DB637967013e6C6A116C': {'symbol': 'TORN', 'decimals': 18},
    '0xc2544A32872A91F4A553b404C6950e89De901fdb': {'symbol': 'FPIS', 'decimals': 18},
    '0x155ff1A85F440EE0A382eA949f24CE4E0b751c65': {'symbol': 'EYE', 'decimals': 18},
    '0x8dB253a1943DdDf1AF9bcF8706ac9A0Ce939d922': {'symbol': 'UNB', 'decimals': 18},
    '0x69b148395Ce0015C13e36BFfBAd63f49EF874E03': {'symbol': 'DTA', 'decimals': 18},
    '0x3B27F92C0e212C671EA351827EDF93DB27cc0c65': {'symbol': 'YVUSDT', 'decimals': 6},
    '0xF9c53268e9de692AE1b2ea5216E24e1c3ad7CB1E': {'symbol': 'IDO', 'decimals': 18},
    '0xf56842Af3B56Fd72d17cB103f92d027bBa912e89': {'symbol': 'BAMBOO', 'decimals': 18},
    '0x622dFfCc4e83C64ba959530A5a5580687a57581b': {'symbol': 'AUTO', 'decimals': 18},
    '0xaF8942831F3A096f708b8b31F191B8958Cf176c5': {'symbol': 'NERF', 'decimals': 18},
    '0x0ADCfDFe9E88D28cC4C1dAaC9CC021067aFF9B0C': {'symbol': 'TOTOFO', 'decimals': 18},
    '0x470EBf5f030Ed85Fc1ed4C2d36B9DD02e77CF1b7': {'symbol': 'TEMPLE', 'decimals': 18},
    '0x32a7C02e79c4ea1008dD6564b35F131428673c41': {'symbol': 'CRU', 'decimals': 18},
    '0xAa843f65872a25D6E9552eA0B360Fb1d5E333124': {'symbol': 'EVC', 'decimals': 18},
    '0xDDD6A0ECc3c6F6C102E5eA3d8Af7B801d1a77aC8': {'symbol': 'UNIX', 'decimals': 18},
    '0x26c8AFBBFE1EBaca03C2bB082E69D0476Bffe099': {'symbol': 'CELL', 'decimals': 18},
    '0x3b58c52C03ca5Eb619EBa171091c86C34d603e5f': {'symbol': 'CYCLUB', 'decimals': 9},
    '0x87b008E57F640D94Ee44Fd893F0323AF933F9195': {'symbol': 'COIN', 'decimals': 18},
    '0xe632ea2eF2CFD8Fc4a2731C76F99078Aef6a4B31': {'symbol': 'THX', 'decimals': 18},
    '0xa71d0588EAf47f12B13cF8eC750430d21DF04974': {'symbol': 'QOM', 'decimals': 18},
    '0xd4342a57eCf2Fe7ffA37c33cb8f63b1500e575E6': {'symbol': 'APN', 'decimals': 18},
    '0x21f15966E07a10554C364b988e91DaB01D32794A': {'symbol': 'SMT', 'decimals': 18},
    '0x333A4823466879eeF910A04D473505da62142069': {'symbol': 'NATION', 'decimals': 18},
    '0x14b40AD2EBA6c1B31db2bA817b07578AFB414415': {'symbol': 'COLR', 'decimals': 18},
    '0x03D1E72765545729A035e909eDd9371a405f77Fb': {'symbol': 'NABOX', 'decimals': 18},
    '0x2C33B28527A63Cdf13C0B24Ce4cf5BF9C9fB3BC6': {'symbol': 'KITTYDINGER', 'decimals': 9},
    '0x98968f0747E0A261532cAcC0BE296375F5c08398': {'symbol': 'MOONCAT', 'decimals': 18},
    '0x009c43B42AEFAC590C719E971020575974122803': {'symbol': 'BIX', 'decimals': 18},
    '0xE0c05ec44775e4AD62CDC2eEcdF337aA7A143363': {'symbol': 'MANC', 'decimals': 2},
    '0x3aaDA3e213aBf8529606924d8D1c55CbDc70Bf74': {'symbol': 'XMON', 'decimals': 18},
    '0xed03Ed872159e199065401b6d0D487d78d9464AA': {'symbol': 'MXNT', 'decimals': 6},
    '0x066e941E168dE3815cd6c8F63c46e8E3a2309ad7': {'symbol': 'BOX', 'decimals': 18},
    '0xaE6e307c3Fe9E922E5674DBD7F830Ed49c014c6B': {'symbol': 'CREDI', 'decimals': 18},
    '0xEB4C2781e4ebA804CE9a9803C67d0893436bB27D': {'symbol': 'RENBTC', 'decimals': 8},
    '0xFc979087305A826c2B2a0056cFAbA50aad3E6439': {'symbol': 'DAFI', 'decimals': 18},
    '0x5ABf88cF3444611D13F6D1B39F3f3EE8575c91a2': {'symbol': 'SAT', 'decimals': 18},
    '0x5A5389bA78F04D783E9F30e5A8C45727c5E018bE': {'symbol': 'CHIP', 'decimals': 9},
    '0xFF75CEd57419bcaEBe5F05254983b013B0646eF5': {'symbol': 'COOK', 'decimals': 18},
    '0xF4Dc48D260C93ad6a96c5Ce563E70CA578987c74': {'symbol': 'BABL', 'decimals': 18},
    '0xA487bF43cF3b10dffc97A9A744cbB7036965d3b9': {'symbol': 'DERI', 'decimals': 18},
    '0xccF4429DB6322D5C611ee964527D42E5d685DD6a': {'symbol': 'CWBTC', 'decimals': 8},
    '0xDc0804eD9ABA9202E2b976C031d3144BB63a7990': {'symbol': 'KISHUTAMA', 'decimals': 9},
    '0x632806BF5c8f062932Dd121244c9fbe7becb8B48': {'symbol': 'PDI', 'decimals': 18},
    '0x69e8b9528CABDA89fe846C67675B5D73d463a916': {'symbol': 'OPEN', 'decimals': 18},
    '0xA1a4E303e9C56962F201C5e834abC1E677A3C4F3': {'symbol': 'CVNX', 'decimals': 18},
    '0x6710c63432A2De02954fc0f851db07146a6c0312': {'symbol': 'MFG', 'decimals': 18},
    '0xed0d5747A9AB03a75fBfEC3228cd55848245B75d': {'symbol': 'NGM', 'decimals': 6},
    '0x83869DE76B9Ad8125e22b857f519F001588c0f62': {'symbol': 'EXM', 'decimals': 8},
    '0x4Ce4C025692B3142dbdE1cd432ef55b9A8D18701': {'symbol': 'DCNT', 'decimals': 9},
    '0xCF3C8Be2e2C42331Da80EF210e9B1b307C03d36A': {'symbol': 'BEPRO', 'decimals': 18},
    '0xEF53462838000184F35f7D991452e5f25110b207': {'symbol': 'KFT', 'decimals': 18},
    '0xb753428af26E81097e7fD17f40c88aaA3E04902c': {'symbol': 'SFI', 'decimals': 18},
    '0xaB37e1358b639Fd877f015027Bb62d3ddAa7557E': {'symbol': 'LIEN', 'decimals': 8},
    '0x2F6f397332B0F8b4536430bCC97980aA838f62f9': {'symbol': 'YUNA', 'decimals': 18},
    '0x777fD20c983d6658c1D50b3958B3A1733d1cd1E1': {'symbol': 'NEWS', 'decimals': 9},
    '0xa52bfFAD02B1FE3f86A543a4e81962d3B3bB01A7': {'symbol': 'DUCKER', 'decimals': 18},
    '0x0Def8d8addE14c9eF7c2a986dF3eA4Bd65826767': {'symbol': 'CLIQ', 'decimals': 18},
    '0x3a1BC4014C4C493DB3dBfBDD8ee1417113b462BF': {'symbol': 'BPTL', 'decimals': 18},
    '0xFEEf77d3f69374f66429C91d732A244f074bdf74': {'symbol': 'CVXFXS', 'decimals': 18},
    '0x466912bAA9430a4a460b141ee8C580D817441449': {'symbol': 'OCB', 'decimals': 18},
    '0x103c3A209da59d3E7C4A89307e66521e081CFDF0': {'symbol': 'GVT', 'decimals': 18},
    '0x9d71CE49ab8A0E6D2a1e7BFB89374C9392FD6804': {'symbol': 'NVIR', 'decimals': 18},
    '0x05fE069626543842439Ef90D9FA1633640c50cF1': {'symbol': 'EVEAI', 'decimals': 18},
    '0xFFC97d72E13E01096502Cb8Eb52dEe56f74DAD7B': {'symbol': 'AAAVE', 'decimals': 18},
    '0xeDADeB5fAa413e6c8623461849DFD0B7C3790c32': {'symbol': 'OBOT', 'decimals': 18},
    '0x5fA54fdDF1870C344DbFaBb37dFab8700Ec0Def1': {'symbol': 'FROGEX', 'decimals': 9},
    '0x59B44e9AC4AB8e29db6Faf7baCd726C6B7bC94b9': {'symbol': 'BLEYD', 'decimals': 9},
    '0x83cee9e086A77e492eE0bB93C2B0437aD6fdECCc': {'symbol': 'MNTP', 'decimals': 18},
    '0x375aBB85C329753b1Ba849a601438AE77eEc9893': {'symbol': 'PDT', 'decimals': 18},
    '0x1C4853Ec0d55e420002c5eFaBc7Ed8e0bA7A4121': {'symbol': 'OKINAMI', 'decimals': 9},
    '0x801dE502C493aa9d3F9F79A0348fbc00521b3001': {'symbol': 'RELAX', 'decimals': 18},
    '0x73d7c860998CA3c01Ce8c808F5577d94d545d1b4': {'symbol': 'IXS', 'decimals': 18},
    '0x6c936D4AE98E6d2172dB18c16C4b601C99918EE6': {'symbol': 'LIFE', 'decimals': 18},
    '0xF80D589b3Dbe130c270a69F1a69D050f268786Df': {'symbol': 'DAM', 'decimals': 18},
    '0xfca89d55A768375Ab7cA04485A35a964BEA828dd': {'symbol': 'DELREY', 'decimals': 18},
    '0x1410434b0346f5bE678d0FB554E5c7ab620f8f4a': {'symbol': 'KAN', 'decimals': 18},
    '0x0c943300D604c732117DB2755873A0A0683ee7a9': {'symbol': 'PDF', 'decimals': 18},
    '0xBc17729fDf562723f0267F79FF25aDE441056d87': {'symbol': 'KST', 'decimals': 18},
    '0x86EFc496DcA70bcFD92D19194290e8457a375773': {'symbol': 'UBSN', 'decimals': 0},
    '0x8a6ACA71A218301c7081d4e96D64292D3B275ce0': {'symbol': 'SFG', 'decimals': 18},
    '0x1BeEF31946fbbb40B877a72E4ae04a8D1A5Cee06': {'symbol': 'PAR', 'decimals': 18},
    '0xd076C4Ba62c57b3Fa10800bcfD8dA66742110E0E': {'symbol': 'HVH', 'decimals': 18},
    '0xe0b9BcD54bF8A730EA5d3f1fFCe0885E911a502c': {'symbol': 'ZUM', 'decimals': 8},
    '0x4Af698B479D0098229DC715655c667Ceb6cd8433': {'symbol': 'MAID', 'decimals': 18},
    '0x0789dbAE94fb18e5789B8e4489BCB7A1ADB58622': {'symbol': 'FSCC', 'decimals': 8},
    '0x69A95185ee2a045CDC4bCd1b1Df10710395e4e23': {'symbol': 'POOLZ', 'decimals': 18},
    '0xA6F7645ed967FAF708A614a2fcA8D4790138586f': {'symbol': 'MNY', 'decimals': 18},
    '0x5552E5a89A70cB2eF5AdBbC45a6BE442fE7160Ec': {'symbol': 'KAWA', 'decimals': 9},
    '0xCdeee767beD58c5325f68500115d4B722b3724EE': {'symbol': 'CRBN', 'decimals': 18},
    '0xF4b5470523cCD314C6B9dA041076e7D79E0Df267': {'symbol': 'BBANK', 'decimals': 18},
    '0x20c3fa331A385b63EE39137e99d0cF2db142fCe1': {'symbol': 'SHIL', 'decimals': 18},
    '0xf4CD3d3Fda8d7Fd6C5a500203e38640A70Bf9577': {'symbol': 'YF-DAI', 'decimals': 18},
    '0x616e8BfA43F920657B3497DBf40D6b1A02D4608d': {'symbol': 'AURABAL', 'decimals': 18},
    '0x05Ec93c0365baAeAbF7AefFb0972ea7ECdD39CF1': {'symbol': 'ABAT', 'decimals': 18},
    '0x9695e0114e12C0d3A3636fAb5A18e6b737529023': {'symbol': 'DFYN', 'decimals': 18},
    '0x744d70FDBE2Ba4CF95131626614a1763DF805B9E': {'symbol': 'SNT', 'decimals': 18},
    '0xD528cf2E081f72908e086F8800977df826B5a483': {'symbol': 'PBX', 'decimals': 18},
    '0x1cA02DD95f3F1e33DA7F5AfE15EA866dAb07aF04': {'symbol': 'BKING', 'decimals': 9},
    '0x7697B462A7c4Ff5F8b55BDBC2F4076c2aF9cF51A': {'symbol': 'SARCO', 'decimals': 18},
    '0xCd2828fc4D8E8a0eDe91bB38CF64B1a81De65Bf6': {'symbol': 'ODDZ', 'decimals': 18},
    '0x0a6E18fB2842855C3AF925310B0F50a4BfA17909': {'symbol': 'CHP', 'decimals': 18},
    '0x995a6C0944322b7285D0712824acf0Cf9BF3993a': {'symbol': 'FEN', 'decimals': 18},
    '0x3D3D35bb9bEC23b06Ca00fe472b50E7A4c692C30': {'symbol': 'VIDYA', 'decimals': 18},
    '0x89DcFF5Fd892f2bfc8B75dBA12804B651f769579': {'symbol': 'STKR', 'decimals': 18},
    '0xA4e8C3Ec456107eA67d3075bF9e3DF3A75823DB0': {'symbol': 'LOOMOLD', 'decimals': 18},
    '0xfB22cED41B1267dA411F68c879f4Defd0bD4796a': {'symbol': 'ZIOT', 'decimals': 18},
    '0x20a8CEC5fffea65Be7122BCaB2FFe32ED4Ebf03a': {'symbol': 'DNXC', 'decimals': 18},
    '0x5c3a228510D246b78a3765C20221Cbf3082b44a4': {'symbol': 'STQ', 'decimals': 18},
    '0x24BCeC1AFda63E622a97F17cFf9a61FFCfd9b735': {'symbol': 'CRAB', 'decimals': 18},
    '0x2dA719DB753dFA10a62E140f436E1d67F2ddB0d6': {'symbol': 'CERE', 'decimals': 10},
    '0x00000000000045166C45aF0FC6E4Cf31D9E14B9A': {'symbol': 'BID', 'decimals': 18},
    '0xee1ae38BE4Ce0074C4A4A8DC821CC784778f378c': {'symbol': 'VYNC', 'decimals': 4},
    '0x3CBc780D2934d55A06069e837fABd3e6Fc23DAB0': {'symbol': 'DBX', 'decimals': 18},
    '0xE1Fc4455f62a6E89476f1072530C20CF1A0622dA': {'symbol': 'PHTR', 'decimals': 18},
    '0xB67bEB26eBeb0DCEec354aE0942256d03c01771b': {'symbol': 'DS', 'decimals': 18},
    '0x2C9C19cE3b15ae77C6d80aEc3C1194Cfd6F7F3fA': {'symbol': '2CRZ', 'decimals': 18},
    '0x686f2404e77Ab0d9070a46cdfb0B7feCDD2318b0': {'symbol': 'LORDS', 'decimals': 18},
    '0x84bA4aEcfDe39D69686a841BAb434C32d179a169': {'symbol': 'MTHD', 'decimals': 18},
    '0xE4B91fAf8810F8895772E7cA065D4CB889120f94': {'symbol': 'BOXFEE', 'decimals': 18},
    '0x79650799e7899A802cB96C0Bc33a6a8d4CE4936C': {'symbol': 'AIT', 'decimals': 18},
    '0x24A6A37576377F63f194Caa5F518a60f45b42921': {'symbol': 'BANK', 'decimals': 18},
    '0xC0bA369c8Db6eB3924965e5c4FD0b4C1B91e305F': {'symbol': 'DUCK', 'decimals': 18},
    '0xDd1Ad9A21Ce722C151A836373baBe42c868cE9a4': {'symbol': 'UBI', 'decimals': 18},
    '0x0B4BdC478791897274652DC15eF5C135cae61E60': {'symbol': 'DAX', 'decimals': 18},
    '0x9ad37205d608B8b219e6a2573f922094CEc5c200': {'symbol': 'IZI', 'decimals': 18},
    '0x58FaD9E3C3AE54c9BA98c3f0E4bF88aB3E8Cf3c5': {'symbol': 'SPAY', 'decimals': 18},
    '0x06576eB3B212d605B797dC15523d9Dc9F4F66DB4': {'symbol': 'TCP', 'decimals': 18},
    '0x2f4b36442687E943E587b45cCcc92D60C3c3502F': {'symbol': 'BETT', 'decimals': 18},
    '0x8770b7Dd89E5f759eE3b226e0C45e890f87dDc48': {'symbol': 'MIRL', 'decimals': 18},
    '0xAA2ce7Ae64066175E0B90497CE7d9c190c315DB4': {'symbol': 'SUTER', 'decimals': 18},
    '0x8Af078d5A3e4a18Ff62e3AEeD4b1c448aB90b304': {'symbol': 'TOC', 'decimals': 18},
    '0xB49fa25978abf9a248b8212Ab4b87277682301c0': {'symbol': 'SOFI', 'decimals': 18},
    '0x05bBe7240De66F6480C9aeda77C1376B13393F83': {'symbol': 'XNO', 'decimals': 18},
    '0x10BC518c32fbAE5e38Ecb50A612160571bD81e44': {'symbol': 'VRO', 'decimals': 8},
    '0x149D8290F653dEB8e34c037D239d3d8EeE9de5aD': {'symbol': 'KING', 'decimals': 18},
    '0xEe9E7Bb7E55bBc86414047B61D65c9C0d91fFBd0': {'symbol': 'FT', 'decimals': 18},
    '0xbb9FD9Fa4863C03c574007fF3370787B9cE65ff6': {'symbol': 'HILO', 'decimals': 18},
    '0x464eBE77c293E473B48cFe96dDCf88fcF7bFDAC0': {'symbol': 'KRL', 'decimals': 18},
    '0x5319e86F0e41a06E49eb37046b8c11D78bcAd68C': {'symbol': 'ZLW', 'decimals': 18},
    '0x0c9b3aB1bd0CF0745625381F5C3Aa1CD9BBc7Abb': {'symbol': 'EXN', 'decimals': 18},
    '0xe7976c4Efc60d9f4C200Cc1bCEF1A1e3B02c73e7': {'symbol': 'MAX', 'decimals': 18},
    '0x431ad2ff6a9C365805eBaD47Ee021148d6f7DBe0': {'symbol': 'DF', 'decimals': 18},
    '0x641927E970222B10b2E8CDBC96b1B4F427316f16': {'symbol': 'MEEB', 'decimals': 18},
    '0x26607aC599266b21d13c7aCF7942c7701a8b699c': {'symbol': 'PIPT', 'decimals': 18},
    '0x429876c4a6f89FB470E92456B8313879DF98B63c': {'symbol': 'CNT', 'decimals': 18},
    '0x0557E0d15aeC0b9026dD17aA874fDf7d182A2cEB': {'symbol': 'CFXQ', 'decimals': 6},
    '0x08d32b0da63e2C3bcF8019c9c5d849d7a9d791e6': {'symbol': 'DCN', 'decimals': 0},
    '0x2aF72850c504dDD3c1876C66a914cAee7Ff8a46A': {'symbol': 'WHL', 'decimals': 18},
    '0x67Cc621AB2d086a101Cff3340df0A065Ac75827c': {'symbol': 'FLOKI', 'decimals': 18},
    '0x36a28c7c9B3DEA22F07F4dF67833cBe764fEEeb4': {'symbol': 'CRT', 'decimals': 18},
    '0xBF0741E995F469D39e4f96C0780f9a8E43f4b978': {'symbol': 'IMC', 'decimals': 18},
    '0x201332Bd45c8628D814F870bFb584B385A7C351e': {'symbol': 'ASTRA', 'decimals': 18},
    '0x19ac2659599fD01c853dE846919544276aD26F50': {'symbol': 'COVN', 'decimals': 18},
    '0x9c354503C38481a7A7a51629142963F98eCC12D0': {'symbol': 'OGV', 'decimals': 18},
    '0xEd0439EACf4c4965AE4613D77a5C2Efe10e5f183': {'symbol': 'SHROOM', 'decimals': 18},
    '0x09637E374Ec5baB8C9e9654a97C9593e3038c5B3': {'symbol': 'ZKP', 'decimals': 9},
    '0xA36FDBBAE3c9d55a1d67EE5821d53B50B63A1aB9': {'symbol': 'TEMP', 'decimals': 18},
    '0xd2287a52287dd64AE2c690D74322E2e345dC8F69': {'symbol': 'BBW', 'decimals': 18},
    '0x99d36e97676A68313ffDc627fd6b56382a2a08B6': {'symbol': 'BABYTRUMP', 'decimals': 9},
    '0xa1116930326D21fB917d5A27F1E9943A9595fb47': {'symbol': 'STKABPT', 'decimals': 18},
    '0x4BDCb66B968060D9390C1D12bd29734496205581': {'symbol': 'ACQ', 'decimals': 18},
    '0xB6ff96B8A8d214544Ca0dBc9B33f7AD6503eFD32': {'symbol': 'SYNC', 'decimals': 18},
    '0xa3ad8c7AB6b731045b5b16E3fDf77975c71ABe79': {'symbol': 'DINERO', 'decimals': 18},
    '0xda4Dd9586D27202A338843Dd6B9824d267006783': {'symbol': 'ECT', 'decimals': 9},
    '0xBe428c3867F05deA2A89Fc76a102b544eaC7f772': {'symbol': 'CVT', 'decimals': 18},
    '0xA849EaaE994fb86Afa73382e9Bd88c2B6b18Dc71': {'symbol': 'MVL', 'decimals': 18},
    '0xFc0d6Cf33e38bcE7CA7D89c0E292274031b7157A': {'symbol': 'NTVRK', 'decimals': 18},
    '0xD23Ac27148aF6A2f339BD82D0e3CFF380b5093de': {'symbol': 'SI', 'decimals': 18},
    '0x62959c699A52eC647622c91e79CE73344e4099f5': {'symbol': 'DFA', 'decimals': 18},
    '0xb186035490C8602EaD853EC98bE05E3461521Ab2': {'symbol': 'PACK', 'decimals': 18},
    '0x65F9A292f1AEED5D755Aa2fD2Fb17AB2E9431447': {'symbol': 'SOMEE', 'decimals': 18},
    '0x56123908ACA89bf60cE9659aE9F4589E46c8D2cF': {'symbol': 'XMU', 'decimals': 9},
    '0xc9F1016d336ef77AeE75Fc11Ad64C5Ecf9121332': {'symbol': 'SAT', 'decimals': 18},
    '0xa8B61CfF52564758A204F841E636265bEBC8db9B': {'symbol': 'YIELD', 'decimals': 18},
    '0x18b52F500e6D9C8b0455eC3483846D9f8edd7e1b': {'symbol': 'MPM', 'decimals': 18},
    '0x66F73D0fD4161cfad4302DC145Ff994375c13475': {'symbol': 'DXGM', 'decimals': 18},
    '0xbf30461210b37012783957D90dC26B95Ce3b6f2d': {'symbol': 'SWIV', 'decimals': 18},
    '0xfd4168e642EbD04C3684A6cDb3A5E86DE85d3908': {'symbol': 'API', 'decimals': 18},
    '0x7240aC91f01233BaAf8b064248E80feaA5912BA3': {'symbol': 'OCTO', 'decimals': 18},
    '0xBe1a001FE942f96Eea22bA08783140B9Dcc09D28': {'symbol': 'BETA', 'decimals': 18},
    '0x446C9033E7516D820cc9a2ce2d0B7328b579406F': {'symbol': 'SOLVE', 'decimals': 8},
    '0xB4371dA53140417CBb3362055374B10D97e420bB': {'symbol': 'SWTH', 'decimals': 8},
    '0x2022760aCDe82079cbEDa17D3afC67dC1F3D97C9': {'symbol': 'RAID', 'decimals': 18},
    '0xE80C0cd204D654CEbe8dd64A4857cAb6Be8345a3': {'symbol': 'JPEG', 'decimals': 18},
    '0x0AaCfbeC6a24756c20D41914F2caba817C0d8521': {'symbol': 'YAM', 'decimals': 18},
    '0x70e8dE73cE538DA2bEEd35d14187F6959a8ecA96': {'symbol': 'XSGD', 'decimals': 6},
    '0x92E187a03B6CD19CB6AF293ba17F2745Fd2357D5': {'symbol': 'DUCK', 'decimals': 18},
    '0x8355DBE8B0e275ABAd27eB843F3eaF3FC855e525': {'symbol': 'WOOL', 'decimals': 18},
    '0xf650C3d88D12dB855b8bf7D11Be6C55A4e07dCC9': {'symbol': 'CUSDT', 'decimals': 8},
    '0xF55cD1e399e1cc3D95303048897a680be3313308': {'symbol': 'TRIO', 'decimals': 8},
    '0x581911b360B6eB3a14eF295a83a91DC2bCE2D6f7': {'symbol': 'MVC', 'decimals': 18},
    '0x0AbdAce70D3790235af448C88547603b945604ea': {'symbol': 'DNT', 'decimals': 18},
    '0xFE5F69dfa2d4501E78078266F6d430c079098f90': {'symbol': 'ARCHIE', 'decimals': 9},
    '0x3EF389f264e07fFF3106A3926F2a166d1393086F': {'symbol': 'SAO', 'decimals': 9},
    '0x17a79792Fe6fE5C95dFE95Fe3fCEE3CAf4fE4Cb7': {'symbol': 'AAMMUSDT', 'decimals': 6},
    '0x44197A4c44D6A059297cAf6be4F7e172BD56Caaf': {'symbol': 'ELTCOIN', 'decimals': 8},
    '0xEDB171C18cE90B633DB442f2A6F72874093b49Ef': {'symbol': 'WAMPL', 'decimals': 18},
    '0x9E46A38F5DaaBe8683E10793b06749EEF7D733d1': {'symbol': 'NCT', 'decimals': 18},
    '0x54B8D105aA09342FAD6b352D41a0bAd3E1a9aA9d': {'symbol': 'PANTO', 'decimals': 18},
    '0x343c6de13833bc7d9890eb6B1cd3FBeBC730EBec': {'symbol': 'DACT', 'decimals': 9},
    '0x5e3346444010135322268a4630d2ED5F8D09446c': {'symbol': 'LOC', 'decimals': 18},
    '0xb26C4B3Ca601136Daf98593feAeff9E0CA702a8D': {'symbol': 'ALD', 'decimals': 18},
    '0x62359Ed7505Efc61FF1D56fEF82158CcaffA23D7': {'symbol': 'CORE', 'decimals': 18},
    '0xd07D9Fe2d2cc067015E2b4917D24933804f42cFA': {'symbol': 'TOL', 'decimals': 18},
    '0xAaE3Cf9968D26925BDb73cE3864e0084a20f4687': {'symbol': 'FAR', 'decimals': 18},
    '0xCD6Adc6b8Bd396E2D53cCD7D7257B4De55bE4fbe': {'symbol': 'CFL365', 'decimals': 18},
    '0x625aE63000f46200499120B906716420bd059240': {'symbol': 'ASUSD', 'decimals': 18},
    '0xf8483E2d6560585C02D46bF7B3186Bf154a96166': {'symbol': 'ICH', 'decimals': 8},
    '0xADA86b1b313D1D5267E3FC0bB303f0A2b66D0Ea7': {'symbol': 'COV', 'decimals': 18},
    '0x4DF812F6064def1e5e029f1ca858777CC98D2D81': {'symbol': 'XAUR', 'decimals': 8},
    '0x35156b404C3f9bdaf45ab65Ba315419bcDe3775c': {'symbol': 'CHIRO', 'decimals': 9},
    '0x2C974B2d0BA1716E644c1FC59982a89DDD2fF724': {'symbol': 'VIB', 'decimals': 18},
    '0xeC5483804e637D45cDe22Fa0869656B64b5AB1ab': {'symbol': 'ACE', 'decimals': 18},
    '0xc40F23A3E9613E012944f7957edCe97899Fa920d': {'symbol': 'DHP', 'decimals': 18},
    '0xB0e977e0214BCfEec0D685064244Ad2b2487380b': {'symbol': 'COLLAR', 'decimals': 18},
    '0x738865301A9b7Dd80Dc3666dD48cF034ec42bdDa': {'symbol': 'AGRS', 'decimals': 8},
    '0xA4C612296aa97057c160B95255f902F9dDe6704c': {'symbol': 'MMC', 'decimals': 18},
    '0x668DbF100635f593A3847c0bDaF21f0a09380188': {'symbol': 'BNSD', 'decimals': 18},
    '0x0FD10b9899882a6f2fcb5c371E17e70FdEe00C38': {'symbol': 'PUNDIX', 'decimals': 18},
    '0x02D3A27Ac3f55d5D91Fb0f52759842696a864217': {'symbol': 'IONX', 'decimals': 18},
    '0x81994b9607e06ab3d5cF3AffF9a67374f05F27d7': {'symbol': 'FUSDT', 'decimals': 8},
    '0xbB126042235E6bD38B17744cb31a8bF4A206c045': {'symbol': 'FANC', 'decimals': 18},
    '0x66eFF5221ca926636224650Fd3B9c497FF828F7D': {'symbol': 'MULTIBTC', 'decimals': 8},
    '0xA3c4Dc4A9cE2a6B40B57F25F8b50DeCc2c64deC2': {'symbol': 'SNFT', 'decimals': 18},
    '0xbA8A621b4a54e61C442F5Ec623687e2a942225ef': {'symbol': 'QUARTZ', 'decimals': 18},
    '0x4E114d405b9Ba2F59524941733e505Ae03Fb1FB5': {'symbol': 'JOY', 'decimals': 18},
    '0x95392f142Af1c12F6e39897Ff9B09c599666B50c': {'symbol': 'BLOOD', 'decimals': 18},
    '0x3A529A8d4f2ea64D206339fA12A3aF4D431F53c3': {'symbol': 'VEN', 'decimals': 18},
    '0xe5feeaC09D36B18b3FA757E5Cf3F8dA6B8e27F4C': {'symbol': 'NFTI', 'decimals': 18},
    '0x31c2415c946928e9FD1Af83cdFA38d3eDBD4326f': {'symbol': 'UMAD', 'decimals': 8},
    '0x805EA9c07B49dd23cE11ec66dC6d8a2957385035': {'symbol': 'TSCT', 'decimals': 18},
    '0x166D45e55143ea15032672Ac9f591A4401CC612D': {'symbol': 'GYOKUTO', 'decimals': 9},
    '0xB8647e90C0645152Fccf4d9AbB6B59Eb4AA99052': {'symbol': 'KEYFI', 'decimals': 18},
    '0x4730fB1463A6F1F44AEB45F6c5c422427f37F4D0': {'symbol': 'FOUR', 'decimals': 18},
    '0xb7cB1C96dB6B22b0D3d9536E0108d062BD488F74': {'symbol': 'WTC', 'decimals': 18},
    '0x77f9CF0bd8C500CfFdF420e72343893aeCC2EC0b': {'symbol': 'LAIKA', 'decimals': 18},
    '0xa02120696c7B8fE16C09C749E4598819b2B0E915': {'symbol': 'WXT', 'decimals': 18},
    '0x2a7e415C169cE3a580c6F374Dc26f6Aaad1EccFe': {'symbol': 'HACHI', 'decimals': 18},
    '0x87DE305311D5788e8da38D19bb427645b09CB4e5': {'symbol': 'VRX', 'decimals': 18},
    '0x62Dc4817588d53a056cBbD18231d91ffCcd34b2A': {'symbol': 'DHV', 'decimals': 18},
    '0x10086399DD8c1e3De736724AF52587a2044c9fA2': {'symbol': 'TMTG', 'decimals': 18},
    '0xE50365f5D679CB98a1dd62D6F6e58e59321BcdDf': {'symbol': 'LA', 'decimals': 18},
    '0x9C7376b5060C47E555f5F53005aA3D5A7C66f2fB': {'symbol': 'SWIFY', 'decimals': 9},
    '0xf56408077487cB879c992909C5b5C66D68c02Eb4': {'symbol': 'RIOT', 'decimals': 18},
    '0xCB0D82f4DFA503C9e3b8Abc7A3Caa01175b2dA39': {'symbol': 'AX', 'decimals': 18},
    '0x62B9c7356A2Dc64a1969e19C23e4f579F9810Aa7': {'symbol': 'CVXCRV', 'decimals': 18},
    '0xfAd45E47083e4607302aa43c65fB3106F1cd7607': {'symbol': 'HOGE', 'decimals': 9},
    '0xD0D42005e7B3c0812b1268F0e5FAF97Ff2423651': {'symbol': 'KOROMARU', 'decimals': 9},
    '0x1D6405138A335cE5FD7364086334EfB3E4F28B59': {'symbol': 'CCX', 'decimals': 18},
    '0xE0C8b298db4cfFE05d1bEA0bb1BA414522B33C1B': {'symbol': 'NCDT', 'decimals': 18},
    '0x24C19F7101c1731b85F1127EaA0407732E36EcDD': {'symbol': 'SGTV2', 'decimals': 18},
    '0x4D4f3715050571A447FfFa2Cd4Cf091C7014CA5c': {'symbol': 'SUMMER', 'decimals': 18},
    '0x10Be9a8dAe441d276a5027936c3aADEd2d82bC15': {'symbol': 'UMX', 'decimals': 18},
    '0x409c4D8cd5d2924b9bc5509230d16a61289c8153': {'symbol': 'TOS', 'decimals': 18},
    '0xc666081073E8DfF8D3d1c2292A29aE1A2153eC09': {'symbol': 'DGTX', 'decimals': 18},
    '0xe88f8313e61A97cEc1871EE37fBbe2a8bf3ed1E4': {'symbol': 'VAL', 'decimals': 18},
    '0x486F4641eF2B50cC130Dadbd27B6F271723873b8': {'symbol': 'GOLD', 'decimals': 18},
    '0x1cBb83EbcD552D5EBf8131eF8c9CD9d9BAB342bC': {'symbol': 'NFY', 'decimals': 18},
    '0xd4fa1460F537bb9085d22C7bcCB5DD450Ef28e3a': {'symbol': 'PPT', 'decimals': 8},
    '0x394A16744Dcd805BB0cA7252e70691f0dcAc56AA': {'symbol': 'TRZ', 'decimals': 18},
    '0x506B8f75BDeF0Edac36B0A6f9CF313485E4341b0': {'symbol': 'WCA', 'decimals': 18},
    '0xfc09c7cFD9c175DD9423ca02aE1249579AB12F12': {'symbol': 'TOTORO', 'decimals': 9},
    '0xf8E9F10c22840b613cdA05A0c5Fdb59A4d6cd7eF': {'symbol': 'DOE', 'decimals': 18},
    '0xAACa86B876ca011844b5798ECA7a67591A9743C8': {'symbol': 'BIOS', 'decimals': 18},
    '0x3758e00b100876C854636eF8Db61988931BB8025': {'symbol': 'UNIQ', 'decimals': 18},
    '0x198065e69A86cb8a9154B333AAD8eFE7A3c256F8': {'symbol': 'KOY', 'decimals': 18},
    '0x412D397DDCa07D753E3E0C61e367fb1b474B3E7D': {'symbol': 'EXE', 'decimals': 18},
    '0x38e3b07D607dEF629b4f3f46a0082006fdd6CDa2': {'symbol': 'SHIBO', 'decimals': 18},
    '0x88800092fF476844f74dC2FC427974BBee2794Ae': {'symbol': 'WALLET', 'decimals': 18},
    '0x94E0BAb2F6Ab1F19F4750E42d7349f2740513aD5': {'symbol': 'UNIC', 'decimals': 18},
    '0x0Ebb614204E47c09B6C3FeB9AAeCad8EE060E23E': {'symbol': 'CPAY', 'decimals': 0},
    '0x692aCCdD8b86692427E0aa4752AE917Df01CC56F': {'symbol': 'SUNC', 'decimals': 18},
    '0x5C8C8D560048F34E5f7f8ad71f2f81a89DBd273e': {'symbol': 'CART', 'decimals': 18},
    '0x5394794Be8b6eD5572FCd6b27103F46b5F390E8f': {'symbol': 'AAMMUNIYFIWETH', 'decimals': 18},
    '0xd779eEA9936B4e323cDdff2529eb6F13d0A4d66e': {'symbol': 'ENTR', 'decimals': 18},
    '0xe0ffDdF8fd7d63Cf907e433c9293069704D4d40D': {'symbol': 'FGHT', 'decimals': 18},
    '0x0f71B8De197A1C84d31de0F1fA7926c365F052B3': {'symbol': 'ARCONA', 'decimals': 18},
    '0xC64500DD7B0f1794807e67802F8Abbf5F8Ffb054': {'symbol': 'LOCUS', 'decimals': 18},
    '0x3832d2F059E55934220881F831bE501D180671A7': {'symbol': 'RENDOGE', 'decimals': 8},
    '0x6f38e0f1a73c96cB3f42598613EA3474F09cB200': {'symbol': 'DAKS', 'decimals': 0},
    '0x08c32b0726C5684024ea6e141C50aDe9690bBdcc': {'symbol': 'STOS', 'decimals': 18},
    '0x2F4404C4012476929b6503E1397707480bf23B7f': {'symbol': 'TAI', 'decimals': 9},
    '0x1afb69DBC9f54d08DAB1bD3436F8Da1af819E647': {'symbol': 'MELOS', 'decimals': 18},
    '0xf2354F740f31704820f6FcfBA70B9dA065459b62': {'symbol': 'ISDT', 'decimals': 18},
    '0x0eC9F76202a7061eB9b3a7D6B59D36215A7e37da': {'symbol': 'BPT', 'decimals': 18},
    '0x327673aE6B33Bd3d90f0096870059994f30Dc8AF': {'symbol': 'LMT', 'decimals': 18},
    '0x1CDD2EaB61112697626F7b4bB0e23Da4FeBF7B7C': {'symbol': 'USDTSO', 'decimals': 6},
    '0xC4e8A9D47000Ab8E59c7031e311762c68215e467': {'symbol': 'STARX', 'decimals': 18},
    '0x616ef40D55C0D2c506f4d6873Bda8090b79BF8fC': {'symbol': 'KTO', 'decimals': 9},
    '0x70eFDc485a10210B056eF8e0A32993Bc6529995E': {'symbol': 'BLZN', 'decimals': 18},
    '0x70401dFD142A16dC7031c56E862Fc88Cb9537Ce0': {'symbol': 'BIRD', 'decimals': 18},
    '0x986EE2B944c42D017F52Af21c4c69B84DBeA35d8': {'symbol': 'BMX', 'decimals': 18},
    '0xdF290B162a7D3E0A328cF198308D421954f08b94': {'symbol': 'BP', 'decimals': 18},
    '0x61107a409fFFe1965126aa456Af679719695C69C': {'symbol': 'UMI', 'decimals': 18},
    '0x3cBb7f5d7499Af626026E96a2f05df806F2200DC': {'symbol': 'PANDA', 'decimals': 18},
    '0x3Ea8ea4237344C9931214796d9417Af1A1180770': {'symbol': 'FLX', 'decimals': 18},
    '0x3543638eD4a9006E4840B105944271Bcea15605D': {'symbol': 'UUU', 'decimals': 18},
    '0x7CDA79830Faf07Ed696Fe220566116951CED36A7': {'symbol': 'MAYP', 'decimals': 18},
    '0x48Fb253446873234F2fEBbF9BdeAA72d9d387f94': {'symbol': 'VBNT', 'decimals': 18},
    '0xaE697F994Fc5eBC000F8e22EbFfeE04612f98A0d': {'symbol': 'LGCY', 'decimals': 18},
    '0x07150e919B4De5fD6a63DE1F9384828396f25fDC': {'symbol': 'BASE', 'decimals': 9},
    '0x38A94e92A19E970c144DEd0B2DD47278CA11CC1F': {'symbol': 'F9', 'decimals': 9},
    '0xbf2179859fc6D5BEE9Bf9158632Dc51678a4100e': {'symbol': 'ELF', 'decimals': 18},
    '0xF970b8E36e23F7fC3FD752EeA86f8Be8D83375A6': {'symbol': 'RCN', 'decimals': 18},
    '0xb5588C411ba0bb7D38865fdC51D082d004e519F7': {'symbol': 'WAIT', 'decimals': 0},
    '0x126c121f99e1E211dF2e5f8De2d96Fa36647c855': {'symbol': 'DEGEN', 'decimals': 18},
    '0x456125Cd98107ae0480Ba566f1b716D48Ba31453': {'symbol': 'CHAMP', 'decimals': 18},
    '0x607C794cDa77efB21F8848B7910ecf27451Ae842': {'symbol': 'PIE', 'decimals': 18},
    '0xD228ceCa8560DDB74137f44D9c582e1a646Be763': {'symbol': 'TB', 'decimals': 9},
    '0x31429d1856aD1377A8A0079410B297e1a9e214c2': {'symbol': 'ANGLE', 'decimals': 18},
    '0xc813EA5e3b48BEbeedb796ab42A30C5599b01740': {'symbol': 'NIOX', 'decimals': 4},
    '0x8baEF8c9568c21B1a2b2fd048F8b4Da835691fD0': {'symbol': 'USDZ', 'decimals': 18},
    '0x2ba592F78dB6436527729929AAf6c908497cB200': {'symbol': 'CREAM', 'decimals': 18},
    '0x505a84a03e382331A1Be487b632Cf357748b65d6': {'symbol': 'SHIBGF', 'decimals': 18},
    '0x1b5036bEc1B82D44d52Fa953A370b3c6Cd9328B5': {'symbol': 'ELAN', 'decimals': 18},
    '0x8eB1779A32678452eB273A22d413207299904d90': {'symbol': 'POCHI', 'decimals': 18},
    '0x6caDf6ABBCeb53e631C288778DAAcF125481C1BB': {'symbol': 'CITADEL', 'decimals': 18},
    '0x0cEC1A9154Ff802e7934Fc916Ed7Ca50bDE6844e': {'symbol': 'POOL', 'decimals': 18},
    '0x4f81C790581b240A5C948Afd173620EcC8C71c8d': {'symbol': 'XDG', 'decimals': 18},
    '0xbc6E06778708177a18210181b073DA747C88490a': {'symbol': 'SYNR', 'decimals': 18},
    '0x90DE74265a416e1393A450752175AED98fe11517': {'symbol': 'UDT', 'decimals': 18},
    '0xce20BB92CCF9BbF5091ef85649E71E552819AD8c': {'symbol': 'SMART', 'decimals': 18},
    '0x6ADb2E268de2aA1aBF6578E4a8119b960E02928F': {'symbol': 'SHIBDOGE', 'decimals': 9},
    '0xae78736Cd615f374D3085123A210448E74Fc6393': {'symbol': 'RETH', 'decimals': 18},
    '0x7690202e2C2297bcD03664e31116d1dFfE7e3B73': {'symbol': 'BOXETH', 'decimals': 18},
    '0x662b67d00A13FAf93254714DD601F5Ed49Ef2F51': {'symbol': 'ORC', 'decimals': 18},
    '0x40EB746DEE876aC1E78697b7Ca85142D178A1Fc8': {'symbol': 'IAG', 'decimals': 18},
    '0xf03A7Eb46d01d9EcAA104558C732Cf82f6B6B645': {'symbol': 'MATICX', 'decimals': 18},
    '0x925206b8a707096Ed26ae47C84747fE0bb734F59': {'symbol': 'WBT', 'decimals': 8},
    '0x4C2e59D098DF7b6cBaE0848d66DE2f8A4889b9C3': {'symbol': 'FODL', 'decimals': 18},
    '0xEda8B016efA8b1161208Cf041cD86972eeE0F31E': {'symbol': 'IHT', 'decimals': 18},
    '0x6FF2241756549B5816A177659E766EAf14B34429': {'symbol': 'AQTIS', 'decimals': 18},
    '0x8d610E20481F4C4f3aCB87bBa9c46BeF7795fdFe': {'symbol': 'UNT', 'decimals': 18},
    '0x04DC37B220A055c5F93680815F670babCD912c2C': {'symbol': 'ODA', 'decimals': 9},
    '0xC8C424B91D8ce0137bAB4B832B7F7D154156BA6c': {'symbol': 'APM', 'decimals': 18},
    '0xcB84d72e61e383767C4DFEb2d8ff7f4FB89abc6e': {'symbol': 'VEGA', 'decimals': 18},
    '0x9cB1AEaFcc8A9406632C5B084246Ea72f62d37b6': {'symbol': 'LBK', 'decimals': 8},
    '0xB67718b98d52318240c52E71A898335da4A28c42': {'symbol': 'INNBC', 'decimals': 6},
    '0x2d886570A0dA04885bfD6eb48eD8b8ff01A0eb7e': {'symbol': 'BCB', 'decimals': 9},
    '0x8400D94A5cb0fa0D041a3788e395285d61c9ee5e': {'symbol': 'UBT', 'decimals': 8},
    '0x3Ab6Ed69Ef663bd986Ee59205CCaD8A20F98b4c2': {'symbol': 'DREP', 'decimals': 18},
    '0xCcDb064a41AFcD432BCEa6F6d0D1E7c371d0B002': {'symbol': 'QR', 'decimals': 9},
    '0x2a2550e0A75aCec6D811AE3930732F7f3ad67588': {'symbol': 'PATH', 'decimals': 18},
    '0x35b08722AA26bE119c1608029CcbC976ac5C1082': {'symbol': 'EM', 'decimals': 8},
    '0x0E192d382a36De7011F795Acc4391Cd302003606': {'symbol': 'FST', 'decimals': 18},
    '0xd82Df0ABD3f51425Eb15ef7580fDA55727875f14': {'symbol': 'DAV', 'decimals': 18},
    '0xc834Fa996fA3BeC7aAD3693af486ae53D8aA8B50': {'symbol': 'CONV', 'decimals': 18},
    '0x1a7e4e63778B4f12a199C062f3eFdD288afCBce8': {'symbol': 'AGEUR', 'decimals': 18},
    '0x4FC15c91a9c4A9efB404174464687E8e128730C2': {'symbol': 'STAT', 'decimals': 18},
    '0x6F620EC89B8479e97A6985792d0c64F237566746': {'symbol': 'WPC', 'decimals': 18},
    '0x6A6AA13393B7d1100c00a57c76c39e8B6C835041': {'symbol': 'OPENAIERC', 'decimals': 9},
    '0x6b32022693210cD2Cfc466b9Ac0085DE8fC34eA6': {'symbol': 'DECI', 'decimals': 8},
    '0x1900E8B5619a3596745F715d0427Fe617c729BA9': {'symbol': 'CBG', 'decimals': 18},
    '0xb32aC3C79A94aC1eb258f3C830bBDbc676483c93': {'symbol': 'OSWAP', 'decimals': 18},
    '0xb48EE8F8FA3885Fc08831ff366ec8df608159EE9': {'symbol': 'LOCK', 'decimals': 18},
    '0xC9fE6E1C76210bE83DC1B5b20ec7FD010B0b1D15': {'symbol': 'FRIN', 'decimals': 18},
    '0xB7c9E99Da8A857cE576A830A9c19312114d9dE02': {'symbol': 'TEAM', 'decimals': 8},
    '0xe6602B34D8510B033E000975B3322537c7172441': {'symbol': 'FRR', 'decimals': 18},
    '0xEa4a2327E75252517535fd013b7C6706609819DB': {'symbol': 'SNS', 'decimals': 18},
    '0x956F47F50A910163D8BF957Cf5846D573E7f87CA': {'symbol': 'FEI', 'decimals': 18},
    '0xF59ae934f6fe444afC309586cC60a84a0F89Aaea': {'symbol': 'PDEX', 'decimals': 18},
    '0x07e3c70653548B04f0A75970C1F81B4CBbFB606f': {'symbol': 'DLT', 'decimals': 18},
    '0x91dFbEE3965baAEE32784c2d546B7a0C62F268c9': {'symbol': 'BONDLY', 'decimals': 18},
    '0x0A58153a0CD1cfaea94cE1f7FdC5D7E679eCa936': {'symbol': 'IM', 'decimals': 18},
    '0x4C6eC08CF3fc987c6C4BEB03184D335A2dFc4042': {'symbol': 'PAINT', 'decimals': 18},
    '0x9C32185b81766a051E08dE671207b34466DD1021': {'symbol': 'BTCPX', 'decimals': 8},
    '0xBC6DA0FE9aD5f3b0d58160288917AA56653660E9': {'symbol': 'ALUSD', 'decimals': 18},
    '0xAa6E8127831c9DE45ae56bB1b0d4D4Da6e5665BD': {'symbol': 'ETH2X-FLI', 'decimals': 18},
    '0x7777777777697cFEECF846A76326dA79CC606517': {'symbol': 'SIG', 'decimals': 18},
    '0x06B884e60794Ce02AafAb13791B59A2e6A07442f': {'symbol': 'UNBNK', 'decimals': 18},
    '0x491E136FF7FF03E6aB097E54734697Bb5802FC1C': {'symbol': 'KTN', 'decimals': 18},
    '0xDD2a36AE937bc134ea694d77fC7e2E36f5d86dE0': {'symbol': 'WELD', 'decimals': 18},
    '0x1b396CF90504686b7edE2760eebb053d30d1f389': {'symbol': 'RLTM', 'decimals': 18},
    '0xB1f66997A5760428D3a87D68b90BfE0aE64121cC': {'symbol': 'LUA', 'decimals': 18},
    '0x1B073382E63411E3BcfFE90aC1B9A43feFa1Ec6F': {'symbol': 'BEST', 'decimals': 8},
    '0x49849C98ae39Fff122806C06791Fa73784FB3675': {'symbol': 'RENBTCCURVE', 'decimals': 18},
    '0x73B708E84837FFCcDE2933e3A1531fe61D5e80Ef': {'symbol': 'BTE', 'decimals': 18},
    '0x6Ef6610d24593805144d73b13d4405E00A4E4aC7': {'symbol': 'DIE', 'decimals': 18},
    '0x71fc860F7D3A592A4a98740e39dB31d25db65ae8': {'symbol': 'AUSDT', 'decimals': 6},
    '0x65Def5029A0e7591e46B38742bFEdd1Fb7b24436': {'symbol': 'KAE', 'decimals': 18},
    '0xca1207647Ff814039530D7d35df0e1Dd2e91Fa84': {'symbol': 'DHT', 'decimals': 18},
    '0x4057Db5bD9f67A566aA10E5587b1a964afFc6a16': {'symbol': 'TFBX', 'decimals': 18},
    '0x59E9261255644c411AfDd00bD89162d09D862e38': {'symbol': 'ETHA', 'decimals': 18},
    '0x20c36f062a31865bED8a5B1e512D9a1A20AA333A': {'symbol': 'DFD', 'decimals': 18},
    '0xa393473d64d2F9F026B60b6Df7859A689715d092': {'symbol': 'LTX', 'decimals': 8},
    '0x8eEF5a82E6Aa222a60F009ac18c24EE12dBf4b41': {'symbol': 'TXL', 'decimals': 18},
    '0x866f8a50A64e68ca66E97e032c5Da99538B3F942': {'symbol': 'EBSO', 'decimals': 4},
    '0xA9Aad2dC3a8315caeee5F458B1d8EDc31D8467BD': {'symbol': 'BTCM', 'decimals': 18},
    '0xBd2949F67DcdC549c6Ebe98696449Fa79D988A9F': {'symbol': 'EMTRG', 'decimals': 18},
    '0xD502F487e1841Fdc805130e13eae80c61186Bc98': {'symbol': 'ITGR', 'decimals': 18},
    '0xA01199c61841Fce3b3daFB83FeFC1899715c8756': {'symbol': 'CIRUS', 'decimals': 18},
    '0xb624FdE1a972B1C89eC1dAD691442d5E8E891469': {'symbol': 'SPORK', 'decimals': 18},
    '0x8bbe1a2961B41340468D0548c2cd5B7DFA9b684c': {'symbol': 'HANDY', 'decimals': 18},
    '0xDc5864eDe28BD4405aa04d93E05A0531797D9D59': {'symbol': 'FNT', 'decimals': 6},
    '0x723CbfC05e2cfcc71d3d89e770D32801A5eEf5Ab': {'symbol': 'BTCP', 'decimals': 8},
    '0xDDdddd4301A082e62E84e43F474f044423921918': {'symbol': 'DVF', 'decimals': 18},
    '0xde4EE8057785A7e8e800Db58F9784845A5C2Cbd6': {'symbol': 'DEXE', 'decimals': 18},
    '0xfffffffFf15AbF397dA76f1dcc1A1604F45126DB': {'symbol': 'FSW', 'decimals': 18},
    '0x347a96a5BD06D2E15199b032F46fB724d6c73047': {'symbol': 'ASIC', 'decimals': 12},
    '0x1b40183EFB4Dd766f11bDa7A7c3AD8982e998421': {'symbol': 'VSP', 'decimals': 18},
    '0xd6710559656f2ecD336b153B0719500Fff8C2435': {'symbol': 'PLAQ', 'decimals': 18},
    '0x464FdB8AFFC9bac185A7393fd4298137866DCFB8': {'symbol': 'REALM', 'decimals': 18},
    '0xb59490aB09A0f526Cc7305822aC65f2Ab12f9723': {'symbol': 'LIT', 'decimals': 18},
    '0x922e2708462c7a3d014D8344F7C4d92b27ECf332': {'symbol': 'NEURONI', 'decimals': 18},
    '0xa27A6FA698a63f82e559914c11C6Cd888060660b': {'symbol': 'SHIN', 'decimals': 8},
    '0xb17C88bDA07D28B3838E0c1dE6a30eAfBCF52D85': {'symbol': 'SHFT', 'decimals': 18},
    '0x8e14c88aB0644eF41bd7138ab91C0160D8c1583A': {'symbol': 'FREN', 'decimals': 18},
    '0x36ac219f90f5A6A3C77f2a7B660E3cC701f68e25': {'symbol': 'XCM', 'decimals': 18},
    '0xE9b7B5D5e8d2BCC78884F9F9099bfA42A9e5c1A5': {'symbol': 'ZENF', 'decimals': 18},
    '0x54Cb643ab007f47882E8120A8c57B639005c2688': {'symbol': 'IXP', 'decimals': 18},
    '0xBd3de9a069648c84d27d74d701C9fa3253098B15': {'symbol': 'EQX', 'decimals': 18},
    '0x4575f41308EC1483f3d399aa9a2826d74Da13Deb': {'symbol': 'OXT', 'decimals': 18},
    '0x64Df3aAB3b21cC275bB76c4A581Cf8B726478ee0': {'symbol': 'CRAMER', 'decimals': 18},
    '0x8B3870Df408fF4D7C3A26DF852D41034eDa11d81': {'symbol': 'IOI', 'decimals': 6},
    '0x38e4adB44ef08F22F5B5b76A8f0c2d0dCbE7DcA1': {'symbol': 'CVP', 'decimals': 18},
    '0xaa61D5dec73971CD4a026ef2820bB87b4a4Ed8d6': {'symbol': 'CRE8R', 'decimals': 18},
    '0x6dDe4fFD6dB302Bc9a46850f61399e082f6c2122': {'symbol': 'IAI', 'decimals': 18},
    '0x356A5160F2B34BC8d88FB084745465eBBbed0174': {'symbol': 'INVI', 'decimals': 13},
    '0x9cc83D580180f0d37d00e5d86ce868f73b6E3D0A': {'symbol': 'BTTY', 'decimals': 18},
    '0xc98D64DA73a6616c42117b582e832812e7B8D57F': {'symbol': 'RSS3', 'decimals': 18},
    '0x0ff5A8451A839f5F0BB3562689D9A44089738D11': {'symbol': 'RDPX', 'decimals': 18},
    '0xe8e8486228753E01Dbc222dA262Aa706Bd67e601': {'symbol': 'WEB3', 'decimals': 18},
    '0x9471d30D78a3C9f076cE206D14867a8d8BE1eFdE': {'symbol': 'ZNX', 'decimals': 6},
    '0xBd0793332e9fB844A52a205A233EF27a5b34B927': {'symbol': 'ZB', 'decimals': 18},
    '0x9301466c40B94942aF9ba63cbaEA1c784FF219B2': {'symbol': 'ECTR', 'decimals': 18},
    '0xFeeB4D0f5463B1b04351823C246bdB84c4320CC2': {'symbol': 'GLDN', 'decimals': 18},
    '0x2786F9C443F323f590dE1881d7215DB305aB846e': {'symbol': 'NKPL', 'decimals': 18},
    '0x43Dfc4159D86F3A37A5A4B3D4580b888ad7d4DDd': {'symbol': 'DODO', 'decimals': 18},
    '0x7453b01D746A72b6397e77C7d9de969FDBdE5a99': {'symbol': 'IJZ', 'decimals': 18},
    '0x6347978f54F75d76630b819897F9Ce68C83ce2a2': {'symbol': 'CORD', 'decimals': 9},
    '0x2365a4890eD8965E564B7E2D27C38Ba67Fec4C6F': {'symbol': 'AAMMUNIWBTCUSDC', 'decimals': 18},
    '0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490': {'symbol': '3CRV', 'decimals': 18},
    '0x0AF5855a348C58a8BbF65BCc7B9BBaCD0e7e2238': {'symbol': 'UAC', 'decimals': 18},
    '0xd31695a1d35E489252CE57b129FD4b1B05E6AcaC': {'symbol': 'TKP', 'decimals': 18},
    '0x4E12EB8e506Ccd1427F6b8F7faa3e88fB698EB28': {'symbol': 'JACK', 'decimals': 18},
    '0x87eDfFDe3E14c7a66c9b9724747a1C5696b742e6': {'symbol': 'SWAG', 'decimals': 18},
    '0x78A52E12c7b63d05c12F9608307587CF654EC3d0': {'symbol': 'CVA', 'decimals': 18},
    '0x35F67c1D929E106FDfF8D1A55226AFe15c34dbE2': {'symbol': 'BETA', 'decimals': 18},
    '0xe9D0331AD47148925c5009D33a428eE301C9D534': {'symbol': 'HRDG', 'decimals': 12},
    '0x68C85B9f78F30A0df5aC5723e4e700037f185415': {'symbol': 'AWO', 'decimals': 18},
    '0xFB19075D77a0F111796FB259819830F4780f1429': {'symbol': 'FB', 'decimals': 6},
    '0x7A939Bb714fd2A48EbeB1E495AA9aaa74BA9fA68': {'symbol': 'EVZ', 'decimals': 18},
    '0x645C52CF6c5C58AA4064494f5b5FFE9C7EC0d7D4': {'symbol': 'HKBY', 'decimals': 18},
    '0xfcA47962D45ADFdfd1Ab2D972315dB4ce7CCf094': {'symbol': 'IXT', 'decimals': 8},
    '0xf0Ee6b27b759C9893Ce4f094b49ad28fd15A23e4': {'symbol': 'ENG', 'decimals': 8},
    '0x2bD0FB740e403B505A3146F9aC02df883Fd5C3Fc': {'symbol': 'K9', 'decimals': 18},
    '0x8326Bf664704966C984a3A46Fa37d7A80A52dCf4': {'symbol': 'DOGU', 'decimals': 18},
    '0x36c833Eed0D376f75D1ff9dFDeE260191336065e': {'symbol': 'GTCETH', 'decimals': 18},
    '0x6d60A8dFb16D09F67D46fcd36A0Cd310078257ca': {'symbol': 'CIX', 'decimals': 18},
    '0xa1d0E215a23d7030842FC67cE582a6aFa3CCaB83': {'symbol': 'YFII', 'decimals': 18},
    '0x076641aF1B8f06B7f8C92587156143C109002cbe': {'symbol': 'SOP', 'decimals': 18},
    '0x7DbbCaE15d4DB168e01673400D7844870cc1e36f': {'symbol': 'WOLFY', 'decimals': 9},
    '0x7995ab36bB307Afa6A683C24a25d90Dc1Ea83566': {'symbol': 'HIT', 'decimals': 6},
    '0x89551b940e2A8ED8eCcF509935bAc9213fE30584': {'symbol': 'DRM', 'decimals': 18},
    '0x39d30828a163713D91c4EADBBA2C497a9139ec5C': {'symbol': 'HBDC', 'decimals': 18},
    '0x72a66e54B66892aE3BBe54DF7bb7Dd5Ae927a6f9': {'symbol': 'CVAG', 'decimals': 18},
    '0xb892249939AdBf6D7851864CA9A5c7D2d537af97': {'symbol': 'GSHIBA', 'decimals': 18},
    '0x883a158c9b28f8d626ACFCFbE1028f49E70c9D75': {'symbol': 'CNG', 'decimals': 18},
    '0x8eEcaad83a1Ea77bD88A818d4628fAfc4CaD7969': {'symbol': 'NFAI', 'decimals': 18},
    '0xf6043bEa98B07F0bEa7813aeB22D0cb70C91C0c4': {'symbol': 'MDAI', 'decimals': 18},
    '0xB13dE094Cc5CEe6C4cC0A3737bf0290166D9Ca5D': {'symbol': 'GMAT', 'decimals': 18},
    '0x2a304FDa5a85182dCA1D03741bB2F07881b9e095': {'symbol': 'DCO', 'decimals': 8},
    '0x8d2BFfCbB19Ff14A698C424FbcDcFd17aab9b905': {'symbol': 'UPUNK', 'decimals': 18},
    '0xB70835D7822eBB9426B56543E391846C107bd32C': {'symbol': 'GTC', 'decimals': 18},
    '0x4A5dec561de897c04062137e687db6093C3278B8': {'symbol': 'GSBL', 'decimals': 18},
    '0x87d73E916D7057945c9BcD8cdd94e42A6F47f776': {'symbol': 'NFTX', 'decimals': 18},
    '0x3F3Cd642E81d030D7b514a2aB5e3a5536bEb90Ec': {'symbol': 'RHO', 'decimals': 18},
    '0x0A41E6d4D4897E8Cec819ce7D10B2D7cDa61Dc94': {'symbol': 'SYC', 'decimals': 18},
    '0xc8871267e07408b89aA5aEcc58AdCA5E574557F8': {'symbol': 'IUSDC', 'decimals': 6},
    '0x341c05c0E9b33C0E38d64de76516b2Ce970bB3BE': {'symbol': 'DSETH', 'decimals': 18},
    '0x72aDadb447784dd7AB1F472467750fC485e4cb2d': {'symbol': 'WRC', 'decimals': 6},
    '0xE9E73E1aE76D17A16cC53E3e87a9a7dA78834d37': {'symbol': 'CAMP', 'decimals': 18},
    '0xfe5F141Bf94fE84bC28deD0AB966c16B17490657': {'symbol': 'LBA', 'decimals': 18},
    '0x1C9922314ED1415c95b9FD453c3818fd41867d0B': {'symbol': 'TOWER', 'decimals': 18},
    '0x095797FD4297fb79883cC912a5Ba6313B15c445d': {'symbol': 'GOLC', 'decimals': 18},
    '0x69Ed89ecd35082E031fE52b75123F801DB083306': {'symbol': 'KAERI', 'decimals': 9},
    '0xF7970499814654CD13Cb7B6E7634A12a7A8A9ABc': {'symbol': 'TOM', 'decimals': 18},
    '0xDFDb7f72c1F195C5951a234e8DB9806EB0635346': {'symbol': 'NFD', 'decimals': 18},
    '0xA91464AbD4625A23aB719e3F0FCE84DaDd54E546': {'symbol': 'IVI', 'decimals': 18},
    '0x4a9d711100afF9A444a3c40284F70269Bb3f0487': {'symbol': 'CHLT', 'decimals': 18},
    '0xF90c7f66eAc7E2130bF677D69a250b2136Cf6697': {'symbol': 'HAKI', 'decimals': 18},
    '0xA64dFe8D86963151E6496BEe513E366F6e42ED79': {'symbol': 'GOKU', 'decimals': 9},
    '0x2c000c0093dE75a8fA2FccD3d97b314e20b431C3': {'symbol': 'HLTC', 'decimals': 18},
    '0x593114f03A0A575aece9ED675e52Ed68D2172B8c': {'symbol': 'BDP', 'decimals': 18},
    '0xdC47f2Ba852669B178699449E50682D6CEAF8C07': {'symbol': 'STON', 'decimals': 18},
    '0xaAC679720204aAA68B6C5000AA87D789a3cA0Aa5': {'symbol': 'HBCH', 'decimals': 18},
    '0x990f341946A3fdB507aE7e52d17851B87168017c': {'symbol': 'STRONG', 'decimals': 18},
    '0x301C755bA0fcA00B1923768Fffb3Df7f4E63aF31': {'symbol': 'GDC', 'decimals': 18},
    '0x6e8908cfa881C9f6f2C64d3436E7b80b1bf0093F': {'symbol': 'BIST', 'decimals': 18},
    '0x6247C86B016Bc4d9aE141849C0a9Eb38C004B742': {'symbol': 'HTL', 'decimals': 18},
    '0x80C62FE4487E1351b47Ba49809EBD60ED085bf52': {'symbol': 'CLV', 'decimals': 18},
    '0x3b484b82567a09e2588A13D54D032153f0c0aEe0': {'symbol': 'SOS', 'decimals': 18},
    '0xf65B5C5104c4faFD4b709d9D60a185eAE063276c': {'symbol': 'TRU', 'decimals': 18},
    '0x362bc847A3a9637d3af6624EeC853618a43ed7D2': {'symbol': 'PRQ', 'decimals': 18},
    '0x1E053D89E08c24aA2CE5C5B4206744Dc2D7BD8F5': {'symbol': 'TT', 'decimals': 18},
    '0x17d2628D30F8e9E966c9Ba831c9B9b01ea8Ea75C': {'symbol': 'ISK', 'decimals': 18},
    '0xe53EC727dbDEB9E2d5456c3be40cFF031AB40A55': {'symbol': 'SUPER', 'decimals': 18},
    '0x5Ca381bBfb58f0092df149bD3D243b08B9a8386e': {'symbol': 'MXC', 'decimals': 18},
    '0x9B2b931d6AB97B6A887b2C5d8529537E6FE73Ebe': {'symbol': 'ALLIN', 'decimals': 9},
    '0xa0246c9032bC3A600820415aE600c6388619A14D': {'symbol': 'FARM', 'decimals': 18},
    '0x1796ae0b0fa4862485106a0de9b654eFE301D0b2': {'symbol': 'PMON', 'decimals': 18},
    '0x4B1E80cAC91e2216EEb63e29B957eB91Ae9C2Be8': {'symbol': 'JUP', 'decimals': 18},
    '0x83e6f1E41cdd28eAcEB20Cb649155049Fac3D5Aa': {'symbol': 'POLS', 'decimals': 18},
    '0xa1817B6d8D890F3943b61648992730373B71f156': {'symbol': 'MONGOOSE', 'decimals': 9},
    '0x196f4727526eA7FB1e17b2071B3d8eAA38486988': {'symbol': 'RSV', 'decimals': 18},
    '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2': {'symbol': 'WETH', 'decimals': 18},
    '0x3Dd98C8A089dBCFF7e8FC8d4f532BD493501Ab7F': {'symbol': 'TOWN', 'decimals': 8},
    '0xfB7B4564402E5500dB5bB6d63Ae671302777C75a': {'symbol': 'DEXT', 'decimals': 18},
    '0x68a3637bA6E75c0f66B61A42639c4e9fCD3D4824': {'symbol': 'MOON', 'decimals': 18},
    '0xE1D7C7a4596B038CEd2A84bF65B8647271C53208': {'symbol': 'NFTY', 'decimals': 18},
    '0x728f30fa2f100742C7949D1961804FA8E0B1387d': {'symbol': 'GHX', 'decimals': 18},
    '0x67675239Fa58c84e75f947c14f566842Dccb69Ae': {'symbol': 'PIXIA', 'decimals': 18},
    '0xe5597f0723EeaBa1B26948e06f008Bf0fc1e37e6': {'symbol': 'GM', 'decimals': 18},
    '0x1776e1F26f98b1A5dF9cD347953a26dd3Cb46671': {'symbol': 'NMR', 'decimals': 18},
    '0x8B39B70E39Aa811b69365398e0aACe9bee238AEb': {'symbol': 'PKF', 'decimals': 18},
    '0x00c83aeCC790e8a4453e5dD3B0B4b3680501a7A7': {'symbol': 'SKL', 'decimals': 18},
    '0x4688a8b1F292FDaB17E9a90c8Bc379dC1DBd8713': {'symbol': 'COVER', 'decimals': 18},
    '0xd49EFA7BC0D339D74f487959C573d518BA3F8437': {'symbol': 'COLI', 'decimals': 18},
    '0x08d967bb0134F2d07f7cfb6E246680c53927DD30': {'symbol': 'MATH', 'decimals': 18},
    '0xDaF88906aC1DE12bA2b1D2f7bfC94E9638Ac40c4': {'symbol': 'EPK', 'decimals': 18},
    '0xf2051511B9b121394FA75B8F7d4E7424337af687': {'symbol': 'HAUS', 'decimals': 18},
    '0xE0e05c43c097B0982Db6c9d626c4eb9e95C3b9ce': {'symbol': 'USF', 'decimals': 18},
    '0xed328E9C1179a30ddC1E7595E036AEd8760C22aF': {'symbol': 'MCADE', 'decimals': 18},
    '0x20945cA1df56D237fD40036d47E866C7DcCD2114': {'symbol': 'NSURE', 'decimals': 18},
    '0x474021845C4643113458ea4414bdb7fB74A01A77': {'symbol': 'UNO', 'decimals': 18},
    '0x6E39A587691b8c9D4341cE0A960998ED6F537AF6': {'symbol': 'MEMAG', 'decimals': 18},
    '0x544c42fBB96B39B21DF61cf322b5EDC285EE7429': {'symbol': 'INSUR', 'decimals': 18},
    '0x57B946008913B82E4dF85f501cbAeD910e58D26C': {'symbol': 'POND', 'decimals': 18},
    '0xDE5ed76E7c05eC5e4572CfC88d1ACEA165109E44': {'symbol': 'DEUS', 'decimals': 18},
    '0x8a854288a5976036A725879164Ca3e91d30c6A1B': {'symbol': 'GET', 'decimals': 18},
    '0xA91ac63D040dEB1b7A5E4d4134aD23eb0ba07e14': {'symbol': 'BEL', 'decimals': 18},
    '0x0BA74fb26Ca523f2dc22fa4318581CC2452eAbA1': {'symbol': 'BOG', 'decimals': 18},
    '0x6fB3e0A217407EFFf7Ca062D46c26E5d60a14d69': {'symbol': 'IOTX', 'decimals': 18},
    '0x1aBaEA1f7C830bD89Acc67eC4af516284b1bC33c': {'symbol': 'EUROC', 'decimals': 6},
    '0x881Ba05de1E78f549cC63a8f6Cabb1d4AD32250D': {'symbol': '00', 'decimals': 18},
    '0xD8912C10681D8B21Fd3742244f44658dBA12264E': {'symbol': 'PLU', 'decimals': 18},
    '0xFe459828c90c0BA4bC8b42F5C5D44F316700B430': {'symbol': 'BBS', 'decimals': 18},
    '0x9fa69536d1cda4A04cFB50688294de75B505a9aE': {'symbol': 'DERC', 'decimals': 18},
    '0x7dA2641000Cbb407C329310C461b2cB9c70C3046': {'symbol': 'AGI', 'decimals': 18},
    '0x1A963Df363D01EEBB2816b366d61C917F20e1EbE': {'symbol': 'MEMEME', 'decimals': 18},
    '0x5fAa989Af96Af85384b8a938c2EdE4A7378D9875': {'symbol': 'GAL', 'decimals': 18},
    '0xC8807f0f5BA3fa45FfBdc66928d71c5289249014': {'symbol': 'ISP', 'decimals': 18},
    '0xE60779CC1b2c1d0580611c526a8DF0E3f870EC48': {'symbol': 'USH', 'decimals': 18},
    '0xDf7FF54aAcAcbFf42dfe29DD6144A69b629f8C9e': {'symbol': 'AZRX', 'decimals': 18},
    '0x50Eec6D765792dCfB0913C8403ef2A12e1B861a6': {'symbol': 'Z3', 'decimals': 18},
    '0x31c8EAcBFFdD875c74b94b077895Bd78CF1E64A3': {'symbol': 'RAD', 'decimals': 18},
    '0x2C537E5624e4af88A7ae4060C022609376C8D0EB': {'symbol': 'TRYB', 'decimals': 6},
    '0x2F141Ce366a2462f02cEA3D12CF93E4DCa49e4Fd': {'symbol': 'FREE', 'decimals': 18},
    '0x19de6b897Ed14A376Dda0Fe53a5420D2aC828a28': {'symbol': 'BGB', 'decimals': 18},
    '0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e': {'symbol': 'YFI', 'decimals': 18},
    '0x9F9c8ec3534c3cE16F928381372BfbFBFb9F4D24': {'symbol': 'GLQ', 'decimals': 18},
    '0xc0a4Df35568F116C370E6a6A6022Ceb908eedDaC': {'symbol': 'UMEE', 'decimals': 6},
    '0xA735A3AF76CC30791C61c10d585833829d36CBe0': {'symbol': 'IMGNAI', 'decimals': 9},
    '0xFE3E6a25e6b192A42a44ecDDCd13796471735ACf': {'symbol': 'REEF', 'decimals': 18},
    '0x5a666c7d92E5fA7Edcb6390E4efD6d0CDd69cF37': {'symbol': 'MARSH', 'decimals': 18},
    '0x42bBFa2e77757C645eeaAd1655E0911a7553Efbc': {'symbol': 'BOBA', 'decimals': 18},
    '0xDC349913d53b446485E98b76800b6254f43Df695': {'symbol': 'BEZOGE', 'decimals': 9},
    '0xA4Eb9C64eC359D093eAc7B65F51Ef933D6e5F7cd': {'symbol': 'STABLZ', 'decimals': 18},
    '0x3593D125a4f7849a1B059E64F4517A86Dd60c95d': {'symbol': 'OM', 'decimals': 18},
    '0x808507121B80c02388fAd14726482e061B8da827': {'symbol': 'PENDLE', 'decimals': 18},
    '0x12b6893cE26Ea6341919FE289212ef77e51688c8': {'symbol': 'TAMA', 'decimals': 18},
    '0xAdc3F2C3D728202658930860158C726d8180a38F': {'symbol': 'SMETA', 'decimals': 18},
    '0x34f797e7190C131cF630524655A618b5BD8738e7': {'symbol': 'BACON', 'decimals': 18},
    '0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b': {'symbol': 'DPI', 'decimals': 18},
    '0x2dE509bf0014ddF697b220bE628213034d320EcE': {'symbol': 'DBI', 'decimals': 18},
    '0x64A60493D888728Cf42616e034a0dfEAe38EFCF0': {'symbol': 'OLT', 'decimals': 18},
    '0x52A8845DF664D76C69d2EEa607CD793565aF42B8': {'symbol': 'APEX', 'decimals': 18},
    '0x4Dd942bAa75810a3C1E876e79d5cD35E09C97A76': {'symbol': 'D2T', 'decimals': 18},
    '0x9D0B65a76274645B29e4cc41B8f23081fA09f4A3': {'symbol': 'LIME', 'decimals': 18},
    '0x037A54AaB062628C9Bbae1FDB1583c195585fe41': {'symbol': 'LCX', 'decimals': 18},
    '0x6D614686550B9E1c1DF4b2cD8f91C9D4df66C810': {'symbol': 'SKEB', 'decimals': 18},
    '0x03ab458634910AaD20eF5f1C8ee96F1D6ac54919': {'symbol': 'RAI', 'decimals': 18},
    '0x2c7f442AaB99D5e18CfAE2291c507c0b5f3c1eb5': {'symbol': 'KEKO', 'decimals': 18},
    '0xBAac2B4491727D78D2b78815144570b9f2Fe8899': {'symbol': 'DOG', 'decimals': 18},
    '0x45080a6531d671DDFf20DB42f93792a489685e32': {'symbol': 'FVT', 'decimals': 18},
    '0x89Ab32156e46F46D02ade3FEcbe5Fc4243B9AAeD': {'symbol': 'PNT', 'decimals': 18},
    '0x0Ae055097C6d159879521C384F1D2123D1f195e6': {'symbol': 'STAKE', 'decimals': 18},
    '0xa589d8868607b8d79eE4288ce192796051263b64': {'symbol': 'TATE', 'decimals': 18},
    '0xEd04915c23f00A313a544955524EB7DBD823143d': {'symbol': 'ACH', 'decimals': 8},
    '0x96610186F3ab8d73EBEe1CF950C750f3B1Fb79C2': {'symbol': 'EJS', 'decimals': 18},
    '0xd26114cd6EE289AccF82350c8d8487fedB8A0C07': {'symbol': 'OMG', 'decimals': 18},
    '0x5888641e3e6cBeA6D84Ba81EDb217bD691d3bE38': {'symbol': 'BOBO', 'decimals': 9},
    '0xAD8D0de33C43eEFe104A279cDB6Ae250C12e6214': {'symbol': 'NARUTO', 'decimals': 9},
    '0x29CbD0510EEc0327992CD6006e63F9Fa8E7f33B7': {'symbol': 'TIDAL', 'decimals': 18},
    '0x888888848B652B3E3a0f34c96E00EEC0F3a23F72': {'symbol': 'TLM', 'decimals': 4},
    '0x66761Fa41377003622aEE3c7675Fc7b5c1C2FaC5': {'symbol': 'CPOOL', 'decimals': 18},
    '0x92D6C1e31e14520e676a687F0a93788B716BEff5': {'symbol': 'DYDX', 'decimals': 18},
    '0xCd7492db29E2ab436e819b249452EE1bbDf52214': {'symbol': 'SMI', 'decimals': 8},
    '0x08ba718F288c3b12B01146816bef9FA03cC635bc': {'symbol': 'CENT', 'decimals': 18},
    '0x7d5121505149065b562C789A0145eD750e6E8cdD': {'symbol': 'VR', 'decimals': 18},
    '0xd38BB40815d2B0c2d2c866e0c72c5728ffC76dd9': {'symbol': 'SIS', 'decimals': 18},
    '0x3A856d4effa670C54585a5D523e96513e148e95d': {'symbol': 'TRIAS', 'decimals': 18},
    '0x54012cDF4119DE84218F7EB90eEB87e25aE6EBd7': {'symbol': 'LUFFY', 'decimals': 9},
    '0xcfcEcFe2bD2FED07A9145222E8a7ad9Cf1Ccd22A': {'symbol': 'ADS', 'decimals': 11},
    '0x2e2364966267B5D7D2cE6CD9A9B5bD19d9C7C6A9': {'symbol': 'VOICE', 'decimals': 18},
    '0xcDa4e840411C00a614aD9205CAEC807c7458a0E3': {'symbol': 'UFI', 'decimals': 18},
    '0xb076BDa1ABc154DdB4CCd9Be45542A823aEE290E': {'symbol': 'FLEX', 'decimals': 18},
    '0x3496B523e5C00a4b4150D6721320CdDb234c3079': {'symbol': 'NUM', 'decimals': 18},
    '0xE7eaec9Bca79d537539C00C58Ae93117fB7280b9': {'symbol': 'DOGEP', 'decimals': 18},
    '0x1AA51bc7eb181ce48CE626Bf62f8956Fa9555136': {'symbol': 'PAW', 'decimals': 18},
    '0x1E2F15302B90EddE696593607b6bD444B64e8F02': {'symbol': 'SHIRYO-INU', 'decimals': 9},
    '0x3B9BE07d622aCcAEd78f479BC0EDabFd6397E320': {'symbol': 'LSS', 'decimals': 18},
    '0xba9d4199faB4f26eFE3551D490E3821486f135Ba': {'symbol': 'CHSB', 'decimals': 8},
    '0x2a3bFF78B79A009976EeA096a51A948a3dC00e34': {'symbol': 'WILD', 'decimals': 18},
    '0x0954906da0Bf32d5479e25f46056d22f08464cab': {'symbol': 'INDEX', 'decimals': 18},
    '0x10010078a54396F62c96dF8532dc2B4847d47ED3': {'symbol': 'HND', 'decimals': 18},
    '0x06F3C323f0238c72BF35011071f2b5B7F43A054c': {'symbol': 'MASQ', 'decimals': 18},
    '0xA8b919680258d369114910511cc87595aec0be6D': {'symbol': 'LYXE', 'decimals': 18},
    '0xc55126051B22eBb829D00368f4B12Bde432de5Da': {'symbol': 'BTRFLY', 'decimals': 18},
    '0x853d955aCEf822Db058eb8505911ED77F175b99e': {'symbol': 'FRAX', 'decimals': 18},
    '0xA876f27F13A9eB6E621202cEFDD5AFc4a90E6457': {'symbol': 'IC', 'decimals': 9},
    '0xe19f85c920B572cA48942315B06D6cAc86585c87': {'symbol': 'PLEB', 'decimals': 18},
    '0x6Bba316c48b49BD1eAc44573c5c871ff02958469': {'symbol': 'GAS', 'decimals': 18},
    '0x55296f69f40Ea6d20E478533C15A6B08B654E758': {'symbol': 'XYO', 'decimals': 18},
    '0x6Bc40d4099f9057b23aF309C08d935b890d7Adc0': {'symbol': 'SNAIL', 'decimals': 18},
    '0xe803178b48A0e560C2b19F3b3d4e504f79D229ce': {'symbol': 'BOBC', 'decimals': 18},
    '0x9813037ee2218799597d83D4a5B6F3b6778218d9': {'symbol': 'BONE', 'decimals': 18},
    '0xE0151763455A8a021e64880C238ba1cff3787fF0': {'symbol': 'APED', 'decimals': 18},
    '0x3c9d6c1C73b31c837832c72E04D3152f051fc1A9': {'symbol': 'BOR', 'decimals': 18},
    '0xbdab72602e9AD40FC6a6852CAf43258113B8F7a5': {'symbol': 'SOV', 'decimals': 18},
    '0xAd1A5B8538a866eCD56dDd328b50ed57cEd5D936': {'symbol': 'GENSLR', 'decimals': 18},
    '0x1fE24F25b1Cf609B9c4e7E12D802e3640dFA5e43': {'symbol': 'CGG', 'decimals': 18},
    '0x59d1e836F7b7210A978b25a855085cc46fd090B5': {'symbol': 'JUSTICE', 'decimals': 18},
    '0xa444EC96Ee01bB219A44B285De47bf33C3447ad5': {'symbol': 'LEOX', 'decimals': 18},
    '0x509A38b7a1cC0dcd83Aa9d06214663D9eC7c7F4a': {'symbol': 'BST', 'decimals': 18},
    '0x7bFDE33d790411A88D46E9e1be32Fc86228891a4': {'symbol': 'SHIBU', 'decimals': 9},
    '0xBBc2AE13b23d715c30720F079fcd9B4a74093505': {'symbol': 'ERN', 'decimals': 18},
    '0xD5930C307d7395Ff807F2921F12C5EB82131a789': {'symbol': 'BOLT', 'decimals': 18},
    '0x3845badAde8e6dFF049820680d1F14bD3903a5d0': {'symbol': 'SAND', 'decimals': 18},
    '0x7dE91B204C1C737bcEe6F000AAA6569Cf7061cb7': {'symbol': 'XRT', 'decimals': 9},
    '0x43A96962254855F16b925556f9e97BE436A43448': {'symbol': 'HORD', 'decimals': 18},
    '0x8260328d0C405D9Ca061D80199102DdC9089e43C': {'symbol': 'DOJO', 'decimals': 9},
    '0x0b8057C3Cc676C329c25C1d0Cd27776efA73762d': {'symbol': 'WEAPON', 'decimals': 9},
    '0x36ff4DaE0E88113D68B1209e245B0e3Af92E9D58': {'symbol': 'XLON', 'decimals': 6},
    '0xdBdb4d16EdA451D0503b854CF79D55697F90c8DF': {'symbol': 'ALCX', 'decimals': 18},
    '0x6468e79A80C0eaB0F9A2B574c8d5bC374Af59414': {'symbol': 'EXRD', 'decimals': 18},
    '0x3bb86d867A9F3adDF994cdaDb210Fa82F0D4157A': {'symbol': 'GHOUL', 'decimals': 18},
    '0x41A3Dba3D677E573636BA691a70ff2D606c29666': {'symbol': 'BLANK', 'decimals': 18},
    '0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0': {'symbol': 'FXS', 'decimals': 18},
    '0xBC19712FEB3a26080eBf6f2F7849b417FdD792CA': {'symbol': 'BORING', 'decimals': 18},
    '0x6BF765C43030387a983f429c1438e9D2025B7E12': {'symbol': 'PEPES', 'decimals': 18},
    '0xD7EFB00d12C2c13131FD319336Fdf952525dA2af': {'symbol': 'XPR', 'decimals': 4},
    '0x9B8D5f3402F74C7a61d9f09c32D3cA07b45c1466': {'symbol': 'GMR', 'decimals': 18},
    '0x53250B5DFA8C911547aFeAf18DB025024C8E919A': {'symbol': 'KERMIT', 'decimals': 9},
    '0x5f3b3c37F47d8532836c800f75e7f262623Aa3BB': {'symbol': 'JAI', 'decimals': 18},
    '0x005D1123878Fc55fbd56b54C73963b234a64af3c': {'symbol': 'KIBA', 'decimals': 18},
    '0x636d585F40A7a445dA7403FCf92E03F89dc3eBd0': {'symbol': 'PEPEGPT', 'decimals': 18},
    '0xAf5191B0De278C7286d6C7CC6ab6BB8A73bA2Cd6': {'symbol': 'STG', 'decimals': 18},
    '0xE0aD1806Fd3E7edF6FF52Fdb822432e847411033': {'symbol': 'ONX', 'decimals': 18},
    '0xC1D9B5A0776d7C8B98b8A838e5a0DD1Bc5Fdd53C': {'symbol': 'ZONE', 'decimals': 18},
    '0xf21661D0D1d76d3ECb8e1B9F1c923DBfffAe4097': {'symbol': 'RIO', 'decimals': 18},
    '0xAC8E13ecC30Da7Ff04b842f21A62a1fb0f10eBd5': {'symbol': 'BABYDOGE', 'decimals': 9},
    '0xB0Ed33f79D89541dfdCb04A8f04Bc2c6be025eCC': {'symbol': 'ZERO', 'decimals': 18},
    '0x33349B282065b0284d756F0577FB39c158F935e6': {'symbol': 'MPL', 'decimals': 18},
    '0xD291E7a03283640FDc51b121aC401383A46cC623': {'symbol': 'RGT', 'decimals': 18},
    '0x6B89B97169a797d94F057F4a0B01E2cA303155e4': {'symbol': 'CHAD', 'decimals': 18},
    '0xdB25f211AB05b1c97D595516F45794528a807ad8': {'symbol': 'EURS', 'decimals': 2},
    '0xAaAAAA20D9E0e2461697782ef11675f668207961': {'symbol': 'AURORA', 'decimals': 18},
    '0x577Fee283e776eec29C9E4d258431982780a38A8': {'symbol': 'PEPA', 'decimals': 9},
    '0xaADA04204e9e1099DaF67cf3D5D137E84E41CF41': {'symbol': 'PEEPO', 'decimals': 18},
    '0x1f05DeE43dF9fd0Cd7F4b011B20b04870a30c29A': {'symbol': 'SWT', 'decimals': 18},
    '0xBBbbCA6A901c926F240b89EacB641d8Aec7AEafD': {'symbol': 'LRC', 'decimals': 18},
    '0x46d0DAc0926fa16707042CAdC23F1EB4141fe86B': {'symbol': 'SNM', 'decimals': 18},
    '0xe5D2e173B120341face9e9970889C9FE64081FfD': {'symbol': 'BLU', 'decimals': 18},
    '0xBB0E17EF65F82Ab018d8EDd776e8DD940327B28b': {'symbol': 'AXS', 'decimals': 18},
    '0x15b7c0c907e4C6b9AdaAaabC300C08991D6CEA05': {'symbol': 'GEL', 'decimals': 18},
    '0x01BA67AAC7f75f647D94220Cc98FB30FCc5105Bf': {'symbol': 'LYRA', 'decimals': 18},
    '0x428dc22668E6F3468273634067e5545ED5417A3E': {'symbol': 'MQL', 'decimals': 18},
    '0x3106a0a076BeDAE847652F42ef07FD58589E001f': {'symbol': 'ADS', 'decimals': 18},
    '0xC91a71A1fFA3d8B22ba615BA1B9c01b2BBBf55ad': {'symbol': 'ZZ', 'decimals': 18},
    '0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF': {'symbol': 'AURA', 'decimals': 18},
    '0x0d02755a5700414B26FF040e1dE35D337DF56218': {'symbol': 'BEND', 'decimals': 18},
    '0xc97d6C52F3ADD91Fa1c5287a453d7444aECBca83': {'symbol': 'DZOO', 'decimals': 18},
    '0x7D29A64504629172a429e64183D6673b9dAcbFCe': {'symbol': 'VXV', 'decimals': 18},
    '0x12AEf5C60C2C86c8ECD3079f22F285f326371340': {'symbol': 'HISAND33', 'decimals': 18},
    '0x6DEA81C8171D0bA574754EF6F8b412F2Ed88c54D': {'symbol': 'LQTY', 'decimals': 18},
    '0xA49d7499271aE71cd8aB9Ac515e6694C755d400c': {'symbol': 'MUTE', 'decimals': 18},
    '0x3FA729B4548beCBAd4EaB6EF18413470e6D5324C': {'symbol': 'MOVE', 'decimals': 18},
    '0x3F382DbD960E3a9bbCeaE22651E88158d2791550': {'symbol': 'GHST', 'decimals': 18},
    '0xcaDC0acd4B445166f12d2C07EAc6E2544FbE2Eef': {'symbol': 'CADC', 'decimals': 18},
    '0xF1182229B71E79E504b1d2bF076C15a277311e05': {'symbol': 'LBR', 'decimals': 18},
    '0x4Bb3205bf648B7F59EF90Dee0F1B62F6116Bc7ca': {'symbol': 'BYN', 'decimals': 18},
    '0xb3Ad645dB386D7F6D753B2b9C3F4B853DA6890B8': {'symbol': 'CTR', 'decimals': 18},
    '0xD13c7342e1ef687C5ad21b27c2b65D772cAb5C8c': {'symbol': 'UOS', 'decimals': 4},
    '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599': {'symbol': 'WBTC', 'decimals': 8},
    '0x3301Ee63Fb29F863f2333Bd4466acb46CD8323E6': {'symbol': 'AKITA', 'decimals': 18},
    '0xF57e7e7C23978C3cAEC3C3548E3D615c346e79fF': {'symbol': 'IMX', 'decimals': 18},
    '0x20CDECBF5d56870b4068a255580a58D068446c92': {'symbol': 'MONKEYS', 'decimals': 9},
    '0x5Aa158404fEd6b4730C13F49d3a7F820e14A636F': {'symbol': 'ULX', 'decimals': 18},
    '0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd': {'symbol': 'GUSD', 'decimals': 2},
    '0xf7de6DEf3D319811418d69Bf56c532A815FC47e8': {'symbol': 'TWOPAW', 'decimals': 18},
    '0x2F573070E6090b3264Fe707e2C9F201716F123c7': {'symbol': 'MUMU', 'decimals': 18},
    '0x41545f8b9472D758bB669ed8EaEEEcD7a9C4Ec29': {'symbol': 'FORT', 'decimals': 18},
    '0x0000000000085d4780B73119b644AE5ecd22b376': {'symbol': 'TUSD', 'decimals': 18},
    '0xD9016A907Dc0ECfA3ca425ab20B6b785B42F2373': {'symbol': 'GMEE', 'decimals': 18},
    '0x8E870D67F660D95d5be530380D0eC0bd388289E1': {'symbol': 'USDP', 'decimals': 18},
    '0x4691937a7508860F876c9c0a2a617E7d9E945D4B': {'symbol': 'WOO', 'decimals': 18},
    '0xB62132e35a6c13ee1EE0f84dC5d40bad8d815206': {'symbol': 'NEXO', 'decimals': 18},
    '0xB8C55c80a1cb7394088a36C6b634dC2Bf3C6fb67': {'symbol': 'PEPEDOGE', 'decimals': 18},
    '0xc00e94Cb662C3520282E6f5717214004A7f26888': {'symbol': 'COMP', 'decimals': 18},
    '0xf8ebf4849F1Fa4FaF0DFF2106A173D3A6CB2eB3A': {'symbol': 'TROLL', 'decimals': 18},
    '0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2': {'symbol': 'MKR', 'decimals': 18},
    '0x3085154623F51b00Dedfc6Ceeb5197277A66B17B': {'symbol': 'NFTY', 'decimals': 18},
    '0x226bb599a12C826476e3A771454697EA52E9E220': {'symbol': 'PRO', 'decimals': 8},
    '0x090185f2135308BaD17527004364eBcC2D37e5F6': {'symbol': 'SPELL', 'decimals': 18},
    '0x11eeF04c884E24d9B7B4760e7476D06ddF797f36': {'symbol': 'MX', 'decimals': 18},
    '0x0F7B3F5a8FeD821c5eb60049538a548dB2D479ce': {'symbol': 'ATOR', 'decimals': 18},
    '0xf3b9569F82B18aEf890De263B84189bd33EBe452': {'symbol': 'CAW', 'decimals': 18},
    '0x9Be776559fED779cAbd67042a7B8987AAe592541': {'symbol': 'BULL', 'decimals': 18},
    '0xEec2bE5c91ae7f8a338e1e5f3b5DE49d07AfdC81': {'symbol': 'DPX', 'decimals': 18},
    '0x979aCA85bA37c675e78322ed5d97fa980B9Bdf00': {'symbol': 'FSN', 'decimals': 18},
    '0x0c572544a4Ee47904d54aaA6A970AF96B6f00E1b': {'symbol': 'WAS', 'decimals': 18},
    '0xE55d97A97ae6A17706ee281486E98A84095d8AAf': {'symbol': 'AIPAD', 'decimals': 18},
    '0x249e38Ea4102D0cf8264d3701f1a0E39C4f2DC3B': {'symbol': 'UFO', 'decimals': 18},
    '0xD13cfD3133239a3c73a9E535A5c4DadEE36b395c': {'symbol': 'VAI', 'decimals': 18},
    '0x3597bfD533a99c9aa083587B074434E61Eb0A258': {'symbol': 'DENT', 'decimals': 8},
    '0x68b429161Ec09a6c1D65ba70727AB1faa5Bc4026': {'symbol': 'ODOGE', 'decimals': 18},
    '0x3506424F91fD33084466F402d5D97f05F8e3b4AF': {'symbol': 'CHZ', 'decimals': 18},
    '0xA2120b9e674d3fC3875f415A7DF52e382F141225': {'symbol': 'ATA', 'decimals': 18},
    '0x8642A849D0dcb7a15a974794668ADcfbe4794B56': {'symbol': 'PROS', 'decimals': 18},
    '0x1E0b2992079B620AA13A7c2E7c88D2e1E18E46E9': {'symbol': 'KOMPETE', 'decimals': 10},
    '0xf36C5f04127F7470834ed6f98bDDC1BE62aba48D': {'symbol': 'CAI', 'decimals': 18},
    '0xD8E163967FED76806Df0097B704bA721B9b37656': {'symbol': 'COPE', 'decimals': 18},
    '0xCC4304A31d09258b0029eA7FE63d032f52e44EFe': {'symbol': 'SWAP', 'decimals': 18},
    '0xAC51066d7bEC65Dc4589368da368b212745d63E8': {'symbol': 'ALICE', 'decimals': 6},
    '0x69af81e73A73B40adF4f3d4223Cd9b1ECE623074': {'symbol': 'MASK', 'decimals': 18},
    '0x8f8221aFbB33998d8584A2B05749bA73c37a938a': {'symbol': 'REQ', 'decimals': 18},
    '0x89B69F2d1adffA9A253d40840B6Baa7fC903D697': {'symbol': 'DIONE', 'decimals': 9},
    '0x777172D858dC1599914a1C4c6c9fC48c99a60990': {'symbol': 'SOLID', 'decimals': 18},
    '0x8FAc8031e079F409135766C7d5De29cf22EF897C': {'symbol': 'HEART', 'decimals': 18},
    '0x2920f7d6134f4669343e70122cA9b8f19Ef8fa5D': {'symbol': 'MONO', 'decimals': 18},
    '0xF411903cbC70a74d22900a5DE66A2dda66507255': {'symbol': 'VRA', 'decimals': 18},
    '0x104F3152D8ebFC3f679392977356962Ff36566aC': {'symbol': 'PORTX', 'decimals': 18},
    '0x88aa4a6C5050b9A1b2Aa7e34D0582025cA6AB745': {'symbol': 'DXP', 'decimals': 18},
    '0x18aAA7115705e8be94bfFEBDE57Af9BFc265B998': {'symbol': 'AUDIO', 'decimals': 18},
    '0xd068C7C941FbbD2300cB2f1841858c2643722DC7': {'symbol': 'DEGAI', 'decimals': 18},
    '0x6cAcDB97e3fC8136805a9E7c342d866ab77D0957': {'symbol': 'SWPR', 'decimals': 18},
    '0xd4c435F5B09F855C3317c8524Cb1F586E42795fa': {'symbol': 'CND', 'decimals': 18},
    '0xd3E4Ba569045546D09CF021ECC5dFe42b1d7f6E4': {'symbol': 'MNW', 'decimals': 18},
    '0xCE3f08e664693ca792caCE4af1364D5e220827B2': {'symbol': 'SAITAMA', 'decimals': 9},
    '0x490e3f4af13e1616EC97A8C6600c1061a8D0253e': {'symbol': 'TRR', 'decimals': 18},
    '0x849A226f327b89E3133D9930d927F9EB9346F8C9': {'symbol': 'CGU', 'decimals': 8},
    '0xa670d7237398238DE01267472C6f13e5B8010FD1': {'symbol': 'SOMM', 'decimals': 6},
    '0xf68D4d917592f3a62417aCE42592F15296cc33A0': {'symbol': 'CHB', 'decimals': 8},
    '0x5E7F20e72C21f6D0Bf0A2814FD4164176401cf8e': {'symbol': 'DEO', 'decimals': 18},
    '0x772722B55cdC2A086aBd064267a17855Eb15e8b3': {'symbol': 'MINTME', 'decimals': 18},
    '0xe20B9e246db5a0d21BF9209E4858Bc9A3ff7A034': {'symbol': 'BAN', 'decimals': 18},
    '0x37613D64258c0FE09d5E53EeCb091DA5b8fA8707': {'symbol': 'PIP', 'decimals': 18},
    '0xd3Ac016b1B8C80EeAdDe4D186A9138C9324e4189': {'symbol': 'OK', 'decimals': 18},
    '0xDd13DEdeCEbDA566322195bc4451D672A148752C': {'symbol': 'PRIMAL', 'decimals': 18},
    '0xc7026a20a640Bc71B9074F7aed52A00cD9147091': {'symbol': 'TGR', 'decimals': 18},
    '0xb990d93C308A31c737Aa91839E8bA8eAF4017D7A': {'symbol': 'PIRATE', 'decimals': 8},
    '0x5F1F11a3dD7a0C39Da1BaA3C7b8585b52a77f435': {'symbol': 'ASAN', 'decimals': 18},
    '0x5BDC32663Ec75E85ff4abc2CAe7ae8B606A2CFCa': {'symbol': 'CP', 'decimals': 18},
    '0x706f280Cdb92260fe2D50EDA545F834ff1fbFd35': {'symbol': 'RABBIT', 'decimals': 9},
    '0xa31DFe74cDA04de990CafC6a0aB2BB6d6C9077CD': {'symbol': 'ABI', 'decimals': 9},
    '0xb2cABf797bc907B049e4cCB5b84d13be3a8CFC21': {'symbol': 'ACRE', 'decimals': 18},
    '0x4E15361FD6b4BB609Fa63C81A2be19d873717870': {'symbol': 'WFTM', 'decimals': 18},
    '0x388D819724dD6d71760A38F00dc01D310d879771': {'symbol': 'JM', 'decimals': 8},
    '0x4B19C70Da4c6fA4bAa0660825e889d2F7eaBc279': {'symbol': 'GMM', 'decimals': 18},
    '0xA0b73E1Ff0B80914AB6fe0444E65848C4C34450b': {'symbol': 'CRO', 'decimals': 8},
    '0x7Cfea0DD176651E7B5a1CeD9c4fAf8Ee295315FD': {'symbol': 'PRNT', 'decimals': 18},
    '0xF33893DE6eB6aE9A67442E066aE9aBd228f5290c': {'symbol': 'GRV', 'decimals': 8},
    '0x3C917054e03485808137EB306eAFA8DA0Ab695cd': {'symbol': 'ORB', 'decimals': 18},
    '0xB8c77482e45F1F44dE1745F52C74426C631bDD52': {'symbol': 'BNB', 'decimals': 18},
    '0x96543ef8d2C75C26387c1a319ae69c0BEE6f3fe7': {'symbol': 'KUJI', 'decimals': 6},
    '0x1B3C515F58857E141A966b33182f2F3feECC10E9': {'symbol': 'USK', 'decimals': 6},
    '0x6F9C26Fa731C7Ea4139Fa669962Cf8F1CE6C8b0B': {'symbol': 'OATH', 'decimals': 18},
    '0xCc503242b574BC01145Da7E2A743b43fB395eC91': {'symbol': 'ROVI', 'decimals': 18},
    '0x1F3F677Ecc58F6A1F9e2CF410dF4776a8546b5DE': {'symbol': 'VNDC', 'decimals': 0},
    '0xA64c3a85ddc4cD351Eeb7aeceBc6a44A64A76392': {'symbol': 'RICE', 'decimals': 18},
    '0x837d904A3799c0769079BE9ecbadDF1aBD4cCD6e': {'symbol': 'TAROT', 'decimals': 18},
    '0xCdB37A4fBC2Da5b78aA4E41a432792f9533e85Cc': {'symbol': 'CDT', 'decimals': 18},
    '0x0a5E677a6A24b2F1A2Bf4F3bFfC443231d2fDEc8': {'symbol': 'USX', 'decimals': 18},
    '0x4184Aa04215e5d716dD4C213FeD519aCADc68f92': {'symbol': 'ONUS', 'decimals': 18},
    '0xD3c325848D7c6E29b574Cb0789998b2ff901f17E': {'symbol': '1ART', 'decimals': 18},
    '0x0FF534801E98A4976246D1f418E441783fc9aa15': {'symbol': 'FUTURE-AI', 'decimals': 18},
    '0xf05897CfE3CE9BBBfE0751CBE6B1B2c686848DCb': {'symbol': 'CATE', 'decimals': 9},
    '0x8f006D1e1D9dC6C98996F50a4c810F17a47fBF19': {'symbol': 'NSFW', 'decimals': 18},
    '0xD5a14081a34d256711B02BbEf17E567da48E80b5': {'symbol': 'WUSDR', 'decimals': 9},
    '0x06fCe0DAB5b26e503E1C4Db48aa165DcaAAa6E11': {'symbol': 'EVCOIN', 'decimals': 9},
    '0xc5A927AD2d1a92Dc5f9662Ce03F7140a8B738B19': {'symbol': 'AIWALLET', 'decimals': 18},
    '0xAE788F80F2756A86aa2F410C651F2aF83639B95b': {'symbol': 'MV', 'decimals': 18},
    '0x888888888888f195E27A2E0F98D712952aB9348e': {'symbol': 'IPISTR', 'decimals': 18},
    '0xC7B89491Bb148551547837ea6ccB4Bb5144d8E47': {'symbol': 'PING', 'decimals': 9},
    '0x9b25889C493aE6Df34cEef1ecb10D77c1Ba73318': {'symbol': 'MEAN', 'decimals': 6},
    '0xB0c7a3Ba49C7a6EaBa6cD4a96C55a1391070Ac9A': {'symbol': 'MAGIC', 'decimals': 18},
    '0xF3A3023e6Dede84AD88a656A3269F2A36e83c9a9': {'symbol': 'SHARBI', 'decimals': 9},
    '0x9f826324bb9BdcF7E7Eb274BEDc417BD45D74F39': {'symbol': 'MILKAI', 'decimals': 8},
    '0x5b52bfB8062Ce664D74bbCd4Cd6DC7Df53Fd7233': {'symbol': 'ZENIQ', 'decimals': 18},
    '0xb012Be90957D70D9a070918027655F998C123a88': {'symbol': 'HMX', 'decimals': 18},
    '0x4c584CD339bdDE73B7F5210486dd8bbeEE3fDe6d': {'symbol': 'SHIBELON', 'decimals': 9},
    '0xA64D5d1eB67748226d84812B45711453f1118c32': {'symbol': 'CINU', 'decimals': 9},
    '0xf3E07812eBC8604fdDB0AA35ff79a03F48f48948': {'symbol': 'JART', 'decimals': 18},
    '0xBcCD27062ae1A2BeA5731c904b96EDfb163Aba21': {'symbol': 'DOGS', 'decimals': 9},
    '0xf124eD9Ec309907808b1FBc6BEdB2a76927B3665': {'symbol': 'EMPIRE', 'decimals': 18},
    '0x9ed7E4B1BFF939ad473dA5E7a218C771D1569456': {'symbol': 'REUNI', 'decimals': 6},
    '0x0d229c3FF2d76E4b88Ca4F9d3D3353F4545EC53F': {'symbol': 'SHINU', 'decimals': 18},
    '0x188fb5f5Ae5BBE4154d5778F2bbb2Fb985c94d25': {'symbol': 'OBX', 'decimals': 18},
    '0x02c3296C6eb50249f290AE596F2bE9454bFfadaB': {'symbol': 'RJV', 'decimals': 6},
    '0xd23Ed8cA350CE2631F7EcDC5E6bf80D0A1DeBB7B': {'symbol': 'PLQ', 'decimals': 18},
    '0x5fab9761d60419C9eeEbe3915A8FA1ed7e8d2E1B': {'symbol': 'DIMO', 'decimals': 18},
    '0x0fAD86Cc1b412b330494E350Bf1dC1994AC55c9e': {'symbol': 'ATC', 'decimals': 6},
    '0x2dfF88A56767223A5529eA5960Da7A3F5f766406': {'symbol': 'ID', 'decimals': 18},
    '0x6731827Cb6879a2091ce3ab3423f7bf20539b579': {'symbol': 'MPWR', 'decimals': 18},
    '0xB50721BCf8d664c30412Cfbc6cf7a15145234ad1': {'symbol': 'ARB', 'decimals': 18},
    '0x77F2BE773CA0887ba2b3Ef8344C8CF13C98d8ca7': {'symbol': 'FYT', 'decimals': 18},
    '0xa89Bf95c5f15a847c8Eb8D348cd7feD719AD7D80': {'symbol': 'AI', 'decimals': 18},
    '0x1C98B54d673C026C8286bADCa3E840aaf72931a3': {'symbol': 'STSW', 'decimals': 6},
    '0x1151CB3d861920e07a38e03eEAd12C32178567F6': {'symbol': 'BONK', 'decimals': 5},
    '0x510975edA48A97E0cA228dD04d1217292487bea6': {'symbol': 'GXE', 'decimals': 18},
    '0xC9C4fd7579133701FA2769b6955e7E56bb386DB1': {'symbol': 'BRG', 'decimals': 18},
    '0xd1420AF453Fd7bf940573431D416cAcE7fF8280c': {'symbol': 'AGOV', 'decimals': 18},
    '0xB4A3B0Faf0Ab53df58001804DdA5Bfc6a3D59008': {'symbol': 'SPA', 'decimals': 18},
    '0x969786c4A8884013d1c9fF18dCCA2aeDbbBfAa8F': {'symbol': 'LFG', 'decimals': 18},
    '0xE9eb76879d332E5c574AA4E8202Ef84956806949': {'symbol': 'PROP', 'decimals': 18},
    '0x152649eA73beAb28c5b49B26eb48f7EAD6d4c898': {'symbol': 'CAKE', 'decimals': 18},
    '0xD38DE430D355a7a8337204c2A4C80392E61475a6': {'symbol': 'DUCKYAI', 'decimals': 18},
    '0x8b9F7e9586633cA4ABffb1F746da8dAed7849cB2': {'symbol': 'CREMAT', 'decimals': 18},
    '0x995C0e3b9AF4da89FC76c5c0784e97C7A36EEc06': {'symbol': 'PIGE', 'decimals': 18},
    '0xdAd33e12e61dC2f2692F2c12e6303B5Ade7277Ba': {'symbol': 'BREWLABS', 'decimals': 9},
    '0xf89060c99853Bb52eaF5F2247D007d73de660252': {'symbol': 'WBESC', 'decimals': 18},
    '0xCc17e34794B6c160a0F61B58CF30AA6a2a268625': {'symbol': 'NRFX', 'decimals': 18},
    '0x87dA7BBd398D6610dED42617E34069D0083ff8aB': {'symbol': 'MIND', 'decimals': 18},
    '0x9d7107c8E30617CAdc11f9692A19C82ae8bbA938': {'symbol': 'ROO', 'decimals': 18},
    '0x9776191F4ebBBa7f358C1663bF82C0a0906c77Fa': {'symbol': 'PHX', 'decimals': 18},
    '0x9d1089802eE608BA84C5c98211afE5f37F96B36C': {'symbol': 'FUSDC', 'decimals': 6},
    '0x703D8574E19428D662a73c301D106dc236aa9BFC': {'symbol': 'TSA', 'decimals': 18},
    '0x9348E94A447bF8B2ec11f374D3F055FD47d936Df': {'symbol': 'FLAG', 'decimals': 18},
    '0xc3589F56B6869824804A5EA29F2c9886Af1B0FcE': {'symbol': 'HNY', 'decimals': 18},
    '0x377d552914E7A104bC22B4F3B6268dDC69615Be7': {'symbol': 'NEXT', 'decimals': 18},
    '0xE2dc070524A6e305ddB64d8513DC444B6a1ec845': {'symbol': 'NEX', 'decimals': 8},
    '0x0F3ADC247E91c3c50bC08721355A41037E89Bc20': {'symbol': 'ANC', 'decimals': 18},
    '0x499568c250Ab2a42292261d6121525d70691894b': {'symbol': 'KRW', 'decimals': 18},
    '0x1350Fbe8Ce27762ec19134bf8FC405a427fe9Bf1': {'symbol': 'TRIPS', 'decimals': 18},
    '0xFeea0bDd3D07eb6FE305938878C0caDBFa169042': {'symbol': '8PAY', 'decimals': 18},
    '0x6DC02164d75651758aC74435806093E421b64605': {'symbol': 'WCHI', 'decimals': 8},
    '0x71ba91dC68C6a206Db0A6A92B4b1De3f9271432d': {'symbol': 'MBX', 'decimals': 18},
    '0xfb62AE373acA027177D1c18Ee0862817f9080d08': {'symbol': 'DPET', 'decimals': 18},
    '0x92a42Db88Ed0F02c71D439e55962Ca7CAB0168b5': {'symbol': 'TRDG', 'decimals': 9},
    '0x3079F61704E9eFa2BcF1db412f735d8d4cFa26f4': {'symbol': 'HAPPY', 'decimals': 18},
    '0xEE1CeA7665bA7aa97e982EdeaeCb26B59a04d035': {'symbol': 'ORAO', 'decimals': 18},
    '0x946112efaB61C3636CBD52DE2E1392D7A75A6f01': {'symbol': 'HYDRO', 'decimals': 18},
    '0x7B4328c127B85369D9f82ca0503B000D09CF9180': {'symbol': 'DC', 'decimals': 18},
    '0x05Fb86775Fd5c16290f1E838F5caaa7342bD9a63': {'symbol': 'HAI', 'decimals': 8},
    '0xA771b49064Da011DF051052848477f18DbA1d2Ac': {'symbol': 'HNS', 'decimals': 6},
    '0x5f6c5C2fB289dB2228d159C69621215e354218d7': {'symbol': 'DMOD', 'decimals': 18},
    '0xc944273b805DeBd35c63011943ABc5aB9eDdb8E3': {'symbol': 'SHIELD', 'decimals': 18},
    '0x9D918eE39a356bE8eF99734599c7e70160dB4Db6': {'symbol': 'REALT-S-12405-SANTA', 'decimals': 18},
    '0xF4657AB08681214Bcb1893aa8E9C7613459250ec': {'symbol': 'REALT-S-1617-SAVERS', 'decimals': 18},
    '0xE7B6dE709fFC3BD237c2f2c800E1002F97A760F3': {'symbol': 'REALT-S-4852-4854-W', 'decimals': 18},
    '0xe887DC4fcb5240c0c080AEAb8870421d3ebd0B28': {'symbol': 'REALT-S-14066-SANTA', 'decimals': 18},
    '0x45fDb1b92a649fb6A64Ef1511D3Ba5Bf60044838': {'symbol': 'USDS', 'decimals': 18},
    '0x8BCd06492416a749C9369009b3429861b7F27F6E': {'symbol': 'BLKC', 'decimals': 8},
    '0x7778360F035C589fCE2f4eA5786CbD8B36e5396B': {'symbol': 'OOE', 'decimals': 18},
    '0x4E9A46EA6A22f3894aBEE2302Ad42fd3b69E21E2': {'symbol': 'BSCGIRL', 'decimals': 8},
    '0xAE12C5930881c53715B369ceC7606B70d8EB229f': {'symbol': 'C98', 'decimals': 18},
    '0x396eC402B42066864C406d1ac3bc86B575003ed8': {'symbol': 'BUY', 'decimals': 2},
    '0x180dAe91D6d56235453a892d2e56a3E40Ba81DF8': {'symbol': 'DOJO', 'decimals': 18},
    '0x8287C7b963b405b7B8D467DB9d79eEC40625b13A': {'symbol': 'SWINGBY', 'decimals': 18},
    '0x1d7Ca62F6Af49ec66f6680b8606E634E55Ef22C1': {'symbol': 'START', 'decimals': 18},
    '0x6C7B97c7e09E790D161769a52F155125FAc6d5A1': {'symbol': 'ANGEL', 'decimals': 18},
    '0x31F69dE127C8A0fF10819C0955490a4Ae46fcc2a': {'symbol': 'GBYTE', 'decimals': 18},
    '0x8848812BD31AEEe33313C10A840FfC3169078C5b': {'symbol': 'CRFI', 'decimals': 18},
    '0x667Fd83E24Ca1D935d36717D305D54fA0CAC991C': {'symbol': 'AGS', 'decimals': 18},
    '0x9393fdc77090F31c7db989390D43F454B1A6E7F3': {'symbol': 'DEC', 'decimals': 3},
    '0x2aaD9Dbc82611485a52325923e1187734e951B78': {'symbol': 'BYTZ', 'decimals': 8},
    '0x8861cfF2366C1128fd699B68304aD99a0764Ef9a': {'symbol': 'CYC', 'decimals': 18},
    '0xdb726152680eCe3c9291f1016f1d36f3995f6941': {'symbol': 'MEDIA', 'decimals': 6},
    '0x34965f73Cfa05Bf8d8aF37cB4af64Fa950605Ea8': {'symbol': 'COW', 'decimals': 18},
    '0x9Ac07635DDBDE5db18648c360DEFb00F5f22537e': {'symbol': 'MOCA', 'decimals': 18},
    '0x7645DdfEecedA57e41f92679c4aCd83c56A81D14': {'symbol': 'FLUX', 'decimals': 18},
    '0xf99d58e463A2E07e5692127302C20A191861b4D6': {'symbol': 'ANY', 'decimals': 18},
    '0x9Fda7cEeC4c18008096C2fE2B85F05dc300F94d0': {'symbol': 'GAJ', 'decimals': 18},
    '0xe0b0C16038845BEd3fCf70304D3e167Df81ce225': {'symbol': 'CSWAP', 'decimals': 18},
    '0x939B462ee3311f8926c047D2B576C389092b1649': {'symbol': 'DAPP', 'decimals': 4},
    '0xef3A930e1FfFFAcd2fc13434aC81bD278B0ecC8d': {'symbol': 'FIS', 'decimals': 18},
    '0x154e35c2b0024B3e079c5c5e4fC31c979c189cCB': {'symbol': 'RAID', 'decimals': 18},
    '0x9D79d5B61De59D882ce90125b18F74af650acB93': {'symbol': 'NSBT', 'decimals': 6},
    '0xDB0170e2d0c1CC1b2E7a90313d9B9afA4f250289': {'symbol': 'ADAPAD', 'decimals': 18},
    '0x43044f861ec040DB59A7e324c40507adDb673142': {'symbol': 'CAP', 'decimals': 18},
    '0xC2708A3A4ba7F64bddc1A49F92f941Bc77Cad23a': {'symbol': 'EGG', 'decimals': 18},
    '0x8bbf1DcCBEdD5c70d8E793d432fB56b848DD1698': {'symbol': 'APEIN', 'decimals': 18},
    '0xb8e3bB633F7276cc17735D86154E0ad5ec9928C0': {'symbol': 'VLXPAD', 'decimals': 18},
    '0x9798dF2f5d213a872c787bD03b2b91F54D0D04A1': {'symbol': 'TBC', 'decimals': 18},
    '0x3405A1bd46B85c5C029483FbECf2F3E611026e45': {'symbol': 'WOW', 'decimals': 18},
    '0x12BB890508c125661E03b09EC06E404bc9289040': {'symbol': 'RACA', 'decimals': 18},
    '0xcA1262e77Fb25c0a4112CFc9bad3ff54F617f2e6': {'symbol': 'WJXN', 'decimals': 0},
    '0x968F6f898a6Df937fC1859b323aC2F14643e3fED': {'symbol': 'NWC', 'decimals': 18},
    '0x39Ae6D231d831756079ec23589D2D37A739F2E89': {'symbol': 'ZPTC', 'decimals': 4},
    '0x5dBB9F64cd96E2DbBcA58d14863d615B67B42f2e': {'symbol': 'OKLG', 'decimals': 9},
    '0xFbC4f3f645C4003a2E4F4e9b51077d2DaA9a9341': {'symbol': 'ZEDXION', 'decimals': 18},
    '0xdb85f6685950E285b1E611037BEBe5B34e2B7d78': {'symbol': 'ZANO', 'decimals': 18},
    '0xf32122561d51E891B823Dec2B42F644884c1Cd91': {'symbol': 'DEFIDO', 'decimals': 9},
    '0xf6719e1A8fcBB1B9c290019E37e004966A8916C9': {'symbol': 'PGEN', 'decimals': 18},
    '0x2fd8BC03D9e827F77faC20B5130eE98B7F80149d': {'symbol': 'CRB', 'decimals': 8},
    '0x16ba8Efe847EBDFef99d399902ec29397D403C30': {'symbol': 'OH', 'decimals': 18},
    '0xd373576a9e738F37Dc6882328358Ff69c4cAf4c6': {'symbol': 'ZAM', 'decimals': 18},
    '0x0106A1122FE94a9CF151097c1Fe17229Ec78FFaD': {'symbol': 'ECOP', 'decimals': 18},
    '0x773258b03c730F84aF10dFcB1BfAa7487558B8Ac': {'symbol': 'SEFI', 'decimals': 6},
    '0x5f0366c9962193fA774cdce9602195593B49f23C': {'symbol': 'FLASH', 'decimals': 18},
    '0x65E6B60Ea01668634D68D0513Fe814679F925BaD': {'symbol': 'PIXEL', 'decimals': 18},
    '0x081131434f93063751813C619Ecca9C4dC7862a3': {'symbol': 'DAR', 'decimals': 6},
    '0x0C75dD36aF9a59BA1d248a98Fe91b2384cfea9be': {'symbol': 'WHIRL', 'decimals': 18},
    '0xAC57De9C1A09FeC648E93EB98875B212DB0d460B': {'symbol': 'BABYDOGE', 'decimals': 9},
    '0x55aF5865807b196bD0197e0902746F31FBcCFa58': {'symbol': 'BOO', 'decimals': 18},
    '0x9A257C90Fa239fBA07771ef7da2d554D148c2E89': {'symbol': 'EYE', 'decimals': 18},
    '0x9fB83c0635De2E815fd1c21b3a292277540C2e8d': {'symbol': 'FEVR', 'decimals': 18},
    '0x0343131C0257aC21Ea5A8dc83841f071eFd9285C': {'symbol': 'ZENITH', 'decimals': 18},
    '0xD8C1232FcD219286E341271385bd70601503B3D7': {'symbol': 'DOGIRA', 'decimals': 9},
    '0x747C4Ce9622EA750EA8048423B38A746B096C8e8': {'symbol': 'BABYFD', 'decimals': 9},
    '0xb6Dd77FD132dcAa10F1858734e838a0FA7431580': {'symbol': 'FCP', 'decimals': 18},
    '0x81C159f7aBAa9139227Aff62959b86B4141F6EB2': {'symbol': 'MARVIN', 'decimals': 9},
    '0x93e32efaFd24973d45f363A76D73ccB9Edf59986': {'symbol': 'BTL', 'decimals': 6},
    '0x42Baf1f659D765C65ADE5BB7E08eb2C680360d9d': {'symbol': 'COPI', 'decimals': 18},
    '0xe1590A6FA0CFf9C960181cb77d8a873601772f64': {'symbol': 'WSB', 'decimals': 18},
    '0xdf09a216Fac5ADC3e640Db418C0b956076509503': {'symbol': 'PKN', 'decimals': 18},
    '0x45C2F8c9B4c0bDC76200448cc26C48ab6ffef83F': {'symbol': 'DOMI', 'decimals': 18},
    '0xA87135285Ae208e22068AcDBFf64B11Ec73EAa5A': {'symbol': 'LUNR', 'decimals': 4},
    '0x965b85D4674F64422c4898C8F8083187f02B32C0': {'symbol': 'SFIL', 'decimals': 8},
    '0xa47c8bf37f92aBed4A126BDA807A7b7498661acD': {'symbol': 'USTC', 'decimals': 18},
    '0x57cd5A91c18D21DeDd72d3bcf255b60Aef290f53': {'symbol': 'SNAP', 'decimals': 18},
    '0x2F25d402829cA4085B8Ea4D3BC68Bf203F5a9faB': {'symbol': 'EAGON', 'decimals': 18},
    '0xaAD483F97f13C6A20B9D05d07c397CE85c42C393': {'symbol': 'WOOP', 'decimals': 18},
    '0xAF9dB9E362e306688AF48c4ACB9618C06db38Ac3': {'symbol': 'ACY', 'decimals': 18},
    '0xBD100d061E120b2c67A24453CF6368E63f1Be056': {'symbol': 'IDYP', 'decimals': 18},
    '0x77A1f4E744d810239F465043E35d067Ca33De259': {'symbol': 'VST', 'decimals': 18},
    '0xE57425F1598f9b0d6219706b77f4b3DA573a3695': {'symbol': 'BTCBR', 'decimals': 18},
    '0x7118057ff0F4Fd0994fb9d2D94de8231d5cca79E': {'symbol': 'SOURCE', 'decimals': 18},
    '0x9Ab70e92319f0b9127df78868Fd3655fB9f1E322': {'symbol': 'WWY', 'decimals': 18},
    '0x86D49fbD3B6f989d641E700a15599d3b165002AB': {'symbol': 'HUH', 'decimals': 9},
    '0x25b24B3c47918b7962B3e49C4F468367F73CC0E0': {'symbol': 'AXL', 'decimals': 18},
    '0xbBa6C7c7d673c48d90069ad2e9d2fE587fcB6bc3': {'symbol': 'LEPA', 'decimals': 18},
    '0xa693B19d2931d498c5B318dF961919BB4aee87a5': {'symbol': 'UST', 'decimals': 6},
    '0x845e2e8B42DCED7DEDCDBa9BdE32C9e338224f97': {'symbol': 'SATOZ', 'decimals': 8},
    '0x2f75113b13D136F861d212Fa9b572F2C79Ac81C4': {'symbol': 'EKTA', 'decimals': 18},
    '0xC669928185DbCE49d2230CC9B0979BE6DC797957': {'symbol': 'BTT', 'decimals': 18},
    '0xCC8Fa225D80b9c7D42F96e9570156c65D6cAAa25': {'symbol': 'SLP', 'decimals': 0},
    '0xc4BB7277A74678f053259CB1F96140347efbfd46': {'symbol': 'COC', 'decimals': 18},
    '0x9b17bAADf0f21F03e35249e0e59723F34994F806': {'symbol': 'GEM', 'decimals': 18},
    '0xEe9801669C6138E84bD50dEB500827b776777d28': {'symbol': 'O3', 'decimals': 18},
    '0x7825e833D495F3d1c28872415a4aee339D26AC88': {'symbol': 'WTLOS', 'decimals': 18},
    '0x2217e5921B7edfB4BB193a6228459974010D2198': {'symbol': 'QMALL', 'decimals': 18},
    '0xD3c51de3E6DD9b53D7f37699AFB3EE3Bf9B9b3F4': {'symbol': 'MCONTENT', 'decimals': 6},
    '0x1F36FB2D91d9951Cf58aE4c1956C0b77e224F1E9': {'symbol': 'VCG', 'decimals': 18},
    '0xe580074A10360404AF3ABfe2d524D5806D993ea3': {'symbol': 'PAY', 'decimals': 18},
    '0x723bD1f87A327e94ceaBf68d8f022E0f54B9cC1a': {'symbol': 'PHL', 'decimals': 18},
    '0xc17fbe1d709dDf6C0B6665Dd0591046815AC7554': {'symbol': 'POL', 'decimals': 18},
    '0x22c5543d1a35178cB03b33f929A959145E538532': {'symbol': 'WAL', 'decimals': 18},
    '0x20d60c6eb195868d4643f2c9B0809e4De6Cc003d': {'symbol': 'PLY', 'decimals': 6},
    '0x809E130E10e787139c54e1d12d3D1971b7A675Bf': {'symbol': 'MTD', 'decimals': 18},
    '0x1aB43204a195a0fD37EdeC621482AFd3792ef90b': {'symbol': 'PLY', 'decimals': 18},
    '0x525eF76138Bf76118d786DbedeaE5F87aaBf4a81': {'symbol': 'NFTI', 'decimals': 18},
    '0xF655C8567E0f213e6C634CD2A68d992152161dC6': {'symbol': 'IBEX', 'decimals': 18},
    '0x19193F450086d0442157b852081976D41657aD56': {'symbol': 'NNT', 'decimals': 18},
    '0xC17c30e98541188614dF99239cABD40280810cA3': {'symbol': 'RISE', 'decimals': 18},
    '0x32462bA310E447eF34FF0D15BCE8613aa8C4A244': {'symbol': 'DHN', 'decimals': 18},
    '0x198d14F2Ad9CE69E76ea330B374DE4957C3F850a': {'symbol': 'NFT', 'decimals': 6},
    '0x16B3E050e9e2f0Ac4f1BEA1b3E4fdc43d7f062Dd': {'symbol': 'SMBR', 'decimals': 9},
    '0xb14784b2a56945AED7b8CD41661D68F8b6CCeC8b': {'symbol': 'MELON', 'decimals': 18},
    '0x446F2A8A39Cc730ef378Be759A3c57f1a3fE824c': {'symbol': 'NBT', 'decimals': 18},
    '0xC84D8d03aA41EF941721A4D77b24bB44D7C7Ac55': {'symbol': 'ECC', 'decimals': 9},
    '0x818f49467021BDaadAE69E071E79AD2fD7226a1E': {'symbol': 'FAME', 'decimals': 18},
    '0x420a24c9c65bd44c48bfB1cC8D6cD1ea8B1Ac840': {'symbol': 'JMPT', 'decimals': 18},
    '0x4550003152F12014558e5CE025707E4DD841100F': {'symbol': 'KZEN', 'decimals': 18},
    '0x0C10bF8FcB7Bf5412187A595ab97a3609160b5c6': {'symbol': 'USDD', 'decimals': 18},
    '0xb5be7557fe8f69a2B5707D25fA0aeE80DfDA512E': {'symbol': 'PS1', 'decimals': 18},
    '0xfcEB206E1a80527908521121358B5e26cAABaa75': {'symbol': 'MAIN', 'decimals': 18},
    '0x614D7f40701132E25fe6fc17801Fbd34212d2Eda': {'symbol': 'BLAST', 'decimals': 9},
    '0x8Fc8f8269ebca376D046Ce292dC7eaC40c8D358A': {'symbol': 'DFI', 'decimals': 8},
    '0xf6e06B54855eFf198a2d9A8686113665499a6134': {'symbol': 'CELT', 'decimals': 18},
    '0x9767203e89dcD34851240B3919d4900d3E5069f1': {'symbol': 'A4', 'decimals': 6},
    '0xd35c06a2781f648c75290976ecF71E71582188B7': {'symbol': 'QUA', 'decimals': 18},
    '0x7a5d3A9Dcd33cb8D527f7b5F96EB4Fef43d55636': {'symbol': 'RADIO', 'decimals': 18},
    '0x26dcFbFa8Bc267b250432c01C982Eaf81cC5480C': {'symbol': 'ANKRMATIC', 'decimals': 18},
    '0xAFCdd4f666c84Fed1d8BD825aA762e3714F652c9': {'symbol': 'VINU', 'decimals': 18},
    '0x9e6b19874e97fe8E8CAD77f2c0AB5E7a693E5dbf': {'symbol': 'ISHND', 'decimals': 18},
    '0x2b95A1Dcc3D405535f9ed33c219ab38E8d7e0884': {'symbol': 'ACRV', 'decimals': 18},
    '0x5f018e73C185aB23647c82bD039e762813877f0e': {'symbol': 'SHACK', 'decimals': 18},
    '0x40fEd5691e547885cABd7A2990De719DCc8497FC': {'symbol': 'SHA', 'decimals': 18},
    '0x18a1Ea69A50A85752b7bC204a2c45A95Ce6E429d': {'symbol': 'SPICE', 'decimals': 18},
    '0xcafE34BAE6F1b23A6B575303EdCc0578d2188131': {'symbol': 'BIP', 'decimals': 18},
    '0x57d579F483854c62FEf850B8a5332B0d8424b7E2': {'symbol': 'OPENX', 'decimals': 18},
    '0x80CD73baDb406eA36B9a7CDeb8df06AeFa7E12d9': {'symbol': 'SLEEPEE', 'decimals': 18},
    '0x471Ea49dd8E60E697f4cac262b5fafCc307506e4': {'symbol': 'RMRK', 'decimals': 10},
    '0x93581991f68DBaE1eA105233b67f7FA0D6BDeE7b': {'symbol': 'EVMOS', 'decimals': 18},
    '0x4C3A8ECeB656Ec63eaE80a4ebD565E4887DB6160': {'symbol': 'SOKU', 'decimals': 18},
    '0x88536C9B2C4701b8dB824e6A16829D5B5Eb84440': {'symbol': 'USV', 'decimals': 9},
    '0xeEeEEb57642040bE42185f49C52F7E9B38f8eeeE': {'symbol': 'ELK', 'decimals': 18},
    '0x2e7B0d4F9B2EaF782eD3D160e3a0a4b1a7930aDA': {'symbol': 'CERES', 'decimals': 18},
    '0x22Ad96aCF613539d7900b5Ec6e0E65dBCC668d2A': {'symbol': 'XERM', 'decimals': 7},
    '0x92DF60c51C710a1b1C20E42D85e221f3A1bFc7f2': {'symbol': 'BANANA', 'decimals': 18},
    '0xC72633F995e98Ac3BB8a89e6a9C4Af335C3D6E44': {'symbol': 'OSEA', 'decimals': 18},
    '0x2b867efD2dE4Ad2B583Ca0CB3dF9C4040Ef4D329': {'symbol': 'LBLOCK', 'decimals': 9},
    '0xe3c408BD53c31C085a1746AF401A4042954ff740': {'symbol': 'GMT', 'decimals': 8},
    '0x35609dC59E15d03c5c865507e1348FA5abB319A8': {'symbol': 'GM', 'decimals': 8},
    '0x34d31446a522252270b89b09016296ec4c98e23d': {'symbol': 'SAUDISHIB', 'decimals': 8},
    '0x4e568ab95F029E8dF1e39b30c9d6D076eaA15945': {'symbol': 'FLY', 'decimals': 18},
    '0x2C4F1DF9c7DE0C59778936C9b145fF56813F3295': {'symbol': 'MNTL', 'decimals': 6},
    '0x57b59f981730c6257dF57cF6F0D98283749A9Eeb': {'symbol': 'BUILD', 'decimals': 18},
    '0xde16Ce60804a881e9F8c4eBB3824646EDecd478D': {'symbol': 'MCRT', 'decimals': 9},
    '0x839e71613f9aA06E5701CF6de63E303616B0DDE3': {'symbol': 'VVS', 'decimals': 18},
    '0x8616E8EA83f048ab9A5eC513c9412Dd2993bcE3F': {'symbol': 'FXUSD', 'decimals': 18},
    '0xbFc43a35b3aE8F7463c5Ac88a0C46107cFCe6f67': {'symbol': 'OCTSMM', 'decimals': 18},
    '0x256D1fCE1b1221e8398f65F9B36033CE50B2D497': {'symbol': 'WALV', 'decimals': 18},
    '0xb28f803A8772E6584A65Ab6dfC535aE6fEf8a0b2': {'symbol': 'LFI', 'decimals': 18},
    '0x6CD13E1856286dE773a435C8Ba93A7C73A04F82d': {'symbol': 'MIDA', 'decimals': 18},
    '0xC285B7E09A4584D027E5BC36571785B515898246': {'symbol': 'CUSD', 'decimals': 18},
    '0x2f32b39023dA7d6A6486A85d12B346EB9C2A0D19': {'symbol': 'FER', 'decimals': 18},
    '0xD6e460F70e1CF60E55B770f66E568B44bf3657d0': {'symbol': 'FUNEX', 'decimals': 18},
    '0x8D6CeBD76f18E1558D4DB88138e2DeFB3909fAD6': {'symbol': 'MIMATIC', 'decimals': 18},
    '0x7252d57Ae4Ec9aF61DB3B82A6eeDAB74EEC5D1dc': {'symbol': 'MONDAY', 'decimals': 18},
    '0x97Bbbc5d96875fB78D2F14b7FF8d7a3a74106F17': {'symbol': 'ASTRAFER', 'decimals': 18},
    '0xB0B195aEFA3650A6908f15CdaC7D92F8a5791B0B': {'symbol': 'BOB', 'decimals': 18},
    '0xA719CB79Af39A9C10eDA2755E0938bCE35e9DE24': {'symbol': 'SEAN', 'decimals': 18},
    '0x3b79a28264fC52c7b4CEA90558AA0B162f7Faf57': {'symbol': 'WMEMO', 'decimals': 18},
    '0xe4A6f23Fb9e00Fca037Aa0EA0a6954dE0a6C53bF': {'symbol': 'TXAU', 'decimals': 18},
    '0x34ABce75D2f8f33940c721dCA0f562617787bfF3': {'symbol': 'TXAG', 'decimals': 18},
    '0x967Fb0D760ED3ce53AfE2f0A071674cccAe73550': {'symbol': 'XETA', 'decimals': 18},
    '0xcb3c5438Dae9Fe30B18eA53dA3Dab0E7dcaA0E4b': {'symbol': 'RISU', 'decimals': 9},
    }

token_symbol_lookup = {
    'LBR': {'address': '0xC98835e792553e505AE46e73a6fD27a23985aCcA', 'decimals': 18},
    'ETH': {'address': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'decimals': 18},
    'XLS': {'address': '0x6c862f803fF42A97D4A483AB761256ad8c90f4F8', 'decimals': 18},
    'BNA': {'address': '0x20910e5b5f087f6439DFcB0ddA4e27d1014Ac2b8', 'decimals': 18},
    'BDOT': {'address': '0x7884F51dC1410387371ce61747CB6264E1dAeE0B', 'decimals': 10},
    'QTF': {'address': '0x0FCbC31c503b4A9eD90E87f8Ff46C318a4a14260', 'decimals': 8},
    'SMC': {'address': '0xB6eDA82597808c96969C21a88bf6c749B441c44A', 'decimals': 18},
    'ALLUO': {'address': '0x1E5193ccC53f25638Aa22a940af899B692e10B09', 'decimals': 18},
    'HAIR': {'address': '0x9Ce115f0341ae5daBC8B477b74E83db2018A6f42', 'decimals': 18},
    'MDM': {'address': '0xd24DFf6117936B6ff97108CF26c1dD8865743d87', 'decimals': 18},
    'WFAIR': {'address': '0xC6065B9fc8171Ad3D29bad510709249681758972', 'decimals': 18},
    'ONE': {'address': '0x946551DD05C5Abd7CC808927480225ce36D8c475', 'decimals': 18},
    'DOKI': {'address': '0x9cEB84f92A0561fa3Cc4132aB9c0b76A59787544', 'decimals': 18},
    'ACRIA': {'address': '0x44F5909E97E1CBf5FBbDF0FC92FD83cde5d5c58A', 'decimals': 18},
    'SHINTAMA': {'address': '0x7e794eD35788b698AE60cefC98eE48015C4876dA', 'decimals': 9},
    'MONONOKE-INU': {'address': '0x4da08a1Bff50BE96bdeD5C7019227164b49C2bFc', 'decimals': 9},
    'EYE': {'address': '0x92d529163c5e880b9De86f01De0cB8924D790357', 'decimals': 18},
    'BHIG': {'address': '0x996229D0c6a485c7F4B52E092EAa907cB2def5C6', 'decimals': 18},
    'PUSUKE': {'address': '0x5DD0D493EA59d512EfC13D5C1F9d325775192877', 'decimals': 18},
    'YVWETH': {'address': '0xa258C4606Ca8206D8aA700cE2143D7db854D168c', 'decimals': 18},
    'FANS': {'address': '0xd33D0EB1c0c0295c3A2409a9101dd511823Bf217', 'decimals': 18},
    'POINT': {'address': '0x2d64750eC85f6eC442E3093D74C7b1b0a133BE6a', 'decimals': 18},
    'BANANA': {'address': '0x94e496474F1725f1c1824cB5BDb92d7691A4F03a', 'decimals': 18},
    'MAXR': {'address': '0x8a2eAd49D2123eDEE893181FB69527A022235C36', 'decimals': 9},
    'SIGN': {'address': '0xb06B8186cc008A79FD6722B1eefad07c14E97dA0', 'decimals': 18},
    'KNCL': {'address': '0xdd974D5C2e2928deA5F71b9825b8b646686BD200', 'decimals': 18},
    'DINU': {'address': '0xbb1EE07d6c7BAeB702949904080eb61f5D5e7732', 'decimals': 18},
    'EVED': {'address': '0x5aaEFe84E0fB3DD1f0fCfF6fA7468124986B91bd', 'decimals': 18},
    'DVP': {'address': '0x8E30ea2329D95802Fd804f4291220b0e2F579812', 'decimals': 18},
    'NXL': {'address': '0x6839aC5Ea8082688B706C773D5d2aFd7d503FDD4', 'decimals': 0},
    'NEPT': {'address': '0x355a824bEa1adc22733978A3748271E1bbB34130', 'decimals': 18},
    'X': {'address': '0x7f3141c4D6b047fb930991b450f1eD996a51CB26', 'decimals': 18},
    'BBTC': {'address': '0x9BE89D2a4cd102D8Fecc6BF9dA793be995C22541', 'decimals': 8},
    'BERRY': {'address': '0x6aEB95F06CDA84cA345c2dE0F3B7f96923a44f4c', 'decimals': 14},
    'GET': {'address': '0xCC1F757E3fa67B70E761C71F4b75B1E9F72263AF', 'decimals': 18},
    'CLM': {'address': '0x0B4C2708F052dca413600e237675e4d6778A9375', 'decimals': 16},
    'CIG': {'address': '0xCB56b52316041A62B6b5D0583DcE4A8AE7a3C629', 'decimals': 18},
    'TIME': {'address': '0x485d17A6f1B8780392d53D64751824253011A260', 'decimals': 8},
    'DAWN': {'address': '0x580c8520dEDA0a441522AEAe0f9F7A5f29629aFa', 'decimals': 18},
    'OTHR': {'address': '0x407a3E019c655B779ccD098Ff50377E4C5F1C334', 'decimals': 9},
    'AAMMUNILINKWETH': {'address': '0xb8db81B84d30E2387de0FF330420A4AAA6688134', 'decimals': 18},
    'RR': {'address': '0xfe98C2B85cAaFE841ca7Fa8b3B912a178A3f68eD', 'decimals': 18},
    'EKO': {'address': '0xa6a840E50bCaa50dA017b91A0D86B8b2d41156EE', 'decimals': 18},
    'IDLEUSDCSAFE': {'address': '0x3391bc034f2935eF0E1e41619445F998b2680D35', 'decimals': 18},
    'BSI': {'address': '0xEB50455805ebF8396d9177BBC4A371A376D00ECC', 'decimals': 18},
    'ALPHR': {'address': '0xaa99199d1e9644b588796F3215089878440D58e0', 'decimals': 18},
    'LOVE': {'address': '0xdE4CE5447ce0c67920a1371605A39187Cb6847c8', 'decimals': 18},
    'INDI': {'address': '0x3392D8A60B77F8d3eAa4FB58F09d835bD31ADD29', 'decimals': 18},
    'EUBC': {'address': '0xc37E8a31BA2d110c12f09f0239954A68b00bC599', 'decimals': 8},
    'SWT': {'address': '0xB9e7F8568e08d5659f5D29C4997173d84CdF2607', 'decimals': 18},
    'PCNT': {'address': '0x657B83A0336561C8f64389a6f5aDE675C04b0C3b', 'decimals': 18},
    'JESUS': {'address': '0xba386A4Ca26B85FD057ab1Ef86e3DC7BdeB5ce70', 'decimals': 18},
    'DTX': {'address': '0x82fdedfB7635441aA5A92791D001fA7388da8025', 'decimals': 18},
    'IJC': {'address': '0xc7fF1e126cC81e816915FF48c940ED9d4E6d05d6', 'decimals': 18},
    'ZZC': {'address': '0x51A673e060E1ea091B23B53d192058fFE0F2D6b6', 'decimals': 18},
    'TCT': {'address': '0x4824A7b64E3966B0133f4f4FFB1b9D6bEb75FFF7', 'decimals': 18},
    'GDG': {'address': '0xFD020998A1bB316dfE7B136Fe59Ae4B365D79978', 'decimals': 18},
    'BXR': {'address': '0x97A3BD8a445cC187c6A751F392e15C3B2134D695', 'decimals': 18},
    'HNB': {'address': '0x9c197c4b58527fAAAb67CB35E3145166B23D242e', 'decimals': 18},
    'CON': {'address': '0x4Dd672e77c795844fe3A464eF8eF0FAAe617C8fB', 'decimals': 18},
    'SX': {'address': '0x99fE3B1391503A1bC1788051347A1324bff41452', 'decimals': 18},
    'DST': {'address': '0xe3A46b2Bc1d83C731D58cAb765D3b45bCe789095', 'decimals': 18},
    'TEN': {'address': '0xDD16eC0F66E54d453e6756713E533355989040E4', 'decimals': 18},
    'BF': {'address': '0x5b71BEE9D961b1B848f8485EEC8d8787f80217F5', 'decimals': 18},
    'CCOMP': {'address': '0x70e36f6BF80a52b3B46b3aF8e106CC0ed743E8e4', 'decimals': 8},
    'CPU': {'address': '0x6D52DfeFb16BB9Cdc78bfCA09061e44574886626', 'decimals': 18},
    'XFT': {'address': '0x76BC677d444f1E9d57dAf5187ee2b7dC852745aE', 'decimals': 18},
    'VOLT': {'address': '0x7db5af2B9624e1b3B4Bb69D6DeBd9aD1016A58Ac', 'decimals': 9},
    'VENT': {'address': '0x5F0bc16D50F72d10b719dBF6845DE2E599eb5624', 'decimals': 18},
    'RFUEL': {'address': '0xaf9f549774ecEDbD0966C52f250aCc548D3F36E5', 'decimals': 18},
    'ALPHA': {'address': '0x48AF7b1c9dac8871C064f62FcEC0d9d6F7c269f5', 'decimals': 18},
    'BIGCAP': {'address': '0xE29F1241E6015A4949cb7E3f07234BA794006181', 'decimals': 18},
    'AMDG': {'address': '0xFf743A38efCF6f46fC0768B46a5Bdc264F7e92B3', 'decimals': 18},
    'DDDD': {'address': '0x8cA9a0fbd8DB501F013F2e9e33a1B9dC129A48E0', 'decimals': 18},
    'CGT': {'address': '0xF56b164efd3CFc02BA739b719B6526A6FA1cA32a', 'decimals': 18},
    'MULTI': {'address': '0x65Ef703f5594D2573eb71Aaf55BC0CB548492df4', 'decimals': 18},
    'DDD': {'address': '0x9F5F3CFD7a32700C93F971637407ff17b91c7342', 'decimals': 18},
    'ARTH': {'address': '0x8CC0F052fff7eaD7f2EdCCcaC895502E884a8a71', 'decimals': 18},
    'NXRA': {'address': '0x644192291cc835A93d6330b24EA5f5FEdD0eEF9e', 'decimals': 18},
    'LWD': {'address': '0x786e995A975C2ab7C1dd252A9019fB0c91c9Ca83', 'decimals': 18},
    'MCB': {'address': '0x4e352cF164E64ADCBad318C3a1e222E9EBa4Ce42', 'decimals': 18},
    'CEL': {'address': '0xaaAEBE6Fe48E54f431b0C390CfaF0b017d09D42d', 'decimals': 4},
    'FDAI': {'address': '0xe2bA8693cE7474900A045757fe0efCa900F6530b', 'decimals': 8},
    'AEGIS': {'address': '0x15Dd37a83564559927bBDcDbfcF85F60A2C6a419', 'decimals': 18},
    'KANGAL': {'address': '0x6E765D26388A17A6e86c49A8E41DF3F58aBcd337', 'decimals': 18},
    '1PECO': {'address': '0x7169bA45cF03E0377B464462AACbccdf1f721125', 'decimals': 18},
    'VITE': {'address': '0xadd5E881984783dD432F80381Fb52F45B53f3e70', 'decimals': 18},
    'XDEFI': {'address': '0x72B886d09C117654aB7dA13A14d603001dE0B777', 'decimals': 18},
    'CAJ': {'address': '0x3C6A7aB47B5F058Be0e7C7fE1A4b7925B8aCA40e', 'decimals': 18},
    'ZIP': {'address': '0xA9d2927d3a04309E008B6af6E2e282AE2952e7fD', 'decimals': 18},
    'BOTTO': {'address': '0x9DFAD1b7102D46b1b197b90095B5c4E9f5845BBA', 'decimals': 18},
    'BOB': {'address': '0x7D8146cf21e8D7cbe46054e01588207b51198729', 'decimals': 18},
    'ZOGI': {'address': '0x7e80e4d7d5725499791CF8b17A4586F1f0672A0C', 'decimals': 18},
    'SRC': {'address': '0x16587cF43F044aBa0165ffA00AcF412631194e4b', 'decimals': 18},
    'JGN': {'address': '0x73374Ea518De7adDD4c2B624C0e8B113955ee041', 'decimals': 18},
    'HRP': {'address': '0xbd15Ad921e1B480209AF549874a2fCb80DC312Bf', 'decimals': 18},
    'DFL': {'address': '0x09ce2B746C32528B7d864A1e3979Bd97d2f095AB', 'decimals': 18},
    'XMT': {'address': '0x3E5D9D8a63CC8a88748f229999CF59487e90721e', 'decimals': 18},
    'NKCLC': {'address': '0xd1b624f07a4D9B3e3746E33cb58f42dF079b5444', 'decimals': 18},
    'ARTM': {'address': '0x19ebAA7F212b09de2aee2a32D40338553C70e2e3', 'decimals': 18},
    'HAHA': {'address': '0xd87D72248093597df8D56d2a53C1ab7c1A0Cc8DA', 'decimals': 18},
    'WSB': {'address': '0x15F20f9DFDf96CCf6AC96653b7C0aBfe4A9C9f0f', 'decimals': 18},
    'MONA': {'address': '0x275f5Ad03be0Fa221B4C6649B8AeE09a42D9412A', 'decimals': 18},
    'SMTY': {'address': '0xbF776e4FCa664D791C4Ee3A71e2722990E003283', 'decimals': 18},
    'LOL': {'address': '0x5978708d6ccE1CC9640Eed47422D64c91BbD5171', 'decimals': 18},
    'BNS': {'address': '0x19E2a43Fbbc643C3B2D9667D858D49cAd17bc2b5', 'decimals': 8},
    'JCC': {'address': '0xeA7aA1eDd21735A5ab05EE3E90869016191e274E', 'decimals': 18},
    'REVV': {'address': '0x557B933a7C2c45672B610F8954A3deB39a51A8Ca', 'decimals': 18},
    'BUY': {'address': '0x31Fdd1c6607F47C14a2821f599211c67AC20FA96', 'decimals': 18},
    'DV': {'address': '0x2a03A891ADD2dc6d0F7b94419086630ba5cB65b6', 'decimals': 18},
    'JDB': {'address': '0xBaD96Ea5D43156a99A94Cd1Fd36a330aA7E2273e', 'decimals': 18},
    'MTN': {'address': '0x41dBECc1cdC5517C6f76f6a6E836aDbEe2754DE3', 'decimals': 18},
    'LIMIT': {'address': '0x1712Aad2C773ee04bdC9114b32163c058321CD85', 'decimals': 18},
    'ARMOR': {'address': '0x1337DEF16F9B486fAEd0293eb623Dc8395dFE46a', 'decimals': 18},
    'UNDEAD': {'address': '0x310C8F00b9dE3c31Ab95ea68feb6C877538f7947', 'decimals': 18},
    'STACME': {'address': '0x7AC168c81F4F3820Fa3F22603ce5864D6aB3C547', 'decimals': 8},
    'FLUT': {'address': '0x4F08705FB8F33AffC231ed66e626B40E84A71870', 'decimals': 11},
    'XOR': {'address': '0x40FD72257597aA14C7231A7B1aaa29Fce868F677', 'decimals': 18},
    'PKR': {'address': '0x001A8Ffcb0f03e99141652eBCdecDb0384E3bd6c', 'decimals': 18},
    'LITX': {'address': '0x4B48068864F77261838C7849A12853FB94c77a91', 'decimals': 18},
    'BISC': {'address': '0x4295c8556AFEE00264C0789dde2ddd2dba71AcFe', 'decimals': 18},
    'JPGC': {'address': '0x5891599664Ed15C6e88041B4F5bc08594f026f0e', 'decimals': 18},
    'ISLA': {'address': '0x20a68F9e34076b2dc15ce726d7eEbB83b694702d', 'decimals': 18},
    'KIRO': {'address': '0xB1191F691A355b43542Bea9B8847bc73e7Abb137', 'decimals': 18},
    'HIMOONBIRDS': {'address': '0x0A7b89E66a1DC16633abdfd132bAe05827d3BFa5', 'decimals': 18},
    'PLOT': {'address': '0x72F020f8f3E8fd9382705723Cd26380f8D0c66Bb', 'decimals': 18},
    'CLEV': {'address': '0x72953a5C32413614d24C29c84a66AE4B59581Bbf', 'decimals': 18},
    'ZKP': {'address': '0x909E34d3f6124C324ac83DccA84b74398a6fa173', 'decimals': 18},
    'HAKA': {'address': '0xD85AD783cc94bd04196a13DC042A3054a9B52210', 'decimals': 18},
    'ASH': {'address': '0x64D91f12Ece7362F91A6f8E7940Cd55F05060b92', 'decimals': 18},
    'FORM': {'address': '0x21381e026Ad6d8266244f2A583b35F9E4413FA2a', 'decimals': 18},
    'JET': {'address': '0x8727c112C712c4a03371AC87a74dD6aB104Af768', 'decimals': 18},
    'RFR': {'address': '0xd0929d411954c47438dc1d871dd6081F5C5e149c', 'decimals': 4},
    'SBET': {'address': '0x2ED2Cc2c858A8a8219fD2F2d9E170285dBd02756', 'decimals': 18},
    'LIMEX': {'address': '0x400b1d8a7dd8C471026B2c8CBe1062b27d120538', 'decimals': 8},
    'UUSDC': {'address': '0xBc5991cCd8cAcEba01edC44C2BB9832712c29cAB', 'decimals': 6},
    'DDRT': {'address': '0x94501b6A153C8973fd1f321fCC8188d40dC5d72d', 'decimals': 8},
    'MCO': {'address': '0xB63B606Ac810a52cCa15e44bB630fd42D8d1d83d', 'decimals': 8},
    'IBETH': {'address': '0x67B66C99D3Eb37Fa76Aa3Ed1ff33E8e39F0b9c7A', 'decimals': 18},
    'RADAR': {'address': '0xf9FBE825BFB2bF3E387af0Dc18caC8d87F29DEa8', 'decimals': 18},
    'HRD': {'address': '0x461B71cff4d4334BbA09489acE4b5Dc1A1813445', 'decimals': 9},
    'XVC': {'address': '0x4fABf135bCF8111671870d4399af739683198f96', 'decimals': 18},
    'CNB': {'address': '0xC538143202f3b11382D8606aae90a96b042a19DB', 'decimals': 18},
    'PIKA': {'address': '0x60F5672A271C7E39E787427A18353ba59A4A3578', 'decimals': 18},
    'RICE': {'address': '0xbD9908b0Cdd50386F92efCC8e1d71766C2782Df0', 'decimals': 18},
    'BABYSHIBA': {'address': '0x66A64a8A9cdc54EF97e45a051bA1c8f3D4b41A1D', 'decimals': 18},
    'BAKED': {'address': '0xa4cb0dCe4849BdcAd2d553E9e68644Cf40E26ccE', 'decimals': 18},
    'STPT': {'address': '0xDe7D85157d9714EADf595045CC12Ca4A5f3E2aDb', 'decimals': 18},
    'AAMMWBTC': {'address': '0x13B2f6928D7204328b0E8E4BCd0379aA06EA21FA', 'decimals': 8},
    'BUIDL': {'address': '0x7b123f53421b1bF8533339BFBdc7C98aA94163db', 'decimals': 18},
    'GOL': {'address': '0x083D41d6DD21EE938f0c055CA4fb12268DF0EfaC', 'decimals': 4},
    'VRN': {'address': '0x72377f31e30a405282b522d588AEbbea202b4f23', 'decimals': 18},
    'DAIN': {'address': '0xd7d706ED3598a354A4aDFce5D8C5383dF99a4461', 'decimals': 18},
    'WITCH': {'address': '0xdc524e3c6910257744C1F93Cf15E9F472b5bD236', 'decimals': 18},
    'JT': {'address': '0xEb7355C2f217b3485a591332Fe13C8c5A76A581D', 'decimals': 18},
    'CELR': {'address': '0x4F9254C83EB525f9FCf346490bbb3ed28a81C667', 'decimals': 18},
    'BDS': {'address': '0x1856BC58f5511269aFE2CA1e89Bf08CBDCD01100', 'decimals': 18},
    'VETME': {'address': '0xe7eF051C6EA1026A70967E8F04da143C67Fa4E1f', 'decimals': 9},
    'GIX': {'address': '0xbD434a09191D401da3283a5545bB3515d033B8c4', 'decimals': 18},
    'CSUSHI': {'address': '0x4B0181102A0112A2ef11AbEE5563bb4a3176c9d7', 'decimals': 8},
    'HUB': {'address': '0xba358B6f5b4c0215650444B8C30D870B55050D2D', 'decimals': 18},
    'ATF': {'address': '0x518b63Da813D46556FEa041A88b52e3CAa8C16a8', 'decimals': 18},
    'LIT': {'address': '0x763Fa6806e1acf68130D2D0f0df754C93cC546B2', 'decimals': 18},
    'STATE': {'address': '0x00C2999c8B2AdF4ABC835cc63209533973718eB1', 'decimals': 18},
    'EUM': {'address': '0x6aB4A7d75B0A42B6Bc83E852daB9E121F9C610Aa', 'decimals': 18},
    'EROWAN': {'address': '0x07baC35846e5eD502aA91AdF6A9e7aA210F2DcbE', 'decimals': 18},
    'USDS': {'address': '0xA4Bdb11dc0a2bEC88d24A3aa1E6Bb17201112eBe', 'decimals': 6},
    'USDI': {'address': '0x2A54bA2964C8Cd459Dc568853F79813a60761B58', 'decimals': 18},
    'DIGITS': {'address': '0xBE56ab825fD35678A32dc35bc4EB17e238e1404F', 'decimals': 18},
    'OMC': {'address': '0xd6bD97a26232bA02172Ff86b055d5D7bE789335B', 'decimals': 18},
    'DRE': {'address': '0x1977bE49C33dFacF6590C16ca9a9CfA0463f663c', 'decimals': 18},
    'ARCHA': {'address': '0x36E43065e977bC72CB86Dbd8405fae7057CDC7fD', 'decimals': 9},
    'RLY': {'address': '0xf1f955016EcbCd7321c7266BccFB96c68ea5E49b', 'decimals': 18},
    'RELIC': {'address': '0x77132Bb08B02a7a87732ec289EFaB45edF49884F', 'decimals': 18},
    'DKA': {'address': '0x5dc60C4D5e75D22588FA17fFEB90A63E535efCE0', 'decimals': 18},
    'ESPRO': {'address': '0x29C56E7CB9C840d2b2371b17e28BaB44AD3c3ead', 'decimals': 18},
    'TUP': {'address': '0x7714f320Adca62B149df2579361AfEC729c5FE6A', 'decimals': 18},
    'WPR': {'address': '0x4CF488387F035FF08c371515562CBa712f9015d4', 'decimals': 18},
    'SPUME': {'address': '0xa16a609fF4e1A15b6cCb469E7a5dD14E89305283', 'decimals': 18},
    'FAI': {'address': '0xCda2f16C6Aa895D533506B426AFF827b709c87F5', 'decimals': 18},
    'ADAI': {'address': '0x028171bCA77440897B824Ca71D1c56caC55b68A3', 'decimals': 18},
    'DEFO': {'address': '0xe481f2311C774564D517d015e678c2736A25Ddd3', 'decimals': 18},
    'CHEQ': {'address': '0x70EDF1c215D0ce69E7F16FD4E6276ba0d99d4de7', 'decimals': 9},
    'YVSNX': {'address': '0xF29AE508698bDeF169B89834F76704C3B205aedf', 'decimals': 18},
    'IDK': {'address': '0x61fd1c62551850D0c04C76FcE614cBCeD0094498', 'decimals': 8},
    'SDCRV': {'address': '0xD1b5651E55D4CeeD36251c61c50C889B36F6abB5', 'decimals': 18},
    'AUSDT': {'address': '0x3Ed3B47Dd13EC9a98b44e6204A523E766B225811', 'decimals': 6},
    'AAMMBPTWBTCWETH': {'address': '0x358bD0d980E031E23ebA9AA793926857703783BD', 'decimals': 18},
    'BLXM': {'address': '0x38d9eb07A7b8Df7D86F440A4A5c4a4c1a27E1a08', 'decimals': 18},
    'BORA': {'address': '0x26fb86579e371c7AEdc461b2DdEF0A8628c93d3B', 'decimals': 18},
    'MFI': {'address': '0xEaDC218Ac4cB7895a5A56e6484646b48f841C45a', 'decimals': 18},
    'AWBTC': {'address': '0x9ff58f4fFB29fA2266Ab25e75e2A8b3503311656', 'decimals': 8},
    'ORD': {'address': '0xBe00734799A67a62Af2819825580318aC1B1E4Ec', 'decimals': 18},
    'ABPT': {'address': '0x41A08648C3766F9F9d85598fF102a08f4ef84F84', 'decimals': 18},
    'CNV': {'address': '0x000000007a58f5f58E697e51Ab0357BC9e260A04', 'decimals': 18},
    'LCMS': {'address': '0x86911B82BfA596e9377836838AC1f0d9AD4Ecc19', 'decimals': 18},
    'STIMA': {'address': '0xD2e5decc08A80be6538F89f9AB8ff296e2f724Df', 'decimals': 6},
    'SPICE': {'address': '0x9b6dB7597a74602a5A806E33408e7E2DAFa58193', 'decimals': 18},
    'UWL': {'address': '0xdbDD6F355A37b94e6C7D32fef548e98A280B8Df5', 'decimals': 18},
    'BRCP': {'address': '0x73484A262730d1d422610729e828346F9B2ff480', 'decimals': 18},
    'KEK': {'address': '0x67954768E721FAD0f0f21E33e874497C73ED6a82', 'decimals': 18},
    'STZ': {'address': '0x3f5294DF68F871241c4B18fcF78ebD8Ac18aB654', 'decimals': 18},
    'SIMP': {'address': '0x41C21693e60FC1a5dBb7c50e54E7A6016aA44C99', 'decimals': 18},
    'PARA': {'address': '0xD16fd95D949F996e3808EeEa0e3881C59E76EF1E', 'decimals': 18},
    'TROVE': {'address': '0x59c6900949aD1835f07a04321f4d9934a054E114', 'decimals': 18},
    'WSB': {'address': '0x0414D8C87b271266a5864329fb4932bBE19c0c49', 'decimals': 18},
    'SLOCK': {'address': '0xbcBf09b4b83EB3893D58F0665d21460C04e39D8E', 'decimals': 18},
    'PHX': {'address': '0xAeC65404DdC3af3C897AD89571d5772C1A695F22', 'decimals': 18},
    'PSP': {'address': '0xcAfE001067cDEF266AfB7Eb5A286dCFD277f3dE5', 'decimals': 18},
    'TX': {'address': '0x8954D907520532c1f0d89d42569232Fd0f995Fdf', 'decimals': 8},
    'BUP': {'address': '0xB04DFdb8271ed2d5e13858562C44A77D3CEb9e57', 'decimals': 18},
    'XRT': {'address': '0x95652e39E6FB55d41f6A975079035829293e0E16', 'decimals': 8},
    'CORX': {'address': '0x26a604DFFE3ddaB3BEE816097F81d3C4a2A4CF97', 'decimals': 8},
    '1EARTH': {'address': '0x9e04F519b094F5F8210441e285f603f4d2b50084', 'decimals': 18},
    'RED': {'address': '0x76960Dccd5a1fe799F7c29bE9F19ceB4627aEb2f', 'decimals': 18},
    'KUMA': {'address': '0x48C276e8d03813224bb1e55F953adB6d02FD3E02', 'decimals': 18},
    'PXG': {'address': '0x47e67BA66b0699500f18A53F94E2b9dB3D47437e', 'decimals': 18},
    'UNN': {'address': '0x226f7b842E0F0120b7E194D05432b3fd14773a9D', 'decimals': 18},
    'CLAW': {'address': '0x41B25Ff6431074959532dB7435DAdacA65a21D1C', 'decimals': 18},
    'SML': {'address': '0x1A403E1c96792dFedb8232cF56400Eb72Ab95Acb', 'decimals': 18},
    'HUSL': {'address': '0xa2881F7F441267042f9778fFA0d4F834693426be', 'decimals': 18},
    'AAMMUNIWBTCWETH': {'address': '0xc58F53A8adff2fB4eb16ED56635772075E2EE123', 'decimals': 18},
    'AIN': {'address': '0x3A810ff7211b40c4fA76205a14efe161615d0385', 'decimals': 18},
    'ELT': {'address': '0xc0AE17Eb994fa828540FFa53776B3830233A1B02', 'decimals': 18},
    'HIMEEBITS': {'address': '0xBBeca80A4c99c3b1BD3C10E64595d0Fc7dC26eE0', 'decimals': 18},
    'MTH': {'address': '0xaF4DcE16Da2877f8c9e00544c93B62Ac40631F16', 'decimals': 5},
    'DRGN': {'address': '0x419c4dB4B9e25d6Db2AD9691ccb832C8D9fDA05E', 'decimals': 18},
    'UFEWO': {'address': '0xcccF837f40D334F8602f031e64B52AD4CD2b6601', 'decimals': 18},
    'HEROS': {'address': '0xb622400807765e73107B7196F444866D7EdF6f62', 'decimals': 9},
    'VERI': {'address': '0x8f3470A7388c05eE4e7AF3d01D8C722b0FF52374', 'decimals': 18},
    'PMT': {'address': '0xC1322D8aE3B0e2E437e0AE36388D0CFD2C02f1c9', 'decimals': 4},
    'LUXO': {'address': '0xD39a2CeCBA2657e125Ba6a5c98ad2F6b6D7E83FD', 'decimals': 18},
    'KODACHI': {'address': '0x57c411e9a358e2d2d0a6b058cEDb709175e8fd16', 'decimals': 18},
    'GMB': {'address': '0xA0008F510fE9eE696E7E320C9e5cbf61E27791Ee', 'decimals': 18},
    'POLI': {'address': '0xa1a36D3537bBe375Cc9694795F663DDC8D516Db9', 'decimals': 18},
    'BAO': {'address': '0xCe391315b414D4c7555956120461D21808A69F3A', 'decimals': 18},
    'CRVFRAX': {'address': '0x3175Df0976dFA876431C2E9eE6Bc45b65d3473CC', 'decimals': 18},
    'FND': {'address': '0xB13d747d783BF1A9F1D65Df74C080C890045f17e', 'decimals': 7},
    'KARMA': {'address': '0xdfe691F37b6264a90Ff507EB359C45d55037951C', 'decimals': 4},
    'PHCR': {'address': '0x37E83a94c6B1Bdb816B59aC71dd02CF154d8111F', 'decimals': 18},
    'ABLOCK': {'address': '0xE692c8D72bd4aC7764090d54842a305546dd1dE5', 'decimals': 8},
    'XAUT': {'address': '0x68749665FF8D2d112Fa859AA293F07A622782F38', 'decimals': 6},
    'MATIC': {'address': '0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0', 'decimals': 18},
    'BKR': {'address': '0x77271813bd9167e75B5DF9c230cf58d64f0A58Fd', 'decimals': 18},
    'RVF': {'address': '0xdc8aF07A7861bedD104B8093Ae3e9376fc8596D2', 'decimals': 18},
    'XOT': {'address': '0x22B48E1f20043D1Db5F2a11CbF1d520A4F20b198', 'decimals': 18},
    'RAGE': {'address': '0x94804dc4948184fFd7355f62Ccbb221c9765886F', 'decimals': 18},
    'USDF': {'address': '0x963035453633f6f7433032c958F82Eb3043D8cD2', 'decimals': 18},
    'EMAX': {'address': '0x15874d65e649880c2614e7a480cb7c9A55787FF6', 'decimals': 18},
    'SUSHI': {'address': '0x6B3595068778DD592e39A122f4f5a5cF09C90fE2', 'decimals': 18},
    'QRDO': {'address': '0x4123a133ae3c521FD134D7b13A2dEC35b56c2463', 'decimals': 8},
    'AYFI': {'address': '0x5165d24277cD063F5ac44Efd447B27025e888f37', 'decimals': 18},
    'ARAI': {'address': '0xc9BC48c72154ef3e5425641a3c747242112a46AF', 'decimals': 18},
    'BGC': {'address': '0xD34c9281585866A9fb2af0931adb1fA35ed44039', 'decimals': 8},
    'BAKT': {'address': '0x8379f52d09b9998eCbA731288EE4fE532fD91C0B', 'decimals': 18},
    'TRUTH': {'address': '0xAF75d880b3128981D1fEd3292Fc02E3FB37Acd53', 'decimals': 9},
    'OTB': {'address': '0xA86a0Da9D05d0771955DF05B44Ca120661aF16DE', 'decimals': 18},
    'USDK': {'address': '0x1c48f86ae57291F7686349F12601910BD8D470bb', 'decimals': 18},
    'ZDEX': {'address': '0x5150956E082C748Ca837a5dFa0a7C10CA4697f9c', 'decimals': 18},
    'PRBLY': {'address': '0x6361F338Ab8DEf2AF3f2A1Be7BD8A7dB3156F7E7', 'decimals': 18},
    'SHANG': {'address': '0xee772CEc929D8430b4Fa7a01cD7FbD159a68Aa83', 'decimals': 18},
    'JPYC': {'address': '0x2370f9d504c7a6E775bf6E14B3F12846b594cD53', 'decimals': 18},
    'VRH': {'address': '0xeAd482da0793B00bbAe0E34C8cfaE6DAf29a44b2', 'decimals': 18},
    'ICHIGO': {'address': '0x8254C1C134436F74047F79eaAeA97E3324eF78B5', 'decimals': 9},
    'CHONK': {'address': '0x84679bc467DC6c2c40ab04538813AfF3796351f1', 'decimals': 18},
    'YLD': {'address': '0xF94b5C5651c888d928439aB6514B93944eEE6F48', 'decimals': 18},
    'RFOX': {'address': '0xa1d6Df714F91DeBF4e0802A542E13067f31b8262', 'decimals': 18},
    'WXMR': {'address': '0x465e07d6028830124BE2E4aA551fBe12805dB0f5', 'decimals': 18},
    'LPT': {'address': '0x58b6A8A3302369DAEc383334672404Ee733aB239', 'decimals': 18},
    'YV1INCH': {'address': '0xB8C3B7A2A618C552C23B1E4701109a9E756Bab67', 'decimals': 18},
    'PAA': {'address': '0x3D9Ac8e7a9C9bE11DFac1677dDa901E28d44527f', 'decimals': 8},
    'EUROE': {'address': '0x820802Fa8a99901F52e39acD21177b0BE6EE2974', 'decimals': 6},
    'JACY': {'address': '0x916c5DE09cF63f6602d1e1793FB41F6437814A62', 'decimals': 9},
    'CHER': {'address': '0xa20f77B7ad5A88badC48800C56507B7274c06Fdc', 'decimals': 18},
    'BIPS': {'address': '0x5Cb888182fBfFdb62C08fb4B5a343914F00FdfeE', 'decimals': 18},
    'SWFL': {'address': '0xBa21Ef4c9f433Ede00badEFcC2754B8E74bd538A', 'decimals': 18},
    'THE': {'address': '0xB4a677B0E363c3815d46326954a4E4d2B1ACe357', 'decimals': 18},
    'ASUSD': {'address': '0x6C5024Cd4F8A59110119C56f8933403A539555EB', 'decimals': 18},
    'AUNI': {'address': '0xB9D7CB55f463405CDfBe4E90a6D2Df01C2B92BF1', 'decimals': 18},
    'ANKR': {'address': '0x8290333ceF9e6D528dD5618Fb97a76f268f3EDD4', 'decimals': 18},
    'SST': {'address': '0x2863916C6ebDBBf0c6f02F87b7eB478509299868', 'decimals': 18},
    'MOF': {'address': '0x653430560bE843C4a3D143d0110e896c2Ab8ac0D', 'decimals': 16},
    'VISION': {'address': '0xF406F7A9046793267bc276908778B29563323996', 'decimals': 18},
    'MKONG': {'address': '0xeE6b9CF11D968E0bAc7BFfF547577B8AE35B8065', 'decimals': 9},
    'YVYFI': {'address': '0xdb25cA703181E7484a155DD612b06f57E12Be5F0', 'decimals': 18},
    'CNG': {'address': '0x5C1d9aA868a30795F92fAe903eDc9eFF269044bf', 'decimals': 18},
    'FRONT': {'address': '0xf8C3527CC04340b208C854E985240c02F7B7793f', 'decimals': 18},
    '': {'address': '0xDadb4aE5B5D3099Dd1f586f990B845F2404A1c4c', 'decimals': 18},
    'AGT': {'address': '0x0BF43350076F95e0d16120b4D6bdfA1C9D50BDBD', 'decimals': 18},
    'SYN': {'address': '0x0f2D719407FdBeFF09D87557AbB7232601FD9F29', 'decimals': 18},
    'AGUSD': {'address': '0xD37EE7e4f452C6638c96536e68090De8cBcdb583', 'decimals': 2},
    'GENI': {'address': '0x444444444444C1a66F394025Ac839A535246FCc8', 'decimals': 9},
    'FCC': {'address': '0x171b1DAefac13A0A3524Fcb6BEDdC7B31E58e079', 'decimals': 18},
    'ART': {'address': '0x570936840fA942C96ca4C57A170dDa494DD9B9C6', 'decimals': 18},
    'TRAXX': {'address': '0xD43Be54C1aedf7Ee4099104f2DaE4eA88B18A249', 'decimals': 18},
    'HANU': {'address': '0x72E5390EDb7727E3d4e3436451DADafF675dBCC0', 'decimals': 12},
    'PGPAY': {'address': '0xC949fC82A15964FB5B97e5cf8f9ffed139086821', 'decimals': 18},
    'GAME': {'address': '0x882e5b370D595E50c24b2a0e7a94e87Cc32ADdA1', 'decimals': 18},
    'KIF': {'address': '0x177BA0cac51bFC7eA24BAd39d81dcEFd59d74fAa', 'decimals': 18},
    'FREQAI': {'address': '0xB685145D7F127b9093d7f9278BaE902EF59Ff486', 'decimals': 18},
    'WHITE': {'address': '0x5F0E628B693018f639D10e4A4F59BD4d8B2B6B44', 'decimals': 18},
    'PETH': {'address': '0x836A808d4828586A69364065A1e064609F5078c7', 'decimals': 18},
    'KAP': {'address': '0x9625cE7753ace1fa1865A47aAe2c5C2Ce4418569', 'decimals': 18},
    'LAYER': {'address': '0x0fF6ffcFDa92c53F615a4A75D982f399C989366b', 'decimals': 18},
    'XEN': {'address': '0x06450dEe7FD2Fb8E39061434BAbCFC05599a6Fb8', 'decimals': 18},
    'CVX': {'address': '0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B', 'decimals': 18},
    'NEU': {'address': '0xA823E6722006afe99E91c30FF5295052fe6b8E32', 'decimals': 18},
    'VIBE': {'address': '0xe8Ff5C9c75dEb346acAc493C463C8950Be03Dfba', 'decimals': 18},
    'MILADY': {'address': '0x227c7DF69D3ed1ae7574A1a7685fDEd90292EB48', 'decimals': 18},
    'SEDO': {'address': '0x0F00f1696218EaeFa2D2330Df3D6D1f94813b38f', 'decimals': 8},
    'HERO': {'address': '0x524Ebc93BeEf838F70b4Ae54b675d3e971D5884E', 'decimals': 9},
    'GOLD': {'address': '0x40d1F63B5D2048e67E9bEdB1B4c2F1A9fb4b6817', 'decimals': 18},
    'PUDGY': {'address': '0xAbeA7663c472648d674bd3403D94C858dFeEF728', 'decimals': 18},
    'KRX': {'address': '0x93ad9b819C88D98B4c9641470A96E24769Ae7922', 'decimals': 9},
    'PEOPLE': {'address': '0x7A58c0Be72BE218B41C608b7Fe7C5bB630736C71', 'decimals': 18},
    'BRMV': {'address': '0x501e8726d06Cdef66f3e0cb67f54924CcA1cC894', 'decimals': 18},
    'DRC': {'address': '0xb78B3320493a4EFaa1028130C5Ba26f0B6085Ef8', 'decimals': 18},
    'BASAN': {'address': '0x970cf867Ca0530a989f222bE01FdD67C1ab5b2bF', 'decimals': 18},
    'IDLEUSDCYIELD': {'address': '0x5274891bEC421B39D23760c04A6755eCB444797C', 'decimals': 18},
    'XAMP': {'address': '0xf911a7ec46a2c6fa49193212fe4a2a9B95851c27', 'decimals': 9},
    'SSV': {'address': '0x9D65fF81a3c488d585bBfb0Bfe3c7707c7917f54', 'decimals': 18},
    'COFI': {'address': '0x3136eF851592aCf49CA4C825131E364170FA32b3', 'decimals': 18},
    'HT': {'address': '0x6f259637dcD74C767781E37Bc6133cd6A68aa161', 'decimals': 18},
    'ARTEM': {'address': '0x9B83f827928aBdf18cF1F7e67053572b9bceff3a', 'decimals': 18},
    'ARW': {'address': '0x351cAa9045D65107b9d311D922D15887cfd634E4', 'decimals': 18},
    'DGX': {'address': '0x4f3AfEC4E5a3F2A6a1A411DEF7D7dFe50eE057bF', 'decimals': 9},
    'CHO': {'address': '0xBBa39Fd2935d5769116ce38d46a71bde9cf03099', 'decimals': 18},
    'ECU': {'address': '0x8fc9b6354E839AB1c8B31F4afa53607092B8C2e5', 'decimals': 18},
    'KROM': {'address': '0x3af33bEF05C2dCb3C7288b77fe1C8d2AeBA4d789', 'decimals': 18},
    'MIST': {'address': '0x88ACDd2a6425c3FaAE4Bc9650Fd7E27e0Bebb7aB', 'decimals': 18},
    'PP': {'address': '0x9565c2036963697786705120Fc59310F747bCfD0', 'decimals': 18},
    'MONG': {'address': '0x1ce270557C1f68Cfb577b856766310Bf8B47FD9C', 'decimals': 18},
    'SDAO': {'address': '0x993864E43Caa7F7F12953AD6fEb1d1Ca635B875F', 'decimals': 18},
    'TBC': {'address': '0x03042ae6fcFd53E3A0BAa1FaB5Ce70E0cb74E6fb', 'decimals': 18},
    'PAID': {'address': '0x1614F18Fc94f47967A3Fbe5FfcD46d4e7Da3D787', 'decimals': 18},
    'AGVC': {'address': '0x8b79656FC38a04044E495e22fAD747126ca305C4', 'decimals': 18},
    'XIV': {'address': '0x44f262622248027f8E2a8Fb1090c4Cf85072392C', 'decimals': 18},
    'IDRT': {'address': '0x998FFE1E43fAcffb941dc337dD0468d52bA5b48A', 'decimals': 2},
    'TOK': {'address': '0x4Fb721eF3Bf99e0f2c193847afA296b9257d3C30', 'decimals': 8},
    'DBR': {'address': '0xAD038Eb671c44b853887A7E32528FaB35dC5D710', 'decimals': 18},
    'SRT': {'address': '0x22987407FD1fC5A971e3FDA3B3e74C88666cDa91', 'decimals': 18},
    'SDG': {'address': '0xCF8829AE9384540c886A151fac3a865794cB9a01', 'decimals': 18},
    'MUNCH': {'address': '0x944eeE930933BE5E23b690c8589021Ec8619a301', 'decimals': 9},
    'SPILLWAYS': {'address': '0x8790f2fc7Ca2e7Db841307FB3f4e72a03Baf7b47', 'decimals': 18},
    'WINTER': {'address': '0xCcbA0b2bc4BAbe4cbFb6bD2f1Edc2A9e86b7845f', 'decimals': 18},
    'PIXIU': {'address': '0x37A15C92e67686aA268df03d4C881A76340907E8', 'decimals': 0},
    'BAO': {'address': '0x374CB8C27130E2c9E04F44303f3c8351B9De61C1', 'decimals': 18},
    'CKU': {'address': '0x08F7be99ED83369541501d60f4e66F8e34c3F736', 'decimals': 18},
    'ILV': {'address': '0x767FE9EDC9E0dF98E07454847909b5E959D7ca0E', 'decimals': 18},
    'XCF': {'address': '0x010D14d36C3eA6570D240ae3ac9d660398f7C48e', 'decimals': 18},
    'ONEICHI': {'address': '0x4db2c02831c9ac305FF9311Eb661f80f1dF61e07', 'decimals': 18},
    'KLEE': {'address': '0xA67E9F021B9d208F7e3365B2A155E3C55B27de71', 'decimals': 9},
    'JFIN': {'address': '0x940BdCb99A0Ee5Fb008A606778AE87Ed9789F257', 'decimals': 18},
    'PP': {'address': '0x8442E0e292186854BB6875b2A0fc1308b9Ded793', 'decimals': 9},
    'IDLEWBTCYIELD': {'address': '0x8C81121B15197fA0eEaEE1DC75533419DcfD3151', 'decimals': 18},
    'ARTI': {'address': '0x3625b55f653F8C7c884aC668696881df43631D44', 'decimals': 8},
    'ADVT': {'address': '0x69e37422cB87d963367F73A119C8ce9a4D529b72', 'decimals': 9},
    'IDLETUSDYIELD': {'address': '0xc278041fDD8249FE4c1Aad1193876857EEa3D68c', 'decimals': 18},
    'BCT': {'address': '0x350D3f0f41b5B21F0e252fe2645AE9D55562150a', 'decimals': 9},
    'TRYC': {'address': '0x0000000005c6B7C1fd10915a05f034F90d524D6E', 'decimals': 6},
    'BKC': {'address': '0x34bdf48A8F753de4822a6CFB1FEE275F9b4D662e', 'decimals': 18},
    'KAIROS': {'address': '0x3Be65bd36C1aD28D33Acc1C9dD8338a2706cA000', 'decimals': 9},
    'TRENDAI': {'address': '0x6fc73113fC1aFaB4C28d3DD4c537A1DA6045d47D', 'decimals': 18},
    'REDPANDA': {'address': '0x9ba4CBcBb920F67f54A8F8f0001D721E0B8944b9', 'decimals': 9},
    'REV': {'address': '0x2ef52Ed7De8c5ce03a4eF0efbe9B7450F2D7Edc9', 'decimals': 6},
    'BSL': {'address': '0x0AF55d5fF28A3269d69B98680Fd034f115dd53Ac', 'decimals': 8},
    'MIS': {'address': '0x024B6e7DC26F4d5579bDD936F8d7BC31f2339999', 'decimals': 18},
    'UCASH': {'address': '0x92e52a1A235d9A103D970901066CE910AAceFD37', 'decimals': 8},
    'CCS': {'address': '0x1da4858ad385cc377165A298CC2CE3fce0C5fD31', 'decimals': 0},
    'BABIL': {'address': '0xf92364c2369a2633FfcD7dB1B18D1Fafff6bCbaB', 'decimals': 8},
    'PMD': {'address': '0xf5555732B3925356964695578feFcFfCD31bCbb8', 'decimals': 9},
    'WTL': {'address': '0x9a0587EaE7eF64b2B38A10442a44CfA43EDd7D2A', 'decimals': 18},
    'BAT': {'address': '0x0D8775F648430679A709E98d2b0Cb6250d2887EF', 'decimals': 18},
    'DSS': {'address': '0x213C53C96A01a89E6Dcc5683cF16473203E17513', 'decimals': 18},
    'BNTX': {'address': '0x86d1d12523B65203851c571FcC029bF90903fB6d', 'decimals': 18},
    'XFIT': {'address': '0x4aa41bC1649C9C3177eD16CaaA11482295fC7441', 'decimals': 18},
    'MOPS': {'address': '0x602f65BB8B8098Ad804E99DB6760Fd36208cD967', 'decimals': 18},
    'COCOS': {'address': '0xc4c7Ea4FAB34BD9fb9a5e1B1a98Df76E26E6407c', 'decimals': 18},
    'BBADGER': {'address': '0x19D97D8fA813EE2f51aD4B4e04EA08bAf4DFfC28', 'decimals': 18},
    'CVXFPIS': {'address': '0xa2847348b58CEd0cA58d23c7e9106A49f1427Df6', 'decimals': 18},
    'MORE': {'address': '0x305DE070488C8469dFAC957226c9c900c4BFbA22', 'decimals': 8},
    'ECO': {'address': '0x8dBF9A4c99580fC7Fd4024ee08f3994420035727', 'decimals': 18},
    'COLL': {'address': '0x957891C11616D3E0B0A76a76fb42724C382E0eF3', 'decimals': 18},
    'LEVX': {'address': '0x86F0cdB38a853420be4dbF6a3a9a3fe0756bc701', 'decimals': 9},
    'WCX': {'address': '0xC014186Cf1Ba36032aaEc7f96088f09eb3934347', 'decimals': 18},
    'OPUL': {'address': '0x80D55c03180349Fff4a229102F62328220A96444', 'decimals': 18},
    'ZRX': {'address': '0xE41d2489571d322189246DaFA5ebDe1F4699F498', 'decimals': 18},
    'SHIT': {'address': '0x4e4a47cAC6A28A62dcC20990ed2cdA9BC659469F', 'decimals': 18},
    'COW': {'address': '0xDEf1CA1fb7FBcDC777520aa7f396b4E015F497aB', 'decimals': 18},
    'AMP': {'address': '0xfF20817765cB7f73d4bde2e66e067E58D11095C2', 'decimals': 18},
    'COOL': {'address': '0x114f1388fAB456c4bA31B1850b244Eedcd024136', 'decimals': 18},
    'ZUSD': {'address': '0xc56c2b7e71B54d38Aab6d52E94a04Cbfa8F604fA', 'decimals': 6},
    'UWU': {'address': '0x55C08ca52497e2f1534B59E2917BF524D4765257', 'decimals': 18},
    'NVG': {'address': '0x08F40811C7d6C013744166f3d4CB1A9A92D3D54E', 'decimals': 18},
    'CMT': {'address': '0xE910c2a090516Fb7a7Be07f96a464785f2D5Dc18', 'decimals': 18},
    'GRAIN': {'address': '0xF88Baf18FAB7e330fa0C4F83949E23F52FECECce', 'decimals': 18},
    'CRETH2': {'address': '0x49D72e3973900A195A155a46441F0C08179FdB64', 'decimals': 18},
    'WOOFY': {'address': '0xD0660cD418a64a1d44E9214ad8e459324D8157f1', 'decimals': 12},
    'MAI': {'address': '0x75387e1287Dd85482aB66102DA9f6577E027f609', 'decimals': 18},
    'TLW': {'address': '0x06f3CDabae564B0546529b4DD8FeF1bcD4235753', 'decimals': 8},
    'FREL': {'address': '0x29CeDDCF0Da3c1D8068a7DFbD0FB06c2E438FF70', 'decimals': 18},
    'NOTE': {'address': '0xCFEAead4947f0705A14ec42aC3D44129E1Ef3eD5', 'decimals': 8},
    'X8X': {'address': '0x910Dfc18D6EA3D6a7124A6F8B5458F281060fa4c', 'decimals': 18},
    'UNM': {'address': '0x6570fFe19dA7e2b425329B157d9109b87f18304b', 'decimals': 18},
    'CAVA': {'address': '0x456D8f0D25A4e787eE60c401F8B963a465148f70', 'decimals': 9},
    'ONSTON': {'address': '0x47b9F01B16E9C9cb99191DCA68c9cc5bF6403957', 'decimals': 18},
    'AFY': {'address': '0xa41161AF8D4Ee421ba5fED4328F2B12034796A21', 'decimals': 18},
    'LCE': {'address': '0x2c644C3bBeA053ed95a6bC04A94C9CE928Ff9881', 'decimals': 8},
    'ACR': {'address': '0x76306F029f8F99EFFE509534037Ba7030999E3CF', 'decimals': 18},
    'AAVE': {'address': '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9', 'decimals': 18},
    'AUTH': {'address': '0x0107C4aAa31940eF88760aCb1F32424CA8d8bD80', 'decimals': 8},
    'CTI': {'address': '0xcB8fb2438A805664cD8c3e640b85AC473DA5BE87', 'decimals': 18},
    'VOLT': {'address': '0xfFbF315f70E458e49229654DeA4cE192d26f9b25', 'decimals': 18},
    'AOA': {'address': '0x9ab165D795019b6d8B3e971DdA91071421305e5a', 'decimals': 18},
    'GLO': {'address': '0xC0e6737A29DE7a00e2f6011924eB257106CB082f', 'decimals': 18},
    'ACAP': {'address': '0xd5A98E77d1fEB091344096301Ea336a5C07a6A41', 'decimals': 18},
    'HIFI': {'address': '0x4b9278b94a1112cAD404048903b8d343a810B07e', 'decimals': 18},
    'GHD': {'address': '0x3b544e6fcf6C8dCE9D8B45A4FdF21C9B02f9fDa9', 'decimals': 18},
    'STSR': {'address': '0xD13432fdD2ae4eeD84C3A0233D6c5265087C96c3', 'decimals': 18},
    'DXF': {'address': '0x15Eabb7500E44B7Fdb6e4051cA8DecA430cF9FB8', 'decimals': 18},
    'ADI': {'address': '0x8810C63470d38639954c6B41AaC545848C46484a', 'decimals': 18},
    'EL': {'address': '0x2781246fe707bB15CeE3e5ea354e2154a2877B16', 'decimals': 18},
    'HIFLUF': {'address': '0x79C9e0410b6615e7Ba9DD69614B0519325a2B047', 'decimals': 18},
    'OFI': {'address': '0x419E35E3515c2fDB652C898bF7A0B21FB20497dC', 'decimals': 9},
    'HELIOS': {'address': '0xC6d1f1D5a46de07E73091f1c8793293B203f01a1', 'decimals': 18},
    'TNGL': {'address': '0x5aF28EB9deC75e86CDFBD2e41a474b54B211C1C2', 'decimals': 9},
    'MYCE': {'address': '0xE9966C1184f8552Fcb16F65addba9Dd08fe8f4EA', 'decimals': 18},
    'RHLLOR': {'address': '0x90E2Fa98DfC518317600Dd3DC571DE8D071a7238', 'decimals': 9},
    'SHIB': {'address': '0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE', 'decimals': 18},
    'CPC': {'address': '0xfAE4Ee59CDd86e3Be9e8b90b53AA866327D7c090', 'decimals': 18},
    'MOCHI': {'address': '0xf134519cBE2042B06Ee7cE20Df51d09b55559896', 'decimals': 18},
    'EUL': {'address': '0xd9Fcd98c322942075A5C3860693e9f4f03AAE07b', 'decimals': 18},
    'DINGER': {'address': '0x9e5BD9D9fAd182ff0A93bA8085b664bcab00fA68', 'decimals': 9},
    'BLT': {'address': '0x107c4504cd79C5d2696Ea0030a8dD4e92601B82e', 'decimals': 18},
    'SHUEY': {'address': '0xcDcA1B81DbB543BAA92c97b701396Cd3bA315E5D', 'decimals': 18},
    'CMS': {'address': '0xF83301c5Cd1CCBB86f466A6B3c53316ED2f8465a', 'decimals': 6},
    'AXIAV3': {'address': '0x793786e2dd4Cc492ed366a94B88a3Ff9ba5E7546', 'decimals': 18},
    'UDAI': {'address': '0x4aD0b81f92B16624BBcF46FC0030cFBBf8d02376', 'decimals': 18},
    'SFRXETH': {'address': '0xac3E018457B222d93114458476f3E3416Abbe38F', 'decimals': 18},
    'YO': {'address': '0xeBF4CA5319F406602EEFf68da16261f1216011B5', 'decimals': 18},
    'TKX': {'address': '0x667102BD3413bFEaa3Dffb48fa8288819E480a88', 'decimals': 8},
    'AUSDC': {'address': '0x9bA00D6856a4eDF4665BcA2C2309936572473B7E', 'decimals': 6},
    'TELE': {'address': '0xF1e345Ea7c33fd6c05F5512a780Eb5839EE31674', 'decimals': 18},
    'ENTC': {'address': '0x3Ecab35B64345bfC472477A653e4A3abE70532D9', 'decimals': 18},
    'UNV': {'address': '0xf009f5531dE69067435e32c4b9D36077F4C4A673', 'decimals': 18},
    'CUBT': {'address': '0xBf9e72eEb5adB8B558334c8672950B7a379D4266', 'decimals': 18},
    'JDC': {'address': '0x2fe39f22EAC6d3c1C86DD9D143640EbB94609FCE', 'decimals': 18},
    'ASKO': {'address': '0xeEEE2a622330E6d2036691e983DEe87330588603', 'decimals': 18},
    'INFS': {'address': '0x193408cA0576B73156Ed42A2EA7D6fD3f6507162', 'decimals': 1},
    'SUDO': {'address': '0x3446Dd70B2D52A6Bf4a5a192D9b0A161295aB7F9', 'decimals': 18},
    'UFARM': {'address': '0x40986a85B4cFCDb054A6CBFB1210194fee51af88', 'decimals': 18},
    'PKEX': {'address': '0xE6f143a0e0A8f24F6294CE3432eA10FaD0206920', 'decimals': 18},
    'SHKOOBY': {'address': '0x29A5c1Db7321C5c9EaE57F9366eE8eef00cA11fb', 'decimals': 18},
    'SAIKO': {'address': '0x685Fb6960Ea7F7b24F02e17c8d893B8c33239d48', 'decimals': 9},
    'VLT': {'address': '0x6b785a0322126826d8226d77e173d75DAfb84d11', 'decimals': 18},
    'BSP': {'address': '0xE51b8Ab09008285a0380Dd2680cD9Dd5e13924D3', 'decimals': 18},
    'ULU': {'address': '0x035bfe6057E15Ea692c0DfdcaB3BB41a64Dd2aD4', 'decimals': 18},
    'KYL': {'address': '0x67B6D479c7bB412C54e03dCA8E1Bc6740ce6b99C', 'decimals': 18},
    'RNDR': {'address': '0x6De037ef9aD2725EB40118Bb1702EBb27e4Aeb24', 'decimals': 18},
    'CW': {'address': '0xd55236D48606c295adEbF129dAD04Fc74BFaA708', 'decimals': 18},
    'SMSCT': {'address': '0x668f7dfb8c8D716839fa5dBEa317D8723Ebe6110', 'decimals': 0},
    'GAZE': {'address': '0xD1E06952708771f71E6dd18F06EE418F6e8FC564', 'decimals': 18},
    'BRTR': {'address': '0xF0ACF8949e705E0ebb6CB42c2164B0B986454223', 'decimals': 8},
    'TERA': {'address': '0x009668A9691e456972c8ec4Cc84e99486308b84D', 'decimals': 18},
    'MOV': {'address': '0x40284109c3309A7C3439111bFD93BF5E0fBB706c', 'decimals': 18},
    'ZEON': {'address': '0xE5B826Ca2Ca02F09c1725e9bd98d9a8874C30532', 'decimals': 18},
    'POLAR': {'address': '0x58fcAa970339a9b1f8c0a5b4F3Fcd7af2bA3075E', 'decimals': 18},
    'LYM': {'address': '0xc690F7C7FcfFA6a82b79faB7508c466FEfdfc8c5', 'decimals': 18},
    'MORE': {'address': '0xbEEf3bB9dA340EbdF0f5bae2E85368140d7D85D0', 'decimals': 18},
    'SIPHER': {'address': '0x9F52c8ecbEe10e00D9faaAc5Ee9Ba0fF6550F511', 'decimals': 18},
    'SUKI': {'address': '0xD212046F89680aC9F106B9c63f314cc9808e18d5', 'decimals': 18},
    'HIFIDENZA': {'address': '0xb3e6Ee8d2c586FA03Ab70Aef96B8aE6D12D64eC7', 'decimals': 18},
    'HIENS3': {'address': '0xa88842Ae47dbe87116cF7001DDdD1b246fcd8262', 'decimals': 18},
    'MAXI': {'address': '0x0d86EB9f43C57f6FF3BC9E23D8F9d82503f0e84b', 'decimals': 8},
    'GMI': {'address': '0x47110d43175f7f2C2425E7d15792acC5817EB44f', 'decimals': 18},
    'UWBTC': {'address': '0x3aF5Ba94C29a8407785f5f6d90eF5d69a8EB2436', 'decimals': 8},
    'TRVL': {'address': '0xd47bDF574B4F76210ed503e0EFe81B58Aa061F3d', 'decimals': 18},
    'FOXT': {'address': '0xFbe878CED08132bd8396988671b450793C44bC12', 'decimals': 18},
    'AI': {'address': '0xc91B523A59ACC63A64f61fc7bBfB4bfc82DD25f2', 'decimals': 18},
    'POWER': {'address': '0x0263994554Ec08cc60473dbf8ceE60f9EedbF093', 'decimals': 18},
    'CRV': {'address': '0xD533a949740bb3306d119CC777fa900bA034cd52', 'decimals': 18},
    'AAMMUNISNXWETH': {'address': '0x38E491A71291CD43E8DE63b7253E482622184894', 'decimals': 18},
    'GMM': {'address': '0x7aF89c8A06719271A96e62E290Ea9Ed192E73FC1', 'decimals': 18},
    'FRETH': {'address': '0xB4Bd4628e6EFb0Cb521D9ec35050C75840320374', 'decimals': 18},
    'WPE': {'address': '0xd075e95423C5c4BA1E122CaE0f4CdFA19b82881b', 'decimals': 18},
    'AIPRO': {'address': '0x913c9509b94D32Ee38B0d98431C1FF109d24ff16', 'decimals': 18},
    'KTT': {'address': '0x14cC8DfaF2258E1B8B2869300dBa1B734dc0fE43', 'decimals': 18},
    'MUSD': {'address': '0xe2f2a5C287993345a840Db3B0845fbC70f5935a5', 'decimals': 18},
    'DKYC': {'address': '0x38d1B0D157529Bd5D936719A8a5F8379aFB24fAA', 'decimals': 18},
    'OPP': {'address': '0xf51ccB15d3F2D304Ae6C409303e8b3a3e397A80b', 'decimals': 18},
    'SMS': {'address': '0xe5867608b51A2c9C78B9587355cC093140A49B0A', 'decimals': 3},
    'EC': {'address': '0x12fD19DAC0Fab61bEd5e0F09091B470C452D4d61', 'decimals': 18},
    'HGOLD': {'address': '0x0c93B616933b0CD03B201b29Cd8a22681DD9E0d9', 'decimals': 8},
    'BB-A-USDC': {'address': '0x82698aeCc9E28e9Bb27608Bd52cF57f704BD1B83', 'decimals': 18},
    'MDS': {'address': '0x66186008C1050627F979d464eABb258860563dbE', 'decimals': 18},
    'MGH': {'address': '0x8765b1A0eb57ca49bE7EACD35b24A574D0203656', 'decimals': 18},
    'MFI': {'address': '0xAa4e3edb11AFa93c41db59842b29de64b72E355B', 'decimals': 18},
    'BET': {'address': '0x8610632B514761597d91ECF20F485cd8163a3033', 'decimals': 18},
    'TRXC': {'address': '0xaD5Fe5B0B8eC8fF4565204990E4405B2Da117d8e', 'decimals': 0},
    'YFFI': {'address': '0xCee1d3c3A02267e37E6B373060F79d5d7b9e1669', 'decimals': 18},
    'COFI': {'address': '0x1a23a6BfBAdB59fa563008c0fB7cf96dfCF34Ea1', 'decimals': 18},
    'WAXE': {'address': '0x7a2Bc711E19ba6aff6cE8246C546E8c4B4944DFD', 'decimals': 8},
    'VUSD': {'address': '0x677ddbd918637E5F2c79e164D402454dE7dA8619', 'decimals': 18},
    'MLC': {'address': '0x14449dE7937fE1C1207006E92f89dEe17BbE418A', 'decimals': 18},
    'MVRC': {'address': '0xa291B47EFE802de81Ac425c67D2B8d2925FA31A4', 'decimals': 18},
    'NCR': {'address': '0xDB5C3C46E28B53a39C255AA39A411dD64e5fed9c', 'decimals': 18},
    'VADER': {'address': '0x2602278EE1882889B946eb11DC0E810075650983', 'decimals': 18},
    'LGOLD': {'address': '0x27778E14Ce36d3B85e1efFeB43816a17bBB7088A', 'decimals': 18},
    'AAMMBPTBALWETH': {'address': '0xd109b2A304587569c84308c55465cd9fF0317bFB', 'decimals': 18},
    'VACHI': {'address': '0x337AF08bb6980Ecb68389C5ed8876D08643aBF8a', 'decimals': 18},
    'HIMFERS': {'address': '0xB755D5bC7de83232b9df1886bD5CdB38895933B0', 'decimals': 18},
    'SASHIMI': {'address': '0xC28E27870558cF22ADD83540d2126da2e4b464c2', 'decimals': 18},
    'TECH': {'address': '0x6286A9e6f7e745A6D884561D88F94542d6715698', 'decimals': 18},
    'FWB': {'address': '0x35bD01FC9d6D5D81CA9E055Db88Dc49aa2c699A8', 'decimals': 18},
    'ORION': {'address': '0x727f064A78DC734D33eEc18d5370aef32Ffd46e4', 'decimals': 18},
    'LY': {'address': '0x8686525d6627A25C68De82c228448f43c97999F2', 'decimals': 9},
    'AWBTC': {'address': '0xFC4B8ED459e00e5400be803A9BB3954234FD50e3', 'decimals': 8},
    'DONUT': {'address': '0xC0F9bD5Fa5698B6505F643900FFA515Ea5dF54A9', 'decimals': 18},
    'MOAR': {'address': '0x187Eff9690E1F1A61d578C7c492296eaAB82701a', 'decimals': 18},
    'ENS': {'address': '0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72', 'decimals': 18},
    'ABUSD': {'address': '0xA361718326c15715591c299427c62086F69923D9', 'decimals': 18},
    'CAST': {'address': '0x3FAb0bBAa03BCEAF7C49E2b12877dB0142BE65FC', 'decimals': 8},
    'PPAY': {'address': '0x054D64b73d3D8A21Af3D764eFd76bCaA774f3Bb2', 'decimals': 18},
    'CARE': {'address': '0x329cf160F30D21006bCD24b67EAde561E54CDE4c', 'decimals': 18},
    'CARDS': {'address': '0x3d6F0DEa3AC3C607B3998e6Ce14b6350721752d9', 'decimals': 18},
    'MSOT': {'address': '0xe5eF42D0E5E4Aa6B36c613d00Db8dad303d505F3', 'decimals': 18},
    'XCUR': {'address': '0xE1c7E30C42C24582888C758984f6e382096786bd', 'decimals': 8},
    'LOGT': {'address': '0xba58444C8050Ed9385b7417533A73644036d21Eb', 'decimals': 18},
    'EURE': {'address': '0x3231Cb76718CDeF2155FC47b5286d82e6eDA273f', 'decimals': 18},
    'SKY': {'address': '0x090FC4D5436D98E13473c4d6109De6A18BCfB0d4', 'decimals': 18},
    'RGP': {'address': '0x4AF5ff1A60a6eF6C7c8f9C4E304cD9051fCa3Ec0', 'decimals': 18},
    'GFI': {'address': '0xdab396cCF3d84Cf2D07C4454e10C8A6F5b008D2b', 'decimals': 18},
    'ACYC': {'address': '0xB56A1f3310578f23120182Fb2e58c087EFE6e147', 'decimals': 18},
    'INV': {'address': '0x41D5D79431A913C4aE7d69a668ecdfE5fF9DFB68', 'decimals': 18},
    'FACTR': {'address': '0xe0bCEEF36F3a6eFDd5EEBFACD591423f8549B9D5', 'decimals': 18},
    'LFT': {'address': '0xB620Be8a1949AA9532e6a3510132864EF9Bc3F82', 'decimals': 18},
    'WOJAK': {'address': '0x5026F006B85729a8b14553FAE6af249aD16c9aaB', 'decimals': 18},
    'LKR': {'address': '0x80CE3027a70e0A928d9268994e9B85d03Bd4CDcf', 'decimals': 18},
    'BAND': {'address': '0xBA11D00c5f74255f56a5E366F4F77f5A186d7f55', 'decimals': 18},
    'RBXS': {'address': '0xA9639160481b625ba43677bE753E0A70bf58c647', 'decimals': 18},
    'ASTO': {'address': '0x823556202e86763853b40e9cDE725f412e294689', 'decimals': 18},
    'FUEL': {'address': '0xEA38eAa3C86c8F9B751533Ba2E562deb9acDED40', 'decimals': 18},
    'TKB': {'address': '0x16594930D16f3970e1a4317c6016555cb2e7b7Fc', 'decimals': 18},
    'OBT': {'address': '0x07F9702ce093db82dfdC92c2C6E578d6EA8D5E22', 'decimals': 18},
    'CBANK': {'address': '0xA5E412ba6FcA1e07b15dEFcaA4236Ff7B5A7f086', 'decimals': 18},
    'MORK': {'address': '0xf552b656022c218C26dAd43ad88881Fc04116F76', 'decimals': 4},
    'MNDCC': {'address': '0xCDb9D30a3BA48CdfCB0EcbE19317E6cf783672f1', 'decimals': 18},
    'LEXE': {'address': '0x53011E93F21EC7A74cdfBB7E6548F1ABCE306833', 'decimals': 18},
    'REL': {'address': '0xb6c4267C4877BB0D6b1685Cfd85b0FBe82F105ec', 'decimals': 18},
    'CHR': {'address': '0x8A2279d4A90B6fe1C4B30fa660cC9f926797bAA2', 'decimals': 6},
    'WLEO': {'address': '0x73A9fb46e228628f8f9BB9004eCa4f4F529D3998', 'decimals': 3},
    'MUSK': {'address': '0x719e7f0daDFdeA25B78595da944F44d15d7E6795', 'decimals': 18},
    'AMLT': {'address': '0xCA0e7269600d353F70b14Ad118A49575455C0f2f', 'decimals': 18},
    'MMI': {'address': '0xB396C717105f2F9Ba81007c7FB774fb06d0fb937', 'decimals': 18},
    'RELAY': {'address': '0x5D843Fa9495d23dE997C394296ac7B4D721E841c', 'decimals': 18},
    'FORTH': {'address': '0x77FbA179C79De5B7653F68b5039Af940AdA60ce0', 'decimals': 18},
    '1ECO': {'address': '0x1Af2eAEAf2B1d9dDA800861268e6bbB3995a6C3b', 'decimals': 18},
    'BRIDGE': {'address': '0x92868A5255C628dA08F550a858A802f5351C5223', 'decimals': 18},
    'AAMMUNICRVWETH': {'address': '0x0ea20e7fFB006d4Cfe84df2F72d8c7bD89247DB0', 'decimals': 18},
    'TOON': {'address': '0xaeE433ADeBe0FBB88dAa47eF0C1A513CaA52EF02', 'decimals': 18},
    'DES': {'address': '0x634239cfA331Df0291653139d1a6083B9cf705e3', 'decimals': 18},
    'NOBI': {'address': '0xC5253C32BB72921b77ae18CF329dAd5e6C3de5A4', 'decimals': 18},
    'SALE': {'address': '0xF063fE1aB7a291c5d06a86e14730b00BF24cB589', 'decimals': 18},
    'AGV': {'address': '0xf4F618Eff5eF36Cde2FCa4FBD86554c62Fb1382B', 'decimals': 18},
    'MTA': {'address': '0xa3BeD4E1c75D00fa6f4E5E6922DB7261B5E9AcD2', 'decimals': 18},
    'BSY': {'address': '0x7A647637E75d34A7798Ba06BD1aDF5c084bb5dd5', 'decimals': 18},
    'AAA': {'address': '0x8C6bf16C273636523C29Db7DB04396143770F6A0', 'decimals': 18},
    'HACHI': {'address': '0x967B0c95295ead8fAef70d26A7846AeCD349AAFf', 'decimals': 18},
    'OMI': {'address': '0xeD35af169aF46a02eE13b9d79Eb57d6D68C1749e', 'decimals': 18},
    'KOYO': {'address': '0x9dAeF41E08Bc88183572b03796C96a7ace1Ec9d4', 'decimals': 18},
    'KONO': {'address': '0x850aAB69f0e0171A9a49dB8BE3E71351c8247Df4', 'decimals': 18},
    'VIRTUE': {'address': '0x9416bA76e88D873050A06e5956A3EBF10386b863', 'decimals': 18},
    'VIZ': {'address': '0x2C10c0dE3362FF21F8ED6bC7F4AC5e391153fD2c', 'decimals': 18},
    'DFGL': {'address': '0xE3a64A3c4216B83255b53Ec7eA078B13f21a7daD', 'decimals': 18},
    'NOKU': {'address': '0x8167D3B1024cB51A2DD1B4d889ddf7023420796a', 'decimals': 18},
    'XUC': {'address': '0xc324a2f6b05880503444451B8b27e6f9e63287Cb', 'decimals': 18},
    'MM': {'address': '0x6B4c7A5e3f0B99FCD83e9c089BDDD6c7FCe5c611', 'decimals': 18},
    'STATS': {'address': '0x39142c18B6db2a8A41B7018f49E1478837560caD', 'decimals': 9},
    'KUSUNOKI': {'address': '0x36919a60a2B67B6d2329863093D180d23d5A0308', 'decimals': 18},
    'PLR': {'address': '0xe3818504c1B32bF1557b16C238B2E01Fd3149C17', 'decimals': 18},
    'RUSHAI': {'address': '0x69d47846bb7B884062FaBc08A4A7dEADCd98c45B', 'decimals': 18},
    'LDN': {'address': '0xb29663Aa4E2e81e425294193616c1B102B70a158', 'decimals': 18},
    'RADAR': {'address': '0x44709a920fCcF795fbC57BAA433cc3dd53C44DbE', 'decimals': 18},
    'PEPE': {'address': '0x6982508145454Ce325dDbE47a25d4ec3d2311933', 'decimals': 18},
    'TKING': {'address': '0x24E89bDf2f65326b94E36978A7EDeAc63623DAFA', 'decimals': 18},
    'WBX': {'address': '0xbB97e381F1d1e94ffa2A5844F6875e6146981009', 'decimals': 18},
    'PDX': {'address': '0x6a8fee0E33Cb65a7E8D21BadCA62E87639ef74B3', 'decimals': 18},
    'FOR': {'address': '0x1FCdcE58959f536621d76f5b7FfB955baa5A672F', 'decimals': 18},
    'WAVES': {'address': '0x1cF4592ebfFd730c7dc92c1bdFFDfc3B9EfCf29a', 'decimals': 18},
    'MUSK': {'address': '0x6069c9223e8a5DA1ec49ac5525d4BB757Af72Cd8', 'decimals': 18},
    'SUKO': {'address': '0x46be62D1Fc217063c1bF1f35D788690ceC7a60cA', 'decimals': 18},
    'SNN': {'address': '0xF5717f5DF41eA67Ef67DFD3c1d02F9940bcF5d08', 'decimals': 3},
    'RVST': {'address': '0x120a3879da835A5aF037bB2d1456beBd6B54d4bA', 'decimals': 18},
    'PRV': {'address': '0xB64fde8f199F073F41c132B9eC7aD5b61De0B1B7', 'decimals': 9},
    'UNI': {'address': '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984', 'decimals': 18},
    'DTH': {'address': '0x0cf75471B0FbF76A315A75eBDa6600ab92Fdc924', 'decimals': 18},
    'SPLC': {'address': '0xbe7A4f88c6B15a722aa960B487BAdF9880184Ec5', 'decimals': 18},
    'PSL': {'address': '0xC775C0C30840Cb9F51e21061B054ebf1A00aCC29', 'decimals': 5},
    'BT': {'address': '0x997507cc49FBf0CD6ce5e1EE543218556fAFdEBc', 'decimals': 18},
    'FX1': {'address': '0xED0e2041BFb5a426e5ED426A73765624E08BbB75', 'decimals': 18},
    'SNC': {'address': '0xF4134146AF2d511Dd5EA8cDB1C4AC88C57D60404', 'decimals': 18},
    'PERP': {'address': '0xbC396689893D065F41bc2C6EcbeE5e0085233447', 'decimals': 18},
    'OCCT': {'address': '0x95A1796437BAd6502d1c1cce165Cd76E522409a9', 'decimals': 18},
    'CALI': {'address': '0xB8fA12f8409DA31A4fc43D15c4c78C33d8213B9B', 'decimals': 18},
    'LEO': {'address': '0x2AF5D2aD76741191D15Dfe7bF6aC92d4Bd912Ca3', 'decimals': 18},
    'FSC': {'address': '0x5f4361971667C8198706dEc8C59cF7607e405C6e', 'decimals': 8},
    'ATC': {'address': '0x0eb3032bcAc2BE1fa95E296442F225edb80fc3CD', 'decimals': 18},
    'BNB': {'address': '0x418D75f65a02b3D53B2418FB8E1fe493759c7605', 'decimals': 18},
    'LUSD3CRV': {'address': '0xEd279fDD11cA84bEef15AF5D39BB4d4bEE23F0cA', 'decimals': 18},
    'AAMMUNIRENWETH': {'address': '0xA9e201A4e269d6cd5E9F0FcbcB78520cf815878B', 'decimals': 18},
    'SPROUT': {'address': '0x9b8e9d523D1D6bC8EB209301c82C7D64D10b219E', 'decimals': 18},
    'WAIF': {'address': '0xB2279B6769CFBa691416F00609b16244c0cF4b20', 'decimals': 18},
    'NEEO': {'address': '0xd8446236FA95b9b5f9fd0f8E7Df1a944823c683d', 'decimals': 18},
    'PLBT': {'address': '0x0AfFa06e7Fbe5bC9a764C979aA66E8256A631f02', 'decimals': 6},
    'BITCCA': {'address': '0xe4D75e9b493458D032A5C3CC1eE9B0712C1EcE06', 'decimals': 2},
    'LNKO': {'address': '0x11afE7Fa792589dd1236257f99ba09f510460Ad9', 'decimals': 8},
    'KFC': {'address': '0xE63684BcF2987892CEfB4caA79BD21b34e98A291', 'decimals': 18},
    'MINI': {'address': '0x4D953cf077c0C95Ba090226E59A18FcF97db44EC', 'decimals': 18},
    'SLICE': {'address': '0x0AeE8703D34DD9aE107386d3eFF22AE75Dd616D1', 'decimals': 18},
    'WACME': {'address': '0xDF4Ef6EE483953fE3B84ABd08C6A060445c01170', 'decimals': 8},
    'IPL': {'address': '0x64CdF819d3E75Ac8eC217B3496d7cE167Be42e80', 'decimals': 18},
    'SPO': {'address': '0xcbE771323587EA16dACB6016e269D7F08A7ACC4E', 'decimals': 18},
    'GUP': {'address': '0xf7B098298f7C69Fc14610bf71d5e02c60792894C', 'decimals': 3},
    'FCL': {'address': '0xF4d861575ecC9493420A3f5a14F85B13f0b50EB3', 'decimals': 18},
    'HART': {'address': '0x52928C95C4C7e934E0EfcfAB08853A0E4558861d', 'decimals': 18},
    'EDEN': {'address': '0x1559FA1b8F28238FD5D76D9f434ad86FD20D1559', 'decimals': 18},
    'LET': {'address': '0xFA3118B34522580c35Ae27F6cf52da1dBb756288', 'decimals': 6},
    'XCN': {'address': '0xA2cd3D43c775978A96BdBf12d733D5A1ED94fb18', 'decimals': 18},
    'GEMX': {'address': '0x3B5e2DF103a037509dcC8430E5462B9CD8F75f49', 'decimals': 9},
    'QTO': {'address': '0xEA01906843Ea8D910658a2c485ffCe7C104AB2b6', 'decimals': 18},
    'AXL': {'address': '0x467719aD09025FcC6cF6F8311755809d45a5E5f3', 'decimals': 6},
    'AWETH': {'address': '0x030bA81f1c18d280636F32af80b9AAd02Cf0854e', 'decimals': 18},
    'SNX': {'address': '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F', 'decimals': 18},
    'NHT': {'address': '0x84342e932797FC62814189f01F0Fb05F52519708', 'decimals': 18},
    'TENSHI': {'address': '0x52662717e448be36Cb54588499D5A8328BD95292', 'decimals': 18},
    'HSC': {'address': '0x2bBA3CF6DE6058cc1B4457Ce00deb359E2703d7F', 'decimals': 18},
    'BST': {'address': '0x0B4680e848f842b86E4Df38D9FA0d9f9419F37EC', 'decimals': 18},
    'TRL': {'address': '0x58f9102bF53Cf186682Bd9a281d3Cd3c616eEc41', 'decimals': 18},
    'AVG': {'address': '0xa41F142b6eb2b164f8164CAE0716892Ce02f311f', 'decimals': 18},
    'QUIDD': {'address': '0xDA9FDAb21bC4A5811134A6E0Ba6CA06624e67c07', 'decimals': 18},
    'TON': {'address': '0x6a6c2adA3Ce053561C2FbC3eE211F23d9b8C520a', 'decimals': 18},
    'WMLX': {'address': '0x77D0cb0ab54F9E74b9405a5B3f60DA06A78F1AAd', 'decimals': 0},
    'CQT': {'address': '0xD417144312DbF50465b1C641d016962017Ef6240', 'decimals': 18},
    'BRIGHT': {'address': '0xBEaB712832112bd7664226db7CD025B153D3af55', 'decimals': 18},
    'ELEC': {'address': '0xD49ff13661451313cA1553fd6954BD1d9b6E02b9', 'decimals': 18},
    'LEAD': {'address': '0x1dD80016e3d4ae146Ee2EBB484e8edD92dacC4ce', 'decimals': 18},
    'UTT': {'address': '0x16f812Be7FfF02cAF662B85d5d58a5da6572D4Df', 'decimals': 8},
    'WAGMIGAMES': {'address': '0x3B604747ad1720C01ded0455728b62c0d2F100F0', 'decimals': 18},
    'GN': {'address': '0xc5019E129b75D380d3d837B8e609dEc6c8f5d044', 'decimals': 9},
    'LOOT': {'address': '0x7b3D36Eb606f873A75A6aB68f8c999848B04F935', 'decimals': 18},
    'ENV': {'address': '0xF1d1A5306daaE314AF6c5d027a492B313e07e1A0', 'decimals': 18},
    'GDO': {'address': '0x16F78145AD0B9Af58747e9A97EBd99175378bd3D', 'decimals': 18},
    'DMX': {'address': '0xf058501585023d040eA9493134ED72C083553eED', 'decimals': 18},
    'GSC': {'address': '0x228ba514309FFDF03A81a205a6D040E429d6E80C', 'decimals': 18},
    'MVEDA': {'address': '0xCBe7142F5c16755D8683BA329EFA1ABF7b54482d', 'decimals': 8},
    'USD1': {'address': '0xf6c0aA7eBFE9992200C67E5388E4F42da49E1783', 'decimals': 2},
    'SILV': {'address': '0x628aB8b061FEA2AF1239B68eFA5e46135D186666', 'decimals': 8},
    'CTPL': {'address': '0xF7e40AF830a2EcbFe72C5210C1Ca272e3Aa7cA1B', 'decimals': 18},
    'AUTUMN': {'address': '0x4c3bAe16c79c30eEB1004Fb03C878d89695e3a99', 'decimals': 18},
    'FUND': {'address': '0xe9B076B476D8865cDF79D1Cf7DF420EE397a7f75', 'decimals': 9},
    'TSHP': {'address': '0x525794473F7ab5715C81d06d10f52d11cC052804', 'decimals': 18},
    'COMBO': {'address': '0xfFffFffF2ba8F66D4e51811C5190992176930278', 'decimals': 18},
    'DVT': {'address': '0x29FA1fee0F4F0Ab0e36eF7Ab8d7a35439eC6be75', 'decimals': 18},
    'NODE': {'address': '0xDa007777D86AC6d989cC9f79A73261b3fC5e0DA0', 'decimals': 18},
    'AAMMUNIDAIUSDC': {'address': '0xE340B25fE32B1011616bb8EC495A4d503e322177', 'decimals': 18},
    'NCASH': {'address': '0x809826cceAb68c387726af962713b64Cb5Cb3CCA', 'decimals': 18},
    'HLP': {'address': '0x308564DC5217c39386F5eaE96545159e1D396661', 'decimals': 18},
    'PAN': {'address': '0xD56daC73A4d6766464b38ec6D91eB45Ce7457c44', 'decimals': 18},
    'HNB': {'address': '0x6E0615a03eD9527a6013FcD5B556E36EF4DaB1FF', 'decimals': 18},
    'FFRAX': {'address': '0x1C9A2d6b33B4826757273D47ebEe0e2DddcD978B', 'decimals': 8},
    'BUSD': {'address': '0x4Fabb145d64652a948d72533023f6E7A623C7C53', 'decimals': 18},
    'MATSUSWAP': {'address': '0x2F0F622a93c85C43Fd47f4Ab119360f5846026C4', 'decimals': 9},
    'MYB': {'address': '0x5d60d8d7eF6d37E16EBABc324de3bE57f135e0BC', 'decimals': 18},
    'PLAY': {'address': '0xE477292f1B3268687A29376116B0ED27A9c76170', 'decimals': 18},
    'CPI': {'address': '0x2b67D1a87A8D8B280A23e97bC55095215eE0ec53', 'decimals': 18},
    'ONES': {'address': '0x0B342C51d1592C41068d5D4b4DA4A68C0a04d5A4', 'decimals': 18},
    'TZKI': {'address': '0xF527d24391C767B86b8e91385e1cE9C54D230A2B', 'decimals': 9},
    'GIGA': {'address': '0x83249c6794BCa5a77Eb8c0AF9F1A86E055459cea', 'decimals': 9},
    'ATD': {'address': '0x8052327F1BAF94A9DC8B26b9100f211eE3774f54', 'decimals': 18},
    'BTBS': {'address': '0x32E6C34Cd57087aBBD59B5A4AECC4cB495924356', 'decimals': 18},
    'HOOT': {'address': '0x12A7530D6f9e1a9B0351D78aB711f7C2c033873A', 'decimals': 18},
    'NEKO': {'address': '0x1E9D0bB190Ac34492aa11B80D28c1C86487a341F', 'decimals': 18},
    'WFLOW': {'address': '0x5c147e74D63B1D31AA3Fd78Eb229B65161983B2b', 'decimals': 18},
    'MADHAT': {'address': '0x425c5b7B55F9C981C71935418ED044E79D8080E2', 'decimals': 9},
    'RSC': {'address': '0xD101dCC414F310268c37eEb4cD376CcFA507F571', 'decimals': 18},
    'FRM': {'address': '0xE5CAeF4Af8780E59Df925470b050Fb23C43CA68C', 'decimals': 6},
    'TEL': {'address': '0x467Bccd9d29f223BcE8043b84E8C8B282827790F', 'decimals': 2},
    'FIT': {'address': '0x3c72fca8523686fd9e5740b0826FA4bB376E0241', 'decimals': 18},
    'IDLEDAIYIELD': {'address': '0x3fE7940616e5Bc47b0775a0dccf6237893353bB4', 'decimals': 18},
    'LML': {'address': '0x25B6325f5BB1c1E03cfbC3e53F470E1F1ca022E3', 'decimals': 18},
    'DINGO': {'address': '0x1f961BCEAEF8eDF6fb2797C0293FfBDe3E994614', 'decimals': 9},
    'WNXM': {'address': '0x0d438F3b5175Bebc262bF23753C1E53d03432bDE', 'decimals': 18},
    'MXT': {'address': '0x6251E725CD45Fb1AF99354035a414A2C0890B929', 'decimals': 18},
    'CVT': {'address': '0x24B47299E756aF0571F512232A3629E0DaBb52Ed', 'decimals': 18},
    'RUFF': {'address': '0xf278c1CA969095ffddDED020290cf8B5C424AcE2', 'decimals': 18},
    'AXPR': {'address': '0xdD0020B1D5Ba47A54E2EB16800D73Beb6546f91A', 'decimals': 18},
    'KAT': {'address': '0x14da230d6726C50F759Bc1838717F8CE6373509C', 'decimals': 18},
    'HOME': {'address': '0xb8919522331C59f5C16bDfAA6A121a6E03A91F62', 'decimals': 6},
    'DOTR': {'address': '0x26524e35E44e265eB181C7Ba6eC084983181aaE3', 'decimals': 9},
    'CTCN': {'address': '0xFD6C31bb6F05Fc8dB64F4b740Ab758605c271FD8', 'decimals': 18},
    'SHIDO': {'address': '0x173E552Bf97BBD50b455514ac52991Ef639ba703', 'decimals': 9},
    'SWAI': {'address': '0x3540DFCaD7CF102a2E44AA0e2132FAb1c89d7eAe', 'decimals': 18},
    'HOP': {'address': '0xc5102fE9359FD9a28f877a67E36B0F050d81a3CC', 'decimals': 18},
    'TRU': {'address': '0x4C19596f5aAfF459fA38B0f7eD92F11AE6543784', 'decimals': 8},
    'HIGAZERS': {'address': '0xc1AD0aA69454603a5dEe55Cf9BD9341e01328544', 'decimals': 18},
    'TIA': {'address': '0x824E35f7A75324f99300aFAC75ECF7354E17Ea26', 'decimals': 9},
    'DGMV': {'address': '0x8EEdEFe828A0f16C8fc80e46a87Bc0f1De2d960c', 'decimals': 18},
    'TMED': {'address': '0xd32641191578Ea9b208125dDD4EC5E7B84FcaB4C', 'decimals': 18},
    'PRIME': {'address': '0xb23d80f5FefcDDaa212212F028021B41DEd428CF', 'decimals': 18},
    'HEAL': {'address': '0xf6Fd82dEdBBe0ffadb5e1ecc2a283AB52B9ED2B0', 'decimals': 18},
    'POE': {'address': '0x0e0989b1f9B8A38983c2BA8053269Ca62Ec9B195', 'decimals': 8},
    'HEDG': {'address': '0xF1290473E210b2108A85237fbCd7b6eb42Cc654F', 'decimals': 18},
    'CTC': {'address': '0x00B7db6B4431e345eee5cc23D21E8dbC1d5cADA3', 'decimals': 18},
    'IZE': {'address': '0xf7e945FcE8F19302AaCc7E1418b0A0bdef89327B', 'decimals': 8},
    'ANOM': {'address': '0x5224F552f110eC78E6E0468138950AE5F3040942', 'decimals': 18},
    'TYPE': {'address': '0xeaf61FC150CD5c3BeA75744e830D916E60EA5A9F', 'decimals': 4},
    'LBT': {'address': '0x5AC83BfbFcebb3397A40fD259dBE7a4bE04237d3', 'decimals': 18},
    'XAI': {'address': '0xd7C9F0e536dC865Ae858b0C0453Fe76D13c3bEAc', 'decimals': 18},
    'MU': {'address': '0x05030203674173Fa6dF6F9f7E34D6E70E9a761D7', 'decimals': 18},
    'MTRA': {'address': '0xC62DEF1701309bb76e6B39B6ab8B5fAC910A3C87', 'decimals': 18},
    'STND': {'address': '0x9040e237C3bF18347bb00957Dc22167D0f2b999d', 'decimals': 18},
    'JEM': {'address': '0x21cf09BC065082478Dcc9ccB5fd215A978Dc8d86', 'decimals': 18},
    'WHITE': {'address': '0xFe4BEb9217cdDf2422d4bd65449b76d807b30fe1', 'decimals': 9},
    'SVS': {'address': '0x7ca62545a380e7d71F8F5CFa14B9211002075930', 'decimals': 18},
    'DPR': {'address': '0xf3AE5d769e153Ef72b4e3591aC004E89F48107a1', 'decimals': 18},
    'AGI': {'address': '0x5F18ea482ad5cc6BC65803817C99f477043DcE85', 'decimals': 18},
    'FLEXUSD': {'address': '0xa774FFB4AF6B0A91331C084E1aebAE6Ad535e6F3', 'decimals': 18},
    'ACOIN': {'address': '0xB62D46A21fef0fADb5b92fC5eF7077E71e65631c', 'decimals': 18},
    'ROG': {'address': '0x5d43b66da68706D39f6C97F7f1415615672b446b', 'decimals': 18},
    'BED': {'address': '0x2aF1dF3AB0ab157e1E2Ad8F88A7D04fbea0c7dc6', 'decimals': 18},
    'CTASK': {'address': '0x196c81385Bc536467433014042788Eb707703934', 'decimals': 18},
    'MTO': {'address': '0xE66b3AA360bB78468c00Bebe163630269DB3324F', 'decimals': 18},
    'HMT': {'address': '0xd1ba9BAC957322D6e8c07a160a3A8dA11A0d2867', 'decimals': 18},
    'OKINU': {'address': '0x7D1356BfcAA08AF4Ab0D643AF1B6b4cd75Ccc2d0', 'decimals': 18},
    'WXTZ': {'address': '0x2A69655C22eDa32ff48d315bB26ED45f150700b4', 'decimals': 6},
    'PAT': {'address': '0xF3b3Cad094B89392fcE5faFD40bC03b80F2Bc624', 'decimals': 18},
    'PP': {'address': '0xeCe8531550Fc599ccB2c7734494FBc358d0a9483', 'decimals': 18},
    'TRV': {'address': '0x72955eCFf76E48F2C8AbCCe11d54e5734D6f3657', 'decimals': 18},
    'YOURWALLET': {'address': '0xDa70FBaC9e1C9Ac3475c24A3A847f85f1388C429', 'decimals': 18},
    'ALBT': {'address': '0x00a8b738E453fFd858a7edf03bcCfe20412f0Eb0', 'decimals': 18},
    'QWLA': {'address': '0x1D96Fd43EE07Aa79f8Fd003cbdF404fb5ce41AD2', 'decimals': 18},
    'PUNK': {'address': '0x269616D549D7e8Eaa82DFb17028d0B212D11232A', 'decimals': 18},
    'PROB': {'address': '0xfB559CE67Ff522ec0b9Ba7f5dC9dc7EF6c139803', 'decimals': 18},
    'BLUESPARROW': {'address': '0x4D67EDef87a5fF910954899f4e5a0AaF107afd42', 'decimals': 9},
    'ANG': {'address': '0x130914E1B240a7F4c5D460B7d3a2Fd3846b576fa', 'decimals': 18},
    'AIRX': {'address': '0x8cb1d155a5a1d5d667611b7710920fD9D1CD727F', 'decimals': 8},
    'COTI': {'address': '0xDDB3422497E61e13543BeA06989C0789117555c5', 'decimals': 18},
    'DENA': {'address': '0x15F0EEDF9Ce24fc4b6826E590A8292CE5524a1DA', 'decimals': 18},
    'TEXAN': {'address': '0xcFCFfE432A48dB53F59c301422d2EdD77B2A88d7', 'decimals': 18},
    'KCT': {'address': '0x4c601dc69AfFb0D4Fc8dE1Ac303705e432A4A27E', 'decimals': 18},
    'ZYX': {'address': '0xf974b5f9Ac9c6632FeE8b76C61B0242ce69C839D', 'decimals': 18},
    'UNIFI': {'address': '0x9E78b8274e1D6a76a0dBbf90418894DF27cBCEb5', 'decimals': 18},
    'LEMO': {'address': '0x60C24407d01782C2175D32fe7C8921ed732371D1', 'decimals': 18},
    'HINA': {'address': '0xbD0a4bf098261673d5E6e600Fd87ddcd756e6764', 'decimals': 9},
    'EPHIAT': {'address': '0xE9F721E7419423f11863e83DbD710b5D6127b5b0', 'decimals': 18},
    'JGBP': {'address': '0x7409856CAE628f5d578B285B45669b36E7005283', 'decimals': 18},
    'UCX': {'address': '0x3D3af44cf092a49280e316f09c8f20ecf97BC933', 'decimals': 18},
    'ATUSD': {'address': '0x4DA9b813057D04BAef4e5800E36083717b4a0341', 'decimals': 18},
    'STA': {'address': '0xa7DE087329BFcda5639247F96140f9DAbe3DeED1', 'decimals': 18},
    'UNIDX': {'address': '0x95b3497bBcCcc46a8F45F5Cf54b0878b39f8D96C', 'decimals': 18},
    'AXIS': {'address': '0xF0c5831EC3Da15f3696B4DAd8B21c7Ce2f007f28', 'decimals': 8},
    'SIDUS': {'address': '0x549020a9Cb845220D66d3E9c6D9F9eF61C981102', 'decimals': 18},
    'TYC': {'address': '0x3A82D3111aB5faF39d847D46023d9090261A658F', 'decimals': 18},
    'GEAR': {'address': '0xBa3335588D9403515223F109EdC4eB7269a9Ab5D', 'decimals': 18},
    'EDC': {'address': '0x9d5963ba32e877871dfF3e2E697283dc64066271', 'decimals': 18},
    'GIV': {'address': '0x900dB999074d9277c5DA2A43F252D74366230DA0', 'decimals': 18},
    'LOCG': {'address': '0x60Eb57d085C59932d5fAa6c6026268A4386927d0', 'decimals': 18},
    'XED': {'address': '0xee573a945B01B788B9287CE062A0CFC15bE9fd86', 'decimals': 18},
    'LON': {'address': '0x0000000000095413afC295d19EDeb1Ad7B71c952', 'decimals': 18},
    'STRP': {'address': '0x97872EAfd79940C7b24f7BCc1EADb1457347ADc9', 'decimals': 18},
    'RYOSHI': {'address': '0x777E2ae845272a2F540ebf6a3D03734A5a8f618e', 'decimals': 18},
    'BCP': {'address': '0x72e203a17adD19A3099137c9d7015fD3e2b7DBa9', 'decimals': 18},
    'MEF': {'address': '0x64B5a241b829Bca14Eb42d00097Ba8fD8135841E', 'decimals': 18},
    'BOMB': {'address': '0x1C95b093d6C236d3EF7c796fE33f9CC6b8606714', 'decimals': 0},
    'AAMMUNIDAIWETH': {'address': '0x9303EabC860a743aABcc3A1629014CaBcc3F8D36', 'decimals': 18},
    'DGP': {'address': '0x927159670C50042109d7C0f4aEd0Cee89452433E', 'decimals': 18},
    'KOK': {'address': '0x9B9647431632AF44be02ddd22477Ed94d14AacAa', 'decimals': 18},
    'ATNT': {'address': '0xAA2d8c9a8bd0F7945143bfD509bE3ff23dd78918', 'decimals': 3},
    'PET': {'address': '0xB870679a7Fa65b924026F496De7f27c1DD0e5C5F', 'decimals': 18},
    'K21': {'address': '0xB9d99C33eA2d86EC5eC6b8A4dD816EBBA64404AF', 'decimals': 18},
    'BRKL': {'address': '0x4674a4F24C5f63D53F22490Fb3A08eAAAD739ff8', 'decimals': 18},
    'MVRS': {'address': '0xBEDE1F59Fa4412556fEF69f1B9969F2003E3f8c1', 'decimals': 18},
    'TERA2': {'address': '0x33909C9CE97Ce509daB3A038B3eC7ac3d1Be3231', 'decimals': 4},
    'NEZUKO': {'address': '0x049715C70fDbdD2be4814f76a53Dc3D6F4367756', 'decimals': 18},
    'BAXS': {'address': '0xACf3D402e5E2C3eDD5b8129e966017D293F12a4C', 'decimals': 18},
    'KIRA': {'address': '0xf98AB0874b13a7Fdc39D7295DEdd49850A5D426B', 'decimals': 8},
    'NOA': {'address': '0x0FD3822072ad001aac1c90A09d9506f097F24458', 'decimals': 9},
    'USHI': {'address': '0x6Dca182ac5E3f99985bC4Ee0f726d6472aB1ec55', 'decimals': 18},
    'FTUSD': {'address': '0x0B319dB00d07C8fAdfaAEf13C910141a5dA0aa8F', 'decimals': 18},
    'DEGO': {'address': '0x3Da932456D082CBa208FEB0B096d49b202Bf89c8', 'decimals': 18},
    'NPAS': {'address': '0xEBe807bFDf8b2092E36e964C75463A8Aaa642139', 'decimals': 18},
    'BPRO': {'address': '0xbbBBBBB5AA847A2003fbC6b5C16DF0Bd1E725f61', 'decimals': 18},
    'CHS': {'address': '0xb41380174d0B06181513A5677b60200b93b5Efb4', 'decimals': 18},
    'CXO': {'address': '0xb6EE9668771a79be7967ee29a63D4184F8097143', 'decimals': 18},
    'METR': {'address': '0x405CE8B2eAeeA7D4Ba5Fc160848cB2A6569e03f0', 'decimals': 18},
    'PRE': {'address': '0xEC213F83defB583af3A000B1c0ada660b1902A0F', 'decimals': 18},
    'DOGES': {'address': '0xb4FBed161bEbcb37afB1Cb4a6F7cA18b977cCB25', 'decimals': 18},
    'NOWAI': {'address': '0x987441856BA4f463544FC68ccBF6a80F434a7956', 'decimals': 18},
    'WINGS': {'address': '0x667088b212ce3d06a1b553a7221E1fD19000d9aF', 'decimals': 18},
    'UCT': {'address': '0x6d1DC3928604b00180Bb570BdAe94b9698d33b79', 'decimals': 18},
    'CHFT': {'address': '0x58002A6B6E659A16dE9F02F529B10536E307b0d9', 'decimals': 18},
    'ASTETH': {'address': '0x1982b2F5814301d4e9a8b0201555376e62F82428', 'decimals': 18},
    'BST': {'address': '0x4E4bFfAA8df6F0dC3e5600bBACf7DA55f37134FC', 'decimals': 18},
    'DTX': {'address': '0x765f0C16D1Ddc279295c1a7C24B0883F62d33F75', 'decimals': 18},
    'TOKU': {'address': '0x9eEC1A4814323a7396c938bc86aeC46b97F1BD82', 'decimals': 18},
    'GALA': {'address': '0x15D4c048F83bd7e37d49eA4C83a07267Ec4203dA', 'decimals': 8},
    'CNTR': {'address': '0x03042482d64577A7bdb282260e2eA4c8a89C064B', 'decimals': 18},
    'HETH': {'address': '0x5bBe36152d3CD3eB7183A82470b39b29EedF068B', 'decimals': 18},
    'TGT': {'address': '0x108a850856Db3f85d0269a2693D896B394C80325', 'decimals': 18},
    'PNODE': {'address': '0xAF691508BA57d416f895e32a1616dA1024e882D2', 'decimals': 18},
    'CREDS': {'address': '0xc13F4F0F865bAc08F62654B57E38669EbC4747a3', 'decimals': 18},
    'USDGLO': {'address': '0x4F604735c1cF31399C6E711D5962b2B3E0225AD3', 'decimals': 18},
    'DRC': {'address': '0xd7f5CABdF696D7d1bf384D7688926A4bdB092c67', 'decimals': 18},
    'MAUSDT': {'address': '0xAFe7131a57E44f832cb2dE78ade38CaD644aaC2f', 'decimals': 18},
    'UMA': {'address': '0x04Fa0d235C4abf4BcF4787aF4CF447DE572eF828', 'decimals': 18},
    'WPCI': {'address': '0x3C2A309d9005433c1BC2C92EF1bE06489e5bf258', 'decimals': 8},
    'ROUTE': {'address': '0x16ECCfDbb4eE1A85A33f3A9B21175Cd7Ae753dB4', 'decimals': 18},
    'FEG': {'address': '0x389999216860AB8E0175387A0c90E5c52522C945', 'decimals': 9},
    'KTLYO': {'address': '0x24E3794605C84E580EEA4972738D633E8a7127c8', 'decimals': 18},
    'DINO': {'address': '0x49642110B712C1FD7261Bc074105E9E44676c68F', 'decimals': 18},
    'CORN': {'address': '0xa456b515303B2Ce344E9d2601f91270f8c2Fea5E', 'decimals': 18},
    'BITH': {'address': '0x8811e4Dd5EC5Eb8764b97CC814B1339089717adA', 'decimals': 8},
    'CV': {'address': '0x50bC2Ecc0bfDf5666640048038C1ABA7B7525683', 'decimals': 18},
    'EGT': {'address': '0x8e1b448EC7aDFc7Fa35FC2e885678bD323176E34', 'decimals': 18},
    'MOFI': {'address': '0xB2dbF14D0b47ED3Ba02bDb7C954e05A72deB7544', 'decimals': 18},
    'RUSH': {'address': '0x382A1667C9062F0621362F49076Ef6e4fE4C9eC7', 'decimals': 18},
    'ALETH': {'address': '0x0100546F2cD4C9D97f798fFC9755E47865FF7Ee6', 'decimals': 18},
    'PAPA': {'address': '0x6C4C193bff0a117F0C2B516802abbA961A1Eeb12', 'decimals': 18},
    'ABAL': {'address': '0x272F97b7a56a387aE942350bBC7Df5700f8a4576', 'decimals': 18},
    'ZIG': {'address': '0xb2617246d0c6c0087f18703d576831899ca94f01', 'decimals': 18},
    'GAI': {'address': '0x0d8ca4b20b115D4DA5c13DC45Dd582A5de3e78BF', 'decimals': 18},
    'MG': {'address': '0xB444Cb2bcB19180602328fA774738cB8DbD6677B', 'decimals': 9},
    'LOA': {'address': '0x7458fd786B2fe8CD801C0381f88b61C5071A006F', 'decimals': 18},
    'REN': {'address': '0x408e41876cCCDC0F92210600ef50372656052a38', 'decimals': 18},
    'ZCN': {'address': '0xb9EF770B6A5e12E45983C5D80545258aA38F3B78', 'decimals': 10},
    'MATRIX': {'address': '0xc8D3DCb63C38607Cb0c9d3F55E8eccE628A01C36', 'decimals': 18},
    'JASMY': {'address': '0x7420B4b9a0110cdC71fB720908340C03F9Bc03EC', 'decimals': 18},
    'LBXC': {'address': '0xfFE510a92434a0Df346C5E72a3494b043Cf249eB', 'decimals': 18},
    'ANT': {'address': '0xa117000000f279D81A1D3cc75430fAA017FA5A2e', 'decimals': 18},
    'IDLE': {'address': '0x875773784Af8135eA0ef43b5a374AaD105c5D39e', 'decimals': 18},
    'SYMM': {'address': '0x57dB3FfCa78dBbE0eFa0EC745D55f62aa0Cbd345', 'decimals': 18},
    'KALE': {'address': '0x508626d9a29D13EbA26F843A2BD7bF7b00a45be5', 'decimals': 18},
    'SWAPP': {'address': '0x8CB924583681cbFE487A62140a994A49F833c244', 'decimals': 18},
    'BABYFLOKI': {'address': '0xdF35988d795d90711e785B488Bb2127692e6f956', 'decimals': 9},
    'FTI': {'address': '0x943ED852DadB5C3938ECdC6883718df8142DE4C8', 'decimals': 18},
    'ORME': {'address': '0xc96DF921009B790dfFcA412375251ed1A2b75c60', 'decimals': 8},
    'SBLX': {'address': '0xf6af18F94d55B8888B557cb3da0Ec623BA245F80', 'decimals': 18},
    'VALOR': {'address': '0x297E4e5e59Ad72B1B0A2fd446929e76117be0E0a', 'decimals': 18},
    'GTR': {'address': '0x32DebF31D5ED90ca2e287D306fc23FaFAD9361c6', 'decimals': 18},
    'CVR': {'address': '0x3C03b4EC9477809072FF9CC9292C9B25d4A8e6c6', 'decimals': 18},
    'BOLLY': {'address': '0x6bd361e10c1afed0d95259e7C0115f3A60e4ea99', 'decimals': 18},
    'WSCRT': {'address': '0x2B89bF8ba858cd2FCee1faDa378D5cd6936968Be', 'decimals': 6},
    'CRE8': {'address': '0xc36b4311B21Fc0c2eAd46f1eA6ce97c9C4D98D3d', 'decimals': 18},
    'DOBE': {'address': '0xe7aB45162f5979F09B0bDa1CC7DfC97C270EA3d5', 'decimals': 18},
    'HUB': {'address': '0x8e9A29e7Ed21DB7c5B2E1cd75e676dA0236dfB45', 'decimals': 18},
    'TAIRYO': {'address': '0x14d4c7A788908fbbBD3c1a4Bac4AFf86fE1573EB', 'decimals': 9},
    'FR': {'address': '0xC626e0619aC79AFEa9281c8eB9b1a9f9D3Fab532', 'decimals': 18},
    'GOTG': {'address': '0xceEB07Dd26b36287B6d109f0b06d7e8202Ce8c1D', 'decimals': 18},
    'ABYSS': {'address': '0x0E8d6b471e332F140e7d9dbB99E5E3822F728DA6', 'decimals': 18},
    'ETHMAXY': {'address': '0x0FE20E0Fa9C78278702B05c333Cc000034bb69E2', 'decimals': 18},
    'UIP': {'address': '0x4290563C2D7c255B5EEC87f2D3bD10389f991d68', 'decimals': 18},
    'ITP': {'address': '0x2b1D36f5B61AdDAf7DA7ebbd11B35FD8cfb0DE31', 'decimals': 18},
    'PWRC': {'address': '0x6Aa40D02115090D40Dc33C7C5F3Cf05112fA4f83', 'decimals': 18},
    'WAIT': {'address': '0x2559813BBb508C4C79e9CCCE4703Bcb1f149EDd7', 'decimals': 9},
    'SWAT': {'address': '0xc0F1728d9513EFC316D0E93A0758c992f88b0809', 'decimals': 8},
    'PINA': {'address': '0x02814F435dD04e254Be7ae69F61FCa19881a780D', 'decimals': 18},
    'ESGC': {'address': '0xe857734840dc188B4283d5aF14aB8685467aB87D', 'decimals': 18},
    'FDAI': {'address': '0x244517Dc59943E8CdFbD424Bdb3262c5f04a1387', 'decimals': 18},
    'LSETH': {'address': '0x8c1BEd5b9a0928467c9B1341Da1D7BD5e10b6549', 'decimals': 18},
    'VXON': {'address': '0x048Aa6E69D5117BD665aF4128383a6982B8367b3', 'decimals': 4},
    'THALES': {'address': '0x8947da500Eb47F82df21143D0C01A29862a8C3c5', 'decimals': 18},
    'MUNI': {'address': '0xA7dE94227Eb6828497f4343B9923959E41Ae9CD1', 'decimals': 18},
    'KOL': {'address': '0x1CC30e2EAc975416060Ec6FE682041408420d414', 'decimals': 18},
    'OIN': {'address': '0x9aeB50f542050172359A0e1a25a9933Bc8c01259', 'decimals': 8},
    'BNV': {'address': '0x072987D5B36aD8d45552aEd98879a7101cCdd749', 'decimals': 18},
    'PBTC35A': {'address': '0xA8b12Cc90AbF65191532a12bb5394A714A46d358', 'decimals': 18},
    'TRISM': {'address': '0x56b4F8C39E07D4d5D91692acf9d0F6d4d3493763', 'decimals': 18},
    'FLOT': {'address': '0x049399a6B048D52971F7D122aE21A1532722285F', 'decimals': 18},
    'MASK20': {'address': '0xc2BdE1A2fA26890c8E6AcB10C91CC6D9c11F4a73', 'decimals': 18},
    'YAXIS': {'address': '0x0adA190c81b814548ddC2F6AdC4a689ce7C1FE73', 'decimals': 18},
    'WGR': {'address': '0xC237868a9c5729bdF3173dDDacaa336a0a5BB6e0', 'decimals': 8},
    'OCEAN': {'address': '0x967da4048cD07aB37855c090aAF366e4ce1b9F48', 'decimals': 18},
    'NAVI': {'address': '0xFc1C93a2507975E98b9d0E9260Ded61A00152BF1', 'decimals': 18},
    'XAV': {'address': '0x40370aEd88933021e20CF7C4D67E00417CdA2202', 'decimals': 18},
    'EFI': {'address': '0x656C00e1BcD96f256F224AD9112FF426Ef053733', 'decimals': 18},
    'X1': {'address': '0xf6c2A37AcC8Ba45874808837486110dC0Afc63C8', 'decimals': 2},
    'TCR': {'address': '0x9C4A4204B79dd291D6b6571C5BE8BbcD0622F050', 'decimals': 18},
    'RET': {'address': '0xD7394087E1DBBE477FE4F1CF373B9Ac9459565fF', 'decimals': 8},
    'MR': {'address': '0xa4Cb3ef5f41a4D89D6FCed22ea8a1C57957629Aa', 'decimals': 18},
    'FROG': {'address': '0x4fEe21439F2b95b72da2F9f901b3956f27fE91D5', 'decimals': 18},
    'UNIETH': {'address': '0xF1376bceF0f78459C0Ed0ba5ddce976F1ddF51F4', 'decimals': 18},
    'HIENS4': {'address': '0x00A7EC2F2b451cb0233E8aDbf4C9a951027C2B02', 'decimals': 18},
    'PAYB': {'address': '0xdCB5645eDa1ed34C5641d81b927D33EBaE9cF2A4', 'decimals': 18},
    'RBX': {'address': '0x8254e26e453EB5aBd29B3c37AC9E8Da32E5d3299', 'decimals': 18},
    'IFEX': {'address': '0xA891CF72AEDa692570928eFe1A832342e9783CDC', 'decimals': 18},
    'INXT': {'address': '0x4A8f5F96D5436e43112c2fBC6a9f70DA9e4E16D4', 'decimals': 8},
    'CEJI': {'address': '0x189fC141854BFc6146777406d53fbC89F4e70407', 'decimals': 18},
    'THEO': {'address': '0xfAc0403a24229d7e2Edd994D50F5940624CBeac2', 'decimals': 9},
    'N': {'address': '0x36c86a1c23f4D17f3B3F28B7266cD7DB1D74A0B4', 'decimals': 9},
    'X7101': {'address': '0x7101a9392EAc53B01e7c07ca3baCa945A56EE105', 'decimals': 18},
    'WANATHA': {'address': '0x3383c5a8969Dc413bfdDc9656Eb80A1408E4bA20', 'decimals': 18},
    'ADF': {'address': '0x7220e92D418E2EB59D0C25d195FA004bfD3aFC42', 'decimals': 18},
    'GUNTHY': {'address': '0x3684b581dB1F94b721EE0022624329FEb16Ab653', 'decimals': 18},
    'CNW': {'address': '0x433FCe7dfbEc729A79999EAf056Cb073B2153ebA', 'decimals': 6},
    'TOKR': {'address': '0xA885B6d3C87535de688194Ec3645199ee76D6772', 'decimals': 9},
    'AGIX': {'address': '0x5B7533812759B45C2B44C19e320ba2cD2681b542', 'decimals': 8},
    'YVTUSD': {'address': '0xFD0877d9095789cAF24c98F7CCe092fa8E120775', 'decimals': 18},
    'CTR': {'address': '0x923b83c26B3809d960fF80332Ed00aA46D7Ed375', 'decimals': 18},
    'NPICK': {'address': '0xC3DCA8F61B275d1e88f3ea31B3e311c49f56b24B', 'decimals': 18},
    'POLL': {'address': '0x705EE96c1c160842C92c1aeCfCFfccc9C412e3D9', 'decimals': 18},
    'TOKE': {'address': '0x2e9d63788249371f1DFC918a52f8d799F4a38C94', 'decimals': 18},
    'MEME': {'address': '0x60c7AEa107EA3cDAb21455e187cFB7e54E09b760', 'decimals': 18},
    'YVSUSD': {'address': '0xa5cA62D95D24A4a350983D5B8ac4EB8638887396', 'decimals': 18},
    'CMKR': {'address': '0x95b4eF2869eBD94BEb4eEE400a99824BF5DC325b', 'decimals': 8},
    'EVN': {'address': '0x9aF15D7B8776fa296019979E70a5BE53c714A7ec', 'decimals': 18},
    'SNGLS': {'address': '0xaeC2E87E0A235266D9C5ADc9DEb4b2E29b54D009', 'decimals': 0},
    'GOM2': {'address': '0x48783486ddD7fa85ECa6B0C4AE8920Bc25DfbcD7', 'decimals': 0},
    'INU': {'address': '0xF32aa187d5Bc16A2C02A6aFb7Df1459d0D107574', 'decimals': 18},
    'COLLIE': {'address': '0xB8A768Cd332e4461FCcFbdb1d0292cEFFbB2Bb8A', 'decimals': 18},
    'WIZ': {'address': '0x33d20575f20C6A1881F8ab08E69F6fbAFfAEedF2', 'decimals': 12},
    'EVRY': {'address': '0xd7DCd9B99787C619b4D57979521258D1A7267ad7', 'decimals': 18},
    'WSTA': {'address': '0xeDEec5691f23E4914cF0183A4196bBEb30d027a0', 'decimals': 18},
    'D2D': {'address': '0x43D4A3cd90ddD2F8f4f693170C9c8098163502ad', 'decimals': 18},
    'SOBA': {'address': '0x65032604Dab202aFf9ADf89300CdB4bD0d059F55', 'decimals': 18},
    'PRVG': {'address': '0x2Ec3275f7aCe4044e499823F511cd58250be8E3d', 'decimals': 8},
    'MDCY': {'address': '0xFEead860FFa43e5660Ca62ea5953a74F695C1D5B', 'decimals': 18},
    'DFIAT': {'address': '0x1045F5cCb01DAEA4f8eaB055F5FcBB7C0E7c89F0', 'decimals': 18},
    'FLIKS': {'address': '0x9280abF1AF455d6228def9c93b5E6D85F3F1ecc8', 'decimals': 18},
    'JRT': {'address': '0x8A9C67fee641579dEbA04928c4BC45F66e26343A', 'decimals': 18},
    'SDL': {'address': '0xA95C5ebB86E0dE73B4fB8c47A45B792CFeA28C23', 'decimals': 18},
    'OWN': {'address': '0xcC6F15Be8573cB8243C42d300565566D328213Dd', 'decimals': 18},
    'MIKAWA': {'address': '0x0888Fd2960Ddf57Cfb561B59d73Dd2b9524F6A26', 'decimals': 9},
    'LSD': {'address': '0x97D4F49eEb0E2c96d5EbAA71AB8418e563ecd9fd', 'decimals': 9},
    'EZY': {'address': '0xA6D5C720a9af5A405dFB6b9F44Fc44FaB5D4A58D', 'decimals': 8},
    'YVDAI': {'address': '0xdA816459F1AB5631232FE5e97a05BBBb94970c95', 'decimals': 18},
    'XCHF': {'address': '0xB4272071eCAdd69d933AdcD19cA99fe80664fc08', 'decimals': 18},
    'PIN': {'address': '0xc1f976B91217E240885536aF8b63bc8b5269a9BE', 'decimals': 18},
    'TFS': {'address': '0xc2A81eb482cB4677136D8812cC6Db6E0cB580883', 'decimals': 18},
    'PZ': {'address': '0x578Dc16c47327429fF1594fA3eB77A96F38438E0', 'decimals': 9},
    'FX': {'address': '0x8c15Ef5b4B21951d50E53E4fbdA8298FFAD25057', 'decimals': 18},
    'BEAR': {'address': '0x17837004eA685690b32dbEaD02a274Ec4333A26a', 'decimals': 18},
    'O': {'address': '0xb53ecF1345caBeE6eA1a65100Ebb153cEbcac40f', 'decimals': 18},
    'HBTC': {'address': '0x0316EB71485b0Ab14103307bf65a021042c6d380', 'decimals': 18},
    'DFND': {'address': '0xd2adC1C84443Ad06f0017aDCa346Bd9b6fc52CaB', 'decimals': 18},
    'RBIS': {'address': '0xf34B1Db61ACa1a371fE97BAd2606c9f534fb9D7D', 'decimals': 18},
    'SAN': {'address': '0x7C5A0CE9267ED19B22F8cae653F198e3E8daf098', 'decimals': 18},
    'XPN': {'address': '0x3b9e094D56103611f0ACEfDAb43182347BA60dF4', 'decimals': 18},
    'KUKY': {'address': '0xE6b7743e2B9Aa2d0A9B163C4e69186ABb57817D9', 'decimals': 18},
    '1WO': {'address': '0xfDBc1aDc26F0F8f8606a5d63b7D3a3CD21c22B23', 'decimals': 8},
    'GOG': {'address': '0x9AB7bb7FdC60f4357ECFef43986818A2A3569c62', 'decimals': 18},
    'RHEGIC2': {'address': '0xAd7Ca17e23f13982796D27d1E6406366Def6eE5f', 'decimals': 18},
    'OLE': {'address': '0x92CfbEC26C206C90aeE3b7C66A9AE673754FaB7e', 'decimals': 18},
    'SUNDER': {'address': '0xbDbf245c690d54b67C6e610A28486A2C6dE08bE6', 'decimals': 18},
    'INU': {'address': '0x050D94685c6B0477E1Fc555888AF6e2bB8dFBda5', 'decimals': 18},
    'T': {'address': '0xCdF7028ceAB81fA0C6971208e83fa7872994beE5', 'decimals': 18},
    'ALINK': {'address': '0xa06bC25B5805d5F8d82847D191Cb4Af5A3e873E0', 'decimals': 18},
    'UJENNY': {'address': '0xa499648fD0e80FD911972BbEb069e4c20e68bF22', 'decimals': 18},
    'SHADOWCATS': {'address': '0x0018d5E01e53878F90FEAB02F1b2019a21adf8b1', 'decimals': 18},
    'VCHF': {'address': '0x79d4f0232A66c4c91b89c76362016A1707CFBF4f', 'decimals': 18},
    'CPRX': {'address': '0xc6e145421FD494B26dCF2BFeB1b02b7c5721978f', 'decimals': 18},
    'RPG': {'address': '0x0E5C8C387C5EBa2eCbc137aD012aeD5Fe729e251', 'decimals': 18},
    'DOGPAD': {'address': '0x6f3277ad0782a7DA3eb676b85a8346A100BF9C1c', 'decimals': 18},
    'DAWGS': {'address': '0x64A77277e37D44957FE5815d6FF442ab8b16cC29', 'decimals': 9},
    'COT': {'address': '0x5CAc718A3AE330d361e39244BF9e67AB17514CE8', 'decimals': 18},
    'PULSE': {'address': '0x52A047Ee205701895Ee06A375492490EC9C597cE', 'decimals': 18},
    'YCO': {'address': '0x3D0293f06daF4311B482564330D57C8Db6C10893', 'decimals': 8},
    'SG': {'address': '0xdDF7Fd345D54ff4B40079579d4C4670415DbfD0A', 'decimals': 18},
    'VPP': {'address': '0x5EeAA2DCb23056F4E8654a349E57eBE5e76b5e6e', 'decimals': 18},
    'ROBOT': {'address': '0xfb5453340C03db5aDe474b27E68B6a9c6b2823Eb', 'decimals': 18},
    'TCH': {'address': '0x9B39A0B97319a9bd5fed217c1dB7b030453bac91', 'decimals': 18},
    'MIMIR': {'address': '0x71dc40668682a124231301414167E4CF7f55383C', 'decimals': 18},
    'API3': {'address': '0x0b38210ea11411557c13457D4dA7dC6ea731B88a', 'decimals': 18},
    'DAPPX': {'address': '0x00d8318E44780EdEEFcF3020A5448F636788883C', 'decimals': 18},
    'CET': {'address': '0x081F67aFA0cCF8c7B17540767BBe95DF2bA8D97F', 'decimals': 18},
    'HMQ': {'address': '0xcbCC0F036ED4788F63FC0fEE32873d6A7487b908', 'decimals': 8},
    'MIRO': {'address': '0x0168703872fa06741Ecaa9Dff7803168e83f7aE0', 'decimals': 8},
    'OAX': {'address': '0x701C244b988a513c945973dEFA05de933b23Fe1D', 'decimals': 18},
    'TAG': {'address': '0x7797C85b46F548eAcC07C229f6Cd207d6370442f', 'decimals': 18},
    'TPRO': {'address': '0x3540abE4f288B280A0740ad5121aEc337c404D15', 'decimals': 18},
    'GTON': {'address': '0x01e0E2e61f554eCAaeC0cC933E739Ad90f24a86d', 'decimals': 18},
    'EMPIRE': {'address': '0x9A2Af0AbB12bee5369B180976Be01E8c80D0e7B6', 'decimals': 9},
    'COSHI': {'address': '0x668C50B1c7f46EFFBE3f242687071d7908AAB00A', 'decimals': 9},
    'WSTETH': {'address': '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', 'decimals': 18},
    'RNT': {'address': '0xFF603F43946A3A28DF5E6A73172555D8C8b02386', 'decimals': 18},
    'HIFRIENDS': {'address': '0xeb2f5A4e1441dF330670Dc7B0cf4eaC0f7ab5fa5', 'decimals': 18},
    'GRG': {'address': '0x4FbB350052Bca5417566f188eB2EBCE5b19BC964', 'decimals': 18},
    'POMI': {'address': '0x96ef7f9cF1B6eCC66E482A6598fc9F009E9277DA', 'decimals': 8},
    'SHARE': {'address': '0x39795344CBCc76cC3Fb94B9D1b15C23c2070C66D', 'decimals': 9},
    'FKX': {'address': '0x16484d73Ac08d2355F466d448D2b79D2039F6EBB', 'decimals': 18},
    'BTU': {'address': '0xb683D83a532e2Cb7DFa5275eED3698436371cc9f', 'decimals': 18},
    'HIPUNKS': {'address': '0xb1f136a74e18E4E2921feBbf25820d1Bb65B5647', 'decimals': 18},
    'ONEPIECE': {'address': '0xf40eDF22e4aB146a32eCD8d659Cedc7A2a55Cd43', 'decimals': 9},
    'BRT': {'address': '0xd578779dbC9252218E12d18D628e3cb27E4a56f2', 'decimals': 18},
    'DOUGH': {'address': '0xad32A8e6220741182940c5aBF610bDE99E737b2D', 'decimals': 18},
    'BLUESPARROW': {'address': '0x24cCeDEBF841544C9e6a62Af4E8c2fA6e5a46FdE', 'decimals': 9},
    'UETH': {'address': '0x77607588222e01bf892a29Abab45796A2047fc7b', 'decimals': 18},
    'ULT': {'address': '0x09617F6fD6cF8A71278ec86e23bBab29C04353a7', 'decimals': 18},
    'CENNZ': {'address': '0x1122B6a0E00DCe0563082b6e2953f3A943855c1F', 'decimals': 18},
    'BARK': {'address': '0x53340a1EF3a0dDeBA1D94BBd1E2fF55936F0ea60', 'decimals': 18},
    'BASE': {'address': '0xe9f84d418B008888A992Ff8c6D22389C2C3504e0', 'decimals': 8},
    'LUCKY': {'address': '0x6B0956258fF7bd7645aa35369B55B61b8e6d6140', 'decimals': 8},
    'BUMP': {'address': '0x785c34312dfA6B74F6f1829f79ADe39042222168', 'decimals': 18},
    'DEVT': {'address': '0xB5c578947de0fd71303F71F2C3d41767438bD0de', 'decimals': 18},
    'LIT': {'address': '0xfd0205066521550D7d7AB19DA8F72bb004b4C341', 'decimals': 18},
    'GINU': {'address': '0x74bE64B45D394FA57816c1950E94dBB8d7A7B306', 'decimals': 9},
    'AISC': {'address': '0xB7Df0f42FAe30acf30C9A5BA147D6B792b5eB9d9', 'decimals': 18},
    'IETH': {'address': '0xc383a3833A87009fD9597F8184979AF5eDFad019', 'decimals': 18},
    'SCCN': {'address': '0xdfdDF7a69716124Bc346bA556d4b9f9E74C4A8Bc', 'decimals': 18},
    'MDF': {'address': '0x1a57367C6194199e5D9aEa1cE027431682dfB411', 'decimals': 18},
    'SELECT': {'address': '0x01FB70eA8AcFDd5435c8e8bd1531d96D57cEd0ec', 'decimals': 18},
    'AUR': {'address': '0x6E98E5401ADCB0D76F4DEBfc3d794B3031F48790', 'decimals': 18},
    'ARIA20': {'address': '0xeDF6568618A00C6F0908Bf7758A16F76B6E04aF9', 'decimals': 18},
    'TON': {'address': '0x2be5e8c109e2197D077D13A82dAead6a9b3433C5', 'decimals': 18},
    'DOS': {'address': '0x0A913beaD80F321E7Ac35285Ee10d9d922659cB7', 'decimals': 18},
    'DSP': {'address': '0x8C661806f716652B637728355cC4e2620D428F99', 'decimals': 18},
    'WFX': {'address': '0xBa1ED22C69ad00739EE2b4aBD70e270BE9e87eE2', 'decimals': 18},
    'FIWT': {'address': '0x62406995CaFd18f57e7375e8E0060725Acebce58', 'decimals': 18},
    'VEE': {'address': '0x340D2bdE5Eb28c1eed91B2f790723E3B160613B7', 'decimals': 18},
    'PARTY': {'address': '0x0fE0Ed7F146Cb12e4B9759afF4FA8d34571802ca', 'decimals': 18},
    'KEES': {'address': '0xfCC63e6459936216b7b887F6dA5fCD90c0C62b7c', 'decimals': 18},
    'TNS': {'address': '0xb0280743b44bF7db4B6bE482b2Ba7b75E5dA096C', 'decimals': 18},
    'ARA': {'address': '0x9Ac5C63dDcb93612E316ab31dFc8192bC8961988', 'decimals': 18},
    'LEMD': {'address': '0xf45f6c8bB3d77eA762175b8f7ca4D251941649fA', 'decimals': 18},
    'SHAKE': {'address': '0x6006FC2a849fEdABa8330ce36F5133DE01F96189', 'decimals': 18},
    'ARC': {'address': '0xC82E3dB60A52CF7529253b4eC688f631aad9e7c2', 'decimals': 18},
    'RIFI': {'address': '0xC8EF1460277EA47d179dEc66D1c5f8b7f7aE5a28', 'decimals': 18},
    'AMKT': {'address': '0xF17A3fE536F8F7847F1385ec1bC967b2Ca9caE8D', 'decimals': 18},
    'DOGEZ': {'address': '0x4D8046B2f7986FEB5a4b0781EbF5Fa2D7a808617', 'decimals': 18},
    'TPT': {'address': '0x4161725D019690a3E0de50f6bE67b07a86A9fAe1', 'decimals': 4},
    'DVPN': {'address': '0xa44E5137293E855B1b7bC7E2C6f8cD796fFCB037', 'decimals': 8},
    'FLC': {'address': '0x32C4ADB9cF57f972bc375129de91C897b4F364F1', 'decimals': 18},
    'MTV': {'address': '0x6226e00bCAc68b0Fe55583B90A1d727C14fAB77f', 'decimals': 18},
    'EXRT': {'address': '0xb20043F149817bff5322F1b928e89aBFC65A9925', 'decimals': 8},
    'BULK': {'address': '0xa143ac515dCA260A46C742C7251eF3B268639593', 'decimals': 18},
    'PRTG': {'address': '0xBD04ccc050058a6A422851fA6c0F92BB65EB06ca', 'decimals': 18},
    'WNCG': {'address': '0xf203Ca1769ca8e9e8FE1DA9D147DB68B6c919817', 'decimals': 18},
    'IDLESUSDYIELD': {'address': '0xF52CDcD458bf455aeD77751743180eC4A595Fd3F', 'decimals': 18},
    'ANTE': {'address': '0xbb70AdbE39408cB1E5258702ea8ADa7c81165b73', 'decimals': 18},
    'FETCH': {'address': '0xcC779bbBD17b2E68ECdCCaeA3d8C791B1573c66b', 'decimals': 18},
    'KOACOMBAT': {'address': '0x6769D86f9C430f5AC6d9c861a0173613F1C5544C', 'decimals': 9},
    'CLB': {'address': '0x0AD896863ce4Cd84f10a9D30D4f509cefFd53c84', 'decimals': 18},
    'GETH': {'address': '0x3802c218221390025BCEABBaD5D8C59F40eB74B8', 'decimals': 18},
    'CBETH': {'address': '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704', 'decimals': 18},
    'AMANA': {'address': '0x6FCE4A401B6B80ACe52baAefE4421Bd188e76F6f', 'decimals': 18},
    'TIC': {'address': '0x72430A612Adc007c50e3b6946dBb1Bb0fd3101D1', 'decimals': 8},
    'NOONE': {'address': '0x296233E84c1D7bfF11121BF6D60f0FFA39C3f0cf', 'decimals': 9},
    'SOUL': {'address': '0x75858677e27C930FB622759FeafFeE2b754Af07F', 'decimals': 8},
    'IMBTC': {'address': '0x3212b29E33587A00FB1C83346f5dBFA69A458923', 'decimals': 8},
    'FNK': {'address': '0xB5FE099475d3030DDe498c3BB6F3854F762A48Ad', 'decimals': 18},
    'STBU': {'address': '0xa6422E3E219ee6d4C1B18895275FE43556fd50eD', 'decimals': 18},
    'THEOS': {'address': '0x9e10f61749c4952C320412A6B26901605Ff6Da1d', 'decimals': 18},
    'UERII': {'address': '0xc1ABb8c93be6811aFfC70675b0432926c4BFBb5D', 'decimals': 18},
    'VXL': {'address': '0x16CC8367055aE7e9157DBcB9d86Fd6CE82522b31', 'decimals': 18},
    'SWFTC': {'address': '0x0bb217E40F8a5Cb79Adf04E1aAb60E5abd0dfC1e', 'decimals': 8},
    'M2': {'address': '0x965d79F1A1016B574a62986e13Ca8Ab04DfdD15C', 'decimals': 18},
    'DAI': {'address': '0x6B175474E89094C44Da98b954EedeAC495271d0F', 'decimals': 18},
    'XOY': {'address': '0xe9F1d62c671EFe99896492766c0B416bD3FB9e52', 'decimals': 8},
    'UMOON': {'address': '0x71de2285ec83BB6517D2B3eC8DEb2A6c44D8C5da', 'decimals': 18},
    'FYZ': {'address': '0x6BFf2fE249601ed0Db3a87424a2E923118BB0312', 'decimals': 18},
    'LAMB': {'address': '0x8971f9fd7196e5cEE2C1032B50F656855af7Dd26', 'decimals': 18},
    'DXY': {'address': '0x39Fa206c1648944f92E8F7B626e1CBdf78d7E9dB', 'decimals': 18},
    'SDEX': {'address': '0x5DE8ab7E27f6E7A1fFf3E5B337584Aa43961BEeF', 'decimals': 18},
    'MS': {'address': '0x6d6554939D646f274d0FC3cEcB7dab5d76bc908F', 'decimals': 18},
    'KPOP': {'address': '0x7B7983967409Fce461EA8bbDF9ED37631b1d59C9', 'decimals': 18},
    'DOGTIC': {'address': '0x3c79EB9d7a76F4EB7B2a5286aAcFbbD4FeAbe79A', 'decimals': 18},
    'SBTC': {'address': '0xb8e103b60A33597136EA9511F46b6dBeB643a3a5', 'decimals': 18},
    'MODEX': {'address': '0x4bceA5E4d0F6eD53cf45e7a28FebB2d3621D7438', 'decimals': 18},
    'MOOV': {'address': '0x24EC2Ca132abf8F6f8a6E24A1B97943e31f256a7', 'decimals': 18},
    'ADD': {'address': '0x635d081fD8F6670135D8a3640E2cF78220787d56', 'decimals': 18},
    'BOWL': {'address': '0x760Ea31711aC14cb9Ba81aC25453C6f378fA5794', 'decimals': 9},
    'MPH': {'address': '0x8888801aF4d980682e47f1A9036e589479e835C5', 'decimals': 18},
    'MOC': {'address': '0x865ec58b06bF6305B886793AA20A2da31D034E68', 'decimals': 18},
    'DMG': {'address': '0xEd91879919B71bB6905f23af0A68d231EcF87b14', 'decimals': 18},
    'ALINK': {'address': '0xA64BD6C70Cb9051F6A9ba1F163Fdc07E0DfB5F84', 'decimals': 18},
    'DOGZ': {'address': '0x82A77710495A35549D2aDD797412b4a4497D33eF', 'decimals': 18},
    'HIDOODLES': {'address': '0x3700Adfd26d5bC062cB8b8a77e68fbd43F58eCAB', 'decimals': 18},
    'IETHV2': {'address': '0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78', 'decimals': 18},
    'RAK': {'address': '0xa8B0F154A688c22142E361707df64277e0A0bE66', 'decimals': 18},
    'RMATIC': {'address': '0x3DBB00c9bE5A327E25CAf4f650844C5DbA81E34B', 'decimals': 18},
    'MPAY': {'address': '0x3810A4Ddf41E586Fa0dbA1463A7951B748cEcFca', 'decimals': 18},
    'SPC': {'address': '0x86ed939B500E121C0C5f493F399084Db596dAd20', 'decimals': 18},
    'BABYSAITAMA': {'address': '0xf79F9020560963422eCC9C0c04D3a21190BBf045', 'decimals': 9},
    'BLY': {'address': '0xf8aD7dFe656188A23e89da09506Adf7ad9290D5d', 'decimals': 18},
    'TDS': {'address': '0x6295Ab2BE04A617747481B292c390BfcA592Cf28', 'decimals': 18},
    'UMBR': {'address': '0xa4bBE66f151B22B167127c770016b15fF97Dd35C', 'decimals': 18},
    'MASTERMIND': {'address': '0x8b0FdE007458Ee153bd0F66CD448Af5FB3D99b43', 'decimals': 18},
    'LMR': {'address': '0x4b1D0b9F081468D780Ca1d5d79132b64301085d1', 'decimals': 8},
    'SHIBN': {'address': '0xd346E8ADA104093ADcf5f4186087E1AD309BB3B3', 'decimals': 18},
    'WIS': {'address': '0xDecade1c6Bf2cD9fb89aFad73e4a519C867adcF5', 'decimals': 18},
    'HULK': {'address': '0x91a5de30e57831529a3c1aF636A78a7E4E83f3aa', 'decimals': 18},
    'ISIKC': {'address': '0x42726d074BBa68Ccc15200442B72Afa2D495A783', 'decimals': 4},
    'OPCT': {'address': '0xDb05EA0877A2622883941b939f0bb11d1ac7c400', 'decimals': 18},
    'SPELLFIRE': {'address': '0x3A0b022f32b3191D44E5847da12dc0B63fb07C91', 'decimals': 18},
    'ISR': {'address': '0xB16d3Ed603D62b125c6bd45519EDa40829549489', 'decimals': 18},
    'ARTIC': {'address': '0xE83D5FB2c60B3A2597452e248CF7b2f52a7E731e', 'decimals': 18},
    'FRANK': {'address': '0xdC4c9bFBB37C7cDeeBc53C0BA5C7474FEA5E106D', 'decimals': 9},
    'EHASH': {'address': '0x2942E3B38E33123965bfbc21E802bE943a76bbC6', 'decimals': 18},
    '0XMR': {'address': '0x035dF12E0F3ac6671126525f1015E47D79dFEDDF', 'decimals': 18},
    'FEG': {'address': '0xbededDf2eF49E87037c4fb2cA34d1FF3D3992A11', 'decimals': 18},
    'MTL': {'address': '0xF433089366899D83a9f26A773D59ec7eCF30355e', 'decimals': 8},
    'STRNGR': {'address': '0xDc0327D50E6C73db2F8117760592C8BBf1CDCF38', 'decimals': 18},
    'RBC': {'address': '0x3330BFb7332cA23cd071631837dC289B09C33333', 'decimals': 18},
    'ASW': {'address': '0x56694577564FdD577a0ABB20FE95C1E2756C2a11', 'decimals': 18},
    'AC': {'address': '0x9A0aBA393aac4dFbFf4333B06c407458002C6183', 'decimals': 18},
    'LKN': {'address': '0x9f549ebFD4974cD4eD4A1550D40394B44A7382AA', 'decimals': 18},
    'DDOS': {'address': '0x7FBEC0bb6A7152e77C30D005B5D49cbC08A602C3', 'decimals': 18},
    'VEMP': {'address': '0xcFEB09C3c5F0f78aD72166D55f9e6E9A60e96eEC', 'decimals': 18},
    'SNOV': {'address': '0xBDC5bAC39Dbe132B1E030e898aE3830017D7d969', 'decimals': 18},
    'WBETH': {'address': '0xa2E3356610840701BDf5611a53974510Ae27E2e1', 'decimals': 18},
    'ASTRA': {'address': '0x7E9c15C43f0D6C4a12E6bdfF7c7D55D0f80e3E23', 'decimals': 18},
    'MYOBU': {'address': '0xDfDd3459D4F87234751696840092ee20C970Fb07', 'decimals': 9},
    'PAYT': {'address': '0x8EF47555856f6Ce2E0cd7C36AeF4FAb317d2e2E2', 'decimals': 18},
    'MCO2': {'address': '0xfC98e825A2264D890F9a1e68ed50E1526abCcacD', 'decimals': 18},
    'CSMATIC': {'address': '0x38b7Bf4eeCF3EB530b1529c9401FC37d2a71a912', 'decimals': 18},
    'ETH20': {'address': '0x04A6b6DE116Fb8bF57e5eE8b05e0293EA3639fE8', 'decimals': 18},
    'WATT': {'address': '0xE67F943af5Eb6051ef56f05979cc30B732717fa6', 'decimals': 4},
    'ALPHA': {'address': '0x138C2F1123cF3f82E4596d097c118eAc6684940B', 'decimals': 18},
    'GUILD': {'address': '0x83E9F223e1edb3486F876EE888d76bFba26c475A', 'decimals': 18},
    'XTK': {'address': '0x7F3EDcdD180Dbe4819Bd98FeE8929b5cEdB3AdEB', 'decimals': 18},
    'SAK3': {'address': '0xe9F84dE264E91529aF07Fa2C746e934397810334', 'decimals': 18},
    'GOLD': {'address': '0xf1b8762a7fa8C244e36F7234EDF40cFaE24394e3', 'decimals': 18},
    'FUC': {'address': '0x62A116B9fFe7338bEe13c87E69552De9cC67627D', 'decimals': 18},
    'QNT': {'address': '0x4a220E6096B25EADb88358cb44068A3248254675', 'decimals': 18},
    'MTU': {'address': '0x822f1A51E534723ec1305E6EB5C96AEcfeb4c629', 'decimals': 18},
    'BSPT': {'address': '0xa350DA05405cc204e551C4eeD19C3039646528d5', 'decimals': 18},
    'GEM': {'address': '0xc7BbA5b765581eFb2Cdd2679DB5Bea9eE79b201f', 'decimals': 18},
    'UQC': {'address': '0x8806926Ab68EB5a7b909DcAf6FdBe5d93271D6e2', 'decimals': 18},
    'CPTINU': {'address': '0x7Cca2e1c9B0519f52029467914A15E782BF66971', 'decimals': 18},
    'GHT': {'address': '0xbe30F684d62C9F7883a75A29c162c332c0d98f23', 'decimals': 18},
    'HUNT': {'address': '0x9AAb071B4129B083B01cB5A0Cb513Ce7ecA26fa5', 'decimals': 18},
    'NEBO': {'address': '0x7f0c8B125040f707441cad9e5eD8a8408673b455', 'decimals': 18},
    'X7DAO': {'address': '0x7105E64bF67ECA3Ae9b123F0e5Ca2b83b2eF2dA0', 'decimals': 18},
    'SDT': {'address': '0x73968b9a57c6E53d41345FD57a6E6ae27d6CDB2F', 'decimals': 18},
    'MFT': {'address': '0xDF2C7238198Ad8B389666574f2d8bc411A4b7428', 'decimals': 18},
    'EMT': {'address': '0x9501BFc48897DCEEadf73113EF635d2fF7ee4B97', 'decimals': 18},
    'R34P': {'address': '0xCAEaf8381D4B20b43AFA42061D6f80319A8881F6', 'decimals': 8},
    'UBXN': {'address': '0x7A73839bd0e5cdED853cB01aa9773F8989381065', 'decimals': 18},
    'ZEUS': {'address': '0x6ef460EB3563CfCc73F8147B0A77DAFFeE71F867', 'decimals': 18},
    'YMNT': {'address': '0xe59A1fe295f940E5226DcF6411Ef2Bdf3089C254', 'decimals': 9},
    'NCT': {'address': '0x8A9c4dfe8b9D8962B31e4e16F8321C44d48e246E', 'decimals': 18},
    'RSFT': {'address': '0x78223d31298107f3E310B09797b07967832046A6', 'decimals': 18},
    'OBTC': {'address': '0xfF770E4c68e35DB85c6e0E89a43750eC02bDB2AC', 'decimals': 18},
    'GLTM': {'address': '0x556d4F40982CB95E0714989E0c229C42be8b1499', 'decimals': 18},
    'MYST': {'address': '0x4Cf89ca06ad997bC732Dc876ed2A7F26a9E7f361', 'decimals': 18},
    'CLT': {'address': '0x2001f2A0Cf801EcFda622f6C28fb6E10d803D969', 'decimals': 8},
    'ASX': {'address': '0x083DFb3566bde80380446718eEDa55B800464636', 'decimals': 18},
    'POM': {'address': '0x24ffe459F51ea20C5d8ad49843529fc33654e7E4', 'decimals': 9},
    'VIDY': {'address': '0xC77b230F31b517F1ef362e59c173C2BE6540B5E8', 'decimals': 18},
    'CIX100': {'address': '0x6393E822874728f8Afa7e1C9944E417D37CA5878', 'decimals': 18},
    'AI': {'address': '0x5d259E3FbcAdef1abfeA5582e456Bc7F0aebC2a1', 'decimals': 18},
    'FUND': {'address': '0x7d8d7c26179b7A6aebbF66A91C38ED92d5B4996b', 'decimals': 18},
    'POLL': {'address': '0x4b0f027d0b694Aae2761ED2d426295d4f949F5d0', 'decimals': 18},
    'GYOSHI': {'address': '0x1F17D72cBe65Df609315dF5c4f5F729eFbd00Ade', 'decimals': 18},
    'ARTEQ': {'address': '0x805C2077f3ab224D889f9c3992B41B2F4722c787', 'decimals': 0},
    'LMY': {'address': '0x66fD97a78d8854fEc445cd1C80a07896B0b4851f', 'decimals': 18},
    'STORJ': {'address': '0xB64ef51C888972c908CFacf59B47C1AfBC0Ab8aC', 'decimals': 8},
    'QASH': {'address': '0x618E75Ac90b12c6049Ba3b27f5d5F8651b0037F6', 'decimals': 6},
    '1INCH': {'address': '0x111111111117dC0aa78b770fA6A738034120C302', 'decimals': 18},
    'ELONONE': {'address': '0x97b65710D03E12775189F0D113202cc1443b0aa2', 'decimals': 9},
    'AAG': {'address': '0x5ba19d656B65f1684cFea4Af428C23B9f3628F97', 'decimals': 18},
    'BZR': {'address': '0x8d96B4ab6C741a4C8679AE323a100d74f085BA8F', 'decimals': 18},
    'BANCA': {'address': '0x998b3B82bC9dBA173990Be7afb772788B5aCB8Bd', 'decimals': 18},
    'ZIK': {'address': '0x88303FeD02b31dB9C7a9eafB711dA9eF4A03e5D3', 'decimals': 18},
    'DYNMT': {'address': '0x3B7f247f21BF3A07088C2D3423F64233d4B069F7', 'decimals': 2},
    'CATT': {'address': '0x6E605c269E0C92e70BEeB85486f1fC550f9380BD', 'decimals': 18},
    'HUB': {'address': '0x56bd0C900acAF04125Ee26f546d6214634fD970F', 'decimals': 18},
    'CNC': {'address': '0x9aE380F0272E2162340a5bB646c354271c0F5cFC', 'decimals': 18},
    'SAK': {'address': '0xb269Ab3B20E09e6fd17C3ED5B4CECF3dEA1a5141', 'decimals': 18},
    'OLEA': {'address': '0xc6Cc3d07C705E39D11c7f60d8836C7C78D4aC5f1', 'decimals': 18},
    'CARD': {'address': '0x954b890704693af242613edEf1B603825afcD708', 'decimals': 18},
    'ECASH': {'address': '0x5D21eF5f25a985380B65c8e943A0082fEDa0Db84', 'decimals': 18},
    'ROBO': {'address': '0x6FC2f1044A3b9bB3e43A43EC8F840843Ed753061', 'decimals': 18},
    'EUR-C': {'address': '0xf7790914Dc335B20Aa19D7c9C9171e14e278A134', 'decimals': 18},
    'SEREN': {'address': '0x33D6064F0DFb62462A74049f30909DDD4f683Ba2', 'decimals': 9},
    'SX': {'address': '0x96Eb50804d0eF2790f2E1A33670FEff6040CF89D', 'decimals': 18},
    'G': {'address': '0x97E19E2a5458294842036404E6A05571D8Bc2fA3', 'decimals': 18},
    'DFX': {'address': '0xdD3A88EF8535Ff7b19178B735C52eA277eF25623', 'decimals': 18},
    'DMTR': {'address': '0x51cB253744189f11241becb29BeDd3F1b5384fdB', 'decimals': 18},
    'OKB': {'address': '0x75231F58b43240C9718Dd58B4967c5114342a86c', 'decimals': 18},
    'VCK': {'address': '0xfa5B75a9e13Df9775cf5b996A049D9cc07c15731', 'decimals': 18},
    'HATI': {'address': '0x251457b7c5d85251Ca1aB384361c821330bE2520', 'decimals': 18},
    'HIOD': {'address': '0x7a83FF237e7870d8d6C3485920EBe654D2841315', 'decimals': 18},
    'APE': {'address': '0x4d224452801ACEd8B2F0aebE155379bb5D594381', 'decimals': 18},
    'FET': {'address': '0xaea46A60368A7bD060eec7DF8CBa43b7EF41Ad85', 'decimals': 18},
    'SODATSU': {'address': '0xf8497f1dab075A3674363B5609Ba679F7dc9C885', 'decimals': 18},
    'CSR': {'address': '0x75Ecb52e403C617679FBd3e77A50f9d10A842387', 'decimals': 18},
    'SKI': {'address': '0x996Dc5dfc819408Dd98Cd92c9a76f64b0738Dc3D', 'decimals': 18},
    'IQN': {'address': '0x0DB8D8b76BC361bAcbB72E2C491E06085A97Ab31', 'decimals': 18},
    'VDG': {'address': '0x57C75ECCc8557136D32619a191fBCDc88560d711', 'decimals': 0},
    'MDT': {'address': '0x814e0908b12A99FeCf5BC101bB5d0b8B5cDf7d26', 'decimals': 18},
    'XIDR': {'address': '0xebF2096E01455108bAdCbAF86cE30b6e5A72aa52', 'decimals': 6},
    'SWISSNFTFUND': {'address': '0x64C1C9a38038AE73C3699B8dc347fEFDa2221E93', 'decimals': 9},
    'FLEX': {'address': '0xFcF8eda095e37A41e002E266DaAD7efC1579bc0A', 'decimals': 18},
    'PAPER': {'address': '0x7aE1D57b58fA6411F32948314BadD83583eE0e8C', 'decimals': 18},
    'BOXA': {'address': '0x7662c015D845EF487fDe32cb884653bE9c9E0110', 'decimals': 9},
    'EST': {'address': '0x259AB9b9EAB62b0fD98729B97BE121073D5B3479', 'decimals': 18},
    'MP3': {'address': '0x018fb5Af9d015Af25592a014C4266a84143De7a0', 'decimals': 18},
    'CTK': {'address': '0x42a501903AFaa1086B5975773375c80E363f4063', 'decimals': 8},
    'PROT': {'address': '0x2341Dd0A96a0Dab62Aa1efB93D59FF7F3bDB8932', 'decimals': 18},
    'WFIO': {'address': '0xbEA269038Eb75BdAB47a9C04D0F5c572d94b93D5', 'decimals': 9},
    'PHUNK': {'address': '0xB39185e33E8c28e0BB3DbBCe24DA5dEA6379Ae91', 'decimals': 18},
    'SOC': {'address': '0xEb6026D3BEAA308D5822C44cDd2Ca8c7714237EC', 'decimals': 18},
    'RLC': {'address': '0x607F4C5BB672230e8672085532f7e901544a7375', 'decimals': 9},
    'YGG': {'address': '0x25f8087EAD173b73D6e8B84329989A8eEA16CF73', 'decimals': 18},
    'BCNT': {'address': '0x9669890e48f330ACD88b78D63E1A6b3482652CD9', 'decimals': 18},
    'PRIX': {'address': '0x3ADfc4999F77D04c8341BAC5F3A76f58DfF5B37A', 'decimals': 8},
    'FIEF': {'address': '0xeA068Fba19CE95f12d252aD8Cb2939225C4Ea02D', 'decimals': 18},
    'SCURVE': {'address': '0xC25a3A3b969415c80451098fa907EC722572917F', 'decimals': 18},
    'WTBT': {'address': '0xD38e031f4529a07996aaB977d2B79f0e00656C56', 'decimals': 18},
    'DFX': {'address': '0x888888435FDe8e7d4c54cAb67f206e4199454c60', 'decimals': 18},
    'IETHV': {'address': '0x3A707d56D538e85B783E8CE12B346e7fB6511F90', 'decimals': 18},
    'AXIS': {'address': '0xecC0F1F860a82aB3b442382D93853C02d6384389', 'decimals': 18},
    'COMFI': {'address': '0x752Efadc0a7E05ad1BCCcDA22c141D01a75EF1e4', 'decimals': 18},
    'DGD': {'address': '0xE0B7927c4aF23765Cb51314A0E0521A9645F0E2A', 'decimals': 9},
    'USDC': {'address': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', 'decimals': 6},
    'USPC': {'address': '0x79B87d52a03ED26A19C1914be4CE37F812e2056c', 'decimals': 18},
    'B-80BAL-20WETH': {'address': '0x5c6Ee304399DBdB9C8Ef030aB642B10820DB8F56', 'decimals': 18},
    'MNT': {'address': '0x149caC67F1cd5d80651E7c9Bb359ec285D821a05', 'decimals': 18},
    'VIX': {'address': '0x49Bf0220C9Ce17E52dCcA3d217231746D676085B', 'decimals': 18},
    'MCT': {'address': '0x6876EbA317272FE221C67405C5e8EB3B24535547', 'decimals': 18},
    'MYC': {'address': '0x4b13006980aCB09645131b91D259eaA111eaF5Ba', 'decimals': 18},
    'DEV': {'address': '0x5cAf454Ba92e6F2c929DF14667Ee360eD9fD5b26', 'decimals': 18},
    'MARX': {'address': '0x2b100Be232F90Dd33d3B6e4b9ad70D1E8AC0E208', 'decimals': 18},
    'DORA': {'address': '0xbc4171f45EF0EF66E76F979dF021a34B46DCc81d', 'decimals': 18},
    'SKYRIM': {'address': '0x2610F0bFC21EF389fe4D03CFB7De9ac1E6C99D6E', 'decimals': 18},
    'ELONGD': {'address': '0x348B7f3106B5Da47405332534d06069fF9CE4d1B', 'decimals': 9},
    'MOOI': {'address': '0xd3C89cAC4A4283EDbA6927e2910fD1eBC14fe006', 'decimals': 18},
    'MTRL': {'address': '0x13C99770694f07279607A6274F28a28c33086424', 'decimals': 18},
    'PLANB': {'address': '0xe77f82f288A6a7EB4Ac295a86FccB1F6D240499B', 'decimals': 18},
    'INST': {'address': '0x6f40d4A6237C257fff2dB00FA0510DeEECd303eb', 'decimals': 18},
    'BPEG': {'address': '0x02AD335DD3ca11c18cEBBbB583b9613b6289d75F', 'decimals': 18},
    'ASIA': {'address': '0xF519381791C03DD7666C142D4E49Fd94d3536011', 'decimals': 18},
    'DDIM': {'address': '0xFbEEa1C75E4c4465CB2FCCc9c6d6afe984558E20', 'decimals': 18},
    'EDG': {'address': '0x08711D3B02C8758F2FB3ab4e80228418a7F8e39c', 'decimals': 0},
    'DSLA': {'address': '0x3aFfCCa64c2A6f4e3B6Bd9c64CD2C969EFd1ECBe', 'decimals': 18},
    'STFX': {'address': '0x9343e24716659A3551eB10Aff9472A2dcAD5Db2d', 'decimals': 18},
    'NUX': {'address': '0x89bD2E7e388fAB44AE88BEf4e1AD12b4F1E0911c', 'decimals': 18},
    'BOSON': {'address': '0xC477D038d5420C6A9e0b031712f61c5120090de9', 'decimals': 18},
    'CLS': {'address': '0x675BBC7514013E2073DB7a919F6e4cbeF576de37', 'decimals': 18},
    'DTO': {'address': '0xB57420FaD6731B004309D5a0ec7C6C906Adb8df7', 'decimals': 18},
    'VBIT': {'address': '0xB8366948B4A3F07BcBf14EB1739daA42A26b07c4', 'decimals': 18},
    'ETHX': {'address': '0x21eAD867C8c5181854f6f8Ce71f75b173d2Bc16A', 'decimals': 18},
    'ID': {'address': '0xEBd9D99A3982d547C5Bb4DB7E3b1F9F14b67Eb83', 'decimals': 18},
    'POWER': {'address': '0xF2f9A7e93f845b3ce154EfbeB64fB9346FCCE509', 'decimals': 18},
    'BUH': {'address': '0x66bcD0C22Bb15cD1bA7199C7a383DC89528f4427', 'decimals': 18},
    'TEM': {'address': '0xaE0585a259A3BCAB258D6EE02FB583f7B33C2a12', 'decimals': 18},
    'DOGEGF': {'address': '0xfb130d93E49DcA13264344966A611dc79a456Bc5', 'decimals': 18},
    'SKT': {'address': '0x012E0e6342308b247F36Ee500ECb14dc77a7a8C1', 'decimals': 8},
    'CYFI': {'address': '0x80a2AE356fc9ef4305676f7a3E2Ed04e12C33946', 'decimals': 8},
    'STONKS': {'address': '0x3d9a4D8ab4F5bd1D5D08Ae3A95e8ED8BB4d7e3B9', 'decimals': 18},
    'LITH': {'address': '0x188E817B02e635D482AE4D81e25DdA98A97C4a42', 'decimals': 18},
    'NMS': {'address': '0x77252494C25444F8598A0c74Ffc90ADc535291a9', 'decimals': 18},
    'JPYC': {'address': '0x431D5dfF03120AFA4bDf332c61A6e1766eF37BDB', 'decimals': 18},
    '0XENCRYPT': {'address': '0x397b102dECcACE4AA8E5bA63EeDb8e65ad83E20c', 'decimals': 9},
    'EWW': {'address': '0x3d38F87BeF0123f5764635C7E11048027E0721a8', 'decimals': 9},
    'ARDX': {'address': '0xB8E2e2101eD11e9138803cd3e06e16dd19910647', 'decimals': 2},
    'BZRX': {'address': '0x56d811088235F11C8920698a204A5010a788f4b3', 'decimals': 18},
    'LINK': {'address': '0x514910771AF9Ca656af840dff83E8264EcF986CA', 'decimals': 18},
    'FOHO': {'address': '0xDd2e93924BDD4E20C3CF4A8736E5955224FA450E', 'decimals': 8},
    'SIFU': {'address': '0x29127fE04ffa4c32AcAC0fFe17280ABD74eAC313', 'decimals': 18},
    'BNK': {'address': '0x7707aAdA3Ce7722Ac63B91727DAF1999849F6835', 'decimals': 8},
    'GEEQ': {'address': '0x6B9f031D718dDed0d681c20cB754F97b3BB81b78', 'decimals': 18},
    'TON': {'address': '0x582d872A1B094FC48F5DE31D3B73F2D9bE47def1', 'decimals': 9},
    'YVAAVE': {'address': '0xd9788f3931Ede4D5018184E198699dC6d66C1915', 'decimals': 18},
    'SEED': {'address': '0x30cF203b48edaA42c3B4918E955fED26Cd012A3F', 'decimals': 18},
    'DC': {'address': '0x0B44547be0A0Df5dCd5327de8EA73680517c5a54', 'decimals': 18},
    'NTX': {'address': '0xFDb15e5E6799Be72798b1ccFAEcbF186Bf73A0c4', 'decimals': 8},
    'GCAKE': {'address': '0x5f944B0c4315Cb7c3a846b025AB4045da44abf6c', 'decimals': 18},
    'OVL': {'address': '0xdc77aCC82ccE1Cc095CbA197474Cc06824adE6F7', 'decimals': 18},
    'ONIT': {'address': '0x410e731c2970Dce3AdD351064AcF5cE9E33FDBf0', 'decimals': 18},
    'FAKT': {'address': '0xC3FFbE26E9446aC52008FfC8C1191d60A12bdC48', 'decimals': 18},
    'SMT': {'address': '0x76dbCba41e11784BFA95cE30e529fA533C3fA4aD', 'decimals': 18},
    'RUN': {'address': '0x5f4c148D17Effd165C2e2d46b46d2BD6e3eBDC3e', 'decimals': 18},
    'IPOR': {'address': '0x1e4746dC744503b53b4A082cB3607B169a289090', 'decimals': 18},
    'B2M': {'address': '0xd7c302fc3ac829C7E896a32c4Bd126f3e8Bd0a1f', 'decimals': 18},
    'BB-A-USDT': {'address': '0x2F4eb100552ef93840d5aDC30560E5513DFfFACb', 'decimals': 18},
    'MAV': {'address': '0xbe1dBe6741fB988fb571AB1E28CfFB36e3C62629', 'decimals': 18},
    'GOHM': {'address': '0x0ab87046fBb341D058F17CBC4c1133F25a20a52f', 'decimals': 18},
    'YVAULT-LP-YCURVE': {'address': '0x5dbcF33D8c2E976c6b560249878e6F1491Bca25c', 'decimals': 18},
    'MEED': {'address': '0x8503a7b00B4b52692cC6c14e5b96F142E30547b7', 'decimals': 18},
    'IBY': {'address': '0x6A68DE599E8E0b1856E322CE5Bd11c5C3C79712B', 'decimals': 18},
    'QDT': {'address': '0x4e432A62733a7ee38Ad2e16b3cc0731457eA5B55', 'decimals': 4},
    'THOL': {'address': '0x207e14389183A94343942de7aFbC607F57460618', 'decimals': 18},
    'PPIZZA': {'address': '0xab306326bC72c2335Bd08F42cbec383691eF8446', 'decimals': 18},
    'TPY': {'address': '0x968Cbe62c830A0Ccf4381614662398505657A2A9', 'decimals': 8},
    'NAAL': {'address': '0xc133529e57681b2999708F9458Be5634e293995E', 'decimals': 18},
    'MFC': {'address': '0x337DD23d05c27bff099d3604938BFc23A9B25820', 'decimals': 18},
    'ELT': {'address': '0xfC1Cb4920dC1110fD61AfaB75Cf085C1f871b8C6', 'decimals': 18},
    'BCUG': {'address': '0x14Da7b27b2E0FedEfe0a664118b0c9bc68e2E9AF', 'decimals': 18},
    'IYKYK': {'address': '0x7cB683151A83c2B10A30cBB003CDA9996228a2bA', 'decimals': 18},
    'FFRAX': {'address': '0x2bE1e42BF263AaB47D27Ba92E72c14823e101D7C', 'decimals': 18},
    'BRIGHT': {'address': '0x5dD57Da40e6866C9FcC34F4b6DDC89F1BA740DfE', 'decimals': 18},
    'FPI': {'address': '0x5Ca135cB8527d76e932f34B5145575F9d8cbE08E', 'decimals': 18},
    'DAMA': {'address': '0x2C0da41C89AdB5a1d4430E5761b9B400911426B0', 'decimals': 18},
    'PRUF': {'address': '0xa49811140E1d6f653dEc28037Be0924C811C4538', 'decimals': 18},
    'CFI': {'address': '0x63b4f3e3fa4e438698CE330e365E831F7cCD1eF4', 'decimals': 18},
    'CRI': {'address': '0x12E951934246186F50146235d541D3bD1D463E4d', 'decimals': 18},
    'DZAR': {'address': '0x9Cb2f26A23b8d89973F08c957C4d7cdf75CD341c', 'decimals': 6},
    'ASTRO': {'address': '0xcbd55D4fFc43467142761A764763652b48b969ff', 'decimals': 18},
    'ADCO': {'address': '0xB6c3DC857845a713d3531cea5ac546F6767992F4', 'decimals': 6},
    'ALOHA': {'address': '0x455F7Ef6D8BCfc35f9337e85aEe1B0600a59FabE', 'decimals': 18},
    'BDP': {'address': '0xf3dcbc6D72a4E1892f7917b7C43b74131Df8480e', 'decimals': 18},
    'UMC': {'address': '0x190fB342aa6a15eB82903323ae78066fF8616746', 'decimals': 6},
    'SONO': {'address': '0x0D15009896EfE9972F8E086bdD3BcBa5c1f74bF3', 'decimals': 8},
    'CBT': {'address': '0xfA93660C3f6a848556Bb8E265f994160A1F2B289', 'decimals': 18},
    'HAPPY': {'address': '0x651e44D0df89055c00513d0efb9E8d1A0C843378', 'decimals': 18},
    'LEVL': {'address': '0x09970aec766b6f3223aCA9111555E99DC50Ff13a', 'decimals': 18},
    'AUC': {'address': '0x87a92428bBc876d463C21C8e51b903f127d9A9F4', 'decimals': 18},
    'X7105': {'address': '0x7105FAA4a26eD1c67B8B2b41BEc98F06Ee21D105', 'decimals': 18},
    'HXRO': {'address': '0x4bD70556ae3F8a6eC6C4080A0C327B24325438f3', 'decimals': 18},
    'HOKK': {'address': '0xe87e15B9c7d989474Cb6d8c56b3DB4eFAD5b21E8', 'decimals': 18},
    'AWRT': {'address': '0x27Ad8e47c48063b209fa5CA14c8c46eCe49B82d2', 'decimals': 9},
    'BOB': {'address': '0xDF347911910b6c9A4286bA8E2EE5ea4a39eB2134', 'decimals': 18},
    'SYM': {'address': '0x2fd61567c29E7ADB4Ca17e60E1f4a3Fcfe68aCb8', 'decimals': 18},
    'PHA': {'address': '0x6c5bA91642F10282b576d91922Ae6448C9d52f4E', 'decimals': 18},
    'BITE': {'address': '0x34103e1190b824a44C6a638438d425cbA21143BA', 'decimals': 18},
    'MANA': {'address': '0x0F5D2fB29fb7d3CFeE444a200298f468908cC942', 'decimals': 18},
    'P2PS': {'address': '0x4527a3B4A8A150403090a99b87efFC96F2195047', 'decimals': 8},
    'TRAC': {'address': '0xaA7a9CA87d3694B5755f213B5D04094b8d0F0A6F', 'decimals': 18},
    'SRM': {'address': '0x476c5E26a75bd202a9683ffD34359C0CC15be0fF', 'decimals': 6},
    'RVUSD': {'address': '0x84777600c51996B2ce92a514FF2583B7d7C2239b', 'decimals': 18},
    'MPT': {'address': '0x47481c1b44F2A1c0135c45AA402CE4F4dDE4D30e', 'decimals': 18},
    'TICKR': {'address': '0xf0a163a26601D9dc8Aef26b388Eadb7B1c620d24', 'decimals': 9},
    'MCT': {'address': '0xBEE709aa42375acE642338F7f657ff859a19c9bC', 'decimals': 18},
    'ARTE': {'address': '0x34612903Db071e888a4dADcaA416d3EE263a87b9', 'decimals': 18},
    'BMC': {'address': '0xd945d2031b4C63C0E363304FB771F709b502DC0a', 'decimals': 18},
    'LTY': {'address': '0x85Ffb35957203dfD12061eAeCD708dB623Bd567C', 'decimals': 18},
    'EHIVE': {'address': '0x10B506Ae42b5466d548EdC4C02a4CE9828386C75', 'decimals': 18},
    'FYP': {'address': '0x8F0921f30555624143d427b340b1156914882C10', 'decimals': 18},
    'YVHEGIC': {'address': '0xe11ba472F74869176652C35D30dB89854b5ae84D', 'decimals': 18},
    'HDAO': {'address': '0x74faaB6986560fD1140508e4266D8a7b87274Ffd', 'decimals': 18},
    'SHIBAMON': {'address': '0x36b00c4c6CE3653a091c7940fC98C3acb0043871', 'decimals': 9},
    'HUM': {'address': '0x07327a00ba28D413f745C931bbe6bE053B0AD2a6', 'decimals': 18},
    'ALPHA': {'address': '0xa1faa113cbE53436Df28FF0aEe54275c13B40975', 'decimals': 18},
    'ECTE': {'address': '0xe9fa21E671BcfB04E6868784b89C19d5aa2424Ea', 'decimals': 18},
    'BLUR': {'address': '0x5283D291DBCF85356A21bA090E6db59121208b44', 'decimals': 18},
    'MM': {'address': '0xC3c221fE28c33814c28c822b631fd76047eF1a63', 'decimals': 18},
    'SPACE': {'address': '0xcc7ab8d78dBA187dC95bF3bB86e65E0C26d0041f', 'decimals': 18},
    'XJP': {'address': '0x604026696fDB3C6720AE3049C46d59AC604dEA0A', 'decimals': 18},
    'ASNX': {'address': '0x328C4c80BC7aCa0834Db37e6600A6c49E12Da4DE', 'decimals': 18},
    'KVERSE': {'address': '0x0AB39AC604f992aAeC3C36dE337c3CD3917a7d26', 'decimals': 18},
    'CNHT': {'address': '0x6E109E9dD7Fa1a58BC3eff667e8e41fC3cc07AEF', 'decimals': 6},
    'DMT': {'address': '0x79126d32a86e6663F3aAac4527732d0701c1AE6c', 'decimals': 18},
    'GRNC': {'address': '0x37d299eF5548BF19503Cb72C085799e331F9097d', 'decimals': 18},
    'TR3': {'address': '0xF61BF4d1A948487d61b8fa63808AAC06bda55F98', 'decimals': 18},
    'DBD': {'address': '0xA5f1DbB0e55bc31f32c6d032BeE330288490e722', 'decimals': 18},
    'PAF': {'address': '0x6a969D379700B2E5EA4E684d273d63C1c050Ba49', 'decimals': 18},
    'OETH': {'address': '0x856c4Efb76C1D1AE02e20CEB03A2A6a08b0b8dC3', 'decimals': 18},
    'GOGO': {'address': '0x9a96E767bFCcE8E80370BE00821ED5BA283D4A17', 'decimals': 18},
    'NTB': {'address': '0xbE393AA534F82c0ffAC31BF06A23e283AcB3352B', 'decimals': 18},
    'KGC': {'address': '0xa8262Eb913FccEa4C3f77fc95b8b4043B384cFbB', 'decimals': 18},
    'MOCHI': {'address': '0x60EF10EDfF6D600cD91caeCA04caED2a2e605Fe5', 'decimals': 18},
    'CGT': {'address': '0xF5238462E7235c7B62811567E63Dd17d12C2EAA0', 'decimals': 8},
    'LIFE': {'address': '0x8640353CDc9778DEaB0dF45D12FB3013dEAC079C', 'decimals': 18},
    'ZERO': {'address': '0x0eC78ED49C2D27b315D462d43B5BAB94d2C79bf8', 'decimals': 18},
    'PSDNOCEAN': {'address': '0x51Fa2efd62ee56a493f24AE963eAce7D0051929E', 'decimals': 18},
    'VB': {'address': '0xCFe4EB08e33272d98cb31e37A7BE78d5C1b740c1', 'decimals': 18},
    'YAE': {'address': '0x4eE438be38F8682ABB089F2BFeA48851C5E71EAF', 'decimals': 18},
    'SNET': {'address': '0xFf19138b039D938db46bDDA0067DC4BA132ec71C', 'decimals': 8},
    'AGG': {'address': '0x492798Fb464e77Cb3cDA62b9a2C3c65162db198e', 'decimals': 18},
    'AAMMUNIUSDCWETH': {'address': '0x391E86e2C002C70dEe155eAceB88F7A3c38f5976', 'decimals': 18},
    'RUG': {'address': '0xDD782657af86492771BdB369CaEB8c246bE362f5', 'decimals': 12},
    'FNB': {'address': '0xE6D2c3cB986db66818c14C7032DB05D1d2A6ee74', 'decimals': 8},
    'MINDS': {'address': '0xB26631c6dda06aD89B93C71400D25692de89c068', 'decimals': 18},
    'DMR': {'address': '0xF74941046389c78F12fE44784C0EC9ca7CEb7dc2', 'decimals': 18},
    'GLO': {'address': '0x70c5f366dB60A2a0C59C4C24754803Ee47Ed7284', 'decimals': 18},
    'TSANGNYON': {'address': '0xd031edafac6a6ae5425e77F936022E506444C242', 'decimals': 18},
    'POC': {'address': '0x095cF7f3e82a1dcAdBF0FbC59023F419883eA296', 'decimals': 18},
    'STACY': {'address': '0xf12EC0D3Dab64DdEfBdC96474bDe25af3FE1B327', 'decimals': 18},
    'ARA': {'address': '0xa92E7c82B11d10716aB534051B271D2f6aEf7Df5', 'decimals': 18},
    'SRN': {'address': '0x68d57c9a1C35f63E2c83eE8e49A64e9d70528D25', 'decimals': 18},
    'SHIRD': {'address': '0x8916187112fd6e3624a80C42FA354Cd9756E695f', 'decimals': 9},
    'LOOKS': {'address': '0xf4d2888d29D722226FafA5d9B24F9164c092421E', 'decimals': 18},
    'ADAI': {'address': '0xfC1E690f61EFd961294b3e1Ce3313fBD8aa4f85d', 'decimals': 18},
    'REF': {'address': '0x89303500a7Abfb178B274FD89F2469C264951e1f', 'decimals': 8},
    'CRTS': {'address': '0x678e840C640F619E17848045D23072844224dD37', 'decimals': 18},
    'SRM': {'address': '0x99B600D0a4abdbc4a6796225a160bCf3D5Ce2a89', 'decimals': 18},
    'H2O': {'address': '0x0642026E7f0B6cCaC5925b4E7Fa61384250e1701', 'decimals': 18},
    'CLNY': {'address': '0x3E828ac5C480069D4765654Fb4b8733b910b13b2', 'decimals': 18},
    'PLA': {'address': '0x307d45Afbb7E84F82ef3D251A6bb0F00Edf632E4', 'decimals': 18},
    'NTK': {'address': '0x69BEaB403438253f13b6e92Db91F7FB849258263', 'decimals': 18},
    'MTAO': {'address': '0x1E8E29CA51363D923725aB9DaC73Bd7e9C440f71', 'decimals': 9},
    'MSTO': {'address': '0x8D26ccA0D58913932157dD4b294d4C95066172dF', 'decimals': 18},
    'DIVI': {'address': '0x246908BfF0b1ba6ECaDCF57fb94F6AE2FcD43a77', 'decimals': 8},
    'WISE': {'address': '0x66a0f676479Cee1d7373f3DC2e2952778BfF5bd6', 'decimals': 18},
    'SRLTY': {'address': '0x142a774E8b52550E88E196CedD7A5835acB646D0', 'decimals': 9},
    'PUGL': {'address': '0xdb69c3f8977f78952303987E87153c1988CF65a9', 'decimals': 18},
    'BFLY': {'address': '0xf680429328caaaCabee69b7A9FdB21a71419c063', 'decimals': 18},
    'NTRS': {'address': '0xeCcf15A4B5976a1365BAeD5297058B4cA42777C0', 'decimals': 18},
    'SAKURA': {'address': '0xb208b1a5664574F72d22E9e9981837310cACFeD6', 'decimals': 9},
    'WPT': {'address': '0x4FD51Cb87ffEFDF1711112b5Bd8aB682E54988eA', 'decimals': 18},
    'LUD': {'address': '0xe64b47931f28f89Cc7A0C6965Ecf89EaDB4975f5', 'decimals': 18},
    'NFUP': {'address': '0x26CBC7008cD879f4B63B69a915378f2D9b17bBF0', 'decimals': 18},
    'UTK': {'address': '0xdc9Ac3C20D1ed0B540dF9b1feDC10039Df13F99c', 'decimals': 18},
    'TRIBL': {'address': '0x6988A804C74Fd04F37DA1EA4781CEa68c9C00F86', 'decimals': 18},
    'ORBN': {'address': '0xA7fBd9254f10F8e20a31a593C9E8BC0D041e15F6', 'decimals': 9},
    'METIS': {'address': '0x9E32b13ce7f2E80A01932B42553652E053D6ed8e', 'decimals': 18},
    'CDS': {'address': '0x3c48Ca59bf2699E51d4974d4B6D284AE52076e5e', 'decimals': 18},
    'ARNX': {'address': '0x0C37Bcf456bC661C14D596683325623076D7e283', 'decimals': 18},
    'DEFX': {'address': '0x5F474906637bdCDA05f29C74653F6962bb0f8eDa', 'decimals': 18},
    'CMS': {'address': '0x1f22AD99C334e6689bAe751CC225bCC38fe54F82', 'decimals': 18},
    'ADX': {'address': '0xADE00C28244d5CE17D72E40330B1c318cD12B7c3', 'decimals': 18},
    'ZENI': {'address': '0x9c6666d5ff4B53b5eB3bd866664c15d0BFcECaa7', 'decimals': 18},
    'TSL': {'address': '0x03806Ce5ef69Bd9780EDFb04c29da1F23Db96294', 'decimals': 18},
    'SETH': {'address': '0x5e74C9036fb86BD7eCdcb084a0673EFc32eA31cb', 'decimals': 18},
    'QUID': {'address': '0x9d38F670D15C14716BE1F109a4F453E966A2B6D4', 'decimals': 9},
    'CPT': {'address': '0x88d50B466BE55222019D71F9E8fAe17f5f45FCA1', 'decimals': 8},
    'BDT': {'address': '0x4Efe8665e564bF454cCF5C90Ee16817F7485d5Cf', 'decimals': 18},
    'VERSE': {'address': '0x7aE0d42f23C33338dE15bFa89C7405c068d9dC0a', 'decimals': 18},
    'WBI': {'address': '0x17Ce5A8187c817D659B21083455455A092a173BB', 'decimals': 18},
    'SHINJI': {'address': '0x1F31DD60b84ea2a273cfC56876437E069aE80afd', 'decimals': 9},
    'WOETH': {'address': '0xDcEe70654261AF21C44c093C300eD3Bb97b78192', 'decimals': 18},
    'JCG': {'address': '0xbE601dD49da9EE1d2F64D422c4AECf8EB83c119f', 'decimals': 18},
    'UNCL': {'address': '0x2f4eb47A1b1F4488C71fc10e39a4aa56AF33Dd49', 'decimals': 18},
    'MERC': {'address': '0xa203eB78Fee91c8459C6d4eF3a899d8724Ee5B35', 'decimals': 18},
    'SHIBGUN': {'address': '0xa3BA969DD831D4eFf2bAF930e7F66D2eef5D929c', 'decimals': 18},
    'ESG': {'address': '0x20cD2E7Ec8F5d8b337fe46a4F565Ccef1561b9a9', 'decimals': 18},
    'JOB': {'address': '0xdfbc9050F5B01DF53512DCC39B4f2B2BBaCD517A', 'decimals': 8},
    'DOSE': {'address': '0xb31eF9e52d94D4120eb44Fe1ddfDe5B4654A6515', 'decimals': 18},
    'CON': {'address': '0x40d2025Ed2E89632D3A41d8541DF9Ed2AC0e2b1c', 'decimals': 18},
    'MO': {'address': '0x4fD5b9B5dcc9a5D5931d007ba4aE573e760d9B64', 'decimals': 18},
    'JSHIBA': {'address': '0x1426cC6D52D1B14e2B3b1Cb04d57ea42B39c4c7c', 'decimals': 9},
    'DOKI': {'address': '0x869DBe51Dc214fcb663604b0F7B548592F8C71f5', 'decimals': 9},
    'VIS': {'address': '0x469084939d1c20Fae3C73704FE963941C51bE863', 'decimals': 18},
    'ONIGIRI': {'address': '0xcf9c692F7e62Af3c571D4173fd4ABf9A3E5330D0', 'decimals': 18},
    'HAO': {'address': '0x3C4008eca800Ec1283E4cf500E68D06BFaBc00a8', 'decimals': 18},
    'SNTVT': {'address': '0x7865af71cf0b288b4E7F654f4F7851EB46a2B7F8', 'decimals': 18},
    '1MIL': {'address': '0xa4eF4b0B23C1fc81d3f9ecF93510e64f58A4A016', 'decimals': 18},
    'UMB': {'address': '0x6fC13EACE26590B80cCCAB1ba5d51890577D83B2', 'decimals': 18},
    'QLINDO': {'address': '0xC18c07a18198A6340cf4d94855fE5eb6DD33b46E', 'decimals': 0},
    'BAS': {'address': '0x106538CC16F938776c7c180186975BCA23875287', 'decimals': 18},
    'AAC': {'address': '0xe75ad3aAB14E4B0dF8c5da4286608DaBb21Bd864', 'decimals': 5},
    'TRDL': {'address': '0x297D33e17e61C2Ddd812389C2105193f8348188a', 'decimals': 18},
    'TRENDX': {'address': '0xC89d9aa9D9E54bb196319c6195AEA1038d2bc936', 'decimals': 18},
    'DAF': {'address': '0x5E031a5bC1c6b6E86A49e0B0F2e757800F1D0FFf', 'decimals': 18},
    'STA': {'address': '0x06874F973Dc3c96dc22A10eF0D0609F877f335EA', 'decimals': 18},
    'WG0': {'address': '0xa10740ff9FF6852eac84cdcfF9184e1D6d27C057', 'decimals': 18},
    'DBOMB': {'address': '0x26A9c0c6F28ae2e69270BC39F13dB87A27Db4CE5', 'decimals': 18},
    'EBOX': {'address': '0x33840024177A7DacA3468912363BeD8b425015c5', 'decimals': 18},
    'BGAN': {'address': '0xc3B5284B2c0cfa1871a6ac63B6d6ee43c08BDC79', 'decimals': 18},
    'FLOKI': {'address': '0xcf0C122c6b73ff809C693DB761e7BaeBe62b6a2E', 'decimals': 9},
    'CCP': {'address': '0xc8d07671aFbA9492Da95819De4AD10859e00aB7F', 'decimals': 18},
    'NEWS': {'address': '0xc39a048353842CAf1B94ce5967A04149Bb435b98', 'decimals': 9},
    'RFX': {'address': '0x159A1dFAe19057de57dFfFcbB3DA1aE784678965', 'decimals': 8},
    'MT': {'address': '0x9b4e2B4B13d125238Aa0480dD42B4f6fC71b37CC', 'decimals': 18},
    'COGE': {'address': '0xC382e04099A435439725BB40647e2B32dC136806', 'decimals': 18},
    'RUNE': {'address': '0x3155BA85D5F96b2d030a4966AF206230e46849cb', 'decimals': 18},
    'MAWETH': {'address': '0x490BBbc2485e99989Ba39b34802faFa58e26ABa4', 'decimals': 18},
    'SWUSD': {'address': '0x77C6E4a580c0dCE4E5c7a17d0bc077188a83A059', 'decimals': 18},
    'TEMCO': {'address': '0x2Fc246aA66F0da5bB1368F688548ecBBE9bdee5d', 'decimals': 18},
    'TKN': {'address': '0xaAAf91D9b90dF800Df4F55c205fd6989c977E73a', 'decimals': 8},
    'DIA': {'address': '0x84cA8bc7997272c7CfB4D0Cd3D55cd942B3c9419', 'decimals': 18},
    'SYLO': {'address': '0xf293d23BF2CDc05411Ca0edDD588eb1977e8dcd4', 'decimals': 18},
    'KMASK': {'address': '0xc3CaB0d0918c39E29088c495a077EcD022758798', 'decimals': 18},
    'NFT': {'address': '0xcB8d1260F9c92A3A545d409466280fFdD7AF7042', 'decimals': 18},
    'LSD': {'address': '0xfAC77A24E52B463bA9857d6b758ba41aE20e31FF', 'decimals': 18},
    'FF': {'address': '0x7E9D8f07A64e363e97A648904a89fb4cd5fB94CD', 'decimals': 18},
    'TAXHAVENINU': {'address': '0x664C6E221c77313307467B121528ad563107bD01', 'decimals': 9},
    'WSTR': {'address': '0xF0dC76C22139ab22618dDFb498BE1283254612B1', 'decimals': 18},
    'FINT': {'address': '0xd61143652aF94f570C7D9429356662dD859cA6EC', 'decimals': 18},
    'DOV': {'address': '0xac3211a5025414Af2866FF09c23FC18bc97e79b1', 'decimals': 18},
    'USDT': {'address': '0xdAC17F958D2ee523a2206206994597C13D831ec7', 'decimals': 6},
    'RI': {'address': '0x37E808F084101F75783612407e7C3f5F92d8ee3F', 'decimals': 18},
    'VEST': {'address': '0x1f19f83fC9a25F3C861260143E36c17706257986', 'decimals': 18},
    'BCPT': {'address': '0x1c4481750daa5Ff521A2a7490d9981eD46465Dbd', 'decimals': 18},
    'MAUSDC': {'address': '0xA5269A8e31B93Ff27B887B56720A25F844db0529', 'decimals': 18},
    'SHINO': {'address': '0xE9287543684f440b2f29983A3472987bF7b0ed12', 'decimals': 18},
    'PAR': {'address': '0x68037790A0229e9Ce6EaA8A99ea92964106C4703', 'decimals': 18},
    'AAMMUNIBATWETH': {'address': '0xA1B0edF4460CC4d8bFAA18Ed871bFF15E5b57Eb4', 'decimals': 18},
    'PRT': {'address': '0x6D0F5149c502faf215C89ab306ec3E50b15e2892', 'decimals': 18},
    'INNIT': {'address': '0x62A8C2818BD7034DC24CD22368C3E53E8eB47c18', 'decimals': 18},
    'ELEV': {'address': '0x045109cF1Be9eDEC048AA0B3D7a323154a1aEA65', 'decimals': 18},
    'SOL': {'address': '0xD31a59c85aE9D8edEFeC411D448f90841571b89c', 'decimals': 9},
    'AID': {'address': '0x37E8789bB9996CaC9156cD5F5Fd32599E6b91289', 'decimals': 18},
    'NPXS': {'address': '0xA15C7Ebe1f07CaF6bFF097D8a589fb8AC49Ae5B3', 'decimals': 18},
    'CAAVE': {'address': '0xe65cdB6479BaC1e22340E4E755fAE7E509EcD06c', 'decimals': 8},
    'HTB': {'address': '0x6be61833FC4381990e82D7D4a9F4c9B3F67eA941', 'decimals': 18},
    'FLUFFY': {'address': '0xc2a2E9B87b37938D934Fd51CfEC0C5Fd5cB14c31', 'decimals': 18},
    'FUZE': {'address': '0x187D1018E8ef879BE4194d6eD7590987463eAD85', 'decimals': 18},
    'DWEB': {'address': '0xE7f58A92476056627f9FdB92286778aBd83b285F', 'decimals': 18},
    'BALPHA': {'address': '0x7a5ce6abD131EA6B148a022CB76fc180ae3315A6', 'decimals': 18},
    'AMM': {'address': '0x8B1F49491477e0fB46a29fef53F1EA320D13c349', 'decimals': 6},
    'CBC': {'address': '0x790bFaCaE71576107C068f494c8A6302aea640cb', 'decimals': 18},
    'LOOT': {'address': '0x721A1B990699eE9D90b6327FaaD0A3E840aE8335', 'decimals': 18},
    'SPANK': {'address': '0x42d6622deCe394b54999Fbd73D108123806f6a18', 'decimals': 18},
    'PUSSY': {'address': '0x9196E18Bc349B1F64Bc08784eaE259525329a1ad', 'decimals': 18},
    'GDAO': {'address': '0x515d7E9D75E2b76DB60F8a051Cd890eBa23286Bc', 'decimals': 18},
    '0XBTC': {'address': '0xB6eD7644C69416d67B522e20bC294A9a9B405B31', 'decimals': 8},
    'UNCX': {'address': '0xaDB2437e6F65682B85F814fBc12FeC0508A7B1D0', 'decimals': 18},
    'SETS': {'address': '0x04E0Af0af1b7f0023c6B12af5a94Df59B0e8cF59', 'decimals': 18},
    'KENNEL': {'address': '0x25d4e7fe624FcC6E7F0b52D142F40c9ACEcF039a', 'decimals': 18},
    'LDO': {'address': '0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32', 'decimals': 18},
    'XFUND': {'address': '0x892A6f9dF0147e5f079b0993F486F9acA3c87881', 'decimals': 9},
    'GMT': {'address': '0x7Ddc52c4De30e94Be3A6A0A2b259b2850f421989', 'decimals': 18},
    'BMP': {'address': '0x01b23286FF60a543ec29366aE8D6B6274cA20541', 'decimals': 18},
    'PAPER': {'address': '0x21018CBC9ad730542130bE180b577b74DB2a9397', 'decimals': 18},
    'WJAURA': {'address': '0x198d7387Fa97A73F05b8578CdEFf8F2A1f34Cd1F', 'decimals': 18},
    'MOH': {'address': '0xa59e341e8047498700eD244814b01b34547fb21B', 'decimals': 18},
    'SANI': {'address': '0x4521C9aD6A3D4230803aB752Ed238BE11F8B342F', 'decimals': 18},
    'ALIEN': {'address': '0xf42965f82f9e3171d1205C5e3058Caf324a09432', 'decimals': 9},
    'DOGGER': {'address': '0xb3Cc3d7E656893F22d2372b0aE57106F6b155Cbe', 'decimals': 18},
    'FLETA': {'address': '0x40897C872214303b6F479a37E549eE1516B264A2', 'decimals': 18},
    'KLTR': {'address': '0xa92c49C403386111C1629AEe00936eED2A9E74a6', 'decimals': 18},
    'XHT': {'address': '0xD3c625F54dec647DB8780dBBe0E880eF21BA4329', 'decimals': 18},
    'CCV2': {'address': '0x612E1726435fE38dD49A0B35b4065B56f49c8F11', 'decimals': 18},
    'ASTA': {'address': '0xf2ddae89449b7D26309a5D54614B1FC99C608AF5', 'decimals': 18},
    'S4F': {'address': '0xAec7d1069e3a914a3EB50f0BFB1796751f2ce48a', 'decimals': 18},
    'DNC': {'address': '0x0fbe9Cc858D7AD6e246Fe9D01Aed22abD2A66f0B', 'decimals': 18},
    'TBT': {'address': '0x07Ac55797D4F43f57cA92a49E65ca582cC287c27', 'decimals': 18},
    'FTX': {'address': '0xd559f20296FF4895da39b5bd9ADd54b442596a61', 'decimals': 18},
    'TBTC': {'address': '0x18084fbA666a33d37592fA2633fD49a74DD93a88', 'decimals': 18},
    'RENA': {'address': '0x56de8BC61346321D4F2211e3aC3c0A7F00dB9b76', 'decimals': 18},
    'GR': {'address': '0xcE593a29905951E8Fc579bC092ecA72577dA575c', 'decimals': 6},
    'ZSC': {'address': '0x7A41e0517a5ecA4FdbC7FbebA4D4c47B9fF6DC63', 'decimals': 18},
    'GEM': {'address': '0x9008064E6cf73e27a3Aba4B10E69f855A4F8efcC', 'decimals': 18},
    'TRADE': {'address': '0x6e5970DBd6fc7eb1f29C6D2eDF2bC4c36124C0C1', 'decimals': 18},
    'BUSY': {'address': '0x5CB3ce6D081fB00d5f6677d196f2d70010EA3f4a', 'decimals': 18},
    'CE': {'address': '0x8F12Dfc7981DE79A8A34070a732471f2D335EecE', 'decimals': 18},
    'ERSDL': {'address': '0x5218E472cFCFE0b64A064F055B43b4cdC9EfD3A6', 'decimals': 18},
    'MAGE': {'address': '0xd52AAE39a2b5CC7812F7B9450EBB61DfEf702B15', 'decimals': 18},
    'FOAM': {'address': '0x4946Fcea7C692606e8908002e55A582af44AC121', 'decimals': 18},
    'XY': {'address': '0x77777777772cf0455fB38eE0e75f38034dFa50DE', 'decimals': 18},
    'MOETA': {'address': '0x238A1FcC2F65ed50c1e1797cF5857CBA3E82a55B', 'decimals': 9},
    'RND': {'address': '0x1c7E83f8C581a967940DBfa7984744646AE46b29', 'decimals': 18},
    'NST': {'address': '0x70Bef3bB2f001dA2fDDb207dAe696cD9FAFf3f5d', 'decimals': 18},
    'WINRY': {'address': '0x1a87077C4F834884691B8ba4fc808D2eC93A9F30', 'decimals': 9},
    'OPIUM': {'address': '0x888888888889C00c67689029D7856AAC1065eC11', 'decimals': 18},
    'BOOP': {'address': '0x8013266cb5c9dd48bE3Ad7D1CE832874d64B3Ce1', 'decimals': 18},
    'CODE': {'address': '0xb24cd494faE4C180A89975F1328Eab2a7D5d8f11', 'decimals': 18},
    'OHM': {'address': '0x64aa3364F17a4D01c6f1751Fd97C2BD3D7e7f1D5', 'decimals': 9},
    'FDZ': {'address': '0x23352036E911A22Cfc692B5E2E196692658ADED9', 'decimals': 18},
    'PROPS': {'address': '0x6fe56C0bcdD471359019FcBC48863d6c3e9d4F41', 'decimals': 18},
    'ILSI': {'address': '0x0acC0FEE1D86D2cD5AF372615bf59b298D50cd69', 'decimals': 18},
    'TOAD': {'address': '0xBfB2b6870501a6Ff17121D676A0A45a38c9eeD1e', 'decimals': 9},
    'SMOL': {'address': '0x553539d40AE81FD0d9C4b991B0b77bE6f6Bc030e', 'decimals': 18},
    'IDEX': {'address': '0xB705268213D593B8FD88d3FDEFF93AFF5CbDcfAE', 'decimals': 18},
    'TWT': {'address': '0xd4Df22556e07148e591B4c7b4f555a17188CF5cF', 'decimals': 9},
    'BLACK': {'address': '0xd714d91A169127e11D8FAb3665d72E8b7ef9Dbe2', 'decimals': 18},
    'SUPERBID': {'address': '0x0563DCe613D559a47877fFD1593549fb9d3510D6', 'decimals': 18},
    'STBT': {'address': '0x530824DA86689C9C17CdC2871Ff29B058345b44a', 'decimals': 18},
    'ZEUS10000': {'address': '0xcbC1Aa6961B28cCcb11E1cdDd84Ad8B42Ec920b1', 'decimals': 4},
    'RPL': {'address': '0xD33526068D116cE69F19A9ee46F0bd304F21A51f', 'decimals': 18},
    'PUMLX': {'address': '0x8c088775e4139af116Ac1FA6f281Bbf71E8c1c73', 'decimals': 18},
    'BOX': {'address': '0x045414e728067Ab3da4bCeafC0D992D59183463A', 'decimals': 18},
    'MBASE': {'address': '0xbA3E5F8b4200a5eb856FF2C3E001aB29444491AA', 'decimals': 18},
    'BLUSD': {'address': '0xB9D7DdDca9a4AC480991865EfEf82E01273F79C3', 'decimals': 18},
    'HONR': {'address': '0x39Ea10E507720783C27eDd5F96Bf2D6e199579B8', 'decimals': 18},
    'AUDT': {'address': '0xB90cb79B72EB10c39CbDF86e50B1C89F6a235f2e', 'decimals': 18},
    'HEAL': {'address': '0x383B0b2ce2e1757b5e1D087D8f36013eA595541a', 'decimals': 9},
    'USDX': {'address': '0x2F6081E3552b1c86cE4479B80062A1ddA8EF23E3', 'decimals': 9},
    'MTLX': {'address': '0x2e1E15C44Ffe4Df6a0cb7371CD00d5028e571d14', 'decimals': 18},
    'AFIN': {'address': '0xEe9E5eFF401ee921b138490d00CA8D1F13f67A72', 'decimals': 8},
    'MEISHU': {'address': '0x406B9DCA8b52f08385014eC1eD1cf6A0D5c01289', 'decimals': 9},
    'BXB': {'address': '0xB69d946D8cdC5238B0658188e1baBB41b8652A29', 'decimals': 18},
    'ICHI': {'address': '0x903bEF1736CDdf2A537176cf3C64579C3867A881', 'decimals': 9},
    'MIX': {'address': '0x5d285F735998F36631F678FF41fb56A10A4d0429', 'decimals': 18},
    'XDATA': {'address': '0x0Cf0Ee63788A0849fE5297F3407f701E122cC023', 'decimals': 18},
    'DATA': {'address': '0x33d63Ba1E57E54779F7dDAeaA7109349344cf5F1', 'decimals': 18},
    'VIDT': {'address': '0x3BE7bF1A5F23BD8336787D0289B70602f1940875', 'decimals': 18},
    'AUC': {'address': '0xc12d099be31567add4e4e4d0D45691C3F58f5663', 'decimals': 18},
    'INUS': {'address': '0x39207D2E2fEEF178FBdA8083914554C59D9f8c00', 'decimals': 18},
    'QUA': {'address': '0x4dAeb4a06F70f4b1A5C329115731fE4b89C0B227', 'decimals': 18},
    'STF': {'address': '0x1F4cb968b76931C494Ff92ED80CCB169ad641cb1', 'decimals': 18},
    'UM': {'address': '0xb19Dd661F076998e3B0456935092a233e12C2280', 'decimals': 18},
    'SHI': {'address': '0x243cACb4D5fF6814AD668C3e225246efA886AD5a', 'decimals': 18},
    'INVECTAI': {'address': '0xCAE7F3aa6D456463bEf0E5B508542b69D96f2904', 'decimals': 8},
    'SDL': {'address': '0xf1Dc500FdE233A4055e25e5BbF516372BC4F6871', 'decimals': 18},
    'AMON': {'address': '0x00059AE69c1622A7542EdC15E8d17b060fE307b6', 'decimals': 18},
    'GNO': {'address': '0x6810e776880C02933D47DB1b9fc05908e5386b96', 'decimals': 18},
    'FNC': {'address': '0x7f280daC515121DcdA3EaC69eB4C13a52392CACE', 'decimals': 18},
    'SILV2': {'address': '0x7E77dCb127F99ECe88230a64Db8d595F31F1b068', 'decimals': 18},
    'CSX': {'address': '0x3aadc3BCe49724ce299fD9F3850211211c399C8c', 'decimals': 18},
    'BSGG': {'address': '0x69570f3E84f51Ea70b7B68055c8d667e77735a25', 'decimals': 18},
    'UUSDT': {'address': '0x178Bf8fD04b47D2De3eF3f6b3D112106375ad584', 'decimals': 6},
    'NRFB': {'address': '0x00281dFCe4cfD72c0b6FDA2AAaF077258743f9E8', 'decimals': 0},
    'ZINU': {'address': '0xC5FDf3569af74f3B3e97E46A187a626352D2d508', 'decimals': 9},
    'MITX': {'address': '0x4a527d8fc13C5203AB24BA0944F4Cb14658D1Db6', 'decimals': 18},
    'PINU': {'address': '0x3a1311B8C404629E38f61D566cefEFed083B9670', 'decimals': 9},
    'AAMMUNIMKRWETH': {'address': '0x370adc71f67f581158Dc56f539dF5F399128Ddf9', 'decimals': 18},
    'MARVIN': {'address': '0xcAa9Ed6D7502595B555113D4517668aE24038C8a', 'decimals': 18},
    'PLA': {'address': '0x0198f46f520F33cd4329bd4bE380a25a90536CD5', 'decimals': 18},
    'OPCH': {'address': '0x4d4d883f920f7c0C36a1bE71a02Aa0cde2AA22d1', 'decimals': 18},
    'DXD': {'address': '0xa1d65E8fB6e87b60FECCBc582F7f97804B725521', 'decimals': 18},
    'PYR': {'address': '0x430EF9263E76DAE63c84292C3409D61c598E9682', 'decimals': 18},
    'AZA': {'address': '0x56322c14cb5F53531c231b8F9Ab178bf5d893A5e', 'decimals': 18},
    'VST': {'address': '0xb7C2fcD6d7922eddd2A7A9B0524074A60D5b472C', 'decimals': 18},
    'CIMO': {'address': '0xC3f65b1c45B0D42a472CD0Fbd37dD6678AEf667a', 'decimals': 9},
    'DDX': {'address': '0x3A880652F47bFaa771908C07Dd8673A787dAEd3A', 'decimals': 18},
    'DNA': {'address': '0xef6344de1fcfC5F48c30234C16c1389e8CdC572C', 'decimals': 18},
    'DAD': {'address': '0x5B322514FF727253292637D9054301600c2C81e8', 'decimals': 9},
    'GAMMA': {'address': '0x6BeA7CFEF803D1e3d5f7C0103f7ded065644e197', 'decimals': 18},
    'HUSD': {'address': '0xdF574c24545E5FfEcb9a659c229253D4111d87e1', 'decimals': 8},
    'JIYUU': {'address': '0x77D9046ee15FAACeB89439FFdDE4be071C2F07bD', 'decimals': 18},
    'PST': {'address': '0xE3feDAeCD47aa8EAb6b23227b0eE56F092C967a9', 'decimals': 18},
    'HIBEANZ': {'address': '0x23ddBD36547d43627afa9B42d4e9FB0515F48B09', 'decimals': 18},
    'TST': {'address': '0xa0b93B9e90aB887E53F9FB8728c009746e989B53', 'decimals': 18},
    'MIYAZAKI': {'address': '0xD5e7D22362BCC9881D06512d3189eAe79DD98d70', 'decimals': 9},
    'BRD': {'address': '0x558EC3152e2eb2174905cd19AeA4e34A23DE9aD6', 'decimals': 18},
    'NFP': {'address': '0x299698B4B44BD6d023981A7317798dEe12860834', 'decimals': 9},
    'PROXY': {'address': '0x82089a9c7c4a07352F7433fBce1D4Ee9a722fF29', 'decimals': 9},
    'BAC': {'address': '0x3449FC1Cd036255BA1EB19d65fF4BA2b8903A69a', 'decimals': 18},
    'AKNC': {'address': '0x9D91BE44C06d373a8a226E1f3b146956083803eB', 'decimals': 18},
    'EGG': {'address': '0x65cCD72c0813CE6f2703593B633202a0F3Ca6a0c', 'decimals': 18},
    'DAISY': {'address': '0x40955D77F87123b71b145098358A60573ac7BE96', 'decimals': 18},
    'IZE': {'address': '0x6944d3e38973C4831dA24E954fbD790c7E688bDd', 'decimals': 18},
    'A5T': {'address': '0xe8272210954eA85DE6D2Ae739806Ab593B5d9c51', 'decimals': 18},
    'OVR': {'address': '0x21BfBDa47A0B4B5b1248c767Ee49F7caA9B23697', 'decimals': 18},
    'DTUBE': {'address': '0xd2be3722B17b616c51ed9B8944A227D1ce579C24', 'decimals': 2},
    'DEB': {'address': '0xA1E770be76BDe604F8Ebb66f640250a787B9422B', 'decimals': 18},
    'MARU': {'address': '0xC2456D2118299a2efDFE6522Ff79aA48fC5d2b00', 'decimals': 9},
    'KDAG': {'address': '0x95E40E065AFB3059dcabe4aaf404c1F92756603a', 'decimals': 18},
    'AZK': {'address': '0x5408245a4D7c685F59cE6D3b8B35916dd6c11A99', 'decimals': 18},
    'MINT': {'address': '0x0CDF9acd87E940837ff21BB40c9fd55F68bba059', 'decimals': 18},
    'BAX': {'address': '0xf920e4F3FBEF5B3aD0A25017514B769bDc4Ac135', 'decimals': 18},
    'CTSI': {'address': '0x491604c0FDF08347Dd1fa4Ee062a822A5DD06B5D', 'decimals': 18},
    'EGT': {'address': '0x450E7F6e3a2f247A51b98C39297a9a5BfbDb3170', 'decimals': 9},
    'WXRP': {'address': '0x39fBBABf11738317a448031930706cd3e612e1B9', 'decimals': 18},
    'DILI': {'address': '0x37F74e99794853777a10ea1dc08a64C86958F06a', 'decimals': 18},
    'WTK': {'address': '0x4cfF49d0A19ed6Ff845a9122fa912AbcFB1F68A6', 'decimals': 18},
    'TALK': {'address': '0xCAabCaA4ca42e1d86dE1a201c818639def0ba7A7', 'decimals': 18},
    'UDO': {'address': '0xea3983Fc6D0fbbC41fb6F6091f68F3e08894dC06', 'decimals': 18},
    'HIAZUKI': {'address': '0x4DB26817c88A57D2ef84cFd90168f535Ee83bDA9', 'decimals': 18},
    'THE9': {'address': '0xEFd113CeA2f656Fe7899A25a06243A6E40406E08', 'decimals': 18},
    'ROOBEE': {'address': '0xA31B1767e09f842ECFd4bc471Fe44F830E3891AA', 'decimals': 18},
    'WSIENNA': {'address': '0x9b00e6E8D787b13756eb919786c9745054DB64f9', 'decimals': 18},
    'SCA': {'address': '0x1FbD3dF007eB8A7477A1Eab2c63483dCc24EfFD6', 'decimals': 18},
    'EGS': {'address': '0xB009BFaAF85e53F55d8657781Eb69feAaed83672', 'decimals': 18},
    'ACXT': {'address': '0x7bE00ed6796B21656732E8f739Fc1b8F1C53DA0D', 'decimals': 18},
    'LOON': {'address': '0x7C5d5100B339Fe7D995a893AF6CB496B9474373c', 'decimals': 18},
    'CUMINU': {'address': '0xd6327ce1fb9D6020E8C2c0E124A1eC23DCAb7536', 'decimals': 18},
    'SHIBMERICAN': {'address': '0x440238CC07186aDEA6653a2E8cb9a24737615609', 'decimals': 9},
    'EXD': {'address': '0x02dE007D412266a2e0Fa9287C103474170F06560', 'decimals': 18},
    'AGLD': {'address': '0x32353A6C91143bfd6C7d363B546e62a9A2489A20', 'decimals': 18},
    'ALEPH': {'address': '0x27702a26126e0B3702af63Ee09aC4d1A084EF628', 'decimals': 18},
    'KNC': {'address': '0xdeFA4e8a7bcBA345F687a2f1456F5Edd9CE97202', 'decimals': 18},
    'MAPS': {'address': '0x9A6aeA7C2b0D563EaB701E3255110d30F08B5a64', 'decimals': 8},
    'CTC': {'address': '0xa3EE21C306A700E682AbCdfe9BaA6A08F3820419', 'decimals': 18},
    'MITH': {'address': '0x3893b9422Cd5D70a81eDeFfe3d5A1c6A978310BB', 'decimals': 18},
    'FTT': {'address': '0x50D1c9771902476076eCFc8B2A83Ad6b9355a4c9', 'decimals': 18},
    'NU': {'address': '0x4fE83213D56308330EC302a8BD641f1d0113A4Cc', 'decimals': 18},
    'BABYPEPE': {'address': '0x5C559F3ee9A81Da83e069c0093471cB05D84052A', 'decimals': 18},
    'ARCAI': {'address': '0x5c8190b76E90B4DD0702740cF6Eb0F7eE01Ab5E9', 'decimals': 9},
    'AD': {'address': '0xe973E453977195422B48e1852A207b7eE9C913C7', 'decimals': 18},
    'IDLEUSDTSAFE': {'address': '0x28fAc5334C9f7262b3A3Fe707e250E01053e07b5', 'decimals': 18},
    'ECOX': {'address': '0xcccD1Ba9f7acD6117834E0D28F25645dECb1736a', 'decimals': 18},
    'GRANX': {'address': '0xbD668EC47e10D3bf23b1d68139E545162a0dfCFA', 'decimals': 18},
    'SKIN': {'address': '0x2bDC0D42996017fCe214b21607a515DA41A9E0C5', 'decimals': 6},
    'PHTK': {'address': '0x1FEE5588cb1De19c70B6aD5399152D8C643FAe7b', 'decimals': 18},
    'NFTFY': {'address': '0xBf6Ff49FfD3d104302Ef0AB0F10f5a84324c091c', 'decimals': 18},
    'ALERT': {'address': '0xaE377ed00DF9b9663B253bE93035EF1CfECBc1fB', 'decimals': 9},
    'CBC': {'address': '0x26DB5439F651CAF491A87d48799dA81F191bDB6b', 'decimals': 8},
    'BIT': {'address': '0x1A4b46696b2bB4794Eb3D4c26f1c55F9170fa4C5', 'decimals': 18},
    'BCDN': {'address': '0x1e797Ce986C3CFF4472F7D38d5C4aba55DfEFE40', 'decimals': 15},
    'BRWL': {'address': '0x4086E77C5E993FDB90a406285d00111a974F877a', 'decimals': 4},
    'RARE': {'address': '0xba5BDe662c17e2aDFF1075610382B9B691296350', 'decimals': 18},
    'SMTX': {'address': '0x102203517ce35AC5Cab9a2cda80DF03f26c7419b', 'decimals': 18},
    'POA': {'address': '0x1d2d542e6D9d85A712deB4D1a7D96a16CE00B8cE', 'decimals': 18},
    'LYFE': {'address': '0xd87de4cCef2c2FE651bc4d130cb1A365248f21fA', 'decimals': 18},
    'MATTER': {'address': '0x9B99CcA871Be05119B2012fd4474731dd653FEBe', 'decimals': 18},
    'FUSDT': {'address': '0xADc234a4e90E2045f353F5d4fCdE66144d23b458', 'decimals': 6},
    'HOODIE': {'address': '0xdffA3a7f5B40789C7A437DbE7B31b47F9B08FE75', 'decimals': 18},
    'CZRX': {'address': '0xB3319f5D18Bc0D84dD1b4825Dcde5d5f7266d407', 'decimals': 8},
    'JS': {'address': '0x5046E860ff274fb8c66106B0Ffb8155849fB0787', 'decimals': 8},
    'ETHER': {'address': '0xAd0891Abb1687Fb994a2C1c9699520422573996F', 'decimals': 18},
    'IDH': {'address': '0x5136C98A80811C3f46bDda8B5c4555CFd9f812F0', 'decimals': 6},
    'RING': {'address': '0x9469D013805bFfB7D3DEBe5E7839237e535ec483', 'decimals': 18},
    'TOTM': {'address': '0x6FF1BFa14A57594a5874B37ff6AC5efbD9F9599A', 'decimals': 18},
    'PVT': {'address': '0x7869c4A1a3f6F8684FBCC422a21aD7Abe3167834', 'decimals': 18},
    'POOF': {'address': '0x888ceA2BBDD5D47a4032cf63668D7525C74af57A', 'decimals': 18},
    'LEAN': {'address': '0xFb83869f7EE52AEBadDEA254F647953AfB1F99F8', 'decimals': 18},
    'TOMOE': {'address': '0x05D3606d5c81EB9b7B18530995eC9B29da05FaBa', 'decimals': 18},
    'HELLSING': {'address': '0xB087C2180e3134Db396977065817aed91FEa6EAD', 'decimals': 9},
    'BNPL': {'address': '0x84d821F7FbDD595c4C4A50842913e6b1E07d7a53', 'decimals': 18},
    'PPC': {'address': '0x044d078F1c86508e13328842Cc75AC021B272958', 'decimals': 6},
    'PERC': {'address': '0x60bE1e1fE41c1370ADaF5d8e66f07Cf1C2Df2268', 'decimals': 18},
    'L2': {'address': '0xBbff34E47E559ef680067a6B1c980639EEb64D24', 'decimals': 18},
    'RAMP': {'address': '0x33D0568941C0C64ff7e0FB4fbA0B11BD37deEd9f', 'decimals': 18},
    'BAYC': {'address': '0xEA47B64e1BFCCb773A0420247C0aa0a3C1D2E5C5', 'decimals': 18},
    'LIBFX': {'address': '0xc0ea83113038987d974FE667831a36E442e661E7', 'decimals': 18},
    'NOIA': {'address': '0xa8c8CfB141A3bB59FEA1E2ea6B79b5ECBCD7b6ca', 'decimals': 18},
    'BST': {'address': '0xD4f6f9Ae14399fD5Eb8DFc7725F0094a1A7F5d80', 'decimals': 18},
    'CMCX': {'address': '0x5B685863494C33F344081F75e5430C260c224A32', 'decimals': 18},
    'ORN': {'address': '0x0258F474786DdFd37ABCE6df6BBb1Dd5dfC4434a', 'decimals': 8},
    'MEVFREE': {'address': '0x1936C91190E901B7dD55229A574AE22B58Ff498a', 'decimals': 18},
    'KCAL': {'address': '0x47C1178F49140ECdBfbdF0aE2935cDB640D579F9', 'decimals': 10},
    'XYZ': {'address': '0x618679dF9EfCd19694BB1daa8D00718Eacfa2883', 'decimals': 18},
    'ITS': {'address': '0xC32cC5b70BEe4bd54Aa62B9Aefb91346d18821C4', 'decimals': 18},
    'WTAO': {'address': '0x77E06c9eCCf2E797fd462A92B6D7642EF85b0A44', 'decimals': 9},
    'LVL': {'address': '0xa8EdA9D4Aee0eb882F8752C6bA7e16d9233C9Ad2', 'decimals': 8},
    'TIC': {'address': '0x549E4D92285ff5A16c9484Ff79211E4358b1f202', 'decimals': 18},
    'MACRV': {'address': '0x9dc7094530cB1bcf5442c3b9389ee386738A190c', 'decimals': 18},
    'STA': {'address': '0xD7d05bDa4bf5876bA1254b3Eaaf8b47D2F5676eb', 'decimals': 18},
    'PLAY': {'address': '0x33e18a092a93ff21aD04746c7Da12e35D34DC7C4', 'decimals': 18},
    'FTG': {'address': '0x3A4CaB3DcfaB144FE7eB2B5a3e288Cc03dC07659', 'decimals': 18},
    'ENT': {'address': '0x675ABaBD3A210566A5e213547523b740Be80041A', 'decimals': 18},
    'CAG': {'address': '0x7d4b8Cce0591C9044a22ee543533b72E976E36C3', 'decimals': 18},
    'HEY': {'address': '0xE61F6e39711cEc14f8D6c637c2f4568bAA9FF7Ee', 'decimals': 18},
    'YVBOOST': {'address': '0x9d409a0A012CFbA9B15F6D4B36Ac57A46966Ab9a', 'decimals': 18},
    'EGGS': {'address': '0x2e516BA5Bf3b7eE47fb99B09eaDb60BDE80a82e0', 'decimals': 18},
    'USDB': {'address': '0x309627af60F0926daa6041B8279484312f2bf060', 'decimals': 18},
    'LOOM': {'address': '0x42476F744292107e34519F9c357927074Ea3F75D', 'decimals': 18},
    'DRT': {'address': '0x9AF4f26941677C706cfEcf6D3379FF01bB85D5Ab', 'decimals': 8},
    'TORA': {'address': '0x7a3f7F6675514D4d611b442A4B76752f6AB77670', 'decimals': 18},
    'ICE': {'address': '0xf16e81dce15B08F326220742020379B855B87DF9', 'decimals': 18},
    'NET': {'address': '0x77A9B858aA8417fd318ecEE01C1d6D9fEF31A71F', 'decimals': 18},
    'XEND': {'address': '0xE4CFE9eAa8Cdb0942A80B7bC68fD8Ab0F6D44903', 'decimals': 18},
    'RDN': {'address': '0x255Aa6DF07540Cb5d3d297f0D0D4D84cb52bc8e6', 'decimals': 18},
    'MADAI': {'address': '0x36F8d0D0573ae92326827C4a82Fe4CE4C244cAb6', 'decimals': 18},
    'MSI': {'address': '0xc4C75F2A0cB1a9ACc33929512dc9733EA1Fd6fde', 'decimals': 18},
    'BRZ': {'address': '0x420412E765BFa6d85aaaC94b4f7b708C89be2e2B', 'decimals': 4},
    'INUSANITY': {'address': '0xd73Cab6a16c67Cb79fc61F8c13DfEc4F32f9DCBF', 'decimals': 9},
    'DEFI++': {'address': '0x8D1ce361eb68e9E05573443C407D4A3Bed23B033', 'decimals': 18},
    'YOUC': {'address': '0x3D371413dd5489F3A04C07c0C2CE369c20986ceb', 'decimals': 10},
    'IDEAS': {'address': '0x443b29fc978058Abe3Fc2f4c3c6b76c57fdecC02', 'decimals': 18},
    'G': {'address': '0xE1005BfBBC9A17d5d844C7a4371CBF6B2B387380', 'decimals': 8},
    'GEC': {'address': '0xe304283C3e60Cefaf7eA514007Cf4E8fdC3d869d', 'decimals': 18},
    'HEEL': {'address': '0xf941D3AAbf2EE0F5589E68Ba6047b8329592B366', 'decimals': 9},
    'SAFEEARTH': {'address': '0xE6F1966d04CfcB9cd1B1dC4E8256D8b501b11CbA', 'decimals': 9},
    'LCS': {'address': '0xAA19961b6B858D9F18a115f25aa1D98ABc1fdBA8', 'decimals': 18},
    'ROCK': {'address': '0x1a6921F7B92113c79306f1A9E148D83445ac42BC', 'decimals': 9},
    'AGS': {'address': '0xdB2F2bCCe3efa95EDA95a233aF45F3e0d4f00e2A', 'decimals': 8},
    'ALCAZAR': {'address': '0x10f44a834097469AC340592d28c479c442E99bFe', 'decimals': 18},
    'KISHIMOTO': {'address': '0xAE36155a55F04a696b8362777620027882B31dB5', 'decimals': 9},
    'VNXLU': {'address': '0x00fC270C9cc13e878Ab5363D00354bebF6f05C15', 'decimals': 18},
    'COLLAR': {'address': '0x9783B81438C24848f85848f8df31845097341771', 'decimals': 18},
    'GPT': {'address': '0xA00425D3e2D3E9FF74F3e112B4D3A7978d7D88c2', 'decimals': 18},
    'AZRX': {'address': '0x6Fb0855c404E09c47C3fBCA25f08d4E41f9F062f', 'decimals': 18},
    'ALY': {'address': '0xf2CDF38e24738bA379ffa38D47bc88a941dF5627', 'decimals': 2},
    'BWT': {'address': '0xcA3Ea3061d638E02113aA960340c98343b5aCd62', 'decimals': 18},
    'ABL': {'address': '0xf8b358b3397a8ea5464f8cc753645d42e14b79EA', 'decimals': 18},
    'WABI': {'address': '0x286BDA1413a2Df81731D4930ce2F862a35A609fE', 'decimals': 18},
    'NAO': {'address': '0xd049206fB408a611E543791F2d8F102a8bC253dc', 'decimals': 18},
    'Y2B': {'address': '0xF9C12B27cE5058ab98ce11BD53900f84E18C0650', 'decimals': 18},
    'CRE': {'address': '0x115eC79F1de567eC68B7AE7eDA501b406626478e', 'decimals': 18},
    'BTC2X-FLI': {'address': '0x0B498ff89709d3838a063f1dFA463091F9801c2b', 'decimals': 18},
    'AAMMWETH': {'address': '0xf9Fb4AD91812b704Ba883B11d2B576E890a6730A', 'decimals': 18},
    'DFCH': {'address': '0xA92cDf011c895780733b2a6A6e15C4c1ed836148', 'decimals': 18},
    'BTR': {'address': '0xd433138d12beB9929FF6fd583DC83663eea6Aaa5', 'decimals': 18},
    'MTRM': {'address': '0xcd17fA52528f37FACB3028688E62ec82d9417581', 'decimals': 0},
    'ELON': {'address': '0x761D38e5ddf6ccf6Cf7c55759d5210750B5D60F3', 'decimals': 18},
    'ARX': {'address': '0x7d8DafF6d70CEAd12c6f077048552Cf89130A2B1', 'decimals': 18},
    'MTX': {'address': '0x0AF44e2784637218dD1D32A322D44e603A8f0c6A', 'decimals': 18},
    'BITO': {'address': '0x93b1E78a3e652cd2e71C4a767595B77282344932', 'decimals': 18},
    'RENQ': {'address': '0xff8C479134A18918059493243943150776cF8CF2', 'decimals': 18},
    'NAME': {'address': '0xe1A4c5BBb704a92599FEdB191f451E0d3a1ed842', 'decimals': 18},
    'TDP': {'address': '0x5b11aAcB6Bddb9ffab908FDCE739Bf4aed554327', 'decimals': 18},
    'XRUN': {'address': '0x5833dBB0749887174b254bA4a5df747FF523a905', 'decimals': 18},
    'SUKU': {'address': '0x0763fdCCF1aE541A5961815C0872A8c5Bc6DE4d7', 'decimals': 18},
    'PUSH': {'address': '0xf418588522d5dd018b425E472991E52EBBeEEEEE', 'decimals': 18},
    'NFTL': {'address': '0x3c8D2FCE49906e11e71cB16Fa0fFeB2B16C29638', 'decimals': 18},
    'ORBR': {'address': '0xdA30f261a962d5AAe94C9ecd170544600d193766', 'decimals': 18},
    'ME': {'address': '0x47140a767A861F7a1f3B0Dd22A2F463421c28814', 'decimals': 18},
    'XMX': {'address': '0x0f8c45B896784A1E408526B9300519ef8660209c', 'decimals': 8},
    'THN': {'address': '0x2E95Cea14dd384429EB3c4331B776c4CFBB6FCD9', 'decimals': 18},
    'ROOT': {'address': '0x77700005BEA4DE0A78b956517f099260C2CA9a26', 'decimals': 18},
    'FLP': {'address': '0x3a1Bda28AdB5B0a812a7CF10A1950c920F79BcD3', 'decimals': 18},
    'HDP': {'address': '0xc4d5545392f5Fc57EBa3AF8981815669bb7E2A48', 'decimals': 4},
    'CWD': {'address': '0x374fB05c96c36348b92E38fB088b26B8511e3b3D', 'decimals': 18},
    'RSR': {'address': '0x320623b8E4fF03373931769A31Fc52A4E78B5d70', 'decimals': 18},
    'PBR': {'address': '0x298d492e8c1d909D3F63Bc4A36C66c64ACB3d695', 'decimals': 18},
    'BMDA': {'address': '0x4236f8aaF2B1f3A28420Eb15B8e0ddF63201a95E', 'decimals': 18},
    'HYVE': {'address': '0xd794DD1CAda4cf79C9EebaAb8327a1B0507ef7d4', 'decimals': 18},
    'YCC': {'address': '0x37E1160184F7dD29f00b78C050Bf13224780b0B0', 'decimals': 8},
    'GEO': {'address': '0x147faF8De9d8D8DAAE129B187F0D02D819126750', 'decimals': 18},
    'COR': {'address': '0x9C2dc0c3CC2BADdE84B0025Cf4df1c5aF288D835', 'decimals': 18},
    'OLY': {'address': '0x6595b8fD9C920C81500dCa94e53Cdc712513Fb1f', 'decimals': 18},
    'DGVC': {'address': '0x26E43759551333e57F073bb0772F50329A957b30', 'decimals': 18},
    'UCM': {'address': '0x722f97A435278B7383a1e3c47F41773bebF3232C', 'decimals': 18},
    'BALTO': {'address': '0x5955f7d312c5d84B004d259D55072C4F0A478dBC', 'decimals': 18},
    'WXBTC': {'address': '0x5592C5Aa89492Ea918d55B804E177B5Ca7dCcd5a', 'decimals': 18},
    'DAPPT': {'address': '0x96184d9C811Ea0624fC30C80233B1d749B9E485B', 'decimals': 18},
    'HRTS': {'address': '0xCeB53519a2B891d33b36E3826962D2a48f878e45', 'decimals': 18},
    'IFARM': {'address': '0x1571eD0bed4D987fe2b498DdBaE7DFA19519F651', 'decimals': 18},
    'HISEALS': {'address': '0x286F851b049CcCE1419e09b6468dC3297F86A703', 'decimals': 18},
    'TOR': {'address': '0x35073CA7bAC0c2eDf58fb34A2CBC39Ea2fbB7056', 'decimals': 18},
    'POS32': {'address': '0x8eb5bD8c9Ab0F8ad28e94693F3c889F490bE2aB0', 'decimals': 18},
    'DBOX': {'address': '0x56c9D5F1E727de03643af220B5cE52DE23D4D973', 'decimals': 18},
    'GRT': {'address': '0xc944E90C64B2c07662A292be6244BDf05Cda44a7', 'decimals': 18},
    'UZUMAKI': {'address': '0xD4c64f0Fe38ecdaF0Baefd7859E18185a73b9aa3', 'decimals': 9},
    'DARUMA': {'address': '0xA888D9616C2222788fa19f05F77221A290eEf704', 'decimals': 9},
    'HEDGEHOG': {'address': '0x6d4CA1177087924edfE0908ef655169EA766FDc3', 'decimals': 18},
    'BAL': {'address': '0xba100000625a3754423978a60c9317c58a424e3D', 'decimals': 18},
    'HID': {'address': '0xB14eBF566511B9e6002bB286016AB2497B9b9c9D', 'decimals': 18},
    'HAN': {'address': '0x0c90C57aaf95A3A87eadda6ec3974c99D786511F', 'decimals': 18},
    'H': {'address': '0xc6956b78e036b87Ba2aB9810BF081eb76EEDd17a', 'decimals': 18},
    'TNT': {'address': '0x6692De64716a177c15360D8d010BC522bBc530a0', 'decimals': 18},
    'SHEESHA': {'address': '0x232FB065D9d24c34708eeDbF03724f2e95ABE768', 'decimals': 18},
    'CUSDC': {'address': '0x39AA39c021dfbaE8faC545936693aC917d5E7563', 'decimals': 8},
    'SAITO': {'address': '0xFa14Fa6958401314851A17d6C5360cA29f74B57B', 'decimals': 18},
    'AAMMUNIAAVEWETH': {'address': '0xe59d2FF6995a926A574390824a657eEd36801E55', 'decimals': 18},
    'KORI': {'address': '0x345DADb10A200F10814Ad8523fCA0f2d958C3370', 'decimals': 9},
    'HOT': {'address': '0x6c6EE5e31d828De241282B9606C8e98Ea48526E2', 'decimals': 18},
    'QCX': {'address': '0xF9e5aF7B42D31D51677c75bbBD37c1986eC79AEE', 'decimals': 8},
    'SQGL': {'address': '0x8d137e3337eb1B58A222Fef2B2Cc7C423903d9cf', 'decimals': 18},
    'CWT': {'address': '0x22143217662334F073652Da5E2F1e0AE7349240c', 'decimals': 18},
    'GOLD': {'address': '0x670f9D9a26D3D42030794ff035d35a67AA092ead', 'decimals': 8},
    'SURF': {'address': '0x46d473a0B3eEEc9F55FADE641bC576d5bc0b2246', 'decimals': 8},
    'COC': {'address': '0x22B6C31c2bEB8f2d0d5373146Eed41Ab9eDe3caf', 'decimals': 8},
    'GAME': {'address': '0x63f88A2298a5c4AEE3c216Aa6D926B184a4b2437', 'decimals': 18},
    'GOLD': {'address': '0x34D6A0F5C2f5D0082141fE73d93B9dd00ca7CE11', 'decimals': 18},
    'KNDX': {'address': '0x7CA5af5bA3472AF6049F63c1AbC324475D44EFC1', 'decimals': 9},
    'ORT': {'address': '0x5c59a5B139B0538CB106D775a022caD98Dd14b5a', 'decimals': 18},
    'ANV': {'address': '0x3D382228C54736d831FAC2748F4734D9177c7332', 'decimals': 18},
    'BBT': {'address': '0x1500205f50bf3FD976466d0662905c9ff254fc9c', 'decimals': 4},
    'POP': {'address': '0x7fC3eC3574d408F3b59CD88709baCb42575EBF2b', 'decimals': 18},
    'Z': {'address': '0x1BE56412c9606E7285280F76A105EBa56996e491', 'decimals': 9},
    'EVA': {'address': '0xd6cAF5Bd23CF057f5FcCCE295Dcc50C01C198707', 'decimals': 18},
    'RATING': {'address': '0xE8663A64A96169ff4d95b4299E7ae9a76b905B31', 'decimals': 8},
    'ARNC': {'address': '0x7495e5Cc8F27e0bd5bD4cb86d17F0D841cA58EE4', 'decimals': 18},
    'DIP': {'address': '0xc719d010B63E5bbF2C0551872CD5316ED26AcD83', 'decimals': 18},
    'ELA': {'address': '0xe6fd75ff38Adca4B97FBCD938c86b98772431867', 'decimals': 18},
    'DEC': {'address': '0x30f271C9E86D2B7d00a6376Cd96A1cFBD5F0b9b3', 'decimals': 18},
    'AXI': {'address': '0x73EE6D7e6b203125aDd89320E9F343d65ec7c39a', 'decimals': 18},
    'NII': {'address': '0x7c8155909cd385F120A56eF90728dD50F9CcbE52', 'decimals': 15},
    'AQT': {'address': '0x2a9bDCFF37aB68B95A53435ADFd8892e86084F93', 'decimals': 18},
    'EXP': {'address': '0xd137561C20BE8e870D62e2aE92269f0950702f77', 'decimals': 18},
    'VAB': {'address': '0xe7aE6D0C56CACaf007b7e4d312f9af686a9E9a04', 'decimals': 18},
    'FINU': {'address': '0x2c022e58C5E3Ee213F06f975fd8a0d3a6fE9Ca1C', 'decimals': 18},
    'CYCE': {'address': '0xEaDD9B69F96140283F9fF75DA5FD33bcF54E6296', 'decimals': 6},
    'NEWO': {'address': '0x98585dFc8d9e7D48F0b1aE47ce33332CF4237D96', 'decimals': 18},
    'MOSOLID': {'address': '0x848578e351D25B6Ec0d486E42677891521c3d743', 'decimals': 18},
    'UNI': {'address': '0x2730d6FdC86C95a74253BefFaA8306B40feDecbb', 'decimals': 8},
    'SENC': {'address': '0xA13f0743951B4f6E3e3AA039f682E17279f52bc3', 'decimals': 18},
    'FKPEPE': {'address': '0x082646B22A3960DA69eF7A778c16dd6fb85Dd999', 'decimals': 18},
    'EARN': {'address': '0xA87Ed75C257f1ec38393bEA0A83d55Ac2279D79c', 'decimals': 9},
    'SPOT': {'address': '0xC1f33e0cf7e40a67375007104B929E49a581bafE', 'decimals': 9},
    'MPS': {'address': '0x96c645D3D3706f793Ef52C19bBACe441900eD47D', 'decimals': 0},
    'HIVALHALLA': {'address': '0x5C0590cDE44569bf39ef79e859B367e39cB000F1', 'decimals': 18},
    'ANB': {'address': '0x2c9acEb63181cd08a093d052ec041e191f229692', 'decimals': 18},
    'STARL': {'address': '0x8E6cd950Ad6ba651F6DD608Dc70e5886B1AA6B24', 'decimals': 18},
    'XI': {'address': '0x295B42684F90c77DA7ea46336001010F2791Ec8c', 'decimals': 18},
    'SATA': {'address': '0x3ebb4A4e91Ad83BE51F8d596533818b246F4bEe1', 'decimals': 18},
    'BOX': {'address': '0x63f584FA56E60e4D0fE8802b27C7e6E3b33E007f', 'decimals': 18},
    'CHEX': {'address': '0x9Ce84F6A69986a83d92C324df10bC8E64771030f', 'decimals': 18},
    'DATA': {'address': '0x8f693ca8D21b157107184d29D398A8D082b38b76', 'decimals': 18},
    'CND': {'address': '0xEc505C81D6a7567b5BDE804870b1038832fE6Da1', 'decimals': 18},
    'MCD': {'address': '0xc66cDac744916AFb6811C71c277d88dE90cE8D5b', 'decimals': 18},
    'ORE': {'address': '0x4f640F2529ee0cF119A2881485845FA8e61A782A', 'decimals': 18},
    'SHAMAN': {'address': '0x5fCe9Fc9B5d62aF082A59D0823A062F7529eFA5a', 'decimals': 18},
    'INFI': {'address': '0x159751323A9E0415DD3d6D42a1212fe9F4a0848C', 'decimals': 18},
    'PRARE': {'address': '0x2C2f7e7C5604D162d75641256b80F1Bf6f4dC796', 'decimals': 18},
    'SKEY': {'address': '0x06A01a4d579479Dd5D884EBf61A31727A3d8D442', 'decimals': 8},
    'EMAID': {'address': '0x329c6E459FFa7475718838145e5e85802Db2a303', 'decimals': 18},
    'VOW': {'address': '0x1BBf25e71EC48B84d773809B4bA55B6F4bE946Fb', 'decimals': 18},
    'LINU': {'address': '0x78132543D8E20D2417d8a07D9ae199D458A0D581', 'decimals': 18},
    'XTM': {'address': '0xCd1fAFf6e578Fa5cAC469d2418C95671bA1a62Fe', 'decimals': 18},
    'DRCT': {'address': '0x9d561d63375672ABd02119b9Bc4FB90EB9E307Ca', 'decimals': 18},
    'NUTS': {'address': '0x981dc247745800bD2cA28a4bF147f0385Eaa0bc0', 'decimals': 18},
    'RARI': {'address': '0xFca59Cd816aB1eaD66534D82bc21E7515cE441CF', 'decimals': 18},
    'CYCE': {'address': '0x9681ee0D91E737C3B60aCEbA7Fbdae61B5462f42', 'decimals': 6},
    'ZAP': {'address': '0x6781a0F84c7E9e846DCb84A9a5bd49333067b104', 'decimals': 18},
    'VERA': {'address': '0xd7f0cC50AD69408aE58be033F4f85D2367C2e468', 'decimals': 18},
    'METADOGE': {'address': '0x8530b66ca3DDf50E0447eae8aD7eA7d5e62762eD', 'decimals': 18},
    'PEEP': {'address': '0x5919DeA604631016c15c805e3D948A0384879892', 'decimals': 9},
    'GEAR': {'address': '0x1b980e05943dE3dB3a459C72325338d327B6F5a9', 'decimals': 18},
    'WMINIMA': {'address': '0x669c01CAF0eDcaD7c2b8Dc771474aD937A7CA4AF', 'decimals': 18},
    'APEFI': {'address': '0x4332f8A38f14BD3D8D1553aF27D7c7Ac6C27278D', 'decimals': 18},
    'COT': {'address': '0x5c872500c00565505F3624AB435c222E558E9ff8', 'decimals': 18},
    'CONI': {'address': '0x4561de8e0C2bbA725d38D266EF62426e62678d82', 'decimals': 18},
    'VETH': {'address': '0x4Bc3263Eb5bb2Ef7Ad9aB6FB68be80E43b43801F', 'decimals': 18},
    'TIGER': {'address': '0xB83c27805aAcA5C7082eB45C868d955Cf04C337F', 'decimals': 18},
    'ATH': {'address': '0x2F9411088cEF82Fd9fB904Eb8092f28eB485C8F6', 'decimals': 18},
    'X7102': {'address': '0x7102DC82EF61bfB0410B1b1bF8EA74575bf0A105', 'decimals': 18},
    'CMOS': {'address': '0x87869A9789291A6cEC99f3c3Ef2fF71fcEb12a8e', 'decimals': 9},
    'MODS': {'address': '0xa43a8FC632d809FB2c16f590c0ff6423e311c67D', 'decimals': 18},
    'IBCHF': {'address': '0x1CC481cE2BD2EC7Bf67d1Be64d4878b16078F309', 'decimals': 18},
    'COVAL': {'address': '0x3D658390460295FB963f54dC0899cfb1c30776Df', 'decimals': 8},
    'DAO': {'address': '0x0f51bb10119727a7e5eA3538074fb341F56B09Ad', 'decimals': 18},
    'HBOT': {'address': '0xE5097D9baeAFB89f9bcB78C9290d545dB5f9e9CB', 'decimals': 18},
    'KNG': {'address': '0x471d113059324321749E097705197a2B44A070FC', 'decimals': 18},
    'BASIC': {'address': '0xF25c91C87e0B1fd9B4064Af0F427157AaB0193A7', 'decimals': 18},
    'PKT': {'address': '0xe28f0982eEeD9cCf816d8496fb742D438056d275', 'decimals': 18},
    'CRNO': {'address': '0xE23311294467654E0CaB14cD32A169A41be5ca8E', 'decimals': 18},
    'XDNA': {'address': '0x8e57c27761EBBd381b0f9d09Bb92CeB51a358AbB', 'decimals': 18},
    'WAR': {'address': '0x4FaDC7A98f2Dc96510e42dD1A74141eEae0C1543', 'decimals': 12},
    'HVN': {'address': '0xC0Eb85285d83217CD7c891702bcbC0FC401E2D9D', 'decimals': 8},
    'ODA': {'address': '0x0C6Bf55F60143Fa1dA202AC8c9b8Ce484b5a5d90', 'decimals': 18},
    'KP4R': {'address': '0xA89ac6e529aCf391CfbBD377F3aC9D93eae9664e', 'decimals': 18},
    'XBP': {'address': '0x28dee01D53FED0Edf5f6E310BF8Ef9311513Ae40', 'decimals': 18},
    'IND': {'address': '0xf8e386EDa857484f5a12e4B5DAa9984E06E73705', 'decimals': 18},
    'YLC': {'address': '0x21d5678A62DFe63a47062469Ebb2fAc2817D8832', 'decimals': 8},
    'AAMMUNIUNIWETH': {'address': '0x3D26dcd840fCC8e4B2193AcE8A092e4a65832F9f', 'decimals': 18},
    'LSILVER': {'address': '0xD64809f5F7d772D9112A6BD379De00A77188199E', 'decimals': 18},
    'CCBCH': {'address': '0xdB792B1D8869A7CFc34916d6c845Ff05A7C9b789', 'decimals': 8},
    'RIA': {'address': '0x9b110Fda4E20DB18Ad7052f8468a455de7449eb6', 'decimals': 18},
    'JAR': {'address': '0xa249DE6948022783765Fee4850d7b85E43118FCc', 'decimals': 18},
    'NTX': {'address': '0xF0d33BeDa4d734C72684b5f9abBEbf715D0a7935', 'decimals': 6},
    'MOTG': {'address': '0x171d76d931529384639BC9aad5b77b77041ED604', 'decimals': 18},
    'VGX': {'address': '0x3C4B6E6e1eA3D4863700D7F76b36B7f3D3f13E3d', 'decimals': 8},
    'GERA': {'address': '0xDe5eA375FFBDc8b25a80fe13D631E8Ba0AB4BB02', 'decimals': 18},
    'MCC': {'address': '0xC146B7CdBaff065090077151d391f4c96Aa09e0C', 'decimals': 9},
    'HSF': {'address': '0xbA6B0dbb2bA8dAA8F5D6817946393Aef8D3A4487', 'decimals': 18},
    'TVT': {'address': '0x98E0438d3eE1404FEA48E38e92853BB08Cfa68bD', 'decimals': 8},
    'BLOCK-E': {'address': '0xa9f9aCB92E4E2f16410511D56839A5Bd1d630a60', 'decimals': 9},
    'STREAMERINU': {'address': '0x0e8D2EB7D6bDF28393c25a1966385Ad32Ff0259a', 'decimals': 18},
    'ECO': {'address': '0x191557728e4d8CAa4Ac94f86af842148c0FA8F7E', 'decimals': 8},
    'RESET': {'address': '0x30df7D7EE52c1b03cd009e656F00AB875AdCEeD2', 'decimals': 18},
    'WSTRAX': {'address': '0xa3C22370de5f9544f0c4De126b1e46cEadF0A51B', 'decimals': 18},
    'AGN': {'address': '0xEDc1004886d010751f74ec0Ad223819F9F3b1910', 'decimals': 18},
    'QTCON': {'address': '0x1bF7Fd22709733cCD7c45AB27Dd02C7EC8E50078', 'decimals': 18},
    'ORO': {'address': '0xc3Eb2622190c57429aac3901808994443b64B466', 'decimals': 18},
    'WAGIE': {'address': '0x492baa7A6450712D4bbCCa01B87F029DEe3Ea3Ec', 'decimals': 9},
    'FIN': {'address': '0x054f76beED60AB6dBEb23502178C52d6C5dEbE40', 'decimals': 18},
    'CPD': {'address': '0x9b31bb425D8263fA1b8B9d090b83CF0C31665355', 'decimals': 18},
    'ICB': {'address': '0x00166a5b93FD4f87ecA1c267d31D82dF3f133a7F', 'decimals': 18},
    'AMKR': {'address': '0x7deB5e830be29F91E298ba5FF1356BB7f8146998', 'decimals': 18},
    'UNQT': {'address': '0xa80f2C8f61c56546001f5FC2eb8D6E4e72c45d4C', 'decimals': 18},
    'VLX': {'address': '0x8C543AED163909142695f2d2aCd0D55791a9Edb9', 'decimals': 18},
    'MEME': {'address': '0x74B988156925937bD4E082f0eD7429Da8eAea8Db', 'decimals': 18},
    'METAL': {'address': '0x294559FA758C88D639Fd085751e463FEe7806eab', 'decimals': 18},
    'FOLO': {'address': '0xB2a63A5Dd36C91eC2DA59B188ff047F66fac122a', 'decimals': 18},
    'CNNS': {'address': '0x6c3BE406174349cfa4501654313d97e6a31072e1', 'decimals': 18},
    'ROYA': {'address': '0x7eaF9C89037e4814DC0d9952Ac7F888C784548DB', 'decimals': 18},
    'PETS': {'address': '0x6afCFF9189e8ed3fCc1CFfa184FEB1276f6A82A5', 'decimals': 18},
    'STC': {'address': '0x15B543e986b8c34074DFc9901136d9355a537e7E', 'decimals': 18},
    'GATE': {'address': '0x9d7630aDF7ab0b0CB00Af747Db76864df0EC82E4', 'decimals': 18},
    'BITT': {'address': '0x9F9913853f749b3fE6D6D4e16a1Cc3C1656B6D51', 'decimals': 18},
    'FORTUNE': {'address': '0x9F009D03E1b7F02065017C90e8e0D5Cb378eB015', 'decimals': 9},
    'CSM': {'address': '0x2620638EDA99F9e7E902Ea24a285456EE9438861', 'decimals': 18},
    'USHIBA': {'address': '0xB893A8049f250b57eFA8C62D51527a22404D7c9A', 'decimals': 9},
    'LIFT': {'address': '0x513C3200F227ebB62e3B3d00B7a83779643a71CF', 'decimals': 18},
    'IOEN': {'address': '0x1e4E46b7BF03ECE908c88FF7cC4975560010893A', 'decimals': 18},
    'SMARTCREDIT': {'address': '0x72e9D9038cE484EE986FEa183f8d8Df93f9aDA13', 'decimals': 18},
    'SATT': {'address': '0xDf49C9f599A0A9049D97CFF34D0C30E468987389', 'decimals': 18},
    'TGL': {'address': '0xCc4450EDEfbBEb450E2f205f4ECEfF7232D54e92', 'decimals': 18},
    'BOTX': {'address': '0xEF19F4E48830093Ce5bC8b3Ff7f903A0AE3E9Fa1', 'decimals': 18},
    'ELG': {'address': '0xA2085073878152aC3090eA13D1e41bD69e60Dc99', 'decimals': 18},
    'SENT': {'address': '0x97AbEe33Cd075c58BFdd174e0885e08E8f03556F', 'decimals': 18},
    'OAP': {'address': '0x1788430620960F9a70e3DC14202a3A35ddE1A316', 'decimals': 18},
    'TXT': {'address': '0x547b2F82ceCfAb9C2B1D36fddA96eF9F58C63B8C', 'decimals': 18},
    'ALI': {'address': '0x6B0b3a982b4634aC68dD83a4DBF02311cE324181', 'decimals': 18},
    'SHIBDAO': {'address': '0xB197a6Fdd8AB825332eDEbA1AB5C4D1Bf97fb9A8', 'decimals': 18},
    'PERI': {'address': '0x5d30aD9C6374Bf925D0A75454fa327AACf778492', 'decimals': 18},
    'ARGO': {'address': '0x28Cca76f6e8eC81e4550ecd761f899110b060E97', 'decimals': 18},
    'FOY': {'address': '0xAf91E8aFbe87642dC628786188a54B78580A4d76', 'decimals': 18},
    'APYS': {'address': '0xf7413489c474ca4399eeE604716c72879Eea3615', 'decimals': 18},
    'CUDOS': {'address': '0x817bbDbC3e8A1204f3691d14bB44992841e3dB35', 'decimals': 18},
    'VID': {'address': '0x2C9023bBc572ff8dc1228c7858A280046Ea8C9E5', 'decimals': 18},
    'OPEN': {'address': '0x9D86b1B2554ec410ecCFfBf111A6994910111340', 'decimals': 8},
    'HPO': {'address': '0xa0ED3C520dC0632657AD2EaaF19E26C4fD431a84', 'decimals': 18},
    'TUF': {'address': '0x6f8b23296394d20eC048FbDec8eBc0CA90f5c8f1', 'decimals': 18},
    'HOPR': {'address': '0xF5581dFeFD8Fb0e4aeC526bE659CFaB1f8c781dA', 'decimals': 18},
    'ADT': {'address': '0xD0D6D6C5Fe4a677D343cC433536BB717bAe167dD', 'decimals': 9},
    'YASHA': {'address': '0xD75f1f81B69bDd4Df8EfbB70e9C6F4609009D753', 'decimals': 18},
    'GWD': {'address': '0x674c964AC0E89d847d6B0aBd144b797bf78BA56b', 'decimals': 6},
    'FREE': {'address': '0x4CD0c43B0D53bc318cc5342b77EB6f124E47f526', 'decimals': 18},
    'GBPT': {'address': '0x86B4dBE5D203e634a12364C0e428fa242A3FbA98', 'decimals': 18},
    'WARP': {'address': '0xEd40834A13129509A89be39a9bE9C0E96A0DDd71', 'decimals': 18},
    'UDOO': {'address': '0x12f649A9E821F90BB143089a6e56846945892ffB', 'decimals': 18},
    'BLUSGD': {'address': '0x92830ef7c8d651Ed3A708053c602E807bAd7db22', 'decimals': 18},
    'SYRANT': {'address': '0x976bf616F61685e0E7E36419Aee3f7Cde24B0Dc9', 'decimals': 18},
    'BLES': {'address': '0xE796d6ca1ceb1b022EcE5296226BF784110031Cd', 'decimals': 18},
    'DMT': {'address': '0x2ccbFF3A042c68716Ed2a2Cb0c544A9f1d1935E1', 'decimals': 8},
    'NBC': {'address': '0x9F195617fA8fbAD9540C5D113A99A0a0172aaEDC', 'decimals': 18},
    'GOB': {'address': '0xd6929179D752d5D25c5efe2d9729eb77D7138A80', 'decimals': 18},
    'TORG': {'address': '0xCfa0885131F602D11D4DA248D2c65A62063567A9', 'decimals': 18},
    'BREED': {'address': '0x94E9EB8b5Ab9fd6B9ea3169D55FFAde62a01702e', 'decimals': 18},
    'FLOOR': {'address': '0xf59257E961883636290411c11ec5Ae622d19455e', 'decimals': 9},
    'IMP': {'address': '0x2D5C73f3597B07F23C2bB3F2422932E67eca4543', 'decimals': 18},
    'QDT': {'address': '0x9Adc7710E9d1b29d8a78c04d52D32532297C2Ef3', 'decimals': 18},
    'HEGIC': {'address': '0x584bC13c7D411c00c01A62e8019472dE68768430', 'decimals': 18},
    'MPH': {'address': '0x6369c3DadfC00054A42BA8B2c09c48131dd4Aa38', 'decimals': 18},
    'NINEFI': {'address': '0xe78BFF28D3c5195C752E734E95421bb4bd742f1d', 'decimals': 18},
    'RAIL': {'address': '0xe76C6c83af64e4C60245D8C7dE953DF673a7A33D', 'decimals': 18},
    'CAPS': {'address': '0x03Be5C903c727Ee2C8C4e9bc0AcC860Cca4715e2', 'decimals': 18},
    'RBN': {'address': '0x6123B0049F904d730dB3C36a31167D9d4121fA6B', 'decimals': 18},
    'LEVER': {'address': '0x4B5f49487ea7B3609b1aD05459BE420548789f1f', 'decimals': 18},
    'LEV': {'address': '0xbc194e6f748a222754C3E8b9946922c09E7d4e91', 'decimals': 18},
    'UBX': {'address': '0xF5B5eFc906513b4344EbAbCF47A04901f99F09f3', 'decimals': 0},
    'PEAK': {'address': '0x630d98424eFe0Ea27fB1b3Ab7741907DFFEaAd78', 'decimals': 8},
    'HDAO': {'address': '0xdaC657ffD44a3B9d8aba8749830Bf14BEB66fF2D', 'decimals': 18},
    'BNT': {'address': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'decimals': 18},
    'OCT': {'address': '0xF5cFBC74057C610c8EF151A439252680AC68c6DC', 'decimals': 18},
    'YVUNI': {'address': '0xFBEB78a723b8087fD2ea7Ef1afEc93d35E8Bed42', 'decimals': 18},
    'PEBBLE': {'address': '0xDC98c5543F3004DEBfaad8966ec403093D0aa4A8', 'decimals': 18},
    'XCAD': {'address': '0x7659CE147D0e714454073a5dd7003544234b6Aa0', 'decimals': 18},
    'CHART': {'address': '0x1d37986F252d0e349522EA6C3B98Cb935495E63E', 'decimals': 18},
    'SCX': {'address': '0x1B8568FbB47708E9E9D31Ff303254f748805bF21', 'decimals': 18},
    'MRI': {'address': '0x0913dDAE242839f8995c0375493f9a1A3Bddc977', 'decimals': 18},
    'DAMEX': {'address': '0x00E679Ba63B509182c349F5614F0A07cDd0CE0c5', 'decimals': 18},
    'DED': {'address': '0xdBBE358Fead311b4089B8BF30F455c041E67f5f4', 'decimals': 18},
    'STUB': {'address': '0x54C890411e97755ca69321f28Bed31CAFBb3C5A0', 'decimals': 18},
    'DFT': {'address': '0xA2A54f1Ec1f09316eF12c1770D32ed8F21B1Fb6A', 'decimals': 8},
    'GSWAP': {'address': '0xaac41EC512808d64625576EDdd580e7Ea40ef8B2', 'decimals': 18},
    'MONKED': {'address': '0xE7AC8545E34771de3706598Abb3dB9a19af2A07F', 'decimals': 8},
    'OPTI': {'address': '0x562E362876c8Aee4744FC2c6aaC8394C312d215d', 'decimals': 9},
    'CXD': {'address': '0x5A56Da75c50aA2733F5Fa9A2442AaEfcBc60B2e6', 'decimals': 18},
    'ICETH': {'address': '0x7C07F7aBe10CE8e33DC6C5aD68FE033085256A84', 'decimals': 18},
    'BOA': {'address': '0x746DdA2ea243400D5a63e0700F190aB79f06489e', 'decimals': 7},
    'VFY': {'address': '0x00DD5Ad8F5d9c5118BcEeb526c55eA6b2C07e922', 'decimals': 18},
    'QSP': {'address': '0x99ea4dB9EE77ACD40B119BD1dC4E33e1C070b80d', 'decimals': 18},
    'BLOCKS': {'address': '0x8a6D4C8735371EBAF8874fBd518b56Edd66024eB', 'decimals': 18},
    'SPHRI': {'address': '0x8A0cdfaB62eD35b836DC0633482798421C81b3Ec', 'decimals': 18},
    'NKN': {'address': '0x5Cf04716BA20127F1E2297AdDCf4B5035000c9eb', 'decimals': 18},
    'BNANA': {'address': '0x07eF9E82721AC16809D24DAfBE1792Ce01654DB4', 'decimals': 18},
    'MIDAS': {'address': '0x97e6e31aFb2d93d437301e006D9DA714616766A5', 'decimals': 18},
    'HONK': {'address': '0x5efcEA234f7547de4569aad1215FA5D2adACeD38', 'decimals': 18},
    'HIRENGA': {'address': '0x1De852CD18323Bc5eBDa842b8EEF46C6E551aa47', 'decimals': 18},
    'MIB': {'address': '0x146D8D942048ad517479C9bab1788712Af180Fde', 'decimals': 18},
    'YDR': {'address': '0x3757232B55E60da4A8793183aC030CfCE4c3865d', 'decimals': 18},
    'YSHORI': {'address': '0x25644eA5bE09d1266B5df3F96559Ecdc2D0ad69b', 'decimals': 18},
    'EZ': {'address': '0x00AbA6fE5557De1a1d565658cBDdddf7C710a1eb', 'decimals': 18},
    'KSC': {'address': '0x990E081A7B7d3Ccba26a2f49746A68CC4fF73280', 'decimals': 18},
    'XGC': {'address': '0x89d3c0249307Ae570A316030764D12f53BB191FD', 'decimals': 14},
    'ONS': {'address': '0x5BB29c33C4A3C29f56F8ACa40B4dB91d8a5fe2c5', 'decimals': 18},
    'IQQ': {'address': '0x68a9d92Fe19399FEEBEd6A9a0980a7ea7638074C', 'decimals': 18},
    'JAY': {'address': '0xDA7C0810cE6F8329786160bb3d1734cf6661CA6E', 'decimals': 18},
    'RBX': {'address': '0x3Ba925fdeAe6B46d0BB4d424D829982Cb2F7309e', 'decimals': 18},
    'DXC': {'address': '0xb0E99627bC29adEf1178f16117BF495351E81997', 'decimals': 18},
    'VIDA': {'address': '0xE35f19E4457A114A951781aaF421EC5266eF25Fe', 'decimals': 18},
    'MF1': {'address': '0x9b5161a41B58498Eb9c5FEBf89d60714089d2253', 'decimals': 18},
    'FOLD': {'address': '0xd084944d3c05CD115C09d072B9F44bA3E0E45921', 'decimals': 18},
    'NEZUKO': {'address': '0xbC298DfAA2EDda095b924F1390cC38fB7c5f6250', 'decimals': 9},
    'OCC': {'address': '0x2F109021aFe75B949429fe30523Ee7C0D5B27207', 'decimals': 18},
    'KTON': {'address': '0x9F284E1337A815fe77D2Ff4aE46544645B20c5ff', 'decimals': 18},
    'KOI': {'address': '0xe84d9e32dC8cE819b8D6c83e50EDAfD46c6354dB', 'decimals': 18},
    'BECOIN': {'address': '0x8F081Eb884Fd47b79536D28E2DD9d4886773F783', 'decimals': 6},
    'INTX': {'address': '0x7533D63A2558965472398Ef473908e1320520AE2', 'decimals': 9},
    'JIZZ': {'address': '0x8b937AF714aC7E2129bD33D93641F52b665Ca352', 'decimals': 18},
    'ELFI': {'address': '0x4dA34f8264CB33A5c9F17081B9EF5Ff6091116f4', 'decimals': 18},
    'CIND': {'address': '0xaC0968a3E2020aC8ca83e60CcF69081EBc6d3bC3', 'decimals': 18},
    'TTX': {'address': '0xeF2e239b21A81719abAabC6239e2e90859B44D9B', 'decimals': 18},
    'DBET': {'address': '0x9b68bFaE21DF5A510931A262CECf63f41338F264', 'decimals': 18},
    'PMGT': {'address': '0xAFFCDd96531bCd66faED95FC61e443D08F79eFEf', 'decimals': 5},
    'EDDA': {'address': '0xFbbE9b1142C699512545f47937Ee6fae0e4B0aA9', 'decimals': 18},
    'AAPX': {'address': '0xbfD815347d024F449886c171f78Fa5B8E6790811', 'decimals': 18},
    'MSHIBA': {'address': '0x9cF77be84214beb066F26a4ea1c38ddcc2AFbcf7', 'decimals': 18},
    'PLUG': {'address': '0x47DA5456bC2e1ce391b645Ce80F2E97192e4976a', 'decimals': 18},
    'HAPI': {'address': '0xD9c2D319Cd7e6177336b0a9c93c21cb48d84Fb54', 'decimals': 18},
    'OUSG': {'address': '0x1B19C19393e2d034D8Ff31ff34c81252FcBbee92', 'decimals': 18},
    'BLZ': {'address': '0x5732046A883704404F284Ce41FfADd5b007FD668', 'decimals': 18},
    'PUBLX': {'address': '0x1a6658F40e51b372E593B7d2144c1402d5cf33E8', 'decimals': 18},
    'AIR': {'address': '0xF725f73CAEe250AE384ec38bB2C77C38ef2CcCeA', 'decimals': 18},
    'MARBLEX7': {'address': '0x1AD82FAc4dfE68e68f22Df9Cb892b018E4541DbA', 'decimals': 18},
    'YFO': {'address': '0xAc0C8dA4A4748d8d821A0973d00b157aA78C473D', 'decimals': 18},
    'OCN': {'address': '0x4092678e4E78230F46A1534C0fbc8fA39780892B', 'decimals': 18},
    'BTRST': {'address': '0x799ebfABE77a6E34311eeEe9825190B9ECe32824', 'decimals': 18},
    '4SHIBA': {'address': '0xB04bf60E468743418E87291d7c9301a5299d984d', 'decimals': 18},
    'KIBSHI': {'address': '0x02e7F808990638E9e67E1f00313037EDe2362361', 'decimals': 18},
    'FRXETH': {'address': '0x5E8422345238F34275888049021821E8E08CAa1f', 'decimals': 18},
    'XOLO': {'address': '0x3252d4221f92b7Af3774Da8312F01600eC84B252', 'decimals': 18},
    'BCMC': {'address': '0x2BA8349123de45E931a8C8264c332E6e9CF593F9', 'decimals': 18},
    'CAT': {'address': '0x56015BBE3C01fE05bc30A8a9a9Fd9A88917e7dB3', 'decimals': 18},
    'HGET': {'address': '0x7968bc6a03017eA2de509AAA816F163Db0f35148', 'decimals': 6},
    'RCG': {'address': '0xe74bE071f3b62f6A4aC23cA68E5E2A39797A3c30', 'decimals': 18},
    'INJ': {'address': '0xe28b3B32B6c345A34Ff64674606124Dd5Aceca30', 'decimals': 18},
    'PTM': {'address': '0x7c32DB0645A259FaE61353c1f891151A2e7f8c1e', 'decimals': 18},
    'IMPT': {'address': '0x04C17b9D3b29A78F7Bd062a57CF44FC633e71f85', 'decimals': 18},
    'ATH': {'address': '0xAA3573CC3761133375b0C504146B93F6827F945A', 'decimals': 18},
    'SRK': {'address': '0x0488401c3F535193Fa8Df029d9fFe615A06E74E6', 'decimals': 18},
    'PMA': {'address': '0x846C66cf71C43f80403B51fE3906B3599D63336f', 'decimals': 18},
    'OXAI': {'address': '0x428dca9537116148616a5A3E44035Af17238Fe9d', 'decimals': 18},
    'MARS4': {'address': '0x16CDA4028e9E872a38AcB903176719299beAed87', 'decimals': 18},
    'DBNB': {'address': '0x833850BE8858722Cfc5E5e75f2Fe6275E055d888', 'decimals': 9},
    'EDR': {'address': '0xc528c28FEC0A90C083328BC45f587eE215760A0F', 'decimals': 18},
    'STRK': {'address': '0x74232704659ef37c08995e386A2E26cc27a8d7B1', 'decimals': 18},
    'APCG': {'address': '0x434Cb4fC4b952872967914D430878Eee53Ebd502', 'decimals': 18},
    'TKINU': {'address': '0xda23d301761E4e2bF474951f978f6DFB6F3C9F14', 'decimals': 9},
    'ZCO': {'address': '0x2008e3057BD734e10AD13c9EAe45Ff132aBc1722', 'decimals': 8},
    'ZOOT': {'address': '0x1341A2257fa7b770420Ef70616f888056f90926c', 'decimals': 9},
    'MDX': {'address': '0x947AEb02304391f8fbE5B25D7D98D649b57b1788', 'decimals': 18},
    'IHF': {'address': '0xaF1250fa68D7DECD34fD75dE8742Bc03B29BD58e', 'decimals': 18},
    'INSURE': {'address': '0xd83AE04c9eD29d6D3E6Bf720C71bc7BeB424393E', 'decimals': 18},
    'IDLEDAISAFE': {'address': '0xa14eA0E11121e6E951E87c66AFe460A00BCD6A16', 'decimals': 18},
    'TOS': {'address': '0x34748FEDAd669C788aaBe4fb325A4587941F7B48', 'decimals': 18},
    'RNB': {'address': '0x2A039B1D9bBDccBb91be28691b730ca893e5e743', 'decimals': 18},
    'GTC': {'address': '0xDe30da39c46104798bB5aA3fe8B9e0e1F348163F', 'decimals': 18},
    'SLD': {'address': '0x1ef6A7e2c966fb7C5403EFEFdE38338b1a95a084', 'decimals': 18},
    'YCURVE': {'address': '0xdF5e0e81Dff6FAF3A7e52BA697820c5e32D806A8', 'decimals': 18},
    'ODE': {'address': '0xbf52F2ab39e26E0951d2a02b49B7702aBe30406a', 'decimals': 18},
    'IBZ': {'address': '0x5AA7C403c7dE4B3bb0cc07079a03e389671a4771', 'decimals': 18},
    'VTX': {'address': '0xceb286C9604c542d3cc08b41AA6C9675B078A832', 'decimals': 18},
    'OPTIG': {'address': '0xD5DF655087d99B7B720A5bc8711F296180A4F44b', 'decimals': 18},
    'AMPL': {'address': '0xD46bA6D942050d489DBd938a2C909A5d5039A161', 'decimals': 9},
    'BBF': {'address': '0xDE075D9ADbD0240b4462F124af926452Ad0BAC91', 'decimals': 18},
    'ONIGI': {'address': '0xb4615AAD766f6De3c55330099E907fF7F13f1582', 'decimals': 9},
    'BIFI': {'address': '0x2791BfD60D232150Bff86b39B7146c0eaAA2BA81', 'decimals': 18},
    'UTU': {'address': '0xa58a4f5c4Bb043d2CC1E170613B74e767c94189B', 'decimals': 18},
    'LBL': {'address': '0x2162f572B25f7358db9376AB58a947a4e45CeDE1', 'decimals': 18},
    'BTSG': {'address': '0x05079687D35b93538cbd59fe5596380cae9054A9', 'decimals': 18},
    'ZEFU': {'address': '0xB1e9157c2Fdcc5a856C8DA8b2d89b6C32b3c1229', 'decimals': 18},
    'WHALE': {'address': '0x9355372396e3F6daF13359B7b607a3374cc638e0', 'decimals': 4},
    'TRADE': {'address': '0x6F87D756DAf0503d08Eb8993686c7Fc01Dc44fB1', 'decimals': 18},
    'PICKLE': {'address': '0x429881672B9AE42b8EbA0E26cD9C73711b891Ca5', 'decimals': 18},
    'MCF': {'address': '0xE33AE4E795114279721047484E5ad5cc7DF24fcB', 'decimals': 10},
    'MESA': {'address': '0x5AFFF9876C1F98b7d2b53bCB69EB57e92408319F', 'decimals': 18},
    'VERSE': {'address': '0x249cA82617eC3DfB2589c4c17ab7EC9765350a18', 'decimals': 18},
    'TOSHI': {'address': '0x77a90B04d64189d4D09508612C09219bC6816BdC', 'decimals': 9},
    'HZM': {'address': '0x069f967be0CA21C7D793D8C343f71e597D9a49b3', 'decimals': 8},
    'ALN': {'address': '0x8185Bc4757572Da2a610f887561c32298f1A5748', 'decimals': 18},
    'OUSD': {'address': '0x2A8e1E676Ec238d8A992307B495b45B3fEAa5e86', 'decimals': 18},
    'AGF': {'address': '0xb3ED706B564BBa9CAB64042f4e1b391bE7BEbCe5', 'decimals': 18},
    'GSE': {'address': '0xe530441f4f73bDB6DC2fA5aF7c3fC5fD551Ec838', 'decimals': 4},
    'MOVD': {'address': '0x12300ED54F65648524c1bFAf86712927dA6cC6fC', 'decimals': 18},
    'LEASH': {'address': '0x27C70Cd1946795B66be9d954418546998b546634', 'decimals': 18},
    'UFR': {'address': '0xEA097A2b1dB00627B2Fa17460Ad260c016016977', 'decimals': 18},
    'HIT': {'address': '0x74b1Af114274335598da72f5C6ed7b954a016EeD', 'decimals': 18},
    'SSP': {'address': '0x624d520BAB2E4aD83935Fa503fB130614374E850', 'decimals': 4},
    'SD': {'address': '0x30D20208d987713f46DFD34EF128Bb16C404D10f', 'decimals': 18},
    'KABOSUINU': {'address': '0xD34A2051aC17BDD44D76AA84EA7Fcf8419130E55', 'decimals': 18},
    'PITCHFXS': {'address': '0x11EBe21e9d7BF541A18e1E3aC94939018Ce88F0b', 'decimals': 18},
    'LETSGO': {'address': '0xA6586E19EF681b1AC0ED3D46413D199a555dBB95', 'decimals': 18},
    'YDF': {'address': '0x30dcBa0405004cF124045793E1933C798Af9E66a', 'decimals': 18},
    'SNOW': {'address': '0xfe9A29aB92522D14Fc65880d817214261D8479AE', 'decimals': 18},
    'PLA': {'address': '0x3a4f40631a4f906c2BaD353Ed06De7A5D3fCb430', 'decimals': 18},
    'SMT': {'address': '0xB17548c7B510427baAc4e267BEa62e800b247173', 'decimals': 18},
    'RAE': {'address': '0xE5a3229CCb22b6484594973A03a3851dCd948756', 'decimals': 18},
    'SWEAT': {'address': '0xB4b9DC1C77bdbb135eA907fd5a08094d98883A35', 'decimals': 18},
    'PINE': {'address': '0x569424c5EE13884A193773fDC5d1c5f79C443a51', 'decimals': 18},
    'LOTT': {'address': '0xbA93EF534094F8b7001ECe2691168140965341ab', 'decimals': 18},
    'X7103': {'address': '0x7103eBdbF1f89be2d53EFF9B3CF996C9E775c105', 'decimals': 18},
    'ICNQ': {'address': '0xB3e2Cb7CccfE139f8FF84013823Bf22dA6B6390A', 'decimals': 18},
    'APPC': {'address': '0x1a7a8BD9106F2B8D977E08582DC7d24c723ab0DB', 'decimals': 18},
    'DIGI': {'address': '0x3CbF23c081fAA5419810ce0F6BC1ECb73006d848', 'decimals': 18},
    'KMT': {'address': '0x184b6f31aeAa6a19Eb207dFeA01F4289629e0f0f', 'decimals': 18},
    'X7104': {'address': '0x7104D1f179Cc9cc7fb5c79Be6Da846E3FBC4C105', 'decimals': 18},
    'BC': {'address': '0xe03B2642A5111aD0EFc0cbCe766498c2dd562Ae9', 'decimals': 9},
    'BB-A-DAI': {'address': '0xae37D54Ae477268B9997d4161B96b8200755935c', 'decimals': 18},
    'CHAT': {'address': '0x442Bc47357919446eAbC18C7211E57a13d983469', 'decimals': 18},
    'THE': {'address': '0x0cbA60Ca5eF4D42f92A5070A8fEDD13BE93E2861', 'decimals': 18},
    'STETH': {'address': '0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84', 'decimals': 18},
    'UMINT': {'address': '0xE99De844EF3Ef72806cf006224EF3b813e82662f', 'decimals': 18},
    'SATS': {'address': '0x6C22910c6F75F828B305e57c6a54855D8adeAbf8', 'decimals': 9},
    'SFCP': {'address': '0x8b6CdA5CC518c904e8844f445E1A7C7d2DB0fF16', 'decimals': 18},
    'RTH': {'address': '0x3FD8f39A962eFDA04956981C31AB89FAB5FB8bC8', 'decimals': 18},
    'ONC': {'address': '0xD90E69f67203EBE02c917B5128629E77B4cd92dc', 'decimals': 18},
    'MLT': {'address': '0x9506d37f70eB4C3d79C398d326C871aBBf10521d', 'decimals': 18},
    'IMPACTXP': {'address': '0xb12494C8824fc069757F47d177E666c571Cd49aE', 'decimals': 9},
    'KEX': {'address': '0x16980b3B4a3f9D89E33311B5aa8f80303E5ca4F8', 'decimals': 6},
    'SUSD': {'address': '0x57Ab1ec28D129707052df4dF418D58a2D46d5f51', 'decimals': 18},
    'QLT': {'address': '0x0173661769325565d4f011b2e5Cda688689Cc87c', 'decimals': 9},
    'PAN': {'address': '0x536381a8628dBcC8C70aC9A30A7258442eAb4c92', 'decimals': 8},
    'KEI': {'address': '0xc0114F14638a333A4D5C3b04F09b96372348A842', 'decimals': 9},
    'VPAD': {'address': '0x51FE2E572e97BFEB1D719809d743Ec2675924EDc', 'decimals': 18},
    'PROM': {'address': '0xfc82bb4ba86045Af6F327323a46E80412b91b27d', 'decimals': 18},
    'POLY': {'address': '0x9992eC3cF6A55b00978cdDF2b27BC6882d88D1eC', 'decimals': 18},
    'SENSO': {'address': '0xC19B6A4Ac7C7Cc24459F08984Bbd09664af17bD1', 'decimals': 0},
    'SOV': {'address': '0x2C5BC2Ba3614fD27FCc7022eA71d9172E2632c16', 'decimals': 18},
    'ZURR': {'address': '0x89509aA1D14a8E1E5364Ec4C3B041213bcDbe08D', 'decimals': 18},
    'GPOOL': {'address': '0x797de1Dc0b9fAf5E25c1f7Efe8dF9599138fA09d', 'decimals': 18},
    'ZORA': {'address': '0xD8E3FB3b08eBA982F2754988d70D57eDc0055ae6', 'decimals': 9},
    'PHM': {'address': '0x3F9BEC82c776c47405BcB38070d2395Fd18F89d3', 'decimals': 18},
    'KITTY': {'address': '0x044727e50ff30DB57fad06Ff4F5846eAb5eA52a2', 'decimals': 9},
    'XTN': {'address': '0x674C6Ad92Fd080e4004b2312b45f796a192D27a0', 'decimals': 18},
    'PRINTS': {'address': '0x4dd28568D05f09b02220b09C2cb307bFd837cb95', 'decimals': 18},
    'LEAG': {'address': '0x7b39917f9562C8Bc83c7a6c2950FF571375D505D', 'decimals': 18},
    'LHC': {'address': '0x320d31183100280CcdF69366CD56180Ea442A3E8', 'decimals': 8},
    'JAM': {'address': '0x23894DC9da6c94ECb439911cAF7d337746575A72', 'decimals': 18},
    'CWS': {'address': '0xaC0104Cca91D167873B8601d2e71EB3D4D8c33e0', 'decimals': 18},
    'DACXI': {'address': '0xEfaB7248D36585e2340E5d25F8a8D243E6e3193F', 'decimals': 18},
    'EQUAD': {'address': '0xC28e931814725BbEB9e670676FaBBCb694Fe7DF2', 'decimals': 18},
    'FIRE': {'address': '0xF921ae2DAC5fa128DC0F6168Bf153ea0943d2D43', 'decimals': 8},
    'AVT': {'address': '0x0d88eD6E74bbFD96B831231638b66C05571e824F', 'decimals': 18},
    'DEPAY': {'address': '0xa0bEd124a09ac2Bd941b10349d8d224fe3c955eb', 'decimals': 18},
    'BAOUSD': {'address': '0x7945b0A6674b175695e5d1D08aE1e6F13744Abb0', 'decimals': 18},
    'MIM': {'address': '0x99D8a9C45b2ecA8864373A26D1459e3Dff1e17F3', 'decimals': 18},
    'TQUEEN': {'address': '0x1C720999A4F8B2B0E9199039D353D2c9A77A73fC', 'decimals': 18},
    'MCG': {'address': '0x9cCf191169A613Eca433a249d3F40131178CAeBc', 'decimals': 18},
    'NAOS': {'address': '0x4a615bB7166210CCe20E6642a6f8Fb5d4D044496', 'decimals': 18},
    'SDG': {'address': '0x0F5dEF84CED3e9e295DAe28df96d0B846dE92C1A', 'decimals': 18},
    'HFT': {'address': '0xb3999F658C0391d94A37f7FF328F3feC942BcADC', 'decimals': 18},
    'EDGE': {'address': '0x4EC1b60B96193a64AcAe44778e51f7BfF2007831', 'decimals': 18},
    'WLITI': {'address': '0x0b63128C40737B13647552e0C926bCFEccC35f93', 'decimals': 18},
    'CNFI': {'address': '0xEABB8996eA1662cAd2f7fB715127852cd3262Ae9', 'decimals': 18},
    'EWC': {'address': '0xcD74Cf66c43e45Abd5703f3642F73d0675D6AFF7', 'decimals': 18},
    'RANKER': {'address': '0x6431FA4B812a2DCC062A38CB55cc7D18135AdEAd', 'decimals': 18},
    'JEUR': {'address': '0x0f17BC9a994b87b5225cFb6a2Cd4D667ADb4F20B', 'decimals': 18},
    'MDA': {'address': '0x51DB5Ad35C671a87207d88fC11d593AC0C8415bd', 'decimals': 18},
    'DEFY': {'address': '0x205eD31C867bF715E4182137Af95AFE9177Cd8E7', 'decimals': 18},
    'TXA': {'address': '0x4463e6A3dEd0dBE3F6e15bC8420dFc55e5FeA830', 'decimals': 18},
    'VEUR': {'address': '0x6bA75D640bEbfe5dA1197bb5A2aff3327789b5d3', 'decimals': 18},
    'GAME': {'address': '0xD567B5F02b9073aD3a982a099a23Bf019FF11d1c', 'decimals': 5},
    'OOKS': {'address': '0x69D9905B2e5f6F5433212b7F3c954433F23C1572', 'decimals': 18},
    'USDP': {'address': '0x1456688345527bE1f37E9e627DA0837D6f08C925', 'decimals': 18},
    'TNB': {'address': '0xF7920B0768Ecb20A123fAc32311d07D193381d6f', 'decimals': 18},
    'JPG': {'address': '0x02e7Ac540409D32C90BfB51114003a9E1fF0249c', 'decimals': 18},
    'OVO': {'address': '0xbfC66D8cCE39E668fd5d3C10fd1B1eAbB82c27B7', 'decimals': 18},
    'HIGH': {'address': '0x71Ab77b7dbB4fa7e017BC15090b2163221420282', 'decimals': 18},
    'KIMCHI': {'address': '0x1E18821E69B9FAA8e6e75DFFe54E7E25754beDa0', 'decimals': 18},
    'GTCOIN': {'address': '0x5d5E244660cA05C42073c9a526616d99f2c99516', 'decimals': 18},
    'KINE': {'address': '0xCbfef8fdd706cde6F208460f2Bf39Aa9c785F05D', 'decimals': 18},
    'SANSHU': {'address': '0xc73C167E7a4Ba109e4052f70D5466D0C312A344D', 'decimals': 9},
    'TVK': {'address': '0xd084B83C305daFD76AE3E1b4E1F1fe2eCcCb3988', 'decimals': 18},
    'PEPEBET': {'address': '0x33Cf519030235F75A13F29aFcFF5D6ff4Fd10350', 'decimals': 18},
    'LOL': {'address': '0xf91AC30E9b517f6D57e99446ee44894e6C22C032', 'decimals': 9},
    'FEAR': {'address': '0x88A9A52F944315D5B4e917b9689e65445C401E83', 'decimals': 18},
    'WNK': {'address': '0xE1BDA0c3Bfa2bE7f740f0119B6a34F057BD58Eba', 'decimals': 18},
    'FLUX': {'address': '0x469eDA64aEd3A3Ad6f868c44564291aA415cB1d9', 'decimals': 18},
    'AMAS': {'address': '0x65a8fbA02F641a13Bb7B01d5E1129b0521004f52', 'decimals': 18},
    'NGC': {'address': '0x72dD4b6bd852A3AA172Be4d6C5a6dbEc588cf131', 'decimals': 18},
    'OGN': {'address': '0x8207c1FfC5B6804F6024322CcF34F29c3541Ae26', 'decimals': 18},
    'AENJ': {'address': '0x712DB54daA836B53Ef1EcBb9c6ba3b9Efb073F40', 'decimals': 18},
    'GOLD1': {'address': '0x0Ec72CD6690db40B16be166858299F19D4f8E5B0', 'decimals': 9},
    'GFX': {'address': '0x65ad6A2288b2Dd23E466226397c8F5D1794e58fC', 'decimals': 18},
    'MLP': {'address': '0xE22020F47B7378dfedcedd2C81d4137c22fE1152', 'decimals': 18},
    'MEL': {'address': '0xed0889F7E1c7C7267407222Be277e1f1Ef4d4892', 'decimals': 18},
    'DOGY': {'address': '0x9c405acf8688AfB61B3197421cDeeC1A266c6839', 'decimals': 18},
    'LM': {'address': '0x7BEC98609cB6378D6F995e8f8097Ee78376fbec9', 'decimals': 18},
    'URUS': {'address': '0xc6DdDB5bc6E61e0841C54f3e723Ae1f3A807260b', 'decimals': 18},
    'FMT': {'address': '0x99c6e435eC259A7E8d65E1955C9423DB624bA54C', 'decimals': 18},
    'CWEB': {'address': '0x505B5eDa5E25a67E1c24A2BF1a527Ed9eb88Bf04', 'decimals': 18},
    'DFE': {'address': '0xd35Ce4FD815522Ff52243EB587020d557617f4dB', 'decimals': 18},
    'ANKRETH': {'address': '0xE95A203B1a91a908F9B9CE46459d101078c2c3cb', 'decimals': 18},
    'DUSK': {'address': '0x940a2dB1B7008B6C776d4faaCa729d6d4A4AA551', 'decimals': 18},
    'PNL': {'address': '0x9FC8f0ca1668E87294941b7f627e9C15eA06B459', 'decimals': 18},
    'ZERO': {'address': '0xF0939011a9bb95c3B791f0cb546377Ed2693a574', 'decimals': 18},
    'GLDX': {'address': '0xC631120155621Ee625835ec810B9885cDd764cd6', 'decimals': 8},
    'THERADIO': {'address': '0xAA2DEd323944b25C0B6f1F891Bc96F010b65622C', 'decimals': 18},
    'SAITOKI': {'address': '0xa3c56427683a19F7574b9fc219CFD27d5d6e87Fa', 'decimals': 9},
    'AUCTION': {'address': '0xA9B1Eb5908CfC3cdf91F9B8B3a74108598009096', 'decimals': 18},
    'RED': {'address': '0xC744df3419A8c9Bd4D6b9852A503eb1C5308A326', 'decimals': 18},
    'ISLA': {'address': '0x697eF32B4a3F5a4C39dE1cB7563f24CA7BfC5947', 'decimals': 18},
    'MFTU': {'address': '0xbA745513ACEbcBb977497C569D4F7d340f2A936B', 'decimals': 18},
    'AKNC': {'address': '0x39C6b3e42d6A679d7D776778Fe880BC9487C2EDA', 'decimals': 18},
    'XRUNE': {'address': '0x69fa0feE221AD11012BAb0FdB45d444D3D2Ce71c', 'decimals': 18},
    'XFI': {'address': '0x5BEfBB272290dD5b8521D4a938f6c4757742c430', 'decimals': 18},
    'FXF': {'address': '0x8a40c222996f9F3431f63Bf80244C36822060f12', 'decimals': 18},
    'TUSHI': {'address': '0x68FfB43026CEdC070bfC01B2203d90caB62CcF30', 'decimals': 18},
    'SETH2': {'address': '0xFe2e637202056d30016725477c5da089Ab0A043A', 'decimals': 18},
    'CWAP': {'address': '0xE74dC43867E0cbEB208F1a012fc60DcBbF0E3044', 'decimals': 18},
    'EUSD': {'address': '0xA0d69E286B938e21CBf7E51D71F6A4c8918f482F', 'decimals': 18},
    'KALLY': {'address': '0xfd30C9BEA1A952FEeEd2eF2C6B2Ff8A8FC4aAD07', 'decimals': 18},
    'AEL': {'address': '0xd337382da15d12Bb6e56498e91DF64f86c8f1EA8', 'decimals': 18},
    'GNFT': {'address': '0xc502002aEB1b9309FcCb016adF50507987Fc6C2b', 'decimals': 18},
    'FOREX': {'address': '0xa4E9584DAa093Cb1205E17bA737c3fd015748087', 'decimals': 18},
    'FUSE': {'address': '0x970B9bB2C0444F5E81e9d0eFb84C8ccdcdcAf84d', 'decimals': 18},
    'KEI': {'address': '0xF75C7a59bCD9bd207C4Ab1BEB0b32EEd3B6392f3', 'decimals': 8},
    'NMT': {'address': '0xd81b71cBb89B2800CDb000AA277Dc1491dc923C3', 'decimals': 18},
    'LUNCH': {'address': '0x71a28feAEe902966DC8D355e7B8Aa427D421e7e0', 'decimals': 18},
    'FIDU': {'address': '0x6a445E9F40e0b97c92d0b8a3366cEF1d67F700BF', 'decimals': 18},
    'MBET': {'address': '0x37c4822DFcEa7211d1D9876dABd48c62D46dbAAb', 'decimals': 18},
    'ACX': {'address': '0x44108f0223A3C3028F5Fe7AEC7f9bb2E66beF82F', 'decimals': 18},
    'SEURO': {'address': '0xb399511642FE1666c6a07f83483e6E4feAed9A00', 'decimals': 18},
    'HEZ': {'address': '0xEEF9f339514298C6A857EfCfC1A762aF84438dEE', 'decimals': 18},
    'KEYS': {'address': '0xe0a189C975e4928222978A74517442239a0b86ff', 'decimals': 9},
    'WUKONG': {'address': '0xA3ec7DF93090f4DE46FaEA09F73cc40ebe7dD714', 'decimals': 18},
    'LUN': {'address': '0xfa05A73FfE78ef8f1a739473e462c54bae6567D9', 'decimals': 18},
    'ALPA': {'address': '0x7cA4408137eb639570F8E647d9bD7B7E8717514A', 'decimals': 18},
    'IXI': {'address': '0x179Cd91631d96e8faFEe6A744eAC6fFdbB923520', 'decimals': 8},
    'DBUY': {'address': '0x4EcE5C5CfB9B960A49aAe739E15cDB6cfDcc5782', 'decimals': 9},
    'SHIWA': {'address': '0x5022Cb6D39001CDD6F996e8a66500c86128f1cc4', 'decimals': 9},
    'LSETH': {'address': '0x628eBC64A38269E031AFBDd3C5BA857483B5d048', 'decimals': 18},
    'BXX': {'address': '0x6b1a8f210Ec6b7B6643cea3583Fb0c079f367898', 'decimals': 18},
    'NITRO': {'address': '0x0335A7610D817aeCA1bEBbEfbd392ecC2eD587B8', 'decimals': 18},
    'QQQ': {'address': '0x2822f6D1B2f41F93f33d937bc7d84A8Dfa4f4C21', 'decimals': 18},
    'CEEK': {'address': '0xb056c38f6b7Dc4064367403E26424CD2c60655e1', 'decimals': 18},
    'SHIBP': {'address': '0xC90906d45046059221e908de5Cc8fcfaCA859235', 'decimals': 18},
    'BICO': {'address': '0xF17e65822b568B3903685a7c9F496CF7656Cc6C2', 'decimals': 18},
    'UGT': {'address': '0x9cf98Eb8A8B28c83E8612046cf55701Ce3Eb0063', 'decimals': 18},
    'MUU': {'address': '0x849C479D7A90Eb378dbD00E8f166371176244eb1', 'decimals': 9},
    'AAMMDAI': {'address': '0x79bE75FFC64DD58e66787E4Eae470c8a1FD08ba4', 'decimals': 18},
    'SILO': {'address': '0x6f80310CA7F2C654691D1383149Fa1A57d8AB1f8', 'decimals': 18},
    'ICONS': {'address': '0x3F68e7B44e9bCB486C2FeAdB7A2289D9cdFC9088', 'decimals': 18},
    'ASD': {'address': '0xff742d05420B6Aca4481F635aD8341F81A6300C2', 'decimals': 18},
    'GTH': {'address': '0xeb986DA994E4a118d5956b02d8b7c3C7CE373674', 'decimals': 18},
    'IUSD': {'address': '0x0A3BB08b3a15A19b4De82F8AcFc862606FB69A2D', 'decimals': 18},
    'STN': {'address': '0xe63d6B308BCe0F6193AeC6b7E6eBa005f41e36AB', 'decimals': 18},
    'TYP': {'address': '0xb46eda6219ba121cE9280388e7afb7dc84be3fF2', 'decimals': 0},
    'DGTV': {'address': '0xFEb6d5238Ed8F1d59DCaB2db381AA948e625966D', 'decimals': 18},
    'MC': {'address': '0x949D48EcA67b17269629c7194F4b727d4Ef9E5d6', 'decimals': 18},
    'PAY': {'address': '0xB97048628DB6B661D4C2aA833e95Dbe1A905B280', 'decimals': 18},
    'APIX': {'address': '0xf51EBf9a26DbC02B13F8B3a9110dac47a4d62D78', 'decimals': 18},
    'DOOMER': {'address': '0x04b77A1be2981c1Ca353aaf251b0F11398413bFA', 'decimals': 18},
    'GENE': {'address': '0x21413c119b0C11C5d96aE1bD328917bC5C8ED67E', 'decimals': 18},
    'YCRV': {'address': '0xFCc5c47bE19d06BF83eB04298b026F81069ff65b', 'decimals': 18},
    'MTC': {'address': '0x905E337c6c8645263D3521205Aa37bf4d034e745', 'decimals': 18},
    'DODI': {'address': '0x4E08F03079c5CD3083eA331Ec61bCC87538B7665', 'decimals': 18},
    'QANX': {'address': '0xAAA9214F675316182Eaa21C85f0Ca99160CC3AAA', 'decimals': 18},
    'KAINET': {'address': '0xB097Dd6a23175C10eAA45A1F410d7eB4cFDAFC8a', 'decimals': 9},
    'BFC': {'address': '0x0c7D5ae016f806603CB1782bEa29AC69471CAb9c', 'decimals': 18},
    'POLY': {'address': '0x9d93692E826A4bd9e903e2A27D7FbD1e116efdad', 'decimals': 9},
    'ROOK': {'address': '0xfA5047c9c78B8877af97BDcb85Db743fD7313d4a', 'decimals': 18},
    'CFXT': {'address': '0x368BF9F1A1CA767935E39F20439D9041707E2634', 'decimals': 18},
    'MTS': {'address': '0xA9598333B99d14D90Bc81CAd8Af82C4C70625e75', 'decimals': 18},
    'CTX': {'address': '0x321C2fE4446C7c963dc41Dd58879AF648838f98D', 'decimals': 18},
    'GUM': {'address': '0x4f5fa8f2d12e5eB780f6082Dd656C565C48E0f24', 'decimals': 18},
    'EVER': {'address': '0x29d578CEc46B50Fa5C88a99C6A4B70184C062953', 'decimals': 9},
    'WIZARD': {'address': '0x87931E7AD81914e7898d07c68F145fC0A553D8Fb', 'decimals': 18},
    'MET': {'address': '0x2Ebd53d035150f328bd754D6DC66B99B0eDB89aa', 'decimals': 18},
    'NIFT': {'address': '0x4a1d542b52a95AD01Ddc70C2e7df0c7bBAAdc56f', 'decimals': 18},
    'KKO': {'address': '0x368C5290b13cAA10284Db58B4ad4F3E9eE8bf4c9', 'decimals': 18},
    'BTZ': {'address': '0x5fBc3cb8B428cC00A04808870295D39962CC7FeE', 'decimals': 18},
    'SHIBAKEN': {'address': '0xa4Cf2aFD3B165975afFFBf7e487CDd40C894Ab6B', 'decimals': 0},
    'ASNX': {'address': '0x35f6B052C598d933D69A4EEC4D04c73A191fE6c2', 'decimals': 18},
    'BES': {'address': '0x805856f176625843d14097017618E224F4607D2D', 'decimals': 18},
    'BMT': {'address': '0xf028ADEe51533b1B47BEaa890fEb54a457f51E89', 'decimals': 18},
    'YVWBTC': {'address': '0xA696a63cc78DfFa1a63E9E50587C197387FF6C7E', 'decimals': 8},
    'BHNY': {'address': '0xCc802c45B55581713cEcd1Eb17BE9Ab7fcCb0844', 'decimals': 18},
    'SUB': {'address': '0x8D75959f1E61EC2571aa72798237101F084DE63a', 'decimals': 18},
    'MNTE': {'address': '0x94D40B49F020BfeBbA1a80A0191EB3737B90E8d3', 'decimals': 18},
    'BURP': {'address': '0x33f391F4c4fE802b70B77AE37670037A92114A7c', 'decimals': 18},
    'SBTC': {'address': '0xfE18be6b3Bd88A2D2A7f928d00292E7a9963CfC6', 'decimals': 18},
    'REVO': {'address': '0x155040625D7ae3e9caDA9a73E3E44f76D3Ed1409', 'decimals': 18},
    'DOGENS': {'address': '0x1b4dD5eA240732dDAc8d74FD1Cd9026Addc02e3c', 'decimals': 9},
    'BITE': {'address': '0x4eED0fa8dE12D5a86517f214C2f11586Ba2ED88D', 'decimals': 18},
    'HDOT': {'address': '0x9ffc3bCDe7B68C46a6dC34f0718009925c1867cB', 'decimals': 18},
    'GLCH': {'address': '0x038a68FF68c393373eC894015816e33Ad41BD564', 'decimals': 18},
    'SWASH': {'address': '0xA130E3a33a4d84b04c3918c4E5762223Ae252F80', 'decimals': 18},
    'IBEUR': {'address': '0x96E61422b6A9bA0e068B6c5ADd4fFaBC6a4aae27', 'decimals': 18},
    'ABR': {'address': '0xa11bD36801d8fa4448F0ac4ea7A62e3634cE8C7C', 'decimals': 18},
    'NPX': {'address': '0x28b5E12CcE51f15594B0b91d5b5AdaA70F684a02', 'decimals': 2},
    'PSTAKE': {'address': '0xfB5c6815cA3AC72Ce9F5006869AE67f18bF77006', 'decimals': 18},
    'YOP': {'address': '0xAE1eaAE3F627AAca434127644371b67B18444051', 'decimals': 8},
    'AIOZ': {'address': '0x626E8036dEB333b408Be468F951bdB42433cBF18', 'decimals': 18},
    'SAI': {'address': '0x89d24A6b4CcB1B6fAA2625fE562bDD9a23260359', 'decimals': 18},
    'HIBAYC': {'address': '0x83031984B45553070a088273f341BfF7fB4f2F46', 'decimals': 18},
    'BTCRED': {'address': '0x6Aac8CB9861E42bf8259F5AbDC6aE3Ae89909E11', 'decimals': 8},
    'BIOT': {'address': '0xc07A150ECAdF2cc352f5586396e344A6b17625EB', 'decimals': 9},
    'APOLLO': {'address': '0xADf86E75d8f0F57e0288D0970E7407eaA49b3CAb', 'decimals': 9},
    'WOOF': {'address': '0x6BC08509B36A98E829dFfAD49Fde5e412645d0a3', 'decimals': 18},
    'LTO': {'address': '0xd01409314aCb3b245CEa9500eCE3F6Fd4d70ea30', 'decimals': 8},
    'TONE': {'address': '0x2Ab6Bb8408ca3199B8Fa6C92d5b455F820Af03c4', 'decimals': 18},
    'LPNT': {'address': '0x6a4C76874e686A7d080D173987A35A9c48905583', 'decimals': 18},
    'GS': {'address': '0xe0B9a2C3E9f40CF74B2C7F591B2b0CCa055c3112', 'decimals': 18},
    'CUNI': {'address': '0x35A18000230DA775CAc24873d00Ff85BccdeD550', 'decimals': 8},
    'SPGBB': {'address': '0x64551fA4093c2D6c298cf1792740F884084a8F69', 'decimals': 18},
    'UCOIL': {'address': '0x549044000a94870ab7c5017cd8fB0AEfa9239a13', 'decimals': 18},
    'SKRT': {'address': '0x887168120cb89Fb06F3E74Dc4AF20D67dF0977f6', 'decimals': 18},
    'FTN': {'address': '0xaeDf386B755465871fF874E3E37Af5976E247064', 'decimals': 18},
    'FOREX': {'address': '0xDb298285FE4C5410B05390cA80e8Fbe9DE1F259B', 'decimals': 18},
    'APOLLO': {'address': '0x425d105913a04f4F95311788195bA13c3E82c3B9', 'decimals': 9},
    '4ART': {'address': '0xFF44b5719f0B77A9951636fc5e69d3a1fc9E7d73', 'decimals': 18},
    'WOM': {'address': '0xBd356a39BFf2cAda8E9248532DD879147221Cf76', 'decimals': 18},
    'SPH': {'address': '0xA0CF46eb152656C7090e769916eb44a138aaa406', 'decimals': 18},
    'YEET': {'address': '0x429acA1cCd47296483D1281c85B24e842de0b758', 'decimals': 18},
    'LSVR': {'address': '0x79A06aCb8bdd138BEEECcE0f1605971f3AC7c09B', 'decimals': 18},
    'LUCHOW': {'address': '0xA5Ef74068d04ba0809B7379dD76Af5Ce34Ab7C57', 'decimals': 18},
    'QRX': {'address': '0x6e0daDE58D2d89eBBe7aFc384e3E4f15b70b14D8', 'decimals': 18},
    'SFG': {'address': '0xA71DCaaf5ba390B1B2012323A8810fAc121F90dd', 'decimals': 18},
    'SURE': {'address': '0xcb86c6A22CB56B6cf40CaFEDb06BA0DF188a416E', 'decimals': 18},
    'NIIFI': {'address': '0x852e5427c86A3b46DD25e5FE027bb15f53c4BCb8', 'decimals': 15},
    'HMR': {'address': '0xB1A30851E3f7d841b231B086479608e17198363A', 'decimals': 18},
    'PLSB': {'address': '0x5EE84583f67D5EcEa5420dBb42b462896E7f8D06', 'decimals': 12},
    'PARA': {'address': '0x3a8d5BC8A8948b68DfC0Ce9C14aC4150e083518c', 'decimals': 18},
    'M': {'address': '0xC3E486F614e297d8E016aC2805e81707C627b2d5', 'decimals': 18},
    'AKRO': {'address': '0x8Ab7404063Ec4DBcfd4598215992DC3F8EC853d7', 'decimals': 18},
    'PERL': {'address': '0xeca82185adCE47f39c684352B0439f030f860318', 'decimals': 18},
    'CRPT': {'address': '0x08389495D7456E1951ddF7c3a1314A4bfb646d8B', 'decimals': 18},
    'SHIBTAMA': {'address': '0xBc276f51184ec085cd1437601eCb4bBc3D981896', 'decimals': 18},
    'PAW': {'address': '0xDc63269eA166b70d4780b3A11F5C825C2b761B01', 'decimals': 18},
    'SOCKS': {'address': '0x23B608675a2B2fB1890d3ABBd85c5775c51691d5', 'decimals': 18},
    'DETO': {'address': '0xaB93dF617F51E1E415b5b4f8111f122d6b48e55C', 'decimals': 18},
    'CHOO': {'address': '0x9c5476Af06590A9277C7706fe70c0a42A480F8A0', 'decimals': 18},
    'CYFM': {'address': '0x4a621d9f1b19296d1C0f87637b3A8D4978e9bf82', 'decimals': 18},
    'POW': {'address': '0x43Ab765ee05075d78AD8aa79dcb1978CA3079258', 'decimals': 18},
    'PYRO': {'address': '0x1e2D230C7A7F4C679Fb1378F1f51dEDeAe85Cd72', 'decimals': 18},
    'KNIGHT': {'address': '0x3541A5C1b04AdABA0B83F161747815cd7B1516bC', 'decimals': 18},
    'PC': {'address': '0xe1eC350ea16D1DdAFF57F31387B2d9708Eb7ce28', 'decimals': 9},
    'CHAIN': {'address': '0xC4C2614E694cF534D407Ee49F8E44D125E4681c4', 'decimals': 18},
    'AVINOC': {'address': '0xF1cA9cb74685755965c7458528A36934Df52A3EF', 'decimals': 18},
    'DXN': {'address': '0x80f0C1c49891dcFDD40b6e0F960F84E6042bcB6F', 'decimals': 18},
    'XIO': {'address': '0x0f7F961648aE6Db43C75663aC7E5414Eb79b5704', 'decimals': 18},
    'MSB': {'address': '0x84c722e6F1363E8D5C6dB3eA600bEF9a006Da824', 'decimals': 18},
    'CBAT': {'address': '0x6C8c6b02E7b2BE14d4fA6022Dfd6d75921D90E4E', 'decimals': 8},
    'IDTT': {'address': '0x6fB1E018f107d3352506c23777e4cd62e063584a', 'decimals': 18},
    'VALUE': {'address': '0x49E833337ECe7aFE375e44F4E3e8481029218E5c', 'decimals': 18},
    'CHEW': {'address': '0xf5d126077096e5b01BC30FFa5d9324D7202d7Cb3', 'decimals': 18},
    'BIGSB': {'address': '0x131157c6760f78f7dDF877C0019Eba175BA4b6F6', 'decimals': 18},
    'FLZ': {'address': '0x8E964e35A76103Af4C7D7318e1B1a82c682ae296', 'decimals': 18},
    'BLID': {'address': '0x8A7aDc1B690E81c758F1BD0F72DFe27Ae6eC56A5', 'decimals': 18},
    'BMI': {'address': '0x725C263e32c72dDC3A19bEa12C5a0479a81eE688', 'decimals': 18},
    'GNX': {'address': '0x6EC8a24CaBdc339A06a172F8223ea557055aDAa5', 'decimals': 9},
    'STREETH': {'address': '0xB840d10D840eF47c233FEC1fd040F5B145a6DfA5', 'decimals': 18},
    'XDEX': {'address': '0x000000000000d0151E748d25b766e77efe2A6c83', 'decimals': 18},
    'ARPA': {'address': '0xBA50933C268F567BDC86E1aC131BE072C6B0b71a', 'decimals': 18},
    'IDLEUSDTYIELD': {'address': '0xF34842d05A1c888Ca02769A633DF37177415C2f8', 'decimals': 18},
    'DREAM': {'address': '0xb44377B74Ef1773639B663d0754cB8410A847d02', 'decimals': 18},
    'SER': {'address': '0xb19189Fb36c816f3e0f16065057B07B790998fDC', 'decimals': 18},
    'EVOAI': {'address': '0x6Fbc20483b53CEa47839Bb8e171Abd6D67C3c696', 'decimals': 9},
    'CREDIT': {'address': '0xC4cB5793BD58BaD06bF51FB37717b86B02CBe8A4', 'decimals': 18},
    'VNXAU': {'address': '0x6d57B2E05F26C26b549231c866bdd39779e4a488', 'decimals': 18},
    'GODS': {'address': '0xccC8cb5229B0ac8069C51fd58367Fd1e622aFD97', 'decimals': 18},
    'EVN': {'address': '0xd780Ae2Bf04cD96E577D3D014762f831d97129d0', 'decimals': 18},
    'FNF': {'address': '0xAcf8D5E515Ed005655DfeFA09C22673a37A7CDEE', 'decimals': 18},
    'LUNC': {'address': '0xd2877702675e6cEb975b4A1dFf9fb7BAF4C91ea9', 'decimals': 18},
    'FLURRY': {'address': '0x60F63B76E2Fc1649E57a3489162732A90ACf59FE', 'decimals': 18},
    'ORAI': {'address': '0x4c11249814f11b9346808179Cf06e71ac328c1b5', 'decimals': 18},
    'NAMI': {'address': '0x2f7B618993cc3848d6C7ed9CdD5e835E4Fe22b98', 'decimals': 18},
    'MVD': {'address': '0x788B6D2B37Aa51D916F2837Ae25b05f0e61339d1', 'decimals': 9},
    'REVOAI': {'address': '0x8793Fb615Eb92822F482f88B3137B00aad4C00D2', 'decimals': 9},
    'TRND': {'address': '0xc3dD23A0a854b4f9aE80670f528094E9Eb607CCb', 'decimals': 18},
    'ESS': {'address': '0xfc05987bd2be489ACCF0f509E44B0145d68240f7', 'decimals': 18},
    'JIG': {'address': '0xe2Fc859C838F31C1450448f4FeD92e3284aE49E9', 'decimals': 18},
    'CHEDDA': {'address': '0x16756EC1DEb89A2106C35E0B586a799D0A61837D', 'decimals': 18},
    'APY': {'address': '0x95a4492F028aa1fd432Ea71146b433E7B4446611', 'decimals': 18},
    'INSAI': {'address': '0xceec4aE5CF5300aC9A49a816961Af9393b214674', 'decimals': 6},
    'ANARCHY': {'address': '0x53fD2342B43eCD24AEf1535BC3797F509616Ce8c', 'decimals': 9},
    'GT': {'address': '0xE66747a101bFF2dBA3697199DCcE5b743b454759', 'decimals': 18},
    'MAD': {'address': '0x5B09A0371C1DA44A8E24D36Bf5DEb1141a84d875', 'decimals': 18},
    'FDT': {'address': '0xEd1480d12bE41d92F36f5f7bDd88212E381A3677', 'decimals': 18},
    'SCAN': {'address': '0x234D51eE02be808A0160b19b689660Fb7BFA871b', 'decimals': 9},
    'MOMA': {'address': '0xbd1848e1491d4308Ad18287A745DD4DB2A4BD55B', 'decimals': 18},
    'ZCX': {'address': '0xC52C326331E9Ce41F04484d3B5E5648158028804', 'decimals': 18},
    'ORS': {'address': '0xEB9A4B185816C354dB92DB09cC3B50bE60b901b6', 'decimals': 18},
    'SAITANOBI': {'address': '0x5e9F35E8163c44cD7e606BdD716AbED32AD2F1C6', 'decimals': 9},
    'EGC': {'address': '0xc0EC8CaEC55F37D47fBfA595727418868A21fd48', 'decimals': 8},
    'PTOY': {'address': '0x8Ae4BF2C33a8e667de34B54938B0ccD03Eb8CC06', 'decimals': 8},
    'OST': {'address': '0x2C4e8f2D746113d0696cE89B35F0d8bF88E0AEcA', 'decimals': 18},
    'NFTD': {'address': '0x8E0fE2947752BE0d5ACF73aaE77362Daf79cB379', 'decimals': 18},
    'X2Y2': {'address': '0x1E4EDE388cbc9F4b5c79681B7f94d36a11ABEBC9', 'decimals': 18},
    'OIL': {'address': '0x0275E1001e293C46CFe158B3702AADe0B99f88a5', 'decimals': 18},
    'ICOM': {'address': '0xB131F337C45D386cEeC234e194b2663D5c3d9dCF', 'decimals': 18},
    'EMON': {'address': '0xd6A5aB46ead26f49b03bBB1F9EB1Ad5c1767974a', 'decimals': 18},
    'LKT': {'address': '0xd3E133a0A14Bb8B595e5Fbf9851c7c783685BA69', 'decimals': 18},
    'DVC': {'address': '0x5E6FFe7B174A50c81fF3f3c54c04fD3c11E20830', 'decimals': 18},
    'GYSR': {'address': '0xbEa98c05eEAe2f3bC8c3565Db7551Eb738c8CCAb', 'decimals': 18},
    'LYNK': {'address': '0xef952363C1d990A2fa58F8b379A9fa33bad1dfd1', 'decimals': 8},
    'MRCH': {'address': '0xBeD4AB0019ff361d83ddeB74883DAC8a70f5ea1e', 'decimals': 18},
    'GETH': {'address': '0xdD69DB25F6D620A7baD3023c5d32761D353D3De9', 'decimals': 18},
    'ETHY': {'address': '0xd1AFBCCC9A2c2187ea544363B986EA0AB6EF08B5', 'decimals': 18},
    'GLM': {'address': '0x7DD9c5Cba05E151C895FDe1CF355C9A1D5DA6429', 'decimals': 18},
    'MCHC': {'address': '0xD69F306549e9d96f183B1AecA30B8f4353c2ECC3', 'decimals': 18},
    'DEXTF': {'address': '0x5F64Ab1544D28732F0A24F4713c2C8ec0dA089f0', 'decimals': 18},
    'MAYC': {'address': '0x94c9cEb2F9741230FAD3a62781b27Cc79a9460d4', 'decimals': 18},
    'BCUBE': {'address': '0x93C9175E26F57d2888c7Df8B470C9eeA5C0b0A93', 'decimals': 18},
    'GST-ETH': {'address': '0x473037de59cf9484632f4A27B509CFE8d4a31404', 'decimals': 8},
    'STAK': {'address': '0x1F8A626883d7724DBd59eF51CBD4BF1Cf2016D13', 'decimals': 18},
    'BERRY': {'address': '0x2d787D4f5005Bd66aC910c2E821241625c406ed5', 'decimals': 18},
    'QUAD': {'address': '0xAB2A7B5876D707e0126B3A75EF7781c77c8877EE', 'decimals': 18},
    'CETH': {'address': '0x4Ddc2D193948926D02f9B1fE9e1daa0718270ED5', 'decimals': 8},
    'KISHU': {'address': '0xA2b4C0Af19cC16a6CfAcCe81F192B024d625817D', 'decimals': 9},
    'PPAI': {'address': '0xe616ADb3f729f6f3df19C876Eda66472a308D397', 'decimals': 18},
    'INU': {'address': '0x00F29171D7bCDC464a0758cF3217fE83173772b9', 'decimals': 9},
    'DPY': {'address': '0x6C2adC2073994fb2CCC5032cC2906Fa221e9B391', 'decimals': 18},
    'BENT': {'address': '0x01597E397605Bf280674Bf292623460b4204C375', 'decimals': 18},
    'GAINS': {'address': '0xd9b312D77Bc7BEd9B9CecB56636300bED4Fe5Ce9', 'decimals': 18},
    'STLINK': {'address': '0xb8b295df2cd735b15BE5Eb419517Aa626fc43cD5', 'decimals': 18},
    'SKRIMP': {'address': '0xec6d73557937974077911a0B6fDc436b0ff70296', 'decimals': 18},
    'HPT': {'address': '0xa66Daa57432024023DB65477BA87D4E7F5f95213', 'decimals': 18},
    'NOISEGPT': {'address': '0xf23a2BF7619Ad93300499c3fC7d7145A06CcA562', 'decimals': 18},
    'LEAP': {'address': '0x53263d9EF74Db583b15fbC6D5D4e8B83833fa134', 'decimals': 18},
    'PLSD': {'address': '0x34F0915a5f15a66Eba86F6a58bE1A471FB7836A7', 'decimals': 12},
    'SPOOL': {'address': '0x40803cEA2b2A32BdA1bE61d3604af6a814E70976', 'decimals': 18},
    'XP': {'address': '0x948c70Dc6169Bfb10028FdBE96cbC72E9562b2Ac', 'decimals': 18},
    'CULT': {'address': '0xf0f9D895aCa5c8678f706FB8216fa22957685A13', 'decimals': 18},
    'INUINU': {'address': '0xC6bDb96E29c38DC43f014Eed44dE4106A6A8eB5f', 'decimals': 18},
    'MOD': {'address': '0xEA1ea0972fa092dd463f2968F9bB51Cc4c981D71', 'decimals': 18},
    'ARTX': {'address': '0x741b0428Efdf4372A8DF6FB54B018dB5e5aB7710', 'decimals': 18},
    'XELS': {'address': '0x397Deb686C72384FAd502A81f4d7fDb89e1f1280', 'decimals': 8},
    'SEUR': {'address': '0xD71eCFF9342A5Ced620049e616c5035F1dB98620', 'decimals': 18},
    'FLONA': {'address': '0xEf27252B567F6B3fe35b34A85bE322917abE524A', 'decimals': 18},
    'RAIN': {'address': '0x71Fc1F555a39E0B698653AB0b475488EC3c34D57', 'decimals': 18},
    'ZEE': {'address': '0x2eDf094dB69d6Dcd487f1B3dB9febE2eeC0dd4c5', 'decimals': 18},
    'BTO': {'address': '0x36905Fc93280f52362A1CBAB151F25DC46742Fb5', 'decimals': 18},
    'DMT': {'address': '0x5b1D655C93185b06B00f7925791106132Cb3ad75', 'decimals': 18},
    'PREMIA': {'address': '0x6399C842dD2bE3dE30BF99Bc7D1bBF6Fa3650E70', 'decimals': 18},
    'HEX': {'address': '0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39', 'decimals': 8},
    '0X0': {'address': '0x5a3e6A77ba2f983eC0d371ea3B475F8Bc0811AD5', 'decimals': 9},
    'GFARM2': {'address': '0x831091dA075665168E01898c6DAC004A867f1e1B', 'decimals': 18},
    'AMKR': {'address': '0xc713e5E149D5D0715DcD1c156a020976e7E56B88', 'decimals': 18},
    'GPT': {'address': '0xD04E772BC0d591fBD288f2E2a86aFA3D3CB647F8', 'decimals': 18},
    'LINA': {'address': '0xC05d14442A510De4D3d71a3d316585aA0CE32b50', 'decimals': 18},
    'MNMC': {'address': '0xF45091f25d374BbE956c0bb64bB85e02D07Aa741', 'decimals': 8},
    'GRLC': {'address': '0x58f7345b5295E43aA454911571f13be186655BE9', 'decimals': 8},
    'CRYN': {'address': '0xC31CEBf8F9E825d1D1244D73d0a65e44bD5210DB', 'decimals': 8},
    'MON': {'address': '0x1EA48B9965bb5086F3b468E50ED93888a661fc17', 'decimals': 18},
    'QKC': {'address': '0xEA26c4aC16D4a5A106820BC8AEE85fd0b7b2b664', 'decimals': 18},
    'SKULL': {'address': '0xBcc66ed2aB491e9aE7Bf8386541Fb17421Fa9d35', 'decimals': 4},
    'KEY': {'address': '0x4CC19356f2D37338b9802aa8E8fc58B0373296E7', 'decimals': 18},
    'RWS': {'address': '0x08AD83D779BDf2BBE1ad9cc0f78aa0D24AB97802', 'decimals': 18},
    'AMANA': {'address': '0xa685a61171bb30d4072B338c80Cb7b2c865c873E', 'decimals': 18},
    'DWAGON': {'address': '0xdf1A2f85F3af80E85d14DdAAB2933C8caA09294a', 'decimals': 18},
    'FLASH': {'address': '0xB1f1F47061A7Be15C69f378CB3f69423bD58F2F8', 'decimals': 18},
    'ST-YCRV': {'address': '0x27B5739e22ad9033bcBf192059122d163b60349D', 'decimals': 18},
    'GDR': {'address': '0x874D4C9B980f1a13dD44CBcDB912e24Ef0671eD0', 'decimals': 18},
    'SAITAX': {'address': '0xC795fBa221f7920F1C6ac0f1598886742D8Ea661', 'decimals': 9},
    'VEE': {'address': '0x7616113782AaDAB041d7B10d474F8A0c04EFf258', 'decimals': 18},
    'AENJ': {'address': '0xaC6Df26a590F08dcC95D5a4705ae8abbc88509Ef', 'decimals': 18},
    'N1': {'address': '0xaCbd826394189Cf2623C6DF98a18b41fC8fFC16D', 'decimals': 18},
    'EVY': {'address': '0xEEd3aE7b0F8b5B9BB8C035A9941382B1822671CD', 'decimals': 12},
    'EQZ': {'address': '0x1Da87b114f35E1DC91F72bF57fc07A768Ad40Bb0', 'decimals': 18},
    'BCDT': {'address': '0xAcfa209Fb73bF3Dd5bBfb1101B9Bc999C49062a5', 'decimals': 18},
    'DFP2': {'address': '0x2F57430a6ceDA85a67121757785877b4a71b8E6D', 'decimals': 18},
    'BCP': {'address': '0xE4f726Adc8e89C6a6017F01eadA77865dB22dA14', 'decimals': 18},
    'JBX': {'address': '0x3abF2A4f8452cCC2CF7b4C1e4663147600646f66', 'decimals': 18},
    'APW': {'address': '0x4104b135DBC9609Fc1A9490E61369036497660c8', 'decimals': 18},
    'CASTLE': {'address': '0xf9d4DAaE1300CfF251979722C4A3C45857973079', 'decimals': 18},
    'POOH': {'address': '0xB69753c06BB5c366BE51E73bFc0cC2e3DC07E371', 'decimals': 18},
    'EMP': {'address': '0x13A0599c493Cc502ED8C4Dd26F22F5cEDC248fC4', 'decimals': 18},
    'TRACE': {'address': '0x9F7fC686CfD64aA5Ae15b351d03071e91533094b', 'decimals': 18},
    'RINIA': {'address': '0x8f828a0644f12Fa352888E645a90333d30f6FD7D', 'decimals': 8},
    'FUN': {'address': '0x419D0d8BdD9aF5e606Ae2232ed285Aff190E711b', 'decimals': 8},
    'SPRING': {'address': '0xf04aF3f4E4929F7CD25A751E6149A3318373d4FE', 'decimals': 18},
    'UPP': {'address': '0xC86D054809623432210c107af2e3F619DcFbf652', 'decimals': 18},
    'MORPHO': {'address': '0x9994E35Db50125E0DF82e4c2dde62496CE330999', 'decimals': 18},
    'PAL': {'address': '0xAB846Fb6C81370327e784Ae7CbB6d6a6af6Ff4BF', 'decimals': 18},
    'BYTES': {'address': '0x7d647b1A0dcD5525e9C6B3D14BE58f27674f8c95', 'decimals': 18},
    'TITAN': {'address': '0x3A8cCCB969a61532d1E6005e2CE12C200caeCe87', 'decimals': 18},
    'HLD': {'address': '0xCA1308E38340C69E848061aA2C3880daeB958187', 'decimals': 9},
    'ZRO': {'address': '0xd79F43113B22D1eA9F29cfcC7BB287489F8EE5e0', 'decimals': 18},
    'MNB': {'address': '0x42b91f1d05afeA671a2dA3c780eDa2aBf0A2A366', 'decimals': 18},
    'TNT': {'address': '0x08f5a9235B08173b7569F83645d2c7fB55e8cCD8', 'decimals': 8},
    'ZEUM': {'address': '0x436dA116249044E8B4464F0Cf21Dd93311d88190', 'decimals': 18},
    'D2O': {'address': '0x2FdA8c6783Aa36BeD645baD28a4cDC8769dCD252', 'decimals': 18},
    'ENJ': {'address': '0xF629cBd94d3791C9250152BD8dfBDF380E2a3B9c', 'decimals': 18},
    'OSQTH': {'address': '0xf1B99e3E573A1a9C5E6B2Ce818b617F0E664E86B', 'decimals': 18},
    'ASM': {'address': '0x2565ae0385659badCada1031DB704442E1b69982', 'decimals': 18},
    'NOW': {'address': '0xE9A95d175a5f4C9369f3b74222402eB1b837693b', 'decimals': 8},
    'GF': {'address': '0xAaEf88cEa01475125522e117BFe45cF32044E238', 'decimals': 18},
    '1SOL': {'address': '0x009178997aFf09A67D4cACcFeB897Fb79d036214', 'decimals': 18},
    'NEXM': {'address': '0xe831F96A7a1DcE1aa2EB760b1e296c6A74CaA9d5', 'decimals': 8},
    'ARCH': {'address': '0x73C69d24ad28e2d43D03CBf35F79fE26EBDE1011', 'decimals': 18},
    'DON': {'address': '0x217ddEad61a42369A266F1Fb754EB5d3EBadc88a', 'decimals': 18},
    'CBX': {'address': '0x37FC4b48CE93469dbEA9918468993C735049642a', 'decimals': 18},
    'LAKE': {'address': '0xF9Ca9523E5b5A42C3018C62B084Db8543478C400', 'decimals': 18},
    'DUBBZ': {'address': '0x38029C62DfA30D9FD3CaDf4C64e9b2ab21DbDa17', 'decimals': 18},
    'BANK': {'address': '0x2d94AA3e47d9D5024503Ca8491fcE9A2fB4DA198', 'decimals': 18},
    'SUZUME': {'address': '0x0B452278223D3954F4AC050949D7998e373e7E43', 'decimals': 18},
    'ETHPAD': {'address': '0x8dB1D28Ee0d822367aF8d220C0dc7cB6fe9DC442', 'decimals': 18},
    'DOLA': {'address': '0x865377367054516e17014CcdED1e7d814EDC9ce4', 'decimals': 18},
    'MYTH': {'address': '0xBA41Ddf06B7fFD89D1267b5A93BFeF2424eb2003', 'decimals': 18},
    'MIN': {'address': '0x5D64D850c8368008aFB39224E92aD0DcEFf3CF38', 'decimals': 18},
    'SHINSHU': {'address': '0x2965395F71B7d97ede251E9B63e44dfA9647cC0A', 'decimals': 18},
    'SHIELD': {'address': '0xcf4C68Db4C2FA0bf58dF07B14f45cE7709a716AC', 'decimals': 18},
    'IWBTC': {'address': '0xEC363faa5c4dd0e51f3D9B5d0101263760E7cdeB', 'decimals': 8},
    'YVRAI': {'address': '0x873fB544277FD7b977B196a826459a69E27eA4ea', 'decimals': 18},
    'EGAME': {'address': '0x62d3c05b9C3d916FBC111819bbD3CEE52906C1Ae', 'decimals': 18},
    'HIKARI': {'address': '0xd4126f195a8de772EeFfa61a4aB6dd43462F4e39', 'decimals': 18},
    'XDEUS': {'address': '0x953Cd009a490176FcEB3a26b9753e6F01645ff28', 'decimals': 18},
    'POLC': {'address': '0xaA8330FB2B4D5D07ABFE7A72262752a8505C6B37', 'decimals': 18},
    'STORE': {'address': '0x31ea0de8119307aA264Bb4b38727aAb4E36b074f', 'decimals': 18},
    'GOV': {'address': '0x274E7Eb07B485CFdE53d02270555213447570aC6', 'decimals': 18},
    'DIS': {'address': '0x220B71671b649c03714dA9c621285943f3cbcDC6', 'decimals': 18},
    'SHR': {'address': '0xd98F75b1A3261dab9eEd4956c93F33749027a964', 'decimals': 2},
    'SQUEEZE': {'address': '0xaBd4dc8fDe9848CBc4Ff2c0Ee81d4A49F4803Da4', 'decimals': 9},
    'YFX': {'address': '0xF55a93b613D172b86c2Ba3981a849DaE2aeCDE2f', 'decimals': 18},
    'BNF': {'address': '0x1DE5e000C41C8d35b9f1f4985C23988f05831057', 'decimals': 18},
    'BPLC': {'address': '0x426FC8BE95573230f6e6bc4af91873F0c67b21b4', 'decimals': 18},
    'GOF': {'address': '0x488E0369f9BC5C40C002eA7c1fe4fd01A198801c', 'decimals': 18},
    'GYEN': {'address': '0xC08512927D12348F6620a698105e1BAac6EcD911', 'decimals': 6},
    'GST': {'address': '0x3AFA1902b1f8a802aBC18e5aD982D1bCd34AfE22', 'decimals': 18},
    'ETHO': {'address': '0x0b5326Da634f9270FB84481DD6F94d3dC2cA7096', 'decimals': 18},
    'IDAI': {'address': '0x40a9d39aa50871Df092538c5999b107f34409061', 'decimals': 18},
    'DG': {'address': '0x53C8395465A84955c95159814461466053DedEDE', 'decimals': 18},
    'IGG': {'address': '0x8FfE40A3D0f80C0CE6b203D5cDC1A6a86d9AcaeA', 'decimals': 6},
    'DIA': {'address': '0xFde00bD1Da57349b0DF85F134b18124266F3bC5b', 'decimals': 18},
    'MONTE': {'address': '0x318Ee488AF8881f9945B6d3D69e8B395Fb559bB1', 'decimals': 18},
    'CRVRENWSBTC': {'address': '0x075b1bb99792c9E1041bA13afEf80C91a1e70fB3', 'decimals': 18},
    'AETH': {'address': '0x3a3A65aAb0dd2A17E3F1947bA16138cd37d08c04', 'decimals': 18},
    'PAXG': {'address': '0x45804880De22913dAFE09f4980848ECE6EcbAf78', 'decimals': 18},
    'LOVE': {'address': '0x5380442d3C4EC4f5777f551f5EDD2FA0F691A27C', 'decimals': 18},
    'SPIZ': {'address': '0xa7A5c1058194Af8F00c187adB7FcC0c95f1C6c2d', 'decimals': 18},
    'KP3R': {'address': '0x1cEB5cB57C4D4E2b2433641b95Dd330A33185A44', 'decimals': 18},
    'MECHA': {'address': '0xC5bcc8BA3F33Ab0d64f3473e861BDC0685b19Ef5', 'decimals': 18},
    'MNS': {'address': '0x53884b61963351C283118a8E1Fc05BA464a11959', 'decimals': 18},
    'PXP': {'address': '0x95aA5d2DbD3c16ee3fdea82D5C6EC3E38CE3314f', 'decimals': 18},
    'XMETA': {'address': '0x591127253E40d4f63bF29CcF3D81FD062A149C8c', 'decimals': 18},
    'MSG': {'address': '0x308516e6dAaC88d719Af5a6D7eA2A82d8C8e7356', 'decimals': 18},
    'KIT': {'address': '0x7866E48C74CbFB8183cd1a929cd9b95a7a5CB4F4', 'decimals': 18},
    'TYRANT': {'address': '0x8EE325AE3E54e83956eF2d5952d3C8Bc1fa6ec27', 'decimals': 9},
    'TERRAFORM': {'address': '0x9C5673002d854de18e9F6Feb7f9825F88c5bf1E4', 'decimals': 18},
    'DELTA': {'address': '0x9EA3b5b4EC044b70375236A281986106457b20EF', 'decimals': 18},
    'MANDOX': {'address': '0x33D203FA03bb30b133De0fE2d6533C268bA286B6', 'decimals': 9},
    'TCR': {'address': '0xE38B72d6595FD3885d1D2F770aa23E94757F91a1', 'decimals': 8},
    'YUP': {'address': '0x69bBC3F8787d573F1BBDd0a5f40C7bA0Aee9BCC9', 'decimals': 18},
    'WZM': {'address': '0xD15A1A2A3211b58113e45809f05934252E34e2F8', 'decimals': 18},
    'PSWAP': {'address': '0x519C1001D550C0a1DaE7d1fC220f7d14c2A521BB', 'decimals': 18},
    'XIDO': {'address': '0xF6650117017FFD48B725B4EC5A00B414097108A7', 'decimals': 18},
    'NFTL': {'address': '0xe7F72Bc0252cA7B16dBb72eEee1AfcDb2429F2DD', 'decimals': 18},
    'SPDR': {'address': '0xbcD4b7dE6fde81025f74426D43165a5b0D790Fdd', 'decimals': 18},
    'ADP': {'address': '0xc314b0E758D5FF74f63e307A86EbfE183C95767b', 'decimals': 18},
    'MRG': {'address': '0xcbee6459728019CB1f2bB971dDe2eE3271BC7617', 'decimals': 18},
    'HICOOLCATS': {'address': '0x082716b6734b31791407d7Dd0e2a2c41260029b2', 'decimals': 18},
    'BORED': {'address': '0x2047ab3072B52561596Ce5E0131BDbB7C848538D', 'decimals': 9},
    'HAKKA': {'address': '0x0E29e5AbbB5FD88e28b2d355774e73BD47dE3bcd', 'decimals': 18},
    'CARR': {'address': '0x884DDBb5DC6c2cEf77d3E74c6CcCa315797d655B', 'decimals': 18},
    'EASE': {'address': '0xEa5eDef1287AfDF9Eb8A46f9773AbFc10820c61c', 'decimals': 18},
    'EYES': {'address': '0x2DCA19E944453e46d9130950Ca135461b3Bc0c30', 'decimals': 18},
    'HITOP': {'address': '0x01022591fCe0609B57670bebA60102dFd2b95D77', 'decimals': 18},
    'BID': {'address': '0x25e1474170c4c0aA64fa98123bdc8dB49D7802fa', 'decimals': 18},
    'ABAT': {'address': '0xE1BA0FB44CCb0D11b80F92f4f8Ed94CA3fF51D00', 'decimals': 18},
    'RETH': {'address': '0x9559Aaa82d9649C7A7b220E7c461d2E74c9a3593', 'decimals': 18},
    'ROCKI': {'address': '0xFf44b937788215ecA197BAaf9AF69dbdC214aa04', 'decimals': 18},
    'BITANT': {'address': '0x15Ee120fD69BEc86C1d38502299af7366a41D1a6', 'decimals': 18},
    'BCP': {'address': '0x4612021C75809160Be60DB21fBc9D6adD0B32deF', 'decimals': 18},
    'TAIL': {'address': '0x4384b85FE228AE727B129230211194E4A50877c4', 'decimals': 9},
    'RETH2': {'address': '0x20BC832ca081b91433ff6c17f85701B6e92486c5', 'decimals': 18},
    'MARS': {'address': '0x66C0DDEd8433c9EA86C8cf91237B14e10b4d70B7', 'decimals': 18},
    'MEM': {'address': '0x42dbBd5ae373FEA2FC320F62d44C058522Bb3758', 'decimals': 18},
    'BMEX': {'address': '0xB113c6CF239F60D380359b762E95C13817275277', 'decimals': 6},
    'TRG': {'address': '0x93eEB426782BD88fCD4B48D7b0368CF061044928', 'decimals': 18},
    'KEEP': {'address': '0x85Eee30c52B0b379b046Fb0F85F4f3Dc3009aFEC', 'decimals': 18},
    'JENNY': {'address': '0xcb4D749FD1FF081D6fEA3bd978de87603E098E63', 'decimals': 18},
    'NFTE': {'address': '0x0f9b80fc3c8b9123D0aEf43Df58ebDBC034A8901', 'decimals': 18},
    'OHMI': {'address': '0x4159862bcF6B4393A80550b1ed03DFfa6f90533C', 'decimals': 18},
    'ZIN': {'address': '0x033e223870f766644f7f7a4B7dc2E91573707d06', 'decimals': 18},
    'B20': {'address': '0xc4De189Abf94c57f396bD4c52ab13b954FebEfD8', 'decimals': 18},
    'PHONON': {'address': '0x758B4684BE769E92eeFeA93f60DDA0181eA303Ec', 'decimals': 18},
    'FRZ': {'address': '0x55b1e2D8b13E7acad03353FAD58fc3FA065C5822', 'decimals': 18},
    'EVN': {'address': '0x68909e586eeAC8F47315e84B4c9788DD54Ef65Bb', 'decimals': 18},
    'FLX': {'address': '0x6243d8CEA23066d098a15582d81a598b4e8391F4', 'decimals': 18},
    'DKUMA': {'address': '0x3f5dd1A1538a4F9f82E543098f01F22480B0A3a8', 'decimals': 18},
    'INX': {'address': '0xBBC7f7A6AADAc103769C66CBC69AB720f7F9Eae3', 'decimals': 18},
    'EPIK': {'address': '0x4da0C48376C277cdBd7Fc6FdC6936DEE3e4AdF75', 'decimals': 18},
    'PKT': {'address': '0x2604FA406Be957E542BEb89E6754fCdE6815e83f', 'decimals': 18},
    'COM': {'address': '0x5A9780Bfe63f3ec57f01b087cD65BD656C9034A8', 'decimals': 12},
    'HOICHI': {'address': '0xC4EE0aA2d993ca7C9263eCFa26c6f7e13009d2b6', 'decimals': 18},
    'HVE2': {'address': '0xD487892BB4C57edBe7aB401d9fe801c8FE6473f5', 'decimals': 18},
    'IBGBP': {'address': '0x69681f8fde45345C3870BCD5eaf4A05a60E7D227', 'decimals': 18},
    'MIMO': {'address': '0x90B831fa3Bebf58E9744A14D638E25B4eE06f9Bc', 'decimals': 18},
    'XSUSHI': {'address': '0x8798249c2E607446EfB7Ad49eC89dD1865Ff4272', 'decimals': 18},
    'RAINI': {'address': '0xeB953eDA0DC65e3246f43DC8fa13f35623bDd5eD', 'decimals': 18},
    'QETH': {'address': '0x93ef1Ea305D11A9b2a3EbB9bB4FCc34695292E7d', 'decimals': 18},
    'X7R': {'address': '0x70008F18Fc58928dcE982b0A69C2c21ff80Dca54', 'decimals': 18},
    'WIX': {'address': '0x8fc6a71Be7f82373e045a97859287676a399934A', 'decimals': 18},
    'HI': {'address': '0xC4f6E93AEDdc11dc22268488465bAbcAF09399aC', 'decimals': 18},
    'WASABI': {'address': '0x896e145568624a498c5a909187363AE947631503', 'decimals': 18},
    'RNBW': {'address': '0xE94B97b6b43639E238c851A7e693F50033EfD75C', 'decimals': 18},
    'FONT': {'address': '0x4C25Bdf026Ea05F32713F00f73Ca55857Fbf6342', 'decimals': 18},
    'SEELE': {'address': '0xB1e93236ab6073fdAC58adA5564897177D4bcC43', 'decimals': 18},
    'ABT': {'address': '0xB98d4C97425d9908E66E53A6fDf673ACcA0BE986', 'decimals': 18},
    'AOG': {'address': '0x8578530205CEcbe5DB83F7F29EcfEEC860C297C2', 'decimals': 18},
    'DNS': {'address': '0x6C5Aa4244113E77414386630B7F963dbE1B4d73D', 'decimals': 18},
    'SHOP': {'address': '0x64b78325d7495D6d4be92f234fa3f3B8d8964B8b', 'decimals': 18},
    'OG': {'address': '0xE89C20096b636fFec9fd26d1a623F42A33eaD309', 'decimals': 18},
    'WOZX': {'address': '0x34950Ff2b487d9E5282c5aB342d08A2f712eb79F', 'decimals': 18},
    'YESP': {'address': '0x46ccA329970B33e1a007DD4ef0594A1cedb3E72a', 'decimals': 18},
    'XTP': {'address': '0x6368e1E18c4C419DDFC608A0BEd1ccb87b9250fc', 'decimals': 18},
    'HOTCROSS': {'address': '0x4297394c20800E8a38A619A243E9BbE7681Ff24E', 'decimals': 18},
    'BAG': {'address': '0x235C8EE913d93c68D2902a8e0b5a643755705726', 'decimals': 18},
    'POLK': {'address': '0xD478161C952357F05f0292B56012Cd8457F1cfbF', 'decimals': 18},
    'AKI': {'address': '0x9eFdFC77017cB9b878300eCdcA21fAB0529a22A0', 'decimals': 9},
    'COIL': {'address': '0x823E1B82cE1Dc147Bbdb25a203f046aFab1CE918', 'decimals': 18},
    'NGL': {'address': '0x2653891204F463fb2a2F4f412564b19e955166aE', 'decimals': 18},
    'UCO': {'address': '0x8a3d77e9d6968b780564936d15B09805827C21fa', 'decimals': 18},
    'DG': {'address': '0x4b520c812E8430659FC9f12f6d0c39026C83588D', 'decimals': 18},
    'WRLD': {'address': '0xD5d86FC8d5C0Ea1aC1Ac5Dfab6E529c9967a45E9', 'decimals': 18},
    'ATT': {'address': '0x89Fb927240750c1B15d4743cd58440fc5f14A11C', 'decimals': 18},
    'ROGE': {'address': '0x45734927Fa2f616FbE19E65f42A0ef3d37d1c80A', 'decimals': 9},
    'PILOT': {'address': '0x37C997B35C619C21323F3518B9357914E8B99525', 'decimals': 18},
    'FNCT': {'address': '0x8Af78f0c818302164F73B2365fE152C2D1FE80e1', 'decimals': 18},
    'INVOX': {'address': '0x4485561Db76614Ff727f8E0a3Ea95690b8b16022', 'decimals': 18},
    'WOLVERINU': {'address': '0x7Cc97Bf17C5AdaBe25F9D19D15A1eC8A1ad65f14', 'decimals': 18},
    'BTB': {'address': '0x06e0feB0D74106c7adA8497754074D222Ec6BCDf', 'decimals': 18},
    'BUMBLEC': {'address': '0xAD83b92cdD542Db07445597f7F06963B31Cf9444', 'decimals': 9},
    'SILK': {'address': '0xB045f7f363fE4949954811b113bd56d208c67B23', 'decimals': 8},
    'CAPP': {'address': '0x11613b1f840bb5A40F8866d857e24DA126B79D73', 'decimals': 2},
    'FTRB': {'address': '0x2596825a84888E8f24b747DF29e11b5dd03C81D7', 'decimals': 18},
    'POLA': {'address': '0xc691bc298a304d591Ad9b352c7A8D216dE9f2CED', 'decimals': 18},
    'DRNT': {'address': '0xA3665c69213BB9AffCf92AB90100fBe273e3aB74', 'decimals': 18},
    'HARAMBE': {'address': '0xADe6FDAba1643E4D1eeF68Da7170F234470938c6', 'decimals': 18},
    'YVS': {'address': '0xEC681F28f4561c2a9534799AA38E0d36A83Cf478', 'decimals': 18},
    'UNISTAKE': {'address': '0x9Ed8e7C9604790F7Ec589F99b94361d8AAB64E5E', 'decimals': 18},
    'DG': {'address': '0xEE06A81a695750E71a662B51066F2c74CF4478a0', 'decimals': 18},
    'QUICK': {'address': '0x6c28AeF8977c9B773996d0e8376d2EE379446F2f', 'decimals': 18},
    'NAS': {'address': '0x5d65D971895Edc438f465c17DB6992698a52318D', 'decimals': 18},
    'ARV': {'address': '0x28fDA76721a8077A5dE802Ab0212849B8c38429E', 'decimals': 18},
    'SWRV': {'address': '0xB8BAa0e4287890a5F79863aB62b7F175ceCbD433', 'decimals': 18},
    'REAP': {'address': '0x1fc5EF0337AEA85C5f9198853a6E3A579a7A6987', 'decimals': 18},
    'UGAS': {'address': '0x8716Fc5Da009D3A208f0178b637a50F4ef42400F', 'decimals': 18},
    'CUBE': {'address': '0xDf801468a808a32656D2eD2D2d80B72A129739f4', 'decimals': 8},
    'NTO': {'address': '0xB668473944d2E25B6aF6D46917Eb0233DBaC53ae', 'decimals': 18},
    'SIGIL': {'address': '0xF029Fd6b5ED35C825Dd02B7A02952C30002840E4', 'decimals': 18},
    'RAZE': {'address': '0x5Eaa69B29f99C84Fe5dE8200340b4e9b4Ab38EaC', 'decimals': 18},
    'ATRI': {'address': '0xdacD69347dE42baBfAEcD09dC88958378780FB62', 'decimals': 0},
    'SALT': {'address': '0x4156D3342D5c385a87D264F90653733592000581', 'decimals': 8},
    'YVLUSD': {'address': '0x378cb52b00F9D0921cb46dFc099CFf73b42419dC', 'decimals': 18},
    'SENATE': {'address': '0x34Be5b8C30eE4fDe069DC878989686aBE9884470', 'decimals': 18},
    'MIR': {'address': '0x09a3EcAFa817268f77BE1283176B946C4ff2E608', 'decimals': 18},
    'UNFI': {'address': '0x441761326490cACF7aF299725B6292597EE822c2', 'decimals': 18},
    'BTR': {'address': '0x5EA82C27efc7634f1C5Ad20a3561c453433a2f3a', 'decimals': 18},
    'BMAX': {'address': '0x116c4B65e14449947bC6fa1bbe844CB16A162d53', 'decimals': 18},
    'ARES': {'address': '0x358AA737e033F34df7c54306960a38d09AaBd523', 'decimals': 18},
    'IDV': {'address': '0x45448E05020576929fcDEabC228E35b420098840', 'decimals': 18},
    'ENXS': {'address': '0x18a3563c21062897950BB09339C82b9686a35667', 'decimals': 18},
    'SHIB05': {'address': '0x8Eb94A06b4716093DBfE335cbdb098DEb2dcdE1b', 'decimals': 18},
    'LOOMI': {'address': '0xEb57Bf569Ad976974C1F861a5923A59F40222451', 'decimals': 18},
    'ZKS': {'address': '0xe4815AE53B124e7263F08dcDBBB757d41Ed658c6', 'decimals': 18},
    'VITO': {'address': '0x757dA0E5c253082b0f2Bd5105119F71817Fe0911', 'decimals': 18},
    'FOX': {'address': '0xc770EEfAd204B5180dF6a14Ee197D99d808ee52d', 'decimals': 18},
    'CIV': {'address': '0x37fE0f067FA808fFBDd12891C0858532CFE7361d', 'decimals': 18},
    'STANDARD': {'address': '0xDA0c94c73D127eE191955FB46bACd7FF999b2bcd', 'decimals': 18},
    'SHOE': {'address': '0x0fD67B4ceb9b607Ef206904eC73459c4880132c9', 'decimals': 18},
    'INX': {'address': '0x84fE25f3921f3426395c883707950d0c00367576', 'decimals': 18},
    'MEGA': {'address': '0x1892F6ff5FBE11c31158F8c6f6F6e33106c5B10E', 'decimals': 18},
    'DGRN': {'address': '0x71E368Ed06814Bf35d4E663eFf946400a5BC8115', 'decimals': 18},
    'GXT': {'address': '0x4674672BcDdDA2ea5300F5207E1158185c944bc0', 'decimals': 18},
    'IFV': {'address': '0x3656bD0f3f07623Bb7f429B390D208f894e44eCE', 'decimals': 18},
    'SHI': {'address': '0xaD996A45fd2373ed0B10Efa4A8eCB9de445A4302', 'decimals': 18},
    'DCHF': {'address': '0x045da4bFe02B320f4403674B3b7d121737727A36', 'decimals': 18},
    'AWC': {'address': '0xaD22f63404f7305e4713CcBd4F296f34770513f4', 'decimals': 8},
    'KATA': {'address': '0x2e85ae1C47602f7927bCabc2Ff99C40aA222aE15', 'decimals': 18},
    'PANDO': {'address': '0x252b9F56359901a0BDE52d0675b1f1130d86F471', 'decimals': 18},
    'FABRIC': {'address': '0x8c6Fa66c21aE3fC435790E451946a9Ea82E6e523', 'decimals': 18},
    'GOVI': {'address': '0xeEAA40B28A2d1b0B08f6f97bB1DD4B75316c6107', 'decimals': 18},
    'BZZ': {'address': '0x19062190B1925b5b6689D7073fDfC8c2976EF8Cb', 'decimals': 16},
    'DALC': {'address': '0x07D9e49Ea402194bf48A8276dAfB16E4eD633317', 'decimals': 8},
    'AZUKI': {'address': '0x910524678C0B1B23FFB9285a81f99C29C11CBaEd', 'decimals': 18},
    'CVNT': {'address': '0x6400B5522f8D448C0803e6245436DD1c81dF09ce', 'decimals': 8},
    'CLINK': {'address': '0xFAce851a4921ce59e912d19329929CE6da6EB0c7', 'decimals': 8},
    'PACHA': {'address': '0x2Dd1B9410c73E16b60240E529e38E30425F00d4E', 'decimals': 10},
    'CATAI': {'address': '0xBC1A4555745e9C5334eBF9DcfEC0B51285aF8d17', 'decimals': 18},
    'RARE': {'address': '0x93dfaf57D986B9cA77Df9376c50878E013D9c7C8', 'decimals': 18},
    'NYM': {'address': '0x525A8F6F3Ba4752868cde25164382BfbaE3990e1', 'decimals': 6},
    'MMAI': {'address': '0xaa0c5B3567fd1bac8a2a11EB16c3F81a49eea90F', 'decimals': 7},
    'SJM': {'address': '0x4448d67EA26a2Eaf286bc1045005f4CF15aaBa11', 'decimals': 9},
    'MEV': {'address': '0xb3Cb8d5AEFF0F4d1f432f353309f47b885E404E3', 'decimals': 18},
    'CCT': {'address': '0x336F646F87D9f6bC6Ed42Dd46E8b3fD9DbD15C22', 'decimals': 18},
    'OHM': {'address': '0x383518188C0C6d7730D91b2c03a03C837814a899', 'decimals': 9},
    'LOKA': {'address': '0x61E90A50137E1F645c9eF4a0d3A4f01477738406', 'decimals': 18},
    'HIMAYC': {'address': '0x635F15eb7Aa2e62d122f6B1f9f519fDCcf4aBdda', 'decimals': 18},
    'IETH': {'address': '0xC71D8BaAa436aa7E42DA1f40bEc48F11AB3c9E4A', 'decimals': 8},
    'YVE-CRVDAO': {'address': '0xc5bDdf9843308380375a611c18B50Fb9341f502A', 'decimals': 18},
    'NRCH': {'address': '0x69fa8e7F6bf1ca1fB0de61e1366f7412b827CC51', 'decimals': 9},
    'ETHM': {'address': '0xFD957F21bd95E723645C07C48a2d8ACB8Ffb3794', 'decimals': 18},
    'RFI': {'address': '0xA1AFFfE3F4D611d252010E3EAf6f4D77088b0cd7', 'decimals': 9},
    'EVX': {'address': '0xf3Db5Fa2C66B7aF3Eb0C0b782510816cbe4813b8', 'decimals': 4},
    'REKT': {'address': '0x8a7664E782860E856031D6c31EB3bde721Bc362b', 'decimals': 18},
    'LINA': {'address': '0x3E9BC21C9b189C09dF3eF1B824798658d5011937', 'decimals': 18},
    'FUSDC': {'address': '0x465a5a630482f3abD6d3b84B39B29b07214d19e5', 'decimals': 8},
    'BOND': {'address': '0x0391D2021f89DC339F60Fff84546EA23E337750f', 'decimals': 18},
    'GCR': {'address': '0x6307B25A665Efc992EC1C1bC403c38F3dDd7c661', 'decimals': 4},
    'ECELL': {'address': '0x9b62Ec1453cEa5Dde760AAf662048cA6eEB66E7f', 'decimals': 2},
    'RAC': {'address': '0xc22B30E4cce6b78aaaADae91E44E73593929a3e9', 'decimals': 18},
    'PIF': {'address': '0xB30F5d11b94efBBfDeaa4de38eDFFCeEc0bE6513', 'decimals': 18},
    'IQ': {'address': '0x579CEa1889991f68aCc35Ff5c3dd0621fF29b0C9', 'decimals': 18},
    'STARS': {'address': '0xc55c2175E90A46602fD42e931f62B3Acc1A013Ca', 'decimals': 18},
    'PRMX': {'address': '0xe4dae00bc1c46ea2F44ae71B1BEb8b171C15d812', 'decimals': 18},
    'MEZZ': {'address': '0xC4c346eDc55504574cCEB00AA1091d22404A4bC3', 'decimals': 18},
    'REM': {'address': '0x83984d6142934bb535793A82ADB0a46EF0F66B6d', 'decimals': 4},
    'PERX': {'address': '0x3C6ff50c9Ec362efa359317009428d52115fe643', 'decimals': 18},
    'YFL': {'address': '0x28cb7e841ee97947a86B06fA4090C8451f64c0be', 'decimals': 18},
    'ZMT': {'address': '0xaa602dE53347579f86b996D2Add74bb6F79462b2', 'decimals': 18},
    'ARCH': {'address': '0x1F3f9D3068568F8040775be2e8C03C103C61f3aF', 'decimals': 18},
    'RVP': {'address': '0x17EF75AA22dD5f6C2763b8304Ab24f40eE54D48a', 'decimals': 18},
    'LHCOIN': {'address': '0x0778Cc2E8bBAd3d483E82371606D100cc8604522', 'decimals': 8},
    'EUSD': {'address': '0x97de57eC338AB5d51557DA3434828C5DbFaDA371', 'decimals': 18},
    'ES': {'address': '0x72108a8CC3254813C6BE2F1b77be53E185abFdD9', 'decimals': 18},
    'SHIK': {'address': '0x24dA31e7BB182cb2cABfEF1d88db19C2AE1f5572', 'decimals': 18},
    'TAIL': {'address': '0xFeeeef4D7b4Bf3cc8BD012D02D32Ba5fD3D51e31', 'decimals': 18},
    'ANRX': {'address': '0xCae72A7A0Fd9046cf6b165CA54c9e3a3872109E0', 'decimals': 18},
    'DYP': {'address': '0x961C8c0B1aaD0c0b10a51FeF6a867E3091BCef17', 'decimals': 18},
    'TSUKA': {'address': '0xc5fB36dd2fb59d3B98dEfF88425a3F425Ee469eD', 'decimals': 9},
    'BT': {'address': '0x6628606c321FaF52b7230A57b26c01B19aA68e82', 'decimals': 18},
    'THOR': {'address': '0xa5f2211B9b8170F694421f2046281775E8468044', 'decimals': 18},
    'GEN': {'address': '0x543Ff227F64Aa17eA132Bf9886cAb5DB55DCAddf', 'decimals': 18},
    '0NE': {'address': '0x73A83269b9bbAFC427E76Be0A2C1a1db2a26f4C2', 'decimals': 18},
    'CDAI': {'address': '0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643', 'decimals': 8},
    'HUSKY': {'address': '0xd5281BB2d1eE94866B03A0fcCDd4e900c8Cb5091', 'decimals': 9},
    'WUSD': {'address': '0x068E3563b1c19590F822c0e13445c4FA1b9EEFa5', 'decimals': 18},
    'DPAY': {'address': '0xE5A733681bbE6cd8c764Bb8078ef8E13a576Dd78', 'decimals': 18},
    'VDR': {'address': '0xEd3D4e446A96dC3b181B64B75c3c70dA41dC3CBe', 'decimals': 18},
    'WQT': {'address': '0x06677Dc4fE12d3ba3C7CCfD0dF8Cd45e4D4095bF', 'decimals': 18},
    'PEPEG': {'address': '0x8BCbef61ACd66537362f38167F11875134FfcD63', 'decimals': 18},
    'NEST': {'address': '0x04abEdA201850aC0124161F037Efd70c74ddC74C', 'decimals': 18},
    'BADGER': {'address': '0x3472A5A71965499acd81997a54BBA8D852C6E53d', 'decimals': 18},
    'XNL': {'address': '0x06A00715E6F92210Af9d7680B584931FAF71A833', 'decimals': 18},
    'DX': {'address': '0x973e52691176d36453868D9d86572788d27041A9', 'decimals': 18},
    'MAHA': {'address': '0x745407c86DF8DB893011912d3aB28e68B62E49B0', 'decimals': 18},
    'MARK': {'address': '0x67c597624B17b16fb77959217360B7cD18284253', 'decimals': 9},
    'BIZZ': {'address': '0x7A8Ca2f815A260660158a38C34ca321A3605eCFE', 'decimals': 8},
    'VETH': {'address': '0xc3D088842DcF02C13699F936BB83DFBBc6f721Ab', 'decimals': 18},
    'ETHIX': {'address': '0xFd09911130e6930Bf87F2B0554c44F400bD80D3e', 'decimals': 18},
    'LBP': {'address': '0xa23C1194d421F252b4e6D5edcc3205F7650a4eBE', 'decimals': 18},
    'ANONUSD': {'address': '0x5a7E6C8204A1359DB9AAcab7bA5Fc309B7981eFd', 'decimals': 18},
    'STRM': {'address': '0x0eDF9bc41Bbc1354c70e2107F80C42caE7FBBcA8', 'decimals': 18},
    'PTF': {'address': '0xC57d533c50bC22247d49a368880fb49a1caA39F7', 'decimals': 18},
    'DAOVC': {'address': '0x284b59cf2539544559C6EfA11E2795e06D535345', 'decimals': 18},
    'SPWN': {'address': '0xe516D78d784C77D479977BE58905B3f2b1111126', 'decimals': 18},
    'TACO': {'address': '0x00D1793D7C3aAE506257Ba985b34C76AaF642557', 'decimals': 18},
    'FIFTY': {'address': '0xf7DD746A613Fb6362D44eCEDEB743f62ade6c3aa', 'decimals': 9},
    'TOP': {'address': '0xdcD85914b8aE28c1E62f1C488E1D968D5aaFfE2b', 'decimals': 18},
    'YVLINK': {'address': '0x671a912C10bba0CFA74Cfc2d6Fba9BA1ed9530B2', 'decimals': 18},
    'POWR': {'address': '0x595832F8FC6BF59c85C527fEC3740A1b7a361269', 'decimals': 6},
    'AERGO': {'address': '0x91Af0fBB28ABA7E31403Cb457106Ce79397FD4E6', 'decimals': 18},
    'OJA': {'address': '0x0aA7eFE4945Db24d95cA6E117BBa65Ed326e291A', 'decimals': 18},
    'MONKEYS': {'address': '0xf2A22B900dde3ba18Ec2AeF67D4c8C1a0DAB6aAC', 'decimals': 9},
    'SHINJA': {'address': '0x7DAc25b1A665e1c70F25F1fC37d88C99274984ed', 'decimals': 9},
    'METANO': {'address': '0x9D9e399e5385e2b9A58d4F775A1E16441b571afb', 'decimals': 18},
    'COVA': {'address': '0xB37a769B37224449d92AAc57dE379E1267Cd3B00', 'decimals': 18},
    'LPOOL': {'address': '0x6149C26Cd2f7b5CCdb32029aF817123F6E37Df5B', 'decimals': 18},
    'XT': {'address': '0x4BE10dA47A07716af28Ad199FbE020501BddD7aF', 'decimals': 18},
    'WCFG': {'address': '0xc221b7E65FfC80DE234bbB6667aBDd46593D34F0', 'decimals': 18},
    'PTU': {'address': '0xC229c69eB3BB51828D0cAA3509A05a51083898dd', 'decimals': 18},
    'XAI': {'address': '0x35e78b3982E87ecfD5b3f3265B601c046cDBe232', 'decimals': 18},
    'AE': {'address': '0x5CA9a71B1d01849C0a95490Cc00559717fCF0D1d', 'decimals': 18},
    'SHIH': {'address': '0x841FB148863454A3b3570f515414759BE9091465', 'decimals': 18},
    'TOKO': {'address': '0xa0F0546Eb5E3eE7e8cfC5DA12e5949F3AE622675', 'decimals': 18},
    'TRIBE': {'address': '0xc7283b66Eb1EB5FB86327f08e1B5816b0720212B', 'decimals': 18},
    'COLLAR': {'address': '0x1a16B2B93B37A7EF5cB0A4F978CeD45FE14a5486', 'decimals': 18},
    'FLY': {'address': '0x85f6eB2BD5a062f5F8560BE93FB7147e16c81472', 'decimals': 4},
    'OKAGE': {'address': '0x8dfc8cc3201425669FaE803e1eB125cddd4189eC', 'decimals': 18},
    'BFT': {'address': '0x01fF50f8b7f74E4f00580d9596cd3D0d6d6E326f', 'decimals': 18},
    'DIGG': {'address': '0x798D1bE841a82a273720CE31c822C61a67a601C3', 'decimals': 9},
    'CAH': {'address': '0x8e0E57DCb1ce8d9091dF38ec1BfC3b224529754A', 'decimals': 18},
    'STMX': {'address': '0xa62cc35625B0C8dc1fAEA39d33625Bb4C15bD71C', 'decimals': 18},
    'CBM': {'address': '0x95eFD1Fe6099F65a7ED524DEF487483221094947', 'decimals': 18},
    'KEL': {'address': '0x4ABB9cC67BD3da9Eb966d1159A71a0e68BD15432', 'decimals': 18},
    'GGTK': {'address': '0xFA99A87b14B02e2240C79240C5a20F945ca5EF76', 'decimals': 18},
    'PBTC': {'address': '0x5228a22e72ccC52d415EcFd199F99D0665E7733b', 'decimals': 18},
    'DVI': {'address': '0x10633216E7E8281e33c86F02Bf8e565a635D9770', 'decimals': 18},
    'ASX': {'address': '0x67d85A291fcDC862A78812a3C26d55e28FFB2701', 'decimals': 18},
    'ZINU': {'address': '0xc50EF449171a51FbeAFd7c562b064B6471C36caA', 'decimals': 9},
    'LED': {'address': '0x72De803b67B6AB05B61EFab2Efdcd414D16eBF6D', 'decimals': 18},
    'CYMI': {'address': '0x7449c93ABe66457e83B3799BCf1A99e92D58A93B', 'decimals': 18},
    'IFT': {'address': '0x2731d151CBDf84A8A4C6d9D0BaE74012Db51E428', 'decimals': 18},
    'SHFT': {'address': '0xF5a56bD9Ee4a3E2D493ab8e072658794B38b6d19', 'decimals': 18},
    'UFT': {'address': '0x0202Be363B8a4820f3F4DE7FaF5224fF05943AB1', 'decimals': 18},
    'RDT': {'address': '0x4740735AA98Dc8aa232BD049f8F0210458E7fCa3', 'decimals': 18},
    'REP': {'address': '0x221657776846890989a759BA2973e427DfF5C9bB', 'decimals': 18},
    'PBTC': {'address': '0x62199B909FB8B8cf870f97BEf2cE6783493c4908', 'decimals': 18},
    'PLSCX': {'address': '0x6cD079F296c10EB84A64239b893D4EdfA2E1d779', 'decimals': 18},
    'MVI': {'address': '0x72e364F2ABdC788b7E918bc238B21f109Cd634D7', 'decimals': 18},
    'PNK': {'address': '0x93ED3FBe21207Ec2E8f2d3c3de6e058Cb73Bc04d', 'decimals': 18},
    'GUESS': {'address': '0xBDCFbf5C4D91Abc0bC9709C7286d00063c0e6F22', 'decimals': 2},
    'UPR': {'address': '0xf720E38F678B29B243F7D53B56Acbf5dE98F2385', 'decimals': 18},
    'KISHIMOTO': {'address': '0xf5b1Fd29d23e98Db2D9EBb8435E1082e3B38FB65', 'decimals': 9},
    'GDT': {'address': '0xc67B12049c2D0CF6e476BC64c7F82fc6C63cFFc5', 'decimals': 8},
    'CYL': {'address': '0x26CB3641aaA43911f1D4cB2ce544eb652AAc7c47', 'decimals': 18},
    'PHNX': {'address': '0x38A2fDc11f526Ddd5a607C1F251C065f40fBF2f7', 'decimals': 18},
    'ACRV': {'address': '0x8dAE6Cb04688C62d939ed9B68d32Bc62e49970b1', 'decimals': 18},
    'MGG': {'address': '0x7237C0B30B1355f1B76355582f182F6f04B08740', 'decimals': 18},
    'BEAN': {'address': '0xBEA0000029AD1c77D3d5D23Ba2D8893dB9d1Efab', 'decimals': 6},
    'POP': {'address': '0xD0Cd466b34A24fcB2f87676278AF2005Ca8A78c4', 'decimals': 18},
    'ASTRAL': {'address': '0xC74B43cC61b627667a608c3F650c2558F88028a1', 'decimals': 18},
    'ECOREAL': {'address': '0x7ECbb21346C501Fd07eb165E406120fA32381C16', 'decimals': 18},
    'RBIF': {'address': '0x7B32e70e8D73ac87c1B342e063528B2930b15ceB', 'decimals': 9},
    'FEY': {'address': '0xe8E06a5613dC86D459bC8Fb989e173bB8b256072', 'decimals': 18},
    'AUSDC': {'address': '0xBcca60bB61934080951369a648Fb03DF4F96263C', 'decimals': 6},
    'GZLR': {'address': '0x9f4909Cc95FB870BF48C128C1Fdbb5F482797632', 'decimals': 18},
    'FWT': {'address': '0x20e7125677311Fca903A8897042b9983f22Ea295', 'decimals': 18},
    'BAAS': {'address': '0x5d929AA919E489505CcAAd8A199619c6DCA0c2de', 'decimals': 18},
    'NXM': {'address': '0xd7c49CEE7E9188cCa6AD8FF264C1DA2e69D4Cf3B', 'decimals': 18},
    'KDOE': {'address': '0x5f190F9082878cA141F858c1c90B4C59fe2782C5', 'decimals': 18},
    'CUT': {'address': '0x135783B60cf5d71DAFF7a377f9eb7dB8D2dEAb9e', 'decimals': 18},
    'PRIMATE': {'address': '0x46e98FFE40E408bA6412bEb670507e083C8B95ff', 'decimals': 18},
    'STX': {'address': '0x006BeA43Baa3f7A6f765F14f10A1a1b08334EF45', 'decimals': 18},
    'ORBS': {'address': '0xff56Cc6b1E6dEd347aA0B7676C85AB0B3D08B0FA', 'decimals': 18},
    'MLN': {'address': '0xec67005c4E498Ec7f55E092bd1d35cbC47C91892', 'decimals': 18},
    'CNTM': {'address': '0x0E5f00DA8AAef196a719d045DB89b5DA8F371b32', 'decimals': 18},
    'FCT': {'address': '0xE1bAD922F84b198A08292FB600319300ae32471b', 'decimals': 18},
    'MODA': {'address': '0x1117aC6Ad6Cdf1A3BC543baD3B133724620522d5', 'decimals': 18},
    'SWISE': {'address': '0x48C3399719B582dD63eB5AADf12A40B4C3f52FA2', 'decimals': 18},
    'MUSE': {'address': '0xB6Ca7399B4F9CA56FC27cBfF44F4d2e4Eef1fc81', 'decimals': 18},
    'HOBBES': {'address': '0x819c1A1568934Ee59d9F3C8B9640908556c44140', 'decimals': 18},
    'BT': {'address': '0x76c5449F4950f6338A393F53CdA8b53B0cd3Ca3a', 'decimals': 18},
    'M87': {'address': '0x8AF5FedC0f263841C18F31D9DbCC97A47e1aB462', 'decimals': 18},
    'LEND': {'address': '0x80fB784B7eD66730e8b1DBd9820aFD29931aab03', 'decimals': 18},
    'GNY': {'address': '0xb1f871Ae9462F1b2C6826E88A7827e76f86751d4', 'decimals': 18},
    'SPD': {'address': '0x1dEa979ae76f26071870F824088dA78979eb91C8', 'decimals': 18},
    'GRO': {'address': '0x3Ec8798B81485A254928B70CDA1cf0A2BB0B74D7', 'decimals': 18},
    'DRC': {'address': '0xa150Db9b1Fa65b44799d4dD949D922c0a33Ee606', 'decimals': 0},
    'RAZOR': {'address': '0x50DE6856358Cc35f3A9a57eAAA34BD4cB707d2cd', 'decimals': 18},
    'YIN': {'address': '0x794Baab6b878467F93EF17e2f2851ce04E3E34C8', 'decimals': 18},
    'FOL': {'address': '0xA8580F3363684d76055bdC6660CaeFe8709744e1', 'decimals': 18},
    'ZKT': {'address': '0x76fcA1adb104770B38581B64d55e67fA5A0f3966', 'decimals': 9},
    'LPL': {'address': '0x99295f1141d58A99e939F7bE6BBe734916a875B8', 'decimals': 18},
    'COLT': {'address': '0x1A3cBdA3853494acAb67648EE59AfeB7ec3E9334', 'decimals': 18},
    'GATSBY': {'address': '0xE22910D04bDc9d55570f3BC52E786b49DddF37b4', 'decimals': 18},
    'DOP': {'address': '0x6bB61215298F296C55b19Ad842D3Df69021DA2ef', 'decimals': 18},
    'SRUNE': {'address': '0x0352557B007A4Aae1511C114409b932F06F9E2f4', 'decimals': 18},
    'JCHF': {'address': '0x53dfEa0A8CC2A2A2e425E1C174Bc162999723ea0', 'decimals': 18},
    'HFIL': {'address': '0x9AFb950948c2370975fb91a441F36FDC02737cD4', 'decimals': 18},
    'MEMEX': {'address': '0xD89310F4BaeDb33AfB36d7cc45BB8847f4463060', 'decimals': 18},
    'EOSDAC': {'address': '0x7e9e431a0B8c4D532C745B1043c7FA29a48D4fBa', 'decimals': 18},
    'PAD': {'address': '0xea7Cc765eBC94C4805e3BFf28D7E4aE48D06468A', 'decimals': 18},
    'SWM': {'address': '0x3505F494c3f0fed0B594E01Fa41Dd3967645ca39', 'decimals': 18},
    'AREN': {'address': '0x69948cC03f478B95283F7dbf1CE764d0fc7EC54C', 'decimals': 18},
    'HIODBS': {'address': '0x33bD66c334274989b673a8e8C8d1A3f1B8De5889', 'decimals': 18},
    'MAP': {'address': '0x9E976F211daea0D652912AB99b0Dc21a7fD728e4', 'decimals': 18},
    'CBK': {'address': '0xD85a6Ae55a7f33B0ee113C234d2EE308EdeAF7fD', 'decimals': 18},
    'HICLONEX': {'address': '0x764104dc24DAdFf01150253a58C82337984B4319', 'decimals': 18},
    'IDEA': {'address': '0x5d3a4F62124498092Ce665f865E0b38fF6F5FbEa', 'decimals': 18},
    'CVC': {'address': '0x41e5560054824eA6B0732E656E3Ad64E20e94E45', 'decimals': 8},
    'DOKE': {'address': '0x517ABf1fCDBD76BC75B532683ada9113E313A128', 'decimals': 9},
    'CRDC': {'address': '0xF81421fc15300c5a8CCa9aFE12F5CBAd502Fa756', 'decimals': 18},
    'NDX': {'address': '0x86772b1409b61c639EaAc9Ba0AcfBb6E238e5F83', 'decimals': 18},
    'KXA': {'address': '0x2223bF1D7c19EF7C06DAB88938EC7B85952cCd89', 'decimals': 18},
    'LAND': {'address': '0x0e2ef8AeCB3c01Ad5D596f1B67134e178199984D', 'decimals': 18},
    'GENE': {'address': '0xf6ec87DFE1Ed3a7256Cc0c38e3c8139103e9aF3b', 'decimals': 18},
    'GPX': {'address': '0x3e7804c51a70Ba26E904c2e0ab440C5623a8A83F', 'decimals': 8},
    'HIPENGUINS': {'address': '0x669dB4c47f89f21554Ebd825A744888725FD9491', 'decimals': 18},
    'SONIC': {'address': '0xd1cD47746B8e72359b28C1c84a4F6a19dc1A0ee5', 'decimals': 18},
    'SERP': {'address': '0xFd414e39155F91E94443a9Fe97E856569d0F5Eec', 'decimals': 9},
    'YVUSDC': {'address': '0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE', 'decimals': 6},
    'HERB': {'address': '0x04A020325024F130988782bd5276e53595e8d16E', 'decimals': 8},
    'DOWS': {'address': '0x661Ab0Ed68000491d98C796146bcF28c20d7c559', 'decimals': 18},
    'CHANGE': {'address': '0x7051faED0775f664a0286Af4F75ef5ed74e02754', 'decimals': 18},
    'WIZ': {'address': '0x603778e5e07e180E300bD10df7fA626313a206B2', 'decimals': 4},
    'POP': {'address': '0x5D858bcd53E085920620549214a8b27CE2f04670', 'decimals': 18},
    'TCAP': {'address': '0x16c52CeeCE2ed57dAd87319D91B5e3637d50aFa4', 'decimals': 18},
    'ICHI': {'address': '0x111111517e4929D3dcbdfa7CCe55d30d4B6BC4d6', 'decimals': 18},
    'PAWTH': {'address': '0xAEcc217a749c2405b5ebC9857a16d58Bdc1c367F', 'decimals': 9},
    'XDAO': {'address': '0x71eebA415A523F5C952Cc2f06361D5443545Ad28', 'decimals': 18},
    'TOMI': {'address': '0x4385328cc4D643Ca98DfEA734360C0F596C83449', 'decimals': 18},
    'DLTA': {'address': '0x0000000DE40dfa9B17854cBC7869D80f9F98D823', 'decimals': 18},
    'VOLT': {'address': '0x3F7Aff0EF20AA2E646290DfA4E67611B2220C597', 'decimals': 9},
    'CCO': {'address': '0x679BADc551626e01B23CeecEFBc9B877EA18fc46', 'decimals': 18},
    'AXSUSHI': {'address': '0xF256CC7847E919FAc9B808cC216cAc87CCF2f47a', 'decimals': 18},
    'XLAB': {'address': '0x8c4E7f814d40f8929F9112C5D09016F923d34472', 'decimals': 18},
    'SHOPX': {'address': '0x7BEF710a5759d197EC0Bf621c3Df802C2D60D848', 'decimals': 18},
    'GLR': {'address': '0xd54619E0b9899D74Cc9B981354Eb6b59732c43B1', 'decimals': 18},
    'LIT': {'address': '0xC5b3D3231001a776123194Cf1290068e8b0C783b', 'decimals': 18},
    'ELAND': {'address': '0x33E07f5055173cF8FeBedE8B21B12D1e2b523205', 'decimals': 18},
    'EMS': {'address': '0x247Dc9CBBAADabce6e30E2A84eC6C53A419913Ad', 'decimals': 18},
    'SKM': {'address': '0x048Fe49BE32adfC9ED68C37D32B5ec9Df17b3603', 'decimals': 18},
    'WVG0': {'address': '0x25C7b64A93Eb1261E130eC21a3e9918CaA38b611', 'decimals': 18},
    'MARAN': {'address': '0x5F9123d661459AF6f398B6f1566f53222612541e', 'decimals': 18},
    'VERA': {'address': '0x166F1a7eCAe00bd43876A25B10a63C575e05c0e7', 'decimals': 18},
    'GVC': {'address': '0x8dc89F4716E027394Bba225b82328C1ea2Ea58Bf', 'decimals': 18},
    'META': {'address': '0xDE2F7766C8BF14CA67193128535E5c7454f8387C', 'decimals': 18},
    'TSX': {'address': '0x734C90044a0bA31B3F2e640c10dC5d3540499Bfd', 'decimals': 18},
    'SNP': {'address': '0x6911F552842236bd9E8ea8DDBB3fb414e2C5FA9d', 'decimals': 18},
    'MILK2': {'address': '0x80c8C3dCfB854f9542567c8Dac3f44D709eBc1de', 'decimals': 18},
    'NORD': {'address': '0x6e9730EcFfBed43fD876A264C982e254ef05a0DE', 'decimals': 18},
    'EPAN': {'address': '0x72630B1e3B42874bf335020Ba0249e3E9e47Bafc', 'decimals': 18},
    'ETHV': {'address': '0xC53342fd7575f572b0fF4569e31941A5B821aC76', 'decimals': 18},
    'WGMI': {'address': '0xE23FaEfcdAAe646527eb86b1C746A25Ae4a36f75', 'decimals': 18},
    'BZN': {'address': '0x6524B87960c2d573AE514fd4181777E7842435d4', 'decimals': 18},
    'EAC': {'address': '0xcAe636167db2369bd746F2bcBa79a6e8b0d4E000', 'decimals': 18},
    'ONEBTC': {'address': '0xC88F47067dB2E25851317A2FDaE73a22c0777c37', 'decimals': 9},
    'SURF': {'address': '0xEa319e87Cf06203DAe107Dd8E5672175e3Ee976c', 'decimals': 18},
    'VITA': {'address': '0x81f8f0bb1cB2A06649E51913A151F0E7Ef6FA321', 'decimals': 18},
    'AST': {'address': '0x27054b13b1B798B345b591a4d22e6562d47eA75a', 'decimals': 4},
    'EXRN': {'address': '0xe469c4473af82217B30CF17b10BcDb6C8c796e75', 'decimals': 0},
    'YVCOMP': {'address': '0x4A3FE75762017DB0eD73a71C9A06db7768DB5e66', 'decimals': 18},
    'YEL': {'address': '0x7815bDa662050D84718B988735218CFfd32f75ea', 'decimals': 18},
    'MUST': {'address': '0x9C78EE466D6Cb57A4d01Fd887D2b5dFb2D46288f', 'decimals': 18},
    'PLT': {'address': '0x0944D5848BD9F60A34Ba92aEa300d4286696Eb76', 'decimals': 18},
    'NDS': {'address': '0x747f564d258612ec5c4E24742C5fd4110bCbe46B', 'decimals': 18},
    'SPIRAL': {'address': '0x01792e1548Dc317bDE6123fe92da1Fe6d7311C3c', 'decimals': 9},
    'SOON': {'address': '0x6100dd79fCAA88420750DceE3F735d168aBcB771', 'decimals': 18},
    'D11': {'address': '0x9bf02cF6B0435A0523E6f6e0D2f35A920144F5FA', 'decimals': 18},
    'DIVER': {'address': '0xFb782396c9b20E564A64896181c7AC8d8979d5F4', 'decimals': 18},
    'PRCY': {'address': '0xdFC3829b127761a3218bFceE7fc92e1232c9D116', 'decimals': 8},
    'AAMMUSDC': {'address': '0xd24946147829DEaA935bE2aD85A3291dbf109c80', 'decimals': 6},
    'CIA': {'address': '0x52F4D5EE6c91e01Be67CA1F64b11eD0EE370817d', 'decimals': 9},
    'AREN': {'address': '0xCC12AbE4ff81c9378D670De1b57F8e0Dd228D77a', 'decimals': 18},
    'SHPING': {'address': '0x7C84e62859D0715eb77d1b1C4154Ecd6aBB21BEC', 'decimals': 18},
    'ADEL': {'address': '0x94d863173EE77439E4292284fF13fAD54b3BA182', 'decimals': 18},
    'STACK': {'address': '0x56A86d648c435DC707c8405B78e2Ae8eB4E60Ba4', 'decimals': 18},
    'BTFA': {'address': '0xC631bE100F6Cf9A7012C23De5a6ccb990EAFC133', 'decimals': 9},
    'EURT': {'address': '0xC581b735A1688071A1746c968e0798D642EDE491', 'decimals': 6},
    'NFTS': {'address': '0x08037036451C768465369431Da5C671ad9B37dBc', 'decimals': 18},
    'HDRN': {'address': '0x3819f64f282bf135d62168C1e513280dAF905e06', 'decimals': 9},
    'TRIX': {'address': '0x056354F3Ff20743aa4c0DA365603871c7000b081', 'decimals': 18},
    'BTSE': {'address': '0x666d875C600AA06AC1cf15641361dEC3b00432Ef', 'decimals': 8},
    'SOUL': {'address': '0x992D339532A9C42F1b0E59A57e95F38da38c66F6', 'decimals': 18},
    'ALCA': {'address': '0xBb556b0eE2CBd89ed95DdEA881477723A3Aa8F8b', 'decimals': 18},
    'ICSA': {'address': '0xfc4913214444aF5c715cc9F7b52655e788A569ed', 'decimals': 9},
    'PSDN': {'address': '0xA5B947687163FE88C3e6af5b17Ae69896F4abccf', 'decimals': 18},
    'COREDAO': {'address': '0xf66Cd2f8755a21d3c8683a10269F795c0532Dd58', 'decimals': 18},
    'AZUM': {'address': '0xd26a9C3437f7D121098c8C05C7413F5Cc70BB070', 'decimals': 18},
    'CINO': {'address': '0x9b0B23b35AD8136E6181F22b346134cE5f426090', 'decimals': 18},
    'COIN': {'address': '0xE61fDAF474Fac07063f2234Fb9e60C1163Cfa850', 'decimals': 18},
    'ITAMCUBE': {'address': '0xBBaB3bDb291b0F22BC9881895ff488A5Db67BeC8', 'decimals': 18},
    'CDT': {'address': '0x177d39AC676ED1C67A2b268AD7F1E58826E5B0af', 'decimals': 18},
    'MNTO': {'address': '0x97a9bac06f90940bCe9CAeC2b880ff17707519e4', 'decimals': 18},
    'BKK': {'address': '0x1df7AA5551E801e280007dc0Fc0454E2D06C1A89', 'decimals': 18},
    'YVSUSHI': {'address': '0x6d765CbE5bC922694afE112C140b8878b9FB0390', 'decimals': 18},
    'OOKI': {'address': '0x0De05F6447ab4D22c8827449EE4bA2D5C288379B', 'decimals': 18},
    'VVT': {'address': '0x755be920943eA95e39eE2DC437b268917B580D6e', 'decimals': 18},
    'CLH': {'address': '0xD7D8f3b8Bc8bC48d3AcC37879EABA7b85889FA52', 'decimals': 18},
    'BABY': {'address': '0x1cCF27211e8bf052f6255329ed641B4E94E80603', 'decimals': 18},
    'BNTY': {'address': '0xd2d6158683aeE4Cc838067727209a0aAF4359de3', 'decimals': 18},
    'JEJUDOGE': {'address': '0x939A7A577D93ad29B64C1595B1284ce660A479B9', 'decimals': 18},
    'DEP': {'address': '0x1A3496C18d558bd9C6C8f609E1B129f67AB08163', 'decimals': 18},
    'TRB': {'address': '0x88dF592F8eb5D7Bd38bFeF7dEb0fBc02cf3778a0', 'decimals': 18},
    'CLAW': {'address': '0xe115401B985306837B9f1495c8e9d7f8f8328cc3', 'decimals': 18},
    'ALG': {'address': '0x16B0a1a87ae8aF5C792faBC429C4FE248834842B', 'decimals': 18},
    'YETIC': {'address': '0xdF96bDE075d59e9143b325C75aF38e208c986E6F', 'decimals': 9},
    'STMATIC': {'address': '0x9ee91F9f426fA633d227f7a9b000E28b9dfd8599', 'decimals': 18},
    'ROKO': {'address': '0x6f222E04F6c53Cc688FfB0Abe7206aAc66A8FF98', 'decimals': 18},
    'AGA': {'address': '0x2D80f5F5328FdcB6ECeb7Cacf5DD8AEDaEC94e20', 'decimals': 4},
    'ZOO': {'address': '0x09F098B155D561Fc9F7BcCc97038b7e3d20bAF74', 'decimals': 18},
    'PLUMS': {'address': '0x666CBfaA3baa2FAccFAC8854fea1E5Db140FB104', 'decimals': 18},
    'UPI': {'address': '0x70D2b7C19352bB76e4409858FF5746e500f2B67c', 'decimals': 18},
    'T': {'address': '0x5b985B4F827072FebE33091b42729522B557Bba1', 'decimals': 18},
    'SXP': {'address': '0x8CE9137d39326AD0cD6491fb5CC0CbA0e089b6A9', 'decimals': 18},
    'SMBSWAP': {'address': '0x53Bd789F2cDb846b227d8ffc7B46eD4263231FDf', 'decimals': 18},
    'LWC': {'address': '0xa6a1Cc527d48585538b137E6Abc14B2a55489D11', 'decimals': 8},
    'LUSD': {'address': '0x5f98805A4E8be255a32880FDeC7F6728C6568bA0', 'decimals': 18},
    'ATUSD': {'address': '0x101cc05f4A51C0319f570d5E146a8C625198e636', 'decimals': 18},
    'PURE': {'address': '0x2904b9b16652d7d0408EcCfA23A19D4A3358230f', 'decimals': 18},
    'MOUSEWORM': {'address': '0xA848a1D33D8ef1633397A6acf617620fab8E5dA8', 'decimals': 6},
    'CHI': {'address': '0x0000000000004946c0e9F43F4Dee607b0eF1fA1c', 'decimals': 0},
    'ABUSD': {'address': '0x6Ee0f7BB50a54AB5253dA0667B0Dc2ee526C30a8', 'decimals': 18},
    'REL': {'address': '0x61bFC979EA8160Ede9b862798B7833a97baFa02a', 'decimals': 18},
    'WCSOV': {'address': '0x68E9C0d9aA450254aed2CD102503d4DFf6B3C37c', 'decimals': 18},
    'TOMS': {'address': '0xbE9Ab37a414c517b2be2bfA5945665bB07379054', 'decimals': 18},
    'FBX': {'address': '0x60BB16c4A931b1a0B8A7D945C651DD90f41D42Cf', 'decimals': 18},
    'WOMBAT': {'address': '0x0C9c7712C83B3C70e7c5E11100D33D9401BdF9dd', 'decimals': 18},
    'PIB': {'address': '0x1864cE27E9F7517047933CaAE530674e8C70b8A7', 'decimals': 18},
    'URQA': {'address': '0x1735Db6AB5BAa19eA55d0AdcEeD7bcDc008B3136', 'decimals': 18},
    'SPN': {'address': '0x32EA3Dc70E2962334864A9665254d2433E4ddbfD', 'decimals': 18},
    'WACO': {'address': '0x910c4DA718CAF4EE38cE5c2490CDDAEca689204e', 'decimals': 18},
    'AMO': {'address': '0x38c87AA89B2B8cD9B95b736e1Fa7b612EA972169', 'decimals': 18},
    'KYOKO': {'address': '0x14a32f050FACF226Ec60882398A9BF36d91DBaC2', 'decimals': 18},
    'MOONEY': {'address': '0x20d4DB1946859E2Adb0e5ACC2eac58047aD41395', 'decimals': 18},
    'HISQUIGGLE': {'address': '0x51395AdE06Eae126f590e7B06Dc8F6Baf511f13F', 'decimals': 18},
    'CUP': {'address': '0x3209d14Ff61766359E64aceFf91877cEC2AD968E', 'decimals': 18},
    'CHX': {'address': '0x1460a58096d80a50a2F1f956DDA497611Fa4f165', 'decimals': 18},
    'DSD': {'address': '0xBD2F0Cd039E0BFcf88901C98c0bFAc5ab27566e3', 'decimals': 18},
    'WOMI': {'address': '0x04969cD041C0cafB6AC462Bd65B536A5bDB3A670', 'decimals': 18},
    'CRF': {'address': '0x508df5aa4746bE37b5b6A69684DfD8BDC322219d', 'decimals': 18},
    'PPDEX': {'address': '0xf1F508c7C9f0d1b15a76fbA564eEf2d956220cf7', 'decimals': 18},
    'CYOP': {'address': '0xddaC9C604BA6Bc4ACEc0FBB485B83f390ECF2f31', 'decimals': 9},
    'STILT': {'address': '0xB5F1457d6FBa1956Fb8d31B0B7CAcA14BDe0bE4b', 'decimals': 9},
    'DAMM': {'address': '0xb3207935FF56120f3499e8aD08461Dd403bF16b8', 'decimals': 18},
    'SAKE': {'address': '0x066798d9ef0833ccc719076Dab77199eCbd178b0', 'decimals': 18},
    'TORN': {'address': '0x77777FeDdddFfC19Ff86DB637967013e6C6A116C', 'decimals': 18},
    'FPIS': {'address': '0xc2544A32872A91F4A553b404C6950e89De901fdb', 'decimals': 18},
    'EYE': {'address': '0x155ff1A85F440EE0A382eA949f24CE4E0b751c65', 'decimals': 18},
    'UNB': {'address': '0x8dB253a1943DdDf1AF9bcF8706ac9A0Ce939d922', 'decimals': 18},
    'DTA': {'address': '0x69b148395Ce0015C13e36BFfBAd63f49EF874E03', 'decimals': 18},
    'YVUSDT': {'address': '0x3B27F92C0e212C671EA351827EDF93DB27cc0c65', 'decimals': 6},
    'IDO': {'address': '0xF9c53268e9de692AE1b2ea5216E24e1c3ad7CB1E', 'decimals': 18},
    'BAMBOO': {'address': '0xf56842Af3B56Fd72d17cB103f92d027bBa912e89', 'decimals': 18},
    'AUTO': {'address': '0x622dFfCc4e83C64ba959530A5a5580687a57581b', 'decimals': 18},
    'NERF': {'address': '0xaF8942831F3A096f708b8b31F191B8958Cf176c5', 'decimals': 18},
    'TOTOFO': {'address': '0x0ADCfDFe9E88D28cC4C1dAaC9CC021067aFF9B0C', 'decimals': 18},
    'TEMPLE': {'address': '0x470EBf5f030Ed85Fc1ed4C2d36B9DD02e77CF1b7', 'decimals': 18},
    'CRU': {'address': '0x32a7C02e79c4ea1008dD6564b35F131428673c41', 'decimals': 18},
    'EVC': {'address': '0xAa843f65872a25D6E9552eA0B360Fb1d5E333124', 'decimals': 18},
    'UNIX': {'address': '0xDDD6A0ECc3c6F6C102E5eA3d8Af7B801d1a77aC8', 'decimals': 18},
    'CELL': {'address': '0x26c8AFBBFE1EBaca03C2bB082E69D0476Bffe099', 'decimals': 18},
    'CYCLUB': {'address': '0x3b58c52C03ca5Eb619EBa171091c86C34d603e5f', 'decimals': 9},
    'COIN': {'address': '0x87b008E57F640D94Ee44Fd893F0323AF933F9195', 'decimals': 18},
    'THX': {'address': '0xe632ea2eF2CFD8Fc4a2731C76F99078Aef6a4B31', 'decimals': 18},
    'QOM': {'address': '0xa71d0588EAf47f12B13cF8eC750430d21DF04974', 'decimals': 18},
    'APN': {'address': '0xd4342a57eCf2Fe7ffA37c33cb8f63b1500e575E6', 'decimals': 18},
    'SMT': {'address': '0x21f15966E07a10554C364b988e91DaB01D32794A', 'decimals': 18},
    'NATION': {'address': '0x333A4823466879eeF910A04D473505da62142069', 'decimals': 18},
    'COLR': {'address': '0x14b40AD2EBA6c1B31db2bA817b07578AFB414415', 'decimals': 18},
    'NABOX': {'address': '0x03D1E72765545729A035e909eDd9371a405f77Fb', 'decimals': 18},
    'KITTYDINGER': {'address': '0x2C33B28527A63Cdf13C0B24Ce4cf5BF9C9fB3BC6', 'decimals': 9},
    'MOONCAT': {'address': '0x98968f0747E0A261532cAcC0BE296375F5c08398', 'decimals': 18},
    'BIX': {'address': '0x009c43B42AEFAC590C719E971020575974122803', 'decimals': 18},
    'MANC': {'address': '0xE0c05ec44775e4AD62CDC2eEcdF337aA7A143363', 'decimals': 2},
    'XMON': {'address': '0x3aaDA3e213aBf8529606924d8D1c55CbDc70Bf74', 'decimals': 18},
    'MXNT': {'address': '0xed03Ed872159e199065401b6d0D487d78d9464AA', 'decimals': 6},
    'BOX': {'address': '0x066e941E168dE3815cd6c8F63c46e8E3a2309ad7', 'decimals': 18},
    'CREDI': {'address': '0xaE6e307c3Fe9E922E5674DBD7F830Ed49c014c6B', 'decimals': 18},
    'RENBTC': {'address': '0xEB4C2781e4ebA804CE9a9803C67d0893436bB27D', 'decimals': 8},
    'DAFI': {'address': '0xFc979087305A826c2B2a0056cFAbA50aad3E6439', 'decimals': 18},
    'SAT': {'address': '0x5ABf88cF3444611D13F6D1B39F3f3EE8575c91a2', 'decimals': 18},
    'CHIP': {'address': '0x5A5389bA78F04D783E9F30e5A8C45727c5E018bE', 'decimals': 9},
    'COOK': {'address': '0xFF75CEd57419bcaEBe5F05254983b013B0646eF5', 'decimals': 18},
    'BABL': {'address': '0xF4Dc48D260C93ad6a96c5Ce563E70CA578987c74', 'decimals': 18},
    'DERI': {'address': '0xA487bF43cF3b10dffc97A9A744cbB7036965d3b9', 'decimals': 18},
    'CWBTC': {'address': '0xccF4429DB6322D5C611ee964527D42E5d685DD6a', 'decimals': 8},
    'KISHUTAMA': {'address': '0xDc0804eD9ABA9202E2b976C031d3144BB63a7990', 'decimals': 9},
    'PDI': {'address': '0x632806BF5c8f062932Dd121244c9fbe7becb8B48', 'decimals': 18},
    'OPEN': {'address': '0x69e8b9528CABDA89fe846C67675B5D73d463a916', 'decimals': 18},
    'CVNX': {'address': '0xA1a4E303e9C56962F201C5e834abC1E677A3C4F3', 'decimals': 18},
    'MFG': {'address': '0x6710c63432A2De02954fc0f851db07146a6c0312', 'decimals': 18},
    'NGM': {'address': '0xed0d5747A9AB03a75fBfEC3228cd55848245B75d', 'decimals': 6},
    'EXM': {'address': '0x83869DE76B9Ad8125e22b857f519F001588c0f62', 'decimals': 8},
    'DCNT': {'address': '0x4Ce4C025692B3142dbdE1cd432ef55b9A8D18701', 'decimals': 9},
    'BEPRO': {'address': '0xCF3C8Be2e2C42331Da80EF210e9B1b307C03d36A', 'decimals': 18},
    'KFT': {'address': '0xEF53462838000184F35f7D991452e5f25110b207', 'decimals': 18},
    'SFI': {'address': '0xb753428af26E81097e7fD17f40c88aaA3E04902c', 'decimals': 18},
    'LIEN': {'address': '0xaB37e1358b639Fd877f015027Bb62d3ddAa7557E', 'decimals': 8},
    'YUNA': {'address': '0x2F6f397332B0F8b4536430bCC97980aA838f62f9', 'decimals': 18},
    'NEWS': {'address': '0x777fD20c983d6658c1D50b3958B3A1733d1cd1E1', 'decimals': 9},
    'DUCKER': {'address': '0xa52bfFAD02B1FE3f86A543a4e81962d3B3bB01A7', 'decimals': 18},
    'CLIQ': {'address': '0x0Def8d8addE14c9eF7c2a986dF3eA4Bd65826767', 'decimals': 18},
    'BPTL': {'address': '0x3a1BC4014C4C493DB3dBfBDD8ee1417113b462BF', 'decimals': 18},
    'CVXFXS': {'address': '0xFEEf77d3f69374f66429C91d732A244f074bdf74', 'decimals': 18},
    'OCB': {'address': '0x466912bAA9430a4a460b141ee8C580D817441449', 'decimals': 18},
    'GVT': {'address': '0x103c3A209da59d3E7C4A89307e66521e081CFDF0', 'decimals': 18},
    'NVIR': {'address': '0x9d71CE49ab8A0E6D2a1e7BFB89374C9392FD6804', 'decimals': 18},
    'EVEAI': {'address': '0x05fE069626543842439Ef90D9FA1633640c50cF1', 'decimals': 18},
    'AAAVE': {'address': '0xFFC97d72E13E01096502Cb8Eb52dEe56f74DAD7B', 'decimals': 18},
    'OBOT': {'address': '0xeDADeB5fAa413e6c8623461849DFD0B7C3790c32', 'decimals': 18},
    'FROGEX': {'address': '0x5fA54fdDF1870C344DbFaBb37dFab8700Ec0Def1', 'decimals': 9},
    'BLEYD': {'address': '0x59B44e9AC4AB8e29db6Faf7baCd726C6B7bC94b9', 'decimals': 9},
    'MNTP': {'address': '0x83cee9e086A77e492eE0bB93C2B0437aD6fdECCc', 'decimals': 18},
    'PDT': {'address': '0x375aBB85C329753b1Ba849a601438AE77eEc9893', 'decimals': 18},
    'OKINAMI': {'address': '0x1C4853Ec0d55e420002c5eFaBc7Ed8e0bA7A4121', 'decimals': 9},
    'RELAX': {'address': '0x801dE502C493aa9d3F9F79A0348fbc00521b3001', 'decimals': 18},
    'IXS': {'address': '0x73d7c860998CA3c01Ce8c808F5577d94d545d1b4', 'decimals': 18},
    'LIFE': {'address': '0x6c936D4AE98E6d2172dB18c16C4b601C99918EE6', 'decimals': 18},
    'DAM': {'address': '0xF80D589b3Dbe130c270a69F1a69D050f268786Df', 'decimals': 18},
    'DELREY': {'address': '0xfca89d55A768375Ab7cA04485A35a964BEA828dd', 'decimals': 18},
    'KAN': {'address': '0x1410434b0346f5bE678d0FB554E5c7ab620f8f4a', 'decimals': 18},
    'PDF': {'address': '0x0c943300D604c732117DB2755873A0A0683ee7a9', 'decimals': 18},
    'KST': {'address': '0xBc17729fDf562723f0267F79FF25aDE441056d87', 'decimals': 18},
    'UBSN': {'address': '0x86EFc496DcA70bcFD92D19194290e8457a375773', 'decimals': 0},
    'SFG': {'address': '0x8a6ACA71A218301c7081d4e96D64292D3B275ce0', 'decimals': 18},
    'PAR': {'address': '0x1BeEF31946fbbb40B877a72E4ae04a8D1A5Cee06', 'decimals': 18},
    'HVH': {'address': '0xd076C4Ba62c57b3Fa10800bcfD8dA66742110E0E', 'decimals': 18},
    'ZUM': {'address': '0xe0b9BcD54bF8A730EA5d3f1fFCe0885E911a502c', 'decimals': 8},
    'MAID': {'address': '0x4Af698B479D0098229DC715655c667Ceb6cd8433', 'decimals': 18},
    'FSCC': {'address': '0x0789dbAE94fb18e5789B8e4489BCB7A1ADB58622', 'decimals': 8},
    'POOLZ': {'address': '0x69A95185ee2a045CDC4bCd1b1Df10710395e4e23', 'decimals': 18},
    'MNY': {'address': '0xA6F7645ed967FAF708A614a2fcA8D4790138586f', 'decimals': 18},
    'KAWA': {'address': '0x5552E5a89A70cB2eF5AdBbC45a6BE442fE7160Ec', 'decimals': 9},
    'CRBN': {'address': '0xCdeee767beD58c5325f68500115d4B722b3724EE', 'decimals': 18},
    'BBANK': {'address': '0xF4b5470523cCD314C6B9dA041076e7D79E0Df267', 'decimals': 18},
    'SHIL': {'address': '0x20c3fa331A385b63EE39137e99d0cF2db142fCe1', 'decimals': 18},
    'YF-DAI': {'address': '0xf4CD3d3Fda8d7Fd6C5a500203e38640A70Bf9577', 'decimals': 18},
    'AURABAL': {'address': '0x616e8BfA43F920657B3497DBf40D6b1A02D4608d', 'decimals': 18},
    'ABAT': {'address': '0x05Ec93c0365baAeAbF7AefFb0972ea7ECdD39CF1', 'decimals': 18},
    'DFYN': {'address': '0x9695e0114e12C0d3A3636fAb5A18e6b737529023', 'decimals': 18},
    'SNT': {'address': '0x744d70FDBE2Ba4CF95131626614a1763DF805B9E', 'decimals': 18},
    'PBX': {'address': '0xD528cf2E081f72908e086F8800977df826B5a483', 'decimals': 18},
    'BKING': {'address': '0x1cA02DD95f3F1e33DA7F5AfE15EA866dAb07aF04', 'decimals': 9},
    'SARCO': {'address': '0x7697B462A7c4Ff5F8b55BDBC2F4076c2aF9cF51A', 'decimals': 18},
    'ODDZ': {'address': '0xCd2828fc4D8E8a0eDe91bB38CF64B1a81De65Bf6', 'decimals': 18},
    'CHP': {'address': '0x0a6E18fB2842855C3AF925310B0F50a4BfA17909', 'decimals': 18},
    'FEN': {'address': '0x995a6C0944322b7285D0712824acf0Cf9BF3993a', 'decimals': 18},
    'VIDYA': {'address': '0x3D3D35bb9bEC23b06Ca00fe472b50E7A4c692C30', 'decimals': 18},
    'STKR': {'address': '0x89DcFF5Fd892f2bfc8B75dBA12804B651f769579', 'decimals': 18},
    'LOOMOLD': {'address': '0xA4e8C3Ec456107eA67d3075bF9e3DF3A75823DB0', 'decimals': 18},
    'ZIOT': {'address': '0xfB22cED41B1267dA411F68c879f4Defd0bD4796a', 'decimals': 18},
    'DNXC': {'address': '0x20a8CEC5fffea65Be7122BCaB2FFe32ED4Ebf03a', 'decimals': 18},
    'STQ': {'address': '0x5c3a228510D246b78a3765C20221Cbf3082b44a4', 'decimals': 18},
    'CRAB': {'address': '0x24BCeC1AFda63E622a97F17cFf9a61FFCfd9b735', 'decimals': 18},
    'CERE': {'address': '0x2dA719DB753dFA10a62E140f436E1d67F2ddB0d6', 'decimals': 10},
    'BID': {'address': '0x00000000000045166C45aF0FC6E4Cf31D9E14B9A', 'decimals': 18},
    'VYNC': {'address': '0xee1ae38BE4Ce0074C4A4A8DC821CC784778f378c', 'decimals': 4},
    'DBX': {'address': '0x3CBc780D2934d55A06069e837fABd3e6Fc23DAB0', 'decimals': 18},
    'PHTR': {'address': '0xE1Fc4455f62a6E89476f1072530C20CF1A0622dA', 'decimals': 18},
    'DS': {'address': '0xB67bEB26eBeb0DCEec354aE0942256d03c01771b', 'decimals': 18},
    '2CRZ': {'address': '0x2C9C19cE3b15ae77C6d80aEc3C1194Cfd6F7F3fA', 'decimals': 18},
    'LORDS': {'address': '0x686f2404e77Ab0d9070a46cdfb0B7feCDD2318b0', 'decimals': 18},
    'MTHD': {'address': '0x84bA4aEcfDe39D69686a841BAb434C32d179a169', 'decimals': 18},
    'BOXFEE': {'address': '0xE4B91fAf8810F8895772E7cA065D4CB889120f94', 'decimals': 18},
    'AIT': {'address': '0x79650799e7899A802cB96C0Bc33a6a8d4CE4936C', 'decimals': 18},
    'BANK': {'address': '0x24A6A37576377F63f194Caa5F518a60f45b42921', 'decimals': 18},
    'DUCK': {'address': '0xC0bA369c8Db6eB3924965e5c4FD0b4C1B91e305F', 'decimals': 18},
    'UBI': {'address': '0xDd1Ad9A21Ce722C151A836373baBe42c868cE9a4', 'decimals': 18},
    'DAX': {'address': '0x0B4BdC478791897274652DC15eF5C135cae61E60', 'decimals': 18},
    'IZI': {'address': '0x9ad37205d608B8b219e6a2573f922094CEc5c200', 'decimals': 18},
    'SPAY': {'address': '0x58FaD9E3C3AE54c9BA98c3f0E4bF88aB3E8Cf3c5', 'decimals': 18},
    'TCP': {'address': '0x06576eB3B212d605B797dC15523d9Dc9F4F66DB4', 'decimals': 18},
    'BETT': {'address': '0x2f4b36442687E943E587b45cCcc92D60C3c3502F', 'decimals': 18},
    'MIRL': {'address': '0x8770b7Dd89E5f759eE3b226e0C45e890f87dDc48', 'decimals': 18},
    'SUTER': {'address': '0xAA2ce7Ae64066175E0B90497CE7d9c190c315DB4', 'decimals': 18},
    'TOC': {'address': '0x8Af078d5A3e4a18Ff62e3AEeD4b1c448aB90b304', 'decimals': 18},
    'SOFI': {'address': '0xB49fa25978abf9a248b8212Ab4b87277682301c0', 'decimals': 18},
    'XNO': {'address': '0x05bBe7240De66F6480C9aeda77C1376B13393F83', 'decimals': 18},
    'VRO': {'address': '0x10BC518c32fbAE5e38Ecb50A612160571bD81e44', 'decimals': 8},
    'KING': {'address': '0x149D8290F653dEB8e34c037D239d3d8EeE9de5aD', 'decimals': 18},
    'FT': {'address': '0xEe9E7Bb7E55bBc86414047B61D65c9C0d91fFBd0', 'decimals': 18},
    'HILO': {'address': '0xbb9FD9Fa4863C03c574007fF3370787B9cE65ff6', 'decimals': 18},
    'KRL': {'address': '0x464eBE77c293E473B48cFe96dDCf88fcF7bFDAC0', 'decimals': 18},
    'ZLW': {'address': '0x5319e86F0e41a06E49eb37046b8c11D78bcAd68C', 'decimals': 18},
    'EXN': {'address': '0x0c9b3aB1bd0CF0745625381F5C3Aa1CD9BBc7Abb', 'decimals': 18},
    'MAX': {'address': '0xe7976c4Efc60d9f4C200Cc1bCEF1A1e3B02c73e7', 'decimals': 18},
    'DF': {'address': '0x431ad2ff6a9C365805eBaD47Ee021148d6f7DBe0', 'decimals': 18},
    'MEEB': {'address': '0x641927E970222B10b2E8CDBC96b1B4F427316f16', 'decimals': 18},
    'PIPT': {'address': '0x26607aC599266b21d13c7aCF7942c7701a8b699c', 'decimals': 18},
    'CNT': {'address': '0x429876c4a6f89FB470E92456B8313879DF98B63c', 'decimals': 18},
    'CFXQ': {'address': '0x0557E0d15aeC0b9026dD17aA874fDf7d182A2cEB', 'decimals': 6},
    'DCN': {'address': '0x08d32b0da63e2C3bcF8019c9c5d849d7a9d791e6', 'decimals': 0},
    'WHL': {'address': '0x2aF72850c504dDD3c1876C66a914cAee7Ff8a46A', 'decimals': 18},
    'FLOKI': {'address': '0x67Cc621AB2d086a101Cff3340df0A065Ac75827c', 'decimals': 18},
    'CRT': {'address': '0x36a28c7c9B3DEA22F07F4dF67833cBe764fEEeb4', 'decimals': 18},
    'IMC': {'address': '0xBF0741E995F469D39e4f96C0780f9a8E43f4b978', 'decimals': 18},
    'ASTRA': {'address': '0x201332Bd45c8628D814F870bFb584B385A7C351e', 'decimals': 18},
    'COVN': {'address': '0x19ac2659599fD01c853dE846919544276aD26F50', 'decimals': 18},
    'OGV': {'address': '0x9c354503C38481a7A7a51629142963F98eCC12D0', 'decimals': 18},
    'SHROOM': {'address': '0xEd0439EACf4c4965AE4613D77a5C2Efe10e5f183', 'decimals': 18},
    'ZKP': {'address': '0x09637E374Ec5baB8C9e9654a97C9593e3038c5B3', 'decimals': 9},
    'TEMP': {'address': '0xA36FDBBAE3c9d55a1d67EE5821d53B50B63A1aB9', 'decimals': 18},
    'BBW': {'address': '0xd2287a52287dd64AE2c690D74322E2e345dC8F69', 'decimals': 18},
    'BABYTRUMP': {'address': '0x99d36e97676A68313ffDc627fd6b56382a2a08B6', 'decimals': 9},
    'STKABPT': {'address': '0xa1116930326D21fB917d5A27F1E9943A9595fb47', 'decimals': 18},
    'ACQ': {'address': '0x4BDCb66B968060D9390C1D12bd29734496205581', 'decimals': 18},
    'SYNC': {'address': '0xB6ff96B8A8d214544Ca0dBc9B33f7AD6503eFD32', 'decimals': 18},
    'DINERO': {'address': '0xa3ad8c7AB6b731045b5b16E3fDf77975c71ABe79', 'decimals': 18},
    'ECT': {'address': '0xda4Dd9586D27202A338843Dd6B9824d267006783', 'decimals': 9},
    'CVT': {'address': '0xBe428c3867F05deA2A89Fc76a102b544eaC7f772', 'decimals': 18},
    'MVL': {'address': '0xA849EaaE994fb86Afa73382e9Bd88c2B6b18Dc71', 'decimals': 18},
    'NTVRK': {'address': '0xFc0d6Cf33e38bcE7CA7D89c0E292274031b7157A', 'decimals': 18},
    'SI': {'address': '0xD23Ac27148aF6A2f339BD82D0e3CFF380b5093de', 'decimals': 18},
    'DFA': {'address': '0x62959c699A52eC647622c91e79CE73344e4099f5', 'decimals': 18},
    'PACK': {'address': '0xb186035490C8602EaD853EC98bE05E3461521Ab2', 'decimals': 18},
    'SOMEE': {'address': '0x65F9A292f1AEED5D755Aa2fD2Fb17AB2E9431447', 'decimals': 18},
    'XMU': {'address': '0x56123908ACA89bf60cE9659aE9F4589E46c8D2cF', 'decimals': 9},
    'SAT': {'address': '0xc9F1016d336ef77AeE75Fc11Ad64C5Ecf9121332', 'decimals': 18},
    'YIELD': {'address': '0xa8B61CfF52564758A204F841E636265bEBC8db9B', 'decimals': 18},
    'MPM': {'address': '0x18b52F500e6D9C8b0455eC3483846D9f8edd7e1b', 'decimals': 18},
    'DXGM': {'address': '0x66F73D0fD4161cfad4302DC145Ff994375c13475', 'decimals': 18},
    'SWIV': {'address': '0xbf30461210b37012783957D90dC26B95Ce3b6f2d', 'decimals': 18},
    'API': {'address': '0xfd4168e642EbD04C3684A6cDb3A5E86DE85d3908', 'decimals': 18},
    'OCTO': {'address': '0x7240aC91f01233BaAf8b064248E80feaA5912BA3', 'decimals': 18},
    'BETA': {'address': '0xBe1a001FE942f96Eea22bA08783140B9Dcc09D28', 'decimals': 18},
    'SOLVE': {'address': '0x446C9033E7516D820cc9a2ce2d0B7328b579406F', 'decimals': 8},
    'SWTH': {'address': '0xB4371dA53140417CBb3362055374B10D97e420bB', 'decimals': 8},
    'RAID': {'address': '0x2022760aCDe82079cbEDa17D3afC67dC1F3D97C9', 'decimals': 18},
    'JPEG': {'address': '0xE80C0cd204D654CEbe8dd64A4857cAb6Be8345a3', 'decimals': 18},
    'YAM': {'address': '0x0AaCfbeC6a24756c20D41914F2caba817C0d8521', 'decimals': 18},
    'XSGD': {'address': '0x70e8dE73cE538DA2bEEd35d14187F6959a8ecA96', 'decimals': 6},
    'DUCK': {'address': '0x92E187a03B6CD19CB6AF293ba17F2745Fd2357D5', 'decimals': 18},
    'WOOL': {'address': '0x8355DBE8B0e275ABAd27eB843F3eaF3FC855e525', 'decimals': 18},
    'CUSDT': {'address': '0xf650C3d88D12dB855b8bf7D11Be6C55A4e07dCC9', 'decimals': 8},
    'TRIO': {'address': '0xF55cD1e399e1cc3D95303048897a680be3313308', 'decimals': 8},
    'MVC': {'address': '0x581911b360B6eB3a14eF295a83a91DC2bCE2D6f7', 'decimals': 18},
    'DNT': {'address': '0x0AbdAce70D3790235af448C88547603b945604ea', 'decimals': 18},
    'ARCHIE': {'address': '0xFE5F69dfa2d4501E78078266F6d430c079098f90', 'decimals': 9},
    'SAO': {'address': '0x3EF389f264e07fFF3106A3926F2a166d1393086F', 'decimals': 9},
    'AAMMUSDT': {'address': '0x17a79792Fe6fE5C95dFE95Fe3fCEE3CAf4fE4Cb7', 'decimals': 6},
    'ELTCOIN': {'address': '0x44197A4c44D6A059297cAf6be4F7e172BD56Caaf', 'decimals': 8},
    'WAMPL': {'address': '0xEDB171C18cE90B633DB442f2A6F72874093b49Ef', 'decimals': 18},
    'NCT': {'address': '0x9E46A38F5DaaBe8683E10793b06749EEF7D733d1', 'decimals': 18},
    'PANTO': {'address': '0x54B8D105aA09342FAD6b352D41a0bAd3E1a9aA9d', 'decimals': 18},
    'DACT': {'address': '0x343c6de13833bc7d9890eb6B1cd3FBeBC730EBec', 'decimals': 9},
    'LOC': {'address': '0x5e3346444010135322268a4630d2ED5F8D09446c', 'decimals': 18},
    'ALD': {'address': '0xb26C4B3Ca601136Daf98593feAeff9E0CA702a8D', 'decimals': 18},
    'CORE': {'address': '0x62359Ed7505Efc61FF1D56fEF82158CcaffA23D7', 'decimals': 18},
    'TOL': {'address': '0xd07D9Fe2d2cc067015E2b4917D24933804f42cFA', 'decimals': 18},
    'FAR': {'address': '0xAaE3Cf9968D26925BDb73cE3864e0084a20f4687', 'decimals': 18},
    'CFL365': {'address': '0xCD6Adc6b8Bd396E2D53cCD7D7257B4De55bE4fbe', 'decimals': 18},
    'ASUSD': {'address': '0x625aE63000f46200499120B906716420bd059240', 'decimals': 18},
    'ICH': {'address': '0xf8483E2d6560585C02D46bF7B3186Bf154a96166', 'decimals': 8},
    'COV': {'address': '0xADA86b1b313D1D5267E3FC0bB303f0A2b66D0Ea7', 'decimals': 18},
    'XAUR': {'address': '0x4DF812F6064def1e5e029f1ca858777CC98D2D81', 'decimals': 8},
    'CHIRO': {'address': '0x35156b404C3f9bdaf45ab65Ba315419bcDe3775c', 'decimals': 9},
    'VIB': {'address': '0x2C974B2d0BA1716E644c1FC59982a89DDD2fF724', 'decimals': 18},
    'ACE': {'address': '0xeC5483804e637D45cDe22Fa0869656B64b5AB1ab', 'decimals': 18},
    'DHP': {'address': '0xc40F23A3E9613E012944f7957edCe97899Fa920d', 'decimals': 18},
    'COLLAR': {'address': '0xB0e977e0214BCfEec0D685064244Ad2b2487380b', 'decimals': 18},
    'AGRS': {'address': '0x738865301A9b7Dd80Dc3666dD48cF034ec42bdDa', 'decimals': 8},
    'MMC': {'address': '0xA4C612296aa97057c160B95255f902F9dDe6704c', 'decimals': 18},
    'BNSD': {'address': '0x668DbF100635f593A3847c0bDaF21f0a09380188', 'decimals': 18},
    'PUNDIX': {'address': '0x0FD10b9899882a6f2fcb5c371E17e70FdEe00C38', 'decimals': 18},
    'IONX': {'address': '0x02D3A27Ac3f55d5D91Fb0f52759842696a864217', 'decimals': 18},
    'FUSDT': {'address': '0x81994b9607e06ab3d5cF3AffF9a67374f05F27d7', 'decimals': 8},
    'FANC': {'address': '0xbB126042235E6bD38B17744cb31a8bF4A206c045', 'decimals': 18},
    'MULTIBTC': {'address': '0x66eFF5221ca926636224650Fd3B9c497FF828F7D', 'decimals': 8},
    'SNFT': {'address': '0xA3c4Dc4A9cE2a6B40B57F25F8b50DeCc2c64deC2', 'decimals': 18},
    'QUARTZ': {'address': '0xbA8A621b4a54e61C442F5Ec623687e2a942225ef', 'decimals': 18},
    'JOY': {'address': '0x4E114d405b9Ba2F59524941733e505Ae03Fb1FB5', 'decimals': 18},
    'BLOOD': {'address': '0x95392f142Af1c12F6e39897Ff9B09c599666B50c', 'decimals': 18},
    'VEN': {'address': '0x3A529A8d4f2ea64D206339fA12A3aF4D431F53c3', 'decimals': 18},
    'NFTI': {'address': '0xe5feeaC09D36B18b3FA757E5Cf3F8dA6B8e27F4C', 'decimals': 18},
    'UMAD': {'address': '0x31c2415c946928e9FD1Af83cdFA38d3eDBD4326f', 'decimals': 8},
    'TSCT': {'address': '0x805EA9c07B49dd23cE11ec66dC6d8a2957385035', 'decimals': 18},
    'GYOKUTO': {'address': '0x166D45e55143ea15032672Ac9f591A4401CC612D', 'decimals': 9},
    'KEYFI': {'address': '0xB8647e90C0645152Fccf4d9AbB6B59Eb4AA99052', 'decimals': 18},
    'FOUR': {'address': '0x4730fB1463A6F1F44AEB45F6c5c422427f37F4D0', 'decimals': 18},
    'WTC': {'address': '0xb7cB1C96dB6B22b0D3d9536E0108d062BD488F74', 'decimals': 18},
    'LAIKA': {'address': '0x77f9CF0bd8C500CfFdF420e72343893aeCC2EC0b', 'decimals': 18},
    'WXT': {'address': '0xa02120696c7B8fE16C09C749E4598819b2B0E915', 'decimals': 18},
    'HACHI': {'address': '0x2a7e415C169cE3a580c6F374Dc26f6Aaad1EccFe', 'decimals': 18},
    'VRX': {'address': '0x87DE305311D5788e8da38D19bb427645b09CB4e5', 'decimals': 18},
    'DHV': {'address': '0x62Dc4817588d53a056cBbD18231d91ffCcd34b2A', 'decimals': 18},
    'TMTG': {'address': '0x10086399DD8c1e3De736724AF52587a2044c9fA2', 'decimals': 18},
    'LA': {'address': '0xE50365f5D679CB98a1dd62D6F6e58e59321BcdDf', 'decimals': 18},
    'SWIFY': {'address': '0x9C7376b5060C47E555f5F53005aA3D5A7C66f2fB', 'decimals': 9},
    'RIOT': {'address': '0xf56408077487cB879c992909C5b5C66D68c02Eb4', 'decimals': 18},
    'AX': {'address': '0xCB0D82f4DFA503C9e3b8Abc7A3Caa01175b2dA39', 'decimals': 18},
    'CVXCRV': {'address': '0x62B9c7356A2Dc64a1969e19C23e4f579F9810Aa7', 'decimals': 18},
    'HOGE': {'address': '0xfAd45E47083e4607302aa43c65fB3106F1cd7607', 'decimals': 9},
    'KOROMARU': {'address': '0xD0D42005e7B3c0812b1268F0e5FAF97Ff2423651', 'decimals': 9},
    'CCX': {'address': '0x1D6405138A335cE5FD7364086334EfB3E4F28B59', 'decimals': 18},
    'NCDT': {'address': '0xE0C8b298db4cfFE05d1bEA0bb1BA414522B33C1B', 'decimals': 18},
    'SGTV2': {'address': '0x24C19F7101c1731b85F1127EaA0407732E36EcDD', 'decimals': 18},
    'SUMMER': {'address': '0x4D4f3715050571A447FfFa2Cd4Cf091C7014CA5c', 'decimals': 18},
    'UMX': {'address': '0x10Be9a8dAe441d276a5027936c3aADEd2d82bC15', 'decimals': 18},
    'TOS': {'address': '0x409c4D8cd5d2924b9bc5509230d16a61289c8153', 'decimals': 18},
    'DGTX': {'address': '0xc666081073E8DfF8D3d1c2292A29aE1A2153eC09', 'decimals': 18},
    'VAL': {'address': '0xe88f8313e61A97cEc1871EE37fBbe2a8bf3ed1E4', 'decimals': 18},
    'GOLD': {'address': '0x486F4641eF2B50cC130Dadbd27B6F271723873b8', 'decimals': 18},
    'NFY': {'address': '0x1cBb83EbcD552D5EBf8131eF8c9CD9d9BAB342bC', 'decimals': 18},
    'PPT': {'address': '0xd4fa1460F537bb9085d22C7bcCB5DD450Ef28e3a', 'decimals': 8},
    'TRZ': {'address': '0x394A16744Dcd805BB0cA7252e70691f0dcAc56AA', 'decimals': 18},
    'WCA': {'address': '0x506B8f75BDeF0Edac36B0A6f9CF313485E4341b0', 'decimals': 18},
    'TOTORO': {'address': '0xfc09c7cFD9c175DD9423ca02aE1249579AB12F12', 'decimals': 9},
    'DOE': {'address': '0xf8E9F10c22840b613cdA05A0c5Fdb59A4d6cd7eF', 'decimals': 18},
    'BIOS': {'address': '0xAACa86B876ca011844b5798ECA7a67591A9743C8', 'decimals': 18},
    'UNIQ': {'address': '0x3758e00b100876C854636eF8Db61988931BB8025', 'decimals': 18},
    'KOY': {'address': '0x198065e69A86cb8a9154B333AAD8eFE7A3c256F8', 'decimals': 18},
    'EXE': {'address': '0x412D397DDCa07D753E3E0C61e367fb1b474B3E7D', 'decimals': 18},
    'SHIBO': {'address': '0x38e3b07D607dEF629b4f3f46a0082006fdd6CDa2', 'decimals': 18},
    'WALLET': {'address': '0x88800092fF476844f74dC2FC427974BBee2794Ae', 'decimals': 18},
    'UNIC': {'address': '0x94E0BAb2F6Ab1F19F4750E42d7349f2740513aD5', 'decimals': 18},
    'CPAY': {'address': '0x0Ebb614204E47c09B6C3FeB9AAeCad8EE060E23E', 'decimals': 0},
    'SUNC': {'address': '0x692aCCdD8b86692427E0aa4752AE917Df01CC56F', 'decimals': 18},
    'CART': {'address': '0x5C8C8D560048F34E5f7f8ad71f2f81a89DBd273e', 'decimals': 18},
    'AAMMUNIYFIWETH': {'address': '0x5394794Be8b6eD5572FCd6b27103F46b5F390E8f', 'decimals': 18},
    'ENTR': {'address': '0xd779eEA9936B4e323cDdff2529eb6F13d0A4d66e', 'decimals': 18},
    'FGHT': {'address': '0xe0ffDdF8fd7d63Cf907e433c9293069704D4d40D', 'decimals': 18},
    'ARCONA': {'address': '0x0f71B8De197A1C84d31de0F1fA7926c365F052B3', 'decimals': 18},
    'LOCUS': {'address': '0xC64500DD7B0f1794807e67802F8Abbf5F8Ffb054', 'decimals': 18},
    'RENDOGE': {'address': '0x3832d2F059E55934220881F831bE501D180671A7', 'decimals': 8},
    'DAKS': {'address': '0x6f38e0f1a73c96cB3f42598613EA3474F09cB200', 'decimals': 0},
    'STOS': {'address': '0x08c32b0726C5684024ea6e141C50aDe9690bBdcc', 'decimals': 18},
    'TAI': {'address': '0x2F4404C4012476929b6503E1397707480bf23B7f', 'decimals': 9},
    'MELOS': {'address': '0x1afb69DBC9f54d08DAB1bD3436F8Da1af819E647', 'decimals': 18},
    'ISDT': {'address': '0xf2354F740f31704820f6FcfBA70B9dA065459b62', 'decimals': 18},
    'BPT': {'address': '0x0eC9F76202a7061eB9b3a7D6B59D36215A7e37da', 'decimals': 18},
    'LMT': {'address': '0x327673aE6B33Bd3d90f0096870059994f30Dc8AF', 'decimals': 18},
    'USDTSO': {'address': '0x1CDD2EaB61112697626F7b4bB0e23Da4FeBF7B7C', 'decimals': 6},
    'STARX': {'address': '0xC4e8A9D47000Ab8E59c7031e311762c68215e467', 'decimals': 18},
    'KTO': {'address': '0x616ef40D55C0D2c506f4d6873Bda8090b79BF8fC', 'decimals': 9},
    'BLZN': {'address': '0x70eFDc485a10210B056eF8e0A32993Bc6529995E', 'decimals': 18},
    'BIRD': {'address': '0x70401dFD142A16dC7031c56E862Fc88Cb9537Ce0', 'decimals': 18},
    'BMX': {'address': '0x986EE2B944c42D017F52Af21c4c69B84DBeA35d8', 'decimals': 18},
    'BP': {'address': '0xdF290B162a7D3E0A328cF198308D421954f08b94', 'decimals': 18},
    'UMI': {'address': '0x61107a409fFFe1965126aa456Af679719695C69C', 'decimals': 18},
    'PANDA': {'address': '0x3cBb7f5d7499Af626026E96a2f05df806F2200DC', 'decimals': 18},
    'FLX': {'address': '0x3Ea8ea4237344C9931214796d9417Af1A1180770', 'decimals': 18},
    'UUU': {'address': '0x3543638eD4a9006E4840B105944271Bcea15605D', 'decimals': 18},
    'MAYP': {'address': '0x7CDA79830Faf07Ed696Fe220566116951CED36A7', 'decimals': 18},
    'VBNT': {'address': '0x48Fb253446873234F2fEBbF9BdeAA72d9d387f94', 'decimals': 18},
    'LGCY': {'address': '0xaE697F994Fc5eBC000F8e22EbFfeE04612f98A0d', 'decimals': 18},
    'BASE': {'address': '0x07150e919B4De5fD6a63DE1F9384828396f25fDC', 'decimals': 9},
    'F9': {'address': '0x38A94e92A19E970c144DEd0B2DD47278CA11CC1F', 'decimals': 9},
    'ELF': {'address': '0xbf2179859fc6D5BEE9Bf9158632Dc51678a4100e', 'decimals': 18},
    'RCN': {'address': '0xF970b8E36e23F7fC3FD752EeA86f8Be8D83375A6', 'decimals': 18},
    'WAIT': {'address': '0xb5588C411ba0bb7D38865fdC51D082d004e519F7', 'decimals': 0},
    'DEGEN': {'address': '0x126c121f99e1E211dF2e5f8De2d96Fa36647c855', 'decimals': 18},
    'CHAMP': {'address': '0x456125Cd98107ae0480Ba566f1b716D48Ba31453', 'decimals': 18},
    'PIE': {'address': '0x607C794cDa77efB21F8848B7910ecf27451Ae842', 'decimals': 18},
    'TB': {'address': '0xD228ceCa8560DDB74137f44D9c582e1a646Be763', 'decimals': 9},
    'ANGLE': {'address': '0x31429d1856aD1377A8A0079410B297e1a9e214c2', 'decimals': 18},
    'NIOX': {'address': '0xc813EA5e3b48BEbeedb796ab42A30C5599b01740', 'decimals': 4},
    'USDZ': {'address': '0x8baEF8c9568c21B1a2b2fd048F8b4Da835691fD0', 'decimals': 18},
    'CREAM': {'address': '0x2ba592F78dB6436527729929AAf6c908497cB200', 'decimals': 18},
    'SHIBGF': {'address': '0x505a84a03e382331A1Be487b632Cf357748b65d6', 'decimals': 18},
    'ELAN': {'address': '0x1b5036bEc1B82D44d52Fa953A370b3c6Cd9328B5', 'decimals': 18},
    'POCHI': {'address': '0x8eB1779A32678452eB273A22d413207299904d90', 'decimals': 18},
    'CITADEL': {'address': '0x6caDf6ABBCeb53e631C288778DAAcF125481C1BB', 'decimals': 18},
    'POOL': {'address': '0x0cEC1A9154Ff802e7934Fc916Ed7Ca50bDE6844e', 'decimals': 18},
    'XDG': {'address': '0x4f81C790581b240A5C948Afd173620EcC8C71c8d', 'decimals': 18},
    'SYNR': {'address': '0xbc6E06778708177a18210181b073DA747C88490a', 'decimals': 18},
    'UDT': {'address': '0x90DE74265a416e1393A450752175AED98fe11517', 'decimals': 18},
    'SMART': {'address': '0xce20BB92CCF9BbF5091ef85649E71E552819AD8c', 'decimals': 18},
    'SHIBDOGE': {'address': '0x6ADb2E268de2aA1aBF6578E4a8119b960E02928F', 'decimals': 9},
    'RETH': {'address': '0xae78736Cd615f374D3085123A210448E74Fc6393', 'decimals': 18},
    'BOXETH': {'address': '0x7690202e2C2297bcD03664e31116d1dFfE7e3B73', 'decimals': 18},
    'ORC': {'address': '0x662b67d00A13FAf93254714DD601F5Ed49Ef2F51', 'decimals': 18},
    'IAG': {'address': '0x40EB746DEE876aC1E78697b7Ca85142D178A1Fc8', 'decimals': 18},
    'MATICX': {'address': '0xf03A7Eb46d01d9EcAA104558C732Cf82f6B6B645', 'decimals': 18},
    'WBT': {'address': '0x925206b8a707096Ed26ae47C84747fE0bb734F59', 'decimals': 8},
    'FODL': {'address': '0x4C2e59D098DF7b6cBaE0848d66DE2f8A4889b9C3', 'decimals': 18},
    'IHT': {'address': '0xEda8B016efA8b1161208Cf041cD86972eeE0F31E', 'decimals': 18},
    'AQTIS': {'address': '0x6FF2241756549B5816A177659E766EAf14B34429', 'decimals': 18},
    'UNT': {'address': '0x8d610E20481F4C4f3aCB87bBa9c46BeF7795fdFe', 'decimals': 18},
    'ODA': {'address': '0x04DC37B220A055c5F93680815F670babCD912c2C', 'decimals': 9},
    'APM': {'address': '0xC8C424B91D8ce0137bAB4B832B7F7D154156BA6c', 'decimals': 18},
    'VEGA': {'address': '0xcB84d72e61e383767C4DFEb2d8ff7f4FB89abc6e', 'decimals': 18},
    'LBK': {'address': '0x9cB1AEaFcc8A9406632C5B084246Ea72f62d37b6', 'decimals': 8},
    'INNBC': {'address': '0xB67718b98d52318240c52E71A898335da4A28c42', 'decimals': 6},
    'BCB': {'address': '0x2d886570A0dA04885bfD6eb48eD8b8ff01A0eb7e', 'decimals': 9},
    'UBT': {'address': '0x8400D94A5cb0fa0D041a3788e395285d61c9ee5e', 'decimals': 8},
    'DREP': {'address': '0x3Ab6Ed69Ef663bd986Ee59205CCaD8A20F98b4c2', 'decimals': 18},
    'QR': {'address': '0xCcDb064a41AFcD432BCEa6F6d0D1E7c371d0B002', 'decimals': 9},
    'PATH': {'address': '0x2a2550e0A75aCec6D811AE3930732F7f3ad67588', 'decimals': 18},
    'EM': {'address': '0x35b08722AA26bE119c1608029CcbC976ac5C1082', 'decimals': 8},
    'FST': {'address': '0x0E192d382a36De7011F795Acc4391Cd302003606', 'decimals': 18},
    'DAV': {'address': '0xd82Df0ABD3f51425Eb15ef7580fDA55727875f14', 'decimals': 18},
    'CONV': {'address': '0xc834Fa996fA3BeC7aAD3693af486ae53D8aA8B50', 'decimals': 18},
    'AGEUR': {'address': '0x1a7e4e63778B4f12a199C062f3eFdD288afCBce8', 'decimals': 18},
    'STAT': {'address': '0x4FC15c91a9c4A9efB404174464687E8e128730C2', 'decimals': 18},
    'WPC': {'address': '0x6F620EC89B8479e97A6985792d0c64F237566746', 'decimals': 18},
    'OPENAIERC': {'address': '0x6A6AA13393B7d1100c00a57c76c39e8B6C835041', 'decimals': 9},
    'DECI': {'address': '0x6b32022693210cD2Cfc466b9Ac0085DE8fC34eA6', 'decimals': 8},
    'CBG': {'address': '0x1900E8B5619a3596745F715d0427Fe617c729BA9', 'decimals': 18},
    'OSWAP': {'address': '0xb32aC3C79A94aC1eb258f3C830bBDbc676483c93', 'decimals': 18},
    'LOCK': {'address': '0xb48EE8F8FA3885Fc08831ff366ec8df608159EE9', 'decimals': 18},
    'FRIN': {'address': '0xC9fE6E1C76210bE83DC1B5b20ec7FD010B0b1D15', 'decimals': 18},
    'TEAM': {'address': '0xB7c9E99Da8A857cE576A830A9c19312114d9dE02', 'decimals': 8},
    'FRR': {'address': '0xe6602B34D8510B033E000975B3322537c7172441', 'decimals': 18},
    'SNS': {'address': '0xEa4a2327E75252517535fd013b7C6706609819DB', 'decimals': 18},
    'FEI': {'address': '0x956F47F50A910163D8BF957Cf5846D573E7f87CA', 'decimals': 18},
    'PDEX': {'address': '0xF59ae934f6fe444afC309586cC60a84a0F89Aaea', 'decimals': 18},
    'DLT': {'address': '0x07e3c70653548B04f0A75970C1F81B4CBbFB606f', 'decimals': 18},
    'BONDLY': {'address': '0x91dFbEE3965baAEE32784c2d546B7a0C62F268c9', 'decimals': 18},
    'IM': {'address': '0x0A58153a0CD1cfaea94cE1f7FdC5D7E679eCa936', 'decimals': 18},
    'PAINT': {'address': '0x4C6eC08CF3fc987c6C4BEB03184D335A2dFc4042', 'decimals': 18},
    'BTCPX': {'address': '0x9C32185b81766a051E08dE671207b34466DD1021', 'decimals': 8},
    'ALUSD': {'address': '0xBC6DA0FE9aD5f3b0d58160288917AA56653660E9', 'decimals': 18},
    'ETH2X-FLI': {'address': '0xAa6E8127831c9DE45ae56bB1b0d4D4Da6e5665BD', 'decimals': 18},
    'SIG': {'address': '0x7777777777697cFEECF846A76326dA79CC606517', 'decimals': 18},
    'UNBNK': {'address': '0x06B884e60794Ce02AafAb13791B59A2e6A07442f', 'decimals': 18},
    'KTN': {'address': '0x491E136FF7FF03E6aB097E54734697Bb5802FC1C', 'decimals': 18},
    'WELD': {'address': '0xDD2a36AE937bc134ea694d77fC7e2E36f5d86dE0', 'decimals': 18},
    'RLTM': {'address': '0x1b396CF90504686b7edE2760eebb053d30d1f389', 'decimals': 18},
    'LUA': {'address': '0xB1f66997A5760428D3a87D68b90BfE0aE64121cC', 'decimals': 18},
    'BEST': {'address': '0x1B073382E63411E3BcfFE90aC1B9A43feFa1Ec6F', 'decimals': 8},
    'RENBTCCURVE': {'address': '0x49849C98ae39Fff122806C06791Fa73784FB3675', 'decimals': 18},
    'BTE': {'address': '0x73B708E84837FFCcDE2933e3A1531fe61D5e80Ef', 'decimals': 18},
    'DIE': {'address': '0x6Ef6610d24593805144d73b13d4405E00A4E4aC7', 'decimals': 18},
    'AUSDT': {'address': '0x71fc860F7D3A592A4a98740e39dB31d25db65ae8', 'decimals': 6},
    'KAE': {'address': '0x65Def5029A0e7591e46B38742bFEdd1Fb7b24436', 'decimals': 18},
    'DHT': {'address': '0xca1207647Ff814039530D7d35df0e1Dd2e91Fa84', 'decimals': 18},
    'TFBX': {'address': '0x4057Db5bD9f67A566aA10E5587b1a964afFc6a16', 'decimals': 18},
    'ETHA': {'address': '0x59E9261255644c411AfDd00bD89162d09D862e38', 'decimals': 18},
    'DFD': {'address': '0x20c36f062a31865bED8a5B1e512D9a1A20AA333A', 'decimals': 18},
    'LTX': {'address': '0xa393473d64d2F9F026B60b6Df7859A689715d092', 'decimals': 8},
    'TXL': {'address': '0x8eEF5a82E6Aa222a60F009ac18c24EE12dBf4b41', 'decimals': 18},
    'EBSO': {'address': '0x866f8a50A64e68ca66E97e032c5Da99538B3F942', 'decimals': 4},
    'BTCM': {'address': '0xA9Aad2dC3a8315caeee5F458B1d8EDc31D8467BD', 'decimals': 18},
    'EMTRG': {'address': '0xBd2949F67DcdC549c6Ebe98696449Fa79D988A9F', 'decimals': 18},
    'ITGR': {'address': '0xD502F487e1841Fdc805130e13eae80c61186Bc98', 'decimals': 18},
    'CIRUS': {'address': '0xA01199c61841Fce3b3daFB83FeFC1899715c8756', 'decimals': 18},
    'SPORK': {'address': '0xb624FdE1a972B1C89eC1dAD691442d5E8E891469', 'decimals': 18},
    'HANDY': {'address': '0x8bbe1a2961B41340468D0548c2cd5B7DFA9b684c', 'decimals': 18},
    'FNT': {'address': '0xDc5864eDe28BD4405aa04d93E05A0531797D9D59', 'decimals': 6},
    'BTCP': {'address': '0x723CbfC05e2cfcc71d3d89e770D32801A5eEf5Ab', 'decimals': 8},
    'DVF': {'address': '0xDDdddd4301A082e62E84e43F474f044423921918', 'decimals': 18},
    'DEXE': {'address': '0xde4EE8057785A7e8e800Db58F9784845A5C2Cbd6', 'decimals': 18},
    'FSW': {'address': '0xfffffffFf15AbF397dA76f1dcc1A1604F45126DB', 'decimals': 18},
    'ASIC': {'address': '0x347a96a5BD06D2E15199b032F46fB724d6c73047', 'decimals': 12},
    'VSP': {'address': '0x1b40183EFB4Dd766f11bDa7A7c3AD8982e998421', 'decimals': 18},
    'PLAQ': {'address': '0xd6710559656f2ecD336b153B0719500Fff8C2435', 'decimals': 18},
    'REALM': {'address': '0x464FdB8AFFC9bac185A7393fd4298137866DCFB8', 'decimals': 18},
    'LIT': {'address': '0xb59490aB09A0f526Cc7305822aC65f2Ab12f9723', 'decimals': 18},
    'NEURONI': {'address': '0x922e2708462c7a3d014D8344F7C4d92b27ECf332', 'decimals': 18},
    'SHIN': {'address': '0xa27A6FA698a63f82e559914c11C6Cd888060660b', 'decimals': 8},
    'SHFT': {'address': '0xb17C88bDA07D28B3838E0c1dE6a30eAfBCF52D85', 'decimals': 18},
    'FREN': {'address': '0x8e14c88aB0644eF41bd7138ab91C0160D8c1583A', 'decimals': 18},
    'XCM': {'address': '0x36ac219f90f5A6A3C77f2a7B660E3cC701f68e25', 'decimals': 18},
    'ZENF': {'address': '0xE9b7B5D5e8d2BCC78884F9F9099bfA42A9e5c1A5', 'decimals': 18},
    'IXP': {'address': '0x54Cb643ab007f47882E8120A8c57B639005c2688', 'decimals': 18},
    'EQX': {'address': '0xBd3de9a069648c84d27d74d701C9fa3253098B15', 'decimals': 18},
    'OXT': {'address': '0x4575f41308EC1483f3d399aa9a2826d74Da13Deb', 'decimals': 18},
    'CRAMER': {'address': '0x64Df3aAB3b21cC275bB76c4A581Cf8B726478ee0', 'decimals': 18},
    'IOI': {'address': '0x8B3870Df408fF4D7C3A26DF852D41034eDa11d81', 'decimals': 6},
    'CVP': {'address': '0x38e4adB44ef08F22F5B5b76A8f0c2d0dCbE7DcA1', 'decimals': 18},
    'CRE8R': {'address': '0xaa61D5dec73971CD4a026ef2820bB87b4a4Ed8d6', 'decimals': 18},
    'IAI': {'address': '0x6dDe4fFD6dB302Bc9a46850f61399e082f6c2122', 'decimals': 18},
    'INVI': {'address': '0x356A5160F2B34BC8d88FB084745465eBBbed0174', 'decimals': 13},
    'BTTY': {'address': '0x9cc83D580180f0d37d00e5d86ce868f73b6E3D0A', 'decimals': 18},
    'RSS3': {'address': '0xc98D64DA73a6616c42117b582e832812e7B8D57F', 'decimals': 18},
    'RDPX': {'address': '0x0ff5A8451A839f5F0BB3562689D9A44089738D11', 'decimals': 18},
    'WEB3': {'address': '0xe8e8486228753E01Dbc222dA262Aa706Bd67e601', 'decimals': 18},
    'ZNX': {'address': '0x9471d30D78a3C9f076cE206D14867a8d8BE1eFdE', 'decimals': 6},
    'ZB': {'address': '0xBd0793332e9fB844A52a205A233EF27a5b34B927', 'decimals': 18},
    'ECTR': {'address': '0x9301466c40B94942aF9ba63cbaEA1c784FF219B2', 'decimals': 18},
    'GLDN': {'address': '0xFeeB4D0f5463B1b04351823C246bdB84c4320CC2', 'decimals': 18},
    'NKPL': {'address': '0x2786F9C443F323f590dE1881d7215DB305aB846e', 'decimals': 18},
    'DODO': {'address': '0x43Dfc4159D86F3A37A5A4B3D4580b888ad7d4DDd', 'decimals': 18},
    'IJZ': {'address': '0x7453b01D746A72b6397e77C7d9de969FDBdE5a99', 'decimals': 18},
    'CORD': {'address': '0x6347978f54F75d76630b819897F9Ce68C83ce2a2', 'decimals': 9},
    'AAMMUNIWBTCUSDC': {'address': '0x2365a4890eD8965E564B7E2D27C38Ba67Fec4C6F', 'decimals': 18},
    '3CRV': {'address': '0x6c3F90f043a72FA612cbac8115EE7e52BDe6E490', 'decimals': 18},
    'UAC': {'address': '0x0AF5855a348C58a8BbF65BCc7B9BBaCD0e7e2238', 'decimals': 18},
    'TKP': {'address': '0xd31695a1d35E489252CE57b129FD4b1B05E6AcaC', 'decimals': 18},
    'JACK': {'address': '0x4E12EB8e506Ccd1427F6b8F7faa3e88fB698EB28', 'decimals': 18},
    'SWAG': {'address': '0x87eDfFDe3E14c7a66c9b9724747a1C5696b742e6', 'decimals': 18},
    'CVA': {'address': '0x78A52E12c7b63d05c12F9608307587CF654EC3d0', 'decimals': 18},
    'BETA': {'address': '0x35F67c1D929E106FDfF8D1A55226AFe15c34dbE2', 'decimals': 18},
    'HRDG': {'address': '0xe9D0331AD47148925c5009D33a428eE301C9D534', 'decimals': 12},
    'AWO': {'address': '0x68C85B9f78F30A0df5aC5723e4e700037f185415', 'decimals': 18},
    'FB': {'address': '0xFB19075D77a0F111796FB259819830F4780f1429', 'decimals': 6},
    'EVZ': {'address': '0x7A939Bb714fd2A48EbeB1E495AA9aaa74BA9fA68', 'decimals': 18},
    'HKBY': {'address': '0x645C52CF6c5C58AA4064494f5b5FFE9C7EC0d7D4', 'decimals': 18},
    'IXT': {'address': '0xfcA47962D45ADFdfd1Ab2D972315dB4ce7CCf094', 'decimals': 8},
    'ENG': {'address': '0xf0Ee6b27b759C9893Ce4f094b49ad28fd15A23e4', 'decimals': 8},
    'K9': {'address': '0x2bD0FB740e403B505A3146F9aC02df883Fd5C3Fc', 'decimals': 18},
    'DOGU': {'address': '0x8326Bf664704966C984a3A46Fa37d7A80A52dCf4', 'decimals': 18},
    'GTCETH': {'address': '0x36c833Eed0D376f75D1ff9dFDeE260191336065e', 'decimals': 18},
    'CIX': {'address': '0x6d60A8dFb16D09F67D46fcd36A0Cd310078257ca', 'decimals': 18},
    'YFII': {'address': '0xa1d0E215a23d7030842FC67cE582a6aFa3CCaB83', 'decimals': 18},
    'SOP': {'address': '0x076641aF1B8f06B7f8C92587156143C109002cbe', 'decimals': 18},
    'WOLFY': {'address': '0x7DbbCaE15d4DB168e01673400D7844870cc1e36f', 'decimals': 9},
    'HIT': {'address': '0x7995ab36bB307Afa6A683C24a25d90Dc1Ea83566', 'decimals': 6},
    'DRM': {'address': '0x89551b940e2A8ED8eCcF509935bAc9213fE30584', 'decimals': 18},
    'HBDC': {'address': '0x39d30828a163713D91c4EADBBA2C497a9139ec5C', 'decimals': 18},
    'CVAG': {'address': '0x72a66e54B66892aE3BBe54DF7bb7Dd5Ae927a6f9', 'decimals': 18},
    'GSHIBA': {'address': '0xb892249939AdBf6D7851864CA9A5c7D2d537af97', 'decimals': 18},
    'CNG': {'address': '0x883a158c9b28f8d626ACFCFbE1028f49E70c9D75', 'decimals': 18},
    'NFAI': {'address': '0x8eEcaad83a1Ea77bD88A818d4628fAfc4CaD7969', 'decimals': 18},
    'MDAI': {'address': '0xf6043bEa98B07F0bEa7813aeB22D0cb70C91C0c4', 'decimals': 18},
    'GMAT': {'address': '0xB13dE094Cc5CEe6C4cC0A3737bf0290166D9Ca5D', 'decimals': 18},
    'DCO': {'address': '0x2a304FDa5a85182dCA1D03741bB2F07881b9e095', 'decimals': 8},
    'UPUNK': {'address': '0x8d2BFfCbB19Ff14A698C424FbcDcFd17aab9b905', 'decimals': 18},
    'GTC': {'address': '0xB70835D7822eBB9426B56543E391846C107bd32C', 'decimals': 18},
    'GSBL': {'address': '0x4A5dec561de897c04062137e687db6093C3278B8', 'decimals': 18},
    'NFTX': {'address': '0x87d73E916D7057945c9BcD8cdd94e42A6F47f776', 'decimals': 18},
    'RHO': {'address': '0x3F3Cd642E81d030D7b514a2aB5e3a5536bEb90Ec', 'decimals': 18},
    'SYC': {'address': '0x0A41E6d4D4897E8Cec819ce7D10B2D7cDa61Dc94', 'decimals': 18},
    'IUSDC': {'address': '0xc8871267e07408b89aA5aEcc58AdCA5E574557F8', 'decimals': 6},
    'DSETH': {'address': '0x341c05c0E9b33C0E38d64de76516b2Ce970bB3BE', 'decimals': 18},
    'WRC': {'address': '0x72aDadb447784dd7AB1F472467750fC485e4cb2d', 'decimals': 6},
    'CAMP': {'address': '0xE9E73E1aE76D17A16cC53E3e87a9a7dA78834d37', 'decimals': 18},
    'LBA': {'address': '0xfe5F141Bf94fE84bC28deD0AB966c16B17490657', 'decimals': 18},
    'TOWER': {'address': '0x1C9922314ED1415c95b9FD453c3818fd41867d0B', 'decimals': 18},
    'GOLC': {'address': '0x095797FD4297fb79883cC912a5Ba6313B15c445d', 'decimals': 18},
    'KAERI': {'address': '0x69Ed89ecd35082E031fE52b75123F801DB083306', 'decimals': 9},
    'TOM': {'address': '0xF7970499814654CD13Cb7B6E7634A12a7A8A9ABc', 'decimals': 18},
    'NFD': {'address': '0xDFDb7f72c1F195C5951a234e8DB9806EB0635346', 'decimals': 18},
    'IVI': {'address': '0xA91464AbD4625A23aB719e3F0FCE84DaDd54E546', 'decimals': 18},
    'CHLT': {'address': '0x4a9d711100afF9A444a3c40284F70269Bb3f0487', 'decimals': 18},
    'HAKI': {'address': '0xF90c7f66eAc7E2130bF677D69a250b2136Cf6697', 'decimals': 18},
    'GOKU': {'address': '0xA64dFe8D86963151E6496BEe513E366F6e42ED79', 'decimals': 9},
    'HLTC': {'address': '0x2c000c0093dE75a8fA2FccD3d97b314e20b431C3', 'decimals': 18},
    'BDP': {'address': '0x593114f03A0A575aece9ED675e52Ed68D2172B8c', 'decimals': 18},
    'STON': {'address': '0xdC47f2Ba852669B178699449E50682D6CEAF8C07', 'decimals': 18},
    'HBCH': {'address': '0xaAC679720204aAA68B6C5000AA87D789a3cA0Aa5', 'decimals': 18},
    'STRONG': {'address': '0x990f341946A3fdB507aE7e52d17851B87168017c', 'decimals': 18},
    'GDC': {'address': '0x301C755bA0fcA00B1923768Fffb3Df7f4E63aF31', 'decimals': 18},
    'BIST': {'address': '0x6e8908cfa881C9f6f2C64d3436E7b80b1bf0093F', 'decimals': 18},
    'HTL': {'address': '0x6247C86B016Bc4d9aE141849C0a9Eb38C004B742', 'decimals': 18},
    'CLV': {'address': '0x80C62FE4487E1351b47Ba49809EBD60ED085bf52', 'decimals': 18},
    'SOS': {'address': '0x3b484b82567a09e2588A13D54D032153f0c0aEe0', 'decimals': 18},
    'TRU': {'address': '0xf65B5C5104c4faFD4b709d9D60a185eAE063276c', 'decimals': 18},
    'PRQ': {'address': '0x362bc847A3a9637d3af6624EeC853618a43ed7D2', 'decimals': 18},
    'TT': {'address': '0x1E053D89E08c24aA2CE5C5B4206744Dc2D7BD8F5', 'decimals': 18},
    'ISK': {'address': '0x17d2628D30F8e9E966c9Ba831c9B9b01ea8Ea75C', 'decimals': 18},
    'SUPER': {'address': '0xe53EC727dbDEB9E2d5456c3be40cFF031AB40A55', 'decimals': 18},
    'MXC': {'address': '0x5Ca381bBfb58f0092df149bD3D243b08B9a8386e', 'decimals': 18},
    'ALLIN': {'address': '0x9B2b931d6AB97B6A887b2C5d8529537E6FE73Ebe', 'decimals': 9},
    'FARM': {'address': '0xa0246c9032bC3A600820415aE600c6388619A14D', 'decimals': 18},
    'PMON': {'address': '0x1796ae0b0fa4862485106a0de9b654eFE301D0b2', 'decimals': 18},
    'JUP': {'address': '0x4B1E80cAC91e2216EEb63e29B957eB91Ae9C2Be8', 'decimals': 18},
    'POLS': {'address': '0x83e6f1E41cdd28eAcEB20Cb649155049Fac3D5Aa', 'decimals': 18},
    'MONGOOSE': {'address': '0xa1817B6d8D890F3943b61648992730373B71f156', 'decimals': 9},
    'RSV': {'address': '0x196f4727526eA7FB1e17b2071B3d8eAA38486988', 'decimals': 18},
    'WETH': {'address': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'decimals': 18},
    'TOWN': {'address': '0x3Dd98C8A089dBCFF7e8FC8d4f532BD493501Ab7F', 'decimals': 8},
    'DEXT': {'address': '0xfB7B4564402E5500dB5bB6d63Ae671302777C75a', 'decimals': 18},
    'MOON': {'address': '0x68a3637bA6E75c0f66B61A42639c4e9fCD3D4824', 'decimals': 18},
    'NFTY': {'address': '0xE1D7C7a4596B038CEd2A84bF65B8647271C53208', 'decimals': 18},
    'GHX': {'address': '0x728f30fa2f100742C7949D1961804FA8E0B1387d', 'decimals': 18},
    'PIXIA': {'address': '0x67675239Fa58c84e75f947c14f566842Dccb69Ae', 'decimals': 18},
    'GM': {'address': '0xe5597f0723EeaBa1B26948e06f008Bf0fc1e37e6', 'decimals': 18},
    'NMR': {'address': '0x1776e1F26f98b1A5dF9cD347953a26dd3Cb46671', 'decimals': 18},
    'PKF': {'address': '0x8B39B70E39Aa811b69365398e0aACe9bee238AEb', 'decimals': 18},
    'SKL': {'address': '0x00c83aeCC790e8a4453e5dD3B0B4b3680501a7A7', 'decimals': 18},
    'COVER': {'address': '0x4688a8b1F292FDaB17E9a90c8Bc379dC1DBd8713', 'decimals': 18},
    'COLI': {'address': '0xd49EFA7BC0D339D74f487959C573d518BA3F8437', 'decimals': 18},
    'MATH': {'address': '0x08d967bb0134F2d07f7cfb6E246680c53927DD30', 'decimals': 18},
    'EPK': {'address': '0xDaF88906aC1DE12bA2b1D2f7bfC94E9638Ac40c4', 'decimals': 18},
    'HAUS': {'address': '0xf2051511B9b121394FA75B8F7d4E7424337af687', 'decimals': 18},
    'USF': {'address': '0xE0e05c43c097B0982Db6c9d626c4eb9e95C3b9ce', 'decimals': 18},
    'MCADE': {'address': '0xed328E9C1179a30ddC1E7595E036AEd8760C22aF', 'decimals': 18},
    'NSURE': {'address': '0x20945cA1df56D237fD40036d47E866C7DcCD2114', 'decimals': 18},
    'UNO': {'address': '0x474021845C4643113458ea4414bdb7fB74A01A77', 'decimals': 18},
    'MEMAG': {'address': '0x6E39A587691b8c9D4341cE0A960998ED6F537AF6', 'decimals': 18},
    'INSUR': {'address': '0x544c42fBB96B39B21DF61cf322b5EDC285EE7429', 'decimals': 18},
    'POND': {'address': '0x57B946008913B82E4dF85f501cbAeD910e58D26C', 'decimals': 18},
    'DEUS': {'address': '0xDE5ed76E7c05eC5e4572CfC88d1ACEA165109E44', 'decimals': 18},
    'GET': {'address': '0x8a854288a5976036A725879164Ca3e91d30c6A1B', 'decimals': 18},
    'BEL': {'address': '0xA91ac63D040dEB1b7A5E4d4134aD23eb0ba07e14', 'decimals': 18},
    'BOG': {'address': '0x0BA74fb26Ca523f2dc22fa4318581CC2452eAbA1', 'decimals': 18},
    'IOTX': {'address': '0x6fB3e0A217407EFFf7Ca062D46c26E5d60a14d69', 'decimals': 18},
    'EUROC': {'address': '0x1aBaEA1f7C830bD89Acc67eC4af516284b1bC33c', 'decimals': 6},
    '00': {'address': '0x881Ba05de1E78f549cC63a8f6Cabb1d4AD32250D', 'decimals': 18},
    'PLU': {'address': '0xD8912C10681D8B21Fd3742244f44658dBA12264E', 'decimals': 18},
    'BBS': {'address': '0xFe459828c90c0BA4bC8b42F5C5D44F316700B430', 'decimals': 18},
    'DERC': {'address': '0x9fa69536d1cda4A04cFB50688294de75B505a9aE', 'decimals': 18},
    'AGI': {'address': '0x7dA2641000Cbb407C329310C461b2cB9c70C3046', 'decimals': 18},
    'MEMEME': {'address': '0x1A963Df363D01EEBB2816b366d61C917F20e1EbE', 'decimals': 18},
    'GAL': {'address': '0x5fAa989Af96Af85384b8a938c2EdE4A7378D9875', 'decimals': 18},
    'ISP': {'address': '0xC8807f0f5BA3fa45FfBdc66928d71c5289249014', 'decimals': 18},
    'USH': {'address': '0xE60779CC1b2c1d0580611c526a8DF0E3f870EC48', 'decimals': 18},
    'AZRX': {'address': '0xDf7FF54aAcAcbFf42dfe29DD6144A69b629f8C9e', 'decimals': 18},
    'Z3': {'address': '0x50Eec6D765792dCfB0913C8403ef2A12e1B861a6', 'decimals': 18},
    'RAD': {'address': '0x31c8EAcBFFdD875c74b94b077895Bd78CF1E64A3', 'decimals': 18},
    'TRYB': {'address': '0x2C537E5624e4af88A7ae4060C022609376C8D0EB', 'decimals': 6},
    'FREE': {'address': '0x2F141Ce366a2462f02cEA3D12CF93E4DCa49e4Fd', 'decimals': 18},
    'BGB': {'address': '0x19de6b897Ed14A376Dda0Fe53a5420D2aC828a28', 'decimals': 18},
    'YFI': {'address': '0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e', 'decimals': 18},
    'GLQ': {'address': '0x9F9c8ec3534c3cE16F928381372BfbFBFb9F4D24', 'decimals': 18},
    'UMEE': {'address': '0xc0a4Df35568F116C370E6a6A6022Ceb908eedDaC', 'decimals': 6},
    'IMGNAI': {'address': '0xA735A3AF76CC30791C61c10d585833829d36CBe0', 'decimals': 9},
    'REEF': {'address': '0xFE3E6a25e6b192A42a44ecDDCd13796471735ACf', 'decimals': 18},
    'MARSH': {'address': '0x5a666c7d92E5fA7Edcb6390E4efD6d0CDd69cF37', 'decimals': 18},
    'BOBA': {'address': '0x42bBFa2e77757C645eeaAd1655E0911a7553Efbc', 'decimals': 18},
    'BEZOGE': {'address': '0xDC349913d53b446485E98b76800b6254f43Df695', 'decimals': 9},
    'STABLZ': {'address': '0xA4Eb9C64eC359D093eAc7B65F51Ef933D6e5F7cd', 'decimals': 18},
    'OM': {'address': '0x3593D125a4f7849a1B059E64F4517A86Dd60c95d', 'decimals': 18},
    'PENDLE': {'address': '0x808507121B80c02388fAd14726482e061B8da827', 'decimals': 18},
    'TAMA': {'address': '0x12b6893cE26Ea6341919FE289212ef77e51688c8', 'decimals': 18},
    'SMETA': {'address': '0xAdc3F2C3D728202658930860158C726d8180a38F', 'decimals': 18},
    'BACON': {'address': '0x34f797e7190C131cF630524655A618b5BD8738e7', 'decimals': 18},
    'DPI': {'address': '0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b', 'decimals': 18},
    'DBI': {'address': '0x2dE509bf0014ddF697b220bE628213034d320EcE', 'decimals': 18},
    'OLT': {'address': '0x64A60493D888728Cf42616e034a0dfEAe38EFCF0', 'decimals': 18},
    'APEX': {'address': '0x52A8845DF664D76C69d2EEa607CD793565aF42B8', 'decimals': 18},
    'D2T': {'address': '0x4Dd942bAa75810a3C1E876e79d5cD35E09C97A76', 'decimals': 18},
    'LIME': {'address': '0x9D0B65a76274645B29e4cc41B8f23081fA09f4A3', 'decimals': 18},
    'LCX': {'address': '0x037A54AaB062628C9Bbae1FDB1583c195585fe41', 'decimals': 18},
    'SKEB': {'address': '0x6D614686550B9E1c1DF4b2cD8f91C9D4df66C810', 'decimals': 18},
    'RAI': {'address': '0x03ab458634910AaD20eF5f1C8ee96F1D6ac54919', 'decimals': 18},
    'KEKO': {'address': '0x2c7f442AaB99D5e18CfAE2291c507c0b5f3c1eb5', 'decimals': 18},
    'DOG': {'address': '0xBAac2B4491727D78D2b78815144570b9f2Fe8899', 'decimals': 18},
    'FVT': {'address': '0x45080a6531d671DDFf20DB42f93792a489685e32', 'decimals': 18},
    'PNT': {'address': '0x89Ab32156e46F46D02ade3FEcbe5Fc4243B9AAeD', 'decimals': 18},
    'STAKE': {'address': '0x0Ae055097C6d159879521C384F1D2123D1f195e6', 'decimals': 18},
    'TATE': {'address': '0xa589d8868607b8d79eE4288ce192796051263b64', 'decimals': 18},
    'ACH': {'address': '0xEd04915c23f00A313a544955524EB7DBD823143d', 'decimals': 8},
    'EJS': {'address': '0x96610186F3ab8d73EBEe1CF950C750f3B1Fb79C2', 'decimals': 18},
    'OMG': {'address': '0xd26114cd6EE289AccF82350c8d8487fedB8A0C07', 'decimals': 18},
    'BOBO': {'address': '0x5888641e3e6cBeA6D84Ba81EDb217bD691d3bE38', 'decimals': 9},
    'NARUTO': {'address': '0xAD8D0de33C43eEFe104A279cDB6Ae250C12e6214', 'decimals': 9},
    'TIDAL': {'address': '0x29CbD0510EEc0327992CD6006e63F9Fa8E7f33B7', 'decimals': 18},
    'TLM': {'address': '0x888888848B652B3E3a0f34c96E00EEC0F3a23F72', 'decimals': 4},
    'CPOOL': {'address': '0x66761Fa41377003622aEE3c7675Fc7b5c1C2FaC5', 'decimals': 18},
    'DYDX': {'address': '0x92D6C1e31e14520e676a687F0a93788B716BEff5', 'decimals': 18},
    'SMI': {'address': '0xCd7492db29E2ab436e819b249452EE1bbDf52214', 'decimals': 8},
    'CENT': {'address': '0x08ba718F288c3b12B01146816bef9FA03cC635bc', 'decimals': 18},
    'VR': {'address': '0x7d5121505149065b562C789A0145eD750e6E8cdD', 'decimals': 18},
    'SIS': {'address': '0xd38BB40815d2B0c2d2c866e0c72c5728ffC76dd9', 'decimals': 18},
    'TRIAS': {'address': '0x3A856d4effa670C54585a5D523e96513e148e95d', 'decimals': 18},
    'LUFFY': {'address': '0x54012cDF4119DE84218F7EB90eEB87e25aE6EBd7', 'decimals': 9},
    'ADS': {'address': '0xcfcEcFe2bD2FED07A9145222E8a7ad9Cf1Ccd22A', 'decimals': 11},
    'VOICE': {'address': '0x2e2364966267B5D7D2cE6CD9A9B5bD19d9C7C6A9', 'decimals': 18},
    'UFI': {'address': '0xcDa4e840411C00a614aD9205CAEC807c7458a0E3', 'decimals': 18},
    'FLEX': {'address': '0xb076BDa1ABc154DdB4CCd9Be45542A823aEE290E', 'decimals': 18},
    'NUM': {'address': '0x3496B523e5C00a4b4150D6721320CdDb234c3079', 'decimals': 18},
    'DOGEP': {'address': '0xE7eaec9Bca79d537539C00C58Ae93117fB7280b9', 'decimals': 18},
    'PAW': {'address': '0x1AA51bc7eb181ce48CE626Bf62f8956Fa9555136', 'decimals': 18},
    'SHIRYO-INU': {'address': '0x1E2F15302B90EddE696593607b6bD444B64e8F02', 'decimals': 9},
    'LSS': {'address': '0x3B9BE07d622aCcAEd78f479BC0EDabFd6397E320', 'decimals': 18},
    'CHSB': {'address': '0xba9d4199faB4f26eFE3551D490E3821486f135Ba', 'decimals': 8},
    'WILD': {'address': '0x2a3bFF78B79A009976EeA096a51A948a3dC00e34', 'decimals': 18},
    'INDEX': {'address': '0x0954906da0Bf32d5479e25f46056d22f08464cab', 'decimals': 18},
    'HND': {'address': '0x10010078a54396F62c96dF8532dc2B4847d47ED3', 'decimals': 18},
    'MASQ': {'address': '0x06F3C323f0238c72BF35011071f2b5B7F43A054c', 'decimals': 18},
    'LYXE': {'address': '0xA8b919680258d369114910511cc87595aec0be6D', 'decimals': 18},
    'BTRFLY': {'address': '0xc55126051B22eBb829D00368f4B12Bde432de5Da', 'decimals': 18},
    'FRAX': {'address': '0x853d955aCEf822Db058eb8505911ED77F175b99e', 'decimals': 18},
    'IC': {'address': '0xA876f27F13A9eB6E621202cEFDD5AFc4a90E6457', 'decimals': 9},
    'PLEB': {'address': '0xe19f85c920B572cA48942315B06D6cAc86585c87', 'decimals': 18},
    'GAS': {'address': '0x6Bba316c48b49BD1eAc44573c5c871ff02958469', 'decimals': 18},
    'XYO': {'address': '0x55296f69f40Ea6d20E478533C15A6B08B654E758', 'decimals': 18},
    'SNAIL': {'address': '0x6Bc40d4099f9057b23aF309C08d935b890d7Adc0', 'decimals': 18},
    'BOBC': {'address': '0xe803178b48A0e560C2b19F3b3d4e504f79D229ce', 'decimals': 18},
    'BONE': {'address': '0x9813037ee2218799597d83D4a5B6F3b6778218d9', 'decimals': 18},
    'APED': {'address': '0xE0151763455A8a021e64880C238ba1cff3787fF0', 'decimals': 18},
    'BOR': {'address': '0x3c9d6c1C73b31c837832c72E04D3152f051fc1A9', 'decimals': 18},
    'SOV': {'address': '0xbdab72602e9AD40FC6a6852CAf43258113B8F7a5', 'decimals': 18},
    'GENSLR': {'address': '0xAd1A5B8538a866eCD56dDd328b50ed57cEd5D936', 'decimals': 18},
    'CGG': {'address': '0x1fE24F25b1Cf609B9c4e7E12D802e3640dFA5e43', 'decimals': 18},
    'JUSTICE': {'address': '0x59d1e836F7b7210A978b25a855085cc46fd090B5', 'decimals': 18},
    'LEOX': {'address': '0xa444EC96Ee01bB219A44B285De47bf33C3447ad5', 'decimals': 18},
    'BST': {'address': '0x509A38b7a1cC0dcd83Aa9d06214663D9eC7c7F4a', 'decimals': 18},
    'SHIBU': {'address': '0x7bFDE33d790411A88D46E9e1be32Fc86228891a4', 'decimals': 9},
    'ERN': {'address': '0xBBc2AE13b23d715c30720F079fcd9B4a74093505', 'decimals': 18},
    'BOLT': {'address': '0xD5930C307d7395Ff807F2921F12C5EB82131a789', 'decimals': 18},
    'SAND': {'address': '0x3845badAde8e6dFF049820680d1F14bD3903a5d0', 'decimals': 18},
    'XRT': {'address': '0x7dE91B204C1C737bcEe6F000AAA6569Cf7061cb7', 'decimals': 9},
    'HORD': {'address': '0x43A96962254855F16b925556f9e97BE436A43448', 'decimals': 18},
    'DOJO': {'address': '0x8260328d0C405D9Ca061D80199102DdC9089e43C', 'decimals': 9},
    'WEAPON': {'address': '0x0b8057C3Cc676C329c25C1d0Cd27776efA73762d', 'decimals': 9},
    'XLON': {'address': '0x36ff4DaE0E88113D68B1209e245B0e3Af92E9D58', 'decimals': 6},
    'ALCX': {'address': '0xdBdb4d16EdA451D0503b854CF79D55697F90c8DF', 'decimals': 18},
    'EXRD': {'address': '0x6468e79A80C0eaB0F9A2B574c8d5bC374Af59414', 'decimals': 18},
    'GHOUL': {'address': '0x3bb86d867A9F3adDF994cdaDb210Fa82F0D4157A', 'decimals': 18},
    'BLANK': {'address': '0x41A3Dba3D677E573636BA691a70ff2D606c29666', 'decimals': 18},
    'FXS': {'address': '0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0', 'decimals': 18},
    'BORING': {'address': '0xBC19712FEB3a26080eBf6f2F7849b417FdD792CA', 'decimals': 18},
    'PEPES': {'address': '0x6BF765C43030387a983f429c1438e9D2025B7E12', 'decimals': 18},
    'XPR': {'address': '0xD7EFB00d12C2c13131FD319336Fdf952525dA2af', 'decimals': 4},
    'GMR': {'address': '0x9B8D5f3402F74C7a61d9f09c32D3cA07b45c1466', 'decimals': 18},
    'KERMIT': {'address': '0x53250B5DFA8C911547aFeAf18DB025024C8E919A', 'decimals': 9},
    'JAI': {'address': '0x5f3b3c37F47d8532836c800f75e7f262623Aa3BB', 'decimals': 18},
    'KIBA': {'address': '0x005D1123878Fc55fbd56b54C73963b234a64af3c', 'decimals': 18},
    'PEPEGPT': {'address': '0x636d585F40A7a445dA7403FCf92E03F89dc3eBd0', 'decimals': 18},
    'STG': {'address': '0xAf5191B0De278C7286d6C7CC6ab6BB8A73bA2Cd6', 'decimals': 18},
    'ONX': {'address': '0xE0aD1806Fd3E7edF6FF52Fdb822432e847411033', 'decimals': 18},
    'ZONE': {'address': '0xC1D9B5A0776d7C8B98b8A838e5a0DD1Bc5Fdd53C', 'decimals': 18},
    'RIO': {'address': '0xf21661D0D1d76d3ECb8e1B9F1c923DBfffAe4097', 'decimals': 18},
    'BABYDOGE': {'address': '0xAC8E13ecC30Da7Ff04b842f21A62a1fb0f10eBd5', 'decimals': 9},
    'ZERO': {'address': '0xB0Ed33f79D89541dfdCb04A8f04Bc2c6be025eCC', 'decimals': 18},
    'MPL': {'address': '0x33349B282065b0284d756F0577FB39c158F935e6', 'decimals': 18},
    'RGT': {'address': '0xD291E7a03283640FDc51b121aC401383A46cC623', 'decimals': 18},
    'CHAD': {'address': '0x6B89B97169a797d94F057F4a0B01E2cA303155e4', 'decimals': 18},
    'EURS': {'address': '0xdB25f211AB05b1c97D595516F45794528a807ad8', 'decimals': 2},
    'AURORA': {'address': '0xAaAAAA20D9E0e2461697782ef11675f668207961', 'decimals': 18},
    'PEPA': {'address': '0x577Fee283e776eec29C9E4d258431982780a38A8', 'decimals': 9},
    'PEEPO': {'address': '0xaADA04204e9e1099DaF67cf3D5D137E84E41CF41', 'decimals': 18},
    'SWT': {'address': '0x1f05DeE43dF9fd0Cd7F4b011B20b04870a30c29A', 'decimals': 18},
    'LRC': {'address': '0xBBbbCA6A901c926F240b89EacB641d8Aec7AEafD', 'decimals': 18},
    'SNM': {'address': '0x46d0DAc0926fa16707042CAdC23F1EB4141fe86B', 'decimals': 18},
    'BLU': {'address': '0xe5D2e173B120341face9e9970889C9FE64081FfD', 'decimals': 18},
    'AXS': {'address': '0xBB0E17EF65F82Ab018d8EDd776e8DD940327B28b', 'decimals': 18},
    'GEL': {'address': '0x15b7c0c907e4C6b9AdaAaabC300C08991D6CEA05', 'decimals': 18},
    'LYRA': {'address': '0x01BA67AAC7f75f647D94220Cc98FB30FCc5105Bf', 'decimals': 18},
    'MQL': {'address': '0x428dc22668E6F3468273634067e5545ED5417A3E', 'decimals': 18},
    'ADS': {'address': '0x3106a0a076BeDAE847652F42ef07FD58589E001f', 'decimals': 18},
    'ZZ': {'address': '0xC91a71A1fFA3d8B22ba615BA1B9c01b2BBBf55ad', 'decimals': 18},
    'AURA': {'address': '0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF', 'decimals': 18},
    'BEND': {'address': '0x0d02755a5700414B26FF040e1dE35D337DF56218', 'decimals': 18},
    'DZOO': {'address': '0xc97d6C52F3ADD91Fa1c5287a453d7444aECBca83', 'decimals': 18},
    'VXV': {'address': '0x7D29A64504629172a429e64183D6673b9dAcbFCe', 'decimals': 18},
    'HISAND33': {'address': '0x12AEf5C60C2C86c8ECD3079f22F285f326371340', 'decimals': 18},
    'LQTY': {'address': '0x6DEA81C8171D0bA574754EF6F8b412F2Ed88c54D', 'decimals': 18},
    'MUTE': {'address': '0xA49d7499271aE71cd8aB9Ac515e6694C755d400c', 'decimals': 18},
    'MOVE': {'address': '0x3FA729B4548beCBAd4EaB6EF18413470e6D5324C', 'decimals': 18},
    'GHST': {'address': '0x3F382DbD960E3a9bbCeaE22651E88158d2791550', 'decimals': 18},
    'CADC': {'address': '0xcaDC0acd4B445166f12d2C07EAc6E2544FbE2Eef', 'decimals': 18},
    'LBR': {'address': '0xF1182229B71E79E504b1d2bF076C15a277311e05', 'decimals': 18},
    'BYN': {'address': '0x4Bb3205bf648B7F59EF90Dee0F1B62F6116Bc7ca', 'decimals': 18},
    'CTR': {'address': '0xb3Ad645dB386D7F6D753B2b9C3F4B853DA6890B8', 'decimals': 18},
    'UOS': {'address': '0xD13c7342e1ef687C5ad21b27c2b65D772cAb5C8c', 'decimals': 4},
    'WBTC': {'address': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', 'decimals': 8},
    'AKITA': {'address': '0x3301Ee63Fb29F863f2333Bd4466acb46CD8323E6', 'decimals': 18},
    'IMX': {'address': '0xF57e7e7C23978C3cAEC3C3548E3D615c346e79fF', 'decimals': 18},
    'MONKEYS': {'address': '0x20CDECBF5d56870b4068a255580a58D068446c92', 'decimals': 9},
    'ULX': {'address': '0x5Aa158404fEd6b4730C13F49d3a7F820e14A636F', 'decimals': 18},
    'GUSD': {'address': '0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd', 'decimals': 2},
    'TWOPAW': {'address': '0xf7de6DEf3D319811418d69Bf56c532A815FC47e8', 'decimals': 18},
    'MUMU': {'address': '0x2F573070E6090b3264Fe707e2C9F201716F123c7', 'decimals': 18},
    'FORT': {'address': '0x41545f8b9472D758bB669ed8EaEEEcD7a9C4Ec29', 'decimals': 18},
    'TUSD': {'address': '0x0000000000085d4780B73119b644AE5ecd22b376', 'decimals': 18},
    'GMEE': {'address': '0xD9016A907Dc0ECfA3ca425ab20B6b785B42F2373', 'decimals': 18},
    'USDP': {'address': '0x8E870D67F660D95d5be530380D0eC0bd388289E1', 'decimals': 18},
    'WOO': {'address': '0x4691937a7508860F876c9c0a2a617E7d9E945D4B', 'decimals': 18},
    'NEXO': {'address': '0xB62132e35a6c13ee1EE0f84dC5d40bad8d815206', 'decimals': 18},
    'PEPEDOGE': {'address': '0xB8C55c80a1cb7394088a36C6b634dC2Bf3C6fb67', 'decimals': 18},
    'COMP': {'address': '0xc00e94Cb662C3520282E6f5717214004A7f26888', 'decimals': 18},
    'TROLL': {'address': '0xf8ebf4849F1Fa4FaF0DFF2106A173D3A6CB2eB3A', 'decimals': 18},
    'MKR': {'address': '0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2', 'decimals': 18},
    'NFTY': {'address': '0x3085154623F51b00Dedfc6Ceeb5197277A66B17B', 'decimals': 18},
    'PRO': {'address': '0x226bb599a12C826476e3A771454697EA52E9E220', 'decimals': 8},
    'SPELL': {'address': '0x090185f2135308BaD17527004364eBcC2D37e5F6', 'decimals': 18},
    'MX': {'address': '0x11eeF04c884E24d9B7B4760e7476D06ddF797f36', 'decimals': 18},
    'ATOR': {'address': '0x0F7B3F5a8FeD821c5eb60049538a548dB2D479ce', 'decimals': 18},
    'CAW': {'address': '0xf3b9569F82B18aEf890De263B84189bd33EBe452', 'decimals': 18},
    'BULL': {'address': '0x9Be776559fED779cAbd67042a7B8987AAe592541', 'decimals': 18},
    'DPX': {'address': '0xEec2bE5c91ae7f8a338e1e5f3b5DE49d07AfdC81', 'decimals': 18},
    'FSN': {'address': '0x979aCA85bA37c675e78322ed5d97fa980B9Bdf00', 'decimals': 18},
    'WAS': {'address': '0x0c572544a4Ee47904d54aaA6A970AF96B6f00E1b', 'decimals': 18},
    'AIPAD': {'address': '0xE55d97A97ae6A17706ee281486E98A84095d8AAf', 'decimals': 18},
    'UFO': {'address': '0x249e38Ea4102D0cf8264d3701f1a0E39C4f2DC3B', 'decimals': 18},
    'VAI': {'address': '0xD13cfD3133239a3c73a9E535A5c4DadEE36b395c', 'decimals': 18},
    'DENT': {'address': '0x3597bfD533a99c9aa083587B074434E61Eb0A258', 'decimals': 8},
    'ODOGE': {'address': '0x68b429161Ec09a6c1D65ba70727AB1faa5Bc4026', 'decimals': 18},
    'CHZ': {'address': '0x3506424F91fD33084466F402d5D97f05F8e3b4AF', 'decimals': 18},
    'ATA': {'address': '0xA2120b9e674d3fC3875f415A7DF52e382F141225', 'decimals': 18},
    'PROS': {'address': '0x8642A849D0dcb7a15a974794668ADcfbe4794B56', 'decimals': 18},
    'KOMPETE': {'address': '0x1E0b2992079B620AA13A7c2E7c88D2e1E18E46E9', 'decimals': 10},
    'CAI': {'address': '0xf36C5f04127F7470834ed6f98bDDC1BE62aba48D', 'decimals': 18},
    'COPE': {'address': '0xD8E163967FED76806Df0097B704bA721B9b37656', 'decimals': 18},
    'SWAP': {'address': '0xCC4304A31d09258b0029eA7FE63d032f52e44EFe', 'decimals': 18},
    'ALICE': {'address': '0xAC51066d7bEC65Dc4589368da368b212745d63E8', 'decimals': 6},
    'MASK': {'address': '0x69af81e73A73B40adF4f3d4223Cd9b1ECE623074', 'decimals': 18},
    'REQ': {'address': '0x8f8221aFbB33998d8584A2B05749bA73c37a938a', 'decimals': 18},
    'DIONE': {'address': '0x89B69F2d1adffA9A253d40840B6Baa7fC903D697', 'decimals': 9},
    'SOLID': {'address': '0x777172D858dC1599914a1C4c6c9fC48c99a60990', 'decimals': 18},
    'HEART': {'address': '0x8FAc8031e079F409135766C7d5De29cf22EF897C', 'decimals': 18},
    'MONO': {'address': '0x2920f7d6134f4669343e70122cA9b8f19Ef8fa5D', 'decimals': 18},
    'VRA': {'address': '0xF411903cbC70a74d22900a5DE66A2dda66507255', 'decimals': 18},
    'PORTX': {'address': '0x104F3152D8ebFC3f679392977356962Ff36566aC', 'decimals': 18},
    'DXP': {'address': '0x88aa4a6C5050b9A1b2Aa7e34D0582025cA6AB745', 'decimals': 18},
    'AUDIO': {'address': '0x18aAA7115705e8be94bfFEBDE57Af9BFc265B998', 'decimals': 18},
    'DEGAI': {'address': '0xd068C7C941FbbD2300cB2f1841858c2643722DC7', 'decimals': 18},
    'SWPR': {'address': '0x6cAcDB97e3fC8136805a9E7c342d866ab77D0957', 'decimals': 18},
    'CND': {'address': '0xd4c435F5B09F855C3317c8524Cb1F586E42795fa', 'decimals': 18},
    'MNW': {'address': '0xd3E4Ba569045546D09CF021ECC5dFe42b1d7f6E4', 'decimals': 18},
    'SAITAMA': {'address': '0xCE3f08e664693ca792caCE4af1364D5e220827B2', 'decimals': 9},
    'TRR': {'address': '0x490e3f4af13e1616EC97A8C6600c1061a8D0253e', 'decimals': 18},
    'CGU': {'address': '0x849A226f327b89E3133D9930d927F9EB9346F8C9', 'decimals': 8},
    'SOMM': {'address': '0xa670d7237398238DE01267472C6f13e5B8010FD1', 'decimals': 6},
    'CHB': {'address': '0xf68D4d917592f3a62417aCE42592F15296cc33A0', 'decimals': 8},
    'DEO': {'address': '0x5E7F20e72C21f6D0Bf0A2814FD4164176401cf8e', 'decimals': 18},
    'MINTME': {'address': '0x772722B55cdC2A086aBd064267a17855Eb15e8b3', 'decimals': 18},
    'BAN': {'address': '0xe20B9e246db5a0d21BF9209E4858Bc9A3ff7A034', 'decimals': 18},
    'PIP': {'address': '0x37613D64258c0FE09d5E53EeCb091DA5b8fA8707', 'decimals': 18},
    'OK': {'address': '0xd3Ac016b1B8C80EeAdDe4D186A9138C9324e4189', 'decimals': 18},
    'PRIMAL': {'address': '0xDd13DEdeCEbDA566322195bc4451D672A148752C', 'decimals': 18},
    'TGR': {'address': '0xc7026a20a640Bc71B9074F7aed52A00cD9147091', 'decimals': 18},
    'PIRATE': {'address': '0xb990d93C308A31c737Aa91839E8bA8eAF4017D7A', 'decimals': 8},
    'ASAN': {'address': '0x5F1F11a3dD7a0C39Da1BaA3C7b8585b52a77f435', 'decimals': 18},
    'CP': {'address': '0x5BDC32663Ec75E85ff4abc2CAe7ae8B606A2CFCa', 'decimals': 18},
    'RABBIT': {'address': '0x706f280Cdb92260fe2D50EDA545F834ff1fbFd35', 'decimals': 9},
    'ABI': {'address': '0xa31DFe74cDA04de990CafC6a0aB2BB6d6C9077CD', 'decimals': 9},
    'ACRE': {'address': '0xb2cABf797bc907B049e4cCB5b84d13be3a8CFC21', 'decimals': 18},
    'WFTM': {'address': '0x4E15361FD6b4BB609Fa63C81A2be19d873717870', 'decimals': 18},
    'JM': {'address': '0x388D819724dD6d71760A38F00dc01D310d879771', 'decimals': 8},
    'GMM': {'address': '0x4B19C70Da4c6fA4bAa0660825e889d2F7eaBc279', 'decimals': 18},
    'CRO': {'address': '0xA0b73E1Ff0B80914AB6fe0444E65848C4C34450b', 'decimals': 8},
    'PRNT': {'address': '0x7Cfea0DD176651E7B5a1CeD9c4fAf8Ee295315FD', 'decimals': 18},
    'GRV': {'address': '0xF33893DE6eB6aE9A67442E066aE9aBd228f5290c', 'decimals': 8},
    'ORB': {'address': '0x3C917054e03485808137EB306eAFA8DA0Ab695cd', 'decimals': 18},
    'BNB': {'address': '0xB8c77482e45F1F44dE1745F52C74426C631bDD52', 'decimals': 18},
    'KUJI': {'address': '0x96543ef8d2C75C26387c1a319ae69c0BEE6f3fe7', 'decimals': 6},
    'USK': {'address': '0x1B3C515F58857E141A966b33182f2F3feECC10E9', 'decimals': 6},
    'OATH': {'address': '0x6F9C26Fa731C7Ea4139Fa669962Cf8F1CE6C8b0B', 'decimals': 18},
    'ROVI': {'address': '0xCc503242b574BC01145Da7E2A743b43fB395eC91', 'decimals': 18},
    'VNDC': {'address': '0x1F3F677Ecc58F6A1F9e2CF410dF4776a8546b5DE', 'decimals': 0},
    'RICE': {'address': '0xA64c3a85ddc4cD351Eeb7aeceBc6a44A64A76392', 'decimals': 18},
    'TAROT': {'address': '0x837d904A3799c0769079BE9ecbadDF1aBD4cCD6e', 'decimals': 18},
    'CDT': {'address': '0xCdB37A4fBC2Da5b78aA4E41a432792f9533e85Cc', 'decimals': 18},
    'USX': {'address': '0x0a5E677a6A24b2F1A2Bf4F3bFfC443231d2fDEc8', 'decimals': 18},
    'ONUS': {'address': '0x4184Aa04215e5d716dD4C213FeD519aCADc68f92', 'decimals': 18},
    '1ART': {'address': '0xD3c325848D7c6E29b574Cb0789998b2ff901f17E', 'decimals': 18},
    'FUTURE-AI': {'address': '0x0FF534801E98A4976246D1f418E441783fc9aa15', 'decimals': 18},
    'CATE': {'address': '0xf05897CfE3CE9BBBfE0751CBE6B1B2c686848DCb', 'decimals': 9},
    'NSFW': {'address': '0x8f006D1e1D9dC6C98996F50a4c810F17a47fBF19', 'decimals': 18},
    'WUSDR': {'address': '0xD5a14081a34d256711B02BbEf17E567da48E80b5', 'decimals': 9},
    'EVCOIN': {'address': '0x06fCe0DAB5b26e503E1C4Db48aa165DcaAAa6E11', 'decimals': 9},
    'AIWALLET': {'address': '0xc5A927AD2d1a92Dc5f9662Ce03F7140a8B738B19', 'decimals': 18},
    'MV': {'address': '0xAE788F80F2756A86aa2F410C651F2aF83639B95b', 'decimals': 18},
    'IPISTR': {'address': '0x888888888888f195E27A2E0F98D712952aB9348e', 'decimals': 18},
    'PING': {'address': '0xC7B89491Bb148551547837ea6ccB4Bb5144d8E47', 'decimals': 9},
    'MEAN': {'address': '0x9b25889C493aE6Df34cEef1ecb10D77c1Ba73318', 'decimals': 6},
    'MAGIC': {'address': '0xB0c7a3Ba49C7a6EaBa6cD4a96C55a1391070Ac9A', 'decimals': 18},
    'SHARBI': {'address': '0xF3A3023e6Dede84AD88a656A3269F2A36e83c9a9', 'decimals': 9},
    'MILKAI': {'address': '0x9f826324bb9BdcF7E7Eb274BEDc417BD45D74F39', 'decimals': 8},
    'ZENIQ': {'address': '0x5b52bfB8062Ce664D74bbCd4Cd6DC7Df53Fd7233', 'decimals': 18},
    'HMX': {'address': '0xb012Be90957D70D9a070918027655F998C123a88', 'decimals': 18},
    'SHIBELON': {'address': '0x4c584CD339bdDE73B7F5210486dd8bbeEE3fDe6d', 'decimals': 9},
    'CINU': {'address': '0xA64D5d1eB67748226d84812B45711453f1118c32', 'decimals': 9},
    'JART': {'address': '0xf3E07812eBC8604fdDB0AA35ff79a03F48f48948', 'decimals': 18},
    'DOGS': {'address': '0xBcCD27062ae1A2BeA5731c904b96EDfb163Aba21', 'decimals': 9},
    'EMPIRE': {'address': '0xf124eD9Ec309907808b1FBc6BEdB2a76927B3665', 'decimals': 18},
    'REUNI': {'address': '0x9ed7E4B1BFF939ad473dA5E7a218C771D1569456', 'decimals': 6},
    'SHINU': {'address': '0x0d229c3FF2d76E4b88Ca4F9d3D3353F4545EC53F', 'decimals': 18},
    'OBX': {'address': '0x188fb5f5Ae5BBE4154d5778F2bbb2Fb985c94d25', 'decimals': 18},
    'RJV': {'address': '0x02c3296C6eb50249f290AE596F2bE9454bFfadaB', 'decimals': 6},
    'PLQ': {'address': '0xd23Ed8cA350CE2631F7EcDC5E6bf80D0A1DeBB7B', 'decimals': 18},
    'DIMO': {'address': '0x5fab9761d60419C9eeEbe3915A8FA1ed7e8d2E1B', 'decimals': 18},
    'ATC': {'address': '0x0fAD86Cc1b412b330494E350Bf1dC1994AC55c9e', 'decimals': 6},
    'ID': {'address': '0x2dfF88A56767223A5529eA5960Da7A3F5f766406', 'decimals': 18},
    'MPWR': {'address': '0x6731827Cb6879a2091ce3ab3423f7bf20539b579', 'decimals': 18},
    'ARB': {'address': '0xB50721BCf8d664c30412Cfbc6cf7a15145234ad1', 'decimals': 18},
    'FYT': {'address': '0x77F2BE773CA0887ba2b3Ef8344C8CF13C98d8ca7', 'decimals': 18},
    'AI': {'address': '0xa89Bf95c5f15a847c8Eb8D348cd7feD719AD7D80', 'decimals': 18},
    'STSW': {'address': '0x1C98B54d673C026C8286bADCa3E840aaf72931a3', 'decimals': 6},
    'BONK': {'address': '0x1151CB3d861920e07a38e03eEAd12C32178567F6', 'decimals': 5},
    'GXE': {'address': '0x510975edA48A97E0cA228dD04d1217292487bea6', 'decimals': 18},
    'BRG': {'address': '0xC9C4fd7579133701FA2769b6955e7E56bb386DB1', 'decimals': 18},
    'AGOV': {'address': '0xd1420AF453Fd7bf940573431D416cAcE7fF8280c', 'decimals': 18},
    'SPA': {'address': '0xB4A3B0Faf0Ab53df58001804DdA5Bfc6a3D59008', 'decimals': 18},
    'LFG': {'address': '0x969786c4A8884013d1c9fF18dCCA2aeDbbBfAa8F', 'decimals': 18},
    'PROP': {'address': '0xE9eb76879d332E5c574AA4E8202Ef84956806949', 'decimals': 18},
    'CAKE': {'address': '0x152649eA73beAb28c5b49B26eb48f7EAD6d4c898', 'decimals': 18},
    'DUCKYAI': {'address': '0xD38DE430D355a7a8337204c2A4C80392E61475a6', 'decimals': 18},
    'CREMAT': {'address': '0x8b9F7e9586633cA4ABffb1F746da8dAed7849cB2', 'decimals': 18},
    'PIGE': {'address': '0x995C0e3b9AF4da89FC76c5c0784e97C7A36EEc06', 'decimals': 18},
    'BREWLABS': {'address': '0xdAd33e12e61dC2f2692F2c12e6303B5Ade7277Ba', 'decimals': 9},
    'WBESC': {'address': '0xf89060c99853Bb52eaF5F2247D007d73de660252', 'decimals': 18},
    'NRFX': {'address': '0xCc17e34794B6c160a0F61B58CF30AA6a2a268625', 'decimals': 18},
    'MIND': {'address': '0x87dA7BBd398D6610dED42617E34069D0083ff8aB', 'decimals': 18},
    'ROO': {'address': '0x9d7107c8E30617CAdc11f9692A19C82ae8bbA938', 'decimals': 18},
    'PHX': {'address': '0x9776191F4ebBBa7f358C1663bF82C0a0906c77Fa', 'decimals': 18},
    'FUSDC': {'address': '0x9d1089802eE608BA84C5c98211afE5f37F96B36C', 'decimals': 6},
    'TSA': {'address': '0x703D8574E19428D662a73c301D106dc236aa9BFC', 'decimals': 18},
    'FLAG': {'address': '0x9348E94A447bF8B2ec11f374D3F055FD47d936Df', 'decimals': 18},
    'HNY': {'address': '0xc3589F56B6869824804A5EA29F2c9886Af1B0FcE', 'decimals': 18},
    'NEXT': {'address': '0x377d552914E7A104bC22B4F3B6268dDC69615Be7', 'decimals': 18},
    'NEX': {'address': '0xE2dc070524A6e305ddB64d8513DC444B6a1ec845', 'decimals': 8},
    'ANC': {'address': '0x0F3ADC247E91c3c50bC08721355A41037E89Bc20', 'decimals': 18},
    'KRW': {'address': '0x499568c250Ab2a42292261d6121525d70691894b', 'decimals': 18},
    'TRIPS': {'address': '0x1350Fbe8Ce27762ec19134bf8FC405a427fe9Bf1', 'decimals': 18},
    '8PAY': {'address': '0xFeea0bDd3D07eb6FE305938878C0caDBFa169042', 'decimals': 18},
    'WCHI': {'address': '0x6DC02164d75651758aC74435806093E421b64605', 'decimals': 8},
    'MBX': {'address': '0x71ba91dC68C6a206Db0A6A92B4b1De3f9271432d', 'decimals': 18},
    'DPET': {'address': '0xfb62AE373acA027177D1c18Ee0862817f9080d08', 'decimals': 18},
    'TRDG': {'address': '0x92a42Db88Ed0F02c71D439e55962Ca7CAB0168b5', 'decimals': 9},
    'HAPPY': {'address': '0x3079F61704E9eFa2BcF1db412f735d8d4cFa26f4', 'decimals': 18},
    'ORAO': {'address': '0xEE1CeA7665bA7aa97e982EdeaeCb26B59a04d035', 'decimals': 18},
    'HYDRO': {'address': '0x946112efaB61C3636CBD52DE2E1392D7A75A6f01', 'decimals': 18},
    'DC': {'address': '0x7B4328c127B85369D9f82ca0503B000D09CF9180', 'decimals': 18},
    'HAI': {'address': '0x05Fb86775Fd5c16290f1E838F5caaa7342bD9a63', 'decimals': 8},
    'HNS': {'address': '0xA771b49064Da011DF051052848477f18DbA1d2Ac', 'decimals': 6},
    'DMOD': {'address': '0x5f6c5C2fB289dB2228d159C69621215e354218d7', 'decimals': 18},
    'SHIELD': {'address': '0xc944273b805DeBd35c63011943ABc5aB9eDdb8E3', 'decimals': 18},
    'REALT-S-12405-SANTA': {'address': '0x9D918eE39a356bE8eF99734599c7e70160dB4Db6', 'decimals': 18},
    'REALT-S-1617-SAVERS': {'address': '0xF4657AB08681214Bcb1893aa8E9C7613459250ec', 'decimals': 18},
    'REALT-S-4852-4854-W': {'address': '0xE7B6dE709fFC3BD237c2f2c800E1002F97A760F3', 'decimals': 18},
    'REALT-S-14066-SANTA': {'address': '0xe887DC4fcb5240c0c080AEAb8870421d3ebd0B28', 'decimals': 18},
    'USDS': {'address': '0x45fDb1b92a649fb6A64Ef1511D3Ba5Bf60044838', 'decimals': 18},
    'BLKC': {'address': '0x8BCd06492416a749C9369009b3429861b7F27F6E', 'decimals': 8},
    'OOE': {'address': '0x7778360F035C589fCE2f4eA5786CbD8B36e5396B', 'decimals': 18},
    'BSCGIRL': {'address': '0x4E9A46EA6A22f3894aBEE2302Ad42fd3b69E21E2', 'decimals': 8},
    'C98': {'address': '0xAE12C5930881c53715B369ceC7606B70d8EB229f', 'decimals': 18},
    'BUY': {'address': '0x396eC402B42066864C406d1ac3bc86B575003ed8', 'decimals': 2},
    'DOJO': {'address': '0x180dAe91D6d56235453a892d2e56a3E40Ba81DF8', 'decimals': 18},
    'SWINGBY': {'address': '0x8287C7b963b405b7B8D467DB9d79eEC40625b13A', 'decimals': 18},
    'START': {'address': '0x1d7Ca62F6Af49ec66f6680b8606E634E55Ef22C1', 'decimals': 18},
    'ANGEL': {'address': '0x6C7B97c7e09E790D161769a52F155125FAc6d5A1', 'decimals': 18},
    'GBYTE': {'address': '0x31F69dE127C8A0fF10819C0955490a4Ae46fcc2a', 'decimals': 18},
    'CRFI': {'address': '0x8848812BD31AEEe33313C10A840FfC3169078C5b', 'decimals': 18},
    'AGS': {'address': '0x667Fd83E24Ca1D935d36717D305D54fA0CAC991C', 'decimals': 18},
    'DEC': {'address': '0x9393fdc77090F31c7db989390D43F454B1A6E7F3', 'decimals': 3},
    'BYTZ': {'address': '0x2aaD9Dbc82611485a52325923e1187734e951B78', 'decimals': 8},
    'CYC': {'address': '0x8861cfF2366C1128fd699B68304aD99a0764Ef9a', 'decimals': 18},
    'MEDIA': {'address': '0xdb726152680eCe3c9291f1016f1d36f3995f6941', 'decimals': 6},
    'COW': {'address': '0x34965f73Cfa05Bf8d8aF37cB4af64Fa950605Ea8', 'decimals': 18},
    'MOCA': {'address': '0x9Ac07635DDBDE5db18648c360DEFb00F5f22537e', 'decimals': 18},
    'FLUX': {'address': '0x7645DdfEecedA57e41f92679c4aCd83c56A81D14', 'decimals': 18},
    'ANY': {'address': '0xf99d58e463A2E07e5692127302C20A191861b4D6', 'decimals': 18},
    'GAJ': {'address': '0x9Fda7cEeC4c18008096C2fE2B85F05dc300F94d0', 'decimals': 18},
    'CSWAP': {'address': '0xe0b0C16038845BEd3fCf70304D3e167Df81ce225', 'decimals': 18},
    'DAPP': {'address': '0x939B462ee3311f8926c047D2B576C389092b1649', 'decimals': 4},
    'FIS': {'address': '0xef3A930e1FfFFAcd2fc13434aC81bD278B0ecC8d', 'decimals': 18},
    'RAID': {'address': '0x154e35c2b0024B3e079c5c5e4fC31c979c189cCB', 'decimals': 18},
    'NSBT': {'address': '0x9D79d5B61De59D882ce90125b18F74af650acB93', 'decimals': 6},
    'ADAPAD': {'address': '0xDB0170e2d0c1CC1b2E7a90313d9B9afA4f250289', 'decimals': 18},
    'CAP': {'address': '0x43044f861ec040DB59A7e324c40507adDb673142', 'decimals': 18},
    'EGG': {'address': '0xC2708A3A4ba7F64bddc1A49F92f941Bc77Cad23a', 'decimals': 18},
    'APEIN': {'address': '0x8bbf1DcCBEdD5c70d8E793d432fB56b848DD1698', 'decimals': 18},
    'VLXPAD': {'address': '0xb8e3bB633F7276cc17735D86154E0ad5ec9928C0', 'decimals': 18},
    'TBC': {'address': '0x9798dF2f5d213a872c787bD03b2b91F54D0D04A1', 'decimals': 18},
    'WOW': {'address': '0x3405A1bd46B85c5C029483FbECf2F3E611026e45', 'decimals': 18},
    'RACA': {'address': '0x12BB890508c125661E03b09EC06E404bc9289040', 'decimals': 18},
    'WJXN': {'address': '0xcA1262e77Fb25c0a4112CFc9bad3ff54F617f2e6', 'decimals': 0},
    'NWC': {'address': '0x968F6f898a6Df937fC1859b323aC2F14643e3fED', 'decimals': 18},
    'ZPTC': {'address': '0x39Ae6D231d831756079ec23589D2D37A739F2E89', 'decimals': 4},
    'OKLG': {'address': '0x5dBB9F64cd96E2DbBcA58d14863d615B67B42f2e', 'decimals': 9},
    'ZEDXION': {'address': '0xFbC4f3f645C4003a2E4F4e9b51077d2DaA9a9341', 'decimals': 18},
    'ZANO': {'address': '0xdb85f6685950E285b1E611037BEBe5B34e2B7d78', 'decimals': 18},
    'DEFIDO': {'address': '0xf32122561d51E891B823Dec2B42F644884c1Cd91', 'decimals': 9},
    'PGEN': {'address': '0xf6719e1A8fcBB1B9c290019E37e004966A8916C9', 'decimals': 18},
    'CRB': {'address': '0x2fd8BC03D9e827F77faC20B5130eE98B7F80149d', 'decimals': 8},
    'OH': {'address': '0x16ba8Efe847EBDFef99d399902ec29397D403C30', 'decimals': 18},
    'ZAM': {'address': '0xd373576a9e738F37Dc6882328358Ff69c4cAf4c6', 'decimals': 18},
    'ECOP': {'address': '0x0106A1122FE94a9CF151097c1Fe17229Ec78FFaD', 'decimals': 18},
    'SEFI': {'address': '0x773258b03c730F84aF10dFcB1BfAa7487558B8Ac', 'decimals': 6},
    'FLASH': {'address': '0x5f0366c9962193fA774cdce9602195593B49f23C', 'decimals': 18},
    'PIXEL': {'address': '0x65E6B60Ea01668634D68D0513Fe814679F925BaD', 'decimals': 18},
    'DAR': {'address': '0x081131434f93063751813C619Ecca9C4dC7862a3', 'decimals': 6},
    'WHIRL': {'address': '0x0C75dD36aF9a59BA1d248a98Fe91b2384cfea9be', 'decimals': 18},
    'BABYDOGE': {'address': '0xAC57De9C1A09FeC648E93EB98875B212DB0d460B', 'decimals': 9},
    'BOO': {'address': '0x55aF5865807b196bD0197e0902746F31FBcCFa58', 'decimals': 18},
    'EYE': {'address': '0x9A257C90Fa239fBA07771ef7da2d554D148c2E89', 'decimals': 18},
    'FEVR': {'address': '0x9fB83c0635De2E815fd1c21b3a292277540C2e8d', 'decimals': 18},
    'ZENITH': {'address': '0x0343131C0257aC21Ea5A8dc83841f071eFd9285C', 'decimals': 18},
    'DOGIRA': {'address': '0xD8C1232FcD219286E341271385bd70601503B3D7', 'decimals': 9},
    'BABYFD': {'address': '0x747C4Ce9622EA750EA8048423B38A746B096C8e8', 'decimals': 9},
    'FCP': {'address': '0xb6Dd77FD132dcAa10F1858734e838a0FA7431580', 'decimals': 18},
    'MARVIN': {'address': '0x81C159f7aBAa9139227Aff62959b86B4141F6EB2', 'decimals': 9},
    'BTL': {'address': '0x93e32efaFd24973d45f363A76D73ccB9Edf59986', 'decimals': 6},
    'COPI': {'address': '0x42Baf1f659D765C65ADE5BB7E08eb2C680360d9d', 'decimals': 18},
    'WSB': {'address': '0xe1590A6FA0CFf9C960181cb77d8a873601772f64', 'decimals': 18},
    'PKN': {'address': '0xdf09a216Fac5ADC3e640Db418C0b956076509503', 'decimals': 18},
    'DOMI': {'address': '0x45C2F8c9B4c0bDC76200448cc26C48ab6ffef83F', 'decimals': 18},
    'LUNR': {'address': '0xA87135285Ae208e22068AcDBFf64B11Ec73EAa5A', 'decimals': 4},
    'SFIL': {'address': '0x965b85D4674F64422c4898C8F8083187f02B32C0', 'decimals': 8},
    'USTC': {'address': '0xa47c8bf37f92aBed4A126BDA807A7b7498661acD', 'decimals': 18},
    'SNAP': {'address': '0x57cd5A91c18D21DeDd72d3bcf255b60Aef290f53', 'decimals': 18},
    'EAGON': {'address': '0x2F25d402829cA4085B8Ea4D3BC68Bf203F5a9faB', 'decimals': 18},
    'WOOP': {'address': '0xaAD483F97f13C6A20B9D05d07c397CE85c42C393', 'decimals': 18},
    'ACY': {'address': '0xAF9dB9E362e306688AF48c4ACB9618C06db38Ac3', 'decimals': 18},
    'IDYP': {'address': '0xBD100d061E120b2c67A24453CF6368E63f1Be056', 'decimals': 18},
    'VST': {'address': '0x77A1f4E744d810239F465043E35d067Ca33De259', 'decimals': 18},
    'BTCBR': {'address': '0xE57425F1598f9b0d6219706b77f4b3DA573a3695', 'decimals': 18},
    'SOURCE': {'address': '0x7118057ff0F4Fd0994fb9d2D94de8231d5cca79E', 'decimals': 18},
    'WWY': {'address': '0x9Ab70e92319f0b9127df78868Fd3655fB9f1E322', 'decimals': 18},
    'HUH': {'address': '0x86D49fbD3B6f989d641E700a15599d3b165002AB', 'decimals': 9},
    'AXL': {'address': '0x25b24B3c47918b7962B3e49C4F468367F73CC0E0', 'decimals': 18},
    'LEPA': {'address': '0xbBa6C7c7d673c48d90069ad2e9d2fE587fcB6bc3', 'decimals': 18},
    'UST': {'address': '0xa693B19d2931d498c5B318dF961919BB4aee87a5', 'decimals': 6},
    'SATOZ': {'address': '0x845e2e8B42DCED7DEDCDBa9BdE32C9e338224f97', 'decimals': 8},
    'EKTA': {'address': '0x2f75113b13D136F861d212Fa9b572F2C79Ac81C4', 'decimals': 18},
    'BTT': {'address': '0xC669928185DbCE49d2230CC9B0979BE6DC797957', 'decimals': 18},
    'SLP': {'address': '0xCC8Fa225D80b9c7D42F96e9570156c65D6cAAa25', 'decimals': 0},
    'COC': {'address': '0xc4BB7277A74678f053259CB1F96140347efbfd46', 'decimals': 18},
    'GEM': {'address': '0x9b17bAADf0f21F03e35249e0e59723F34994F806', 'decimals': 18},
    'O3': {'address': '0xEe9801669C6138E84bD50dEB500827b776777d28', 'decimals': 18},
    'WTLOS': {'address': '0x7825e833D495F3d1c28872415a4aee339D26AC88', 'decimals': 18},
    'QMALL': {'address': '0x2217e5921B7edfB4BB193a6228459974010D2198', 'decimals': 18},
    'MCONTENT': {'address': '0xD3c51de3E6DD9b53D7f37699AFB3EE3Bf9B9b3F4', 'decimals': 6},
    'VCG': {'address': '0x1F36FB2D91d9951Cf58aE4c1956C0b77e224F1E9', 'decimals': 18},
    'PAY': {'address': '0xe580074A10360404AF3ABfe2d524D5806D993ea3', 'decimals': 18},
    'PHL': {'address': '0x723bD1f87A327e94ceaBf68d8f022E0f54B9cC1a', 'decimals': 18},
    'POL': {'address': '0xc17fbe1d709dDf6C0B6665Dd0591046815AC7554', 'decimals': 18},
    'WAL': {'address': '0x22c5543d1a35178cB03b33f929A959145E538532', 'decimals': 18},
    'PLY': {'address': '0x20d60c6eb195868d4643f2c9B0809e4De6Cc003d', 'decimals': 6},
    'MTD': {'address': '0x809E130E10e787139c54e1d12d3D1971b7A675Bf', 'decimals': 18},
    'PLY': {'address': '0x1aB43204a195a0fD37EdeC621482AFd3792ef90b', 'decimals': 18},
    'NFTI': {'address': '0x525eF76138Bf76118d786DbedeaE5F87aaBf4a81', 'decimals': 18},
    'IBEX': {'address': '0xF655C8567E0f213e6C634CD2A68d992152161dC6', 'decimals': 18},
    'NNT': {'address': '0x19193F450086d0442157b852081976D41657aD56', 'decimals': 18},
    'RISE': {'address': '0xC17c30e98541188614dF99239cABD40280810cA3', 'decimals': 18},
    'DHN': {'address': '0x32462bA310E447eF34FF0D15BCE8613aa8C4A244', 'decimals': 18},
    'NFT': {'address': '0x198d14F2Ad9CE69E76ea330B374DE4957C3F850a', 'decimals': 6},
    'SMBR': {'address': '0x16B3E050e9e2f0Ac4f1BEA1b3E4fdc43d7f062Dd', 'decimals': 9},
    'MELON': {'address': '0xb14784b2a56945AED7b8CD41661D68F8b6CCeC8b', 'decimals': 18},
    'NBT': {'address': '0x446F2A8A39Cc730ef378Be759A3c57f1a3fE824c', 'decimals': 18},
    'ECC': {'address': '0xC84D8d03aA41EF941721A4D77b24bB44D7C7Ac55', 'decimals': 9},
    'FAME': {'address': '0x818f49467021BDaadAE69E071E79AD2fD7226a1E', 'decimals': 18},
    'JMPT': {'address': '0x420a24c9c65bd44c48bfB1cC8D6cD1ea8B1Ac840', 'decimals': 18},
    'KZEN': {'address': '0x4550003152F12014558e5CE025707E4DD841100F', 'decimals': 18},
    'USDD': {'address': '0x0C10bF8FcB7Bf5412187A595ab97a3609160b5c6', 'decimals': 18},
    'PS1': {'address': '0xb5be7557fe8f69a2B5707D25fA0aeE80DfDA512E', 'decimals': 18},
    'MAIN': {'address': '0xfcEB206E1a80527908521121358B5e26cAABaa75', 'decimals': 18},
    'BLAST': {'address': '0x614D7f40701132E25fe6fc17801Fbd34212d2Eda', 'decimals': 9},
    'DFI': {'address': '0x8Fc8f8269ebca376D046Ce292dC7eaC40c8D358A', 'decimals': 8},
    'CELT': {'address': '0xf6e06B54855eFf198a2d9A8686113665499a6134', 'decimals': 18},
    'A4': {'address': '0x9767203e89dcD34851240B3919d4900d3E5069f1', 'decimals': 6},
    'QUA': {'address': '0xd35c06a2781f648c75290976ecF71E71582188B7', 'decimals': 18},
    'RADIO': {'address': '0x7a5d3A9Dcd33cb8D527f7b5F96EB4Fef43d55636', 'decimals': 18},
    'ANKRMATIC': {'address': '0x26dcFbFa8Bc267b250432c01C982Eaf81cC5480C', 'decimals': 18},
    'VINU': {'address': '0xAFCdd4f666c84Fed1d8BD825aA762e3714F652c9', 'decimals': 18},
    'ISHND': {'address': '0x9e6b19874e97fe8E8CAD77f2c0AB5E7a693E5dbf', 'decimals': 18},
    'ACRV': {'address': '0x2b95A1Dcc3D405535f9ed33c219ab38E8d7e0884', 'decimals': 18},
    'SHACK': {'address': '0x5f018e73C185aB23647c82bD039e762813877f0e', 'decimals': 18},
    'SHA': {'address': '0x40fEd5691e547885cABd7A2990De719DCc8497FC', 'decimals': 18},
    'SPICE': {'address': '0x18a1Ea69A50A85752b7bC204a2c45A95Ce6E429d', 'decimals': 18},
    'BIP': {'address': '0xcafE34BAE6F1b23A6B575303EdCc0578d2188131', 'decimals': 18},
    'OPENX': {'address': '0x57d579F483854c62FEf850B8a5332B0d8424b7E2', 'decimals': 18},
    'SLEEPEE': {'address': '0x80CD73baDb406eA36B9a7CDeb8df06AeFa7E12d9', 'decimals': 18},
    'RMRK': {'address': '0x471Ea49dd8E60E697f4cac262b5fafCc307506e4', 'decimals': 10},
    'EVMOS': {'address': '0x93581991f68DBaE1eA105233b67f7FA0D6BDeE7b', 'decimals': 18},
    'SOKU': {'address': '0x4C3A8ECeB656Ec63eaE80a4ebD565E4887DB6160', 'decimals': 18},
    'USV': {'address': '0x88536C9B2C4701b8dB824e6A16829D5B5Eb84440', 'decimals': 9},
    'ELK': {'address': '0xeEeEEb57642040bE42185f49C52F7E9B38f8eeeE', 'decimals': 18},
    'CERES': {'address': '0x2e7B0d4F9B2EaF782eD3D160e3a0a4b1a7930aDA', 'decimals': 18},
    'XERM': {'address': '0x22Ad96aCF613539d7900b5Ec6e0E65dBCC668d2A', 'decimals': 7},
    'BANANA': {'address': '0x92DF60c51C710a1b1C20E42D85e221f3A1bFc7f2', 'decimals': 18},
    'OSEA': {'address': '0xC72633F995e98Ac3BB8a89e6a9C4Af335C3D6E44', 'decimals': 18},
    'LBLOCK': {'address': '0x2b867efD2dE4Ad2B583Ca0CB3dF9C4040Ef4D329', 'decimals': 9},
    'GMT': {'address': '0xe3c408BD53c31C085a1746AF401A4042954ff740', 'decimals': 8},
    'GM': {'address': '0x35609dC59E15d03c5c865507e1348FA5abB319A8', 'decimals': 8},
    'SAUDISHIB': {'address': '0x34d31446a522252270b89b09016296ec4c98e23d', 'decimals': 8},
    'FLY': {'address': '0x4e568ab95F029E8dF1e39b30c9d6D076eaA15945', 'decimals': 18},
    'MNTL': {'address': '0x2C4F1DF9c7DE0C59778936C9b145fF56813F3295', 'decimals': 6},
    'BUILD': {'address': '0x57b59f981730c6257dF57cF6F0D98283749A9Eeb', 'decimals': 18},
    'MCRT': {'address': '0xde16Ce60804a881e9F8c4eBB3824646EDecd478D', 'decimals': 9},
    'VVS': {'address': '0x839e71613f9aA06E5701CF6de63E303616B0DDE3', 'decimals': 18},
    'FXUSD': {'address': '0x8616E8EA83f048ab9A5eC513c9412Dd2993bcE3F', 'decimals': 18},
    'OCTSMM': {'address': '0xbFc43a35b3aE8F7463c5Ac88a0C46107cFCe6f67', 'decimals': 18},
    'WALV': {'address': '0x256D1fCE1b1221e8398f65F9B36033CE50B2D497', 'decimals': 18},
    'LFI': {'address': '0xb28f803A8772E6584A65Ab6dfC535aE6fEf8a0b2', 'decimals': 18},
    'MIDA': {'address': '0x6CD13E1856286dE773a435C8Ba93A7C73A04F82d', 'decimals': 18},
    'CUSD': {'address': '0xC285B7E09A4584D027E5BC36571785B515898246', 'decimals': 18},
    'FER': {'address': '0x2f32b39023dA7d6A6486A85d12B346EB9C2A0D19', 'decimals': 18},
    'FUNEX': {'address': '0xD6e460F70e1CF60E55B770f66E568B44bf3657d0', 'decimals': 18},
    'MIMATIC': {'address': '0x8D6CeBD76f18E1558D4DB88138e2DeFB3909fAD6', 'decimals': 18},
    'MONDAY': {'address': '0x7252d57Ae4Ec9aF61DB3B82A6eeDAB74EEC5D1dc', 'decimals': 18},
    'ASTRAFER': {'address': '0x97Bbbc5d96875fB78D2F14b7FF8d7a3a74106F17', 'decimals': 18},
    'BOB': {'address': '0xB0B195aEFA3650A6908f15CdaC7D92F8a5791B0B', 'decimals': 18},
    'SEAN': {'address': '0xA719CB79Af39A9C10eDA2755E0938bCE35e9DE24', 'decimals': 18},
    'WMEMO': {'address': '0x3b79a28264fC52c7b4CEA90558AA0B162f7Faf57', 'decimals': 18},
    'TXAU': {'address': '0xe4A6f23Fb9e00Fca037Aa0EA0a6954dE0a6C53bF', 'decimals': 18},
    'TXAG': {'address': '0x34ABce75D2f8f33940c721dCA0f562617787bfF3', 'decimals': 18},
    'XETA': {'address': '0x967Fb0D760ED3ce53AfE2f0A071674cccAe73550', 'decimals': 18},
    'RISU': {'address': '0xcb3c5438Dae9Fe30B18eA53dA3Dab0E7dcaA0E4b', 'decimals': 9},
    'CROWN': {'address': '0x444d6088B0F625f8C20192623B3C43001135E0fa', 'decimals': 18},

}

liquid_tkn_addresses_bancor_3 = [
    '0x514910771AF9Ca656af840dff83E8264EcF986CA',
    '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
    '0x903bEF1736CDdf2A537176cf3C64579C3867A881',
    '0x6B175474E89094C44Da98b954EedeAC495271d0F',
    '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
    '0xdAC17F958D2ee523a2206206994597C13D831ec7',
    '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
    '0x48Fb253446873234F2fEBbF9BdeAA72d9d387f94',
    '0x4691937a7508860F876c9c0a2a617E7d9E945D4B',
    '0xF629cBd94d3791C9250152BD8dfBDF380E2a3B9c',
    '0x444d6088B0F625f8C20192623B3C43001135E0fa',
    '0x6c6EE5e31d828De241282B9606C8e98Ea48526E2',
    '0x408e41876cCCDC0F92210600ef50372656052a38',
    '0x0D8775F648430679A709E98d2b0Cb6250d2887EF',
    '0x6710c63432A2De02954fc0f851db07146a6c0312',
    '0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0',
    '0xBC19712FEB3a26080eBf6f2F7849b417FdD792CA',
    '0x4a220E6096B25EADb88358cb44068A3248254675',
    '0x0F5D2fB29fb7d3CFeE444a200298f468908cC942',
    '0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2',
    '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9',
    '0x275f5Ad03be0Fa221B4C6649B8AeE09a42D9412A',
    '0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F',
    '0x0d438F3b5175Bebc262bF23753C1E53d03432bDE',
    '0x99295f1141d58A99e939F7bE6BBe734916a875B8',
    '0x0f51bb10119727a7e5eA3538074fb341F56B09Ad',
    '0xD33526068D116cE69F19A9ee46F0bd304F21A51f',
    '0xC18360217D8F7Ab5e7c516566761Ea12Ce7F9D72',
    '0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
    '0xb9EF770B6A5e12E45983C5D80545258aA38F3B78',
    '0x939B462ee3311f8926c047D2B576C389092b1649',
    '0xb2cABf797bc907B049e4cCB5b84d13be3a8CFC21',
    '0x967da4048cD07aB37855c090aAF366e4ce1b9F48',
    '0xaA7a9CA87d3694B5755f213B5D04094b8d0F0A6F',
    '0xDe30da39c46104798bB5aA3fe8B9e0e1F348163F',
    '0x6f40d4A6237C257fff2dB00FA0510DeEECd303eb',
    '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0',
    '0xFe459828c90c0BA4bC8b42F5C5D44F316700B430',
    '0x1559FA1b8F28238FD5D76D9f434ad86FD20D1559',
    '0x0f71B8De197A1C84d31de0F1fA7926c365F052B3',
    '0x3506424F91fD33084466F402d5D97f05F8e3b4AF',
    '0xD13c7342e1ef687C5ad21b27c2b65D772cAb5C8c',
    '0x0954906da0Bf32d5479e25f46056d22f08464cab',
    '0xFca59Cd816aB1eaD66534D82bc21E7515cE441CF',
    '0xfF20817765cB7f73d4bde2e66e067E58D11095C2',
    '0x607F4C5BB672230e8672085532f7e901544a7375',
    '0x2A039B1D9bBDccBb91be28691b730ca893e5e743',
    '0x8290333ceF9e6D528dD5618Fb97a76f268f3EDD4',
    '0xA36FDBBAE3c9d55a1d67EE5821d53B50B63A1aB9',
    '0x505a84a03e382331A1Be487b632Cf357748b65d6',
    '0xBBbbCA6A901c926F240b89EacB641d8Aec7AEafD',
    '0x3ebb4A4e91Ad83BE51F8d596533818b246F4bEe1',
    '0x8798249c2E607446EfB7Ad49eC89dD1865Ff4272',
    '0xc719d010B63E5bbF2C0551872CD5316ED26AcD83',
    '0xc944E90C64B2c07662A292be6244BDf05Cda44a7',
    '0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e',
    '0x232FB065D9d24c34708eeDbF03724f2e95ABE768',
    '0x1776e1F26f98b1A5dF9cD347953a26dd3Cb46671'
]


def get_token_decimals_from_address(token_address) -> int:
    return token_address_lookup[token_address]["decimals"]


def get_token_symbol_from_address(token_address) -> str:
    if check_if_tkn_in_table_address(token_address):
        return token_address_lookup[token_address]["symbol"] 
    else: 
        return token_address
     


def check_if_tkn_in_table_address(token_address) -> bool:
    return token_address in token_address_lookup


def get_token_decimals_from_symbol(token_symbol) -> int:
    return token_symbol_lookup[token_symbol]["decimals"]


def get_token_address_from_symbol(token_symbol) -> str:
    return token_symbol_lookup[token_symbol]["address"]


def check_if_tkn_in_table_symbol(token_address) -> bool:
    return token_address in token_symbol_lookup

pair_pools = {
    "0x67CeA36eEB36Ace126A3Ca6E21405258130CF33C": {"pair": "USDC_TSUKA",
            "pair_reverse": "TSUKA_USDC",
            "exchange_name": "uniswap",
            "exchange": "uniswap",
            "address": "0x67CeA36eEB36Ace126A3Ca6E21405258130CF33C",
            "address0": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48",
            "address1": "0xc5fB36dd2fb59d3B98dEfF88425a3F425Ee469eD",
            "symbol0": "USDC",
            "symbol1": "TSUKA",
            "fee": 0.003,
            "decimal0": 6,
            "decimal1": 9,
            "exchange_version": "v2",
            "exchange_id": 3,
            "exchange_type": "Constant Product Pool"},

    "0xEb7b5294c79b0369315Ed7fE76b52d3108B0A62D": {
            "pair": "USDC_JAY",
            "pair_reverse": "JAY_USDC",
            "exchange_name": "uniswap",
            "exchange": "uniswap",
            "address": "0x67CeA36eEB36Ace126A3Ca6E21405258130CF33C",
            "address0": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48",
            "address1": "0xDA7C0810cE6F8329786160bb3d1734cf6661CA6E",
            "symbol0": "USDC",
            "symbol1": "JAY",
            "fee": 0.003,
            "decimal0": 6,
            "decimal1": 18,
            "exchange_version": "v2",
            "exchange_id": 3,
            "exchange_type": "Constant Product Pool"},

}



In [91]:
def process_raw_carbon_strategies(strategies, block_updated: int
):
    """
    Takes a list of Carbon Strategies in the raw contract format, processes them, and inserts them into the database.

    Parameters
    ----------
    strategies : List[Any]
        A list of raw Carbon strategies
    block_updated : int, optional
        The last block that was updated, by default None

    """

    carbon_orders = []

    for strategy in strategies:
        # try:
        strategy_id = str(strategy[0])
        tkn0_address, tkn1_address = strategy[2][0], strategy[2][1]
        order0, order1 = strategy[3][0], strategy[3][1]
        y_0, z_0, A_0, B_0 = (
            order0[0],
            order0[1],
            order0[2],
            order0[3],
        )
        y_1, z_1, A_1, B_1 = (
            order1[0],
            order1[1],
            order1[2],
            order1[3],
        )
        tkn0_address = C.w3.toChecksumAddress(tkn0_address)
        tkn1_address = C.w3.toChecksumAddress(tkn1_address)

        if not check_if_tkn_in_table_address(tkn0_address):
            C.logger.info(f"Token not found in database: {tkn0_address}")
            continue
        if not check_if_tkn_in_table_address(tkn1_address):
            C.logger.info(f"Token not found in database: {tkn1_address}")
            continue
        tkn0_decimals = get_token_decimals_from_address(tkn0_address)

        tkn1_decimals = get_token_decimals_from_address(tkn1_address)
        token0_symbol = get_token_symbol_from_address(tkn0_address)
        token1_symbol = get_token_symbol_from_address(tkn1_address)

        tkn0 = ERC20Token(
            address=tkn0_address, symbol=token0_symbol, decimals=tkn0_decimals
        )
        tkn1 = ERC20Token(
            address=tkn1_address, symbol=token1_symbol, decimals=tkn1_decimals
        )

        if y_0 > 0 and B_0 > 0:
            order0 = EncodedOrder(
                y=y_0, z=z_0, A=A_0, B=B_0, token_in=tkn1, token_out=tkn0
            )
            order0 = CarbonV1Order(
                tkn_in=tkn1,
                tkn_out=tkn0,
                encoded_order=order0,
                order_id=strategy_id,
                block_updated=block_updated,
            )
            carbon_orders.append(order0)
            #print(f"Carbon order0: tknin:{tkn1.symbol}, tknout{tkn0.symbol}, amt{(y_0) / 10 ** tkn0.decimals}")

        if y_1 > 0 and B_1 > 0:
            order1 = EncodedOrder(
                y=y_1, z=z_1, A=A_1, B=B_1, token_in=tkn0, token_out=tkn1
            )
            order1 = CarbonV1Order(
                tkn_in=tkn0,
                tkn_out=tkn1,
                encoded_order=order1,
                order_id=strategy_id,
                block_updated=block_updated,
            )
            carbon_orders.append(order1)
            #print(f"Carbon order1: tknin:{tkn0.symbol}, tknout{tkn1.symbol}, amt{(y_1) / 10 ** tkn1.decimals}")

        # except Exception as e:
        #     logger.error(f"Error updating Carbon strategy {strategy} [{e}]")
        #     continue
    return carbon_orders

In [92]:
import brownie

def get_strategies_by_pair(
        token0: str, token1: str, start_idx: int, end_idx: int
):

    return controller.strategiesByPair(
        token0, token1, start_idx, end_idx
    )

def get_strategies_count_by_pair(token0: str, token1: str) -> int:
    return controller.strategiesByPairCount(
        token0, token1
    )

def get_strategies(pair):
    num_strategies = get_strategies_count_by_pair(pair[0], pair[1])
    return get_strategies_by_pair(pair[0], pair[1], 0, num_strategies)

def get_all_carbon_strategies(carbon_pairs):

    with brownie.multicall(address="0x5BA1e12693Dc8F9c48aAD8770482f4739bEeD696"):
        strats = [
            strategy
            for pair in carbon_pairs
            for strategy in get_strategies(pair)
        ]
        block_number = brownie.web3.eth.block_number

    return process_raw_carbon_strategies(
        strategies=strats, block_updated=block_number
    )



In [93]:
import requests
# def request_coingecko_price(token_address: str):

#     token_address = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2" if token_address == "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE" else token_address
#     token_address = str(token_address)
#     url = f"https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={token_address}&vs_currencies=USD&include_market_cap=false&include_24hr_vol=false&include_24hr_change=false&include_last_updated_at=false"

#     reponse = requests.get(url=url)

#     if not reponse:
#         print(f"No price for token: {token_address}")
#         return ("TKN", 1)

#     data = reponse.json()

#     if data:
#         price = data.get(token_address.lower()).get("usd")
#         # item = (token_address.lower(), price)
#         return ("USD", Decimal(str(price)))
#     else:
#         print(f"No price for token: {token_address}")
#         return ("TKN", 1)

import json
import os
from dotenv import load_dotenv
load_dotenv()

price_key = os.environ.get("x-carbon-auth-key")


def get_market_price(token_address):
    carbon_url = f"https://app.carbondefi.xyz/api/marketrate/{token_address}?convert=USD"

    res = requests.get(carbon_url, headers={"x-carbon-auth-key" : price_key}) 



    res_json = json.loads(res.text)
    #print(res_json.get("status").get("error_code"))
    #print(res_json)
    if res_json.get("status").get("error_code") == 0:
        return ("USD" ,Decimal(str(res_json.get("data").get("USD"))))
    else:
        return ("TKN", 1)
        # return -1
    

In [94]:
def generate_carbon_tvl_dict(pairs, carbon_orders) -> dict:
    forwards = "forwards"
    inverted = "inverted"
    pair_dict = {}

    tokens = []

    unknown_pairs = []

    # Build a dict of all pairs
    for pair in pairs:
        pair_combined = pair[0] + "_" + pair[1]
        tokens.append(pair[0])
        tokens.append(pair[1])
        
        # if not check_if_tkn_in_table_address(pair[0]) or not check_if_tkn_in_table_address(pair[1]):
        #     unknown_pairs.append(pair_combined)
        #     continue      
        pair_dict[pair_combined] = {get_token_symbol_from_address(pair[0]): 0,
                                    get_token_symbol_from_address(pair[1]): 0, "num_strats": 0}

    #print(f"Number of unknown pairs: {len(unknown_pairs)}")
    # for p in unknown_pairs:
    #     print(p)

    unique_tokens = set(tokens)

    price_list = {}

    for token in unique_tokens:
        price_list[token.lower()] = get_market_price(token_address=token)
        #time.sleep(1)

    pair_keys = pair_dict.keys()
    # validate that all pairs have been added to the dict
    # assert len(pair_keys) == len(pairs)

    unknown_strategies = []

    for order in carbon_orders:
        # if not check_if_tkn_in_table_address(order.tkn0.address) or not check_if_tkn_in_table_address(
        #         order.tkn1.address):
        #     unknown_strategies.append(order)
        #     continue

        tkn0_tkn1 = order.tkn0.address + "_" + order.tkn1.address
        tkn1_tkn0 = order.tkn1.address + "_" + order.tkn0.address
        key = ""
        direction = ""

        if tkn0_tkn1 in pair_keys:
            key = tkn0_tkn1
        elif tkn1_tkn0 in pair_keys:
            key = tkn1_tkn0
        else:
            raise Exception(
                "Key not found in token pairs. Something went wrong in generate_carbon_tvl_dict function."
            )

        denomination = price_list.get(str(order.tkn1.address).lower())[0]
        pair_dict[key][f"_{order.tkn1.symbol}"] = denomination
        new_total = pair_dict.get(key).get(order.tkn1.symbol) + int(
            order.y * price_list.get(str(order.tkn1.address).lower())[1])
        pair_dict[key]["num_strats"] = pair_dict.get(key).get("num_strats") + 1
        pair_dict[key][order.tkn1.symbol] = new_total
        pair_dict[key][f"token0address"] = order.tkn0.address
        pair_dict[key][f"token1address"] = order.tkn1.address

    # print(f"Number of unknown strategies: {len(unknown_strategies)}")
    # for p in unknown_strategies:
    #     print(p)


    organized_list = []

    for pair in pair_keys:

        p_k = list(pair_dict.get(pair).keys())
        tkn0 = p_k[0]
        tkn1 = p_k[1]


        denom0 = pair_dict.get(pair).get(f"_{tkn0}")
        denom1 = pair_dict.get(pair).get(f"_{tkn1}")


        organized_list.append(
            {"token0": tkn0, "amt0": pair_dict.get(pair).get(tkn0), "token0address": pair_dict.get(pair).get("token0address"),"token1": tkn1,
                "amt1": pair_dict.get(pair).get(tkn1), "token1address": pair_dict.get(pair).get("token1address") ,"num_strats": pair_dict.get(pair).get("num_strats")})


    df = pd.DataFrame(organized_list)

    #df = pd.DataFrame(pair_dict.items())

    path = os.path.normpath(f"{os.getcwd()}/fastlane_bot/data/tvl_report.csv")

    df.to_csv(path)
    return organized_list
    #return pair_dict

In [95]:
def create_unique_token_list(pairs):

    tokens = []
    for pair in pairs:
        tokens.append(pair[0])
        tokens.append(pair[1])

    unique_tokens = set(tokens)
    return unique_tokens


def get_token_prices(tokens):
    prices = {}

    for tkn in tokens:
        price = get_market_price(tkn)
        prices[tkn] = price

    return prices




In [96]:
# all_tkns = create_unique_token_list(all_pairs)
# prices = get_token_prices(all_tkns)


#print(prices)


In [97]:
# def check_if_pair_active(token_pair):

#     strategies = get_strategies(token_pair)
#     #print(f"num of strategies: {len(strategies)}")
#     return True if len(strategies) > 0 else False

# # pair = all_pairs[8]
# # print(f"pair={pair}")
# # print(check_if_pair_active(pair))

# active_pairs = [pair for pair in all_pairs if check_if_pair_active(pair)]



In [98]:
# print(f"all pairs length: {len(all_pairs)}")
# print(f"active pairs length: {len(active_pairs)}")

In [99]:
flt = [T.USDC, T.USDT, T.ETH, T.LINK, T.BNT, T.WBTC, T.DAI]

def get_triangle_exists(tkn0, tkn1):
    
    pair_name = tkn0 + "/" + tkn1
    pair_reverse = tkn1 + "/" + tkn0
    tkn0_weth = "WETH-6Cc2" if tkn0 == "ETH-EEeE" else tkn0
    tkn1_weth = "WETH-6Cc2" if tkn1 == "ETH-EEeE" else tkn1
    pair_name_weth = tkn0_weth + "/" + tkn1_weth   
    pair_reverse_weth = tkn1_weth + "/" + tkn0_weth

    flt_tkn0 = None
    flt_tkn1 = None

    bancor_v2 = None
    bancor_v3 = None
    uniswap_v2 = None
    uniswap_v3 = None
    sushi = None

    # ETH BNT 0BNT 1PEPE PEPE ETH

    for tkn in flt:
        
        if tkn == flt:
            continue

        found0 = False
        found1 = False
        pair_flt_0 = tkn0 + "/" + tkn
        pair_reverse_0 = tkn + "/" + tkn0
        pair_flt_1 = tkn1 + "/" + tkn
        pair_reverse_1 = tkn + "/" + tkn1

        b2_0 = bot.db.get_pool(pair_name=pair_flt_0, exchange_name="bancor_v2")
        b2_0 = bot.db.get_pool(pair_name=pair_reverse_0, exchange_name="bancor_v2") if b2_0 is None else b2_0
        b2_1 = bot.db.get_pool(pair_name=pair_flt_1, exchange_name="bancor_v2")
        b2_1 = bot.db.get_pool(pair_name=pair_reverse_1, exchange_name="bancor_v2") if b2_1 is None else b2_1
        b3_0 = bot.db.get_pool(pair_name=pair_flt_0, exchange_name="bancor_v3")
        b3_1 = bot.db.get_pool(pair_name=pair_flt_1, exchange_name="bancor_v3")
        b3_0 = bot.db.get_pool(pair_name=pair_reverse_0, exchange_name="bancor_v3") if b3_0 is None else b3_0
        b3_1 = bot.db.get_pool(pair_name=pair_reverse_1, exchange_name="bancor_v3") if b3_1 is None else b3_1


        tkn_weth = "WETH-6Cc2" if tkn == "ETH-EEeE" else tkn
        pair_flt_0_weth = tkn0_weth + "/" + tkn_weth
        pair_reverse_0_weth = tkn_weth + "/" + tkn0_weth
        pair_flt_1_weth = tkn1_weth + "/" + tkn_weth
        pair_reverse_1_weth = tkn_weth + "/" + tkn1_weth

        uni_v2_0 = bot.db.get_pool(pair_name=pair_flt_0_weth, exchange_name="uniswap_v2")
        uni_v2_0 = bot.db.get_pool(pair_name=pair_reverse_0_weth, exchange_name="uniswap_v2") if uni_v2_0 is None else uni_v2_0
        uni_v2_1 = bot.db.get_pool(pair_name=pair_flt_1_weth, exchange_name="uniswap_v2")
        uni_v2_1 = bot.db.get_pool(pair_name=pair_reverse_1_weth, exchange_name="uniswap_v2") if uni_v2_1 is None else uni_v2_1

        uni_v3_0 = bot.db.get_pool(pair_name=pair_flt_0_weth, exchange_name="uniswap_v3")
        uni_v3_0 = bot.db.get_pool(pair_name=pair_reverse_0_weth, exchange_name="uniswap_v3") if uni_v3_0 is None else uni_v3_0
        uni_v3_1 = bot.db.get_pool(pair_name=pair_flt_1_weth, exchange_name="uniswap_v3")
        uni_v3_1 = bot.db.get_pool(pair_name=pair_reverse_1_weth, exchange_name="uniswap_v3") if uni_v3_1 is None else uni_v3_1

        sushi_0 = bot.db.get_pool(pair_name=pair_flt_0_weth, exchange_name="sushiswap_v2")
        sushi_0 = bot.db.get_pool(pair_name=pair_reverse_0_weth, exchange_name="sushiswap_v2") if sushi_0 is None else sushi_0
        sushi_1 = bot.db.get_pool(pair_name=pair_flt_1_weth, exchange_name="sushiswap_v2")
        sushi_1 = bot.db.get_pool(pair_name=pair_reverse_1_weth, exchange_name="sushiswap_v2") if sushi_1 is None else sushi_1

        found0 = True if (b2_0 is not None or b3_0 is not None or uni_v2_0 is not None or uni_v3_0 is not None or sushi_0 is not None) else False
        found1 = True if (b2_1 is not None or b3_1 is not None or uni_v2_1 is not None or uni_v3_1 is not None or sushi_1 is not None) else False

        if found0 and found1:
            return True
    
    return False










def external_pool_exists(tkn0, tkn1):
    
    pair_name = tkn0 + "/" + tkn1
    pair_reverse = tkn1 + "/" + tkn0
    bancor_v2 = bot.db.get_pool(pair_name=pair_name, exchange_name="bancor_v2")
    bancor_v2 = bot.db.get_pool(pair_name=pair_reverse, exchange_name="bancor_v2") if bancor_v2 is None else bancor_v2
    bancor_v3 = bot.db.get_pool(pair_name=pair_name, exchange_name="bancor_v3") 
    bancor_v3 = bot.db.get_pool(pair_name=pair_reverse, exchange_name="bancor_v3") if bancor_v3 is None else bancor_v3
    tkn0 = "WETH-6Cc2" if tkn0 == "ETH-EEeE" else tkn0
    tkn1 = "WETH-6Cc2" if tkn1 == "ETH-EEeE" else tkn1
    pair_name = tkn0 + "/" + tkn1   
    pair_reverse = tkn1 + "/" + tkn0
    uniswap_v2 = bot.db.get_pool(pair_name=pair_name, exchange_name="uniswap_v2")
    uniswap_v2 = bot.db.get_pool(pair_name=pair_reverse, exchange_name="uniswap_v2") if uniswap_v2 is None else uniswap_v2
    uniswap_v3 = bot.db.get_pool(pair_name=pair_name, exchange_name="uniswap_v3")
    uniswap_v3 = bot.db.get_pool(pair_name=pair_reverse, exchange_name="uniswap_v3") if uniswap_v3 is None else uniswap_v3
    sushi = bot.db.get_pool(pair_name=pair_name, exchange_name="sushiswap_v2")
    sushi = bot.db.get_pool(pair_name=pair_reverse, exchange_name="sushiswap_v2") if sushi is None else sushi
    covered = True if (bancor_v3 is not None or bancor_v2 is not None or uniswap_v2 is not None or uniswap_v3 is not None or sushi is not None) else False

    # if not covered:
    #     print(f"pair not covered: {pair_name}")

    return covered


In [100]:
# def calculate_pair_coverage(pairs):
#     pools_not_covered_by_pairs = []
#     covered = 0
#     for token_pair in pairs:
#         tkn0 = bot.db.get_token(address=token_pair[0]).key      
#         tkn1 = bot.db.get_token(address=token_pair[1]).key      
#         exists = external_pool_exists(tkn0, tkn1)
#         if exists:
#             covered += 1
#         else:
#             pools_not_covered_by_pairs.append(tkn0 + "/" + tkn1) 
#     return covered, pools_not_covered_by_pairs



# count_of_covered_pairs, pools_not_covered = calculate_pair_coverage(active_pairs)

# coverage_percent = count_of_covered_pairs/len(active_pairs)

# coverage_percent = str("{0:.0%}".format((coverage_percent)))

# # for pool in pools_not_covered:
# #     print(f"pair not covered: {pool}")

# print(f"\n\nNumber of token pairs arbed by pair: {count_of_covered_pairs} of {len(active_pairs)}. \n{coverage_percent} coverage.")

    

In [101]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
#Authorize the API
scope = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/drive.file'
    ]
file_name = os.path.normpath(my_file_path)
creds = ServiceAccountCredentials.from_json_keyfile_name(file_name,scope)
client = gspread.authorize(creds)

sheet = client.open('Arbon Carbon').sheet1



In [102]:

def analyze_tvl(tvl_report):
    total_tvl = 0
    num_strats = 0
    arbed_tvl = 0
    not_covered_tvl = 0
    arbed_strats = 0
    not_covered_strats = 0
    total_pairs = 0
    pairs_covered = 0
    pairs_not_covered = 0
    

    start_col = 12
    start_row = 22

    update_cells = []
    number_of_rows = 0

    for idx, pair in enumerate(tvl_report):
        if pair.get("token0address") is None or pair.get("token1address") is None:
            continue
        number_of_rows += 1
        update_cells.append(idx)

        tvl0 = pair.get("amt0")
        tvl1 = pair.get("amt1")
        total_tvl += tvl0 + tvl1    



        #print(pair.get("token0address"), pair.get("token1address"))    
        tkn0 = bot.db.get_token(address=pair.get("token0address")).key      
        tkn1 = bot.db.get_token(address=pair.get("token1address")).key     

        tkn0_symbol = tkn0[:4]
        tkn1_symbol = tkn1[:4]

        if "-" in tkn0_symbol:
            new = tkn0_symbol.split("-")[0]
            tkn0_symbol = new
        if "-" in tkn1_symbol:
            new = tkn1_symbol.split("-")[0]
            tkn1_symbol = new


        pair_exists = external_pool_exists(tkn0, tkn1)
        number_of_strats = pair.get("num_strats")
        num_strats += number_of_strats
        total_pairs += 1


        update_cells.append(tkn0_symbol)
        update_cells.append(tvl0)
        update_cells.append(tkn1_symbol)
        update_cells.append(tvl1)
        update_cells.append(number_of_strats)
        update_cells.append(tvl0 + tvl1)
        
        triangle_exists = get_triangle_exists(tkn0, tkn1)
        #triangle_exists = False

        if pair_exists:
            arbed_tvl += tvl0 + tvl1
            arbed_strats += number_of_strats
            pairs_covered += 1
            update_cells.append("Yes")
            update_cells.append("Yes") if triangle_exists else update_cells.append("No")
        else:
            #print(f"Not covered: {pair}")
            if triangle_exists:
                arbed_tvl += tvl0 + tvl1
                arbed_strats += number_of_strats
            else:
                not_covered_tvl += tvl0 + tvl1
                not_covered_strats += number_of_strats
                pairs_not_covered += 1
            update_cells.append("No")
            update_cells.append("Yes") if triangle_exists else update_cells.append("No")

    # number_of_rows = len(update_cells) / 9
    cell_list = sheet.range(f'A22:I{number_of_rows + start_row - 1}')

    # print(f"num of update_cells: {len(update_cells)}")
    # print(f"num of cells: {len(cell_list)}")
    # print(f"num of rows: {number_of_rows}")

    for idx, cell in enumerate(cell_list):
        cell.value = update_cells[idx]

    # Update in batch
    sheet.update_cells(cell_list)

    print("\n************************ Arb Coverage Report ************************")
    print(f"\nArbed TVL = ${arbed_tvl}")
    print(f"Not arbed TVL = ${not_covered_tvl}")
    print(f"Total TVL = ${total_tvl}")  
    print(f"Arb coverage of TVL: {'{0:.0%}'.format((arbed_tvl/total_tvl))}")
    print(f"\nArbed strategies = {arbed_strats}")
    print(f"Not arbed strategies = {not_covered_strats}")
    print(f"Total strategies = {num_strats}")  
    print(f"Strategy coverage: {'{0:.0%}'.format((arbed_strats/num_strats))}")  
    print(f"\nArbed pairs = {pairs_covered}")        
    print(f"Not arbed pairs = {pairs_not_covered}") 
    print(f"Total pairs = {total_pairs}") 
    print(f"Pair coverage: {'{0:.0%}'.format((pairs_covered/total_pairs))}")  
    # sheet.update_cell(2,2, arbed_tvl)
    # sheet.update_cell(3,2, not_covered_tvl)
    # sheet.update_cell(4,2, total_tvl)
    # sheet.update_cell(5,2, '{0:.0%}'.format((arbed_tvl/total_tvl)))
    # sheet.update_cell(7,2, arbed_strats)
    # sheet.update_cell(8,2, not_covered_strats)
    # sheet.update_cell(9,2, num_strats)
    # sheet.update_cell(10,2, '{0:.0%}'.format((arbed_strats/num_strats)))
    # sheet.update_cell(12,2, pairs_covered)
    # sheet.update_cell(13,2, pairs_not_covered)  
    # sheet.update_cell(14,2, total_pairs)  
    # sheet.update_cell(15,2, '{0:.0%}'.format((pairs_covered/total_pairs)))

    print("\n**********************************************************************")

    update_fields = []
    update_fields.append(arbed_tvl)
    update_fields.append(not_covered_tvl)
    update_fields.append(total_tvl)
    update_fields.append('{0:.1%}'.format((arbed_tvl/total_tvl)))
    update_fields.append(arbed_strats)
    update_fields.append(not_covered_strats)
    update_fields.append(num_strats)
    update_fields.append('{0:.1%}'.format((arbed_strats/num_strats)))
    update_fields.append(pairs_covered)
    update_fields.append(pairs_not_covered)
    update_fields.append(total_pairs)
    update_fields.append('{0:.1%}'.format((pairs_covered/total_pairs)))

    cell_list = sheet.range(f'B2:B13')
    # print(len(cell_list))
    # print(cell_list)

    for idx, cell in enumerate(cell_list):
        cell.value = update_fields[idx]

    # Update in batch
    sheet.update_cells(cell_list)

    






In [103]:
def get_tvl_by_token(carbon_orders):
    
    tokens = {}
    token_addresses = {}
    
    for order in carbon_orders:
        token_addresses[order.tkn1.symbol] = order.tkn1.address
        if order.tkn1.symbol in tokens:
            tokens[order.tkn1.symbol] = tokens.get(order.tkn1.symbol) + order.y
        else:
            tokens[order.tkn1.symbol] = order.y
    return (tokens, token_addresses)



def update_token_tvl(carbon_orders):
    start_col = 12
    start_row = 22

    start_col_letter = "K"
    end_col_letter = "N"

    tvl_by_token_dict, token_addresses_dict = get_tvl_by_token(carbon_orders)
    tvl_by_token_keys = tvl_by_token_dict.keys()

    update_fields = []
    update_rows = []

    for idx, tkn in enumerate(tvl_by_token_keys):
        #sheet.update_cell(start_row + idx, start_col -1, idx + 1)
        #sheet.update_cell(start_row + idx, start_col, tkn)
        #sheet.update_cell(start_row + idx, start_col + 1, str(tvl_by_token_dict.get(tkn)))

        

        tkn_price = get_market_price(token_addresses_dict.get(tkn))

        tvl_usd = tvl_by_token_dict.get(tkn) * tkn_price[1]
        tvl_usd = str(tvl_usd)
        #sheet.update_cell(start_row + idx, start_col + 2, tvl_usd)
        row = [idx + 1, tkn, str(tvl_by_token_dict.get(tkn)), tvl_usd]
        update_rows.append(row)

        update_fields.append(idx + 1)
        update_fields.append(tkn)
        update_fields.append(str(tvl_by_token_dict.get(tkn)))
        update_fields.append(tvl_usd)

        


    # print(f"Number of update_fields: {len(update_fields)}")
    # print(f"update_fields: {update_fields}")
    # print(f"Number of update_rows: {len(update_rows)}")
    # print(f"update_fields: {update_rows}")
    #Select a range
    # print(f"{start_col_letter}{start_row}:{end_col_letter}{len(update_rows) + start_row}")
    cell_list = sheet.range(f'{start_col_letter}{start_row}:{end_col_letter}{len(update_rows) - 1 + start_row}')
    # print(len(cell_list))
    # print(cell_list)

    for idx, cell in enumerate(cell_list):
        cell.value = update_fields[idx]

    # Update in batch
    sheet.update_cells(cell_list)







In [1]:
# Automatically run the report every 4 hours

while True:
    
    carbon_orders = get_all_carbon_strategies(carbon_pairs=all_pairs)
    tvl_report = generate_carbon_tvl_dict(pairs=all_pairs, carbon_orders=carbon_orders)
    update_token_tvl(carbon_orders)
    analyze_tvl(tvl_report)
    time.sleep(14400)



NameError: name 'get_all_carbon_strategies' is not defined